# Case Study: The site for recommendations - "Gnod"

### Scenario

You have been hired as a Data Analyst for "Gnod".

"Gnod" is a site that provides recommendations for music, art, literature and products based on collaborative filtering algorithms. Their flagship product is the _music recommender_, which you can try at [www.gnoosic.com](www.gnoosic.com). The site asks users to input 3 bands they like, and computes similarity scores with the rest of the users. Then, they recommend to the user bands that users with similar tastes have picked.

"Gnod" is a small company, and its only revenue stream so far are adds in the site. In the future, they would like to explore partnership options with music apps (such as _Deezer_, _Soundcloud_ or even _Apple Music_ and _Spotify_). However, for that to be possible, they need to expand and improve their recommendations.

That's precisely where you come. They have hired you as a Data Analyst, and they expect you to bring a mix of technical expertise and business mindset to the table.

Jane, CTO of Gnod, has sent you an email assigning you with your first task.

### Task(s)

> This is an e-mail Jane - CTO of Gnod - sent over your inbox in the first weeks working there.

_Dear xxxxxxxx,
We are thrilled to welcome you as a Data Analyst for *Gnoosic*!_

_As you know, we are trying to come up with ways to enhance our music recommendations. One of the new features we'd like to research is to recommend songs (not only bands). We're also aware of the limitations of our collaborative filtering algorithms, and would like to give users two new possibilities when searching for recommendations:_

- _Songs that are actually similar to the ones they picked from an acoustic point of view._
- _Songs that are popular around the world right now, independently from their tastes._

_Coming up with the perfect song recommender will take us months - no need to stress out too much. In this first week, we want you to explore new data sources for songs. The Internet is full of information and our first step is to acquire it do an initial exploration. Feel free to use APIs or directly scrape the web to collect as much information as possible from popular songs. Eventually, we'll need to collect data from millions of songs, but we can start with a few hundreds or thousands from each source and see if the collected features are useful._

_Once the data is collected, we want you to create clusters of songs that are similar to each other. The idea is that if a user inputs a song from one group, we'll prioritize giving them recommendations of songs from that same group._

_On Friday, you will present your work to me and Marek, the CEO and founder. Full disclosure: I need you to be very convincing about this whole song-recommender, as this has been my personal push and the main reason we hired you for!_

_Be open minded about this process: we are agile, and that means that we define our products and features on-the-go, while exploring the tools and the data that's available to us. We'd love you to provide your own vision of the product and the next steps to be taken._

_Lots of luck and strength for this first week with us!_

_-Jane_

In [1]:

#general
import pandas as pd
import numpy as np
import random
from IPython.display import display, Image

#Web Scraping
from bs4 import BeautifulSoup
import requests

#Cleaners
import re

#API
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from spotify_credentials import *
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret)) #Initialize SpotiPy with user credentias

#Kmean
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

NameError: name 'ty' is not defined

# Scraping

In [ ]:
def billboard_web_scraper(url):
    response = requests.get(url)
    if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
    
        song = []
        artist = []
        for i in range(0,100):
            song.append(soup.select("div.item-details__title")[i].get_text())
            artist.append(soup.select("div.item-details__artist")[i].get_text())
            
        top_100_df =  pd.DataFrame({
                            "name": song,
                            "artists": artist,
                           })
        
        return top_100_df
    else:
        print("Error, response status code = ", response.status_code)

# Cleaners

In [ ]:
def song_cleaner(x):
    x = re.sub( " +", " ", x.replace("Featuring", "").lower().strip())
    return x

In [ ]:
def char_remover(x, char_del=[",", ".", "'", "[", "]", ".", ":", "-", "'", "&"]):
    for char in char_del:
        x = x.replace(char, "")
    return x
    

In [ ]:
def standardize_features(df):
    df["name"] = df["name"].apply(lambda x: song_cleaner(x))
    df["artists"] = df["artists"].apply(lambda x: song_cleaner(char_remover(str(x))))
    try:
        df["year"] = df["year"].apply(lambda x: str(x)[:4]) 
    except:
        pass
    return df

In [ ]:
top_100_bil = standardize_features(billboard_web_scraper("https://www.billboard.com/charts/hot-100."))
top_100_bil.head(10)

# Spotify API 

In [ ]:
def basic_spotify_search(song):
    tune_name = []
    artists_name = []
    song_id = []
    release_date = []
    explicit = []
    song_info = sp.search(q = song, limit = 20)
    for i in range(len(song_info["tracks"]["items"])):
        try:
            tune_name.append(song_info["tracks"]["items"][i]["name"])   
            artists_name.append([song_info["tracks"]["items"][i]["artists"][j]["name"] for j in range(len(song_info["tracks"]["items"][i]["artists"]))])
            song_id.append(song_info["tracks"]["items"][i]["id"])
            release_date.append(song_info["tracks"]["items"][i]["album"]["release_date"])
            explicit.append(int(song_info["tracks"]["items"][i]["explicit"]))
        except:
            pass
    d = {'name': tune_name, 'artists': artists_name, "song_id": song_id, "year": release_date, "explicit": explicit}
    
    return  pd.DataFrame(data=d)

In [ ]:
df_BSS = basic_spotify_search("lemonade")
df_BSS

In [ ]:
def get_features(spoty_ids): 
    '''Must input List'''
    features = sp.audio_features(tracks=spoty_ids)
    df_infunc = pd.DataFrame(columns = list(features[0].keys()))
    
    for i in range(len(list(spoty_ids))):
        df_infunc.loc[i] = list(features[i].values())
        
    return df_infunc.drop(["type", "id", "uri", "track_href", "analysis_url", "time_signature"], axis = 1)


In [ ]:
df_BSS_clean = standardize_features(df_BSS)
df_BSS_clean

In [ ]:
get_features(list(df_BSS["song_id"])).head()

# Adding Spotify data to billboard

In [ ]:
def SongandArtist_featureizer(bil_df):
    id_list = []
    basic_info = pd.DataFrame(columns = ["name", "artists", "song_id", "year", "explicit"] )
    for i in range(len(list(bil_df["name"]))):
        spot_search = standardize_features(basic_spotify_search( list(bil_df["name"])[i] ))
        df_comparer = spot_search[spot_search["artists"] == list(bil_df["artists"])[i]]
        ndx = df_comparer.index.values.astype(int)
        
        try:    
            id_list.append(spot_search["song_id"][ndx[0]])
            basic_info = basic_info.append(spot_search.iloc[ndx[0]])
        except:
            id_list.append(spot_search["song_id"][0])
            basic_info = basic_info.append(spot_search.iloc[[0]])
    

    return pd.concat([basic_info.reset_index(), get_features(id_list).reset_index()], axis = 1).drop(["index"], axis = 1)
    

In [ ]:
df_bil_feat = SongandArtist_featureizer(top_100_bil)

# Kaggle Spotify Data

In [ ]:
data_spotify_csv = pd.read_csv("data.csv")
data_spotify_csv.head()

In [ ]:
df_kag = data_spotify_csv.copy()

In [ ]:
def kag(df):
    df = df[['name', 'artists', 'id', 'year', 'explicit', 'danceability',
           'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
           'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
    df.columns = ['name', 'artists', 'song_id', 'year', 'explicit', 'danceability',
           'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
           'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    df = standardize_features(df)
    return df

df_kag = kag(df_kag)

# Unsupervised Learning

In [ ]:
def Basic_Kmean(df_song_data):
    X = df_song_data.drop(["name", "song_id", "artists"], axis = 1) 
    X = StandardScaler().fit_transform(X)
    
    kmeans = KMeans(n_clusters=8, random_state=1234)
    kmeans.fit(X)
    
    clusters = kmeans.predict(X)
    
    return pd.concat([df_song_data, pd.Series(clusters)], axis = 1), kmeans

# Models 

In [ ]:
df_kag_grouped, kmeans_spoty = Basic_Kmean(df_kag)

In [ ]:
df_bil_grouped, kmeans_bil = Basic_Kmean(df_bil_feat)

In [ ]:
df_kag_grouped.columns= ['name', 'artists', 'song_id','year', 'explicit', 'danceability','energy','key', 'loudness','mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 'valence','tempo', 'duration_ms', "group"]
df_bil_grouped.columns= ['name', 'artists', 'song_id','year', 'explicit', 'danceability','energy','key', 'loudness','mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 'valence','tempo', 'duration_ms', "group"]

In [ ]:
df_bil_grouped.head()

In [ ]:
df_kag_grouped["name"][0]

# UI

In [ ]:
def similar_songs(df, group_num):
    sim_songs = df[df["group"]  == group_num]
    selected = random.randint(1,sim_songs.shape[0])
    print("You should check out (", list(sim_songs["name"])[selected], ") by (", list(sim_songs["artists"])[selected],")")

In [ ]:
def is_hot(chosen_song, df_bil, bil_model):
    SongAndArtist = standardize_features(basic_spotify_search([chosen_song]))
    
    group_num = bil_model.predict(SongandArtist_featureizer(SongAndArtist.iloc[[song_num]]))
    similar_songs(df_bil, group_num)

In [ ]:
def is_spoty(chosen_song, df_spoty, spoty_model):
    SongAndArtist = standardize_features(basic_spotify_search([chosen_song]))
    display(SongAndArtist)
    
    song_num = int(input("Which on of these is it? "))
    
    SADF = SongandArtist_featureizer(SongAndArtist.iloc[[song_num]]).drop(["name", "song_id", "artists"], axis = 1)
    group_num = spoty_model.predict(SADF)
    
    similar_songs(df_spoty, group_num[0])
    

In [ ]:
def master_UI(chosen_song, df_spoty, spoty_model, df_bil, bil_model):
    if chosen_song == "#":
        return
    
    elif song_cleaner(chosen_song) in list(df_bil["name"]):
        print("Song is curently hot")
        is_hot(chosen_song, df_bil, bil_model)
        
    else:
        print("Song is not in top 100")
        is_spoty(chosen_song, df_spoty, spoty_model)

# Song Recomender!

In [ ]:
master_UI(str(input("Choose a song! I'll recomend a similar one ")), df_kag_grouped, kmeans_spoty, df_bil_grouped, kmeans_bil)

In [ ]:
!jupyter nbconvert --to script CaseStudy.ipynb

In [ ]:
<svg version="1.0" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" x="0px" y="0px" viewBox="0 0 708.08 493.07" style="enable-background:new 0 0 708.08 493.07;" xml:space="preserve">
<style type="text/css">
	.st0{fill-rule:evenodd;clip-rule:evenodd;fill:#878787;stroke:#F2F2F2;stroke-width:0.5;stroke-miterlimit:3.864;}
	.st1{fill-rule:evenodd;clip-rule:evenodd;fill:none;stroke:#000000;stroke-width:0.5;stroke-miterlimit:10;}
	.st2{fill:#878787;stroke:#F2F2F2;stroke-width:0.5;stroke-miterlimit:3.864;}
	.st3{fill:#878787;stroke:#F2F2F2;stroke-width:0.25;stroke-miterlimit:3.864;}
	.st4{fill-rule:evenodd;clip-rule:evenodd;fill:#878787;stroke:#F2F2F2;stroke-width:0.25;stroke-miterlimit:3.864;}
	.st5{fill-rule:evenodd;clip-rule:evenodd;fill:none;stroke:#F2F2F2;stroke-width:0.25;stroke-miterlimit:3.864;}
	.st6{fill:none;stroke:#F2F2F2;stroke-width:0.25;stroke-miterlimit:3.864;}
	.st7{fill:none;stroke:#F2F2F2;stroke-width:0.4;stroke-miterlimit:3.864;}
	.st8{fill:none;stroke:#E6E6E6;stroke-miterlimit:10;}
	.st9{fill:#7CA55B;}
	.st10{fill:#FFCE31;}
	.st11{fill:#EF8F41;}
	.st12{fill:#BF2C37;}
	.st13{fill:none;}
	.st14{fill:#3A3A3A;}
	.st15{font-family:'Roboto-Regular';}
	.st16{font-size:12px;}
</style>
<g id="cantabria">
	<path class="st0 fase_1" d="M366.56,58.27c0.3-0.09,3.29,1.1,5.34-0.19c0.76-0.48,3.38-0.2,4.41-1.5c0.69-0.87,2.01-0.73,2.18-1.38
		c0.12-0.5-1.19-12.41-0.03-13.78c0.68-0.8,6.9-5.44,6.99-5.17c0.42,1.22,0.13,0.88,1.58,0.88c0.88-0.08,1.78,0.47,2.58,0.75
		c1.08,0.37,2.11,0.25,3.23,0.25l0,0l-0.29-1c0-0.88-0.28-1.48-0.11-2.4c0.1-0.57-0.72-1.84-0.41-2.02c0.3-0.17,1.28-0.45,1.38-0.84
		c0.07-0.02,0.04-0.08,0.14-0.09c0.09-0.36,1.41-1.32,1.79-1.34c1.16-0.07,0.76-0.46,0.95-1.25c0.82,0,1.41-0.22,1.99,0.27
		c0.42,0.36,1.76-0.37,1.88-0.84c0.08,0,0.14,0.02,0.2,0.02c0.08,0.24,1.21,2.07,1.49,0.91c0.09-0.02,0.15-0.16,0.25-0.16
		c0.13-0.26,1.14-0.52,1.43-0.59c0.11-0.46,0.35-0.88,0.14-1.45l0.19-1.52l0,0c-0.26-0.09-0.45-0.31-1-0.47
		c-1.02-0.07-1.55-1.49-2.47-1.95c-0.98-0.49-2.12-0.5-3.14-0.93c-2.19-0.92-5.24,0.96-5.5-2.34c-0.1-1.24-2.19-0.43-2.76-1.45
		c-1.16-2.06-2.79-1.35-4.66-0.57c-1.63,0.36-3.2,1.35-4.34,2.49c-1,1.01-2.39,3.36-2.62,1c-0.03-0.29-0.52-0.53-0.5-0.77
		c0.03-0.4,1.94-1.42,0.79-2.11c-0.45-0.27-4.45,1.13-5.38,1.18c-0.9,0.05-0.97,1.25-2.06,1.25c-3.21,0-5.62,1.69-8.75,2.58
		c-0.76,0.28-2.97,0.47-3.82,0.43c-0.9-0.04-1.79-0.84-2.62-0.57c-1.46,0.47-3.91,0.49-5.94,0.39l0,0l-0.8,1.4
		c0.03,0.48-0.19,0.81-0.05,1.34c0.28,1,0.95,2.76,0.02,3.7c-0.71,0.71-0.93-0.53-1.2-0.84c-0.15-0.17-1.22-0.21-1.52-0.32
		c-0.69-0.24-0.83,0.82-1.52,1.27c-1.47,0.96-3.19-0.83-3.33,1.84c-0.09,1.88-1.44,2.73-3.26,2.81l-1.95,0.41l0,0
		c0.13,0.19,0.61,1.3,0.43,1.5c-0.47,0.53-0.96,1.34-0.5,2.09c0.19,0.31,0.52,0.69,0.81,0.88c0.47,0.32,1.18,0.34,1.61,0.77
		c0.86,0.94,0.67,0.96,1.79,1.59c0.54,0.3-0.14,2.06,0.77,2.15l0,0c0.28,0.03,0.94-0.28,1.29-0.34c0.93-0.16,1.31,0.59,2.35,0.07
		c0.57-0.29,1.21-0.98,1.88-0.5c1.92,1.39,1.38-0.82,2.69-0.82c0.95,0,0.93,1.03,1.9-0.27c0.62-0.84,2.13,0.46,3.03,0.91
		c1.17,0.59,0.91,3.25,2.47,3.36c0.73-0.14,2.14,0.13,2.87,0.29c1.11,0.26,1.23,2.35,1.47,3.29c0.25,0.97-0.05,2.48,0.52,3.33
		c0.32,0.28,0.35,0.54,0.81,0.54c0.75,0,0.82,1.78,1.72,2.43c1.5,1.08,1.58-1.77,2.35,0.57C365.97,58.13,366.25,58.36,366.56,58.27
		L366.56,58.27L366.56,58.27z M399.16,32.17l-0.2,0.37c-0.29,0.32-0.97,0.63-0.97-0.03c0-0.32,0.13-0.96-0.11-1.19
		c-0.19-0.18-0.24-0.53-0.16-0.78c0.06-0.17,0.93-0.3,1.14-0.4c0.21-0.09,0.3-0.02,0.47,0.1c0.15,0.1,0.4,0.23,0.41,0.44
		c0.01,0.22-0.07,0.49-0.18,0.67C399.41,31.6,399.32,31.96,399.16,32.17z"></path>
</g>
<g id="asturias">
	<path class="st0 fase_1" d="M349.06,24.02c-0.73-0.04-1.4-0.09-1.95-0.14c-1.62-0.15-3.34-0.8-4.98-0.7c-1.44-0.06-2.75-0.97-4.09-1.31
		l-4.21-1.09c-2.68-0.69-5.16-0.04-7.8-1.02c-1.39-0.52-2.19-1.16-3.85-0.75c-2.07,0.19-3.71-0.72-4.05-2.79
		c-0.66-0.16-2.01-0.22-2.71-0.09c-1.29,0.24-1.11-0.39-1.56,0.86c-0.11,0.32-0.83,0.23-0.81,0.05c0.05-0.52,0.56-1.03,0.52-1.54
		c-0.06-0.74-2.25-0.35-2.74-0.27c-1.95,0.32-3.84-0.29-5.79-0.29c-1.09,0-1.75,0.59-2.56,0.68c-0.94,0.11-1.06-0.79-1.65-1.36
		c-1.01-0.63-2.49-1.16-2.9-2.4c-0.2-0.6-2.03-2.5-2.65-2.65c-0.14,0.41-0.55,1.45-0.97,1.68c-0.66,0.35-1.83-0.08-1.83,0.86
		c0,1.88-2.88,1.99-4.21,1.99c-1.19,0-2.24,1.76-3.03,0.91c-0.41-0.45-1.67-0.3-2.26-0.25c-0.9-0.07-2.26-0.69-2.99-1.18
		c-0.37-0.24-1.65,0.65-2.17,0.84c-1.84,0.65-4.27,1.37-6.24,0.86c-0.35-0.09-0.68-0.63-1-0.63c-0.53,0-0.68,0.75-1.24,0.79
		c-0.64,0.05-1.1-0.25-1.72-0.36c-0.93-0.17-1.94-0.03-2.9,0.05c-1.06-0.15-1.3-0.39-2.51-0.16c-0.66,0.12-2.21,0.34-2.81,0.18
		c-0.49-0.13-0.98-0.43-1.49-0.43c-0.74,0-0.95,0.4-1.9,0.23c-1.11-0.2-2.24-0.17-3.33-0.36c-1.36-0.24-2.47,0.57-3.87,0.59
		l-1.17,0.01l0,0l-0.03,1.35c0.04,0.44-0.29,1.32-0.29,1.99c0,0.66-1.43,1.68-1.81,2.45c-1.03,2.07-1.48,0.89-3.01,1.27
		c-0.05,0.38-0.71,2.01-0.14,2.18c2.69,0.77,0.02,4.51,4.05,4.51c0.36,1.46-0.57,3.04,0.75,4.35c0.36,0.43,0.69,0.69,1.24,0.73
		c0.21,0.01,2.05,0.72,2.1,0.79c0.17,0.25-0.59,1.11-0.59,1.61c0,1.02,0.95,2.03,1.99,1.72c0.71-0.21,2.1-1.88,2.31-2.52
		c0.56,0.14,1.93,2.69,1.74,3.31c-0.81,2.63-4.37,1.64-5.36,3.63c-0.27,0.54-1.04,1.82-0.93,2.4c0.38,1.96,0.92,0.54,2.01,0.54
		c0.28,1.14,3.01,0.9,3.76,2.4l0.32,2.54l0,0c0.9-0.06,0,1.7,0.44,2.48c0.27,0.49,1.88-0.44,2.48-0.92
		c0.84-0.67,1.71-1.36,2.78-1.68c0.56-0.17,1.17-0.23,1.76-0.23c0.67,0,1.29,0.32,1.9,0.32c0.73,0,1.38,0.38,2.06,0.18
		c0.91-0.26,2.99-0.72,3.71-1.18c1.41-0.89,2.24-3.31,2.96-4.87c0.26-0.57,0.7,0,1.34,0.64c0.66,0.66,2.39,1.24,3.41,1.83
		c1.6,0.93,2.77-5.79,4-3c0.36,0.81,1.61,0.51,2.31,0.29c0.93-0.29-0.13-2.29,1.52-2.29c0.78,0.19,1.14,0.52,1.81,0.79
		c0.69,0.28-1.54,4.99-0.86,4.99c0.39,1.56,3.16,1.2,4.66,2.04c1.13,0.63,4.11-0.65,5.36-0.42c2.54,0.47,1.97-5.54,4.28-5.7
		c0.79-0.05,0.63,1.04,1.54,1.21c1.19,0.22,2.95,1.92,4.24,1.62c1.23-0.13,3.74,0.85,3.91-0.5c0.32-2.61,2.29-0.93,2.41-3.51
		c0.1-2.15,1.13,1.27,2.46,2.02c1.34,0.75,2.37-1.92,3.37-1.66c2.17,0.56,2.02-0.94,3.53-0.83c0.54,0.04,1.01-1.09,1.82-1.52
		c0.49-0.26,0.79-2.09,1.02-2.67c0.44-0.53,1.94,1.53,2.4,1.07c0.85-0.85,2.16,0.11,3.08-0.71c0.69-0.62-0.24-2.25,0.42-2.91
		c0.26-0.26,1.2-0.28,1.66-0.17c0.2,0.05,0.53,0.13,0.71,0.33c0.4,0.47,0.97-0.1,1.23,0.5l0,0l1.95-0.41
		c1.82-0.08,3.16-0.93,3.26-2.81c0.13-2.66,1.85-0.87,3.33-1.84c0.68-0.45,0.82-1.51,1.52-1.27c0.3,0.1,1.36,0.14,1.52,0.32
		c0.27,0.31,0.49,1.55,1.2,0.84c0.93-0.93,0.26-2.7-0.02-3.7c-0.15-0.53,0.08-0.85,0.05-1.34L349.06,24.02L349.06,24.02z"></path>
</g>
<g id="la_rioja">
	<path class="st0 fase_1" d="M413.6,98.22L413.6,98.22c-0.04-0.05-0.08-0.11-0.12-0.18c-0.51-0.97-2.78-2.56-4.22-2.2
		c-1.8,0.45-4.36-6.88-4.87-7.14c0-0.39,0.99-10.5,1.27-11.31c0.24-0.68,0.38-7.28-0.97-7.28l-0.02-0.02c0-0.78,1.25-1.08,1.58-1.75
		c0.25-0.5,1.13-1.46,1.02-1.99c-0.11-0.51-1.02-1.45-0.75-1.63c0.57-0.38,1.56-0.12,2.22-0.16c0.47-0.03,0.75-0.32,1.22-0.32
		c0.54,0,1.23,0.16,1.79,0.2c0.64,0.05,1.28,1.21,1.95,0.7l1.43-0.59l0,0l0.95,0.39c0,0.67-0.32,1.13,0.61,1.13
		c0,0.91-0.65,1.62-0.05,2.31c0.61,0.69,2.27-1.27,2.4-1.88c0.22-1.06,0.97-1.4,1.67-0.5c0.42,0.54,1.5,0.5,1.49,1.36l-0.02,2.15
		c-0.01,1.26,1.01,1.65,1.58,2.88c0.35,0.75,0.97-0.72,1.83-0.11c1.26,0.88,0.73,2.65,2.35,0.11c1.43-2.23,2.84,0.65,4.37-0.02l0,0
		c0.55,0.08,1.26,0.65,1.81,0.79c1.9,0.5,3.86,1.62,5.34,2.27c1.51,0.66,6.09,0.13,6.61,1.95c0.85,3.04,1.68,1.76,4.21,2.67
		c1.38,0.5,2.03,2.57,2.85,3.54c1.76,2.08,5,2.82,7.37,4.08c0.37,0.2,1.16,0.69,2,1.32l-1.01,2.22c-0.03,1.13-0.88,0.22-1.27,0
		c-0.74-0.42-0.99,0.46-1.54,0.11c-1.95-1.21-1.36-0.09-2.38,0.84c-0.93,0.85-2.97,2.94-2.65,4.31c0.2,0.83,0.28,1.34,1.11,1.65
		c0.84,0.32,1.44,0.19,1.72,1.31l0,0c-0.26,0.6-0.6,1.38-0.79,1.83l-0.09,0.21c-0.32,1.67-2.09,2.48-3.23,2.96
		c-1.76,0.74-2.16-0.46-3.45-1.25c-0.67-0.41-2.19-0.88-2.95-1.04c-1.72-0.36-0.27-0.75-0.96-2.26c-0.18-1.11,0.68-1.27-0.2-2.15
		c-0.25-0.25-1.69-1.96-1.54-2.13c0.23-0.26,1.42-0.59,1.49-0.88c0.77-0.19-0.12-0.54-0.5-0.64c-0.56-0.15-1.19,0.5-1.79,0.5
		c-0.99,0-2.98,1.42-3.37,1.04c-0.34-0.34-0.86-3.38-0.93-3.87c-1.97-0.49-4.07,0.25-5.77,1.67c-0.85,0.71-1.4,0.59-2.2,0.79
		c0,2.13-0.24,3.57-1.08,5.24c-1.27,2.51-2.46,0.35-4.45,0.79c-0.72,0.24-4.71-1.06-5.1-1.82c-0.79-1.53,1.23-2.84,1.58-4.28
		c0.38-1.52-0.56-0.73-1.48-0.3c-0.49,0.23-0.62,4.68-1.58,5.45C417.53,99.74,414.25,98.99,413.6,98.22z"></path>
</g>
<g id="pais_vasco">
	<g id="gipuzkoa">
		<path class="st0 fase_1" d="M460.69,27.87l-2.04,1.29c-0.56,1.37-0.8,2.19-2.1,2.99c-1.09,0.67-1.48,0.63-2.69,0.63
			c-0.26,0-0.64,1.62-0.66,1.95c-0.07,1.3,0.17,0.94,0.43,1.97c0.09,0.35-1.47,2.31-1.79,2.74c-0.44,0.6-1.7,0.81-2.26,1.34
			c-0.68,0.64-0.23,1.07-0.63,1.63c-0.98,1.37-0.52,0.6-0.41,2.2c0.1,1.49-1.21,1.01-2.22,1.75c-0.51,0.94-1.43,1.68-2.51,1.79
			c-1.04,0.11-1.25-0.67-1.9-0.61c-1.19,0.11-1.93,1.97-1.8,3.13l0,0l-1.84-0.82c-2.03-3.15-6.13-2.83-9.43-2.83
			c-0.86,0-4.32-0.66-2.53-1.81c1.43-0.92,2.44-1.81,2.44-3.7l-0.23-1.02l0,0l0.68-0.77c1.21-1.67,1.73-2.34,1.04-4.4
			c-0.62-1.86-0.14-2.73,1.67-3.54c1.28-0.57,0.54-1.89,0.75-2.83l0.75-2.11l0,0c1.09,0.78,2.37,1.51,3.69,1.73
			c1.22,0.2,2.01-0.71,3.01-0.66c1.52,0.08,2.59,0.72,4.34,0.52c1.3-0.15,4.13-0.82,5.32-1.29l3.37-0.86
			c1.74-0.45,3.14-3.26,5.2-2.92l0,0c0.07,0.22,0.43,1.09,0.32,1.13c0,0.95,0.36,1.17,1.2,1.59
			C460.61,26.46,460.45,27.15,460.69,27.87L460.69,27.87z"></path>
	</g>
	<g id="bizkaia">
		<path class="st0 fase_1" d="M433.41,27.71c-0.09-0.07-0.18-0.13-0.27-0.2c-1.38-1.04-2.99-1.12-4.39-2.45c-0.85-0.81-1.48-0.74-2.53-1.13
			c-1.54-0.1-1.74-1.49-3.53-1.11c-0.82,0.17-2.67-0.56-2.99-1.43c-0.34-0.95,0.01-1.09-1.33-0.5c-1.36,0.6-2.05,0.88-3.64,0.88
			c-2.56,0-2.36-0.68-4.09,1.43c-0.59,0.72-1.82,1-2.24,1.79c-0.46,0.86,0.59,1.65,0.59,2.43c-1.42,0-2.64-2.16-4.18-1.66
			c-0.5,0.16-0.74,0.16-0.94,0.09l0,0l-0.19,1.52c0.22,0.57-0.02,0.99-0.14,1.45c-0.29,0.07-1.29,0.32-1.43,0.59
			c-0.1,0-0.16,0.14-0.25,0.16c-0.29,1.16-1.42-0.67-1.49-0.91c-0.07,0-0.13-0.02-0.2-0.02c-0.12,0.47-1.45,1.2-1.88,0.84
			c-0.58-0.49-1.17-0.27-1.99-0.27c-0.2,0.79,0.21,1.18-0.95,1.25c-0.38,0.02-1.7,0.98-1.79,1.34c-0.1,0.01-0.06,0.07-0.14,0.09
			c-0.1,0.39-1.08,0.67-1.38,0.84c-0.31,0.17,0.51,1.45,0.41,2.02c-0.17,0.92,0.11,1.52,0.11,2.4l0.29,1l0,0l2.04-0.73
			c0.89-0.19,1.66-1.17,2.44-1.52c0.6-0.27,0.87-1.12,1.47-1.38c0.71-0.31,1.36,1.1,2.01,1.43c0.86,0.42,2.03,0.05,2.94,0.05
			c0.01,0.04,0,0.03,0.05,0.05c0.02,0.09,0.14,0.34-0.02,0.39l0,0l2.13,1.38c3.11,0.99,2.98-4.03,4.93-2.54
			c0.87,0.67,1.38,0.81,1.33,1.99c-0.03,0.79-0.66,1.67-0.57,2.43c0.17,1.38,1.71,3.14,3.05,3.42c1.39,0.29,2.06,0.95,3.28,1.47
			c1.13,0.49,3.71,0.23,5.04,0.23c0.71,0,1.13-0.17,1.02-0.91c-0.2-1.35-0.37-1.64,0.97-2.43l3.55-0.11l0,0l0.68-0.77
			c1.21-1.67,1.73-2.34,1.04-4.4c-0.62-1.86-0.14-2.73,1.67-3.54c1.28-0.57,0.54-1.89,0.75-2.83L433.41,27.71L433.41,27.71
			L433.41,27.71z M399.16,32.17l-0.2,0.37c-0.29,0.32-0.97,0.63-0.97-0.03c0-0.32,0.13-0.96-0.11-1.19
			c-0.19-0.18-0.24-0.53-0.16-0.78c0.06-0.17,0.93-0.3,1.14-0.4c0.21-0.09,0.3-0.02,0.47,0.1c0.15,0.1,0.4,0.23,0.41,0.44
			c0.01,0.22-0.07,0.49-0.18,0.67C399.41,31.6,399.32,31.96,399.16,32.17z"></path>
	</g>
	<g id="alava">
		<path class="st0 fase_1" d="M403.77,35.56l2.13,1.38c3.11,0.99,2.98-4.03,4.93-2.54c0.87,0.67,1.38,0.81,1.33,1.99
			c-0.03,0.79-0.66,1.67-0.57,2.43c0.17,1.38,1.71,3.14,3.05,3.42c1.39,0.29,2.06,0.95,3.28,1.47c1.13,0.49,3.71,0.23,5.04,0.23
			c0.71,0,1.13-0.17,1.02-0.91c-0.2-1.35-0.37-1.64,0.97-2.43l3.55-0.11l0,0l0.23,1.02c0,1.88-1.02,2.77-2.44,3.7
			c-1.78,1.15,1.68,1.81,2.53,1.81c3.31,0,7.4-0.32,9.43,2.83l1.84,0.82l0,0c0.02,0.17,0.05,0.32,0.11,0.45
			c1.11,2.69-2.07,3.47-1.2,6.1c0.39,1.17-0.32,1.83-1.4,1.95c-1.22,0.14-0.29,1.82-0.29,2.83c0,0.87,0.19,0.26,0.63,0.23
			c0.18-0.01,0.27,1.07,0.27,1.25c0,0.49-0.62,0.26-0.9,0.27c-0.36,0.02-0.82,0.63-1.2,0.73c-1.14,0.29-1.13-0.01-1.67-0.88
			c-0.84-1.37-2.48,0.9-3.05,1.47c-0.27,0.27-1.06,0.33-1.09,0.7c-0.01,0.18,0.25,1.21,0.34,1.27c0.39,0.25,0.71-0.09,0.84,0.5
			c0.03,0.12-0.15,0.98,0.16,0.73c0.66-0.54,0.91-1.34,1.47-1.97c0.21-0.23,0.68,0.35,0.84,0.57c0.15,0.21,0.04,1.82-0.02,2.15
			l-0.36,1.95l-1.27,1.41l0,0c-1.53,0.68-2.93-2.21-4.37,0.02c-1.63,2.53-1.09,0.77-2.35-0.11c-0.86-0.6-1.48,0.86-1.83,0.11
			c-0.57-1.23-1.6-1.62-1.58-2.88l0.02-2.15c0.01-0.86-1.07-0.82-1.49-1.36c-0.71-0.9-1.46-0.56-1.67,0.5
			c-0.12,0.61-1.79,2.57-2.4,1.88c-0.61-0.69,0.05-1.4,0.05-2.31c-0.93,0-0.61-0.46-0.61-1.13l-0.95-0.39l0,0l-0.4-0.63l-6.95-1.73
			c0,0-3.1-3.5-4.49-3.49c0,0-2.58-4.21-3.05-4.79c-0.47-0.59-0.41-1.25-1.04-1.88c-1.11-1.11-0.93-1.41,0.11-2.4
			c0.42-0.4,0.61-0.96,0.83-1.48c0.36-0.91,0.44-0.77,1.26-0.58c0.72,0.17,1.36-3.66,1.36-4.3c1.55-0.52,1.44-1.15,1.04-2.61
			c0-0.74-1.33-1.82-1.33-2.77C402.44,37.26,403.64,36.08,403.77,35.56L403.77,35.56z"></path>
	</g>
</g>
<g id="navarra">
	<path class="st0 fase_1" d="M483.9,76.84c0.09-0.24,0.11-0.52,0.2-0.75c0.08-0.21,0.13-0.41,0.09-0.65c-0.09-0.56-0.52-0.03-0.72,0.16
		c-0.39,0.36-0.51,0.82-0.7,1.28c-0.05,0.13-0.16,0.47-0.03,0.58c0.24,0.2,0.66-0.25,0.88-0.39L483.9,76.84L483.9,76.84z
		 M487.28,73.5c-0.45,0.1-0.85-1.39-1.41-0.56c-0.15,0.22-0.15,0.44-0.26,0.68c-0.17,0.38-0.73,0.93-0.4,1.35
		c0.49,0.62,1.51,0.45,2.22,0.45c1.32,0-0.1-0.86-0.03-1.51L487.28,73.5L487.28,73.5z M495.77,57.43L495.77,57.43
		c-0.6,0.45-1.39,0.78-2.04,1.16c-1.11,0.65-1.78,2.26-3.14,2.49c-1.16,0.2-1.41-0.66-1.88,0.86c-0.29,0.94,0.25,2.82-1.54,2.13
		c-0.87,0.17-2.26-0.49-3.01,0.05c-0.37,0.26-0.17,1.86-0.63,1.65c-0.34-0.15-0.97-0.17-0.63,0.41c0.64,1.11-0.16,0.94-0.77,1.65
		c-0.38,0.45-0.04,0.78-0.25,1.18c-0.84,1.58-2.31-0.35-2.67,0.05c-0.35,0.38-0.45,1.74-0.61,2.29c-0.16,0.55,0.35,1.42,0.32,2.09
		c-0.05,1.14-1.3,1.22-1.79,2.09c-0.62,0.7-0.62,1.66-1.11,2.43c-0.45,0.71-0.33,1.2,0.59,1.25c1.06,0.06-1.54,3.17-1.74,3.54
		c-0.73,1.32-0.48,2.11-0.88,3.42c-0.1,0.34-0.62,1.1-0.54,1.41c0.12,0.45,0.62,0.93,0.84,1.41c0.34,0.76-0.11,1.79-0.23,2.58
		c-0.2,0.81,1.04,1.86,1.61,2.36c0.77,0.68,0.01,1.78,1.65,1.59c1.29-0.15-1.65,3.91-1.83,4.38c-0.47,1.17-0.56,2.86-2.17,2.86
		c-1.82,0-2.93,0.43-4.59,1.07c-2.34,0.89-3.31-2.61-5.18-2.61c-1.36,0-2.94,0.91-3.53-0.95c-0.21-0.38-0.38-0.7-0.88-0.62
		c-0.47,0.07-0.78,0.4-1.17-0.02c-0.16-0.17-0.35-0.21-0.58-0.19c-0.32,0.03-0.59,0.04-0.9,0.03l0,0c-0.28-1.12-0.88-0.99-1.72-1.31
		c-0.83-0.32-0.91-0.82-1.11-1.65c-0.33-1.37,1.72-3.46,2.65-4.31c1.01-0.92,0.43-2.05,2.38-0.84c0.55,0.34,0.8-0.53,1.54-0.11
		c0.38,0.22,1.23,1.13,1.27,0l1.01-2.22c-0.85-0.62-1.63-1.12-2-1.32c-2.37-1.26-5.62-2-7.37-4.08c-0.82-0.97-1.47-3.04-2.85-3.54
		c-2.53-0.91-3.35,0.37-4.21-2.67c-0.51-1.82-5.1-1.29-6.61-1.95c-1.48-0.65-3.44-1.76-5.34-2.27c-0.55-0.15-1.26-0.71-1.81-0.79
		l0,0l1.27-1.41l0.36-1.95c0.06-0.33,0.18-1.94,0.02-2.15c-0.15-0.21-0.63-0.8-0.84-0.57c-0.56,0.63-0.81,1.43-1.47,1.97
		c-0.31,0.25-0.13-0.61-0.16-0.73c-0.13-0.59-0.45-0.25-0.84-0.5c-0.09-0.06-0.35-1.09-0.34-1.27c0.03-0.37,0.81-0.43,1.09-0.7
		c0.57-0.57,2.21-2.84,3.05-1.47c0.54,0.88,0.53,1.18,1.67,0.88c0.38-0.1,0.84-0.71,1.2-0.73c0.29-0.01,0.9,0.22,0.9-0.27
		c0-0.18-0.09-1.26-0.27-1.25c-0.44,0.04-0.63,0.64-0.63-0.23c0-1.01-0.93-2.69,0.29-2.83c1.08-0.12,1.79-0.78,1.4-1.95
		c-0.87-2.63,2.31-3.4,1.2-6.1c-0.05-0.13-0.09-0.28-0.11-0.45l0,0c-0.12-1.16,0.62-3.02,1.8-3.13c0.65-0.06,0.86,0.72,1.9,0.61
		c1.08-0.11,2.01-0.85,2.51-1.79c1.01-0.74,2.32-0.25,2.22-1.75c-0.11-1.6-0.58-0.83,0.41-2.2c0.4-0.56-0.05-0.99,0.63-1.63
		c0.57-0.53,1.82-0.74,2.26-1.34c0.32-0.43,1.88-2.39,1.79-2.74c-0.26-1.03-0.5-0.67-0.43-1.97c0.02-0.33,0.4-1.95,0.66-1.95
		c1.21,0,1.61,0.04,2.69-0.63c1.3-0.8,1.55-1.62,2.1-2.99l2.04-1.29l0,0c0.83,0,1.55-0.39,2.44-0.39c0.68,0,1.34,0.32,1.92,0.32
		c0.01,0.03,0.02,0.04,0.05,0.05c0,1.03-0.6,1.68,0.59,2.54c1.18,0.85,1.99,0.23,1.99-1.07c-0.1-0.03,0.03-0.55,0.05-0.63
		c1-0.33,2.06,0.22,2.94,0.66c1.39,0.7,3.66,0.14,4.14,1.86c0.51,1.83-0.93,3.06-0.93,4.51c-0.52,1.61-0.66,2.26-2.26,3.38
		c-2.29,1.6,2.86,3.74,4.09,3.74c0.72,0,0.43-2.7,0.79-3.29c0.71-1.16,1.11-1.29,2.51-1.29c-0.39,0.13-1.18,1.34-1.18,1.86
		c0,1,1.95,2.58,2.78,1.72c0.84-0.86,1.64,0.95,2.62,0.95c0.82,0,0.88,1.41,1.29,1.25c0.78-0.3,0.24-1.04,1.18-0.52
		c0.17,0.1,0.84,0.31,0.88,0.43c0.33,0.95,0.55,0.76,1.54,1c0.31,0.07,0.76-0.02,1.09-0.02c0.47,0,0.25,0.59,0.52,0.86
		c0.21,0.26,1.44,0.79,1.74,0.86c0.09,0.38,1.34,0.94,1.52,0.23c0.03-0.01,0.04-0.02,0.07-0.02c0.16-0.66,1.21,0.08,1.47,0.11
		c0.41,1.62,4.98-1,5.56-0.48c0.41,0.36,0.76,1.56,0.88,2.11c0.03,0.13,0.04,0.3,0.04,0.47l0,0c-1.07,0-2.15,0.13-2.96,0.77
		c-0.76,0.6-0.77,1.63-1.15,2.45c-0.23,0.49-0.67,0.92-0.97,1.34c-0.37,0.52,0.05,1.58-0.32,2.09
		C495.7,56.58,497.01,56.52,495.77,57.43z"></path>
</g>
<g id="extremadura">
	<g id="badajoz">
		<path class="st0 fase_1" d="M341.59,234.16h0.05c0,0.24,0.88,0.9,0.88,1.56c0,1.55-0.82,2.04-1.13,3.35c-0.36,1.53-0.77,2.09-2.49,2.09
			c0.08,0.43,0.83,1.58,1.02,2.31c0.44,1.71,1.45,1.99,2.33,3.31c0.17,0.26,1.41,1.91,1.27,2.09c-0.12,0.14-3.72-0.91-4.57-0.91
			c-0.68,0-2.03-0.74-2.38-0.09c-0.45,0.86-0.21,1.29-1.29,1.68c-0.17,1.83-1.54,2.95,0.38,4.28c0.62,0.43,0.54,1.43-0.11,1.84
			c-0.48,0.3-2.38-0.07-2.6-0.52c-0.61,0-1.22-0.48-1.81-0.48c0,0.44,0.54,1.05,0.72,1.47c0.8,1.91,0.34,3.59,2.92,4.19
			c2.37,0.55,1.25,2.04,0.59,3.35c-0.46,0.33-0.68-0.08-1.13-0.18c-0.59-0.13-1.76-0.04-2.06,0.52c-0.63,1.2-1.11,2.4-1.11,3.81
			c0,1.39-0.07,2.49-1.38,3.31l-0.88,0.61l0,0l-1.7,0.05c-2.04,1.36-1.62,0.21-3.35,0.48c-1.78,0.28-1.38-0.45-1.38,1.81
			c0,1.07-0.99,0.78-1.58,1.11c-0.83,0.45-1.67,1.68-2.53,2.31c-1.47,1.08-1.43,2.04-3.19,1.45c-0.77-0.26-2.36,3.49-3.28,4.1
			c-1.14,0.75-1.04,1.38-1.79,2.29c-0.71,0.97-1.87-0.33-2.85,0.82c-0.57,0.66-1.03,1.99-1.09,2.88c-0.03,0.57,0.64,0.73,0.68,1.16
			c0.06,0.69-0.68,1.98-0.14,2.4c0.44,0.35,0.72,0.37,0.72,1c0,1.37,1.22,2.2,1.13,3.88c-0.05,0.98-0.85,1.39-0.88,1.95
			c-0.03,0.5,0.27,0.75,0.05,1.27c-0.1,0.23-0.67,0.76-1.16,1.12l0,0c-0.25,0.19-0.48,0.33-0.61,0.36c-0.52,0.11-0.91,0-1.45,0.05
			c-1.16,0.31-2.84,3.21-3.98,2.54c-0.53-0.32-1.42-2.76-0.48-2.9c2.64-0.39,1.44-3.85-1.61-2.15c-1.23,0.68-3.76,0.36-4.23,1.9
			c-0.29,0.96-1.43,1.26-1.95,2.27c-0.53,1.04,1.19,1.57-0.14,2.9c-1.2,1.2-0.2,1.67-2.15,2.27c-1.44,0.44-3.13,0.34-4.71,0.73
			c-1.1,0.49-1.94,1.55-2.92,1.95l0,0c-0.74,0.3-0.99-0.79-1.74-0.79c-0.74,0-1.04,0.06-1.65-0.36c-0.65-0.46-1.41-0.2-2.15-0.43
			c-0.52-0.16-2.02-1.68-2.13-2.2c-0.45-2.26-1.16-0.9-2.6-1.86c-1.03-0.68-0.68,0.08-1,0.77l-0.81,1.77
			c-0.09,0.2-2.2-0.82-2.56-0.97c-1.74-0.39-1.92,0.61-1.92-1.61c0-0.56-1.48-1.94-1.95-2.06c-0.68-0.18-1.83,0.22-2.58,0.25
			c-1.96,0.08-1.56,0.51-3.01-0.79c-0.45-0.41-1.17,0.12-1.61-0.36c-1.34-1.48,2.47-2.96-0.27-3.11c-1.09-0.06-2.06-0.21-2.83-1.04
			l-1.65-0.5l0,0c-0.88-1.44-0.73,0.2-1.76-0.05c-1.13-0.27-1.58,1.36-2.96,1.29c-1.83-0.1-1.87,0.43-2.78-1.63
			c-0.45-1.01-1.13-3.1-1.88-3.85c-0.4-0.4-1.15-0.96-1.15-1.61c-0.01,0.04,0.01,0.07,0.02,0.11c-0.42-0.94-1.38-1.54-1.99-2.36
			c-0.49-0.66-1.1-1.66-1.29-2.43c-0.18-0.71-0.29-0.78-0.97-0.93c-0.55-0.12-0.34-1.46-0.18-1.81c0.15-0.33,0.32-0.45,0.2-0.8
			c-0.03-0.1-0.48,0.12-0.72-0.09l0.2-1.81c0.11-0.45,0.21-0.91,0.34-1.36c0.31-1.11,1.16-1.74,1.65-2.65
			c0.19-0.35,0.38-0.7,0.57-1.04c0.24-0.44-0.88-0.1-0.84-0.93c0.05-0.92,0.21-1.92,0.27-2.81c0.03-0.5,0.06-0.94,0.18-1.41
			c0.21-0.86,0.7-0.67,1.22-1.09c0.55-0.44,0.78-1.03,1.27-1.43c0.32-0.26,0.64-0.16,0.9-0.41c0.81-0.75,0.58-1.99,2.08-1.99
			c0.98,0,2.09-2.51,2.83-2.18c0.04-0.36,0.53-1.86,0.39-2.22c-0.11-0.29-0.14-0.55-0.14-0.86c0.3-0.15,0.82-0.93,0.88-1.27
			c0.09-0.52,0.81-0.99,1.09-1.47c0.33-0.58,1.82-2.76,1.65-3.24c-0.12-0.33-0.45-0.42-0.45-0.86c0-0.53,0.18-0.71-0.02-1.25
			c-0.47-0.79-2.01-2.31-3.01-2.31c-1.19,0-3.13,1.11-4.05-0.02c-0.15-0.18,0.23-1.78,0.2-2.24c-0.07-1.25-1.25-0.81-2.31-1.22
			c-1.55-0.61-2.75-2.58-1.97-4.35c0.11-0.25,0.15-0.43,0.15-0.57l0,0l1.84-0.54c0.59-0.34,0.71-3.05,1.22-3.85
			c0.35-0.55,2.41-3.55,3.12-1.77c0.38,0.95-0.18,2.18,0.38,3.2c0.65,1.17,1.9-0.65,1.88-1.5c-0.03-1.05-2.19-5.78,1.02-3.4
			c1.33,0.98,3.32,1.77,4.98,1.84c0.74,0.03,2.87-0.53,3.26,0.27c0.32,0.68,0.04,1.45,0.77,1.95c2.91,1.97,0.06,3.14-0.23,5.44
			c-0.21,0.87,0.37,1.76,0.93,2.38c1.41,1.56,4.04-1.27,5.97,0.34c1.73,1.44,2.63,1.13,4.77,0.93c2.4-0.23,3.88,0.74,6.33-0.23
			c3.23-1.28-0.04,3.74,2.44,3.94c1.26,0.11,1.51,0.82,2.49,1.41c0.75,0.44,1.72,0.51,2.31,1.13c1.55,1.65,3.74-3.32,5.5-3.67
			c0.84,0.85-0.18,2.64,1.02,3.26c0.79,0.41,1.68-1.4,2.26-1.93c0.66-0.6,0.98,0.25,1.45,0.39c0.21,0.06,2.48-0.49,2.69-0.61
			c1.23-0.74,0.86-2.28,2.92-2.02c1.76,0.15,2.06,2.45,3.76,2.15c0.79-0.14,3.19-0.24,3.69-0.7c1.5-1.4-2.73-4.85,1.4-4.96
			c1.58-0.04,3.08,2.01,4.84,2.27c3.99,0.58,2.78-3.71,3.48-6.12c0.15-0.52-0.02-3.21,1.09-2.72c1.71,0.76,2.12,1.5,4.28,1.27
			c1.3-0.36,2-0.33,2.78-1.54c0.52-0.81,1.08-3.63,2.67-2.22c1.02,0.91,3.48-0.03,4.54-0.76l0,0c0.41,0.18,0.82,0.36,1.16,0.6
			c2.11,1.47,2.82-0.05,4.93-0.34c0.28-0.04,1.22-0.56,2.14-1.17l0,0C340.57,235.3,341.59,234.5,341.59,234.16L341.59,234.16z"></path>
	</g>
	<g id="caceres">
		<path class="st0 fase_1" d="M300.99,190.54c0.49,0.17,0.95,0.46,1.18,0.73c0.41,0.47,0.65,1.07,1.38,1.11c1,0.06,0.8,1.25,1.9,1.77
			c1.31,0.61,2.02,1.59,3.6,0.79c0.8,0.07,1.63-0.32,2.13-0.93c0.42-0.51,0.65-1.18,1.18-1.56c0.24-0.17,0.91-0.1,1.22-0.14
			c0.25-0.03,2.33-1.13,1.95-0.29c-0.32,0.68-0.81,1.32-0.81,2.09c0,0.53,0.09,1.04,0.07,1.56c-0.04,0.94-0.29,1.35,0.25,2.22
			c0.36,0.58,0.77,1.06,0.9,1.71l0,0l-1.24,0.76c0.16,0.58-0.11,1.47-0.11,2.13c0,1.14,0.31,2.22,0.09,3.35
			c-0.23,1.21-0.54,2.28-0.86,3.47c-0.21,0.38-1.03,2.57-0.43,2.74c0.43,0.12,0.81-0.25,1.22-0.36c0.68-0.19,1.52-0.41,2.17-0.05
			c0.75,0.42,0.16,3.75-0.11,4.58l-0.18,1.16l1.45,0.91l1.24-0.25l1.23-0.84l1.37-0.98l0.9-0.14l0.31,1.65v0.29
			c-0.9,0.23,0.51,1.39,0.63,1.59c0.41,0.65-0.42,2.86-0.52,3.74c-0.1,0.86-0.69,2.23-1.31,2.86c-1.08,1.08,0.13,0.81,0.93,1.81
			c1.03,1.3,1.77,3.35,3.28,4.1c0.79,0.4,1.48,0.22,1.99,1.04l1.56,2.49c0.38,0.62,1.07,0.93,1.75,1.24l0,0
			c-1.06,0.73-3.51,1.67-4.54,0.76c-1.58-1.41-2.15,1.41-2.67,2.22c-0.78,1.21-1.48,1.19-2.78,1.54c-2.16,0.23-2.56-0.51-4.28-1.27
			c-1.1-0.49-0.94,2.2-1.09,2.72c-0.71,2.41,0.51,6.71-3.48,6.12c-1.76-0.26-3.26-2.31-4.84-2.27c-4.14,0.12,0.1,3.56-1.4,4.96
			c-0.49,0.46-2.9,0.57-3.69,0.7c-1.7,0.3-1.99-2-3.76-2.15c-2.06-0.27-1.69,1.28-2.92,2.02c-0.21,0.13-2.48,0.67-2.69,0.61
			c-0.47-0.14-0.79-0.99-1.45-0.39c-0.58,0.53-1.47,2.34-2.26,1.93c-1.2-0.62-0.17-2.42-1.02-3.26c-1.75,0.36-3.95,5.32-5.5,3.67
			c-0.59-0.62-1.56-0.69-2.31-1.13c-0.98-0.58-1.23-1.29-2.49-1.41c-2.48-0.21,0.78-5.22-2.44-3.94c-2.45,0.97-3.93,0-6.33,0.23
			c-2.15,0.2-3.04,0.52-4.77-0.93c-1.93-1.61-4.56,1.22-5.97-0.34c-0.56-0.62-1.13-1.51-0.93-2.38c0.28-2.3,3.13-3.47,0.23-5.44
			c-0.73-0.5-0.45-1.27-0.77-1.95c-0.38-0.8-2.52-0.24-3.26-0.27c-1.66-0.07-3.65-0.85-4.98-1.84c-3.21-2.38-1.05,2.35-1.02,3.4
			c0.03,0.85-1.23,2.66-1.88,1.5c-0.57-1.02-0.01-2.24-0.38-3.2c-0.71-1.79-2.77,1.21-3.12,1.77c-0.51,0.81-0.63,3.51-1.22,3.85
			l-1.84,0.54l0,0c-0.02-0.42-0.49-0.49-0.94-0.99c-0.44-0.49-1.43-1.72-1.63-2.24c-0.28-0.56-0.18-1.06-0.29-1.65
			c-0.19-1.01-0.48-0.29,0.09-1.36c0.31-0.58,0.71-3.05,0.18-3.4c-0.69-0.46-1.51-0.8-2.26-1.16c-1.32-0.62-0.95-2.33-1.97-2.33
			c-0.87,0-0.98-0.43-1.36-1.02c-0.58-0.9-0.87-1.15-1.65-1.81c-0.37-0.32-1.35-1.93-1.54-2.38l-0.47-1.59
			c0.56-0.11,1.21-0.36,1.8-0.36c0.6,0,1.24,0.34,1.79,0.57c0.69,0.28,1.34,0.32,2.06,0.23c1.16-0.14,1.9,1.11,3.03,0.25
			c1.43-1.08,2.63-1.23,4.28-0.73c0.7,0.22,1.27,0.71,2.1,0.52c1.03-0.23,1.28-0.64,2.51-0.7c0.65-0.03,1.36,0.55,2.04,0.29
			c0.42-0.16,0.78-0.4,1.3-0.6l0.17-1.01c0-0.99,0.52-1.38,0.79-2.2c0.21-0.64-0.12-1.45,0.23-2.04c0.36-0.61,0.16-0.82-0.07-1.38
			c-0.23-0.57,0.17-1.47,0.84-1.47c0.94,0,1.06-1.19,1.79-1.72c1.1-0.46,0.54-1.96,0.54-2.81c0-1.03,1.35-1.48,1.11-2.31
			c-0.5-1.75,1.09-3.67-0.59-5.35c-0.45-0.45-0.82-0.29-1.04-1.02c-0.92-3-1.8-0.93-3.57-2.36c-0.6-0.49-0.47-1.89-0.18-2.52
			c-0.12-0.94-0.92-0.55-0.27-1.75c0.23-0.42,0.63-1.28,0.72-1.68c0.36-0.05,1.07-0.02,1.38-0.14c0.37-0.14,0.44-0.75,0.86-0.95
			c0.43-0.21,2.35-0.28,3.01-0.41l0,0l1.33,0.57c1.23,0.5,22.16-15.62,23.61-13.91l3.54,3.82c0.21,0.25,0.58,0.23,0.88,0.18
			c1.89-0.31-0.53,0.99,1.2,1.5c2.61,0.76-0.29,2.68,0.88,3.26c0.42,0.21,2.44,0.6,2.49,0.66c0.42,0.55,0.22,1.62,1.2,1.84
			c2.26,0.27,2.08,2.73,3.82,0.11c0.67-1,1.06-1.03,2.1-1.31c0.74-0.2,1.2-1.23,2.01-1.43c0.01,0.03,0.02,0.04,0.05,0.05
			c0,0.72,0.34,1.36,0.16,2.09c-0.4,1.58,0.87,1.9,2.28,1.36C300.12,190.32,300.57,190.38,300.99,190.54L300.99,190.54z"></path>
	</g>
</g>
<g id="Region_de_murcia">
	<g id="murcia">
		<path class="st0 fase_1" d="M464.81,340.7l-4.14-3.4c-1.1,0.1-1.27-0.52-2.33,0.02c-1.67,0.86-5.08-5.17-5.99-6.57
			c-0.79-1.22-2.03-3.05-2.44-4.4c-0.27-0.86-0.24-1.42-0.2-2.31c0.02-0.42,0.28-0.83,0.36-1.25c0.12-0.61-0.21-1-0.25-1.56
			c0.25-0.98-0.36-2.72,0.14-3.56c0.37-0.62,0.83-1.73,0.97-2.43c-0.66,0-1.18,0.57-1.95,0.39c-0.57-0.14-1.12-0.83-1.72-0.73
			c-1.39,0.25-1.7-0.47-2.96-0.54c-1.53-0.09-3.9-1.09-4.84-2.4c-0.66-0.92-0.94-1.73-1.47-2.7c-0.4-0.74-1.52-0.9-1.9-1.59l0,0
			l0.25-1.29c0.55-0.37,1.04-1.26,1.54-1.77c0.88-0.9,1.47-1.15,2.17-2.22c0.59-0.89,0.7-1.54,0.86-2.56
			c0.22-1.43,2.03-2.4,3.37-2.61c2.01-0.31,2.1-1.39,3.12-3.06c1.12-0.88,0.93-0.27,1.92-0.02c0.54,1.9,6.94-2.01,7.74-2.47
			c2.47-1.44,2.47-1.89,5.2-1.59c0.47,0.95-0.16,1.43,0.77,2.45c1.58,1.73,4.32,0.1,5.72-0.97c2.26-1.73,1.25-2.09,1.99-4.47
			c-0.16-0.84,0.08-1.87-0.2-2.7c-0.33-0.95-0.98-3.41-0.18-4.26c1.1-1.17,1.72-0.3,1.72-2.38c0-1.42,1.01-3.65,2.35-4.28
			c0.95-0.45,1.07-0.18,1.63,0.39c0.62,0.63,1.97-0.72,2.53-1c0.46-0.41,0.82-0.89,1.2-1.36c0.82-1.02,4.98-1.56,5.36-0.11
			c0.28,1.05,0.38,1.41,1.27,2.18c0.33,0.28,0.8,0.55,1.06,0.84c0.36,0.4,1.06,0.47,1.36,0.77l0.93,0.18l0,0l-0.09,0.93
			c-0.08,0.25,1.52,0.21,1.58,0.55c0.07,0.37,0.74,0.89,0.83,1.25c0.17,0.7,0.89,3.3,0.42,3.91c-0.31,0.94-1.18,1.29-0.83,2.08
			c0.33,0.74,0.69,2.85,0.11,3.27c-1.28,0.94-2.09,2.31-0.53,2.72c1.46,0.38,1.29,1,2.08,2.58c2.03,4.09,0.25,7.29,0.25,10.81
			c0,1.5,1.56,1.43,2.33,2.66c1.51,2.44-0.62,4.99,1.5,6.82c1.24,1.07,1.43,2.1,2.91,2.74l0.25,2.33c0,1.03-0.69,1.54-0.85,1.54
			c-0.06-0.17-1.22-1.38-1.22-0.57c0,0.53-0.88,1.82-1.29,2.13c-1.1,1.65-2.37,1.77-0.54,3.47c0.97,0.9,2.03,2.02,3.35,2.36
			c0.3,0.08,2.21,0.43,2.28,0.66h0.05c0,1.97-5.43,2.37-6.67,2.83c-1.09,0.41-1.63,1.2-2.87,1.02c-1.15-0.16-1.72-1.04-2.49-1.04
			c-0.6,0-1.24,0.5-1.79,0.09c-0.64-0.47-1.39-0.71-2.08-0.09c-0.91,0.81-0.75,1.33-0.75,2.22h-0.05l-2.01-0.75
			c-1.67-0.83-2.09-1.87-4.12-0.61c-1.79,1.11-2.56-0.57-4.61,1.59c-0.86,0.91-2.35,1.47-3.01,2.38c-1.41,1.95-0.14,2.79-2.71,3.24
			C467.61,338.74,466.14,339.69,464.81,340.7L464.81,340.7z"></path>
	</g>
</g>
<g id="canarias">
	<g id="la_graciosa">
		<path class="st0 fase_0" d="M188.24,393.67c-0.04-0.15-0.22-0.37-0.34-0.43c-0.19-0.09-0.51,0.06-0.59,0.23
			c-0.03,0.07-0.02,0.13-0.02,0.2c-0.01,0.16-0.07,0.29,0.05,0.41c0.17,0.18,0.47,0.2,0.7,0.16L188.24,393.67L188.24,393.67z
			 M187.31,401.92c0.58-0.14,1.01-0.97,1.29-1.43c0.09-0.14,0.22-0.34,0.2-0.5c-0.01-0.09-0.31-0.24-0.38-0.34
			c-0.13-0.17-0.53-0.96-0.66-0.48c-0.06,0.23-0.03,0.7-0.14,0.88c-0.15,0.26-0.38,0.68-0.66,0.82c-0.16,0.08-0.2,0.26-0.27,0.41
			c-0.17,0.36-0.05,0.36,0.14,0.66L187.31,401.92L187.31,401.92z"></path>
	</g>
	<g id="lanzarote">
		<path class="st0 fase_1" d="M190.82,402.44l0.45,0.23c0,1.29,0.12,1.98-0.86,2.81c-1.42,1.2,0.06,2.33-0.52,3.79
			c-0.44,1.11-0.25,1.83-1.13,2.83c-0.25,0.29-2.65,2.27-2.92,2.2c-0.55-0.15-1.1-0.23-1.58,0.23c-0.67,0.63-1.31,1.25-2.28,1.25
			c-2.52,0-4.34,1.23-5.16,3.7h-0.05c-0.05,0.2-0.72-0.84-1.02-0.86c-0.46-0.04-2.29,0.38-2.4-0.02c-0.46-1.66,1.34-2.43,1.43-4.19
			c0.24-5.24,4.58-5.73,8.48-7.68c1.43-0.72,0.87-0.62,2.26,0.11c2.36,1.25,1.71-5.79,3.89-5.21
			C189.73,401.69,190.6,402.55,190.82,402.44L190.82,402.44z"></path>
	</g>
	<g id="fuerteventura">
		<path class="st0 fase_1" d="M174.37,427.22c0.04,0.72,0.36,1.49,0.57,2.18c0.16,0.55,0,1.14,0,1.7c0,1.24,0.04,2.39-0.72,3.4
			c-1.23,1.61-0.23,4.05-0.75,5.92l-1.49,4.67c-0.82,2.56-1.22,1.76-3.17,2.36c-3.11,0.95-9.01,1.45-10.56,4.58
			c-0.44,0.88-3.11,3.91-3.91,3.65c-1.11-0.36-2.34-0.31-3.39-1c-1.41-0.93-2.53,0.39-2.53-1.65c4.31,0.85,8.76-1.85,10.93-5.37
			c0.79-1.28,0.2-2.55,0.45-3.94c0.25-1.37,1.84-2.5,2.26-4.33c0.14-0.51-0.01-1.05,0.38-1.45c0.65-0.65,1.49-1.22,1.95-1.99
			c1.64-2.82,2.19-5.77,3.48-8.7c0.07-0.46-0.17-0.92-0.11-1.43c0.18-1.55,3.33-1.68,4.41-2.06c0.8-0.28,1.18,0.55,1.79,0.93
			c1.05,0.65,0.15,1.07,0.38,1.95L174.37,427.22L174.37,427.22z"></path>
	</g>
	<g id="gran_canaria">
		<path class="st0 fase_1" d="M108.84,467.28c1.05-0.35,2.26-0.32,2.26-1.65c0-1.32,1.36-1.35,1.04-2.63c-0.22-0.88-0.06-1.25,0.54-1.9
			c0.32-0.34,0.02-1.86,0.02-2.45c0-1.92-0.58-1.76-1.29-3.24c-0.35-0.73-0.36-3.88-0.36-4.83c-0.03-0.01-0.07-0.09-0.09-0.02
			c-0.62,0-1.05,1.1-1.61,1.16c-0.86,0.08-1.47-0.44-2.26-0.75c-0.99-0.39-4.6,0.63-5.04-0.7c-2.52,0-1.99,1.02-2.71,2.83
			c-0.84,2.11-1.95,3.43-4.32,4.01c-0.13,0.91-0.45,1.7-0.79,2.54c-0.29,0.71,0.59,1.88,0.75,2.72c0.35,0.53,0.47,1.09,0.52,1.7
			c0.07,0.74,1.57,1.57,2.1,2.2c1.03,1.21,3.14,2.21,4.59,2.79c2.64,1.06,3.3-0.85,5.59-1.45L108.84,467.28z"></path>
	</g>
	<g id="tenerife">
		<path class="st0 fase_1" d="M69.39,436.49c-2.19,2.2-4.62,3.7-7.85,3.24c-2.28-0.33-4.79,2.68-7.33,0.41c-0.07-0.06-3.41,1.18-3.33,1.81
			c0.05,0.01,0.29,0.27,0.29,0.29c0.08,0.02,0.36,0.28,0.38,0.36c1.45,0.29,2.33,3.24,2.51,4.44c0.32,2.07,1.61,4.34,3.08,5.92
			l1.95,2.61c0.6,0.8,0.08,1.52,0.48,2.24c0.35,0.64,2.03-0.42,2.38-0.63c1.29-0.82,2.65-1.42,3.96-0.41
			c0.08,0.06,0.22-0.79,0.43-0.95c1.92-1.55,2.64-3.16,4.05-5.15c0.54-0.76,0.26-1.89,0.61-2.7l1.15-2.67
			c1.1-1.45,0.44-2.28,1.06-3.7c0.7-1.58,1.33-1.69,2.53-2.77c1.81-1.63,2.94-3.32,5.25-4.42c1.46-0.7,2.47-1.71,1.27-3.24
			c-0.2-0.25-2.79,0.92-3.53,0.91l-2.69-0.05c-0.73-0.01-0.95-0.4-1.38,0.2c-0.59,0.82-1.87,0.61-2.56,1.25
			c-0.77,0.71-1.1,1.62-1.99,2.29L69.39,436.49L69.39,436.49z"></path>
	</g>
	<g id="el_hierro">
		<path class="st0 fase_1" d="M5.5,471.4c-0.75-0.58-2.32-0.77-3.28-0.77c-1.64,0-0.86-0.53-0.86-1.5c0-1.83,0.75-0.68,1.85-0.68
			c2.39,0,3.69-1.12,4.75-3.04c0.39-0.71,2.32-1.43,3.08-1.11c0.87,0.36,0.52,2.57,0.23,3.2c-0.59,1.25-1.94,1.93-2.08,3.38
			c-0.09,0.9-2.31,1.22-3.01,0.95L5.5,471.4z"></path>
	</g>
	<g id="la_gomera">
		<path class="st0 fase_1" d="M35.66,455.69c-0.9-0.39-0.77-1.25-1.22-1.9c-0.7-1.02-0.74-1.1-0.36-2.31c0.4-1.29,0.56-3.31,2.51-3.31
			c0.48,0,0.89,0.38,1.36,0.5c1.1,0.27,1.75,0.35,2.62,1.16c1.13,1.04,2.74,1.21,2.74,3.08c0,0.81-0.56,1.2-1.06,1.75
			c-0.28,0.3-1.51,1.21-1.58,1.52c-0.53,0.07-1.14,0.6-1.85,0.63c-0.91,0.05-1.61-0.24-2.44-0.57L35.66,455.69L35.66,455.69z"></path>
	</g>
	<g id="la_palma">
		<path class="st0 fase_1" d="M12.56,434c-0.23-1.21-0.55-1.51-1.22-2.43c-0.63-0.85-0.81-0.97-1.09-2.11c-0.21-0.88-1-1.1-1-1.93
			c0-1.67-1.32-2.87-1.67-4.35c-0.33-1.37,1.64-2.64,2.38-3.63c1.11-1.49,1.67,0.15,3.33,0.23c1.28,0.06,2.33-0.92,2.9,0.36
			c0.86,1.95,3.17,3.68,1.31,5.53c-1.87,1.87,0.84,4.99-1.4,7.41c-0.21,0.23-1.54,2.62-1.54,2.9c0,1.75-1.92,0.74-1.92-1.09
			L12.56,434L12.56,434z"></path>
	</g>
	<g id="linea">
		<path class="st1" d="M7.74,369.04h158.54l59.1,37.71v65.14"></path>
	</g>
</g>
<g id="">
	<g id="formentera">
		<path class="st0 fase_1" d="M591.51,274.4c-0.44,0-1.06-0.51-1.45-0.73c-0.19-0.11-1.12-1.26-1.18-1.43c-0.47,0-0.81,0.19-1.33,0.09
			c-0.43-0.08-1.16-0.19-1.56-0.09c-0.13,0.52,0.09,1.49,0.09,2.09c0,0.55-0.04,1.26,0.09,1.77l0.02,0.02
			c0.28,0,1.28-1.67,1.67-1.95c0.99-0.7,2.34,0,3.01,0.75c0.33,0.37,0.65,0.68,1.18,0.68c0.18,0,1.23,0,1.27-0.16
			c0.06-0.01,0.08-0.08,0.14-0.09c0.06-0.4,0.34-1.05,0.05-1.45c-0.29-0.4-0.93,0.29-1.18,0.43L591.51,274.4z"></path>
	</g>
	<g id="ibiza">
		<path class="st0 fase_1" d="M586.49,263.86c-0.3-1.21,4.47-2.12,5.11-3.94c0.37-1.05-0.22-1.37,0.97-1.65c1.41-0.34,1.16-1.77,2.28-2.15
			c-0.42-0.57-0.92-1.23-0.86-2.06c0.1-1.39,0.78-1.29-1.18-1.29c-1.05,0-0.52-0.67-0.97-1.07c-0.06-0.05-1.41,0.95-2.13,0.95
			c-0.98,0-2.25,0.68-3.08,1.2c-0.72,0.45-0.99,0.36-1.81,0.36c-0.68,0-3.61,3.48-2.67,4.1c1.53,1.02,0.47,1.25-0.86,1.25
			c-2.66,0-1.21-0.16-1.56,1.61c-0.13,0.67-0.39,1.19-0.43,1.93c-0.08,1.41,1.17,2.15,1.67,0.66l0.54,0.14
			c0.52,0.19,2.21,0.75,2.67,0.66c0.26,0.08,0.46,0.43,0.58,0.67c0.19,0.36,0.42,0.75,0.87,0.75c0.23,0,0.57,0,0.79,0.07
			c0.12,0.04,0.39,0.17,0.51,0.12c0.13-0.68-0.48-1.37-0.48-2.07L586.49,263.86L586.49,263.86z"></path>
	</g>
	<g id="menorca">
		<path class="st0 fase_1" d="M694.28,201.04c-0.98,1.97-1.82,1.06-3.71,1.36c-1.75,0.28-5.24-0.3-6.79,0.5c-0.28,0.14-1.12,1.3-1.2,1.59
			c-0.39,1.39,0.52,1.11,1.38,1.11c0,1.17-0.02,2.52-0.27,3.65c-0.24,1.07,2.83,0.53,3.51,0.43c1.51-0.22,2.71-0.75,4.21-0.05
			c2.45,1.16,4.5,3.12,6.95,4.22c2.97,1.33,5.11,3.35,4.91-1.65c-0.09-2.26-1.3-4.71-1.81-6.76c-1.24,0-3.5-1.15-3.76-2.24
			c-0.39-1.66-0.64-1.46-1.85-0.82c-1.57,0.83-0.44-1.16-1.33-1.5L694.28,201.04L694.28,201.04z"></path>
	</g>
	<g id="mallorca">
		<path class="st0 fase_1" d="M637.13,217.14c-1.09,0.64-2.07,2.4-3.08,3.26c-0.57,0.49-1.15,0.61-1.85,0.75
			c-1.15,0.22-2.34,1.77-3.39,2.52l-3.01,2.13l-2.15,1.36c-0.44,0.28,2.66,3.11,3.33,3.2c0.41,0.05,1.91-0.12,2.17,0.09
			c0.09,0.08-0.25,0.88-0.2,1.18c0.17,1.02,2.55,2.87,3.12,1.2c0.27-0.79,0.62-2.5,1.4-2.9c0.33-0.17,0.69,0.14,1.04-0.09
			c0.2-0.14,0.25-0.79,0.43-0.79c0.05-0.15,0.59,0.36,0.79,0.16c0.26-0.26,0.3-0.86,0.61-1c0.82-0.21,3.07,1.67,3.78,2.2
			c0.93,0.69-0.29,1.81-0.29,2.72c0,0.82,1.06,1.23,0.77,2.02c-0.42,1.16,0.61,2.54,1.45,3.26c1,0.87,2.9,0.43,4.21,0.43
			c0.61,0.02,0.58,0.38,1.22,0.11c1.12-0.47,1.22-0.63,2.24,0.39c0.58,0.58,0.01,1.7,1,1.88c0.59,0.11,1.56,2.22,2.38,2.67
			c0.63,0.35,2.4-1.95,2.9-2.43c0.38-0.36,2.26-1.06,2.31-1.2c0.27-0.83,0.38-0.9,1.13-1.18c2.1-0.77,2.18-4.05,2.71-5.83
			c0.89-1.06,1.42-2.87,2.44-3.74c0.72-0.62,1.39-1.34,2.17-1.81c0.98-0.59-0.48-0.82,0.02-1.88c0.53-1.12,2.44-1.58,2.44-3.24
			c0-0.76,0.72-2.47,0.36-2.83c-1.22-1.23-3.35-2.57-5.07-3.02c-0.2-0.05-1.46,1.95-1.95,2.31c-1.52,1.15-3.08,0.33-4.48-0.66
			c-0.24-0.54-1.37-1.92-1.83-2.33c-1.58-1.4,1.05-1.36,1.85-1.99c0.51-0.4,0.49-0.79,0.16-1.25c-0.31-0.44,0.36-0.91,0.36-1.5
			c-0.1-0.03-2.2,0.95-2.56,1.07c-0.51,0.17-1.4,0.3-1.4-0.54c0-1.39,1.03-1.97,2.31-2.43c0.58-0.21,2.78-1.08,2.78-1.84
			c-1.17,0-2.44-0.23-3.3,0.63c-1.49,1.49-2.65,0.46-4.37,0.79c-0.8,0.15-0.89,0.72-1.52,1.11c-0.62,0.38-1.6,0.08-2.35,0.2
			c-0.88,0.54-1.8,0.86-2.67,1.47c-0.7,0.5-1.41,0.81-2.28,0.84c-0.6,0.02-1.01,0.27-1.33,0.73c-0.95,1.34-3.48,1.26-4.28,3.02
			L637.13,217.14L637.13,217.14z"></path>
	</g>
</g>
<g id="melilla">
	<path class="st2 fase_1" d="M413.75,447.31c0,1.19-0.97,2.16-2.16,2.16s-2.16-0.97-2.16-2.16c0-1.19,0.97-2.16,2.16-2.16
		S413.75,446.12,413.75,447.31z"></path>
</g>
<g id="ceuta">
	<path class="st2 fase_1" d="M314.84,414.93c0,1.19-0.97,2.16-2.16,2.16s-2.16-0.97-2.16-2.16s0.97-2.16,2.16-2.16
		S314.84,413.74,314.84,414.93z"></path>
</g>
<g id="castilla_la_mancha">
	<g id="ciudad_real">
		<path class="st0 fase_0" d="M331.28,236.89l1.03-2.23c0.7-1.24,1.46-2.63,1.9-3.97c0.18-0.53,0.49-1.17,0.77-1.65
			c0.29-0.51,0.35-0.18,0.75-0.02c1,0.39,3.96,1.13,4.18,2.27c0.24,1.25-0.39,3.4-0.39,4.71l0,0c-0.92,0.61-1.86,1.13-2.14,1.17
			c-2.12,0.29-2.82,1.81-4.93,0.34C332.1,237.25,331.69,237.07,331.28,236.89L331.28,236.89L331.28,236.89z M341.59,234.16h0.05
			c0,0.24,0.88,0.9,0.88,1.56c0,1.55-0.82,2.04-1.13,3.35c-0.36,1.53-0.77,2.09-2.49,2.09c0.08,0.43,0.83,1.58,1.02,2.31
			c0.44,1.71,1.45,1.99,2.33,3.31c0.17,0.26,1.41,1.91,1.27,2.09c-0.12,0.14-3.72-0.91-4.57-0.91c-0.68,0-2.03-0.74-2.38-0.09
			c-0.45,0.86-0.21,1.29-1.29,1.68c-0.17,1.83-1.54,2.95,0.38,4.28c0.62,0.43,0.54,1.43-0.11,1.84c-0.48,0.3-2.38-0.07-2.6-0.52
			c-0.61,0-1.22-0.48-1.81-0.48c0,0.44,0.54,1.05,0.72,1.47c0.8,1.91,0.34,3.59,2.92,4.19c2.37,0.55,1.25,2.04,0.59,3.35
			c-0.46,0.33-0.68-0.08-1.13-0.18c-0.59-0.13-1.76-0.04-2.06,0.52c-0.63,1.2-1.11,2.4-1.11,3.81c0,1.39-0.07,2.49-1.38,3.31
			l-0.88,0.61l0,0l0.61,1.22c0.89,2.53,4.9-0.42,5.16,1.86c0.24,2.11,0.16,3.23,2.65,3.7c0.9,0.17,3.33,0.66,3.96,1.29
			c0.83,0.83,1.52,1.08,2.47,1.7c1.49,0.98,0.86,1.93,2.83,2.54c0.94,0.29,2.07,1.24,2.87,1.88c1.16,1.48,2.48,2.02,4.05,2.86
			c0.65,0.34,2.49,2.16,2.94,2.22l0,0c0.6,0.09,2.01-1.71,2.62-2.15c1.19-0.86,3.31,0.75,4.73,0.75c1.18,0,2.35-0.37,3.53-0.16
			c1.03,0.19,2.18,0.68,3.23,0.68c1.13,0.13,4.63,0.05,5.66-0.41c1.14-0.51,1.33-3.36,3.17-2.22c0.58,0.36,1.08-0.24,1.72-0.16
			c1.22,0.16,2.27,0.75,3.42,1.16c2.78,0.98,0.87-2.54,2.56-2.54c0.84,0,1.22,0.44,1.22,1.25c0,1.05,1.41,1.14,2.24,1.2
			c1.63,0.49,1.77-0.31,3.08-1.13c1.28-0.81,0.49-1.45,1.13-2.27c0.1-0.13,2.09-0.68,2.35-0.68c0.47,0,1.06,0.96,1.63,1.2
			c1.21,0.51,3.05,0.45,4.37,0.73c2.31,0.48,4.08-2.96,5.23-0.5c0.11,0.24,1.31,2.08,1.58,1.84c0.16-0.15,0.35-1.12,0.63-1.47
			c0.4-0.49,0.91-0.88,1.45-1.22c1.32-0.38,0.9,0.28,1.7,0.57c2.02,0.72,3.71-2.3,5-3.31c0.29-0.23,0.58-0.28,0.87-0.23l0,0
			l-0.39-1.65c0.39-1.41-0.1-1.12,1.11-2.09c1.8-1.43,2.62-1.4,3.6-3.72c1.02-2.41-1.02-4.51-3.1-5.3c-0.69-0.26-0.9-2.84-0.9-3.49
			c0-0.85-1.33-2.48-2.13-2.81c-0.57-0.47-1.33-0.44-1.88-0.88c-0.48-0.39-1.11-1.15-1.47-1.65c-0.18-0.25,0.79-0.93,0.95-1.27
			c0.93-1.97,2.97-3.23,2.15-5.69c-1.02-3.08-0.12-4.3,1.97-6.85c0.78-0.82,1.56-1.8,1.56-2.99c0-0.53-0.12-0.92-0.25-1.41
			c-0.13-0.49-0.43-1.13-0.2-1.63l0,0l-1.29-1.38c-0.15-0.45,0.12-2.7-0.68-2.65c-1.32,0.08-1.03,2.53-2.44,2.81
			c-0.79,0.16-1.43-0.22-1.67-1.04c-0.22-0.75,0.56-1.57,0.25-2.2c-0.51-1.03-1.03-1.81-2.04-2.45l0,0c-1.72,0-3.07,0.77-4.82,0.43
			c-1.63-0.31-2.25-2.18-4.09-2.18c-1.33,0-2.18,1.29-3.51,1.29c-1.89,0-2.71-0.28-2.96,1.93c-0.16,1.44-0.62,1.13-1.88,1.13
			c-1.38,0-2.01,0.87-3.21,1.13c-0.83,0.66-1.16,1.93-2.1,2.63c-0.85,0.63-1.54,1.19-2.58,1.43c-1.94,0.44-3.55-0.64-5.45,0.82
			c-1.68,1.29-1.38,0.77-2.67-0.27c-1.35-1.09-2.53,1.37-3.62,1.43c-0.88,0.04-1.47-0.83-2.33-1c-2.34-0.67-1.59-1.93-2.24-3.49
			c-0.21-0.49-3.81-0.74-4.41-0.63c-1.22,0.22-2.01,0.19-2.01-1.27c0-1.14,0.2-2.76,0.47-3.85c0.18-0.74,1.63-0.43,1.99-1.16
			c0.39-0.78,0.09-1.88,0.52-2.52c0.68-1.01,1.42-1.18,0.61-2.58c-1.12-0.14-3.52,0.22-4.64,0.5c-1.19,0.29-0.55,3.36-1.95,3.88
			c-1.17,0.43-2.49,0.27-3.64-0.05c-1.56,0-3.72-1.26-5.18-1.81c-1.39-0.53-2.41,2.55-2.92,0.68c-0.25-0.94-0.35-1.94-0.7-2.81
			c-0.59,0-1.54,1.59-1.9,2.04c-0.76,0.96-1.76,1.66-2.6,2.56l-3.12,2.31H341.59z"></path>
	</g>
	<g id="cuenca">
		<path class="st0 fase_1" d="M472.18,198.05c-0.3,0.71-0.29,1.05,0.38,1.52c0.38,0.27,2.3,0.36,2.3,0.84c0,0.67,0.31,1.3,0.58,2.08
			c0.19,0.54,0.45,1.13,0.91,1.5c0.74,0.59,1.15-0.38,2.08,0.08c1.26,0.63,1.31,0.37,2.66-0.08l2.5,1.16l0,0l0.77,1.97l0,0
			c-0.67,0.91,0.25,1.55,0.06,2.6c-0.08,0.44-0.35,0.52-0.17,0.91c0.75,1.6-0.91,4.97-0.91,6.49c0,1.75-2.05,3.93-2.16,5.65
			c0.86,2.31-2.17,3.31-4.57,1.91c-2.83-1.64-1.22,4.14-3.33,5.07c-1.14,0.5,0.11,1.58-0.25,2.83c-0.59,2.06-3.91,2.09-2.95,2.88
			l0,0l-5.38,3.39c-0.09,0.26-4.93,3.58-5.5,3.7c-1.65,0.33-3.32-0.74-5.2-0.48c-1.12,0.15-2.95,0.37-3.71-0.59
			c-0.58-0.72-1.51-1.65-2.28-2.15c-2.28-1.46-0.79,1.62-0.54,3.04c0.68,3.86-1.54,0.65-3.33,1.34c-1.22,0.35-1.9,0.1-3.03,0.93
			c-2.59,1.89-1.83,0.7-3.46-1.63c-1.08-1.54-1.64,1.96-2.78,2.11c-1.74,0.22-4.05-7.02-6.11-4.85c-1.02,1.07-1.54-0.67-2.56-0.48
			c-1.16,0.22-1.88,0.77-3.23,0.77l-2.53-0.05l0,0l-1.29-1.38c-0.15-0.45,0.12-2.7-0.68-2.65c-1.32,0.08-1.03,2.53-2.44,2.81
			c-0.79,0.16-1.43-0.22-1.67-1.04c-0.22-0.75,0.56-1.57,0.25-2.2c-0.51-1.03-1.03-1.81-2.04-2.45l0,0l0.07-2.04
			c0-1.94-0.05-3.86,0.47-5.69c0.7-2.45-0.2-3.01-1.54-4.81c-0.49-1.1-1.31-2.12-1.83-3.22c-0.47-0.99-0.8-2-1.38-2.95
			c-0.84-1.36-2.69-3.83-2.42-5.46c0.19-1.15,0.73-2.69,0.47-3.85c-0.06-0.27-1.56-0.27-1.97-0.45l-0.79-2.71l0,0
			c3.39-0.64,2.22-3.21,3.6-5.5l0,0c1.58-0.49,2.54-1.42,2.89-2.6c0.51-1.75,1.89,1.17,2.15,1.52c0.51,0.69,1.27,1.03,2.13,0.79
			c1.31-0.36,1.07-1.42,1.7-2.31c0.34-0.48,0.79-0.36,1.06-0.66c0.38-0.41,0.39-1.03,0.47-1.54c1.32-3.3-1.07-6.37,0.93-9.73
			c0.79-1.32,1.41,0.64,1.99,1.2c1.69,1.63,1.86-0.95,2.31-1.52c0.39-0.49,1.25-0.65,1.81-0.86c1.5-0.56,5.03,1.54,5.36,0.73
			c0.37-0.92-0.68-1.84-0.95-2.65c-0.13-0.39-0.53-2.93-0.07-2.74c1.36,0.54,1.36,1.8,3.1,1.2c1.25-1.06,3.71,0.08,2.35-2.45
			c-0.97-1.81-1.7-2.1,0.5-2.2c0.57-0.03,0.85,0.84,1.13,1.31c0.82,1.39,1.67,0.98,2.28-0.32c0.83-1.74-0.78-3.33,1.9-2.27
			c2.5,0.99,3.8-2.15,5.57-0.82c1.04,0.79,5.78,5.12,6.42,1.88c0.59,0,2.88,6.97,3.05,7.82l2.33,3.42l0,0
			c-0.08,0.32,0.23,0.61,1.13,0.68l2.28,2.9c0.38,0.11,0.48,0.59,0.48,0.88c-0.73,0.1-1.06,2.17-0.54,2.24
			c0.08,0.33,0.86-1.37,1.15-1.45c0.84-0.24,1.96,2.84,2.78,3.13c1.18,0.42,1.45,1.09,2.17,2.02c0.63,0.81,0.93,0.89,1.88,0.59
			c0.59-0.19,0.74-0.01,1.2,0.32c0.72,0.52,1.6-0.43,2.17-0.23C472.69,195.32,472.21,197.8,472.18,198.05L472.18,198.05z"></path>
	</g>
	<g id="guadalajara">
		<path class="st0 fase_1" d="M411.37,135.25c0.9-0.73,10.44-2.47,10.44-0.67c0,1.31,12.68,6.96,13.51,7.48c0.3,0.15,12.39,3.1,12.45,2.48
			l0,0c1.6,0.24,2.61-0.82,4.03-0.82c0,0.19,0.49,0.68,0.54,1c0.35,2.06,1.25,0.05,2.04,0.84c0.36,0.36,0.3,1.18,0.81,1.59
			c1.15,0.92,2.53-0.06,3.42,1.72c0.47,0.93,1.71,2.55,2.62,3.13c0.16,0.61,2.56,2.78,3.14,3.17c0.5,0.33,0.75,0.59,0.86,0.84l0,0
			c0.2,0.45-0.1,0.87-0.3,1.63c-0.43,1.7,1.45,3.42,2.62,4.28c1.11,0.82,0.24,3.12,0.09,4.31c-0.13,1.01,1.39,4.73,0.29,5.28v0.07
			c-0.7,0.17,0.23,2.46-0.72,3.26c-0.78,0.66-0.7,0.91-1.88,0.63c-0.29-0.07-2.19-0.78-2.26-1c-0.96,0-1.32,5.74-1.67,6.71
			c-0.3,0.84-1.07-0.03-1.47,0.32c-0.47,0.42-0.68,1.54-1.29,1.79c-0.53,0.22-0.9,0.53-0.97,0.82l0,0l-2.33-3.42
			c-0.17-0.85-2.46-7.82-3.05-7.82c-0.64,3.24-5.38-1.09-6.42-1.88c-1.76-1.34-3.07,1.81-5.57,0.82c-2.68-1.06-1.07,0.52-1.9,2.27
			c-0.62,1.29-1.47,1.7-2.28,0.32c-0.28-0.47-0.56-1.34-1.13-1.31c-2.2,0.1-1.47,0.39-0.5,2.2c1.36,2.53-1.11,1.39-2.35,2.45
			c-1.74,0.59-1.74-0.66-3.1-1.2c-0.46-0.18-0.06,2.36,0.07,2.74c0.27,0.81,1.32,1.74,0.95,2.65c-0.33,0.82-3.86-1.28-5.36-0.73
			c-0.56,0.21-1.42,0.37-1.81,0.86c-0.45,0.56-0.62,3.15-2.31,1.52c-0.58-0.56-1.2-2.52-1.99-1.2c-1.99,3.35,0.39,6.43-0.93,9.73
			c-0.09,0.52-0.09,1.13-0.47,1.54c-0.27,0.29-0.72,0.18-1.06,0.66c-0.63,0.9-0.39,1.95-1.7,2.31c-0.85,0.24-1.62-0.1-2.13-0.79
			c-0.26-0.35-1.64-3.27-2.15-1.52c-0.35,1.18-1.3,2.11-2.89,2.6l0,0l-0.78-1.92c0-1.18-0.31-2.71-0.97-3.72
			c-0.68-1.02-2.37,1.18-2.76,1.81c-0.33,0.53-0.86-0.94-0.79-1.09c0.13-0.29,0.37-0.59,0.43-0.91c0.09-0.44,0.09-0.92,0.18-1.36
			c0.23-1.13,1.56-2.36,1.06-3.56c-0.29-0.7,0.63-1.19,0.77-1.79c0.08-0.35-0.6-1.49-0.9-1.72c-0.75-0.56-1.72-0.06-1.72-1.29
			c0-1.53-0.21-2.57-1.33-3.63c-0.8-0.75-2.38-0.32-2.38-1.5c0-1.17-0.44-0.86-1.13-1.36c-1.12-0.81-0.02-2.31-0.25-3.26
			c-0.13-0.53-1.13,0.22-1.49-0.32c-0.34-0.5-0.08-1.65-0.81-1.65c-0.87,0-1.89,0.91-2.58,0.18c-0.69-0.74-0.24-1.94-0.09-2.81
			c0-0.51,0.29-2.17-0.34-2.43c-0.51-0.21-2.07-0.04-1.74-0.79c0.58-1.33,0.88-2.44,1.31-3.85c0.26-0.86,1.79-0.67,0.63-1.95
			c-0.54-0.6,0.05-0.49,0.27-1c0.29-0.65,0.33-1.33,0.54-2.02c0.28-0.93,0.78-1.74,1.13-2.63c0-0.95-1.1-1.12-1.43-1.86
			c-0.27-0.61,0.52-1.52,0.52-2.18c-0.71-0.18-2.33,0.07-2.56-0.88c-0.18-0.76-0.4-1.62-1.2-1.88c-0.81-0.26-4.23-2.81-4.23-3.6
			c0,0,10.96-5.53,10.98-5.7C395.64,136.05,409.97,136.38,411.37,135.25z"></path>
	</g>
	<g id="toledo">
		<path class="st0 fase_0" d="M411.58,221.06c-0.49-1.1-1.31-2.12-1.83-3.22c-0.47-0.99-0.8-2-1.38-2.95c-0.84-1.36-2.69-3.83-2.42-5.46
			c0.19-1.15,0.73-2.69,0.48-3.85c-0.06-0.27-1.56-0.27-1.97-0.45l-0.79-2.71c-0.52,0.1-1.15,0.15-1.9,0.15
			c-1.43,0-2.69,0.21-3.96,0.42c-0.42-0.75-0.86-2.15-1.99-1.97c-0.37,0.06-0.78,0.31-0.79,0.71c-0.02,0.45-0.19,0.88-0.19,1.35
			l-0.27,0.32c-0.26,0.01-0.53,0.02-0.8,0.02c-1.48,0-2.74-0.01-3.92,0.07c-1.34,1.49-2.94,0.6-4.25,2.02
			c-0.45,0.49-0.8,1.3-1.29,1.68c-0.92,0.7-2.85,0.75-4,1.5c-1.16,0.75-3.66,4.64-4.71,2.31c-0.15-0.34-0.89-2.09-0.23-2.09
			c0.65,0,1.4-0.81,1.97-1.07c0.41-0.18,1.09,0.43,1.63,0.39c0.62-0.05,0.26-0.61,0.45-0.88c1.08-1.49,2.29-2.35,3.8-3.4
			c1.07-0.74,1.04-1.13,1.7-2.02c0.93-1.25,0.14-1.55-0.43-2.58c-1.24-0.52-2.78-1.67-4.09-0.95c-1.8,0.99-2.48-1.95-3.26-1.66
			c-1.43,0.54-3.21-1.94-4.82-1.54c-0.64,0.16-1.39-0.82-2.31-0.97c-1-0.17-1.83-1.88-2.01-1.88c-0.07-0.23-1.23,0.08-1.45,0.09
			c-2.29,0.1-4.18-1.29-6.29-1.29c-0.44,0.16-2.97,4.44-3.71,3.33c-0.62-0.93-1.15-2.95-1.15-4.08c0-0.46-1-2.2-1-0.86
			c0,0.82-2.4,2.72-3.05,3.29c-0.56,0.48-1.28,0.69-1.88,1.2c-0.55,0.48-0.81,1.39-1.79,1.02l-1.15-0.61c-1.3,0-2.04,1.07-3.23,1
			c-1.24-0.08-1.37-1.74-1.56-2.54c-0.15-0.61-0.89-1.07-0.5-1.68c0.36-0.57-2.16,0.4-2.31,0.41c-0.48,0.04-1.32-0.65-1.52-0.54
			c-0.38,0.2-0.38,1.6-0.48,2.04c-0.12,0.55-0.91,2.14-1.61,1.7c-0.61,0.26-1.98,1.13-2.38,1.68c-0.53,0.74,0.66,1.63-0.57,1.99
			c-1.3,0.39-3.48,2.47-3.48,0.02c0-1.65-0.71-1.22-1.95-0.7c-1.48,0.62-2.24,3.09-4.19,3.26c-1.08,0.1-2.27,0.33-2.19-1.16
			l-4.64-0.31l-1.24,0.76c0.16,0.58-0.11,1.47-0.11,2.13c0,1.14,0.31,2.22,0.09,3.35c-0.23,1.21-0.54,2.28-0.86,3.47
			c-0.21,0.38-1.03,2.57-0.43,2.74c0.43,0.12,0.81-0.25,1.22-0.36c0.68-0.19,1.52-0.41,2.17-0.05c0.75,0.42,0.16,3.75-0.11,4.58
			l-0.18,1.16l1.45,0.91l1.24-0.25l1.23-0.84l1.37-0.98l0.9-0.14l0.31,1.65v0.29c-0.9,0.23,0.51,1.39,0.63,1.59
			c0.41,0.65-0.42,2.86-0.52,3.74c-0.1,0.86-0.69,2.23-1.31,2.86c-1.08,1.08,0.13,0.81,0.93,1.81c1.03,1.3,1.77,3.35,3.28,4.1
			c0.79,0.4,1.48,0.22,1.99,1.04l1.56,2.49c0.38,0.62,1.07,0.93,1.75,1.24l1.03-2.23c0.7-1.24,1.46-2.63,1.9-3.97
			c0.18-0.53,0.49-1.17,0.77-1.66c0.29-0.51,0.35-0.18,0.75-0.02c1,0.39,3.96,1.13,4.18,2.27c0.24,1.25-0.39,3.4-0.39,4.71
			c1.04-0.69,2.06-1.49,2.06-1.83h0.05l3.12-2.31c0.84-0.9,1.84-1.61,2.6-2.56c0.36-0.45,1.31-2.04,1.9-2.04
			c0.35,0.87,0.45,1.88,0.7,2.81c0.51,1.87,1.53-1.21,2.92-0.68c1.46,0.56,3.62,1.81,5.18,1.81c1.15,0.31,2.47,0.48,3.64,0.05
			c1.39-0.51,0.76-3.58,1.95-3.88c1.12-0.28,3.52-0.64,4.64-0.5c0.81,1.41,0.07,1.57-0.61,2.58c-0.43,0.64-0.13,1.74-0.52,2.52
			c-0.36,0.73-1.81,0.41-1.99,1.16c-0.27,1.1-0.48,2.72-0.48,3.85c0,1.46,0.79,1.48,2.01,1.27c0.6-0.11,4.2,0.14,4.41,0.63
			c0.65,1.56-0.1,2.82,2.24,3.49c0.86,0.17,1.45,1.04,2.33,1c1.08-0.05,2.27-2.52,3.62-1.43c1.29,1.04,0.99,1.56,2.67,0.27
			c1.9-1.45,3.51-0.37,5.45-0.82c1.04-0.24,1.73-0.8,2.58-1.43c0.94-0.7,1.28-1.97,2.1-2.63c1.2-0.27,1.83-1.13,3.21-1.13
			c1.26,0,1.71,0.31,1.88-1.13c0.25-2.2,1.07-1.93,2.96-1.93c1.32,0,2.17-1.29,3.51-1.29c1.85,0,2.46,1.86,4.09,2.18
			c1.75,0.34,3.1-0.43,4.82-0.43l0.07-2.04c0-1.94-0.05-3.86,0.48-5.69C413.82,223.41,412.92,222.85,411.58,221.06z"></path>
	</g>
	<g id="albacete">
		<path class="st0 fase_0" d="M470.09,235.47l-5.38,3.39c-0.09,0.26-4.93,3.58-5.5,3.7c-1.65,0.33-3.32-0.74-5.2-0.48
			c-1.12,0.15-2.95,0.37-3.71-0.59c-0.58-0.72-1.51-1.65-2.28-2.15c-2.28-1.46-0.79,1.62-0.54,3.04c0.68,3.86-1.54,0.65-3.33,1.34
			c-1.22,0.35-1.9,0.1-3.03,0.93c-2.59,1.89-1.83,0.7-3.46-1.63c-1.08-1.54-1.64,1.96-2.78,2.11c-1.74,0.22-4.05-7.02-6.11-4.85
			c-1.02,1.07-1.54-0.67-2.56-0.48c-1.16,0.22-1.88,0.77-3.23,0.77l-2.53-0.05l0,0c-0.23,0.5,0.08,1.14,0.2,1.63
			c0.12,0.49,0.25,0.88,0.25,1.41c0,1.2-0.78,2.18-1.56,2.99c-2.09,2.54-2.99,3.76-1.97,6.85c0.82,2.46-1.22,3.72-2.15,5.69
			c-0.16,0.34-1.13,1.02-0.95,1.27c0.36,0.5,0.99,1.26,1.47,1.65c0.55,0.44,1.31,0.41,1.88,0.88c0.79,0.33,2.13,1.96,2.13,2.81
			c0,0.66,0.21,3.23,0.9,3.49c2.08,0.79,4.12,2.89,3.1,5.3c-0.98,2.32-1.79,2.28-3.6,3.72c-1.21,0.96-0.72,0.68-1.11,2.09l0.39,1.65
			l0,0c1.04,0.17,2.09,1.69,3.16,1.69c1.3,0,2.45-1.43,3.78-0.36c0.42,0.34,1.18,1.89,1.04,2.43c-0.78,2.97-0.34,1.77,2.08,2.79
			c1.27,0.53,0.48,3.43,0.7,4.78c0.3,0.49,0.42,0.64,1,0.82c1.11,0.34,0.88,0.15,0.88,1.38c0,0.61-0.29,1.22-0.29,1.88
			c0,1.76,0.26,1.86-0.93,3.45c-0.56,0.74-1.17,1.58-1.83,2.24c-0.22,0.22-1.36,1.14-1.36,1.41l0,0c0,0.33,3,1.96,3.51,2.2
			c1.15,0.55,2.36,1.09,3.66,1.2l1.24-0.16l0,0l0.25-1.29c0.55-0.37,1.04-1.26,1.54-1.77c0.88-0.9,1.47-1.15,2.17-2.22
			c0.59-0.89,0.7-1.54,0.86-2.56c0.22-1.43,2.03-2.4,3.37-2.61c2.01-0.31,2.1-1.39,3.12-3.06c1.12-0.88,0.93-0.27,1.92-0.02
			c0.54,1.9,6.94-2.01,7.74-2.47c2.47-1.44,2.47-1.89,5.2-1.59c0.47,0.95-0.16,1.43,0.77,2.45c1.58,1.73,4.32,0.1,5.72-0.97
			c2.26-1.73,1.25-2.09,1.99-4.47c-0.16-0.84,0.08-1.87-0.2-2.7c-0.33-0.95-0.98-3.41-0.18-4.26c1.1-1.17,1.72-0.3,1.72-2.38
			c0-1.42,1.01-3.65,2.35-4.28c0.95-0.45,1.07-0.18,1.63,0.39c0.62,0.63,1.97-0.72,2.53-1c0.46-0.41,0.82-0.89,1.2-1.36
			c0.82-1.02,4.98-1.56,5.36-0.11c0.28,1.05,0.38,1.41,1.27,2.18c0.33,0.28,0.8,0.55,1.06,0.84c0.36,0.4,1.06,0.47,1.36,0.77
			l0.93,0.18l0,0l2.66,2.81c2.58,0.65,3.96-3.95,3.33-4.99c-0.4-0.65-0.38-2.26,0.58-2.49c0.42-0.1-1.26-2.09-1.16-2.33l-0.42-0.83
			c0.21-0.52,1.35,0.13,1.25-0.5c-0.1-0.61,1.06-1.79,1-2.41c0-1.55-4.16-3.39-4.24-4.74c-0.06-1.03-2.99,0.51-3.56,0.86
			c-0.65,0.4-2.29,0.49-3.08,0.41c-2-0.21-0.6-0.7-1.68-2.27c-0.85-1.24-1.68-2.8-3.16-3.49c-1.15-0.54,0.13,0.1-0.17-0.91
			c0.19-0.56,0.63-0.09,1.08-0.5c0.3-0.27,1.65-0.52,1.75-0.91c0.24-0.95-0.24-1.75,0.58-2.58c1.29-1.3,11.44-12.71-1.68-9.21
			c-1.69,0.45-2.82-0.47-4.46-1c-1.01-0.32-0.7-1.56-1.68-1.93c-0.59-0.23-2.1-1.21-2.58-1.25c-1.48-0.11-2.06,1.08-2.66-0.49
			C470.89,236.09,470.44,235.75,470.09,235.47L470.09,235.47z"></path>
	</g>
</g>
<g id="andalucia">
	<g id="sevilla">
		<path class="st0 fase_1" d="M323.19,359.43c0.53-0.42,0.91-0.79,1.52-1.11c0.88-0.47,1.88-0.6,2.76-1.07c1.25-0.65,2.36-1.59,3.35-2.58
			c0.58-0.46,1.73-1.44,1.52-2.29c-0.15-0.61-1.09-0.73-1.33-1.31c-0.9-2.17,2.05,0.01,2.65-0.2c0.78-0.28,0.04-1.37,1.45-1.79
			c0.59-0.17,0.6,0.79,0.9,1.11c0.64,0.69,3.64-0.09,3.66-0.82c0.04-1.22-0.98-2.44,0.72-3.15l1.65-0.36l0,0
			c0-1.08-0.38-1.95-0.43-2.95c-0.03-0.61-1.48-2.25-2.04-2.13c-1.16,0.25-1.43,1.24-2.78,0.84c-1.19-0.35-1.82-2.12-2.08-3.22
			c-0.14-0.58-0.2-1.59-0.57-2.04c-0.49-0.61-1.37-0.42-1.81-1.29c-0.47-0.95-0.3-1.4-1.06-2.24c-1.35-1.35-0.19-3.62-0.38-5.26
			c-0.06-0.55-0.74-0.44-1.06-0.68c-0.62-0.47-0.02-1.3-0.02-1.88c0-0.98-0.74-1.46-1.18-2.22c-0.95-1.68-2.04-0.87-3.55-0.7
			c-2.16,0.24,1.37,4.07,1.22,4.44c-0.41,0-2.14-1.2-2.51-1.61c-0.52-0.58-1.78-0.94-2.51-0.75c-0.89,0.07-1.42,0.66-1.97,1.29
			c-0.83,0.96-1.12,1.13-2.35,1.31c-0.69,0.11-1.01,0.65-1.52,1.04c-0.54,0.42-1.32,0.68-1.7-0.09c-0.32-0.66-0.59-1.88-1-2.43
			c-0.33-0.44-0.68-0.87-0.86-1.41c-0.29-0.87,1.98-1.45,2.67-1.47c0.86-0.03,1.22-0.13,1.31-1.04c0-0.82-0.03-1.93-0.36-2.68
			c-0.44-0.99-0.9-0.86-0.9-2.22c0-2.93-2.64-1.69-3.21-4.38c-0.26-1.2,0.14-2.87-0.84-3.7c-1.82-1.54-2.04-1.96-2.04-4.42
			l-2.04-2.15l0,0c-0.25,0.19-0.48,0.33-0.61,0.36c-0.52,0.11-0.91,0-1.45,0.05c-1.16,0.31-2.84,3.21-3.98,2.54
			c-0.53-0.32-1.42-2.76-0.48-2.9c2.64-0.39,1.44-3.85-1.61-2.15c-1.23,0.68-3.76,0.36-4.23,1.9c-0.29,0.96-1.43,1.26-1.95,2.27
			c-0.53,1.04,1.19,1.57-0.14,2.9c-1.2,1.2-0.2,1.67-2.15,2.27c-1.44,0.44-3.13,0.34-4.71,0.73c-1.1,0.49-1.94,1.55-2.92,1.95l0,0
			l0.07,0.95c0.21,1.07,0.43,0.8,1.31,1.27c0.47,0.25,0.01,1.13-0.05,1.52c-0.2,1.31,1.08,1.87,0.5,3.33
			c-0.62,1.55-0.66,1.73-1.63,0.18c-0.51-0.82-0.56,0.98-0.75,1.22c-1.25,1.65-2.44,0.86-4,0.54c-1.56-0.32-2.84-0.46-4.34,0.39
			c-1.24,1.62-3.67,2.83-4.77,4.51c-1.52,2.32,0.76,1.45,2.47,1.31c1.94-0.16,1.12,0.99,1.56,2.49c0.57,1.94,2,3.9,3.05,5.62
			c1.81,2.97-1.7,1.22-1.7,3.88c0.39,1.36-1.63,2.4,0.2,3.54c1.7,1.05-0.12,4.3,0.18,6.17c0.17,1.09,0.74,2.34,0.34,3.45
			c-0.46,1.26-1.61,1.67-1.61,3.02c0.3,0.85,0.02,1.76,0.36,2.58c0.43,1.04,0.76,1.69,0.34,2.79c-0.3,0.79-0.73,2.1,0.02,2.74
			l1.15,1.2l0,0l4.98,0.59c2.33,0,2.68,1.41,4.64,2.02c1.63,0.5,6.46,1.47,7.83,0.34c1.23-1.02,1.25-2.59,2.71-3.51
			c1.12-0.71,2.94-0.16,4.19-0.16c1.59,0,1.47-2.83,2.71-2.67c0.82,1.15,1.43,0.72,2.26,1.5c0.51,0.47,1.46,3.46,2.04,1.72
			c0.42-1.28,2.12-3.1,3.3-3.7c0.12-0.85-1.24-1.53-1.24-2.52c0-0.55,3.87-2.19,2.53,0.97c-0.91,2.15-0.31,1.92,0.18,3.88
			c0.04,0.01,0.23,0.18,0.29,0.23c0.61-0.12,0.85-1.13,1.2-1.56c0.44-0.55,1.79-1.34,1.97-1.86c0.39-0.47,0.83-1.49,1.54-1.36
			c0.79,0.14,1.81,2.54,2.08,3.22c0.8,2.04,2.25,0.16,3.21-0.61L323.19,359.43L323.19,359.43z"></path>
	</g>
	<g id="malaga">
		<path class="st0 fase_0" d="M323.19,359.43c0.53-0.42,0.91-0.79,1.52-1.11c0.88-0.47,1.88-0.6,2.76-1.07c1.25-0.65,2.36-1.59,3.35-2.58
			c0.58-0.46,1.73-1.44,1.52-2.29c-0.15-0.61-1.09-0.73-1.33-1.31c-0.9-2.17,2.05,0.01,2.65-0.2c0.78-0.28,0.04-1.37,1.45-1.79
			c0.59-0.17,0.6,0.79,0.9,1.11c0.64,0.69,3.64-0.09,3.66-0.82c0.04-1.22-0.98-2.44,0.72-3.15l1.65-0.36l0,0c0.96,0.4,1.41,0,1.83,1
			c0.18,0.43,0.25,0.88,0.61,1.2c0.31,0.27,1.08-0.11,1.47-0.11c0.59,0,1.06,0.63,1.72,0.23c0.87-0.54,1.27-1.41,1.97-2.11
			c0.46-0.42,1.06-1.44,1.79-1.31c0.53,0.09,1.22-0.47,1.72-0.34c0.35,0.09,0.08,1.39,0.14,1.75c0.1,0.59,0.41,1.47,0.95,1.81
			l1.18,0.63l0,0l1.06,3.38c0.43,1.74,0.51,5.03,1.97,6.21c1.38,1.12,3.53,3.58,5.34,3.65c1.8,0.06,1.9,1.38,3.33,2.24
			c1.99,1.21,4.18,0.63,6.11,1.79c2.06,1.24,4.24,1.22,4.8,4.01l-0.51,2.67l0,0c-0.28-0.14-0.57-0.25-0.87-0.31
			c-2.07-0.41-3.77,1.04-5.68,1.04c-0.71,0-2.13-0.82-2.99-1c-1.95-0.4-1.63,1.04-3.03,1.04c-1.98,0.5-4.22,0.95-6.29,0.66
			l-4.68-0.66c-0.88-0.12-4.46,5.74-5.41,6.69c-1.12,1.13-3.23,0.52-3.71,2.72c-1.17,1.61-0.07,1.46-2.08,1.56
			c-1.11,0.06-1.31,1.46-3.05,1.16c-3.38-0.59-5-2.02-8.42-0.11c-1.53,0.86-2.85,1.35-4.52,1.72c-0.76,0.17-1.5,0.68-2.28,0.93
			c-0.7,0.39-2.37,0.55-2.76,1.27c-0.71,1.31-1.95,1.68-1.95,3.38c0,1.24-0.22,0.96-1.02,1.77c-0.31,0.31-0.41,0.87-0.52,1.27l0,0
			l-0.95-1.5c-0.62,0.47-0.68,1.25-1.43,0.27c-0.3-0.39-0.26-1.45-0.34-1.95c-0.15-1.03-0.5-1.77-0.97-2.67
			c-0.57-1.1-1.22-2.49-2.04-3.42c-0.52-0.69-2.68-3.8-3.57-1.45c-0.73,1.93-4.73-0.22-4.5-1.13c1.96-0.25,3.48,1.12,4.82-1.2
			c0.56-0.98,0.82-2.36,2.01-2.83c1.64-0.66,4.12-1.2,4.34-3.45c0.1-1.02,0.37-1.58,0.81-2.52c0.56-1.16-0.23-1.86-0.68-2.95
			c-0.44-0.66-0.57-1.81-0.23-2.52c0.85-1.72,1.59-2.96,3.19-1.27c1.05,1.11,1.47,1.88,3.19,1.88c1.01,0,1.59-1.16,1.74-1.99
			c0.12-0.68,0.72-1.19,0.95-1.81C325.45,362.86,324.35,361.16,323.19,359.43L323.19,359.43z"></path>
	</g>
	<g id="jaen">
		<path class="st0 fase_1" d="M356.33,291.04l2.49,3.38c0.27,1.67,2.49,3.06,1.63,4.99c-0.4,0.89-1.21,1.89-1.81,2.68
			c-0.77,1.01-1.21,2.42-1.67,3.58c-0.42,1.03,0.23,1.66,0.38,2.63c0.28,1.73,0.01,2.93-0.27,4.6l-0.54,3.29
			c-0.41,2.49,3.03,0.38,2.94,1.95c-0.04,0.68-0.88,1.28-0.88,2.06c0,1.29,1.4,0.92,1.72,1.56c0.15,0.77,0.8,0.97,0.16,1.77
			c-1.25,1.56-2.13,2.36,0.36,3.38c1.07,0.44,1.15,1.76,1.56,2.72c0.83,1.94,2.78,1.22,2.78,3.54c0,1.59,2.04,1.02,2.04,2.52
			l-0.11,1.77l0,0c0.87,0.61,1.81,1.48,2.87,1.7c1.24,0.25,1.77-0.17,2.9-0.34c2.27-0.35,2.7-2.35,3.35-4.06
			c0.67-1.77,0.74-1.36,2.35-1.56c1.4-0.18,3.06-1.58,4.5-2.11c1.19-0.86,2.19-1.59,2.92-2.9c0.37-0.67,1-1.77,1.74-2.06
			c0.53-0.21,0.84,0.44,1.15,0.73c0.64,0.57,1.15,1.04,1.92,1.5c0.95,0.56,1.85,0.04,2.78-0.23c3.44,0.82,2.85-3.6,5.93-3.88
			c2.37-0.22,3.97,3.8,6.54,1.72c0.86-0.7,0.97,0.18,1.67,0.18c1.09,0,0.99-1.07,1.4-1.72c0.43-0.69,1.45-0.75,1.95-1.38
			c1.08-1.36,0.9-4.98,0.9-6.76c1.13-1.89,0.78-4.02,1.63-6.05c0.79-1.9,1.91-3.52,3.82-4.4c0.78-0.36,1.63-1.1,1.63-2.02
			c0-0.59,0.05-1.9,0.16-2.49l0,0l8.44,3.13l0,0c0-0.26,1.14-1.19,1.36-1.41c0.66-0.66,1.27-1.5,1.83-2.24
			c1.19-1.59,0.93-1.68,0.93-3.45c0-0.66,0.29-1.27,0.29-1.88c0-1.23,0.23-1.04-0.88-1.38c-0.57-0.18-0.69-0.32-1-0.82
			c-0.22-1.35,0.57-4.25-0.7-4.78c-2.42-1.02-2.86,0.18-2.08-2.79c0.14-0.53-0.62-2.09-1.04-2.43c-1.32-1.07-2.48,0.36-3.78,0.36
			c-1.06,0-2.12-1.51-3.16-1.69l0,0c-0.29-0.05-0.58,0.01-0.87,0.23c-1.29,1.01-2.98,4.03-5,3.31c-0.8-0.28-0.37-0.94-1.7-0.57
			c-0.54,0.34-1.05,0.73-1.45,1.22c-0.29,0.35-0.47,1.33-0.63,1.47c-0.28,0.25-1.47-1.59-1.58-1.84c-1.14-2.46-2.91,0.98-5.23,0.5
			c-1.32-0.28-3.16-0.22-4.37-0.73c-0.57-0.24-1.16-1.2-1.63-1.2c-0.27,0-2.25,0.55-2.35,0.68c-0.64,0.82,0.15,1.46-1.13,2.27
			c-1.3,0.83-1.45,1.63-3.08,1.13c-0.83-0.06-2.24-0.15-2.24-1.2c0-0.81-0.38-1.25-1.22-1.25c-1.69,0,0.23,3.52-2.56,2.54
			c-1.15-0.41-2.19-1-3.42-1.16c-0.64-0.08-1.14,0.52-1.72,0.16c-1.84-1.14-2.03,1.72-3.17,2.22c-1.03,0.46-4.53,0.54-5.66,0.41
			c-1.06,0-2.2-0.49-3.23-0.68c-1.18-0.21-2.35,0.16-3.53,0.16c-1.42,0-3.54-1.61-4.73-0.75
			C358.35,289.32,356.94,291.12,356.33,291.04L356.33,291.04z"></path>
	</g>
	<g id="huelva">
		<path class="st0 fase_1" d="M282.27,311.73l0.07,0.95c0.21,1.07,0.43,0.8,1.31,1.27c0.47,0.25,0.01,1.13-0.05,1.52
			c-0.2,1.31,1.08,1.87,0.5,3.33c-0.62,1.55-0.66,1.73-1.63,0.18c-0.51-0.82-0.56,0.98-0.75,1.22c-1.25,1.65-2.44,0.86-4,0.54
			c-1.56-0.32-2.84-0.46-4.34,0.39c-1.24,1.62-3.67,2.83-4.77,4.51c-1.52,2.32,0.76,1.45,2.47,1.31c1.94-0.16,1.12,0.99,1.56,2.49
			c0.57,1.94,2,3.9,3.05,5.62c1.81,2.97-1.7,1.22-1.7,3.88c0.39,1.36-1.63,2.4,0.2,3.54c1.7,1.05-0.12,4.3,0.18,6.17
			c0.17,1.09,0.74,2.34,0.34,3.45c-0.46,1.26-1.61,1.67-1.61,3.02c0.3,0.85,0.02,1.76,0.36,2.58c0.43,1.04,0.76,1.69,0.34,2.79
			c-0.3,0.79-0.73,2.1,0.02,2.74l1.15,1.2l0,0l-0.9,3.33l0.33,2.81l0,0c0.08-0.43-0.5-0.71-0.81-1.17c-0.44-0.66-0.94-1.51-1.11-2.2
			l-0.88-2.11l-0.97-1.72c-0.76-1.35-1.81-2.35-3.1-3.26c-2.8-1.98-5.66-4.18-8.8-5.58c-1.4-0.62-3.94-2.38-5.41-2.43
			c-2.74-0.68-4.7-2.68-7.67-3.04c-3.11-0.37-5.96,0.19-8.94,0.84c-1.09,0.34-2.53,1.27-3.83,1.43l0,0
			c-0.11-0.59-0.28-1.18-0.35-1.77c-0.04-0.28,0.07-0.42-0.07-0.61c-0.1-0.13-0.49-0.97-0.66-1.16c-0.44-0.52,0.14-1.7,0.14-2.36
			c0-1.07-0.72-2.05-0.72-3.26c0-0.62,0.45-1.3,0.25-1.9c-0.25-0.77-1.01-0.53-0.81-1.56c0.13-0.69,0.04-1.37-0.11-2.04
			c-0.19-0.84-0.27-1.54-0.84-2.24c-0.33-0.41-1.29-1.46-1.6-1.84c0.06-0.58,0.34-1.59,0.62-2.04c0.12-0.2,0.31-0.61,0.25-0.86
			c-0.29-1.22,0.64-0.83,0.9-1.63c0.16-0.5,0.52-0.98,0.79-1.45c0.32-0.55,0.53-1.54,0.41-2.18c0.17-1.01,1.36-1.76,1.18-2.72
			c-0.21-1.09,0.85-0.61,1.29-1.16c0.3-0.38,0.68-0.82,1.04-1.13c1.64-1.45,3.09-2.57,2.94-4.92c-0.05-0.74,1.64-2.38,1.45-3.72
			c-0.1-0.73-0.65-1.13,0.5-1.36c0.67-0.14,1.73-0.67,2.22-0.41c0.6,0.32,2.6,0.04,2.15-0.82c0.26-0.73,0.07-1.04,1.06-1.22
			c0.34-0.06,1.31,0.36,1.56,0.57c0.43,0.36,0.44,0.74,1.11,0.57c1.68-0.44,1.97-2.06,1.97-3.81c0-0.76,0.25-0.49,0.61-1
			c0.75-1.04,0.16-2.44,0.43-3.11c0.19-0.48,1.41-1.46,1.22-1.77l0,0l1.65,0.5c0.77,0.83,1.74,0.99,2.83,1.04
			c2.74,0.14-1.07,1.63,0.27,3.11c0.44,0.48,1.15-0.04,1.61,0.36c1.45,1.3,1.05,0.87,3.01,0.79c0.74-0.03,1.89-0.43,2.58-0.25
			c0.46,0.12,1.95,1.5,1.95,2.06c0,2.22,0.18,1.22,1.92,1.61c0.35,0.15,2.46,1.18,2.56,0.97l0.81-1.77c0.32-0.69-0.03-1.46,1-0.77
			c1.44,0.96,2.15-0.4,2.6,1.86c0.1,0.52,1.6,2.04,2.13,2.2c0.74,0.23,1.49-0.03,2.15,0.43c0.61,0.43,0.91,0.36,1.65,0.36
			C281.28,310.94,281.53,312.03,282.27,311.73L282.27,311.73z"></path>
	</g>
	<g id="granada">
		<path class="st0 fase_0" d="M419.22,301.31c-0.11,0.6-0.16,1.91-0.16,2.49c0,0.92-0.85,1.66-1.63,2.02c-1.91,0.88-3.04,2.5-3.82,4.4
			c-0.84,2.04-0.5,4.16-1.63,6.05c0,1.78,0.18,5.39-0.9,6.76c-0.5,0.63-1.51,0.69-1.95,1.38c-0.41,0.66-0.31,1.72-1.4,1.72
			c-0.7,0-0.81-0.88-1.67-0.18c-2.57,2.07-4.17-1.94-6.54-1.72c-3.07,0.28-2.49,4.7-5.93,3.88c-0.93,0.27-1.83,0.79-2.78,0.23
			c-0.77-0.46-1.28-0.92-1.92-1.5c-0.32-0.28-0.62-0.93-1.15-0.73c-0.75,0.29-1.37,1.39-1.74,2.06c-0.73,1.31-1.73,2.04-2.92,2.9
			c-1.44,0.53-3.11,1.93-4.5,2.11c-1.61,0.21-1.68-0.21-2.35,1.56c-0.65,1.71-1.08,3.71-3.35,4.06c-1.12,0.17-1.65,0.59-2.9,0.34
			c-1.06-0.22-2-1.09-2.87-1.7l0,0c-1.31,0.11-2.83,0.09-3.46,1.56c-0.49,1.16-1.06,1.75-2.33,1.75c-2.44,0-1.74,1.47-2.19,3.33
			c-0.54,2.21-0.89,3.16-2.96,4.4l-0.75,0.11l0,0l1.06,3.38c0.43,1.74,0.51,5.03,1.97,6.21c1.38,1.12,3.53,3.58,5.34,3.65
			c1.8,0.06,1.9,1.38,3.33,2.24c1.99,1.21,4.18,0.63,6.11,1.79c2.06,1.24,4.24,1.22,4.8,4.01l-0.51,2.67l0,0
			c0.91,0.45,1.75,1.15,2.73,1l2.38-0.36c1.21-0.19,2.29-1.09,3.33-0.16c1.41,1.27,2.29-0.11,3.53,1.13
			c1.29,2.27,4.21-0.12,5.56-0.57c0.78-0.25,0.9-1.42,1.95-1.18c1.62,0.38,2.13-0.88,3.85-0.29c1.01,0.34,2.03,0.3,3.08,0.2l0,0
			l-0.03-2.6c0.81-0.89,3.81-2.39,3.96-3.17c0.27-1.42-2.51-2.84-0.5-3.85c0.78-0.39,0.43-0.77,0.45-1.47
			c0.04-1.22,1.85-0.57,1.85-2.09c0-1.25-1.03-3.19-1.49-4.4c-1.3-3.43,1.56-0.54,2.85-2.34c0-1.46,0.32-2.12,1.49-3.08
			c0.86-0.7,1.68-4.34,1.97-5.51c0.38-1.55,2.12,0.4,3.01,1.04c1.15,0.83,1.54,0.36,2.74,0.79c0.15,0.05,1.99,1.17,1.9,0.48
			c-0.34-2.63-0.19-7.06,1.38-9.34c1.78-1.3,2.49-3.59,4.8-4.42c1.46-0.53,1.63-2.18,3.42-2.43c1.16-0.16,2.33,0.05,1.63-1.5
			c-0.44-0.97-1.22-3.68,0.16-3.92c0.39-0.07,1.97,0.24,1.54-0.66c-0.44-0.92-0.69-1.97-0.29-2.99c0.18-0.74,0.59-2.47,0.38-3.17
			c-0.49-1.69-0.59-2.78-0.14-4.53c0.43-1.67-0.97-3.35,0.57-4.58l0.54-0.91l0,0l-1.24,0.16c-1.3-0.11-2.52-0.65-3.66-1.2
			c-0.51-0.24-3.51-1.87-3.51-2.2l0,0L419.22,301.31L419.22,301.31z"></path>
	</g>
	<g id="cordoba">
		<path class="st0 fase_1" d="M306.47,301.83c0.48-0.36,1.06-0.89,1.16-1.12c0.23-0.52-0.07-0.77-0.05-1.27c0.03-0.55,0.83-0.97,0.88-1.95
			c0.09-1.68-1.13-2.5-1.13-3.88c0-0.63-0.28-0.65-0.72-1c-0.54-0.42,0.19-1.71,0.14-2.4c-0.03-0.43-0.71-0.58-0.68-1.16
			c0.05-0.89,0.52-2.22,1.09-2.88c0.98-1.15,2.14,0.16,2.85-0.82c0.75-0.91,0.65-1.54,1.79-2.29c0.92-0.61,2.51-4.36,3.28-4.1
			c1.76,0.59,1.72-0.37,3.19-1.45c0.86-0.64,1.7-1.86,2.53-2.31c0.6-0.33,1.58-0.04,1.58-1.11c0-2.26-0.4-1.54,1.38-1.81
			c1.73-0.27,1.31,0.88,3.35-0.48l1.7-0.05l0,0l0.61,1.22c0.89,2.53,4.9-0.42,5.16,1.86c0.24,2.11,0.16,3.23,2.65,3.7
			c0.9,0.17,3.33,0.66,3.96,1.29c0.83,0.83,1.52,1.08,2.47,1.7c1.49,0.98,0.86,1.93,2.83,2.54c0.94,0.29,2.07,1.24,2.87,1.88
			c1.16,1.48,2.48,2.02,4.05,2.86c0.65,0.34,2.49,2.16,2.94,2.22l0,0l2.49,3.38c0.27,1.67,2.49,3.06,1.63,4.99
			c-0.4,0.89-1.21,1.89-1.81,2.68c-0.77,1.01-1.21,2.42-1.67,3.58c-0.42,1.03,0.23,1.66,0.38,2.63c0.28,1.73,0.01,2.93-0.27,4.6
			l-0.54,3.29c-0.41,2.49,3.03,0.38,2.94,1.95c-0.04,0.68-0.88,1.28-0.88,2.06c0,1.29,1.4,0.92,1.72,1.56
			c0.15,0.77,0.8,0.97,0.16,1.77c-1.25,1.56-2.13,2.36,0.36,3.38c1.07,0.44,1.15,1.76,1.56,2.72c0.83,1.94,2.78,1.22,2.78,3.54
			c0,1.59,2.04,1.02,2.04,2.52l-0.11,1.77l0,0c-1.31,0.11-2.83,0.09-3.46,1.56c-0.49,1.16-1.06,1.75-2.33,1.75
			c-2.44,0-1.74,1.47-2.19,3.33c-0.54,2.21-0.89,3.16-2.96,4.4l-0.75,0.11l0,0l-1.18-0.63c-0.54-0.34-0.85-1.22-0.95-1.81
			c-0.06-0.35,0.22-1.66-0.14-1.75c-0.5-0.13-1.19,0.43-1.72,0.34c-0.73-0.13-1.33,0.9-1.79,1.31c-0.69,0.69-1.1,1.57-1.97,2.11
			c-0.66,0.41-1.13-0.23-1.72-0.23c-0.39,0-1.16,0.39-1.47,0.11c-0.36-0.32-0.43-0.77-0.61-1.2c-0.42-1-0.87-0.59-1.83-1l0,0
			c0-1.08-0.38-1.95-0.43-2.95c-0.03-0.61-1.48-2.25-2.04-2.13c-1.16,0.25-1.43,1.24-2.78,0.84c-1.19-0.35-1.82-2.12-2.08-3.22
			c-0.14-0.58-0.2-1.59-0.57-2.04c-0.49-0.61-1.37-0.42-1.81-1.29c-0.47-0.95-0.3-1.4-1.06-2.24c-1.35-1.35-0.19-3.62-0.38-5.26
			c-0.06-0.55-0.74-0.44-1.06-0.68c-0.62-0.47-0.02-1.3-0.02-1.88c0-0.98-0.74-1.46-1.18-2.22c-0.95-1.68-2.04-0.87-3.55-0.7
			c-2.16,0.24,1.37,4.07,1.22,4.44c-0.41,0-2.14-1.2-2.51-1.61c-0.52-0.58-1.78-0.94-2.51-0.75c-0.89,0.07-1.42,0.66-1.97,1.29
			c-0.83,0.96-1.12,1.13-2.35,1.31c-0.69,0.11-1.01,0.65-1.52,1.04c-0.54,0.42-1.32,0.68-1.7-0.09c-0.32-0.66-0.59-1.88-1-2.43
			c-0.33-0.44-0.68-0.87-0.86-1.41c-0.29-0.87,1.98-1.45,2.67-1.47c0.86-0.03,1.22-0.13,1.31-1.04c0-0.82-0.03-1.93-0.36-2.68
			c-0.44-0.99-0.9-0.86-0.9-2.22c0-2.93-2.64-1.69-3.21-4.38c-0.26-1.2,0.14-2.87-0.84-3.7c-1.82-1.54-2.04-1.96-2.04-4.42
			L306.47,301.83L306.47,301.83z"></path>
	</g>
	<g id="cadiz">
		<path class="st0 fase_1" d="M318.36,395.38l-0.95-1.5c-0.62,0.47-0.68,1.25-1.43,0.27c-0.3-0.39-0.26-1.45-0.34-1.95
			c-0.15-1.03-0.5-1.77-0.97-2.67c-0.57-1.1-1.22-2.49-2.04-3.42c-0.52-0.69-2.68-3.8-3.57-1.45c-0.73,1.93-4.73-0.22-4.5-1.13
			c1.96-0.25,3.48,1.12,4.82-1.2c0.56-0.98,0.82-2.36,2.01-2.83c1.64-0.66,4.12-1.2,4.34-3.45c0.1-1.02,0.37-1.58,0.81-2.52
			c0.56-1.16-0.23-1.86-0.68-2.95c-0.44-0.66-0.57-1.81-0.23-2.52c0.85-1.72,1.59-2.96,3.19-1.27c1.05,1.11,1.47,1.88,3.19,1.88
			c1.01,0,1.59-1.16,1.74-1.99c0.12-0.68,0.72-1.19,0.95-1.81c0.85-2.3-0.25-4-1.4-5.74l0,0l-1.79,1.52
			c-0.96,0.77-2.41,2.65-3.21,0.61c-0.27-0.68-1.29-3.08-2.08-3.22c-0.71-0.13-1.14,0.89-1.54,1.36c-0.17,0.52-1.52,1.31-1.97,1.86
			c-0.35,0.44-0.59,1.44-1.2,1.56c-0.06-0.05-0.26-0.22-0.29-0.23c-0.49-1.95-1.09-1.72-0.18-3.88c1.33-3.16-2.53-1.52-2.53-0.97
			c0,0.98,1.36,1.67,1.24,2.52c-1.19,0.59-2.88,2.41-3.3,3.7c-0.58,1.74-1.53-1.25-2.04-1.72c-0.83-0.78-1.44-0.34-2.26-1.5
			c-1.24-0.15-1.12,2.67-2.71,2.67c-1.25,0-3.06-0.55-4.19,0.16c-1.46,0.92-1.48,2.49-2.71,3.51c-1.36,1.13-6.19,0.16-7.83-0.34
			c-1.96-0.6-2.31-2.02-4.64-2.02l-4.98-0.59l0,0l-0.9,3.33l0.33,2.81l0,0c-0.02,0.1-0.07,0.2-0.17,0.32
			c-0.52,0.61-1.14,1.19-1.88,1.54c-1.32,0.62,0.5,2.4,0.95,3.47c0.37,0.89-0.32,2.06,0.97,2.22c1.01,0.13,1.9,0.1,2.83,0.57
			c1.21,0.6,0.55,1.39,2.17,1.79c1.04,0.26,1.49,1.45,0.7,2.24c-0.21,0.51-0.54,0.98-1.13,0.98c-0.38,0-1.87-1.79-1.45-0.59
			c0.71,2.03,2.07,3.69,2.78,5.69c0.32,0.91,1.15,1.72,1.56,2.56l0.79,1.61c0.21,0.43,0.81,1.56,0.84,1.95
			c0.48,0.01,0.81-0.18,1.2-0.18c0.17,0.7,2.18,4.36,2.71,4.9c0.81,0.81,3.02,1.15,4.09,0.75c2.8-1.04,3.77,2.51,5.2,4.04
			c1.19,1.27,2.06,0.84,3.14,1.61c0.59,0.42,0.93,0.57,1.63,0.43c0.79-0.16,0.97,0.48,1.56,0.77c1.83,0.36,2.23,2.59,4.57,1.18
			c0.81-0.49,1.31-0.96,2.31-1.16c1.24-0.25,2.78-1.06,1.85-2.56c-0.43-0.7-0.09-4.39,1.24-4.28c0.76,0.06,1.24,0.49,1.58,1.05l0,0
			l1.43-0.2l0,0c0.01-0.98,0.03-1.99,0.52-2.3c0.41-0.54,0.33-1.35,0.7-1.81C317.62,396.39,318.19,395.96,318.36,395.38
			L318.36,395.38z"></path>
	</g>
	<g id="almeria">
		<path class="st0 fase_1" d="M464.81,340.7l-4.14-3.4c-1.1,0.1-1.27-0.52-2.33,0.02c-1.67,0.86-5.08-5.17-5.99-6.57
			c-0.79-1.22-2.03-3.05-2.44-4.4c-0.27-0.86-0.24-1.42-0.2-2.31c0.02-0.42,0.28-0.83,0.36-1.25c0.12-0.61-0.21-1-0.25-1.56
			c0.25-0.98-0.36-2.72,0.14-3.56c0.37-0.62,0.83-1.73,0.97-2.43c-0.66,0-1.18,0.57-1.95,0.39c-0.57-0.14-1.12-0.83-1.72-0.73
			c-1.39,0.25-1.7-0.47-2.96-0.54c-1.53-0.09-3.9-1.09-4.84-2.4c-0.66-0.92-0.94-1.73-1.47-2.7c-0.4-0.74-1.52-0.9-1.9-1.59l0,0
			l-0.54,0.91c-1.54,1.23-0.14,2.91-0.57,4.58c-0.45,1.76-0.36,2.84,0.14,4.53c0.2,0.7-0.2,2.43-0.38,3.17
			c-0.4,1.02-0.15,2.07,0.29,2.99c0.43,0.9-1.15,0.59-1.54,0.66c-1.38,0.24-0.6,2.95-0.16,3.92c0.71,1.54-0.47,1.34-1.63,1.5
			c-1.79,0.24-1.95,1.9-3.42,2.43c-2.3,0.83-3.01,3.12-4.8,4.42c-1.57,2.28-1.72,6.71-1.38,9.34c0.09,0.69-1.75-0.42-1.9-0.48
			c-1.2-0.44-1.59,0.03-2.74-0.79c-0.89-0.64-2.63-2.59-3.01-1.04c-0.29,1.17-1.11,4.81-1.97,5.51c-1.18,0.96-1.49,1.62-1.49,3.08
			c-1.29,1.79-4.15-1.09-2.85,2.34c0.46,1.21,1.49,3.15,1.49,4.4c0,1.51-1.81,0.87-1.85,2.09c-0.03,0.71,0.33,1.08-0.45,1.47
			c-2.02,1.01,0.77,2.43,0.5,3.85c-0.15,0.78-3.15,2.28-3.96,3.17l0.03,2.6l0,0c1.53-0.15,3.12-0.43,4.74,0.14
			c1.45,0.51,2.43-0.36,3.73-0.36c1.51,0,1.43,1.41,2.47,1.88c0.24,0.39,0.59,0.79,1.06,0.79c0.7,0,0.86-1.17,1.72-0.54
			c0.66,0.48,0.88,1.54,1.9,1.27c0.39-0.47,2.11,0.02,2.87-0.36c2.02-1.01,2.83-1.63,3.48-3.83c0.43-1.47,4.38-4.62,5.88-3.31
			c0.86,0.75,0.99,1.66,2.22,0.66c0.96-0.79,1.96-1.14,3.14-0.45c1.37,0.16,2.69,2.01,3.42,3.08c0.27,0.4,1.31,2.16,1.56,2.27
			c1.69,0.69,3.55-1.95,4.71-2.49c1.42-0.67,0.42-2.25,2.08-3.08c1.65-0.83,0.63-0.36,1.09-2.06c0.54-2.02,2.61-2.3,3.91-3.6
			c1.17-1.17,0.42-2.83,0.9-4.28c1.76-2.4,2.03-5.2,2.76-8.05c0.79-3.08,3.81-5.21,5.43-7.84
			C463.55,341.69,464.16,341.19,464.81,340.7L464.81,340.7z"></path>
	</g>
</g>
<g id="galicia">
	<g id="pontevedra">
		<path class="st0 fase_1" d="M209.79,53.13c0.97,0.54,0.32,2.14,0.47,3.17c0.19,1.31,0.93,1.74,1.99,2.43c0.46,0.29,1.75,0.67,1.92,1.2
			c0.28,0.86-0.88,2.11-0.7,3.31c0.22,1.46-1.13,1.71-1.13,3.06l-0.45,1.47l0,0c-2.14,0.14-4.22,1.78-6.24,1.41
			c-2.15-0.4-4.55-0.52-6.42,0.88c-0.99,0.75-3.66,1.09-3.66,2.43c0,2.04,0.86,3.84,0.68,6.01c0.11,0.88,0.29,2.29,1.06,2.86
			c0.58,0.42,0.75,0.8,0.75,1.5c0,1.31-0.07,3.26,1.74,3.26c0.84,0,1.3-0.71,1.92-0.63c0.54,0.95,0.24,3.28,0.07,4.38l-0.71,1.71
			l0,0c-0.1-0.13-0.28-0.2-0.6-0.19c-0.1,0.18-5.13,2.94-5.45,2.9c-0.64-0.08-0.92-0.27-1.33,0.41c-0.46,0.74-0.7-0.35-1.33,0.11
			c-1.16,0.84-0.86-0.03-2.01-0.07c-1.06-0.04-1.81-0.13-2.56,0.73c-0.46,0.53-0.64,1.3-1.61,1.07c-2.1-0.5-2.8,0.09-3.87,1.77
			c-0.74,1.18-0.74,1.23-1.99,1.95c-1.58,0.91-0.99,1.16-1.81,2.45c-0.34,0.54-1.24,0.78-1.81,1.07c-0.97,0.49-1.01,1.35-1.76,1.9
			c-0.44,0.33-0.9,0.67-1.35,0.98l0,0c-0.07-0.59-0.07-1.17-0.07-1.84c0-2.35-0.57-4.65-0.2-6.98c0.26-1.62-0.47-2.94-0.47-4.42
			c0.1-0.08,2.07-0.32,2.31-0.32c0.13-0.13,0.13-0.11,0.27-0.32c2.2-1.71-1.16-1.78,0.45-2.49c0.33-0.15,1.73-2.23,1.83-2.63
			c0.24-0.95,1.08-0.91,1.9-1.29c1.03-0.48,1.72-1.18,2.33-2.11c0.61-0.92,2.65-0.18,2.01-1.54c-0.17-0.37-0.09-2.12-0.36-1.99
			c-1.08,0.48-1.13,0.9-0.95,2.06c0.15,1-1.72,1-2.4,1c-1.08,0-2.96,1.91-3.78,1.34c-0.59-0.41-1.19,0.2-1.54-0.05
			c-0.6-0.42,0.05-1.21,0.05-1.65c0.46-0.12,1.24-0.27,1.38-0.82c0.22-0.84-1-2.01,0.41-1.43c1.86,0.77,2.24-1.69,3.19-2.99
			c0.68-0.94,2.14-0.99,1.54-2.02c-1.09-1.87-1.16,0.4-2.71,0.75c-0.72,0.48-0.72,1.09-1.54,0.75c-0.71-0.3-1.26,0.23-1.95,0.36
			c-1.1,0.21-1.15-2.12-1.38-2.77c-0.08-0.22,1.04-0.88,1.18-0.88c0.02-0.05,0.09-0.02,0.11-0.09c0.88,0,0.7-0.1,0.52-0.84
			c-0.27-1.1,0.19-0.77,0.5-1.52c0.17-0.42-1.29-3.39-0.7-3.85c0.85-0.67,1.55-0.35,1.97-1.61c0.25-0.76,1.81-2.63,1.81-3.06l0,0
			l1.27-2.67c1.24-0.48,2.02-0.99,3.44-1.04c2.09-0.07,2.85-2.55,5.27-2.13c0.89,0.15,1.86,0.65,2.83,0.36
			c1.05-0.31,1.77-1.3,2.74-1.86c1.7-0.23,1.08-1.16,1.49-2.47c0.3-0.95,1.26,0.36,2.06,0.36c1.96,0,2.99-1.56,4.89-1.56
			c1.6,0,2.39,0.47,3.82,0.95L209.79,53.13L209.79,53.13z"></path>
	</g>
	<g id="ourense">
		<path class="st0 fase_1" d="M258.04,84.89c0.8-0.82,1.63-1.69,1.63-2.91c0-0.91,0.26-1.96-0.48-2.58c-0.99-0.84-2.57-1.02-3.33-1.9
			c-0.56-0.66,0.17-0.85,0.54-1.38c0.33-0.45-0.44-0.88,0.2-1.77c0.1-0.67-0.05-1.1-0.2-1.72c-0.4-0.1-0.73-1.06-1.15-1.06
			c-0.36,0-2.38-0.35-2.77-0.66c-0.6-0.49-2.01-1.68-2.33-1.17c-1.13,1.8-0.65,0.25-1.61,1.11c-0.17,0.15-1.78,0.67-2.24,0.56l0,0
			l-2.31,2.52c-1.18,1.12-2.02,2.91-2.65,4.38c-0.24,0.55-0.95,3.3-1.52,3.22c-0.17-0.69-0.52-1.36-0.68-2.15
			c-0.14-0.7-0.48-1.56-1.18-1.9c-1.3-0.65-3.61-1.52-5.04-1.22c-1.16,0.24-2.04,0.97-3.14,1.2c-1.86,1.05-1.97,1.92-3.89,0.68
			c-1.29-0.83-2.25,0.95-3.21-0.68c-0.47-0.8-1.36-1.2-1.95-1.86c-0.85-0.95-1.86-0.92-2.69-1.59c-0.41-0.33-0.34-1.02-0.72-1.18
			c-0.74-0.31-1.17,0.36-1.9-0.34c-1.52-1.46-3.62-0.02-2.74-2.88c0.24-0.79-0.36-1.26-0.81-1.81l0,0
			c-2.14,0.14-4.22,1.78-6.24,1.41c-2.15-0.4-4.55-0.52-6.42,0.88c-0.99,0.75-3.66,1.09-3.66,2.43c0,2.04,0.86,3.84,0.68,6.01
			c0.11,0.88,0.29,2.29,1.06,2.86c0.58,0.42,0.75,0.8,0.75,1.5c0,1.31-0.07,3.26,1.74,3.26c0.84,0,1.3-0.71,1.92-0.63
			c0.54,0.95,0.24,3.28,0.07,4.38l-0.71,1.71l0,0c0.16,0.22,0.09,0.62,0.06,1.15c-0.04,0.65-0.49,3.05,0.66,2.74
			c0.98-0.26,1.5-1.03,2.65-0.59c1.01,0.39,1.6,2.7,0.79,3.51c-0.53,0.54-1.26-0.05-1.56,0.63c-0.3,0.68-0.49,0.77-1.27,0.77
			c-0.34,1.03-0.53,1.95-1.38,2.9c-0.69,0.75-0.54,1.32-0.54,2.27c0,1.58,0.66,0.77,1.54,1.07c0,0.52,0.17,0.81,0.07,1.41
			c-0.12,0.67,0.05,1.51,0.7,1.81c0.87,0.41,2.58,0.36,3.39-0.2c0.59-0.74,1.45-0.13,1.95-0.77c1.12-1.46,1.24-2.11,3.1-2.2
			c1.93-0.09,0.76-1.94,1.43-2.67c1.37-1.53,0.27,3.36,1.09,3.81c0.34,0.19,0.51-0.33,0.75-0.45c0.79-0.42,0.44-0.9,0.81-1.11
			c0.23-0.13,1.2,0.05,1.54,0.05c1.15,0,0.9-0.94,2.26-0.68c1.79,0.35,1.01-1.38,3.21,0.2c0.89,0.64,2.12,0.36,3.17,0.36
			c0,0.05,0.01,0.05-0.02,0.07c0.02,0.05-1.03,1.39-1.06,1.97c-0.07,1.1,2.35,0.53,3.12,0.34c0.45-0.11,1.15-0.18,1.15-0.75
			c0-0.32-0.2-0.69,0.09-0.97c0.12-0.12,0.47-0.18,0.47,0.07c0.47,0.12,0.77,0.16,1.36,0.16c0.62,0,0.09,0.76,0.59,1.04
			c0.48,0.63,0.53,2.54,1.49,1.5c0.17-0.19,1.48-2.51,1.88-1.7c0.65,1.3,4.02-1,4.93-1.22c1.18-0.29,2.65-1,2.35-2.52
			c-0.16-0.85-0.18-1.74-0.18-2.61c0-1.34,0.65-0.06,1.02-0.57c0.55-0.76,2.27-0.63,2.9,0c0.93,0.93,0.18,1.83,2.42,1.38
			c0.41,0,2.1-1.08,2.19-1.38l0,0l-0.48-1.07c0.49-0.39,7.08-11.05,8.07-11.05c0-0.66-1.09-1.9-0.26-2.8
			C257.67,85.27,257.86,85.08,258.04,84.89L258.04,84.89z"></path>
	</g>
	<g id="lugo">
		<path class="st0 fase_1" d="M247.65,14.8l-0.03,1.35c0.04,0.44-0.29,1.32-0.29,1.99c0,0.66-1.43,1.68-1.81,2.45
			c-1.03,2.07-1.48,0.89-3.01,1.27c-0.05,0.38-0.71,2.01-0.14,2.18c2.69,0.77,0.02,4.51,4.05,4.51c0.36,1.46-0.57,3.04,0.75,4.35
			c0.36,0.43,0.69,0.69,1.24,0.73c0.21,0.01,2.05,0.72,2.1,0.79c0.17,0.25-0.59,1.11-0.59,1.61c0,1.02,0.95,2.03,1.99,1.72
			c0.71-0.21,2.1-1.88,2.31-2.52c0.56,0.14,1.93,2.69,1.74,3.31c-0.81,2.63-4.37,1.64-5.36,3.63c-0.27,0.54-1.04,1.82-0.93,2.4
			c0.38,1.96,0.92,0.54,2.01,0.54c0.28,1.14,3.01,0.9,3.76,2.4l0.32,2.54l0,0l-0.38,0.77c0.07,0.4,1.16,0.9,0.9,1.27
			c-0.34,0.49-0.85-0.05-1.29-0.05c-1.66,0,0.34,2.65,0.29,3.24c-0.05,0.56-1.16,1.65-1.52,2.15c-0.27,0.39-2.07,2.88-2.6,2.09
			c-0.34-0.51-0.36-1.18-0.79-0.05c-0.16,0.42-0.23,0.85-0.41,1.29c-0.87,0.44-3.53,0.25-2.26,2.4c1.2,2.04-0.25,1.54-0.59,3.22
			c-0.2,0.98,0.75,1.44,0.02,2.43c-0.71,0.96-0.26,2.97-0.84,3.35c0.01,0.03,0.02,0.04,0.02,0.07l0,0l-2.31,2.52
			c-1.18,1.12-2.02,2.91-2.65,4.38c-0.24,0.55-0.95,3.3-1.52,3.22c-0.17-0.69-0.52-1.36-0.68-2.15c-0.14-0.7-0.48-1.56-1.18-1.9
			c-1.3-0.65-3.61-1.52-5.04-1.22c-1.16,0.24-2.04,0.97-3.14,1.2c-1.86,1.05-1.97,1.92-3.89,0.68c-1.29-0.83-2.25,0.95-3.21-0.68
			c-0.47-0.8-1.36-1.2-1.95-1.86c-0.85-0.95-1.86-0.92-2.69-1.59c-0.41-0.33-0.34-1.02-0.72-1.18c-0.74-0.31-1.17,0.36-1.9-0.34
			c-1.52-1.46-3.62-0.02-2.74-2.88c0.24-0.79-0.36-1.26-0.81-1.81l0,0l0.45-1.47c0-1.35,1.35-1.6,1.13-3.06
			c-0.18-1.2,0.98-2.45,0.7-3.31c-0.17-0.53-1.47-0.91-1.92-1.2c-1.06-0.68-1.8-1.12-1.99-2.43c-0.15-1.03,0.5-2.64-0.47-3.17l0,0
			c0.35-0.8,0.1-1.89,1.06-2.29c1.1-0.46,0.66-0.9,0.86-1.88c0.12-0.59,0.8-0.65,0.93-1.18c0.18-0.78,0.18-1.85,0.18-2.65
			c0-1.21-0.36-2.02-0.68-3.15c-0.11-0.54-0.15-1.5,0.25-1.93c0.3-0.32,0.71-0.71,0.32-1.13c-0.93-1-0.58-2.02-0.95-3.22
			c-0.23-0.77-0.38-1.73-0.2-2.54c0.18-0.79,1.22-2.03,1.18-2.74c-0.03-0.58-0.52-1.04-0.52-1.61c0-1.07,0.72-1,1.38-1.52
			c1.34-0.96,0.77-2.97,0.77-4.31c0.69-0.34,0.41-0.73,1.58-0.73c0.4,0,1.39-0.08,1.49-0.5c0.46-0.12,1.54-3.56,1.81-4.17
			c0.53-1.19,0.33-3.13,0.59-4.44c0.09-0.02,0.65-0.45,0.68-0.54c0.44-0.09,0.33-2.07,0.36-2.49c0.09-1.05,0.81-1.19,0.77-2.54
			l-0.16-3.33l0,0c0.48-0.33,0.94-0.93,1.56-0.87c0.14,0.02,1.04,1.57,1.18,1.84c0.31,0.28,0.81,1.07,0.9,1.47l1.72-1.93
			c0.48-0.54,1.45-0.24,2.04-0.14c1.35,0.25,0.5,0.58,1.27,1.02c0.63,0.36,1.4,0.44,2.06,0.75c2.01,0.94,3.6,3.88,5.2,5.37
			c0.79,0.74,1.16,1.93,2.31,1.93c1.08,0,2.59,0.6,3.64,0.32L247.65,14.8L247.65,14.8z"></path>
	</g>
	<g id="a_coruna">
		<path class="st0 fase_1" d="M221.49,5.09l0.16,3.33c0.04,1.35-0.68,1.49-0.77,2.54c-0.03,0.43,0.08,2.41-0.36,2.49
			c-0.02,0.1-0.59,0.53-0.68,0.54c-0.26,1.31-0.06,3.25-0.59,4.44c-0.27,0.61-1.35,4.06-1.81,4.17c-0.1,0.41-1.09,0.5-1.49,0.5
			c-1.17,0-0.9,0.38-1.58,0.73c0,1.34,0.57,3.35-0.77,4.31c-0.66,0.52-1.38,0.45-1.38,1.52c0,0.57,0.49,1.03,0.52,1.61
			c0.04,0.71-1,1.96-1.18,2.74c-0.18,0.81-0.03,1.77,0.2,2.54c0.37,1.2,0.02,2.22,0.95,3.22c0.39,0.42-0.02,0.82-0.32,1.13
			c-0.4,0.42-0.36,1.39-0.25,1.93c0.32,1.13,0.68,1.94,0.68,3.15c0,0.8,0,1.88-0.18,2.65c-0.12,0.53-0.81,0.59-0.93,1.18
			c-0.2,0.98,0.24,1.42-0.86,1.88c-0.96,0.4-0.71,1.49-1.06,2.29l0,0l-2.71,0.18c-1.43-0.48-2.23-0.95-3.82-0.95
			c-1.89,0-2.93,1.56-4.89,1.56c-0.8,0-1.76-1.32-2.06-0.36c-0.42,1.31,0.2,2.24-1.49,2.47c-0.97,0.56-1.69,1.55-2.74,1.86
			c-0.96,0.28-1.94-0.21-2.83-0.36c-2.42-0.42-3.18,2.06-5.27,2.13c-1.42,0.05-2.2,0.56-3.44,1.04l-1.27,2.67l0,0
			c-0.78,0-1.64,0.65-2.31,0.88c-0.66,0.23-0.53-1.33-1.4-1.68l-0.23,0.05c-0.23,0.19-0.91,0.81-0.93,1.09
			c-0.02,0.33,0.5,0.82,0.43,0.84c0,0.2-0.58,0.3-0.7,0.45c-0.1,0.13,0.12,0.6,0.07,0.82c-1.07,0.15-0.49-1.01-0.57-1.45
			c-0.84-0.21-2.03,1.15-2.33,1.84c-0.15,0.34,0.06,0.66-0.23,1c-0.46,0.53-0.86,0.58-1.38,0.97c-0.69,0.53-0.5,1.73-1.15,2.33
			c-0.11,0.1-1.06,0.65-1.13,0.36c-0.23-0.06-0.02-2.02-0.02-2.43c-0.08-0.05-0.07-0.04-0.07-0.14c-0.33-0.19-1.61-0.57-1.61-0.68
			c0-0.86,0.65-1.11,0.97-1.72c0.64-1.2,0.69-2.91,0.93-4.24c0.15-0.44,0.74-2.08,1.02-2.36c0.68-0.68,1.85-0.56,2.35-1.56
			c0.35-0.7,0.02-1,0.86-1.41c0.29-0.14,2.02,0.1,1.22-0.48c-0.65-0.47-1.88-0.29-2.35,0.27c-0.06,0.08-1.25,0.29-1.47,0.34
			c-0.95,0.2-1.02-0.68-1.47-0.68c-0.83,0-1.7,1.05-1.97,1.79c-0.44,1.23-0.53,1.17-1.2,0.34c-0.35-0.43-1.52-1.72-1.52-2.24
			c0-0.37,0.65-0.29,0.93-0.41c1.26-0.52-0.08-1.87-0.57-2.36c-0.02,0.05,0.01,0.05,0.05,0.09c-1.21-0.93-0.14-2.02-0.14-3.24
			c0-0.1-2.17-1.3-2.17-2.49h-0.05c-0.09,0.34,0.54,1.93-0.34,1.93c-0.12-0.47-0.94-0.87-1.02-1.2c-1.6,0-1.3,2.92-2.15,2.92
			c0-1.94-1.09-3.07,0.57-4.49c0.98-0.84,0.15-2.11-0.34-2.65c-1.11-1.21,0.15-1.03,0.41-1.61c0.27-0.61,0.88-3.12,1.9-2.92
			c1.02,0.19,0.8,0.4,1.61-0.39c0.13-0.12,0.85-0.99,0.88-1.11c-0.14-0.04-0.22-0.93-0.41-1.09c-0.24-0.2-0.96,1.28-1.49,0.54
			c-0.58-0.8,0.77-1.83,1.54-2.54c1.14-1.06,2.74,0.26,4.37-0.48c0.91-0.62,0.91-1.47,2.06-1.88c0.82-0.3,2.78,0.91,2.08-0.77
			c-0.26-0.64,0.1-1-0.52-1.38c-0.42-0.26-1.22-0.45-1.22-1.04l0.05-0.02c0.87,0,1.55-0.25,2.38-0.25c1.26,0,0.84-0.75,0.84-1.52
			c0.03-0.01,0.02,0,0.02-0.05l0.02-0.02c1.55,0,2.22,0.36,2.81-1.36c0.08,0,0.27,0.6,0.5,0.73c0.66,0.37,0.94,0.91,1.56,1.27
			c3.39,1.96,4.53-0.59,7.71-0.59c1.15-0.36,2.43,0.46,3.51-0.7c0.75-0.8,2.83-3.39,4.09-2.22c0.93,0.86,0.22,2.48,2.47,1.07
			c0.96-0.61-0.26-2.19,0.27-2.43c1.4-0.62,0.78-0.18,1.47,0.32c0.4,0.28,0.98-0.04,1.33,0.45c0.65,0.9,1.12,2.5,2.35,2.9
			c0.01-0.03,0-0.02,0.05-0.02c0.19-0.58-0.03-2.6-0.23-3.24c-0.24-0.79,1.11-1.75,1.74-1.9c0-0.74-4.1-0.28-4.8-0.91
			c-1.63-1.28,1.42-1.11,2.1-1.27c0.88-0.21,1.62-0.24,2.31-0.93c0.12-0.12,0.36-0.79,0.05-0.63c-1.14,0.57-2.28,0.55-3.51,0.95
			c-1.06,0.35-1.91-0.09-2.9-0.09c0-0.47,1.29-0.87,0.95-1.47c-0.76-1.37,0.38-0.9,0.66-1.9c0.19-0.7-0.94-0.94-1.06-1.45
			c0.48-0.09,0.67-0.57,1.04-0.07c0.96,1.3,4.23-1.48,4.5-2.63c0.28-1.16,0.57,0.42,1.29-0.57c1.04-1.42,1.35-1.22,2.99-1.5
			c1.58-0.15,0.28-0.64,0.61-1.97c0.25-1,0.05-2.27,1.61-1.7c1.65,0.6,2.37-0.87,3.64-1.36c0.86-0.33,1.41-2.15,2.53-2.15
			l-0.14,4.08c-0.06,1.88,2.46-0.5,2.9-0.86c1.3-1.09,3.08-2.02,4.34-2.95c2.35-1.73-0.32,1.64-0.32,2.27
			C221.05,5.33,221.27,5.24,221.49,5.09L221.49,5.09z"></path>
	</g>
</g>
<g id="madrid">
	<path class="st0 fase_0" d="M356.96,171.33c0.13,0.12,0.33,0.16,0.56,0.05c0.3-0.14,0.61-0.18,0.94-0.17c0.1,0,0.3,0.06,0.4,0.02
		c0.12-0.05,0.12-0.54,0.12-0.65l0,0l-0.06-0.41c-0.19-0.46-0.74-0.53-1.12-0.26c-0.25,0.18-0.52,0.34-0.77,0.54
		C356.72,170.72,356.74,171.13,356.96,171.33L356.96,171.33L356.96,171.33z M365.02,166.63c0.32-0.38,1.58-2.65,1.76-2.65
		c1.35,0,3.22,0.89,3.57-0.86c0.07-0.35,0.58-0.43,0.7-0.82c0.21-0.7-3.54-4.33-3.03-5.03c0.52-0.7,4.95-1.61,5.95-2.61
		c0.95-0.95,1.86-0.91,3.05-1.18c1.95-0.43,2.66-3.02,3.85-4.22c0.62-0.63,1.43-0.64,2.06-1.45l1.9-2.45
		c0.36-0.18,0.55-0.39,0.81-0.66c0.24-0.24,0.74-0.54,1.06-0.61c0.18-0.04,0.36-0.09,0.54-0.09c0.27,0,0.53,0.09,0.79,0.09l0,0
		c0,0.79,0.01,1.19,0.81,1.45c0.8,0.26,1.02,1.12,1.2,1.88c0.23,0.95,1.84,0.71,2.56,0.88c0,0.65-0.79,1.57-0.52,2.18
		c0.33,0.74,1.43,0.91,1.43,1.86c-0.35,0.89-0.85,1.7-1.13,2.63c-0.21,0.69-0.26,1.37-0.54,2.02c-0.22,0.5-0.81,0.4-0.27,1
		c1.15,1.28-0.37,1.09-0.63,1.95c-0.43,1.42-0.74,2.53-1.31,3.85c-0.33,0.75,1.24,0.59,1.74,0.79c0.63,0.26,0.34,1.92,0.34,2.43
		c-0.15,0.87-0.6,2.07,0.09,2.81c0.69,0.73,1.71-0.18,2.58-0.18c0.73,0,0.47,1.15,0.81,1.65c0.37,0.53,1.37-0.21,1.49,0.32
		c0.23,0.95-0.87,2.45,0.25,3.26c0.7,0.5,1.13,0.19,1.13,1.36c0,1.18,1.57,0.75,2.38,1.5c1.13,1.05,1.33,2.1,1.33,3.63
		c0,1.23,0.97,0.73,1.72,1.29c0.31,0.23,0.99,1.37,0.9,1.72c-0.14,0.6-1.06,1.09-0.77,1.79c0.5,1.2-0.83,2.43-1.06,3.56
		c-0.09,0.44-0.09,0.92-0.18,1.36c-0.06,0.31-0.3,0.62-0.43,0.91c-0.07,0.15,0.47,1.62,0.79,1.09c0.39-0.63,2.08-2.84,2.76-1.81
		c0.67,1.01,0.97,2.54,0.97,3.72l0.78,1.92l0,0c-1.39,2.29-0.21,4.86-3.6,5.5l0,0c-0.52,0.1-1.15,0.15-1.9,0.15
		c-1.43,0-2.69,0.21-3.96,0.42c-0.42-0.75-0.86-2.15-1.99-1.97c-0.37,0.06-0.78,0.31-0.79,0.71c-0.02,0.45-0.19,0.88-0.19,1.35
		l-0.27,0.32c-0.26,0.01-0.53,0.02-0.8,0.02c-1.48,0-2.74-0.01-3.92,0.07c-1.34,1.49-2.94,0.6-4.25,2.02
		c-0.45,0.49-0.8,1.3-1.29,1.68c-0.92,0.7-2.85,0.75-4,1.5c-1.16,0.75-3.66,4.64-4.71,2.31c-0.15-0.34-0.89-2.09-0.23-2.09
		c0.65,0,1.4-0.81,1.97-1.07c0.41-0.18,1.09,0.43,1.63,0.39c0.62-0.05,0.26-0.61,0.45-0.88c1.08-1.49,2.29-2.35,3.8-3.4
		c1.07-0.74,1.04-1.13,1.7-2.02c0.93-1.25,0.14-1.55-0.43-2.58c-1.24-0.52-2.78-1.67-4.09-0.95c-1.8,0.99-2.48-1.95-3.26-1.66
		c-1.43,0.54-3.21-1.94-4.82-1.54c-0.64,0.16-1.39-0.82-2.31-0.97c-1-0.17-1.83-1.88-2.01-1.88c-0.07-0.23-1.23,0.08-1.45,0.09
		c-2.29,0.1-4.18-1.29-6.29-1.29c-0.44,0.16-2.97,4.44-3.71,3.33c-0.62-0.93-1.15-2.95-1.15-4.08c0-0.46-1-2.2-1-0.86
		c0,0.82-2.4,2.72-3.05,3.29c-0.56,0.48-1.28,0.69-1.88,1.2c-0.55,0.48-0.81,1.39-1.79,1.02l-1.15-0.61l0,0
		c1.11-0.91-3.17-3.64-1.55-4.77c0.58-0.4,2.51-1.63,2.77-2.14c0.09-0.18,0.64-0.1,0.72,0.07c0.28,0.54,0.16,1.44,1,1.41
		c0.24-0.01,1.71-0.08,1.72-0.23c0.03-0.32-0.25-0.72,0.05-0.95c0.65-0.52,0.51-2.82,1.85-3.65c0.61-0.11,2.93-1.26,2.96-1.72
		c0.02-0.29,0.59-2.54,0.63-3.42c0.03-0.48-0.23-0.99-0.23-1.47c0-0.34,0.35-0.54,0.61-0.68c1.03-0.08,0.66-3.95,1.22-4.6
		c0.17-0.2,4.36-0.87,4.45-1.53c0.1-0.16,0.18-0.32,0.27-0.48C363,170.24,364.07,167.74,365.02,166.63z"></path>
</g>
<g id="aragon">
	<g id="huesca">
		<path class="st0 fase_1" d="M556.77,61.55c-0.18,0.08-0.4,0.14-0.68,0.19c-0.61,0-1.14-0.17-1.67-0.36c-0.48-0.17-0.94-0.11-1.38-0.23
			c-1.15-0.32-2.05,0.43-3.1,0.18c-0.6-0.14-1.46,0.01-2.04,0.23c-0.48,0.18-2.06-0.29-2.06-0.86c-0.04-0.01-0.14-0.14-0.16-0.2
			c-0.86-0.22-0.3-0.75-1.56-0.34c-0.33,0.6-0.94,0.68-0.79,1.43c0.16,0.81-0.48,1.19-1.15,0.95c-1.36-0.48-0.79-2.77-2.53-2.54
			c-0.82,0.11-1.26-1.14-2.15-0.59c-0.38,0.23-2.3,1.03-2.6,0.91c-0.44-0.18-1.18-0.52-1.54-0.02c-0.48,0.67-0.54,1.21-1.65,0.93
			c-0.92-0.23-0.61-0.49-1.31,0.39c-0.35,0.43-2.12-0.5-2.56-0.7c-0.39-0.28-1.66-1.01-1.74-1.5c-0.05-0.01-1.97-3.34-2.38-3.63
			c-0.15-0.11-1.2,0.18-1.54,0.16c-1.62-0.11-2.84-3.26-5.18-2.31c-0.51,0.2-2.99,2.7-3.26,1.61c-0.51-0.08-2.11,1.11-2.4-0.07
			c-0.88,0-1.49-1.59-1.95-0.29l-0.25,0.7c-0.29,0.81-2.31,2.31-1.61-0.11c-0.44-1.18-1.06,0.7-1.43-0.32
			c-0.14-0.38-0.18-1.58-0.68-1.65c-0.03-0.12-0.36-0.2-0.38-0.34c-0.75-0.11-1.35-1.73-1.76-1.84c0-0.38-1.37-0.25-1.67-0.29
			c-0.09-0.35-0.61-0.64-0.61-1.22c0-0.19,0.02-0.47,0.02-0.73l0,0c-1.07,0-2.15,0.13-2.96,0.77c-0.76,0.6-0.77,1.63-1.15,2.45
			c-0.23,0.49-0.67,0.92-0.97,1.34c-0.37,0.52,0.05,1.58-0.32,2.09c0.09,0.87,1.39,0.81,0.16,1.72l0.2-0.16l0.39,3.38
			c0.1,0.58-0.69,0.74-0.2,1.34c1.19,1.45,0.47,0.94-0.59,1.99c-0.52,0.52-0.15,1.33,0,1.9c0.32,1.27-0.94,2.06-0.61,3.13
			c0.41,1.34,1.33,2.27,1.33,3.79l0,0c0,0.94-0.38,1.75-0.38,2.74c0,1.12,0.82,1.45,1.67,1.86c1.73,0.83-1.7,5.41-1.7,6.94
			c1.13,0.28,2.61,0.93,3.37-0.39c0.2-0.34,2.11-6.02,2.31-4.96c0.5,2.62-0.09,4.77-0.09,7.37c-0.39,2.34-0.38,5.01-0.47,7.39
			c-0.09,2.2-0.27,2.75-1.9,0.84c-0.29-0.34-1.38,2.09-1.61,2.43c-0.61,0.9-1.55,1.55-0.27,2.34c1.62,1,1.97,1.85,4.05,1.97
			c1.79,0.11,3.01-0.1,4.43,1.07l1.06,1.84c0.87,1.5,2.91,1.24,3.71,2.9c0.87,1.8,0.04,5.81,1.72,6.8c0.98,0.58,2.58,0.61,3.37,1.41
			c0.9,0.9,0.88,1.82,1.43,2.9c0.56,0.56,1.21,0.9,1.81,1.38c0.99,0.8,1.85,2.42,3.17,1.68c1.12-0.63,1.18-0.92,2.56-0.57
			c0.47,0.12,0.78,3.3,1.24,4.01c1.08,1.68,1.53,2.95,2.15,4.83c0.53,1.06,0.32,3.22,1.74,3.63c0.42,0.12,0.5-1.24,0.86-1.34
			c0.35-0.1,1.47,1.88,2.65,1.9c1.88,0.05,3.89-2.08,5.36-3.08c1.58-1.08,3.31-0.3,5.07,0.09l0,0c0.81-0.51,1.07-1.77,0.45-2.54
			c-1.28-1.59,1.21-1.36,1.99-1.61c1.86-0.6,2.51-4.35,2.07-5.68c-0.91,0,1.66-1.56,1.31-2.98c-0.29-1.18-4.69,0.74-5.73-0.23
			c-1.83-1.71,2.28-3.61,2.71-5.4c0.07,0,0.06-0.02-0.02,0.05c0.76-0.6,2.92,0.28,2.92-1.07c0-0.91,0.46-2.47,1.61-2.47
			c0.89,0,1.48-0.16,1.54-1.16c0.05-0.86,1.99-0.68,1.99-1.75c0-0.92-0.5-2.76-0.97-3.58c-0.41-0.71,0.41-1.25,1-1.34
			c0.2-0.03,1.6-1.41,1.81-1.63c0.94-1.02,0.13-1.96,0.59-2.88c0.2-0.4,0.64-0.95,0.66-1.41c0.94-1.88,1-3.7,1-5.8
			c0-1.99,1.22-4.8,1.9-6.69c0.54-1.49-0.66-0.96,0.47-2.54c0.41-0.57-1.08-0.36-1.31-0.68c-0.26-0.36,0.33-1.2,0.18-1.86
			c-0.21-0.85,0.21-1.58-0.63-2.04c-0.84-0.46-0.16-1.13-0.68-1.77c-0.37-0.46-0.41-0.95,0.2-1.18c0.4-0.15,1.58-0.28,1.43-0.91
			c-0.38-1.55-1.03-0.65-0.16-2.4c0.58-1.17,1.11-2.15,0.68-3.42c-0.82,0-1.74-0.13-2.04-0.95c-0.41-1.15-0.37-1.33-1.2-2.22
			L556.77,61.55L556.77,61.55z"></path>
	</g>
	<g id="zaragoza">
		<path id="zaragoza" class="st0 fase_1" d="M487.28,73.5c-0.45,0.1-0.85-1.39-1.41-0.56c-0.15,0.22-0.15,0.44-0.26,0.68
			c-0.17,0.38-0.73,0.93-0.4,1.35c0.49,0.62,1.51,0.45,2.22,0.45c1.32,0-0.1-0.86-0.03-1.51L487.28,73.5L487.28,73.5z M483.9,76.84
			c0.09-0.24,0.11-0.52,0.2-0.75c0.08-0.21,0.13-0.41,0.09-0.65c-0.09-0.56-0.52-0.03-0.72,0.16c-0.39,0.36-0.51,0.82-0.7,1.28
			c-0.05,0.13-0.16,0.47-0.03,0.58c0.24,0.2,0.66-0.25,0.88-0.39L483.9,76.84L483.9,76.84z M543.6,130.77L543.6,130.77
			c-0.65,1.31,0.58,1.61,1.27,2.47c0.47,0.59,0.18,0.95-0.23,1.36c-0.51,0.51-0.03,1.03,0.38,1.31c0.18,0.13,0.29,0.46,0.34,0.88
			l0,0c0.12,0.95-0.03,2.35-0.14,2.8c-0.3,1.29-2.11,0.85-2.47,2.02c-0.25,0.83,0.19,1.9-0.5,2.58c-0.51,0.51-1.32,0.51-1.67,0.86
			c-0.37,0.37-0.03,0.66-0.66,0.97c-0.54,0.27-1.65-0.39-1.65,0.63c0,0.36,0.13,1.12,0.3,1.72l0,0c-1.04-0.22-4.42-0.26-5.39,0.32
			c-0.9,0.54-1.19,1.25-1.7-0.34c-0.74-2.32-1.89-2.97-3.94-4.1c-1.07-0.59-2.81-1.63-3.96-1.93c-2.18-0.56-3.21-1.75-4.86-3.02
			c-0.75-0.57-1.9-0.63-2.81-0.82c-1.28-0.7-2.94-2.58-4-3.7c-0.92-0.96-2.66-1.2-4.12-1.2l-0.05,0.05c0,1.29,3.16,5.11,2.01,6.03
			c-1.02-0.13-1.98-3.69-3.03-3.88c-0.7-0.13-1.51,4.01-1.18,4.67c1.05,2.11,0.53,2.15-0.95,3.83c-2.14,2.42-2.25,0.98-4.28-0.63
			c-0.91,0-1.41,2.46-2.31,3.04c-1.12,0.73-2.01-1.37-2.96-1.13c-0.42,0.92-1.55,2.05-2.56,2.22c-1.42-0.47-1.43-1.11-1.67-2.38
			c-0.32-1.69-0.7-0.51-2.04-1.02c-0.57-0.22-1.1-0.86-1.76-0.86c-1.17,0-1.39,1.28-2.35,1.59c-1.37,0.44-2.26-1.44-3.21,0.11
			c-0.95,1.55-1.35,4.38-3.48,3.79c-0.66-0.18-1.83-1.72-1.83-1.72c-0.34,1.34-4.12,3.24-3.42,4.6c1,1.94-3.11,2.61-4.5,2.99
			l-3.01,1.13l0,0c-0.11-0.25-0.36-0.51-0.86-0.84c-0.59-0.39-2.98-2.56-3.14-3.17c-0.91-0.58-2.16-2.2-2.62-3.13
			c-0.89-1.78-2.27-0.8-3.42-1.72c-0.51-0.41-0.46-1.23-0.81-1.59c-0.78-0.78-1.69,1.22-2.04-0.84c-0.05-0.32-0.54-0.8-0.54-1
			c-1.42,0-2.43,1.06-4.03,0.82l0,0l-0.72-0.66c-0.43-0.34-1.17-0.61-1.7-0.73c-0.13-0.03-1.32-0.24-1.33-0.27
			c-0.15,0,0-1.12,0-1.29c0-0.55,0.02-1.18-0.11-1.7c-0.31-1.23-0.82-2.47-1.06-3.7c0-0.74,0.78-1.14,0.36-1.9
			c-0.31-0.57,0.88-0.57,1.31-0.97c0.95-0.89,0.46-4.09,1.22-4.31c0.49-0.14,3.65-7.39,4.52-8.36c0.8-0.89,7.53-9.6,5.9-10.06
			c0-1.55,1.86-3.62,0.43-5.21c-0.69-0.77-0.88-0.92-0.88-1.99l-0.11-1.9l0.88-2.04c0.31,0.01,0.58,0,0.9-0.03
			c0.23-0.02,0.42,0.02,0.58,0.19c0.39,0.41,0.7,0.09,1.17,0.02c0.5-0.08,0.67,0.24,0.88,0.62c0.59,1.86,2.17,0.95,3.53,0.95
			c1.87,0,2.84,3.5,5.18,2.61c1.66-0.63,2.77-1.07,4.59-1.07c1.61,0,1.71-1.68,2.17-2.86c0.18-0.46,3.12-4.52,1.83-4.38
			c-1.64,0.19-0.89-0.91-1.65-1.59c-0.56-0.5-1.81-1.55-1.61-2.36c0.12-0.8,0.57-1.83,0.23-2.58c-0.22-0.48-0.72-0.95-0.84-1.41
			c-0.08-0.3,0.44-1.07,0.54-1.41c0.4-1.32,0.16-2.1,0.88-3.42c0.2-0.36,2.8-3.48,1.74-3.54c-0.92-0.05-1.04-0.54-0.59-1.25
			c0.49-0.77,0.48-1.72,1.11-2.43c0.49-0.86,1.74-0.95,1.79-2.09c0.03-0.67-0.48-1.54-0.32-2.09c0.16-0.55,0.26-1.91,0.61-2.29
			c0.36-0.39,1.83,1.54,2.67-0.05c0.21-0.4-0.13-0.73,0.25-1.18c0.61-0.72,1.41-0.54,0.77-1.65c-0.33-0.57,0.29-0.56,0.63-0.41
			c0.46,0.21,0.27-1.39,0.63-1.65c0.75-0.54,2.14,0.12,3.01-0.05c1.79,0.68,1.25-1.19,1.54-2.13c0.47-1.52,0.71-0.66,1.88-0.86
			c1.36-0.23,2.04-1.84,3.14-2.49c0.64-0.38,1.43-0.71,2.04-1.16l0.2-0.16l0.39,3.38c0.1,0.58-0.69,0.74-0.2,1.34
			c1.19,1.45,0.47,0.94-0.59,1.99c-0.52,0.52-0.15,1.33,0,1.9c0.32,1.27-0.94,2.06-0.61,3.13c0.41,1.34,1.33,2.27,1.33,3.79l0-0.03
			c0,0.94-0.38,1.78-0.38,2.78c0,1.12,0.82,1.45,1.67,1.86c1.73,0.83-1.7,5.41-1.7,6.94c1.13,0.28,2.61,0.93,3.37-0.39
			c0.2-0.34,2.11-6.02,2.31-4.96c0.5,2.62-0.09,4.77-0.09,7.37c-0.39,2.34-0.38,5.01-0.47,7.39c-0.09,2.2-0.27,2.75-1.9,0.84
			c-0.29-0.34-1.38,2.09-1.61,2.43c-0.61,0.9-1.55,1.55-0.27,2.34c1.62,1,1.97,1.85,4.05,1.97c1.79,0.11,3.01-0.1,4.43,1.07
			l1.06,1.84c0.87,1.5,2.91,1.24,3.71,2.9c0.87,1.8,0.04,5.81,1.72,6.8c0.98,0.58,2.58,0.61,3.37,1.41c0.9,0.9,0.88,1.82,1.43,2.9
			c0.56,0.56,1.21,0.9,1.81,1.38c0.99,0.8,1.85,2.42,3.17,1.68c1.12-0.63,1.18-0.92,2.56-0.57c0.47,0.12,0.78,3.3,1.24,4.01
			c1.08,1.68,1.53,2.95,2.15,4.83c0.53,1.06,0.32,3.22,1.74,3.63c0.42,0.12,0.5-1.24,0.86-1.34c0.35-0.1,1.47,1.88,2.65,1.9
			c1.88,0.05,3.89-2.08,5.36-3.08C540.11,129.6,541.84,130.38,543.6,130.77z"></path>
	</g>
	<g id="teruel">
		<path class="st0 fase_1" d="M484.37,207.12l-0.77-1.97l0,0c1.05-0.35,4.04-1.42,3.33-2.88c-0.62-1.26-3.7-1.23-5.16-1.03
			c-2.16,0.3-1.78-1.25-2.33-2.83c-0.17-0.5-0.91,1.54-1.25,1.08c-0.44-0.61-0.25-2.45-0.25-3.24l0.17,0.5
			c-0.33,1.58,0.34,3.95-3.66,3.08l-2.27-1.77l0,0c0.03-0.24,0.51-2.72,0.2-2.83c-0.57-0.21-1.45,0.75-2.17,0.23
			c-0.46-0.33-0.61-0.51-1.2-0.32c-0.94,0.3-1.25,0.22-1.88-0.59c-0.72-0.93-0.99-1.6-2.17-2.02c-0.82-0.29-1.94-3.36-2.78-3.13
			c-0.3,0.08-1.07,1.78-1.15,1.45c-0.51-0.07-0.18-2.14,0.54-2.24c0-0.3-0.1-0.78-0.48-0.88l-2.28-2.9
			c-0.9-0.07-1.21-0.36-1.13-0.68l0,0c0.07-0.29,0.44-0.6,0.97-0.82c0.61-0.25,0.82-1.37,1.29-1.79c0.4-0.35,1.17,0.52,1.47-0.32
			c0.35-0.97,0.72-6.71,1.67-6.71c0.07,0.22,1.97,0.93,2.26,1c1.18,0.28,1.1,0.03,1.88-0.63c0.95-0.81,0.03-3.09,0.72-3.26v-0.07
			c1.1-0.55-0.42-4.27-0.29-5.28c0.15-1.19,1.02-3.49-0.09-4.31c-1.17-0.86-3.06-2.58-2.62-4.28c0.2-0.77,0.49-1.18,0.3-1.63l0,0
			l3.01-1.13c1.39-0.38,5.5-1.05,4.5-2.99c-0.7-1.36,3.08-3.26,3.42-4.6c0.01,0,1.18,1.54,1.83,1.72c2.13,0.59,2.53-2.24,3.48-3.79
			c0.95-1.55,1.85,0.33,3.21-0.11c0.97-0.31,1.18-1.59,2.35-1.59c0.67,0,1.19,0.64,1.76,0.86c1.33,0.51,1.72-0.67,2.04,1.02
			c0.24,1.27,0.26,1.91,1.67,2.38c1.01-0.17,2.14-1.3,2.56-2.22c0.95-0.24,1.84,1.86,2.96,1.13c0.9-0.58,1.39-3.04,2.31-3.04
			c2.03,1.62,2.14,3.05,4.28,0.63c1.48-1.68,2-1.72,0.95-3.83c-0.33-0.66,0.48-4.79,1.18-4.67c1.06,0.19,2.01,3.75,3.03,3.88
			c1.15-0.92-2.01-4.74-2.01-6.03l0.05-0.05c1.46,0,3.2,0.24,4.12,1.2c1.06,1.11,2.73,2.99,4,3.7c0.9,0.19,2.06,0.25,2.81,0.82
			c1.66,1.27,2.68,2.45,4.86,3.02c1.14,0.3,2.89,1.34,3.96,1.93c2.04,1.13,3.19,1.78,3.94,4.1c0.51,1.59,0.8,0.88,1.7,0.34
			c0.97-0.59,4.35-0.54,5.39-0.32l0,0c0.09,0.31,0.2,0.58,0.31,0.73c0.81,1.24,1.74,1.77,2.33,3.17c1.22,2.92-1.53,4.08-1.13,6.57
			c0.17,1.05,0.73,2.17,0.97,3.17c0.19,0.79-1.01,1.71-1.27,2.47c-0.19,0.48-0.34,0.73-0.81,0.95c-0.47,0.22-0.91,0.26-1.4,0.39
			c-0.25,0.07-0.46,0-0.66,0.18c-0.22,0.2,0.02,0.9,0.09,1.16l0,0l-0.67,0.44c-0.42,0.16-0.85,0.39-1.26,0.08
			c-0.24-0.18-0.3-0.3-0.62-0.27c-0.85,0.07-1.42,0.84-2,1.36c-0.17,0.28-2,1.47-2.21,1.6c-1.05-1.57,0.21-1.33-2.16-1.33
			c-1.46,0-2.68,0.23-3.33-1.16c-0.11-0.23-2.06-0.91-2.33-0.91c-1.29,0-0.69-2.18-1.1-0.93c-0.66,2-0.4,3.05-2.49,4.1
			c-1.48,0.74-1.37,1.41-2.73,0.99c-1.54-0.47,0.19,0.18-0.17,1c-0.5,1.13,1.66,0.73,1.75,0.17c0.07-0.5,0.68,0.77,0.72,1.13
			c0.36,0,1.73,1.62,1.69,1.86c0,0.49-0.33,2.46,0,2.58c0.7,0.25,2.16,3.52,1.41,4.07c-0.56,0.41-2.88-0.2-3.33,0.25
			c-0.85,0.85,1.5,0.83,1.83,1.58c0.24,0.55-0.84,1.93-0.97,2.31c-0.07,0.19-1.45,0.84-1.74,1.07c-0.88,0.88-1.14,1.85-2.35,2.29
			c-1.1,0.4-0.02,1.4-0.02,1.95c0,1.36-2.63,4.28-3.64,3.95c-0.42-0.14-1.94-0.83-2.33-0.83c-0.56,1.12,1.39,3.73,1.08,4.99
			c-0.41,1.67-3.74-0.99-4.61,0.33c-0.56,0.84,0.14,1.39,0.14,2.13c-0.37,0.12-0.77,0.1-1.22,0.14c-1.56,1.95-1.88,2.52-3.04,4.48
			c-0.82,1.39-2.05,0.59-2.33,1.75c-0.2,0.82,1.08,2.27,1.58,2.91c0.19,0.25-0.8,0.8-0.83,0.91c-0.06,0.02-0.4,1.31-0.5,1.33
			c-0.06,0.22-1.02,0.8-1.25,1c-0.35,0.3-2.52-0.61-2.96-0.61c-0.12-0.37-2.35-0.09-2.61-0.47c-0.23-0.33,1.68-2.47,1.21-2.7
			c0-0.91-1.43-0.76-2.13-1.46c-0.55-0.55-1.54-0.01-1.75-0.83c-0.9-0.13-0.84,1.12-1.83,0.91c-0.19-0.59-2.2-0.2-2.49-0.08
			L484.37,207.12L484.37,207.12z"></path>
	</g>
</g>
<g id="catalunya">
	<g id="tierres_de_lebre">
		<path class="st3 fase_0" d="M558.03,174.66l-0.16,0l-0.09,0l-0.16,0l-0.07,0l-0.09,0l-0.16-0.08l-0.08,0.09l-0.49-0.23l-0.32,0.09
			l-0.16,0l-0.09,0l-0.15,0.24l-0.23,0.09l-0.64,0.18l-0.16,0l-0.32-0.07l-0.25-0.24l0.01,0.32l0,0.16l0,0.09l0,0.07l-0.07,0
			l-0.16,0l-0.09-0.07l-0.08-0.08l-0.09,0l0,0.09l-0.08,0.08l-0.24-0.07l-0.09-0.08l0.1,0.48l0.17,0.24l0.17,0.31l0.33,0.24
			l0.32,0.07l0.41,0.07l0.32-0.09l0.4-0.09l0.72-0.18l0.31-0.17l0.63-0.34l0.32-0.25l0.22-0.33l0.35-0.51
			c-0.04-0.05-0.08-0.11-0.11-0.17L558.03,174.66z M565.12,167.22l-0.17-0.47l-0.07,0.08l0,0.16l-0.24,0.01l-0.17-0.23l-0.16,0
			l-0.08,0.08l-0.16,0.09l-0.23,0.08l-0.16,0l-0.24,0.01l-0.16-0.07l-0.25-0.31l-0.33-0.32l-1.09-1.41l-0.43-0.71l-0.16-0.16
			l-0.25-0.15l-0.16-0.15l-0.32-0.08l-0.48,0.01l-0.24,0.01l-0.23,0.01l-0.16,0.09l-0.17,0.16l0.25,0.15l0.24-0.01l0.16,0l0.16,0.07
			l0.25,0.08l0.24,0.07l0.32,0.15l0.25,0.08l0.41,0.23l-0.15,0.16l-0.09,0l-0.15,0.16l-0.16,0.08l-0.08,0.09l-0.23,0.08l-0.57-0.06
			l-0.57-0.23l-0.25-0.15l-0.41-0.47l-0.09-0.08l-0.24-0.23l-0.25-0.31l-0.26-0.56l0.08-0.16l0.15-0.16l0.08-0.09l0.07-0.09
			l0.16-0.08l0.24-0.09l0.07,0l0.08-0.16l0.16-0.08l0.24-0.09l0.07-0.16l0.08-0.08l0.16,0l0,0.07l0.07-0.08l0.08-0.17l-0.09-0.07
			l0-0.09l0.15-0.4l0-0.09l0.09,0l0.07-0.08l-0.01-0.17l0.17-0.16l0.07,0l0.09,0l0.16,0l0.07,0l0.08-0.16l0-0.07l0.16-0.16
			l0.07-0.16l0.08-0.09l0.4-0.09l0.23-0.18l0-0.07l0.09,0l0.07-0.09l0.08-0.08l0.15-0.25l-0.08-0.07l0.07,0l0-0.09l0.08-0.08
			l0.07-0.09l0.08-0.16l0.07-0.09l0,0.09l0,0.07l0.08-0.08l0-0.09l0-0.07l0.07-0.09l0-0.07l0.08-0.09l0.07,0l0.08-0.08l0.16-0.09
			l0.08-0.25l0.16-0.08l0.07-0.16l0-0.09l0.09,0l0-0.07l0-0.09l0-0.07l0.07-0.09l0.08-0.08l0.07,0l-0.01-0.17l0.09,0l0-0.07l0.09,0
			l0.07-0.25l0.16-0.08l0.24-0.25l0.07-0.08l0.08-0.09l0-0.07l0-0.09l0.15-0.16l-0.41-0.07l-0.4-0.15l-0.73-0.05l0.07-0.09
			l-0.58-0.46l-0.35-0.87l-0.17-0.24l-0.43-0.95l-0.16-0.15l-0.08-0.16l0.29-0.89l0.07-0.25l-0.01-0.4l-0.08-0.23l0.15-0.25
			l0.31-0.33l0.31-0.41l0.08-0.16l0.46-0.74l0.25,0.24l0.57,0.3l0.07-0.09l0.24-0.01l0-0.07l0.32-0.09l0.15-0.16l-0.01-0.32
			L564,148.6l0.08-0.16l-0.09-0.24l0.09,0l0.15-0.25l0.16-0.08l0.25,0.07l0,0.09l0,0.09l-0.08,0.16l0.09,0.16l0.07,0l-0.01-0.24
			l0.08-0.16l0.07-0.09l-0.57-0.46l-0.57-0.14l-0.17-0.23l-0.31,0.17l-0.16,0l-0.09-0.08l-0.08,0.09l-0.22,0.49l0,0.07l0.07,0
			l0.09,0.08l0.01,0.24l-0.08,0.32l-0.31,0.24l0,0.16l-0.07,0.09l-0.24,0.01l-0.09-0.08l-0.24-0.15l-0.32-0.15l-0.34-0.38
			l-0.49-0.32l-0.17-0.47l-0.07,0.25l-0.16,0.24l-0.39,0.01l-0.1-0.39l-0.58-0.39l-0.17-0.24l-0.07,0.09l-0.24,0.01l-0.48-0.15
			l0.23-0.25l-0.16-0.15l-0.01-0.24l-0.24-0.15l-0.09-0.23l-0.16,0l-0.16-0.16l-0.08-0.08l-0.09-0.16l0.08-0.09l0.16,0l-0.09-0.16
			l-0.26-0.63l-0.33-0.4l-0.17-0.31l-0.01-0.4l-0.42-0.54l-0.33-0.4l0.31-0.17l0.08-0.09l0.46-0.49l0.16,0l0.86-0.82l-0.49-0.39
			l-0.24,0.08l-0.17-0.31l0.08-0.16l0.07-0.32l-0.09-0.4l0-0.07l0.16-0.09l0.15-0.41l0.31-0.33l-0.01-0.48l-0.18-0.56l0.24-0.24
			l0.3-0.57l0.15-0.49l0.32-0.08l0.4-0.17l0.32-0.01l0.16-0.09l0.4-0.25l0-0.16l-0.88,0.1l-0.47,0.17l-0.33-0.06l-0.23,0.08
			l-0.32-0.07l-0.24,0.08l-0.24,0.17l-0.16,0l-0.4-0.06l-0.48,0.41l-0.48,0.1l-0.47,0.17l-0.31,0.33l-0.4,0.01l-0.41-0.06
			l-0.48-0.15l-0.5-0.64l-0.17-0.47l0-0.16l-0.24,0.01l-0.81-0.46l-0.39,0.41l-0.07,0.41l-0.46,0.65l-0.23,0.57l-0.89-0.05
			l-0.17-0.4l-0.25-0.4l-0.17-0.23l-0.35-0.55l-0.41-0.47l-0.34-0.39l-0.15,0.16l-0.54,0.34l-0.39,0.49l0.04,1.21l-0.06,0.96
			l-0.39,0.17l-1.54-0.19l-0.07,0l-0.38,0.9l0.01,0.48l-0.15,0.41l0.17,0.39l-0.48-0.06l-0.08,0.32l0.01,0.23l-0.16,0.09l-0.16,0
			l-0.32-0.08l-0.32-0.15l-0.09,0l-0.07,0.25l-0.08,0.09l-0.16,0.08l-0.32-0.15l0,0.09l-0.16,0.16l-0.07,0.48l-0.37,1.3l-0.07,0.32
			l-0.08,0.24l-0.05,0.73l-0.08,0.16l-0.16-0.07l-0.23,0.08l0,0.09l-0.17,0l-0.7,0.5l-0.73-0.05l0,0.07l0.08,0.16l0,0.16l-0.15,0.16
			l0.01,0.24l-0.07,0.16l-0.17,0.16l-0.23,0.01l-0.16-0.07l-0.66-0.3l-0.23,0.01l-0.09,0l0.16,0.07l0.09,0.08l0,0.07l-0.08,0.09
			l-0.48,0.09l-0.16,0.09l0,0.16l0.33,0.31l-0.14,0.57l-0.24,0.08l0.03,1.05l0.09,0.16l0.16,0l0.32-0.09l0.25,0.08l0.08,0.16l0,0.07
			l-0.16,0.09l-0.07,0l0,0.07l0.16,0.08l0.01,0.23l-0.15,0.25l0.17,0.31l0.24,0.15l0,0.16l0,0.09l-0.07,0.09l-0.16,0.08l-0.15,0.16
			l0,0.09l0,0.16l0.16,0l0.15-0.16l0.16-0.09l0.09,0l0.08,0.08l0,0.07l-0.07,0.16l-0.08,0.16l0,0.09l0.09,0.08l0.32,0.07l0.07,0
			l0.08-0.08l-0.01-0.24l0-0.09l0.07,0l0.09,0.08l0.08,0.24l0.25,0.23l0.41,0.31l0.16,0l0.16,0.08l0.25,0.15l0,0.09l-0.16,0.08
			l-0.17-0.07l-0.23,0.01l-0.09,0l-0.16,0.08l-0.07,0.09l0.08,0.23l0.16,0.08l0.24-0.01l0.08,0.16l0.09,0.16l-0.16,0.09l-0.08,0.16
			l0,0.16l0.09,0.07l0.08,0.16l0.25,0.24l0.09,0.32l0.01,0.24l0.01,0.23l0.23-0.01l0.16,0.08l0,0.07l-0.22,0.34l-0.09,0l0.01,0.32
			l0,0.07l-0.07,0.16l-0.08,0.17l-0.16,0.08l-0.08,0.09l-0.15,0.24l0.1,0.4l-0.31,0.41l-0.16,0l-0.32,0.33l0.02,0.56l-0.16,0
			l-0.07,0.08l0,0.09l0.17,0.23l-0.16,0.09l0.01,0.4l-0.15,0.16l0.01,0.23l-0.24,0.09l-0.07,0.08l0.08,0.08l0.25,0.15l0.16,0.08
			l0.17,0.23l0.01,0.24l-0.15,0.41l0.08,0.16l0.01,0.23l0.09,0.16l0.27,0.88l0.33,0.4l0.24,0.24l0.09,0.16l-0.08,0.08l-0.07,0
			l-0.08,0.09l-0.23,0.17l-0.31,0.81l-0.39,0.1l-0.24,0.33l-0.4,0.26l0.01,0.4l0,0.16l-0.15,0.48l-0.16,0.08l-0.16,0l-0.48-0.06
			l-0.46,0.58l-0.09-0.08l-0.07,0l-0.08,0.16l-0.15,0.25l-0.09,0l-0.08-0.08l-0.09-0.16l-0.07,0.09l-0.25-0.08l-0.06,0.41l0,0.16
			l0,0.16l-0.17-0.07l-0.17-0.24l-0.16-0.15l-0.07,0l0,0.07l0.17,0.24l0,0.16l0.17,0.47l0.65,0.31l0.24-0.25l0.17,0.4l0.4-0.1
			l0.07-0.16l0.24-0.09l0.16-0.08l0.4-0.01l-0.07,0.48l0.44,1.19l0.24,0.08l0.33-0.17l0.31-0.17l0.16,0l0.25,0.15l0.16,0.07
			l0.25,0.48l0.22,0.16l-0.06,0.12l-0.31,0.1l-0.12,0.31l-0.33,0.08l-0.23,0.54l0.5,0.53l0.01,0.48l0,0.09l-0.08,0.08l0.09,0.33
			l0,0.07l0.16,0.16l0.07,0l0.09,0.24l0.08,0.24l0.41,0.22l0.09,0l0.16-0.08l0.07,0l0.09,0.16l0.49,0.31l0.73,0.3l0.65,0.23
			l1.29,0.04l0.4-0.01l0.16,0.08l0.4,0.15l0.18,0.23l0.08,0.24l0.58,0.39l0.42,0.39l-0.42,0.24h0.08l0.04-0.06l0.15,0.29l0.4-0.08
			l-0.07,0.32l0.09,0.4l0.02,0.56l0.41,0.31l0.48,0.15l0.74,0.38l0.57,0.23l0.41,0.15l0.4,0.06l0.16,0.08l0.08-0.32l0.23-0.57
			l0-0.09l0-0.07l0.38-0.58l0.08-0.16l0.23-0.66l0.31-0.24l0.07,0l0.16,0.15l-0.17-0.24l0.55-0.58l0.85-1.07l0.31-0.4l0.24-0.09
			l0.07-0.08l-0.07,0l-0.09,0l0-0.09l0.16-0.08l0.09,0l-0.08,0.08l0.09,0l0.08,0.08l0.24-0.01l0-0.09l-0.07,0l-0.09-0.07l0,0.07
			l-0.25-0.15l0.08-0.09l0.08,0.08l0-0.09l0.09,0l0.08-0.08l0.07,0l0.48,0.06l1.29-0.11l0.24-0.09l0.23-0.08l0.4-0.1l0.31-0.24
			l0.49-0.17l1.04-0.12l0.64-0.02l0.07,0l0.41,0.07l0.01,0.2c0.07,0.01,0.14,0.01,0.2,0.03l0.05-0.07l0.23-0.25l0.32-0.33l0.71-0.43
			l0.55-0.26l1.59-0.76l0.55-0.35l0.4-0.33l0.38-0.33l0.31-0.57l0.15-0.48l0.15-0.32l0.07-0.16L565.12,167.22z"></path>
	</g>
	<g id="camp_de_tarragona">
		<path class="st4 fase_0" d="M596.02,135.35l0-0.09l-0.32,0.01l-0.17,0.16l0.25,0.08L596.02,135.35z M580.34,146.59l0-0.09l-0.07,0.16
			L580.34,146.59z M572.46,148.99l-0.09,0l0.09,0.08L572.46,148.99z M596.33,140.48l-0.16-0.15l0.06-0.48l-0.17-0.31l-0.31,0.17
			l-0.08,0.16l-0.07,0.25l-0.32,0.08l-0.25-0.23l-0.25-0.24l-0.33-0.24l-0.32,0.09l-0.41-0.07l-0.34-0.71l-0.01-0.32l-0.24-0.31
			l0.96,0.13l0.16,0.15l0,0.09l0.16,0l0.32,0.15l0.32-0.25l0.16,0l0.06-0.4l0-0.09l0.24-0.33l-0.25-0.31l0.24-0.33l0.06-0.41
			l-0.07,0l-0.01-0.24l0.16,0l0.16-0.16l-0.16-0.07l-0.16,0.16l-0.09-0.08l-0.39,0.26l-0.15,0.4l-0.41-0.15l-0.49-0.31l-0.09,0
			l-0.01-0.23l0.23-0.57l0-0.09l-0.66-0.3l-0.16,0l-0.01-0.32l-0.25-0.4l-0.1-0.48l-0.25-0.24l0.47-0.58l0.32-0.08l-0.1-0.4
			l-0.16-0.07l-0.07,0.08l-0.01-0.23l-0.32-0.16l-0.24,0.09l-0.25-0.08l-0.17-0.39l-0.42-0.48l-0.72-0.14l-0.41-0.31l-1.43,0.77
			l0.08-0.25l0.15-0.32l0.15-0.4l0.23-0.18l-0.18-0.63l-0.16-0.15l0.08-0.16l0.07-0.16l-0.32-0.15l0.07-0.41l0.24,0.08l0.4-0.17
			l0-0.09l-0.8,0.02l0-0.16l-0.25-0.31l0.7-0.91l-0.01-0.49l-0.09-0.32l-0.32-0.15l-0.09-0.4l-0.33,0.01l-0.24-0.15l-0.32,0.01
			l0-0.07l-0.5-0.39l-0.73-0.14l-0.08-0.16l-0.08,0.08l-0.24-0.15l-0.32-0.15l-0.09-0.23l-0.16-0.08l-0.01-0.32l0.07-0.32l0.23-0.49
			l0.89,0.13l0.39-0.42l0.48,0.15l-0.02-0.64l-0.08-0.24l-0.49,0.1l-0.17-0.24l0.95-0.75l-1.31-0.69h0l-0.4,0.1l-0.15,0.24
			l-0.64,0.02l0.09,0.49l0.33,0.15l0.01,0.32l-0.49,0.09l-0.32-0.07l-0.17-0.24l-0.49-0.22l-2.18-0.11l-0.32-0.15l-0.47,0.34
			l-0.9-0.38l-0.48,0.25l-0.17-0.23l-0.23,0.08l-0.24,0.17l-0.48-0.15h0l-0.1,0.02l-0.39,0.08v0h0l-0.12,0.27l-0.1,0.22l-0.14,0.65
			l-0.72,0.26l-0.88,0.19l-0.04,0.08l-0.04,0.08l0.33,0.08l0,0.07l-0.08,0.09l-0.32,0.01l0.08,0.08l0.16,0l-0.06,0.64l-0.08,0.08
			l-0.16,0l0.01,0.24l0.02,0.08l0.08,0.32l0.33,0.22l0.18,0.72l-0.32,0.01l-0.23,0.11l-0.48,0.23l-1.04,0.28l0,0.16l-0.4,0.17
			l-0.47,0.17l-0.1-0.39l-0.24-0.08l-0.25-0.07l-0.41-0.16l-0.49-0.38l-0.32-0.08l-0.41-0.31l-0.58-0.39l-0.07,0.16l0.28,0.36
			l0.22,0.28l-0.16,0.16l-0.31,0.24v0l0.25,0.32l0.49,0.38l0.48,0.15l0.33,0.24L572.4,128l-0.15,0.49l-0.16,0.08l-0.32,0.09
			l-0.14,0.56l-0.15,0.49l-0.32,0.01l0.01,0.23l-0.15,0.41l-0.16,0l-0.32,0.33l-0.56-0.06l0.08,0.23l-0.4,0.33l0.01,0.17l-0.07,0.08
			l-0.41-0.15l-0.23,0.41l-0.15,0.4l-0.07,0.25l-0.4,0.33l0.01,0.4l0.16,0l-0.23,0.25l-0.33,0.17l-0.23,0.01l-0.56,0.09l-0.8,0.02
			l-0.33,0.09l-0.23,0.08l-0.56,0.26l-0.88,0.19l-0.67-0.94l-0.1-0.4l-0.56,0.18l-0.07,0.25l-0.15,0.24l-0.21,1.21l-0.32,0.09
			l-0.57-0.07l-0.16,0l-0.41-0.15l-0.57-0.14l-0.16-0.07l-0.32,0.08l-0.15,0.25l-0.15,0.32l0.01,0.4l-0.4,0.25l-0.16,0.09
			l-0.32,0.01l-0.4,0.17l-0.32,0.08l-0.15,0.49l-0.3,0.57l-0.24,0.24l0.18,0.56l0.01,0.48l-0.31,0.33l-0.15,0.41l-0.16,0.09l0,0.07
			l0.09,0.4l-0.07,0.32l-0.08,0.16l0.17,0.31l0.24-0.08l0.49,0.39l-0.86,0.82l-0.16,0l-0.47,0.49l-0.08,0.09l-0.31,0.17l0.33,0.4
			l0.42,0.54l0.01,0.4l0.17,0.31l0.33,0.4l0.26,0.63l0.09,0.16l-0.16,0l-0.08,0.09l0.09,0.16l0.08,0.08l0.16,0.15l0.16,0l0.09,0.23
			l0.24,0.15l0.01,0.24l0.16,0.15l-0.23,0.25l0.48,0.15l0.24-0.01l0.07-0.09l0.17,0.24l0.58,0.39l0.1,0.39l0.39-0.01l0.16-0.24
			l0.07-0.25l0.17,0.47l0.49,0.32l0.34,0.38l0.32,0.15l0.24,0.15l0.09,0.08l0.24-0.01l0.07-0.09l0-0.16l0.31-0.24l0.08-0.32
			l-0.01-0.24l-0.09-0.08l-0.07,0l0-0.07l0.22-0.49l0.08-0.09l0.09,0.08l0.16,0l0.31-0.17l0.17,0.23l0.57,0.14l0.57,0.46l-0.07,0.09
			l-0.08,0.16l0.01,0.24l-0.07,0l-0.09-0.16l0.08-0.16l-0.01-0.17l-0.25-0.07l-0.16,0.08l-0.15,0.25l-0.09,0l0.09,0.24L564,148.6
			l0.09,0.32l0.01,0.32l-0.15,0.16l-0.32,0.09l0,0.07l-0.24,0.01l-0.07,0.09l-0.57-0.3l-0.25-0.24l-0.46,0.74l-0.08,0.16l-0.31,0.41
			l-0.31,0.33l-0.15,0.25l0.08,0.23l0.01,0.4l-0.07,0.25l-0.29,0.89l0.08,0.16l0.16,0.16l0.43,0.95l0.17,0.24l0.35,0.87l0.58,0.46
			l-0.07,0.09l0.73,0.05l0.4,0.15l0.41,0.07l0.3-0.65l0.24-0.18l0.16,0l0.16-0.08l0.15-0.25l0-0.07l0.08-0.16l0.16-0.09l0.38-0.58
			l0.15-0.16l0.17-0.08l0.15-0.16l0.38-0.42l0.24-0.01l0.24-0.09l0.16-0.08l0.21-0.66l0.08-0.08l0.07-0.09l0.7-0.82l0.31-0.33
			l0.17-0.09l0.31-0.33l0.39-0.26l0.24-0.17l0.16,0.08l0.08-0.09l0.16,0l0.4-0.33l0.23-0.08l0.09,0l0.07-0.09l0.09,0l0.07-0.09
			l0.09,0l0.07-0.08l0-0.09l0.17,0.08l0-0.09l0.07-0.08l0.16,0l0.08-0.09l0.07,0l0-0.07l0.24-0.17l0.07,0l0.09,0.07l0.16,0
			l0.16-0.08l0.09,0l0,0.16l-0.08,0.08l0.16-0.08l0-0.09l0.08-0.08l0.23-0.01l0.25,0.07l0.24-0.01l0.31-0.17l0.56-0.09l0.47-0.18
			l0.24-0.08l0.48-0.01l0.24,0.07l0.09,0.17l0.16,0.07l0,0.16l0.24-0.01l0.16,0.08l0,0.16l0.07,0l0,0.07l0.09,0.16l-0.09,0
			l0.09,0.08l0.24-0.09l0,0.09l0.16,0l0.22-0.33l0.09,0l0.07-0.16l-0.08-0.08l0-0.16l0.22-0.49l0.23-0.57l0.31-0.33l0.31-0.25
			l0.09,0.16l0.16,0.16l0.08-0.16l-0.09-0.16l0.4-0.33l0.07,0l-0.07,0.25l0.07-0.09l0,0.16l0.09,0l0.07-0.08l-0.08-0.24l0.07,0
			l0.09,0.24l0.31-0.17l-0.08-0.08l0-0.07l0.16,0.07l0.07,0l-0.16-0.07l0.08-0.09l0.16-0.09l0.16,0.08l-0.08,0.16l0-0.07l-0.07,0.08
			l0.08,0.08l-0.56,0.25l-0.07,0.25l-0.09-0.08l-0.15,0.32l0.09,0.08l-0.24,0.25l0,0.07l0.24-0.24l0.22-0.41l0.55-0.34l0.08-0.16
			l0.07-0.09l0.08-0.16l0.07,0l0-0.09l0.08-0.08l0.08-0.09l0.16-0.08l0.16,0.07l0,0.09l0.07-0.09l0.16,0l0-0.07l0.08-0.09l0-0.07
			l0.31-0.17l0.47-0.26l0.33-0.1l0.64-0.09l0.32,0.07l0.48-0.01l0.34,0.16l0.07-0.17l0.25,0.08l0.39-0.17l0.56-0.18l0.32-0.09
			l0.09,0.08l0.16,0.07l0.24-0.01l0.07-0.08l0.16,0l0.08-0.16l0.07,0l-0.08-0.08l1.5-0.93l0.8-0.5l1.03-0.35l0.47-0.17l1.12-0.28
			l0.07,0l0.15-0.16l0.4-0.1l1.2-0.2l0.4-0.09l1.2-0.04l0.31-0.17l0.09,0.07l0.16-0.08l0.07,0l0.16-0.09l0,0.09l0.56-0.1l0.16-0.08
			l-0.17-0.4l-0.01-0.48l0.24-0.01l0.17,0.23l0.07-0.08L596.33,140.48z"></path>
	</g>
	<g id="lleida">
		<path class="st4 fase_0" d="M590.38,107.86l0,0.07l-0.78,0.5l-0.64,0.51l-0.16,0.16l-0.56,0.09l-0.56-0.06l-0.81-0.06l-0.22,0.41
			l0.08,0.16l-0.22,0.81l-0.15,0.18l0.01,0.39l-0.29,1.14l0.25,0.23l0.41,0.16l0.24,0.23l-0.32,0.09l-1.41,1.41l0.73,0.06
			l-0.23,0.49l0.33,0.06l0.01,0.4l0.73,0.06l-0.07,0.48l-0.16,0.08l0.41,0.23l0.01,0.48l0.24,0.31l0.42,0.48l0.32-0.01l-0.06,0.57
			l-0.24,0.08l-0.24,0.01l-0.16,0.09l-0.07,0.24l-0.33-0.31l-1.21,0.04l-0.15,0.16l-0.07,0.32l-0.72,0.27l0,0.16l-0.4,0.17
			l0.09,0.16l0.32,0.08l0.16-0.09l0.08,0.24l0.41,0.23l0.32,0.15l0.56-0.02l-0.06,0.64l-0.55,0.42l0.01,0.23l0.09,0.49l0.33,0.15
			l0.01,0.32l-0.49,0.09l-0.32-0.07l-0.17-0.24l-0.49-0.22l-2.18-0.11l-0.32-0.15l-0.47,0.34l-0.9-0.38l-0.48,0.25l-0.17-0.23
			l-0.23,0.08l-0.24,0.17l-0.48-0.15l-0.49,0.1l-0.22,0.49l-0.14,0.65l-0.72,0.26l-0.88,0.19l-0.08,0.16l0.33,0.08l0,0.07
			l-0.08,0.09l-0.32,0.01l0.08,0.08l0.16,0l-0.06,0.64l-0.08,0.08l-0.16,0l0.01,0.24l0.1,0.4l0.33,0.22l0.18,0.72l-0.32,0.01
			l-0.71,0.34l-1.04,0.28l0,0.16l-0.4,0.17l-0.47,0.17l-0.1-0.39l-0.24-0.08l-0.25-0.07l-0.41-0.16l-0.49-0.38l-0.32-0.08
			l-0.41-0.31l-0.58-0.39l-0.07,0.16l0.5,0.64l-0.16,0.16l-0.31,0.24l0.25,0.32l0.49,0.38l0.48,0.15l0.33,0.24L572.4,128l-0.15,0.49
			l-0.16,0.08l-0.32,0.09l-0.14,0.56l-0.15,0.49l-0.32,0.01l0.01,0.23l-0.15,0.41l-0.16,0l-0.32,0.33l-0.56-0.06l0.08,0.23
			l-0.4,0.33l0.01,0.17l-0.07,0.08l-0.41-0.15l-0.23,0.41l-0.15,0.4l-0.07,0.25l-0.4,0.33l0.01,0.24l0,0.16l0.16,0l-0.23,0.25
			l-0.33,0.17l-0.23,0.01l-0.56,0.09l-0.8,0.02l-0.33,0.09l-0.23,0.08l-0.56,0.26l-0.88,0.19l-0.67-0.94l-0.1-0.4l-0.56,0.18
			l-0.07,0.25l-0.15,0.24l-0.21,1.21l-0.32,0.09l-0.57-0.07l-0.16,0l-0.41-0.15l-0.57-0.14l-0.16-0.07l-0.32,0.08l-0.15,0.25
			l-0.15,0.32l0.01,0.24l-0.88,0.1l-0.47,0.17l-0.33-0.06l-0.23,0.08l-0.32-0.07l-0.24,0.08l-0.24,0.17l-0.16,0l-0.4-0.06
			l-0.48,0.41l-0.48,0.1l-0.47,0.17l-0.31,0.33l-0.4,0.01l-0.41-0.06l-0.48-0.15l-0.5-0.64l-0.17-0.47l0-0.16l-0.24,0.01l-0.81-0.46
			l-0.39,0.41l-0.07,0.41l-0.46,0.65l-0.23,0.57l-0.89-0.05l-0.17-0.4l-0.25-0.4l-0.17-0.23l-0.35-0.55l-0.41-0.47l-0.34-0.39
			l-0.15,0.16l-0.54,0.34l-0.39,0.49l0.04,1.21l-0.06,0.96l-0.39,0.17l-1.54-0.19l-0.84-1.35l0-0.16l0.06-0.57l-0.65-0.46
			l-0.16-0.15l0.38-0.74l0.32-0.08l0.31-0.41l-0.51-1.11l-1.65-1.32l0.7-0.75l0.3-0.57l0.15-0.4l0.07-0.41l-0.25-0.15l0.46-0.74
			l0-0.16l-0.41-0.31l-0.01-0.49l-0.17-0.39l-0.01-0.24l-0.01-0.32l1.05,0.13l0.81-0.11l0.4-0.17l0.07-0.08l0.54-0.74l0.3-0.65
			l0.46-0.58l0.15-0.25l0.55-0.58l0-0.16l-0.42-0.71l-0.25-0.31l0.15-0.24l-0.17-0.4l0.31-0.41l-0.09-0.48l-0.26-0.56l-1.03,0.51
			l-2.18-0.33l-0.11-1.05l0.07-0.24l0.39-0.42l-0.16-0.15l-0.77-1.43l-0.32-0.07l-0.03-0.88l0.54-0.74l1.01-1.07l0.15-0.16
			l0.06-0.57l0.09,0l0.46-0.82l0.49,0.48l0.13-0.97l-0.01-0.4l0.33-0.01l1.67-0.37l0.64,0.07l0.24-0.25l0.21-1.13l0.22-0.41
			l0.31-0.41l0.49-0.17l0.15-0.41l0.55-0.34l0.32-0.01l0.73,0.06l0.37-0.74l-0.01-0.48l0.23-0.57l0.89,0.13l0.07-0.41l-0.09-0.24
			l0.24-0.17l0.23-0.01l0.49-0.17l0.15-0.4l-0.58-0.47l0.28-1.21l-0.17-0.4l0.15-0.24l0.32-0.01l-1.06-0.45l-0.25-0.16l-0.32-0.07
			l-0.16-0.08l0.14-0.64l0.47-0.65l0.31-0.34l0.31-0.17l0.49,0.32l0.23-0.41l0.06-0.57l0.24-0.25l-0.01-0.23l0.79-0.27l0.19,0.56
			l0,0.09l0.14-0.73l0.39-0.26l0.31-0.65l-0.09-0.16l-0.16,0l-0.1-0.4l-0.08-0.24l0.21-0.73l0.4-0.25l0.13-1.05l0.3-0.57l0.17,0.47
			l0.58,0.46l0.73,0.3l1.13,0.05l0.08,0.24l0.5,0.3l0.48,0.06l1.12-0.27l0.32,0.15l1.21,0.12l0.65,0.23l0.81-0.11l0.63-0.26l0,0.09
			l0.09,0.16l0.24,0.15l0.99,0.7l1.12-0.12l0.41,0.16l0.32,0.07l1.2-0.11l0.49-0.01l0.46-0.66l0.32,0.15l0.63-0.18l0.97-0.1
			l0.26,0.55l0.03,1.05l0.32-0.09l0.36-1.06l-0.17-0.4l-0.5-0.55l0.39-0.41l0.16-0.09l0.31-0.49l0.08-0.32l0.37-0.74l0.32-0.25
			l0.22-0.33l0.15-0.48l0.39-0.66l0.15-0.24l0.25,0.23l0.07-0.08l0.16,0.07l0.15-0.4l0.33-0.01l0.08,0.23l-0.06,0.49l-0.08,0.32
			l0.18,0.79l0.01,0.32l0.32-0.08l0.09,0l0.16-0.16l0.08-0.09l0.16,0.08l0.47-0.17l0.25,0.55l0.25,0.08l0.16-0.09l0.16,0.17
			l0.01,0.39l0.57,0.14l0,0.16l0.16,0.15l0.1,0.56l-0.31,0.17l0.25,0.64l-0.14,0.57l0.09,0.16l0.5,0.71l-0.08,0.08l0.25,0.08
			l0.41,0.31l-0.15,0.48L580.06,99l0.02,0.71l0.16,0.17l-0.43,1.61l-0.15,0.25l0.23-0.08l0.33-0.09l1.02-0.83l0.08,0.07l0.16-0.08
			l0,0.16l0.65,0.14l0.17,0.31l-0.14,0.57l-0.54,0.82l0.56-0.18l0.08,0.32l0.26,0.4l0.01,0.48l0.01,0.4l0.17,0.24l0.08-0.09
			l0.31-0.25l0.15-0.24l0-0.16l0.23-0.25l0.33,0.15l0.21-0.81l0.47-0.26l0.24-0.01l0.16-0.08l0.24-0.09l0.66,0.71l0.09,0l0.23-0.09
			l0.9,0.62l-0.15,0.48l0.56-0.18l0.26,0.47l0.73,0.22l0.46-0.82l0.31-0.33l0.65,0.23l0.1,0.56l0.01,0.39l0.02,0.65l-0.08,0.16
			l0.18,0.63l-0.3,0.57l-0.31,0.41l-0.55,0.34l0.66,0.46l0.17,0.31L590.38,107.86z"></path>
	</g>
	<g id="alt_pirineu_i_aran">
		<path class="st4 fase_0" d="M582.64,98.92l-0.17,0.01l-0.16,0.08l0,0.16l0.17,0.24l0.09,0l0.08-0.09l0-0.16L582.64,98.92z M608.83,73.94
			l1.12-0.12l0.08-0.16l-0.09-0.07l-0.16-0.16l-0.66-0.55l-0.17-0.31l0.22-0.66l0.17-0.01l-0.5-0.46l-0.23,0.01l0,0.09l-0.08,0.16
			l0,0.16l-0.23,0.17l-0.4,0.33l-0.21,1.05l0.16,0.15L608.83,73.94z M610.03,79.06l-0.33-0.24l-0.16,0l-0.25-0.15l-0.25-0.15
			l-0.25-0.4l-0.25-0.15l-0.33-0.31l-0.1-0.48l-0.08-0.16l-0.1-0.56l-0.17-0.4l0.06-0.8l-0.66-0.39l-0.01-0.48l-0.09-0.65
			l-0.32-0.07l-0.4,0.25l-0.48,0.01l-0.09,0l-0.32-0.07l-0.32,0.08l-0.16-0.07l-0.17-0.47l-0.74-0.38l-0.16,0.16l-0.41-0.23
			l-0.17-0.31l-0.5-0.55l-0.41-0.06l-0.24,0.08l-0.58-0.38L600.89,72l-0.64-0.14l-0.32,0.09l-0.25-0.24l-0.32-0.15l-0.64,0.1
			l-0.26-0.56l-0.24,0.08l-0.54,0.51l-0.65-0.15l-0.39,0.01l-0.34-0.47l-0.22,0.49l-0.06,0.73l-0.06,0.41l-0.15,0.41l-0.24,0.01
			l-0.32-0.08l-0.24,0.01l-0.23,0.25l-0.41-0.15l-0.4,0.09l-0.48-0.15l-0.31,0.17l-0.38,0.74l-0.32-0.15l-0.33-0.24l-0.24,0.01
			l-0.08,0.25l0.09,0.32l-0.24,0.33l-0.06,0.41l-0.72,0.11l-0.63,0.18L589.4,75l-0.64-0.06l-0.73-0.06l-0.65-0.14l-0.17-0.31l0.16,0
			l-0.17-0.31l0.14-0.57l-0.24-0.24l0.06-0.57l-0.49-0.38l-0.41-0.07l0.23-0.41l0.79-0.26l0.86-0.84l-0.51-0.87l-0.09-0.55
			l-0.41-0.32l-0.87,0.43l-0.24,0.09l0.23-0.57l0.14-0.57l0.71-0.5l0.07-0.25l-0.16-0.16l-0.49-0.39l-0.01-0.39l0.31-0.17l0.3-0.66
			l-0.16-0.07l-0.24-0.32l-0.17-0.23l-0.01-0.24l-0.25-0.23l-0.27-0.8l-0.09-0.4l-0.01-0.48l-0.32-0.15l-0.26-0.56l-0.48-0.06
			l0.06-0.41l-0.01-0.48l-0.33-0.4l-0.64-0.14l-0.41-0.31l-0.01-0.4l0.15-0.32l-0.01-0.32L583,59.67l-0.25-0.23l-0.24-0.08
			l-0.48,0.26l-0.8,0.02l-0.56,0.25l-0.16-0.07l-0.41-0.07l-0.23,0.17l-0.33,0.01l-0.16-0.15l-0.33-0.38l-0.48-0.16l-0.39,0.34
			l-0.48,0.01l-0.63,0.18l-0.48,0.25l-0.31,0.25l-0.41-0.15l-0.33-0.24l-0.25-0.39l-0.5-0.23l-0.01-0.4l-0.08-0.23l-0.02-0.65
			l-0.33-0.47l-0.41-0.23l-0.17-0.31l-0.74-0.38l-0.25-0.23l-0.17-0.31l-0.16,0.08l-1.2,0.28l-0.09-0.16l-0.24-0.15l-0.4,0.09
			l-0.89-0.29l-0.63,0.26l-0.25-0.08l-0.17-0.24l-0.09-0.23l-0.24-0.15l-0.16-0.16l-0.25-0.15l-0.56,0.25l-0.38,0.33l-0.08,0.33
			l-0.16-0.08l-0.49-0.23l-0.25-0.15l-0.25-0.23l-0.41-0.39l-0.66-0.39l-0.32,0.01l-0.5-0.62l-0.73-0.14l-0.32,0.01l-1.06-0.62
			l-0.4,0.1l-0.48,0.09l-0.41-0.06l-0.49-0.23l-0.98-0.29l-0.17-0.4l-0.49-0.22l-0.16,0l-0.32-0.08l-0.8,0.34l-0.32,0.01l-0.15,0.49
			l-0.63,0.41l0.17,0.49l0.16,0.15l-0.07,0.48l0.1,0.4l0.09,0.39l-0.47,0.34l-0.23,0.49l-0.23,0.25l0.4,0.31l0.33,0.15l-0.32,0.24
			l-0.31,0.25l-0.46,0.58l0.73,0.05l0.25,0.15l0.01,0.4l0.17,0.24l0.17,0.4l0.08,0.07l0.25,0.31l-0.24,0.17l0.34,0.64l-0.22,0.33
			l-0.08,0.32l0.01,0.32l-0.23,0.09l-0.09-0.08l-0.24,0.01l0.26,0.4l0.25,0.07l0.26,0.72l0.34,0.32l0.24,0.15l0.18,0.79l0.03,0.97
			l0.58,0.38l0.49,0.23l0.65,0.14l0.16-0.09l0.41,0.16l0.48,0.06l0,0.09l-0.38,0.9l0.03,0.88l-0.23,0.65l-0.14,0.57l-0.38,0.42
			l0.34,0.54l-0.32-0.07l-0.25-0.24l-0.46,0.65l0.01,0.4l-0.07,0.41l0.32,0.15l0.34,0.24l-0.17,0.08l-0.15,0.41l-0.62,0.5
			l-0.25-0.15l-0.32-0.15l-0.06,0.41l-0.24,0.01l-0.08,0.08l0.69,1.35l0.02,0.55l0.26,0.96l0.58,0.46l0.26,0.56l-0.24,0.25
			l-0.07,0.16l0.4,0.31l0.27,0.88l-0.15,0.32l-0.16,0.08l-0.08,0.25l-0.47,0.17l0.41,0.31l0.24,0.08l0.41,0.15l0.41,0.07l0.07,0
			l0.34,0.22l-0.16,0.16l-0.17,0.09l-0.23,0.08l-0.08,0.16l-0.31,0.18l-0.16,0.08l0,0.16l0.17,0.47l0.32,0.08L560,81.1l0.25,0.07
			l-0.48,0.49l0.09,0.24l-0.46,0.58l-0.07,0.64l-0.15,0.32l-0.14,0.81l-0.06,0.73l-0.22,0.33l-0.14,0.57l-0.47,0.57l0.1,0.33
			l-0.07,0.48l-0.22,0.41l-0.07,0.48l-0.37,1.13l0.09,0.17l0.57,0.3l-0.4,0.25l-0.31,0.41l0.01,0.4l0.02,0.8l0.08,0.16l0.09,0.24
			l-0.07,0.48l-0.21,0.66l-0.24,0.33l-0.23,0.49l-0.14,0.57l0.17,0.47l0.58,0.46l0.73,0.3l1.13,0.05l0.08,0.24l0.5,0.3l0.48,0.06
			l1.12-0.27l0.32,0.15l1.21,0.12l0.65,0.23l0.81-0.11l0.63-0.26l0,0.09l0.09,0.16l0.24,0.15l0.99,0.7l1.12-0.12l0.41,0.16
			l0.32,0.07l1.2-0.11l0.49-0.01l0.46-0.66l0.32,0.15l0.63-0.18l0.97-0.1l0.26,0.55l0.03,1.05l0.32-0.09l0.36-1.06l-0.17-0.4
			l-0.5-0.55l0.39-0.41l0.16-0.09l0.31-0.49l0.08-0.32l0.37-0.74l0.32-0.25l0.22-0.33l0.15-0.48l0.39-0.66l0.15-0.24l0.25,0.23
			l0.07-0.08l0.16,0.07l0.15-0.4l0.33-0.01l0.08,0.23l-0.06,0.49l-0.08,0.32l0.18,0.79l0.01,0.32l0.32-0.08l0.09,0l0.16-0.16
			l0.08-0.09l0.16,0.08l0.47-0.17l0.25,0.55l0.25,0.08l0.16-0.09l0.16,0.17l0.01,0.39l0.57,0.14l0,0.16l0.16,0.16l0.1,0.56
			l-0.31,0.17l0.25,0.64l-0.14,0.57l0.09,0.16l0.5,0.71l-0.08,0.08l0.25,0.08l0.41,0.31l-0.15,0.48L580.06,99l0.02,0.71l0.16,0.17
			l-0.43,1.61l-0.15,0.25l0.23-0.08l0.33-0.09l1.02-0.83l0.08,0.07l0.16-0.08l-0.17-0.24l0.15-0.25l0.08-0.08l0.07-0.25l-0.08-0.07
			l0.16,0l0.15-0.49l0.31-0.33l-0.16-0.15l0-0.07l0.16,0l0.49,0.06l0.16,0.08l0.39-0.01l0.4-0.17l0.16,0l0.62-0.51l-0.17-0.47
			l-0.17-0.47l0.07-0.49l0.07,0l-0.09-0.48l0.15-0.24l-0.16-0.08l0.06-0.41l-0.03-1.04l0-0.16l-0.16-0.07l-0.16,0.16l-0.16,0
			l-0.57-0.23l-0.42-0.87l0.13-1.05l0.31-0.33l0.23-0.09l0.08-0.08l0.24-0.09l0.07-0.24l0.24-0.01l0.16-0.09l0.23-0.49l-0.25-0.15
			l-0.09-0.08l-0.01-0.24l0.09,0l0.16-0.08l0.09,0.07l0-0.07l0.07,0l0.47-0.26l0.33,0.15l0.39-0.01l0.8-0.42l0.56-0.02l0.08,0.32
			l0.58,0.39l1.1-0.68l0.95-0.75l0.78-0.66l0.08-0.25l0.24-0.17l-0.02-0.64l0.72-0.27l0.89,0.06l0.48,0.15l0.73,0.22l0.64-0.02
			l-0.1-0.65l0.53-1.06l0.08-0.32l0.31-0.33l0.15-0.16l0.8-0.34l0.16,0l0.32-0.09l0.56-0.1l0-0.16l0.39-0.25l-0.18-0.72l-0.73-0.14
			l0.08-0.16l0.32-0.09l0.48-0.01l0.24-0.01l0.09,0.08l0.47-0.26l0.17,0.24l0.64-0.02l0.09,0l0.22-0.41l0.65-0.02l0.56-0.02
			l0.32,0.07l0.39-0.25l0.33-0.01l0-0.16l0.31-0.25l0.48,0.15l0.4-0.09l0.31-0.25l0.24-0.01l0.47-0.26l0.31-0.17l0.16,0.08
			l0.32-0.09l0.09,0l0.32-0.08l0.24-0.09l0.15-0.24l0.48-0.1l0.65,0.3l0.73,0.06l0.1,0.39l0.4-0.33l0.16,0.15l0.13-0.89l0.23-0.24
			l0.33-0.01l0.24,0.07l0.47-0.25l-0.01-0.49l0.16,0.17l0.33-0.01l0.47-0.34L610.03,79.06z"></path>
	</g>
	<g id="barcelona">
		<path class="st3 fase_0" d="M634.72,114.83l-0.25-0.31l-0.16-0.08l-0.17-0.31l-0.17-0.47l0.23-0.41l0.22-0.49l-0.17-0.24l0.07-0.4
			l-0.88,0.19l-0.16-0.08l-0.33,0.09l-0.63,0.34l-0.57-0.23l-0.17-0.39l0.08-0.25l-0.25-0.31l-0.41-0.31l0.08-0.09l-0.17-0.24
			l0.16-0.16l-0.09-0.32l-0.8-0.14l-0.09,0l-0.49-0.31l-0.58-0.39l0.14-0.57l-0.63,0.26l-0.49-0.23l-0.1-0.48l-0.49-0.39l-0.48,0.01
			l-0.33,0.17l-0.8-0.14l-0.16-0.15l-0.16-0.07l-0.46,0.74l-0.32,0.24l0.18,0.31l-0.08,0.33l0.01,0.48l-0.71,0.34l-0.48,0.01
			l-0.35-1.19l-0.34-0.24l-0.65-0.37l-0.47,0.25l-0.08,0.25l-0.73-0.3l-0.16-0.15l-0.01-0.33l-0.15,0.41l-0.31,0.41l-0.31,0.49
			l0.01,0.48l0.17,0.24l0,0.16l-0.08,0.16l-0.07,0.25l0.08,0.32l0.33,0.15l0.08,0.24l-0.15,0.32l0.16,0.07l0.16,0.15l-0.31,0.18
			l-0.09-0.08l-0.32,0.01l-0.23,0.09l-0.16-0.17l-0.24,0.18l-0.25-0.08l-0.31,0.17l-0.41-0.06l-0.14,0c0,0.04,0.01,0.08,0.01,0.12
			l-0.03-0.12l-0.39,0.01l-0.35-0.71l0.16-0.41l-0.01-0.4l0.05-0.96l-0.17-0.4l-0.01-0.24l-0.09-0.32l-0.33-0.22l-0.17-0.24
			l-0.4,0.33l-0.07,0.09l-0.16-0.08l-0.16,0l0-0.16l-0.24,0.25l-0.31,0.17l-0.39,0.58l-0.16-0.17l-0.87,0.36l-0.16-0.08l-0.07,0.16
			l-0.17,0.01l-0.08-0.07l-0.25-0.32l-0.09-0.39l-0.16-0.08l-0.08,0.16l-0.25-0.07l-0.32-0.08l-0.48,0.1l-0.4,0.33l0.18,0.72
			l-0.07,0.24l0.49,0.32l-0.4,0.17l0.1,0.48l-0.09,0l-0.23,0.25l-0.15,0.4l-0.32,0.25l0.01,0.24l-0.47,0.17l0.09,0.32l0.18,0.63
			l-0.14,0.65l-0.31,0.17l0.09,0.55l-0.24,0.25l0.01,0.24l0.18,0.47l0.01,0.24l-0.32,0.24l-0.48,0.01l-0.31,0.33l-0.48,0.01
			l-0.4,0.17l-0.16-0.07l-0.32,0.01l-0.16-0.15l-0.16-0.08l-0.01-0.39l-0.18-0.17l-0.16-0.07l-0.15,0.25l-0.41-0.39l-0.56,0.18
			l-0.31,0.33l-0.08,0.09l-0.31,0.4l0.08,0.08l0.11,0.88l0.09,0.56l-0.31,0.24l-0.56,0.18l-0.33-0.31l-0.07,0.25l0.1,0.8l-0.06,0.33
			l-0.4,0.01l-0.8,0.02l-0.58-0.39l-0.25-0.24l-0.27,1.78l-0.31,0.25l-0.08,0.24l0.25,0.24l0.16,0.08l0.24-0.01l0.24,0.07l0.18,0.56
			l0.16,0.07l-0.07,0.09l0.08,0.08l0.42,0.54l0.33,0.24l0.41,0.31l0,0.09l0.01,0.48l-0.24,0.08l-0.23,0.01l-0.15,0.41l0,0l0,0.09
			l-0.55,0.65l-0.47,0.26l-0.24,0.24l0,0.16l-0.48-0.15l-0.08,0.16l0,0.09l0.17,0.31l-0.88,0.19l0.01,0.49l0,0.07l-0.22,0.41
			l-0.82-0.38l0.08-0.32l-0.16,0l-0.41-0.15l-0.32-0.15l-0.48-0.07l-0.18-0.15l-0.07,0l-0.33-0.31l-0.16,0l-0.86-1.74l-0.8,0.02
			l-1.13-0.05l-0.73-0.3l-0.32,0.01l0.17,0.47l-0.16,0l-0.24,0.18l0.01,0.23l0,0.16l-0.71,0.18l-0.39,0.42l-0.07,0.57l0.42,0.55
			l0.41,0.06l0.18,0.56l-0.96,0.43l-0.25-0.4l-0.49-0.06l-0.07,0.24l-0.4,0.1l0.01,0.23l-0.32-0.15l0.08,0.16l-0.95,0.36l-0.4,0.33
			l-0.33-0.39l0,0.09l-0.4,0.17l-0.24-0.08l-0.07,0.41l0.32,0.15l-0.07,0.16l-0.08,0.16l0.16,0.15l0.18,0.63l-0.23,0.18l-0.15,0.4
			l-0.15,0.32l-0.08,0.25l1.43-0.77l0.41,0.31l0.72,0.14l0.42,0.48l0.17,0.39l0.25,0.08l0.24-0.09l0.32,0.16l0.01,0.23l0.07-0.08
			l0.16,0.07l0.1,0.4l-0.32,0.08l-0.47,0.58l0.25,0.24l0.1,0.48l0.25,0.4l0.01,0.32l0.16,0l0.66,0.3l0,0.09l-0.23,0.57l0.01,0.23
			l0.09,0l0.49,0.31l0.41,0.15l0.15-0.4l0.39-0.26l0.09,0.08l0.16-0.16l0.16,0.07l-0.16,0.16l-0.16,0l0.01,0.24l0.07,0l-0.06,0.41
			l-0.24,0.33l0.25,0.31l-0.24,0.33l0,0.09l-0.06,0.4l-0.16,0l-0.32,0.25l-0.32-0.15l-0.16,0l0-0.09l-0.08-0.07l-0.09-0.08
			l-0.96-0.13l0.24,0.31l0.01,0.32l0.34,0.71l0.41,0.07l0.32-0.09l0.33,0.24l0.25,0.24l0.25,0.23l0.32-0.08l0.07-0.25l0.08-0.16
			l0.31-0.17l0.17,0.31l-0.06,0.48l0.16,0.15l0.25,0.48l-0.07,0.08l-0.17-0.23l-0.24,0.01l0.01,0.48l0.17,0.4l0.08-0.09l0.23-0.01
			l-0.07,0.09l-0.24,0.08l0.4-0.01l0-0.07l0-0.09l0.72-0.1l0.24-0.18l0.16-0.08l0.16-0.09l0.23-0.17l0.4-0.09l0.4-0.1l0.23-0.08
			l0.33-0.01l0.08,0.07l-0.08-0.23l0.32-0.09l0.08,0.32l0.09,0l-0.09-0.16l0.08-0.08l0.15-0.17l0.24-0.01l0.16,0l0.16-0.08
			l0.24,0.07l0.08-0.08l0.07,0l0.08-0.09l0.09,0l0.07,0l0.24-0.17l0.16,0.08l0.16-0.09l0,0.09l0.07-0.09l0-0.07l0.17-0.09l0.07-0.08
			l0.16-0.09l0.4-0.17l0.16,0l0.16-0.09l0.16-0.08l0.09,0.07l0,0.09l0.07,0l0.08-0.09l0.08,0.08l0-0.09l0.08-0.08l0.41,0.15
			l0.07-0.09l0.16,0.08l0.08-0.09l0.16,0l0.25,0.08l0.16-0.09l0.07-0.16l0.16,0l0,0.09l0.08-0.09l-0.09-0.07l0.09,0l0-0.09
			l0.56-0.25l0.32-0.09l0.23-0.08l0.15-0.17l0.09,0l0.16-0.08l0.16-0.09l0.72-0.26l-0.07,0.08l-0.08,0.09l0.09,0l0.23-0.17
			l0.24-0.09l0.32-0.08l2.33-0.07l0.88-0.19l0.88-0.19l1.11-0.36l1.2-0.51l0.63-0.34l0.15-0.25l0.24-0.08l0.07,0l0.16,0l0.09,0.07
			l-0.09-0.16l-0.08-0.07l0.07,0l0.09,0l0.16-0.16l-0.16-0.08l0.22-0.49l0.08-0.16l0.16,0.15l0.09,0l-0.18-0.15l0-0.09l0-0.07
			l0.09,0l0.08-0.09l0.07,0l0-0.09l0.08-0.24l0.23-0.01l0-0.09l0.08-0.24l0.15-0.41l-0.09-0.08l-0.16,0l-0.07,0l-0.08,0.09
			l-0.15,0.32l-0.07,0.25l-0.17,0.01l0.08-0.16l-0.16-0.08l0.07-0.08l0.17-0.01l0.06-0.57l0.09,0.16l0,0.07l0.31-0.24l-0.09-0.08
			l0.08-0.16l0.07,0l0.16,0.15l0,0.09l0.23-0.49l-0.16-0.17l0.08-0.08l0.16,0.07l0.07-0.08l-0.08-0.08l-0.09,0l0.08-0.16l0.16,0.08
			l0.07-0.16l-0.16-0.08l-0.01-0.23l0.09,0l0.08,0.07l0.09,0l-0.16-0.15l0.15-0.25l0,0.09l-0.08,0.08l0.09,0.08l0.07-0.16
			l-0.08-0.16l0.07,0l0.09,0l0,0.07l0,0.09l0,0.07l-0.09,0l0.01,0.17l0.08-0.09l0,0.09l-0.08,0.08l0,0.16l0.09,0l0,0.09l0.07,0
			l-0.07,0.16l-0.08,0.32l0.08-0.25l0.07-0.08l0.08-0.32l-0.09,0l0-0.09l0.16-0.41l0.07-0.08l0.08-0.16l0.07-0.09l0-0.07l0-0.09
			l0.09,0l0.07,0l0-0.07l0.24-0.34l0.16-0.16l0.08-0.08l0.07-0.09l0-0.07l0.31-0.25l0-0.09l0.09,0.08l0.07-0.25l0.09,0.08l0-0.09
			l-0.09-0.07l-0.07,0l0.07-0.16l0.09,0.07l0,0.09l0.07-0.09l-0.08-0.07l0.07,0l0-0.16l0.08-0.16l0.08-0.09l0.07-0.09l0-0.07
			l0.08-0.16l0-0.09l0.15-0.32l0.01,0.24l0.07-0.25l0-0.07l0-0.09l0.09,0l0-0.07l0.07-0.17l0.08-0.08l0.07-0.09l0.08-0.16l0.16-0.16
			l0.08-0.08l0.23-0.17l0.15-0.17l0.24-0.17l0.07,0l0.09,0l0.08-0.08l0.39-0.26l0.16-0.08l0.08-0.09l0.07,0l0.33-0.17l0.23-0.01
			l-0.07,0.09l-0.08,0.08l0.09,0l0.07-0.08l0.24-0.18l0.16-0.08l0.16-0.09l0.24-0.01l0.16-0.08l0.23-0.09l0.09,0l0.09,0l0.16-0.08
			l0.16-0.09l0.07,0l0.16-0.08l0.08-0.09l0.07-0.08l0.09,0l0.09,0.07l0.07,0l0.08-0.08l0-0.09l0.07,0l0.08-0.09l0.16-0.08l0.16-0.09
			l0.07-0.08l0.24-0.17l0.09,0l0.07-0.09l0.09,0l0.16-0.16l0.07,0l0.08-0.08l0.07-0.09l0.15-0.25l0.17-0.08l0.07-0.09l0.16-0.08
			l0.08-0.09l0.07,0l0.4-0.33l0.08-0.08l0.16-0.09l0.07,0l0.08-0.16l0,0.07l0.07,0l0-0.07l0.08-0.16l0.07-0.09l0.16-0.08l0.08-0.09
			l0-0.07l0.16-0.09l0.16-0.16l0.16,0l0.31-0.17l0.4-0.1l0.08-0.08l0.16,0l0.07-0.09l0.09,0l0-0.07l0.07,0l0.08-0.16l0.09,0
			l0.07-0.09l0.09,0l0,0.09l-0.08,0.08l0.16-0.08l0-0.09l0.16-0.08l0.09,0l0.23-0.25l0.4-0.17l0.63-0.42l0.33-0.08l0-0.09l0.07,0
			l0.09,0l0.07,0l-0.07,0.25l-0.09,0l0,0.07l0.16-0.08l0.08-0.16l0.23-0.18l0.18-0.18c0,0-0.94-3.54-1.11-3.95l0.09-0.1l0.4,0.06
			l0.8-0.18l0.48,0.07l0.09,0l0.14-0.57l0.38-0.33L634.72,114.83z"></path>
	</g>
	<g id="catalunya_central">
		<path class="st4 fase_0" d="M629.77,96.96l-0.01-0.48l0.16,0l0.15-0.18l0.15-0.32l-0.08-0.07l-0.75-0.7l-0.17-0.31l0.39-0.26l0.16-0.16
			l0-0.16L629.53,94l-0.48,0.01l-0.24,0.17l-0.09-0.32l-0.16-0.17l-0.4-0.15l-0.25-0.07l-0.08,0.24l-0.71,0.51l-0.23,0.08
			l-0.25-0.07l-0.42-0.39l0-0.16l-0.17-0.24l-0.33-0.47l0-0.07l-0.08-0.08l0.22-0.57l-0.08-0.07l-0.72,0.02l-0.17-0.33l-0.1-0.39
			l0.08-0.09l0.22-0.33l-0.32-0.15l-0.33-0.24l-0.58-0.38l-0.09-0.49l0.15-0.24l0.15-0.33l0.63-0.25l0.08-0.16l0.4,0.06l0-0.16
			l-0.32-0.15l-0.24-0.07l-0.01-0.24l-0.4,0.26l-0.09,0l-0.55,0.18l-0.33,0.08l-0.31,0.17l-0.16,0.09l-0.32,0.01l-0.4,0.25
			l-0.24-0.23l-0.33-0.15l-0.23,0.08l-0.32-0.07l-0.56,0.02l0.24,0.07l0.41,0.23l0.09,0.16l0.23-0.01l-0.16,0.16l-0.07,0.16
			l0.24,0.08l0.16-0.16l0.09,0.07l-0.31,0.49l-0.24,0.01l-0.32-0.07L620.56,90l-0.24,0.17l-0.23,0.17l-0.56,0.1l-0.17-0.33
			l-1.04,0.19l-0.08,0.09l-0.07,0.25l0.08,0.32l-0.16,0.08l-0.16-0.16l-0.08,0.25l-0.32,0.01l-0.23,0.01l-0.01-0.32l-0.09-0.16
			l-0.96-0.06l-0.64,0.26l-0.31,0.49l-0.47,0.49l-0.32-0.15l-0.58-0.39l-0.09-0.16l0-0.16l-0.08-0.07l-0.01-0.49l-0.25-0.23
			l-0.31,0.17l-0.57-0.07l0,0.16l-0.31,0.18l-0.08,0.08l-0.08-0.07l-0.25-0.24l-0.16-0.15l-0.32,0.08l-0.25-0.31l-0.33-0.22
			l-0.88,0.19l-0.01-0.24l-0.08-0.16l-0.43-0.79l-0.09-0.24l0.08-0.25l-0.48,0.01l-0.56,0.1l-0.03-1.04l0.32,0.07l0.57,0.14
			l0.8-0.02l-0.4,0.17l0.18,0.33l0.08,0.16l-0.3,0.57l0.33,0.31l0.09,0.16l0.64-0.1l0.24-0.24l0.06-0.33l0.24-0.24l0.33,0.22
			l0.56-0.02l0.15-0.32l-0.1-0.48l0.15-0.16l-0.24-0.08l-0.25-0.07l-0.1-0.81l0.07-0.24l0.08-0.25l-0.41-0.06l-0.4,0.17l-0.16,0.16
			l-0.25-0.56l-0.25-0.31l0.31-0.25l0.08-0.24l0.06-0.57l0-0.09l0.31-0.41l0.23-0.49l0.31-0.33l0.08-0.16l0.55-0.34l0.15-0.32
			l-0.25-0.08l-0.32-0.15l-0.41-0.23l-0.33-0.22l-0.57-0.23l-0.48,0.01l-0.4,0.1l-0.4,0.33l-0.39,0.01l-1.06-0.62l-0.16-0.15
			l-0.4,0.33l-0.1-0.39L606.46,81l-0.65-0.3l-0.48,0.1l-0.15,0.24l-0.24,0.09l-0.32,0.08l-0.09,0l-0.32,0.09l-0.16-0.08l-0.31,0.17
			l-0.47,0.26l-0.24,0.01l-0.31,0.25l-0.4,0.09l-0.48-0.15l-0.31,0.25l0,0.16l-0.33,0.01l-0.39,0.25l-0.32-0.07l-0.56,0.02
			l-0.65,0.02l-0.22,0.41l-0.09,0l-0.64,0.02l-0.17-0.24l-0.47,0.26l-0.09-0.08l-0.24,0.01l-0.48,0.01l-0.32,0.09l-0.08,0.16
			l0.73,0.14l0.18,0.72L597,84.26l0,0.16l-0.56,0.1l-0.32,0.09l-0.16,0l-0.8,0.34l-0.15,0.16l-0.31,0.33l-0.08,0.32l-0.53,1.06
			l0.1,0.65l0.16-0.09l0,0l-0.16,0.09l-0.64,0.02l-0.73-0.22l-0.48-0.15l-0.89-0.06l-0.72,0.27l0.02,0.64l-0.24,0.17l-0.08,0.25
			l-0.78,0.66l-0.95,0.75l-1.1,0.68l-0.58-0.39l-0.08-0.32l-0.56,0.02l-0.8,0.42l-0.39,0.01l-0.33-0.15l-0.47,0.26l-0.07,0l0,0.07
			l-0.09-0.07l-0.16,0.08l-0.09,0l0.01,0.24l0.09,0.08l0.25,0.15l-0.23,0.49l-0.16,0.09l-0.24,0.01l-0.07,0.24l-0.24,0.09
			l-0.08,0.08l-0.23,0.09l-0.31,0.33l-0.13,1.05l0.42,0.87l0.57,0.23l0.16,0l0.15-0.16l0.16,0.07l0,0.16l0.03,1.04l-0.06,0.41
			l0.16,0.08l-0.15,0.24l0.09,0.48l-0.07,0l-0.07,0.49l0.17,0.47l0.17,0.47l-0.62,0.51l-0.16,0l-0.4,0.17l-0.39,0.01l-0.16-0.08
			l-0.49-0.06l-0.16,0l0,0.07l0.16,0.16l-0.31,0.33l-0.15,0.49l-0.16,0l0.08,0.07l-0.07,0.25l-0.08,0.08l-0.15,0.25l0.17,0.24
			l0,0.16l0.65,0.14l0.17,0.31l-0.14,0.57l-0.54,0.82l0.56-0.18l0.08,0.32l0.26,0.4l0.01,0.48l0.01,0.4l0.17,0.24l0.08-0.09
			l0.31-0.25l0.15-0.24l0-0.16l0.23-0.25l0.33,0.15l0.21-0.81l0.47-0.26l0.24-0.01l0.16-0.08l0.24-0.09l0.66,0.71l0.09,0l0.23-0.09
			l0.9,0.62l-0.15,0.48l0.56-0.18l0.26,0.47l0.73,0.22l0.46-0.82l0.31-0.33l0.65,0.23l0.1,0.56l0.01,0.39l0.02,0.65l-0.08,0.16
			l0.18,0.63l-0.3,0.57l-0.31,0.41l-0.55,0.34l0.66,0.46l0.17,0.31l0.63-0.25l0,0.07l-0.78,0.5l-0.64,0.51l-0.15,0.16l-0.06,0.64
			l0.01,0.32l-0.01-0.32l0.06-0.64l-0.56,0.09l-0.56-0.06l-0.81-0.06l-0.22,0.41l0.08,0.16l-0.22,0.81l-0.15,0.18l0.01,0.39
			l-0.29,1.14l0.25,0.23l0.41,0.16l0.24,0.23l-0.32,0.09l-1.41,1.41l0.73,0.06l-0.23,0.49l0.33,0.06l0.01,0.4l0.73,0.06l-0.07,0.48
			l-0.16,0.08l0.41,0.23l0.01,0.48l0.24,0.31l0.42,0.48l0.32-0.01l-0.06,0.57l-0.24,0.08l-0.24,0.01l-0.16,0.09l-0.07,0.24
			l-0.33-0.31l-1.21,0.04l-0.15,0.16l-0.07,0.32l-0.72,0.27l0,0.16l-0.4,0.17l0.09,0.16l0.32,0.08l0.16-0.09l0.08,0.24l0.41,0.23
			l0.32,0.15l0.56-0.02l-0.06,0.64l-0.55,0.42l0.01,0.23l0,0l0.09,0.49l0.33,0.15l0.01,0.32l-0.49,0.09l-0.32-0.07l-0.17-0.24
			l-0.49-0.22l-2.18-0.11l-0.32-0.15l-0.47,0.34l-0.9-0.38l-0.48,0.25l-0.17-0.23l-0.23,0.08l-0.24,0.17l-0.48-0.15l-0.49,0.1
			l-0.22,0.49l-0.14,0.65l-0.26,0.09c0.33,0.41,0.6,0.86,0.91,1.29c0.04,0.06,0.08,0.12,0.13,0.17c0.01,0.01,0.05,0.05,0.06,0.06
			c0.06,0.06,0.12,0.12,0.18,0.17c0.01,0.01,0.03,0.03,0.06,0.06c0,0,0.01,0,0.01,0c0,0,0,0,0.02,0.01c0.01,0,0.02,0.01,0.03,0.01
			c0.09,0.03,0.19,0.05,0.28,0.07c0.56,0.12,1.2,0.23,1.65,0.54c0,0,0.01,0,0.01,0c0.01,0,0.01,0,0.01,0
			c0.55-0.44,1.45-0.16,1.47,0.62c0.01,0.67-0.03,1.36-0.13,2.02c-0.03,0.18-0.07,0.36-0.09,0.55l0.24-0.32l0.08,0.07l0.09,0.24
			l0.02,0.8l0.63-0.18l0.41,0.15l0.56-0.18l0.22-0.33l-0.08-0.24l0.55-0.34l0.82,0.62l0.24-0.24l0.47-0.42l0.48-0.01l0.47-0.26
			l0.24-0.33l0.15-0.24l-0.01-0.33l0.08-0.08l0.24,0.07l0.31-0.4l0.24-0.09v0l0.32,0.15l0.09,0.32l0.01,0.49l-0.7,0.91l0.25,0.31
			l0,0.16l0.8-0.02l0.33,0.4l0.4-0.33l0.95-0.36l-0.08-0.16l0.32,0.15l-0.01-0.23l0.4-0.1l0.07-0.24l0.49,0.06l0.25,0.4l0.96-0.43
			l-0.18-0.56l-0.41-0.06l-0.42-0.55l0.07-0.57l0.39-0.42l0.71-0.18l0-0.16l-0.01-0.23l0.24-0.18l0.16,0l-0.17-0.47l0.32-0.01
			l0.73,0.3l1.13,0.05l0.8-0.02l0.86,1.74l0.16,0l0.33,0.31l0.07,0l0.18,0.15l0.48,0.07l0.32,0.15l0.41,0.15l0.16,0l-0.08,0.32
			l0.82,0.38l0.22-0.41l0-0.07l-0.01-0.49l0.88-0.19l-0.17-0.31l0-0.09l0.08-0.16l0.48,0.15l0-0.16l0.24-0.24l0.47-0.26l0.55-0.66
			l0-0.09l0.15-0.41l0.23-0.01l0.24-0.08l-0.01-0.48l0-0.09l-0.41-0.31l-0.33-0.24l-0.42-0.54l-0.08-0.08l0.07-0.09l-0.16-0.07
			l-0.18-0.56l-0.24-0.07l-0.24,0.01l-0.16-0.08l-0.25-0.24l0.08-0.24l0.31-0.25l0.27-1.78l0.25,0.24l0.58,0.39l0.8-0.02l0.4-0.01
			l0.06-0.33l-0.1-0.8l0.07-0.25l0.33,0.31l0.56-0.18l0.31-0.24l-0.09-0.56l-0.11-0.88l-0.08-0.08l0.31-0.4l0.08-0.09l0.31-0.33
			l0.56-0.18l0.41,0.39l0.15-0.25l0.16,0.07l0.18,0.17l0.01,0.39l0.16,0.08l0.16,0.15l0.32-0.01l0.16,0.07l0.4-0.17l0.48-0.01
			l0.31-0.33l0.48-0.01l0.32-0.24l-0.01-0.24l-0.18-0.47l-0.01-0.24l0.24-0.25l-0.09-0.55l0.31-0.17l0.14-0.65l-0.18-0.63
			l-0.09-0.32l0.47-0.17l-0.01-0.24l0.32-0.25l0.15-0.4l0.23-0.25l0.09,0l-0.1-0.48l0.4-0.17l-0.49-0.32l0.07-0.24l-0.18-0.72
			l0.4-0.33l0.48-0.1l0.32,0.08l0.25,0.07l0.08-0.16l0.16,0.08l0.09,0.39l0.25,0.32l0.08,0.07l0.17-0.01l0.07-0.16l0.16,0.08
			l0.87-0.36l0.16,0.17l0.39-0.58l0.31-0.17l0.24-0.25l0,0.16l0.16,0l0.16,0.08l0.07-0.09l0.4-0.33l0.17,0.24l0.33,0.22l0.09,0.32
			l0.01,0.24l0.17,0.4l-0.05,0.96l0.01,0.4l-0.16,0.41l0.35,0.71l0.55-0.02l0.41,0.06l0.31-0.17l0.25,0.08l0.24-0.18l0.16,0.17
			l0.23-0.09l0.32-0.01l0.09,0.08l0.31-0.18l-0.16-0.15l-0.16-0.07l0.15-0.32l-0.08-0.24l-0.33-0.15l-0.08-0.32l0.07-0.25l0.08-0.16
			l0-0.07l0-0.09l-0.17-0.24l-0.01-0.48l0.31-0.49l0.31-0.41l0.15-0.41l0.01,0.33l0.16,0.15l0.73,0.3l0.08-0.25l0.47-0.25l0.65,0.38
			l0.34,0.24l0.36,1.19l0.48-0.01l0.71-0.34l-0.01-0.48l0.08-0.33l-0.18-0.31l0.32-0.24l0.46-0.74l0.16,0.07l0.16,0.15l0.8,0.14
			l0.33-0.17l-0.25-0.07l0.08-0.16l-0.18-0.24l-0.08-0.32l0.06-0.41l0.24-0.33l0.24-0.17l0.07-0.09l-0.17-0.31l0-0.16l-0.08-0.16
			l0.31-0.25l0.4-0.01l0.22-0.4l0.24-0.09l0.15-0.48l-0.01-0.48l-0.33-0.24l0-0.09l0.07-0.32l0.31-0.17l0.24-0.24l0.31-0.25
			l0.08-0.16l0.39-0.42l-0.01-0.23l-0.01-0.33l-0.26-0.63l-0.4,0.01l-0.08-0.16l-0.16-0.08l-0.25-0.07l-0.24,0.01l-0.07,0.16
			l-0.25-0.31l0.15-0.41l-0.01-0.24l-0.24-0.31l-0.24,0.01l-0.01-0.23l0.4-0.26l0.16,0.16l0.24,0.08l0.09,0.07l0.16,0l0.09,0.08
			l0.23-0.17l0.48-0.01l-0.08-0.33l0.24-0.17l-0.01-0.23l0.31-0.17l-0.74-0.47l0.31-0.17l0.08-0.24l0.08,0.16l0.09,0.16l0.23-0.17
			l0.49-0.17l-0.09-0.24l0.24-0.17L629.77,96.96z"></path>
	</g>
	<g id="girona">
		<path class="st4 fase_0" d="M655.84,105.35l0.07-0.09l-0.07,0L655.84,105.35z M661.45,80.82l-0.07,0.16l0.08,0.08l0.08-0.25L661.45,80.82
			z M655.72,90.48l0.07,0l0-0.07L655.72,90.48z M659.29,78.48l-0.08-0.08l0,0.09L659.29,78.48z M662.46,79.27l-0.09,0l0.09,0.07
			L662.46,79.27z M659.12,94.4l-0.09,0l0,0.09L659.12,94.4z M659.03,94.17l-0.07,0l-0.09,0l-0.07,0.08l0.08,0.08l0.09,0l0.16-0.09
			l0-0.07L659.03,94.17z M653.08,107.69l-0.07,0l0.08,0.08L653.08,107.69z M662.22,79.6l0.24-0.09l-0.09-0.07l-0.33-0.24l0.01,0.24
			l-0.16-0.15l0-0.16l-0.09,0l0,0.07l-0.49-0.22l-0.08,0.08l-0.08-0.07l-0.16,0.16L661,78.82l-0.16,0l-0.09-0.07l-0.08-0.08
			l-0.57-0.22l-0.09,0l0,0.16l-0.07,0.08l0,0.09l-0.24,0.01l-0.07,0l0,0.16l-0.09,0l-0.17-0.31l-0.16,0.08l0-0.16l-0.17-0.23
			l-0.01-0.24L658.88,78l-0.09-0.07l-0.16-0.08l-0.15,0.16l-0.32,0.01L658,78.11l-0.08,0.16l0-0.07l-0.16,0l-0.09,0l-0.15,0.24
			l0.09,0l0.07,0l0,0.09l-0.07,0.16l-0.16,0.08l-0.49-0.38l0.06-0.41l0-0.09l-0.07,0l0,0.09l-0.09,0l-0.09-0.08l0.08-0.08l-0.09,0
			l-0.07,0l-0.09-0.08l-0.07,0l-0.16-0.15l-0.16-0.07l-0.09-0.16l-0.01-0.49l0-0.16l-0.07,0l0,0.09l-0.08,0.08l-0.09,0l-0.01-0.23
			l-0.08-0.24l0.24-0.25l0.15-0.24l-0.09,0l-0.16-0.08l-0.32-0.07l0.15-0.25l0.31-0.25l-0.08-0.16l-0.24,0.01l0.23-0.4l0.07-0.16
			l0.16-0.09l-0.01-0.24l-0.07,0l0.07-0.24l-0.32-0.15l0.08-0.16l0.23-0.01l0.24-0.18l-0.09-0.07l-0.39,0.01l-0.24,0.08l-0.32,0.01
			l-0.24,0.01l-0.73-0.05l-0.24-0.08l-0.4,0.1l-0.24,0.33l0,0.16l-0.48,0.1l-0.16,0l-0.42-0.48l-0.16-0.07l-0.17-0.24l-0.25-0.23
			l-0.49-0.48l-0.17-0.47l-0.26-0.63l-0.32,0.01l-0.39,0.17l-0.32,0.24l-0.49-0.22l-0.49-0.23l-0.39,0.42l-0.47,0.17l-0.58-0.62
			l-0.25-0.24l-0.71,0.59l-0.22,0.65l-0.31,0.17l-0.65-0.14l-0.73-0.14l-0.49-0.31l-0.23,0.25l-0.07,0.48l-0.89-0.22l-0.24,0.33
			l-0.06,0.49l-0.24,0.33l-0.32-0.08l-0.39,0.5l-0.08-0.08l-0.3,0.73l-0.17,0.01l-0.39,0.09l-0.24,0.25l-0.24,0.09l-0.24-0.24
			l-0.33-0.4L639,74.5l-0.65,0.09l-0.62,0.51l-0.63,0.41l-0.4,0.01l-0.4,0.26l0.02,0.8l-0.23,0.09l-0.49-0.15l0.26,0.39l0.1,0.4
			l0,0.16l0.17,0.4l0.32,0.15l0.09,0.56l0.01,0.4l-0.39,0.09l-0.33-0.06l-0.32-0.08l-0.39,0.01l-0.25-0.15l-0.88,0.03l-0.32-0.15
			l-0.57-0.23l0-0.16l-0.48-0.06l-0.8,0.26l-0.07,0.32l-0.15,0.33l-0.32,0.49l-0.71,0.02l-0.57-0.3l-0.73-0.14l-0.32,0.08
			l-0.16-0.07l-0.26-0.4l-0.26-0.47l-0.4-0.23l-0.17-0.4l-0.49-0.06l-0.33-0.4l-0.02-0.72l-1.21-0.04l-0.57-0.31l-0.48-0.06
			l-0.73-0.14l-0.49-0.31l-0.09-0.33l-0.08-0.07l-0.56,0.02L622.21,75l-0.33-0.31l-0.55,0.34l-0.57-0.3l-0.4-0.32l-0.49-0.15
			l-0.32,0.01l-0.23,0.5l-0.39,0.09l-0.8,0.27l-0.31,0.17l-0.31,0.17l-0.8-0.06l-0.5-0.38l-0.56,0.18l-0.31,0.17l-0.31,0.17
			l-0.4,0.26l-0.22,0.65l-0.24,0.24l0.1,0.56l-0.48,0.26l-0.63,0.34l-0.14,0.57l-0.08,0.08l-0.33-0.22l-0.31,0.24l-0.4,0.18
			l-0.64,0.02l-0.56,0.09l-0.47,0.49l-0.47,0.34l-0.33,0.01l-0.16-0.17l0.01,0.49l-0.47,0.25l-0.24-0.07l-0.33,0.01l-0.23,0.24
			l-0.13,0.89l1.06,0.62l0.39-0.01l0.4-0.33l0.4-0.1l0.48-0.01l0.57,0.23l0.33,0.22l0.41,0.23l0.32,0.15l0.25,0.08l-0.15,0.32
			l-0.55,0.34l-0.08,0.16l-0.31,0.33L611.13,84l-0.31,0.41l0,0.09l-0.06,0.57l-0.08,0.24l-0.31,0.25l0.25,0.31l0.25,0.56l0.16-0.16
			l0.4-0.17l0.41,0.06l-0.08,0.25l-0.07,0.24l0.1,0.81l0.25,0.07l0.24,0.08l-0.15,0.16l0.1,0.48l-0.15,0.32l-0.56,0.02l-0.33-0.22
			l-0.24,0.24l-0.06,0.33l-0.24,0.24l-0.64,0.1l-0.09-0.16l-0.33-0.31l0.3-0.57l-0.08-0.16l-0.18-0.33l0.4-0.17l-0.8,0.02
			l-0.57-0.14l-0.32-0.07l0.03,1.04l0.56-0.1l0.48-0.01l-0.08,0.25l0.09,0.24l0.43,0.79l0.08,0.16l0.01,0.24l0.88-0.19l0.33,0.22
			l0.25,0.31l0.32-0.08l0.16,0.15l0.25,0.24l0.08,0.07l0.08-0.08l0.31-0.18l0-0.16l0.57,0.07l0.31-0.17l0.25,0.23l0.01,0.49
			l0.08,0.07l0,0.16l0.09,0.16l0.58,0.39l0.32,0.15l0.46-0.49l0.31-0.49l0.64-0.26l0.96,0.06l0.09,0.16l0.01,0.32l0.23-0.01
			l0.32-0.01l0.08-0.25l0.16,0.15l0.16-0.08l-0.08-0.32l0.07-0.25l0.08-0.09l1.04-0.19l0.17,0.33l0.56-0.1l0.23-0.17l0.24-0.17
			l0.16,0.15l0.32,0.07l0.24-0.01l0.31-0.49l-0.09-0.07l-0.15,0.16l-0.24-0.08l0.07-0.16l0.15-0.16l-0.23,0.01l-0.09-0.16
			l-0.41-0.23l-0.24-0.07l0.56-0.02l0.32,0.07l0.23-0.08l0.33,0.15l0.24,0.23l0.4-0.25l0.32-0.01l0.16-0.09l0.31-0.17l0.33-0.08
			l0.55-0.18l0.09,0l0.4-0.26l0.01,0.24l0.24,0.07l0.32,0.15l0,0.16l-0.4-0.06l-0.08,0.16l-0.63,0.25l-0.15,0.33l-0.15,0.24
			l0.09,0.49l0.58,0.38l0.33,0.24l0.32,0.15l-0.22,0.33l-0.08,0.09l0.1,0.39l0.17,0.33l0.72-0.02l0.08,0.07l-0.22,0.57l0.08,0.08
			l0,0.07l0.33,0.47l0.17,0.24l0,0.16l0.42,0.39l0.25,0.07l0.23-0.08l0.71-0.51l0.08-0.24l0.25,0.07l0.4,0.15l0.16,0.17l0.09,0.32
			l0.24-0.17l0.48-0.01l0.24,0.31l0,0.16l-0.16,0.16l-0.39,0.26l0.17,0.31l0.75,0.7l0.08,0.07l-0.15,0.32l-0.15,0.18l-0.16,0
			l0.01,0.48l0.41,0.15l-0.24,0.17l0.09,0.24l-0.49,0.17l-0.23,0.17l-0.09-0.16l-0.08-0.16l-0.08,0.24l-0.31,0.17l0.74,0.47
			l-0.31,0.17l0.01,0.23l-0.24,0.17l0.08,0.33l-0.48,0.01l-0.23,0.17l-0.09-0.08l-0.16,0l-0.09-0.07l-0.24-0.08l-0.16-0.15
			l-0.4,0.26l0.01,0.23l0.24-0.01l0.24,0.31l0.01,0.24l-0.15,0.41l0.25,0.31l0.07-0.16l0.24-0.01l0.25,0.07l0.16,0.08l0.08,0.16
			l0.4-0.01l0.26,0.63l0.01,0.33l0.01,0.23l-0.39,0.42l-0.08,0.16l-0.31,0.25l-0.24,0.24l-0.31,0.17l-0.07,0.32l0,0.09l0.33,0.24
			l0.01,0.48l-0.15,0.48l-0.24,0.09l-0.22,0.4l-0.4,0.01l-0.31,0.25l0.08,0.16l0,0.16l0.17,0.31l-0.07,0.09l-0.24,0.17l-0.24,0.33
			l-0.06,0.41l0.08,0.32l0.18,0.24l-0.08,0.16l0.25,0.07l0.48-0.01l0.49,0.39l0.1,0.48l0.49,0.23l0.63-0.26l-0.14,0.57l0.58,0.39
			l0.49,0.31l0.09,0l0.8,0.14l0.09,0.32l-0.16,0.16l0.17,0.24l-0.08,0.09l0.41,0.31l0.25,0.31l-0.08,0.25l0.17,0.39l0.57,0.23
			l0.63-0.34l0.33-0.09l0.16,0.08l0.88-0.19l-0.07,0.4l0.17,0.24l-0.22,0.49l-0.23,0.41l0.17,0.47l0.17,0.31l0.16,0.08l0.31,0.39
			l-0.62,0.74l-1.67,0.2c0,0,0.89,3.46,0.98,3.99l2.11-0.86l0.23-0.17l0.32-0.08l1.36-0.61l1.26-0.52l0.4-0.26l0.24-0.08l0.23-0.25
			l0.09,0l0.71-0.43l0.31-0.17l0.8-0.1l0.17-0.18l0,0l0.22-0.65l0.31-0.49l0.15-0.16l0.17-0.09l0,0.09l-0.08,0.08l0.16-0.08
			l0.08-0.09l0-0.07l0.16-0.09l0-0.07l0.23-0.17l0.08-0.09l0-0.09l0.07-0.08l0.17-0.09l0.23-0.08l0-0.09l0.24-0.08l0.07-0.09
			l0.16,0.08l0.08-0.09l0.08-0.08l0.16-0.09l0.31-0.17l0.16,0l0.25,0.08l0.07-0.09l0.16,0l0.16-0.09l0.09,0l0.07,0l0-0.07l0.09,0
			l0.16,0l0.09,0.07l0.07,0l0.08-0.08l0.07,0l0.16-0.09l0.16-0.08l-0.08-0.08l0.16-0.08l0.07,0l0.24-0.01l0.09,0l0.31-0.17l0.07,0
			l0.09,0l0.07-0.09l0.24-0.08l0.09,0l0-0.09l0-0.16l0.07-0.09l0.08-0.08l0.15-0.16l0.16,0l0-0.09l0.07-0.08l-0.08-0.08l0.07,0
			l0.09,0l0.07-0.08l0.08-0.09l0-0.16l0.09,0l0.07,0l0.08-0.09l-0.09,0l0-0.16l0.31-0.4l0.15-0.25l0.16-0.09l0.16-0.24l0.23-0.09
			l0.32-0.08l0.08-0.09l0.07,0l0.08-0.08l0.08-0.09l0.07-0.08l0.24-0.18l0.39-0.17l0.24-0.01l0.09,0l0.08,0.08l0.09,0.07l0-0.07
			l0.07,0l-0.08-0.08l-0.09-0.32l0.09,0l0.16,0l0.07,0l0.09,0.08l0.07,0l0.16-0.09l0-0.07l0.09,0l0.08-0.09l0-0.07l-0.09-0.17
			l0-0.07l0.08-0.09l0.16,0l0.08,0.08l0.09,0.07l0-0.07l0.07-0.16l0-0.09l0.16-0.16l-0.08-0.07l0.16-0.16l-0.09-0.08l0.09,0l0-0.09
			l0.15-0.48l0.08-0.32l0.15-0.25l0.23-0.17l0.39-0.41l0.09,0l0.07-0.25l0.24-0.33l0.31-0.17l0.4-0.1l0.16,0l0.16,0.08l0.08,0.07
			l0,0.09l-0.07,0.08l-0.24,0.01l0,0.09l0.32-0.09l0.09,0l0.07,0l0-0.07l0-0.16l0.09,0l0.08-0.09l0,0.09l0.07,0l0-0.09l0.24-0.08
			l0.07-0.09l-0.08-0.16l-0.01-0.24l0.07,0l0.09,0l0.07-0.08l0-0.09l0.09,0l0.07,0l0-0.07l0.16,0l0,0.16l0.08-0.09l0.16,0l0.08-0.08
			l0.23-0.09l0.08-0.08l0.07-0.09l0.09,0l-0.09-0.16l0.08-0.25l0.16-0.08l-0.16-0.15l0.16-0.16l0-0.16l0.08-0.09l0.23-0.09
			l0.08-0.16l0-0.07l0.07,0l0.08-0.09l0.16,0.15l0.07-0.08l0.07-0.41l0.16-0.08l-0.01-0.33l0.16-0.08l-0.08-0.16l0-0.16l-0.09-0.16
			l0.24-0.01l0.07-0.16l0.16-0.09l-0.16-0.15l0.08-0.16l-0.09-0.08l0.16-0.08l-0.08-0.08l0.07-0.08l-0.08-0.08l-0.09,0l0-0.07
			l0.08-0.17l0-0.16l0.31-0.24l0.24-0.17l-0.09-0.33l0.08-0.08l0.07-0.16l-0.07,0l0,0.07l-0.09-0.07l0-0.09l-0.08-0.07l0.07-0.09
			l0-0.07l-0.08-0.16l-0.18-0.17l-0.07,0l-0.16,0l-0.24,0.01l-0.07,0l-0.01-0.23l-0.09-0.24l-0.26-0.4l-0.18-0.71l-0.08-0.24
			l-0.1-0.64l-0.01-0.49l0.07-0.4l-0.01-0.4l0.07-0.25l0.08-0.16l0-0.07l0.16,0.07l-0.09-0.07l-0.08-0.08l0.07,0l0.09,0l0.09,0
			l0.07,0l0,0.09l0.08-0.09l0-0.07l0.15-0.25l-0.09-0.08l-0.07,0l0-0.07l-0.01-0.24l0-0.07l-0.09-0.08l-0.08-0.07l-0.09-0.08
			l-0.09-0.08l0.09,0l-0.25-0.15l0-0.07l0.08-0.09l0-0.07l-0.09,0l0-0.09l-0.16,0l-0.01-0.23l-0.08-0.08l-0.09-0.16l-0.17-0.31
			l-0.08-0.08l0-0.07l-0.09-0.16l0-0.09l-0.16,0.09l-0.32,0.08l-0.09-0.16l0.08-0.08l0.16-0.09l0.07-0.09l-0.08-0.07l-0.09,0
			l-0.08-0.16l-0.08,0.08l-0.16,0l-0.16-0.07l-0.33-0.24l-0.09-0.07l0,0.07l-0.07,0l-0.09,0l0.09,0.08l-0.08,0.08l-0.07,0l-0.16,0
			l-0.16,0l0.07-0.32l-0.08-0.24l-0.09,0l-0.09,0l-0.32,0.01l-0.08-0.32l0-0.16l-0.09-0.07l-0.17-0.49l-0.2-1.52l-0.13-1.52
			l0.07-0.49l0.23-0.65l0.22-0.65l0.39-0.58l0.15-0.32l0.47-0.42l0.24-0.17l0.4-0.1l0.16,0l0.16,0.17l0.16,0.07l0.08,0.08l0,0.07
			l-0.07,0l0,0.09l0.07,0l0,0.16l-0.07,0l-0.09,0l0,0.07l0.16,0l0,0.09l0.09,0.07l0,0.09l0.32-0.01l0.16-0.09l0.09,0.24l0.24,0.08
			l0.25,0.07l-0.07,0.09l0.32,0.15l0.16,0.15l0.07-0.16l0.16-0.16l0-0.09l0.17-0.01l-0.09-0.16l0.08-0.08l-0.09-0.24l0.08-0.16
			l0.33,0.22l0.07-0.16l0.16,0.08l0.17,0.31l0.34,0.24l0.24,0.07l0.08-0.08l0.07-0.25l-0.08-0.07l-0.16,0l-0.16,0l-0.09-0.08
			l0.08-0.09l-0.01-0.32L660,83.2l0.24-0.01l0.23-0.09l0.33-0.01l0.16-0.08l0-0.16l0.22-0.5l-0.08-0.23l-0.09-0.08l-0.08-0.07
			l-0.17-0.33l0.07-0.08l0-0.09l-0.08-0.07l0-0.09l0.16-0.08l0.16,0.07l0.24,0.15l0.09,0.08l0-0.09l0.16,0l0-0.16l-0.09-0.23
			l-0.16,0l-0.16-0.15l0.08-0.09l0.07-0.16l-0.08-0.08l0.07-0.16l-0.16,0l0.08-0.16l0.16-0.08l0.08,0.07l0.17,0.08l0-0.16l0.15-0.16
			l0.16-0.09l0-0.09l0.23-0.08l-0.08-0.16l0.24,0.15L662.22,79.6z M652.76,107.61l0-0.07l0.09,0.07L652.76,107.61z M651.84,109.09
			l0.16-0.08l-0.24,0.08L651.84,109.09z"></path>
	</g>
	<g id="catalunya_provincias">
		<g id="tarragona_p">
			<path class="st5" d="M596.37,140.24l-0.16-0.15l0.06-0.48l-0.17-0.31l-0.31,0.17l-0.08,0.16l-0.07,0.25l-0.32,0.08l-0.25-0.23
				l-0.25-0.24l-0.33-0.24l-0.32,0.09l-0.41-0.07l-0.34-0.71l-0.01-0.32l-0.24-0.31l0.96,0.13l0.16,0.15l0,0.09l0.16,0l0.32,0.15
				l0.32-0.25l0.16,0l0.06-0.4l0-0.09l0.24-0.33l-0.25-0.31l0.24-0.33l0.06-0.41l-0.07,0l-0.01-0.24l0.16,0l0.16-0.16l-0.16-0.07
				l-0.16,0.16l-0.09-0.08l-0.39,0.26l-0.15,0.4l-0.41-0.15l-0.49-0.3l-0.09,0l-0.01-0.23l0.23-0.57l0-0.09l-0.66-0.3l-0.16,0
				l-0.01-0.32l-0.25-0.4l-0.1-0.48l-0.25-0.24l0.47-0.58l0.32-0.08l-0.1-0.4l-0.16-0.07l-0.07,0.08l-0.01-0.23l-0.32-0.16
				l-0.24,0.09l-0.25-0.08l-0.17-0.39l-0.42-0.48l-0.72-0.14l-0.41-0.31l-1.43,0.77l0.08-0.25l0.15-0.32l0.15-0.4l0.23-0.18
				l-0.18-0.63l-0.16-0.15l0.08-0.16l0.07-0.16l-0.32-0.15l0.07-0.41l0.24,0.08l0.4-0.17l0-0.09l-0.8,0.02l0-0.16l-0.25-0.31
				l0.7-0.9l-0.01-0.49l-0.09-0.32l-0.32-0.15h0l-0.09-0.4l0,0l0,0l-0.33,0.01l-0.24-0.15l-0.32,0.01l0-0.07l-0.5-0.39l-0.73-0.14
				l-0.08-0.16l-0.08,0.08l-0.24-0.15l-0.32-0.15l-0.09-0.23l-0.16-0.08l-0.01-0.32l0.07-0.32l0.23-0.49l0.89,0.13l0.39-0.42
				l0.48,0.15v0l0,0l-0.01-0.32l-0.01-0.32l-0.04-0.12l-0.04-0.12l-0.27,0.06l-0.22,0.04l-0.17-0.24l0.95-0.75l-1.31-0.68l-0.4,0.1
				l-0.15,0.24l-0.64,0.02l0.09,0.49l0.33,0.15l0.01,0.32l-0.49,0.09l-0.32-0.07l-0.17-0.24l-0.49-0.22l-2.18-0.11l-0.32-0.15
				l-0.47,0.34l-0.9-0.38l-0.48,0.25l-0.17-0.23l-0.23,0.08l-0.24,0.17l-0.48-0.15h0l0,0l-0.49,0.1l-0.22,0.49l-0.14,0.65
				l-0.19,0.07l-0.53,0.19l-0.88,0.19l-0.08,0.16l0.33,0.08l0,0l0,0.07l-0.08,0.09l-0.32,0.01l0.08,0.08l0.16,0l-0.06,0.64
				l-0.08,0.08l-0.16,0l0.01,0.24l0.1,0.4l0.33,0.22l0.18,0.72l-0.01,0l-0.31,0.01l-0.71,0.34l-1.04,0.28l0,0.16l-0.4,0.17
				l-0.47,0.17l-0.07-0.29l-0.02-0.1l-0.24-0.08l-0.25-0.07l-0.2-0.08l-0.2-0.08l-0.49-0.38l-0.32-0.08l-0.41-0.31l-0.58-0.39
				l-0.07,0.16l0.5,0.63l-0.16,0.16l-0.31,0.24l0.25,0.32l0.49,0.38l0.48,0.15l0.33,0.24l-0.56,0.34l-0.15,0.49l-0.16,0.08
				l-0.32,0.09l-0.14,0.56l-0.15,0.49l-0.32,0.01l0.01,0.23l-0.15,0.41l-0.16,0l-0.32,0.33l-0.56-0.06l0.08,0.23l-0.4,0.33v0l0,0
				l0.01,0.17l-0.07,0.08l-0.41-0.15L569,131.5l-0.15,0.4l-0.07,0.25l-0.4,0.33l0.01,0.24l0,0.16l0.16,0l-0.23,0.25l-0.33,0.17
				l-0.23,0.01l-0.56,0.09l-0.8,0.02l-0.07,0.02l-0.25,0.07l-0.23,0.08l-0.56,0.26l-0.88,0.19l-0.67-0.94l-0.1-0.4l-0.56,0.18
				l-0.07,0.25l-0.15,0.24l-0.21,1.21l-0.32,0.09l-0.57-0.07l-0.16,0l-0.41-0.15l-0.57-0.14l-0.16-0.07l-0.32,0.08l-0.15,0.25
				l-0.15,0.32l0.01,0.4l-0.11,0.07l0.11-0.07l0-0.16l-0.88,0.1l-0.47,0.17l-0.33-0.06l-0.23,0.08l-0.32-0.06l-0.24,0.08l-0.24,0.17
				l-0.16,0l-0.4-0.06l-0.48,0.41l-0.48,0.1l-0.47,0.17l-0.31,0.33l-0.4,0.01l-0.41-0.06l-0.48-0.15l-0.5-0.63l-0.17-0.47l0-0.16
				l-0.24,0.01l-0.81-0.45l-0.39,0.41l-0.07,0.41l-0.46,0.65l-0.23,0.57l-0.89-0.05l-0.17-0.4l-0.25-0.4l-0.17-0.23l-0.35-0.55
				l-0.41-0.47l-0.34-0.39l-0.16,0.16l-0.54,0.34l-0.39,0.49l0.04,1.21l-0.06,0.96l-0.39,0.17l-1.54-0.19l-0.07,0l-0.38,0.89
				l0.01,0.48l-0.15,0.41l0.17,0.39l-0.48-0.06l-0.08,0.32l0.01,0.23l-0.16,0.09l-0.16,0l-0.32-0.08l-0.32-0.15l-0.09,0l-0.07,0.25
				l-0.08,0.09l-0.16,0.08l-0.32-0.15l0,0.09l-0.16,0.16l-0.07,0.48l-0.37,1.3l-0.07,0.32l-0.08,0.24l-0.05,0.73l-0.08,0.16
				l-0.16-0.07l-0.23,0.08l0,0.09l-0.17,0l-0.7,0.5l-0.73-0.05l0,0.07l0.08,0.16l0,0.16l-0.16,0.16l0.01,0.24l-0.07,0.16l-0.17,0.16
				l-0.23,0.01l-0.16-0.07l-0.66-0.3l-0.23,0.01l-0.09,0l0.16,0.07l0.09,0.08l0,0.07l-0.08,0.09l-0.48,0.09l-0.16,0.09l0,0.16
				l0.33,0.31l-0.14,0.57l-0.24,0.08l0.03,1.05l0.09,0.16l0.16,0l0.32-0.09l0.25,0.08l0.08,0.16l0,0.07l-0.16,0.09l-0.07,0l0,0.07
				l0.16,0.08l0.01,0.23l-0.15,0.25l0.17,0.31l0.24,0.15l0,0.16l0,0.09l-0.07,0.09l-0.16,0.08l-0.16,0.16l0,0.09l0,0.16l0.16,0
				l0.16-0.16l0.16-0.09l0.09,0l0.08,0.08l0,0.07l-0.07,0.16l-0.08,0.16l0,0.09l0.09,0.08l0.32,0.06l0.07,0l0.08-0.08l-0.01-0.24
				l0-0.09l0.07,0l0.09,0.08l0.08,0.24l0.25,0.23l0.41,0.31l0.16,0l0.16,0.08l0.25,0.15l0,0.09l-0.16,0.08l-0.17-0.07l-0.23,0.01
				l-0.09,0l-0.16,0.08l-0.07,0.09l0.08,0.23l0.16,0.08l0.24-0.01l0.08,0.16l0.09,0.16l-0.16,0.09l-0.08,0.16l0,0.16l0.09,0.07
				l0.08,0.16l0.25,0.24l0.09,0.32l0.01,0.24l0.01,0.23l0.23-0.01l0.16,0.08l0,0.07l-0.22,0.34l-0.09,0l0.01,0.32l0,0.07l-0.07,0.16
				l-0.08,0.17l-0.16,0.08l-0.08,0.09l-0.15,0.24l0.1,0.4l-0.31,0.41l-0.16,0l-0.32,0.33l0.02,0.56l-0.16,0l-0.07,0.08l0,0.09
				l0.17,0.23l-0.16,0.09l0.01,0.4l-0.16,0.16l0.01,0.23l-0.24,0.09l-0.07,0.08l0.08,0.08l0.25,0.15l0.16,0.08l0.17,0.23l0.01,0.24
				l-0.15,0.41l0.08,0.16l0.01,0.23l0.09,0.16l0.27,0.88l0.33,0.39l0.24,0.24l0.09,0.16l-0.08,0.08l-0.07,0l-0.08,0.09l-0.23,0.17
				l-0.31,0.81l-0.39,0.1l-0.24,0.33l-0.4,0.26l0.01,0.4l0,0.16l-0.15,0.48l-0.16,0.08l-0.16,0l-0.48-0.06l-0.46,0.58l-0.09-0.08
				l-0.07,0l-0.08,0.16l-0.15,0.25l-0.09,0l-0.08-0.08l-0.09-0.16l-0.07,0.09l-0.25-0.08l-0.06,0.41l0,0.16l0,0.16l-0.17-0.07
				l-0.17-0.24l-0.16-0.15l-0.07,0l0,0.07l0.17,0.24l0,0.16l0.17,0.47l0.65,0.31l0.24-0.25l0.17,0.4l0.4-0.1l0.07-0.16l0.24-0.09
				l0.16-0.08l0.4-0.01l-0.07,0.48l0.44,1.19l0.24,0.08l0.33-0.17l0.31-0.17l0.16,0l0.25,0.15l0.16,0.07l0.25,0.48l0.22,0.16
				l-0.06,0.12l-0.31,0.1l-0.12,0.31l-0.33,0.08l-0.23,0.54l0.5,0.52l0.01,0.48l0,0.09l-0.08,0.08l0.09,0.33l0,0.07l0.16,0.15
				l0.07,0l0.09,0.24l0.08,0.24l0.41,0.22l0.09,0l0.16-0.08l0.07,0l0.09,0.16l0.49,0.3l0.73,0.3l0.65,0.23l1.29,0.04l0.4-0.01
				l0.16,0.08l0.4,0.15l0.18,0.23l0.08,0.24l0.58,0.39l0.42,0.39l-0.42,0.24h0.08l0.04-0.06l0.15,0.29l0.4-0.08l-0.07,0.32l0.09,0.4
				l0.02,0.56l0.41,0.31l0.48,0.15l0.74,0.38l0.57,0.23l0.41,0.15l0.4,0.06l0.16,0.08l0.08-0.32l0.23-0.57l0-0.09l0-0.07l0.38-0.58
				l0.08-0.16l0.23-0.66l0.31-0.24l0.07,0l0.16,0.15l-0.17-0.24l0.55-0.58l0.85-1.07l0.31-0.4l0.24-0.09l0.07-0.08l-0.07,0l-0.09,0
				l0-0.09l0.16-0.08l0.09,0l-0.08,0.08l0.09,0l0.08,0.08l0.25-0.01l0-0.09l-0.07,0l-0.09-0.07l0,0.07l-0.25-0.15l0.08-0.09
				l0.08,0.08l0-0.09l0.09,0l0.08-0.08l0.07,0h0l0.48,0.06l1.29-0.11l0.24-0.09l0.23-0.08l0.4-0.1l0.31-0.24l0.49-0.17l1.04-0.12
				l0.64-0.02l0.07,0l0.41,0.07l0.01,0.2c0.07,0.01,0.14,0.01,0.2,0.03l0.05-0.07l0.23-0.25l0.32-0.33l0.71-0.43l0.55-0.26
				l1.59-0.76l0.55-0.35l0.4-0.33l0.38-0.33l0.31-0.57l0.15-0.48l0.15-0.32l0.07-0.16l-0.17-0.4l-0.17-0.47l-0.07,0.08l0,0.16
				l-0.24,0.01l-0.17-0.23l-0.16,0l-0.08,0.08l-0.16,0.09l-0.23,0.08l-0.16,0l-0.25,0.01l-0.16-0.07l-0.25-0.31l-0.33-0.32
				l-1.09-1.4l-0.43-0.71l-0.16-0.15l-0.25-0.15l-0.16-0.15l-0.32-0.08l-0.48,0.01l-0.24,0.01l-0.23,0.01l-0.16,0.09l-0.17,0.16
				l0.25,0.15l0.24-0.01l0.16,0l0.16,0.07l0.25,0.08l0.24,0.07l0.32,0.15l0.25,0.08l0.41,0.23l-0.15,0.16l-0.09,0l-0.16,0.16
				l-0.16,0.08l-0.08,0.09l-0.23,0.08l-0.57-0.06l-0.57-0.23l-0.25-0.15l-0.41-0.47l-0.09-0.08l-0.24-0.23l-0.25-0.31l-0.26-0.56
				l0.08-0.16l0.16-0.16l0.08-0.09l0.07-0.09l0.16-0.08l0.24-0.09l0.07,0l0.08-0.16l0.16-0.08l0.24-0.09l0.07-0.16l0.08-0.08l0.16,0
				l0,0.07l0.07-0.08l0.08-0.17l-0.09-0.07l0-0.09l0.15-0.4l0-0.09l0.09,0l0.07-0.08l-0.01-0.17l0.17-0.16l0.07,0l0.09,0l0.16,0
				l0.07,0l0.08-0.16l0-0.07l0.16-0.16l0.07-0.16l0.08-0.09l0.4-0.09l0.23-0.18l0-0.07l0.09,0l0.07-0.09l0.08-0.08l0.15-0.25
				l-0.08-0.07l0.07,0l0-0.08l0.08-0.08l0.07-0.09l0.08-0.16l0.07-0.09l0,0.09l0,0.07l0.08-0.08l0-0.09l0-0.07l0.07-0.09l0-0.07
				l0.08-0.09l0.07,0l0.08-0.08l0.16-0.09l0.08-0.25l0.16-0.08l0.07-0.16l0-0.09l0.09,0l0-0.07l0-0.09l0-0.07l0.07-0.09l0.08-0.08
				l0.07,0l-0.01-0.17l0.09,0l0-0.07l0.09,0l0.07-0.25l0.16-0.08l0.24-0.25l0.07-0.08l0.08-0.09l0-0.07l0-0.09l0.15-0.16h0
				l0.15-0.32l0.15-0.32l0.24-0.18l0.16,0l0.16-0.08l0.15-0.25l0-0.07l0.08-0.16l0.16-0.09l0.38-0.58l0.16-0.16l0.17-0.08l0.16-0.16
				l0.38-0.42l0.25-0.01l0.24-0.09l0.16-0.08l0.22-0.66l0.08-0.08l0.07-0.09l0.7-0.82l0.31-0.33l0.17-0.09l0.31-0.33l0.39-0.26
				l0.24-0.17l0.16,0.08l0.08-0.09l0.16,0l0.4-0.33l0.23-0.08l0.09,0l0.07-0.09l0.09,0l0.07-0.09l0.09,0l0.07-0.08l0-0.09l0.17,0.08
				l0-0.09l0.07-0.08l0.16,0l0.08-0.09l0.07,0l0-0.07l0.24-0.17l0.07,0l0.09,0.07l0.16,0l0.16-0.08l0.09,0l0,0.16l-0.08,0.08
				l0.16-0.08l0-0.09l0.08-0.08l0.23-0.01l0.25,0.07l0.24-0.01l0.31-0.17l0.56-0.09l0.47-0.18l0.24-0.08l0.48-0.01l0.24,0.07
				l0.09,0.17l0.16,0.07l0,0.16l0.24-0.01l0.16,0.08l0,0.16l0.07,0l0,0.07l0.09,0.16l-0.09,0l0.09,0.08l0.24-0.09l0,0.09l0.16,0
				l0.22-0.33l0.09,0l0.07-0.16l-0.08-0.08l0-0.16l0.22-0.49l0.23-0.57l0.31-0.33l0.31-0.25l0.09,0.16l0.16,0.15l0.08-0.16
				l-0.09-0.16l0.4-0.33l0.07,0l-0.07,0.25l0.07-0.09l0,0.16l0.09,0l0.07-0.08l-0.08-0.24l0.07,0l0.09,0.24l0.31-0.17l-0.08-0.08
				l0-0.07l0.16,0.07l0.07,0l-0.16-0.07l0.08-0.09l0.16-0.09l0.16,0.08l-0.08,0.16l0-0.07l-0.07,0.08l0.08,0.08l-0.56,0.25
				l-0.07,0.25l-0.09-0.08l-0.15,0.32l0.09,0.08l-0.24,0.25l0,0.07l0.24-0.24l0.22-0.41l0.56-0.34l0.08-0.16l0.07-0.09l0.08-0.16
				l0.07,0l0-0.09l0.08-0.08l0.08-0.09l0.16-0.08l0.16,0.07l0,0.09l0.07-0.09l0.16,0l0-0.07l0.08-0.09l0-0.07l0.31-0.17l0.47-0.26
				l0.33-0.09l0.64-0.09l0.32,0.07l0.48-0.01l0.34,0.16l0.07-0.17l0.25,0.08l0.39-0.17l0.56-0.18l0.32-0.09l0.09,0.08l0.16,0.07
				l0.24-0.01l0.07-0.08l0.16,0l0.08-0.16l0.07,0l-0.08-0.08l1.5-0.93l0.8-0.5l1.03-0.35l0.47-0.17l1.12-0.28l0.07,0l0.16-0.16
				l0.4-0.1l1.2-0.2l0.4-0.09l1.2-0.04l0.31-0.17l0.09,0.07l0.16-0.08l0.07,0l0.16-0.09l0,0.09l0.56-0.1l0.16-0.08l-0.17-0.4
				l-0.01-0.48l0.24-0.01l0.17,0.23l0.07-0.08L596.37,140.24z"></path>
		</g>
		<g id="lleida_p">
			<path class="st5" d="M610.07,78.91l-0.33-0.24l-0.16,0l-0.5-0.3l-0.25-0.4l-0.25-0.15l-0.33-0.31l-0.1-0.48l-0.08-0.16l-0.1-0.56
				l-0.17-0.4l0.06-0.8l-0.66-0.38l-0.01-0.48l-0.09-0.65l-0.32-0.06l-0.4,0.25l-0.48,0.01l-0.09,0l-0.32-0.06l-0.32,0.08
				l-0.16-0.07l-0.17-0.47l-0.74-0.38l-0.16,0.16l-0.41-0.23l-0.17-0.31l-0.5-0.55l-0.41-0.06l-0.24,0.08l-0.58-0.38l-0.72,0.26
				l-0.64-0.14l-0.32,0.09l-0.25-0.24l-0.32-0.15l-0.64,0.1l-0.26-0.56l-0.24,0.08l-0.54,0.51l-0.65-0.15l-0.39,0.01l-0.34-0.47
				l-0.22,0.49l-0.06,0.73l-0.06,0.41l-0.15,0.41L595.58,73l-0.32-0.08l-0.24,0.01l-0.23,0.25l-0.41-0.15l-0.4,0.09l-0.48-0.15
				l-0.31,0.17l-0.38,0.73l0,0l-0.32-0.15l-0.33-0.24l-0.24,0.01l-0.08,0.25l0.09,0.32l-0.24,0.33l-0.06,0.41l-0.72,0.11l-0.63,0.18
				l-0.82-0.22l-0.64-0.06l-0.73-0.06l-0.65-0.14l-0.17-0.31l0.16,0l-0.17-0.31l0.14-0.57l-0.24-0.24l0.06-0.57l-0.49-0.38
				l-0.41-0.07l0.23-0.41l0.79-0.26l0.86-0.83l-0.51-0.87l-0.09-0.55l-0.41-0.32l-0.87,0.43l-0.24,0.09l0.23-0.57l0.14-0.57
				l0.71-0.5l0.07-0.25l-0.16-0.15l-0.49-0.39l-0.01-0.39l0.31-0.17l0.3-0.66l-0.16-0.07l-0.24-0.32l-0.17-0.23l-0.01-0.24
				l-0.25-0.23l-0.27-0.8l-0.09-0.4L585.95,63l-0.32-0.15l-0.26-0.56l-0.48-0.06l0.06-0.41l-0.01-0.48l-0.33-0.39l-0.64-0.14
				l-0.41-0.31l-0.01-0.4l0.15-0.32l-0.01-0.32l-0.64,0.09l-0.25-0.23l-0.24-0.08l-0.48,0.26l-0.8,0.02l-0.56,0.25l-0.16-0.07
				l-0.41-0.07l-0.23,0.17l-0.33,0.01l-0.16-0.15l-0.33-0.38l-0.48-0.16l-0.39,0.34l-0.48,0.01l-0.63,0.18l-0.48,0.25l-0.31,0.25
				l-0.41-0.15l-0.33-0.24l-0.25-0.39l-0.5-0.23l-0.01-0.4l-0.08-0.23l-0.02-0.65l-0.33-0.47l-0.41-0.23l-0.17-0.31l-0.74-0.38
				l-0.25-0.23l-0.17-0.31l-0.16,0.08l-1.2,0.28l-0.09-0.16l-0.24-0.15l-0.4,0.09l-0.89-0.29l-0.63,0.26l-0.25-0.08l-0.17-0.24
				l-0.09-0.23l-0.24-0.15l-0.16-0.15l-0.25-0.15l-0.56,0.25l-0.38,0.33l-0.08,0.33l-0.16-0.08l-0.49-0.23l-0.25-0.15l-0.25-0.23
				l-0.41-0.39l-0.66-0.38l-0.32,0.01l-0.5-0.62l-0.73-0.14l-0.32,0.01l-1.06-0.62l-0.4,0.1l-0.48,0.09l-0.41-0.06l-0.49-0.23
				l-0.98-0.29l-0.17-0.4l-0.49-0.22l-0.16,0l-0.32-0.08l-0.8,0.34l-0.32,0.01l-0.15,0.49l-0.63,0.41l0.17,0.48l0.16,0.15
				l-0.07,0.48l0.1,0.4l0.09,0.39l-0.47,0.34l-0.23,0.49l-0.23,0.25l0.4,0.31l0.33,0.15l-0.32,0.24l-0.31,0.25l-0.46,0.58l0.73,0.05
				l0.25,0.15l0.01,0.4l0.17,0.24l0.17,0.4l0.08,0.07l0.25,0.31l-0.24,0.17l0.34,0.64l-0.22,0.33l-0.08,0.32l0.01,0.32l-0.23,0.09
				l-0.09-0.08l-0.24,0.01l0.26,0.4l0.25,0.07l0.26,0.72l0.34,0.32l0.24,0.15l0.18,0.79l0.03,0.97l0.58,0.38l0.49,0.23l0.65,0.14
				l0.16-0.09l0.41,0.16l0.48,0.06l0,0.09l-0.38,0.89l0.03,0.88l-0.23,0.65l-0.14,0.57l-0.38,0.42l0.34,0.54l-0.32-0.07l-0.25-0.24
				l-0.46,0.65l0.01,0.4l-0.07,0.41l0.32,0.15l0.34,0.23l-0.17,0.08l-0.15,0.41l-0.62,0.5l-0.25-0.15l-0.32-0.15l-0.06,0.41
				l-0.24,0.01l-0.08,0.08l0.69,1.35l0.02,0.55l0.26,0.96l0.58,0.46l0.26,0.56l-0.24,0.25l-0.07,0.16l0.4,0.31l0.27,0.88l-0.15,0.32
				l-0.16,0.08l-0.08,0.25l-0.47,0.17l0.41,0.31l0.24,0.08l0.41,0.15l0.41,0.07l0.07,0l0.34,0.22l-0.16,0.16l-0.17,0.09l-0.23,0.08
				l-0.08,0.16l-0.31,0.18l-0.16,0.08l0,0.16l0.17,0.47l0.32,0.08l0.25,0.08l0.25,0.07l-0.48,0.49l0.09,0.24l-0.46,0.58l-0.07,0.64
				l-0.15,0.32l-0.14,0.81l-0.06,0.73l-0.22,0.33l-0.14,0.57l-0.47,0.57l0.1,0.33l-0.07,0.48l-0.22,0.41L557.9,88l-0.37,1.13
				l0.09,0.17l0.57,0.3l-0.4,0.25l-0.31,0.41l0.01,0.4l0.02,0.8l0.08,0.16l0.09,0.24l-0.07,0.48l-0.22,0.66l-0.24,0.33l-0.23,0.49
				l-0.14,0.57l-0.3,0.57l-0.13,1.05l-0.4,0.25l-0.21,0.73l0.08,0.24l0.1,0.4l0.16,0l0.09,0.16l-0.31,0.65l-0.39,0.26l-0.14,0.73
				l0-0.09l-0.19-0.56l-0.79,0.27l0.01,0.23l-0.24,0.25l-0.06,0.57l-0.23,0.41l-0.49-0.32l-0.31,0.17l-0.31,0.34l-0.47,0.65
				l-0.14,0.64l0.16,0.08l0.32,0.06l0.25,0.16l1.06,0.45l-0.32,0.01l-0.15,0.24l0.17,0.4l-0.28,1.21l0.58,0.47l-0.15,0.4l-0.49,0.17
				l-0.23,0.01l-0.24,0.17l0.09,0.24l-0.07,0.41l-0.89-0.13l-0.23,0.57l0.01,0.48l-0.37,0.73l-0.73-0.06l-0.32,0.01l-0.56,0.34
				l-0.15,0.41l-0.49,0.17l-0.31,0.41l-0.22,0.41l-0.21,1.12l-0.24,0.25l-0.64-0.07l-1.67,0.37l-0.33,0.01l0.01,0.4l-0.13,0.97
				l-0.49-0.47l-0.46,0.82l-0.09,0l-0.06,0.57l-0.16,0.16l-1.01,1.07l-0.54,0.74l0.03,0.88l0.32,0.07l0.77,1.42l0.16,0.15
				l-0.39,0.42l-0.07,0.24l0.11,1.05l2.18,0.33l1.03-0.51l0.26,0.56l0.09,0.48l-0.31,0.41l0.17,0.4l-0.15,0.24l0.25,0.31l0.42,0.71
				l0,0.16l-0.55,0.58l-0.15,0.25l-0.46,0.58l-0.3,0.65l-0.54,0.74l-0.07,0.08l-0.4,0.17l-0.81,0.11l-1.05-0.13l0.01,0.32l0.01,0.24
				l0.17,0.39l0.01,0.49l0.41,0.31l0,0.16l-0.46,0.74l0.25,0.15l-0.07,0.41l-0.15,0.4l-0.3,0.57l-0.7,0.74l1.65,1.31l0.51,1.11
				l-0.31,0.41l-0.32,0.08l-0.38,0.73l0.16,0.15l0.65,0.46l-0.06,0.57l0,0.16l0.84,1.35l1.54,0.19l0.39-0.17l0.06-0.96l-0.04-1.21
				l0.39-0.49l0.54-0.34l0.16-0.16l0.34,0.39l0.41,0.47l0.35,0.55l0.17,0.23l0.25,0.4l0.17,0.4l0.89,0.05l0.23-0.57l0.46-0.65
				l0.07-0.41l0.39-0.41l0.81,0.45l0.24-0.01l0,0.16l0.17,0.47l0.5,0.63l0.48,0.15l0.41,0.06l0.4-0.01l0.31-0.33l0.47-0.17l0.48-0.1
				l0.48-0.41l0.4,0.06l0.16,0l0.24-0.17l0.24-0.08l0.32,0.06l0.23-0.08l0.33,0.06l0.47-0.17l0.88-0.1l-0.01-0.24l0.15-0.32
				l0.15-0.25l0.32-0.08l0.16,0.07l0.57,0.14l0.41,0.15l0.16,0l0.57,0.07l0.32-0.09l0.21-1.21l0.15-0.24l0.07-0.25l0.56-0.18
				l0.1,0.4l0.67,0.94l0.88-0.19l0.56-0.26l0.23-0.08l0.33-0.09l0.8-0.02l0.56-0.09l0.23-0.01l0.33-0.17l0.23-0.25l-0.16,0l0-0.16
				l-0.01-0.24l0.4-0.33l0.07-0.25l0.15-0.4l0.23-0.41l0.41,0.15l0.07-0.08l-0.01-0.17l0.4-0.33l-0.08-0.23l0.56,0.06l0.32-0.33
				l0.16,0l0.15-0.41l-0.01-0.23l0.32-0.01l0.15-0.49l0.14-0.56l0.32-0.09l0.16-0.08l0.15-0.49l0.56-0.34l-0.33-0.24l-0.48-0.15
				l-0.49-0.38l-0.25-0.32l0.31-0.24l0.16-0.16l-0.5-0.63l0.07-0.16l0.58,0.39l0.41,0.31l0.32,0.08l0.49,0.38l0.41,0.16l0.25,0.07
				l0.24,0.08l0.1,0.39l0.47-0.17l0.4-0.17l0-0.16l1.04-0.28l0.71-0.34l0.32-0.01l-0.18-0.72l-0.33-0.22l-0.1-0.4l-0.01-0.24l0.16,0
				l0.08-0.08l0.06-0.64l-0.16,0l-0.08-0.08l0.32-0.01l0.08-0.09l0-0.07l-0.33-0.08l0.08-0.16l0.88-0.19l0.46-0.16
				c0.05,0.06,0.26-0.04,0.26-0.04l0.28-1.12l0.55-0.18l0.47,0.17l0.49-0.3l0.16,0.22l0.47-0.17l0.94,0.35l0.43-0.37l0.31,0.21
				l2.31,0.06c0.01,0,0.93-0.55,0.93-0.55l-0.01-0.2l0.55-0.42l0.06-0.64l-0.56,0.02l-0.32-0.15l-0.41-0.23l-0.08-0.24l-0.16,0.09
				l-0.32-0.08l-0.09-0.16l0.4-0.17l0-0.16l0.72-0.27l0.07-0.32l0.16-0.16l1.21-0.04l0.33,0.31l0.07-0.24l0.16-0.09l0.24-0.01
				l0.24-0.08l0.06-0.57l-0.32,0.01l-0.42-0.48l-0.24-0.31l-0.01-0.48l-0.41-0.23l0.16-0.08l0.07-0.48l-0.73-0.06l-0.01-0.4
				l-0.33-0.06l0.23-0.49l-0.73-0.06l1.41-1.4l0.32-0.09l-0.24-0.23l-0.41-0.16l-0.25-0.23l0.29-1.14l-0.01-0.39l0.15-0.17l0.22-0.8
				l-0.08-0.16l0.22-0.41l0.81,0.06l0.56,0.06l0.56-0.09l-0.06,0.64l0.01,0.32l-0.08,0.25l-0.31,0.17l0.25,0.24l0.16,0l0.07-0.09
				l0.16-0.08l0.25,0.15l0-0.16l0.15-0.17l-0.1-0.48l0.08-0.16l-0.09-0.07l0.81,0.14l0.27,0.72l0,0.16l0.32-0.01l0.17,0.23
				l0.16,0.15l0.25,0.08l-0.16,0.32l0.09,0.24l0.16,0l-0.08-0.24l0.48-0.01l0.21-0.73l0.4-0.09l-0.01-0.33l0.16,0.08l0.31-0.17
				l0.08-0.09l0.33,0.32l0.48-0.1l-0.08-0.16l0.31-0.49l-0.17-0.32l0.07-0.08l-0.16,0l0-0.16l-0.17-0.24l0.15-0.41l0.71-0.41
				l0.14-0.65l-0.17-0.31l0.56-0.18l0.24-0.33l-0.16-0.15l0.15-0.48l-0.17-0.48l0-0.07l-0.35-0.64l-0.48-0.15l-0.24-0.07l-0.48,0.42
				l-0.23,0.17l-0.58-0.39l0.31-0.49l0.31-0.17l0.01,0.23l0.09,0l0.54-0.81l0.16,0.07l0.47-0.17l0.23-0.4l0.38-0.42l0.31-0.41
				l0.17-0.01l0.24,0.08l0.16-0.09l-0.42-0.71l0.23-0.17l-0.01-0.24l-0.16-0.15l0.15-0.24l0-0.09l-0.09-0.4l0.21-0.72l0.08-0.09
				l0.16,0.08l0.07-0.17l0.24-0.17l0.41,0.15l-0.3,0.57l0.16,0.15l0.4,0.07l0.48-0.26l0-0.07l0.47-0.26l0.55-0.42l-0.17-0.31
				l-0.01-0.23l0.07-0.25l-0.4-0.15l0-0.16l-0.34-0.39l-0.16,0l-0.15,0.48l0.42,0.64l-0.16,0.08l-0.07,0.41l-0.08-0.08l-0.09-0.23
				l-0.16-0.15l-0.32-0.08l-0.4,0.1l-0.23,0.08l-0.01-0.48l-0.25-0.15l-0.08,0.16l0.09,0.08l-0.24,0.33l-0.01-0.24l-0.16-0.15
				l0.15-0.25l-0.01-0.39l0.15-0.41l0.16-0.16l0.24-0.01l-0.06,0.48l0.08,0.16l-0.15,0.25l0.16,0.15l0.24-0.25l-0.09-0.32l0.24-0.33
				l0.23-0.17l0.32-0.25l0.32-0.01l0.07-0.16l-0.17-0.24l0.08-0.16l-0.09-0.32l-0.16-0.15l-0.24-0.15l-0.16,0.32l-0.07,0.25
				l-0.25-0.15l-0.01-0.24l0.08-0.32l0.06-0.41l0.54-0.81l0-0.16l-0.01-0.24l0.08-0.32l0.73,0.22l0.31-0.18l0.23-0.24l-0.01-0.48
				l-0.48-0.07l-0.24-0.24l0.16-0.16l0.06-0.4l0.08-0.17l-0.16-0.15l-0.41-0.31l-0.01-0.23l0.61-1.15l0.32,0.15l0.48-0.49l0.15-0.25
				l-0.1-0.4l0.08-0.32l-0.17-0.23L599,88.14l-0.23,0.49l0.09,0.33l-0.47,0.17l-0.31,0.24l-0.17-0.31l-0.25-0.15l-0.41-0.23l-0.16,0
				l0.09,0.24l-0.08,0.16l-0.09-0.4l-0.16,0.33l-0.17-0.24l-0.16,0.08l0.01,0.4l-0.16,0l-0.08-0.07l-0.16,0l-0.34-0.23l0.16-0.09
				l-0.17-0.31l-0.42-0.79l0.07-0.25l-0.57-0.06l-0.16,0l-0.33-0.24l-0.16,0.09h0l-0.1-0.65l0.53-1.06l0.08-0.32l0.47-0.49l0.8-0.34
				l0.16,0l0.32-0.09l0.56-0.1l0-0.16l0.39-0.25l-0.18-0.72L596.51,83l0.08-0.16l0.32-0.09l0.48-0.01l0.24-0.01l0.09,0.08l0.47-0.26
				l0.17,0.24l0.64-0.02l0.09,0l0.22-0.41l0.65-0.02l0.56-0.02l0.32,0.06l0.39-0.25l0.33-0.01l0-0.16l0.31-0.25l0.48,0.15l0.4-0.09
				l0.31-0.25l0.24-0.01l0.47-0.26l0.31-0.17l0.16,0.08l0.32-0.09l0.09,0l0.32-0.08l0.24-0.09l0.15-0.24l0.48-0.1l0.65,0.3
				l0.73,0.06l0.1,0.39l0.4-0.33l0.16,0.15l0.13-0.89l0.23-0.24l0.33-0.01l0.24,0.07l0.47-0.25l-0.01-0.49l0.16,0.17l0.33-0.01
				l0.47-0.34L610.07,78.91z"></path>
		</g>
		<g id="barcelona_p">
			<path id="barcelona_p" class="st6" d="M631.9,120.65l-0.4,0.17l-0.23,0.25l-0.09,0l-0.16,0.08l0,0.09l-0.16,0.08l0.08-0.08
				l0-0.09l-0.09,0l-0.07,0.09l-0.09,0l-0.08,0.16l-0.07,0l0,0.07l-0.09,0l-0.07,0.09l-0.16,0l-0.08,0.08l-0.4,0.1l-0.31,0.17
				l-0.16,0l-0.15,0.16l-0.16,0.09l0,0.07l-0.08,0.09l-0.16,0.08l-0.07,0.09l-0.08,0.16l0,0.07l-0.07,0l0-0.07l-0.08,0.16l-0.07,0
				l-0.16,0.09l-0.08,0.08l-0.4,0.33l-0.07,0l-0.08,0.09l-0.16,0.08l-0.07,0.09l-0.17,0.08l-0.15,0.25l-0.07,0.09l-0.08,0.08
				l-0.07,0l-0.16,0.16l-0.09,0l-0.07,0.09l-0.09,0l-0.24,0.17l-0.07,0.08l-0.16,0.09l-0.16,0.08l-0.08,0.09l-0.07,0l0,0.09
				l-0.08,0.08l-0.07,0l-0.09-0.07l-0.09,0l-0.07,0.08l-0.08,0.09l-0.16,0.08l-0.07,0l-0.16,0.09l-0.16,0.08l-0.17,0.01l-0.23,0.09
				l-0.16,0.08l-0.24,0.01l-0.16,0.09l-0.16,0.08l-0.24,0.18l-0.07,0.08l-0.09,0l0.08-0.08l0.07-0.09l-0.23,0.01l-0.33,0.17l-0.07,0
				l-0.08,0.09l-0.16,0.08l-0.39,0.26l-0.08,0.08l-0.16,0l-0.24,0.17l-0.24-0.32l0.24,0.32l-0.15,0.17l-0.23,0.17l-0.08,0.08
				l-0.16,0.16l-0.08,0.16l-0.07,0.09l-0.08,0.08l-0.07,0.17l0,0.07l-0.09,0l0,0.09l0,0.07l-0.07,0.25l-0.01-0.24l-0.15,0.32l0,0.09
				l-0.08,0.16l0,0.07l-0.07,0.09l-0.08,0.09l-0.08,0.16l0,0.16l-0.07,0l0.08,0.07l-0.07,0.09l0-0.09l-0.09-0.07l-0.07,0.16l0.07,0
				l0.09,0.07l0,0.09l-0.09-0.08l-0.07,0.25l-0.09-0.08l0,0.09l-0.31,0.25l0,0.07l-0.07,0.09l-0.08,0.08l-0.16,0.16l-0.24,0.34
				l0,0.07l-0.07,0l-0.09,0l0,0.09l0,0.07l-0.07,0.09l-0.08,0.16l-0.07,0.08l-0.16,0.41l0,0.09l0.09,0l-0.08,0.32l-0.07,0.08
				l-0.08,0.25l0.08-0.32l0.07-0.16l-0.07,0l0-0.09l-0.09,0l0-0.16l0.08-0.08l0-0.09l-0.08,0.09l-0.01-0.17l0.09,0l0-0.07l0-0.09
				l0-0.07l-0.09,0l-0.07,0l0.08,0.16l-0.07,0.16l-0.09-0.08l0.08-0.08l0-0.09l-0.15,0.25l0.16,0.15l-0.09,0l-0.08-0.07l-0.09,0
				l0.01,0.23l0.16,0.08l-0.07,0.16l-0.16-0.08l-0.08,0.16l0.09,0l0.08,0.08l-0.07,0.08l-0.16-0.07l-0.08,0.08l0.16,0.17l-0.23,0.49
				l0-0.09l-0.16-0.15l-0.07,0l-0.08,0.16l0.09,0.08l-0.31,0.24l0-0.07l-0.09-0.16l-0.06,0.57l-0.17,0.01l-0.07,0.08l0.16,0.08
				l-0.08,0.16l0.17-0.01l0.07-0.25l0.15-0.32l0.08-0.09l0.07,0l0.16,0l0.09,0.08l-0.15,0.41l-0.08,0.24l0,0.09l-0.23,0.01
				l-0.08,0.24l0,0.09l-0.07,0l-0.08,0.09l-0.09,0l0,0.07l0,0.09l0.18,0.15l-0.09,0l-0.16-0.15l-0.08,0.16l-0.22,0.49l0.16,0.08
				l-0.15,0.16l-0.09,0l-0.07,0l0.08,0.07l0.09,0.16l-0.09-0.07l-0.16,0l-0.07,0l-0.24,0.08l-0.15,0.25l-0.63,0.34l-1.2,0.51
				l-1.11,0.36l-0.88,0.19l-0.88,0.19l-2.33,0.07l-0.32,0.08l-0.24,0.09l-0.23,0.17l-0.09,0l0.08-0.09l0.07-0.08l-0.72,0.26
				l-0.16,0.09l-0.16,0.08l-0.09,0l-0.15,0.17l-0.23,0.08l-0.32,0.09l-0.56,0.25l0,0.09l-0.09,0l0.09,0.07l-0.08,0.09l0-0.09
				l-0.16,0l-0.07,0.16l-0.16,0.09l-0.25-0.08l-0.16,0l-0.08,0.09l-0.16-0.08l-0.07,0.09l-0.41-0.15l-0.08,0.08l0,0.09l-0.08-0.08
				l-0.08,0.09l-0.07,0l0-0.09l-0.09-0.07l-0.16,0.08l-0.16,0.09l-0.16,0l-0.4,0.17l-0.16,0.09l-0.07,0.08l-0.17,0.09l0,0.07
				l-0.07,0.09l0-0.09l-0.16,0.09l-0.16-0.08l-0.24,0.17l-0.07,0l-0.09,0l-0.08,0.09l-0.07,0l-0.08,0.08l-0.24-0.07l-0.16,0.08
				l-0.16,0l-0.25,0.01l-0.15,0.17l-0.08,0.08l0.09,0.16l-0.09,0l-0.08-0.32l-0.32,0.09l0.08,0.23l-0.08-0.07l-0.33,0.01l-0.23,0.08
				l-0.4,0.1l-0.4,0.09l-0.23,0.17l-0.16,0.09l-0.16,0.08l-0.24,0.18l-0.72,0.1l0,0.09l0,0.07l-0.4,0.01l0.24-0.08l0.07-0.09
				l-0.23,0.01l-0.08,0.09l-0.17-0.4l-0.01-0.48l0.24-0.01l0.17,0.23l0.07-0.08l-0.25-0.48l-0.16-0.15l0.06-0.48l-0.17-0.31
				l-0.31,0.17l-0.08,0.16l-0.07,0.25l-0.32,0.08l-0.25-0.23l-0.25-0.24l-0.33-0.24l-0.32,0.09l-0.41-0.07l-0.34-0.71l-0.01-0.32
				l-0.24-0.31l0.96,0.13l0.09,0.08l0.08,0.07l0,0.09l0.16,0l0.32,0.15l0.32-0.25l0.16,0l0.06-0.4l0-0.09l0.24-0.33l-0.25-0.31
				l0.24-0.33l0.06-0.41l-0.07,0l-0.01-0.24l0.16,0l0.16-0.16l-0.16-0.07l-0.16,0.16l-0.09-0.08l-0.39,0.26l-0.15,0.4l-0.41-0.15
				l-0.49-0.3l-0.09,0l-0.01-0.23l0.23-0.57l0-0.09l-0.66-0.3l-0.16,0l-0.01-0.32l-0.25-0.4l-0.1-0.48l-0.25-0.24l0.47-0.58
				l0.32-0.08l-0.1-0.4l-0.16-0.07l-0.07,0.08l-0.01-0.23l-0.32-0.16l-0.24,0.09l-0.25-0.08l-0.17-0.39l-0.42-0.48l-0.72-0.14
				l-0.41-0.31l-1.43,0.77l0.08-0.25l0.15-0.32l0.15-0.4l0.23-0.18l-0.18-0.63l-0.16-0.15l0.08-0.16l0.07-0.16l-0.32-0.15l0.07-0.41
				l0.24,0.08l0.4-0.17l0-0.09l-0.8,0.02l0-0.16l-0.25-0.31l0.7-0.9l-0.01-0.49l-0.09-0.32l-0.32-0.15l-0.09-0.4l-0.33,0.01
				l-0.24-0.15l-0.32,0.01l0-0.07l-0.5-0.39l-0.73-0.14l-0.08-0.16l-0.08,0.08l-0.24-0.15l-0.32-0.15l-0.09-0.23l-0.16-0.08
				l-0.01-0.32l0.07-0.32l0.23-0.49l0.89,0.13l0.39-0.42l0.48,0.15l-0.02-0.64l-0.08-0.24l-0.49,0.1l-0.17-0.24l0.95-0.75
				l-1.31-0.68l-0.4,0.1l-0.15,0.24l-0.64,0.02l-0.01-0.23l0.55-0.42l0.06-0.64l-0.56,0.02l-0.32-0.15l-0.41-0.23l-0.08-0.24
				l-0.16,0.09l-0.32-0.08l-0.09-0.16l0.4-0.17l0-0.16l0.72-0.27l0.07-0.32l0.16-0.16l1.21-0.04l0.33,0.31l0.07-0.24l0.16-0.09
				l0.24-0.01l0.24-0.08l0.06-0.57l-0.32,0.01l-0.42-0.48l-0.24-0.31l-0.01-0.48l-0.41-0.23l0.16-0.08l0.07-0.48l-0.73-0.06
				l-0.01-0.4l-0.33-0.06l0.23-0.49l-0.73-0.06l1.41-1.4l0.32-0.09l-0.24-0.23l-0.41-0.16l-0.25-0.23l0.29-1.14l-0.01-0.39
				l0.15-0.17l0.22-0.8l-0.08-0.16l0.22-0.41l0.81,0.06l0.56,0.06l0.56-0.09l-0.06,0.64l0.01,0.32l-0.08,0.25l-0.31,0.17l0.25,0.24
				l0.16,0l0.07-0.09l0.16-0.08l0.25,0.15l0-0.16l0.15-0.17l-0.1-0.48l0.08-0.16l-0.09-0.07l0.81,0.14l0.27,0.72l0,0.16l0.32-0.01
				l0.17,0.23l0.16,0.15l0.25,0.08l-0.16,0.32l0.09,0.24l0.16,0l-0.08-0.24l0.48-0.01l0.21-0.73l0.4-0.09l-0.01-0.33l0.16,0.08
				l0.31-0.17l0.08-0.09l0.33,0.32l0.48-0.1l-0.08-0.16l0.31-0.49l-0.17-0.32l0.07-0.08l-0.16,0l0-0.16l-0.17-0.24l0.15-0.41
				l0.71-0.41l0.14-0.65l-0.17-0.31l0.56-0.18l0.24-0.33l-0.16-0.15l0.15-0.48l-0.17-0.48l0-0.07l-0.35-0.64l-0.48-0.15l-0.24-0.07
				l-0.48,0.42l-0.23,0.17l-0.58-0.39l0.31-0.49l0.31-0.17l0.01,0.23l0.09,0l0.54-0.81l0.16,0.07l0.47-0.17l0.23-0.4l0.38-0.42
				l0.31-0.41l0.17-0.01l0.24,0.08l0.16-0.09l-0.42-0.71l0.23-0.17l-0.01-0.24l-0.16-0.15l0.15-0.24l0-0.09l-0.09-0.4l0.21-0.72
				l0.08-0.09l0.16,0.08l0.07-0.17l0.24-0.17l0.41,0.15l-0.3,0.57l0.16,0.15l0.4,0.07l0.48-0.26l0-0.07l0.47-0.26l0.55-0.42
				l-0.17-0.31l-0.01-0.23l0.07-0.25l-0.4-0.15l0-0.16l-0.34-0.39l-0.16,0l-0.15,0.48l0.42,0.64l-0.16,0.08l-0.07,0.41l-0.08-0.08
				l-0.09-0.23l-0.16-0.15l-0.32-0.08l-0.4,0.1l-0.23,0.08l-0.01-0.48l-0.25-0.15l-0.08,0.16l0.09,0.08l-0.24,0.33l-0.01-0.24
				l-0.16-0.15l0.15-0.25l-0.01-0.39l0.15-0.41l0.16-0.16l0.24-0.01l-0.06,0.48l0.08,0.16l-0.15,0.25l0.16,0.15l0.24-0.25
				l-0.09-0.32l0.24-0.33l0.23-0.17l0.32-0.25l0.32-0.01l0.07-0.16l-0.17-0.24l0.08-0.16l-0.09-0.32l-0.16-0.15l-0.24-0.15
				l-0.16,0.32l-0.07,0.25l-0.25-0.15l-0.01-0.24l0.08-0.32l0.06-0.41l0.54-0.81l0-0.16l-0.01-0.24l0.08-0.32l0.73,0.22l0.31-0.18
				l0.23-0.24l-0.01-0.48l-0.48-0.07l-0.24-0.24l0.16-0.16l0.06-0.4l0.08-0.17l-0.16-0.15l-0.41-0.31l-0.01-0.23l0.61-1.15
				l0.32,0.15l0.48-0.49l0.15-0.25l-0.1-0.4l0.08-0.32l-0.17-0.23L599,88.14l-0.23,0.49l0.1,0.33l-0.47,0.17l-0.31,0.24l-0.17-0.31
				l-0.25-0.15l-0.41-0.23l-0.16,0l0.09,0.24l-0.08,0.16l-0.09-0.4l-0.16,0.33l-0.17-0.24l-0.16,0.08l0.01,0.4l-0.16,0l-0.08-0.07
				l-0.16,0l-0.34-0.23l0.16-0.09l-0.17-0.31l-0.42-0.79l0.07-0.25l-0.57-0.06l-0.16,0l-0.33-0.24l-0.16,0.09l-0.1-0.65l0.53-1.06
				l0.08-0.32l0.47-0.49l0.8-0.34l0.16,0l0.32-0.09l0.56-0.1l0-0.16l0.39-0.25l-0.18-0.72L596.51,83l0.08-0.16l0.32-0.09l0.48-0.01
				l0.24-0.01l0.09,0.08l0.47-0.26l0.17,0.24l0.64-0.02l0.09,0l0.22-0.41l0.65-0.02l0.56-0.02l0.32,0.06l0.39-0.25l0.33-0.01l0-0.16
				l0.31-0.25l0.48,0.15l0.4-0.09l0.31-0.25l0.24-0.01l0.47-0.26l0.31-0.17l0.16,0.08l0.32-0.09l0.09,0l0.32-0.08l0.24-0.09
				l0.15-0.24l0.48-0.1l0.65,0.3l0.73,0.06l0.1,0.39l0.4-0.33l0.16,0.15l1.06,0.62l0.39-0.01l0.4-0.33l0.4-0.1l0.48-0.01l0.57,0.23
				l0.33,0.22l0.41,0.23l0.32,0.15l0.25,0.08l-0.15,0.32l-0.56,0.34l-0.08,0.16l-0.31,0.33l-0.23,0.49l-0.31,0.41l0,0.09l-0.06,0.57
				l-0.08,0.24l-0.31,0.25l0.25,0.31l0.25,0.56l0.16-0.16l0.4-0.17l0.41,0.06l-0.08,0.25l-0.07,0.24l0.1,0.81l0.25,0.07l0.24,0.08
				l-0.15,0.16l0.1,0.48l-0.15,0.32l-0.56,0.02l-0.33-0.22l-0.24,0.24l-0.06,0.33L610.69,89l-0.64,0.1l-0.09-0.16l-0.33-0.31
				l0.3-0.57l-0.08-0.16l-0.18-0.32l0.4-0.17l-0.8,0.02l-0.57-0.14l-0.32-0.06l0.03,1.04l0.56-0.1l0.48-0.01l-0.08,0.25l0.09,0.24
				l0.43,0.79l0.08,0.16l0.01,0.24l0.88-0.19l0.33,0.22l0.25,0.31l0.32-0.08l0.16,0.15l0.25,0.24l0.08,0.07l0.08-0.08l0.31-0.18
				l0-0.16l0.57,0.07l0.31-0.17l0.25,0.23l0.01,0.49l0.08,0.07l0,0.16l0.09,0.16l0.58,0.39l0.32,0.15l0.47-0.49l0.31-0.49l0.64-0.26
				l0.96,0.06l0.09,0.16l0.01,0.32l0.23-0.01l0.32-0.01l0.08-0.25l0.16,0.15l0.16-0.08l-0.08-0.32l0.07-0.25l0.08-0.09l1.04-0.19
				l0.17,0.32l0.56-0.1l0.23-0.17l0.24-0.17l0.16,0.15l0.32,0.06l0.24-0.01l0.31-0.49l-0.09-0.07l-0.16,0.16l-0.24-0.08l0.07-0.16
				l0.16-0.16l-0.23,0.01l-0.09-0.16l-0.41-0.23l-0.24-0.07l0.56-0.02l0.32,0.06l0.23-0.08l0.33,0.15l0.24,0.23l0.4-0.25l0.32-0.01
				l0.16-0.09l0.31-0.17l0.33-0.08l0.55-0.18l0.09,0l0.4-0.26l0.01,0.24l0.24,0.07l0.32,0.15l0,0.16l-0.4-0.06l-0.08,0.16
				l-0.63,0.25l-0.15,0.33l-0.15,0.24l0.09,0.49l0.58,0.38l0.33,0.24l0.32,0.15l-0.22,0.33l-0.08,0.09l0.1,0.39l0.17,0.32l0.72-0.02
				l0.08,0.07l-0.22,0.57l0.08,0.08l0,0.07l0.33,0.47l0.17,0.24l0,0.16l0.42,0.39l0.25,0.07l0.23-0.08l0.71-0.51l0.08-0.24
				l0.25,0.07l0.4,0.15l0.16,0.17l0.09,0.32l0.24-0.17l0.48-0.01l0.24,0.31l0,0.16l-0.16,0.16l-0.39,0.26l0.17,0.31l0.75,0.7
				l0.08,0.07l-0.15,0.32l-0.15,0.17l-0.16,0l0.01,0.48l0.41,0.15l-0.24,0.17l0.09,0.24l-0.49,0.17l-0.23,0.17l-0.09-0.16
				l-0.08-0.16l-0.08,0.24l-0.31,0.17l0.74,0.47l-0.31,0.17l0.01,0.23L629,98.81l0.08,0.33l-0.48,0.01l-0.23,0.17l-0.09-0.08
				l-0.16,0l-0.09-0.07l-0.24-0.08l-0.16-0.15l-0.4,0.26l0.01,0.23l0.24-0.01l0.24,0.31l0.01,0.24l-0.15,0.41l0.25,0.31l0.07-0.16
				l0.25-0.01l0.25,0.07l0.16,0.08l0.08,0.16l0.4-0.01l0.26,0.63l0.01,0.33l0.01,0.23l-0.39,0.42l-0.08,0.16l-0.31,0.25l-0.24,0.24
				l-0.31,0.17l-0.07,0.32l0,0.09l0.33,0.24l0.01,0.48l-0.15,0.48l-0.24,0.09l-0.22,0.4l-0.4,0.01l-0.31,0.25l0.08,0.16l0,0.16
				l0.17,0.31l-0.07,0.09l-0.24,0.17l-0.24,0.33l-0.06,0.41l0.08,0.32l0.18,0.24l-0.08,0.16l0.25,0.07l0.48-0.01l0.49,0.39l0.1,0.48
				l0.49,0.23l0.63-0.26l-0.14,0.57l0.58,0.39l0.49,0.3l0.09,0l0.8,0.14l0.09,0.32l-0.16,0.16l0.17,0.24l-0.08,0.09l0.41,0.31
				l0.25,0.31l-0.08,0.25l0.17,0.39l0.57,0.23l0.63-0.34l0.33-0.09l0.16,0.08l0.88-0.19l-0.07,0.4l0.17,0.24l-0.22,0.49l-0.23,0.41
				l0.17,0.47l0.17,0.31l0.37,0.58l-0.51,0.87l-1.79,0.2l1.09,3.95l-1.24,0.47L631.9,120.65z"></path>
		</g>
		<g id="girona_p">
			<path class="st5" d="M655.89,105.17l0.07-0.09l-0.07,0L655.89,105.17z M661.51,80.68l-0.07,0.16l0.08,0.08l0.08-0.25
				L661.51,80.68z M655.77,90.32l0.07,0l0-0.07L655.77,90.32z M659.35,78.33l-0.08-0.08l0,0.09L659.35,78.33z M662.52,79.12l-0.09,0
				l0.09,0.07L662.52,79.12z M659.18,94.23l-0.09,0l0,0.09L659.18,94.23z M659.09,94l-0.07,0l-0.09,0l-0.07,0.08l0.08,0.08l0.09,0
				l0.16-0.09l0-0.07L659.09,94z M653.14,107.5l-0.07,0l0.08,0.08L653.14,107.5z M662.28,79.45l0.24-0.09l-0.09-0.07l-0.33-0.24
				l0.01,0.24l-0.16-0.15l0-0.16l-0.09,0l0,0.07l-0.49-0.22l-0.08,0.08l-0.08-0.07l-0.16,0.16l-0.01-0.33l-0.16,0l-0.09-0.07
				l-0.08-0.08l-0.57-0.22l-0.09,0l0,0.16l-0.07,0.08l0,0.09l-0.24,0.01l-0.07,0l0,0.16l-0.09,0l-0.17-0.31l-0.16,0.08l0-0.16
				l-0.17-0.23l-0.01-0.24l-0.16-0.08l-0.09-0.07l-0.16-0.08l-0.16,0.16l-0.32,0.01l-0.16,0.09l-0.08,0.16l0-0.07l-0.16,0l-0.09,0
				l-0.15,0.24l0.09,0l0.07,0l0,0.09l-0.07,0.16l-0.16,0.08l-0.49-0.38l0.06-0.41l0-0.09l-0.07,0l0,0.09l-0.09,0l-0.09-0.08
				l0.08-0.08l-0.09,0l-0.07,0l-0.09-0.08l-0.07,0l-0.16-0.15l-0.16-0.07l-0.09-0.16l-0.01-0.49l0-0.16l-0.07,0l0,0.09l-0.08,0.08
				l-0.09,0l-0.01-0.23l-0.08-0.24l0.24-0.25l0.15-0.24l-0.09,0l-0.16-0.08l-0.32-0.06l0.15-0.25l0.31-0.25l-0.08-0.16l-0.24,0.01
				l0.23-0.4l0.07-0.16l0.16-0.09l-0.01-0.24l-0.07,0l0.07-0.24l-0.32-0.15l0.08-0.16l0.23-0.01l0.24-0.18l-0.09-0.07l-0.39,0.01
				l-0.24,0.08l-0.32,0.01l-0.24,0.01l-0.73-0.05l-0.24-0.08l-0.4,0.1l-0.24,0.33l0,0.16l-0.48,0.1l-0.16,0l-0.42-0.48l-0.16-0.07
				l-0.17-0.24l-0.25-0.23l-0.49-0.47l-0.17-0.47l-0.26-0.63l-0.32,0.01l-0.39,0.17l-0.32,0.24l-0.49-0.22l-0.49-0.23l-0.39,0.42
				l-0.47,0.17l-0.58-0.62l-0.25-0.24l-0.71,0.58l-0.22,0.64l-0.31,0.17l-0.65-0.14l-0.73-0.14l-0.49-0.3l-0.23,0.25l-0.07,0.48
				l-0.89-0.22l-0.24,0.33l-0.06,0.49l-0.24,0.33l-0.32-0.08l-0.39,0.5l-0.08-0.08l-0.3,0.73l-0.17,0.01l-0.39,0.09l-0.24,0.25
				l-0.24,0.09l-0.24-0.24l-0.33-0.39l-0.73-0.05l-0.65,0.09l-0.62,0.51l-0.63,0.41l-0.4,0.01l-0.4,0.26l0.02,0.8l-0.23,0.09
				l-0.49-0.14l0.26,0.39l0.1,0.4l0,0.16l0.17,0.4l0.32,0.15l0.09,0.56l0.01,0.4l-0.39,0.09l-0.33-0.06l-0.32-0.08l-0.39,0.01
				l-0.25-0.15l-0.88,0.03l-0.32-0.15l-0.57-0.23l0-0.16l-0.48-0.06l-0.8,0.26l-0.07,0.32l-0.15,0.33l-0.32,0.49l-0.71,0.02
				l-0.57-0.3l-0.73-0.14l-0.32,0.08l-0.16-0.07l-0.26-0.4l-0.26-0.47l-0.4-0.23l-0.17-0.4l-0.49-0.06l-0.33-0.39l-0.02-0.72
				l-1.21-0.04l-0.57-0.31l-0.48-0.06l-0.73-0.14l-0.49-0.3l-0.09-0.33l-0.08-0.07l-0.56,0.02l-0.41-0.31l-0.33-0.31l-0.56,0.34
				l-0.57-0.3l-0.4-0.32l-0.49-0.14l-0.32,0.01l-0.23,0.5l-0.39,0.09l-0.8,0.27l-0.31,0.17l-0.31,0.17l-0.8-0.06l-0.5-0.38
				l-0.56,0.18l-0.31,0.17l-0.31,0.17l-0.4,0.26l-0.22,0.64l-0.24,0.24l0.1,0.56l-0.48,0.26l-0.63,0.34l-0.14,0.57l-0.08,0.08
				l-0.33-0.22l-0.31,0.24l-0.4,0.18l-0.64,0.02l-0.56,0.09l-0.47,0.49l-0.47,0.34l-0.33,0.01l-0.16-0.17l0.01,0.49l-0.47,0.25
				L608.58,80l-0.33,0.01l-0.23,0.24l-0.13,0.89l1.06,0.62l0.39-0.01l0.4-0.33l0.4-0.1l0.48-0.01l0.57,0.23l0.33,0.22l0.41,0.23
				l0.32,0.15l0.25,0.08l-0.15,0.32l-0.56,0.34l-0.08,0.16l-0.31,0.33l-0.23,0.49l-0.31,0.41l0,0.09l-0.06,0.57l-0.08,0.24
				l-0.31,0.25l0.25,0.31l0.25,0.56l0.16-0.16l0.4-0.17l0.41,0.06l-0.08,0.25l-0.07,0.24l0.1,0.81l0.25,0.07l0.24,0.08l-0.16,0.16
				l0.1,0.48l-0.15,0.32l-0.56,0.02l-0.33-0.22l-0.24,0.24l-0.06,0.33L610.69,89l-0.64,0.1l-0.09-0.16l-0.33-0.31l0.3-0.57
				l-0.08-0.16l-0.18-0.32l0.4-0.17l-0.8,0.02l-0.57-0.14l-0.32-0.06l0.03,1.04l0.56-0.1l0.48-0.01l-0.08,0.25l0.09,0.24l0.43,0.79
				l0.08,0.16l0.01,0.24l0.88-0.19l0.33,0.22l0.25,0.31l0.32-0.08l0.16,0.15l0.25,0.24l0.08,0.07l0.08-0.08l0.31-0.18l0-0.16
				l0.57,0.07l0.31-0.17l0.25,0.23l0.01,0.49l0.08,0.07l0,0.16l0.09,0.16l0.58,0.39l0.32,0.15l0.47-0.49l0.31-0.49l0.64-0.26
				l0.96,0.06l0.09,0.16l0.01,0.32l0.23-0.01l0.32-0.01l0.08-0.25l0.16,0.15l0.16-0.08l-0.08-0.32l0.07-0.25l0.08-0.09l1.04-0.19
				l0.17,0.32l0.56-0.1l0.23-0.17l0.24-0.17l0.16,0.15l0.32,0.06l0.25-0.01l0.31-0.49l-0.09-0.07l-0.16,0.16l-0.24-0.08l0.07-0.16
				l0.16-0.16l-0.23,0.01l-0.09-0.16l-0.41-0.23l-0.24-0.07l0.56-0.02l0.32,0.06l0.23-0.08l0.33,0.15l0.24,0.23l0.4-0.25l0.32-0.01
				l0.16-0.09l0.31-0.17l0.33-0.08l0.55-0.18l0.09,0l0.4-0.26l0.01,0.24l0.24,0.07l0.32,0.15l0,0.16l-0.4-0.06l-0.08,0.16
				l-0.63,0.25l-0.15,0.33l-0.15,0.24l0.09,0.49l0.58,0.38l0.33,0.24l0.32,0.15l-0.22,0.33l-0.08,0.09l0.1,0.39l0.17,0.32l0.72-0.02
				l0.08,0.07l-0.22,0.57l0.08,0.08l0,0.07l0.33,0.47l0.17,0.24l0,0.16l0.42,0.39l0.25,0.07l0.23-0.08l0.71-0.51l0.08-0.24
				l0.25,0.07l0.4,0.15l0.16,0.17l0.09,0.32l0.24-0.17l0.48-0.01l0.24,0.31l0,0.16l-0.16,0.16l-0.39,0.26l0.17,0.31l0.75,0.7
				l0.08,0.07l-0.15,0.32l-0.15,0.17l-0.16,0l0.01,0.48l0.41,0.15l-0.24,0.17l0.09,0.24l-0.49,0.17l-0.23,0.17l-0.09-0.16
				l-0.08-0.16l-0.08,0.24l-0.31,0.17l0.74,0.47l-0.31,0.17l0.01,0.23L629,98.81l0.08,0.33l-0.48,0.01l-0.23,0.17l-0.09-0.08
				l-0.16,0l-0.09-0.07l-0.24-0.08l-0.16-0.15l-0.4,0.26l0.01,0.23l0.25-0.01l0.24,0.31l0.01,0.24l-0.15,0.41l0.25,0.31l0.07-0.16
				l0.24-0.01l0.25,0.07l0.16,0.08l0.08,0.16l0.4-0.01l0.26,0.63l0.01,0.33l0.01,0.23l-0.39,0.42l-0.08,0.16l-0.31,0.25l-0.24,0.24
				l-0.31,0.17l-0.07,0.32l0,0.09l0.33,0.24l0.01,0.48l-0.15,0.48l-0.24,0.09l-0.22,0.4l-0.4,0.01l-0.31,0.25l0.08,0.16l0,0.16
				l0.17,0.31l-0.07,0.09l-0.24,0.17l-0.24,0.33l-0.06,0.41l0.08,0.32l0.18,0.24l-0.08,0.16l0.25,0.07l0.48-0.01l0.49,0.39l0.1,0.48
				l0.49,0.23l0.63-0.26l-0.14,0.57l0.58,0.39l0.49,0.3l0.09,0l0.8,0.14l0.09,0.32l-0.16,0.16l0.17,0.24l-0.08,0.09l0.41,0.31
				l0.25,0.31l-0.08,0.25l0.17,0.39l0.57,0.23l0.63-0.34l0.33-0.09l0.16,0.08l0.88-0.19l-0.07,0.4l0.17,0.24l-0.22,0.49l-0.23,0.41
				l0.17,0.47l0.17,0.31l0.16,0.08l0.19,0.39l-0.22,0.49l-0.33,0.54l-1.67,0.2l1.03,3.91l2.12-1.07l0.23-0.17l0.32-0.08l1.36-0.6
				l1.26-0.52l0.4-0.26l0.24-0.08l0.23-0.25l0.09,0l0.71-0.43l0.31-0.17l0.8-0.1l0.17-0.18l0,0l0.22-0.64l0.31-0.49l0.16-0.16
				l0.17-0.09l0,0.09l-0.08,0.08l0.16-0.08l0.08-0.09l0-0.07l0.16-0.09l0-0.07l0.23-0.17l0.08-0.09l0-0.09l0.07-0.08l0.17-0.09
				l0.23-0.08l0-0.09l0.24-0.08l0.07-0.09l0.16,0.08l0.08-0.09l0.08-0.08l0.16-0.09l0.31-0.17l0.16,0l0.25,0.08l0.07-0.09l0.16,0
				l0.16-0.09l0.09,0l0.07,0l0-0.07l0.09,0l0.16,0l0.09,0.07l0.07,0l0.08-0.08l0.07,0l0.16-0.09l0.16-0.08l-0.08-0.08l0.16-0.08
				l0.07,0l0.24-0.01l0.09,0l0.31-0.17l0.07,0l0.09,0l0.07-0.09l0.24-0.08l0.09,0l0-0.09l0-0.16l0.07-0.09l0.08-0.08l0.16-0.16
				l0.16,0l0-0.09l0.07-0.08l-0.08-0.08l0.07,0l0.09,0l0.07-0.08l0.08-0.09l0-0.16l0.09,0l0.07,0l0.08-0.09l-0.09,0l0-0.16l0.31-0.4
				l0.15-0.25l0.16-0.09l0.16-0.24l0.23-0.09l0.32-0.08l0.08-0.09l0.07,0l0.08-0.08l0.08-0.09l0.07-0.08l0.24-0.18l0.39-0.17
				l0.25-0.01l0.09,0l0.08,0.08l0.09,0.07l0-0.07l0.07,0l-0.08-0.08l-0.09-0.32l0.09,0l0.16,0l0.07,0l0.09,0.08l0.07,0l0.16-0.09
				l0-0.07l0.09,0l0.08-0.09l0-0.07l-0.09-0.17l0-0.07l0.08-0.09l0.16,0l0.08,0.08l0.09,0.07l0-0.07l0.07-0.16l0-0.09l0.16-0.16
				l-0.08-0.07l0.16-0.16l-0.09-0.08l0.09,0l0-0.09l0.15-0.48l0.08-0.32l0.15-0.25l0.23-0.17l0.39-0.41l0.09,0l0.07-0.25l0.24-0.33
				l0.31-0.17l0.4-0.1l0.16,0l0.16,0.08l0.08,0.07l0,0.09l-0.07,0.08l-0.24,0.01l0,0.09l0.32-0.09l0.09,0l0.07,0l0-0.07l0-0.16
				l0.09,0l0.08-0.09l0,0.09l0.07,0l0-0.09l0.24-0.08l0.07-0.09l-0.08-0.16l-0.01-0.24l0.07,0l0.09,0l0.07-0.08l0-0.09l0.09,0
				l0.07,0l0-0.07l0.16,0l0,0.16l0.08-0.09l0.16,0l0.08-0.08l0.23-0.09l0.08-0.08l0.07-0.09l0.09,0l-0.09-0.16l0.08-0.25l0.16-0.08
				l-0.16-0.15l0.16-0.16l0-0.16l0.08-0.09l0.23-0.09l0.08-0.16l0-0.07l0.07,0l0.08-0.09l0.16,0.15l0.07-0.08l0.07-0.41l0.16-0.08
				l-0.01-0.33l0.16-0.08l-0.08-0.16l0-0.16l-0.09-0.16l0.24-0.01l0.07-0.16l0.16-0.09l-0.16-0.15l0.08-0.16l-0.09-0.08l0.16-0.08
				l-0.08-0.08l0.07-0.08l-0.08-0.08l-0.09,0l0-0.07l0.08-0.17l0-0.16l0.31-0.24l0.24-0.17l-0.09-0.33l0.08-0.08l0.07-0.16l-0.07,0
				l0,0.07l-0.09-0.07l0-0.09l-0.08-0.07l0.07-0.09l0-0.07l-0.08-0.16l-0.18-0.17l-0.07,0l-0.16,0l-0.24,0.01l-0.07,0l-0.01-0.23
				l-0.09-0.24l-0.26-0.4l-0.18-0.71l-0.08-0.24l-0.1-0.64l-0.01-0.49l0.07-0.4l-0.01-0.4l0.07-0.25l0.08-0.16l0-0.07l0.16,0.07
				l-0.09-0.07l-0.08-0.08l0.07,0l0.09,0l0.09,0l0.07,0l0,0.09l0.08-0.09l0-0.07l0.15-0.25l-0.09-0.08l-0.07,0l0-0.07l-0.01-0.24
				l0-0.07l-0.09-0.08l-0.08-0.07l-0.09-0.08l-0.09-0.08l0.09,0L658,92.5l0-0.07l0.08-0.09l0-0.07l-0.09,0l0-0.09l-0.16,0
				l-0.01-0.23l-0.08-0.08l-0.09-0.16l-0.17-0.31l-0.08-0.08l0-0.07l-0.09-0.16l0-0.09l-0.16,0.09l-0.32,0.08l-0.09-0.16l0.08-0.08
				l0.16-0.09l0.07-0.09l-0.08-0.07l-0.09,0l-0.08-0.16l-0.08,0.08l-0.16,0l-0.16-0.07l-0.33-0.24l-0.09-0.07l0,0.07l-0.07,0
				l-0.09,0l0.09,0.08l-0.08,0.08l-0.07,0l-0.16,0l-0.16,0l0.07-0.32l-0.08-0.24l-0.09,0l-0.09,0l-0.32,0.01l-0.08-0.32l0-0.16
				l-0.09-0.07l-0.17-0.48l-0.2-1.52l-0.13-1.52l0.07-0.49l0.23-0.65l0.22-0.64l0.39-0.58l0.15-0.32l0.47-0.42l0.24-0.17l0.4-0.1
				l0.16,0l0.16,0.17l0.16,0.07l0.08,0.08l0,0.07l-0.07,0l0,0.09l0.07,0l0,0.16l-0.07,0l-0.09,0l0,0.07l0.16,0l0,0.09l0.09,0.07
				l0,0.09l0.32-0.01l0.16-0.09l0.09,0.24l0.24,0.08l0.25,0.07l-0.07,0.09l0.32,0.15l0.16,0.15l0.07-0.16l0.16-0.16l0-0.09
				l0.17-0.01l-0.09-0.16l0.08-0.08l-0.09-0.24l0.08-0.16l0.33,0.22l0.07-0.16l0.16,0.08l0.17,0.31l0.34,0.23l0.24,0.07l0.08-0.08
				l0.07-0.25l-0.08-0.07l-0.16,0l-0.16,0l-0.09-0.08l0.08-0.09l-0.01-0.32l0.08,0.08l0.24-0.01l0.23-0.09l0.33-0.01l0.16-0.08
				l0-0.16l0.22-0.5l-0.08-0.23l-0.09-0.08l-0.08-0.07l-0.17-0.32l0.07-0.08l0-0.09l-0.08-0.07l0-0.09l0.16-0.08l0.16,0.07
				l0.24,0.15l0.09,0.08l0-0.09l0.16,0l0-0.16l-0.09-0.23l-0.16,0l-0.16-0.15l0.08-0.09l0.07-0.16l-0.08-0.08l0.07-0.16l-0.16,0
				l0.08-0.16l0.16-0.08l0.08,0.07l0.17,0.08l0-0.16l0.16-0.16l0.16-0.09l0-0.09l0.23-0.08l-0.08-0.16l0.24,0.15L662.28,79.45z
				 M652.82,107.42l0-0.07l0.09,0.07L652.82,107.42z M651.9,108.89l0.16-0.08l-0.24,0.08L651.9,108.89z"></path>
		</g>
	</g>
</g>
<g id="comunidad_valenciana">
	<g id="manises">
		<path id="manises_3_" class="st3 fase_0" d="M514.94,234.54c-0.13-0.01-0.58-0.04-0.55-0.19c0.56-1.9-7.79-0.43-7.35-2.15
			c-0.52-1.57-1.55-1.91-3.12-1.99c-1.23-0.13-2-1.28-3.14-0.51c-0.56-0.28-1.69-1.24-2.22-0.78c-1.74,1.18-3.18-0.68-4.92,0.49
			c-0.01,0.88-3.09-1.44-2.41,0.15c-0.03,0.67-0.07,1.34,0.15,2.04c1.08,1.9-0.43,3.59,0.1,5.57c0.08,1.09-0.94,2.83,0.04,3.25
			c0.88,0.74,2.68,0.74,3.22,1.81c0.3,0.52,0.17,1.04,0.26,1.49c0.21-0.5,0.41-0.08,0.75-0.08c1.59-0.23-0.16-0.89,1.4-0.44
			c-0.46,0.73,1.08,0.14,0.92,0.57c3.79-3.93,2.32-2.74,3.36-7.56c1.65,0.26,3.58,0.27,5.1,0.08c-0.27-0.57-0.34-1.13-0.62-1.77
			C506.67,235.26,514.61,235.98,514.94,234.54z"></path>
	</g>
	<g id="denia">
		<path id="denia_3_" class="st3 fase_1" d="M524.95,263.36l-0.09,0.09l0,0.19l-0.19,0.18l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.19,0.09
			l-0.09,0l-0.19,0.18l0,0.09l-0.28,0.18l-0.1,0.18l0.09,0.28l-0.09,0.09l-0.28,0.09l-0.75,0.17l-0.19,0l-0.19,0l-0.28-0.19l-0.09,0
			l-0.09-0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.57,0.92l-0.09,0.09l-0.1,0.18
			l-0.09,0.09l-0.09,0l-0.19,0.09l-0.19,0.09l-0.56,0.36l0,0.09l0.37,0.1l0.09,0.19l0.09,0.09l0.09,0.19l0.09,0.09l0.09,0.1
			l0.09,0.09l0,0.28l-0.09,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.19l0.09,0l0,0.09l0,0.09l-0.19,0.28l-0.09,0.09l0,0.09
			l0,0.09l0.09,0.09l0,0.09l0.09,0l0,0.1l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.1-0.19l0.09,0l0-0.09l0.1,0.09l0-0.09l0.09,0
			l0-0.09l0.09,0l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0.19,0l0.19,0l0.46,0.1l0.83,0.01l0.19,0l0.28,0.1l0.19,0l0.09,0
			l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.1l-0.09,0l0,0.09l-0.1,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0
			l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09
			l0.18,0.37l0.09,0l0.09,0.09l0,0.09l0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.1l-0.09,0.09l0.09,0l0,0.09l0.09,0.09l-0.09,0
			l0,0.09l-0.09,0l0,0.09l0,0.09l0.09,0.09l0.09,0l0.09,0l0.09,0.09l0.1,0l0.09,0l0.09,0.09l0.37,0.1l0.18,0.1l0.18,0.1l0.09,0
			l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0.09l0.09,0l0.09-0.09l0.37,0.01l0.18,0.1l0.09,0l0.28,0.1l0.09,0l0-0.19l0.09,0l0.09,0l0-0.19
			l-0.09-0.46l0.1-0.18l0-0.19l0-0.09l0-0.1l0-0.09l0.09,0l0-0.09l0.09-0.09l0.01-0.37l0.18,0.47l0.28,0l0.19,0l0.18,0.19l0.09,0
			l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.1-0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0
			l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0
			l0.09-0.09l0,0.09l0.09,0.09l0.09,0l0,0.1l0.09,0.09l0,0.09l0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09
			l0,0.09l-0.09,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.19l0,0.09l0,0.09
			l0,0.19l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.19,0l0.18,0.1l0.1-0.18l1.01,0.57l0.37,0.19
			l0.18,0.19l0.55,0.29l0,0.09l0.09,0.09l0.1,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09
			l0.09,0l0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0.09-0.09l0.09-0.1
			l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0.1-0.09l0,0.09l0.09,0l0.09,0.1
			l-0.09,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0
			l0-0.09l-0.09,0l0-0.1l0-0.09l-0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09
			l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l-0.09-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0
			l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.1l0.09,0l0-0.09
			l0.09,0l0-0.09l0.09-0.09l0-0.09l0.1,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0.09l0,0.09
			l0,0.09l0,0.09l0.09,0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0
			l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l-0.09-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0
			l0-0.09l0.09,0l0-0.1l0.09,0l0-0.09l0-0.09l0.09,0l0.1,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09
			l0-0.09l-0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0.09-0.09l0.09,0l0,0.09l0,0.09l0,0.09
			l0.09,0l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l-0.09,0
			l0-0.09l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.1l-0.09,0l0-0.09l0-0.09
			l0-0.09l0-0.09l-0.09,0l0,0.09l0,0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0
			l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.09-0.09l0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09
			l-0.1-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.1l0-0.09l0.09,0l0.1,0l0.09,0l0-0.09l0.09,0.09l0.09-0.09l0.09,0
			l0.09-0.09l-0.09-0.09l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.1,0l-0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0
			l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l0-0.09l0-0.09
			l-0.09,0l-0.09,0l-0.09-0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.1,0l0,0.09l-0.09,0l-0.09,0
			l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.09-0.1l0-0.09l-0.09-0.09l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0
			l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.1,0
			l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0
			l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.18-0.1l-0.09-0.09l-0.1,0.09l-0.09,0l-0.09,0
			l-0.09-0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0
			l-0.18-0.1l-0.09-0.09l-0.09-0.09l-0.18-0.1l-0.09-0.09l-0.09-0.1l-0.09-0.09l-0.1,0l0,0.09l0,0.1l0,0.09l0.1,0l0.09,0l0,0.09
			l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0.09l-0.09,0.09l-0.1,0.18l0,0.09l0.09,0l0,0.09l0,0.09l-0.09,0.09l-0.09,0l0,0.09
			l-0.09,0l0,0.09l-0.09,0l-0.1,0l-0.09,0l-0.09,0l0-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09-0.09l-0.19,0l0-0.09l-0.09,0l-0.09,0
			l0-0.09l-0.09-0.09l-0.09,0l-0.09-0.19l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.1l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09,0
			l-0.09,0l-0.09,0l0,0.1l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0.09,0
			l0,0.09l-0.09,0l-0.1,0l0-0.09l-0.09,0l-0.19,0l-0.19,0l-0.09,0.09l-0.28,0.09l-0.28,0.18l-0.09,0.09l-0.19,0l-0.09,0.09
			l-0.09,0.09l-0.28,0.09l-0.28,0.18l-0.09-0.09l-0.19-0.1l0-0.28l0-0.09l0-0.19l-0.09-0.28l-0.18-0.28L524.95,263.36z
			 M540.72,269.62l0.09-0.09l-0.09,0L540.72,269.62z M541.08,270.56l0.09,0l0-0.09l-0.09,0L541.08,270.56z M541.07,271.21l0,0.09
			l0.09,0l0-0.09L541.07,271.21z M536.06,265.28l0.18,0.1l-0.09-0.09L536.06,265.28z M536.42,265.47l0.09,0.28l0-0.28L536.42,265.47
			z"></path>
	</g>
	<g id="xativa-ontinyent">
		<path id="xativa" class="st3 fase_1" d="M519.47,257.71l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09
			l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.1,0l-0.09,0l-0.09-0.09l-0.18-0.1l0-0.09l-0.18-0.1l-0.93-0.01l-0.09,0l0-0.09
			l-0.09-0.1l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09-0.09l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l-0.1,0l-0.09,0l0.09-0.09l-0.09,0l0-0.09
			l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.19,0l0-0.09l-0.19,0l-0.09,0l0-0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0
			l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.1,0l-0.09,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09-0.09l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.18-0.1l-0.09,0l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.09l-0.09,0l0-0.09
			l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0.09l-0.09,0.09l-0.1,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l-0.09,0
			l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09-0.19l0-0.09l-0.09,0l0-0.09l0-0.09
			l0-0.09l0-0.09l-0.1,0l-0.09,0l0-0.1l-0.09,0l-0.09,0l0,0.1l0,0.09l0,0.09l0,0.09l0.09,0.09l0-0.09l0.09,0l0.09,0l-0.09,0.09
			l-0.19,0.09l-0.09,0.09l-0.28,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09
			l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l0,0.1l-0.09,0l-0.09,0l0,0.09l-0.09,0
			l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.19,0.09l0-0.09l0.09,0l0-0.1l0.09,0l0-0.09l-0.09-0.09l-0.18-0.1l-0.09-0.19l-0.09,0
			l-0.18-0.1l-0.19,0l-0.18-0.1l0-0.19l0.09-0.09l0-0.19l-0.09-0.09l-0.18-0.19l0.09-0.09l0-0.09l-0.09-0.19l0-0.28l-0.18-0.19
			l-0.18-0.19l-0.09-0.09l-0.27-0.28l-0.27-0.28l-0.09,0l-0.09-0.19l-0.09-0.19l-0.09,0l-0.09-0.19l0-0.09l0.1-0.37l-0.18-0.38
			l-0.09-0.19l-0.18-0.28l-0.27-0.28l-0.18-0.19l-0.36-0.38l-0.09-0.09l-0.09-0.09l-0.09-0.09l-0.18-0.19l-0.09-0.28l-0.18-0.19
			l-0.18-0.28l-0.55-0.38l-0.09,0.09l0-0.19l-0.09-0.09l-0.09-0.09l0-0.09l0.09-0.09l0,0.09l0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09
			l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.1l0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09
			l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0-0.09l0.19-0.09l0.19,0l0.09,0l0.09,0.09l0-0.09l0-0.09l-0.09-0.09l0-0.19
			l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l-0.1-0.09l-0.09,0.09l-0.09-0.09l0-0.1l0-0.09l0-0.09l-0.09-0.09
			l-0.09,0l-0.09,0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0
			l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09-0.09
			l-0.09-0.09l0-0.09l-0.1,0l0-0.09l-0.09,0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.1
			l0-0.09l-0.09,0l-0.09,0l-0.18-0.1l-0.09,0l-0.09,0l0,0.09l0,0.09l0,0.1l0,0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09-0.1l0-0.09
			l-0.09,0l0,0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0.09l0,0.1l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0
			l0,0.09l-0.1,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.19,0.28l-0.19,0.09l-0.1,0.28l-0.19,0.18l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09
			l-0.93,0.26l-0.01,0.47l0.18,0.19l0.09,0.37l0,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0l-0.19,0.09l0.36,0.65l0.09,0.19l0,0.09
			l0.09,0.09l0.09,0l0.27,0.37l-0.28,0.09l-0.19,0.09l-0.28,0l-0.19,0l-0.09,0l-0.09,0l-0.09,0l-0.19,0l-0.09,0l-0.1,0l-0.09,0
			l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0.09
			l-0.09,0.09l-0.09,0l0,0.09l-0.09,0.09l0.09,0l0,0.09l0.18,0.1l-0.01,0.65l-0.09,0.09l0,0.09l0,0.19l0.09,0.37l0,0.09l0,0.09
			l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.19l0,0.09l0.09,0l0,0.09l0.09,0.1l0.27,0.28l0,0.09l0,0.09l0,0.09l-0.09,0l0.09,0.09l0,0.09
			l0.09,0.09l0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09
			l0,0.19l0,0.1l0,0.09l0,0.09l0.09,0.19l0,0.09l-0.09,0.09l0.09,0.09l0.09,0.19l0,0.09l0.09,0.28l0.09,0l0,0.09l0.09,0.09l0,0.09
			l0,0.09l0.09,0.37l0.1,0.09l0.09,0l0,0.19l0.09,0l0.09,0.19l0,0.09l0.09,0.09l-0.09,0l-0.09,0l0.09,0.09l-0.09,0l0,0.09l-0.09,0
			l0,0.09l-0.09,0l0,0.09l0.09,0l-0.09,0.09l0,0.09l0,0.09l-0.1,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09
			l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.1l-0.09,0l-0.09,0.09l0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.19l-0.09,0l0,0.09l-0.09,0l0,0.09
			l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l-0.09,0.09
			l-0.09,0l-0.09,0.09l-0.19,0.18l-0.09,0l-0.56,0.18l-0.09,0l-0.09,0l-0.09,0.09l0.09,0.28l0,0.28l0,0.09l0,0.19l0,0.09l0.09,0
			l0,0.09l0,0.09l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0.19l0.09,0.19l0.27,0.47l0.09,0l0.28,0.19l0.28,0.19l-0.1,0.37
			l-0.19,0.28l-0.01,0.46l0,0.19l0.09,0.19l-0.19,0.37l-0.01,0.37l0.09,0.37l0.09,0.46l0,0.19l0.27,0.37l-0.1,0.37l-0.19,0.18
			l0,0.28l0,0.19l-0.1,0.46l0.27,0.37l0,0.19l0.37,0.1l0.19,0l-0.1,0.28l0,0.09l0.18,0.28l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0
			l0.09,0l0.09,0l0.19-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.19,0l0.09,0l0.09,0l0.19,0l0.28,0l0,0.09l0.09,0.09
			l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.19l0.09,0.19
			l0.09,0l0,0.09l0,0.09l0,0.19l0.09,0.19l0.09,0l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.19-0.09l0.09,0
			l0,0.1l0.09,0l0,0.09l0.09-0.09l0.09,0l0.09,0l0.19-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.1,0.09
			l0.09,0l0.19-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.09l0.09,0
			l0.09,0l0.19,0l-0.09-0.19l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09-0.09l0.09,0.09l-0.09,0.09l-0.09,0.09l0.09,0.09l0.09,0l0.09,0
			l0.09,0l0.1-0.09l0.09,0l0.18,0.1l0.09,0l0,0.09l0.18,0.28l0,0.09l0.09,0l0.09,0.19l0,0.09l0,0.1l0,0.09l0.09-0.09l0.28-0.09
			l0.09,0l0-0.09l0.28-0.18l0.09,0.09l0,0.09l0.09,0.09l0.09,0.1l0,0.09l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.1,0
			l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09
			l-0.09,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0.09
			l0,0.1l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.46,0.01l0.09-0.09l0-0.09l0.1,0l0.09-0.09l0.09-0.09l0.09-0.1l0.09,0
			l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.19-0.09l0-0.09l0.09,0
			l0.09,0l0.09,0l0.09,0l0-0.09l0.19,0l0.09-0.09l0.09,0l0.09,0l0-0.09l0.1,0l0.09,0l0.09,0l0-0.09l0.09-0.09l0.19-0.18l0.09-0.09
			l0.1-0.18l0-0.09l0.09,0l0-0.09l0.09,0l0-0.1l0.09,0l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09,0.09l0-0.19l0.09,0l0.09,0l0.09-0.09
			l0.09,0l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0-0.28l0-0.19l0-0.09l-0.28,0l-0.28,0l-0.09-0.09l-0.09-0.09l0-0.09l0-0.09
			l-0.18-0.19l-0.09,0l-0.09,0l0,0.09l-0.19,0l-0.09,0.09l0,0.09l-0.09,0l-0.19,0.09l-0.19,0.09l-0.09,0l-0.1,0.09l0,0.09
			l-0.09,0.09l-0.18-0.1l-0.09,0l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09
			l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.1l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0
			l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.19-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0
			l0.19,0l0.09-0.09l0.19-0.09l0.09,0l0.19,0l0.19-0.09l0.09,0l0.09,0l0.19-0.09l0.09-0.09l0.09,0l0.19-0.09l0.09,0l0.09,0
			l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.19,0l0.09,0l0.09,0l0.18,0.1l0.37,0.01l0.28,0l0.37,0.01l0.18,0.1l0.19-0.18l0.28-0.09
			l0-0.09l0.1-0.28l0.28-0.09l0.19-0.18l0.19,0l0,0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0.1-0.09l0.19-0.09l0.09,0l0.19-0.09
			l0.46-0.09l0.09-0.09l0.19,0l0.09-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.18,0.1
			l0-0.09l0.09,0.09l0.09,0l0.19-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.09,0
			l-0.09-0.09l0.09,0l0.09,0l0-0.1l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l-0.09-0.09l-0.09,0.09l0-0.09l0.09,0
			l0.09-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.19,0l0.09,0l0-0.09l0.1-0.09l0.19-0.09
			l0-0.09l0-0.09l0.09,0l0.1-0.18l0-0.09l0.09-0.09l-0.57,0.73l0.66-0.82l0.37-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09-0.1l-0.09,0
			l-0.09,0l-0.19,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l0-0.09l-0.09,0l0.1-0.37l0-0.28l0-0.1l0-0.19l0.09,0l0.09-0.09l0.19-0.28l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l-0.09,0
			l-0.09,0l-0.09,0.09l-0.09-0.09l0-0.09l0-0.09l0.09,0l0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.09
			l0-0.09l0-0.09l0-0.19l0-0.28l0-0.09l0.1-0.18l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09
			L519.47,257.71z M509.12,255.33l0,0.09l-0.1-0.09L509.12,255.33z"></path>
	</g>
	<g id="general_valencia">
		<path class="st3 fase_0" d="M508.47,241.59c-0.2,0-4.93,1.59-4.93,1.59l-1.14,2.21l-0.2,0.93l-1.56-1.05l-1.46-0.77l-1.02-0.75l2.64-3.21
			l0.24-3.71l0.38-0.64l3.18-0.16l1.83,0.14l-0.8-1.73l3.23,0.46l0.1-0.46l-0.37-0.19l-0.26-1.12l0.93,0.01l0.6-2.96l0.31-1.85
			l-0.82-1.04l0.6-0.39c0,0,0.05-0.3,1.09,0.52c1.04,0.82,2.9,3.2,2.9,3.2l-0.47,0.64l0.73,1.31l-0.12,1.3l-0.47,0.36l1.38,1.04
			l1.67,0.3l0.88,2.8l-2.25,1.36l-0.38,0.83l-1.11-0.2l0.08,0.93c0,0,0.4,0.88,0.15,0.88c-0.25,0-3.89-0.41-3.89-0.41"></path>
	</g>
	<g id="clinico-malvarrosa">
		<path id="valencia_clinico_malvarrosa_3_" class="st3 fase_0" d="M516.91,224.17l-0.47,0.27l0,0.09l-0.47,0.18l-0.56,0.27
			l0.27,0.56l0,0.09l0.18,0.28l0,0.09l0.09,0l0,0.28l-0.37-0.01l-0.09,0l-0.56,0.36l-0.09,0l-0.09,0.09l-0.09,0.09l0,0.09
			l-0.09,0.09l-0.09,0.09l0,0.09l-0.1,0.18l0,0.1l-0.1,0.28l-0.1,0.09l-0.1,0.18l-0.09,0.09l-0.1,0.18l-0.09,0.09l0-0.09l-0.09,0
			l-0.09-0.09l0,0.09l0.18,0.28l0.18,0.19l0,0.09l0.28,0.19l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0,0.09
			l-0.09,0.1l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09
			l0.09,0l0,0.09l-0.09,0.09l0.09,0l0.19,0l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0.09l-0.09,0l-0.09,0l0,0.09l0.09,0.09l0,0.09l0,0.19
			l0.09,0l0,0.09l0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09-0.09
			l0.09,0l0,0.09l0.1,0l0,0.09l-0.1,0.09l0,0.09l0,0.1l0.1,0.09l0,0.09l0.18,0.1l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0
			l0.09,0l0-0.09l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.1l0-0.09l0-0.19l0-0.09
			l0.1-0.18l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09
			l0-0.09l0.09-0.09l0-0.1l0.09-0.09l0-0.09l0.09-0.09l0.1-0.18l0.09,0l0-0.09l0-0.09l0.1,0l0-0.09l-0.1,0l0-0.09l-0.37-0.1l0-0.09
			l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.18-0.1l-0.09,0l0-0.09l0.09-0.09l0.09-0.09l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0.09
			l0.09,0l0.09,0.09l0.09,0l0.1,0l0,0.09l0.09,0l0.09,0l0.09,0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09
			l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0.09-0.1l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.1-0.18
			l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.19l0.09-0.09l0.1-0.18l0-0.09l0.09-0.1l0-0.09l-0.09,0.09l-0.09,0l-0.19,0.09l-0.73-0.48
			l-0.37-0.38l-0.09,0l-0.09,0l-0.09,0l-0.28-0.1l-0.74-0.29L516.91,224.17z M513.32,228.3l-0.28,0.09l0.09,0.19l-0.09,0.09
			l-0.19,0.18l-0.09,0.09l0,0.09l-0.09,0.09l0.09,0.09l0.09,0.28l0.09,0l0.36,0.57l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09
			l0.09,0.09l0,0.09l0.18,0.1l-0.09,0.09l-0.19,0.09l0,0.09l-0.09,0l0.09,0.09l0,0.09l0.09,0.09l0.18,0.1l0.09,0l0.09,0.09l0.1,0
			l0,0.09l0,0.09l0,0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0.09l0-0.09l0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09
			l0-0.09l0.09,0l-0.09-0.09l-0.09,0l0-0.09l0.09,0l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0.1-0.18l0-0.09l0.09,0l0-0.09l0-0.09
			l-0.09-0.09l-0.09,0l0-0.09l-0.19,0l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.1l0-0.09l0-0.09l-0.1,0l0-0.09l-0.09,0
			l0-0.09l-0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09l0-0.09
			L513.32,228.3z M514.78,230.27l0.09,0.09l0-0.09L514.78,230.27z"></path>
	</g>
	<g id="la_fe">
		<path class="st3 fase_0" d="M517.96,234.24l-0.19,0l0,0.19l0,0.09l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.19l0,0.09
			l0,0.09l0,0.09l0.09,0l0,0.28l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0.09,0.1l0,0.09l0,0.09l0,0.09
			l0,0.09l0.09,0.1l0,0.19l0.09,0.29l-0.74,0.08l0-0.09l-0.09,0l-0.09-0.1l0-0.19l0-0.28l0.01-0.38l0-0.1l0-0.28l-0.09,0l-0.09,0
			l-0.09,0.09l-0.19,0l-0.09,0l-0.09,0l0-0.09l-0.28-0.1l0-0.09l-0.09-0.1l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09-0.1l-0.09,0
			l-0.09-0.1l-0.09,0.09l-0.1,0.19l-0.09,0l-0.09-0.1l-0.09,0.09l-0.09,0l-0.09-0.1l0-0.09l0-0.09l0-0.19l-0.09,0l-0.19,0l-0.09,0
			l-0.09,0.09l-0.1,0l-0.09,0l-0.09,0l-0.09-0.19l0-0.09l-0.09,0l0-0.09l-0.09-0.1l0-0.09l0-0.09l-0.09,0l-0.09-0.1l0-0.09l0.09,0
			l0.09,0l0.09-0.09l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.1-0.29l0-0.19l0.09-0.09l0-0.09l-0.09,0l-0.19-0.1
			l0.09-0.09l0-0.19l-0.09,0l0-0.09l0.09,0l0.1,0l0,0.09l0.09,0l-0.09-0.29l0.09,0l0-0.09l-0.09-0.1l-0.1,0l-0.18-0.1l-0.09,0
			l-0.09-0.1l-0.09-0.1l0.09,0l-0.09-0.1l0.09,0l-0.09-0.19l-0.09-0.1l0-0.1l0-0.09l-0.09,0l0-0.28l0.09-0.09l0.19,0l0.18,0.1
			l0.09,0l0.09,0.1l0.1,0l0,0.09l0,0.09l0,0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0.1l0-0.09l0.09,0l0-0.09
			l-0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l-0.09-0.1l-0.09,0l0-0.09l0.09,0l-0.09-0.1l0-0.09l-0.09,0l0-0.09l0.1-0.19l0-0.09l0.09,0
			l0-0.09l0.09,0.1l0-0.09l-0.09,0l0-0.09l-0.09-0.1l-0.09,0l0-0.09l-0.19,0l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.1
			l0-0.09l0-0.09l-0.1,0l0-0.09l-0.09,0l0-0.09l-0.09-0.1l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.1l-0.09,0l0-0.09l-0.09,0
			l0-0.09l-0.09-0.1l0-0.09l0-0.09l0.28,0l0.18,0.29l0.18,0.19l0,0.09l0.28,0.19l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0.09,0
			l0,0.09l0,0.09l-0.09,0.1l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.1l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09
			l0.09,0l0-0.09l0.09,0l0,0.09l-0.09,0.09l0.09,0l0.19,0l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0.1l-0.09,0l-0.09,0l0,0.09l0.09,0.1
			l0,0.09l0,0.19l0.09,0l0,0.09l0.09,0.1l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09
			l0.09-0.09l0.09,0l0,0.09l0.1,0l0,0.09l-0.1,0.09l0,0.09l0,0.1l0.1,0.1l0,0.09l0.18,0.1l0.09,0l0.09,0l0.09,0.1l0.09,0l0.09,0
			l0.09,0l0.09,0l0-0.09l0.09,0.1l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.1l0,0.09l0,0.19
			l0,0.19l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0.09l0,0.09l0.09,0.1l0.09,0l0-0.09l0.09,0.1
			l-0.09,0.09l0.19,0l0,0.1l-0.09,0l-0.09,0l0,0.09l0.19,0L517.96,234.24z"></path>
	</g>
	<g id="elche-crevillent">
		<path id="elche_3_" class="st3 fase_0" d="M505.6,292.02l-0.19,0.18l-0.93,0.26l-0.28,0.09l-0.09,0.09l0,0.1l-0.09,0l-0.09,0.09l-0.1,0
			l-0.09,0l-0.09,0.09l-0.09,0l-0.19,0l-0.09,0l-0.19,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09l-0.37,0.27l-0.09,0
			l0,0.09l-0.19,0.09l-0.19,0.09l-0.28,0.09L501.3,294l-0.28,0.27l-0.09,0l-0.19,0.18l-0.09,0l0,0.09l-0.19,0l-0.09,0.09l-0.09,0
			l-0.19,0l-0.28,0.09l-0.09,0l-0.47,0.27l-0.28-0.19l-0.28,0l-0.09,0l-0.56,0.18l-0.09,0l-0.19,0l-0.19,0.09l-0.09,0l-0.47,0.18
			l-0.09,0l-0.1,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l0,0.09l-0.09,0
			l-0.09,0.09l-0.1,0.18l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l0.18,0.19l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0,0.1
			l0.09,0.09l0.09,0.19l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0.09,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09
			l0.09,0.19l0.09,0.09l0,0.09l0.09,0l0,0.19l0.1-0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0,0.1l0.09,0l0.09,0l0,0.09
			l0.09,0.09l0,0.09l0.09,0l0.09,0.19l0.09,0l0,0.09l0.09,0.09l0,0.09l0.36,0.75l0,0.09l0,0.19l0,0.09l0.09,0l0,0.28l0,0.09l0,0.09
			l0.09,0.09l0,0.09l0.09,0.09l-0.28,0.09l-0.09,0.09l-0.28,0.09l0.28,0.1l0.09,0l0.46-0.09l-0.09-0.19l0.19-0.09l0.09,0l0.09-0.09
			l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.1,0l0.19-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.28,0.1l0.09,0l0,0.1
			l0.37,0.19l0,0.09l0.09,0l0,0.09l0.28,0l0.09,0l0,0.09l0.09,0l1.47,1.32l0.56,0.01l0.09,0l0.09,0l0.38-0.09l0.19-0.09l0.19-0.09
			l0.09-0.09l0.28,0l0.37,0.01l0.09,0.19l0.18,0.56l0.19,0l0.09,0l0.09,0l0.28,0l0-0.1l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09
			l0.09-0.09l0-0.09l0-0.09l0-0.09l0.1,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09
			l0.09-0.09l0.09-0.09l0-0.09l0-0.09l0-0.1l0.09,0l0-0.19l0.09,0l0-0.09l0.19-0.18l0.09,0l0-0.09l0.09-0.09l0.09-0.09l0.09-0.09
			l0.09,0l0.09-0.09l0.09-0.09l0.19,0l0.19,0l0.09,0l0.09,0l0.09-0.09l0.28,0l0.28,0l0.09,0l0.09,0l0.09,0l-0.09,0.09l-0.1,0.18
			l0.19-0.18l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.19,0l0.19-0.09l0.09,0
			l0-0.09l0.09,0l0.09-0.09l0.1,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.19l0-0.09l0-0.09l-0.09-0.09l0-0.09l-0.1-0.09l0-0.09l0-0.09
			l0-0.09l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09l0-0.19l-0.09-0.09l0-0.09l0-0.19l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l-0.09,0l-0.09,0l-0.18-0.28l-0.09-0.37l0-0.1l0-0.09l-0.09,0l0-0.09l-0.09-0.28l-0.09-0.09l-0.18-0.28l-0.09,0l-0.09,0
			l-0.09,0.09l-0.09,0l-0.09,0.09l-0.19,0l-0.19,0l-0.28,0.09l0-0.09l-0.09,0l-0.1-0.09l-0.18-0.1l-0.09-0.09l-0.27-0.28l-0.64-0.57
			l-0.09-0.09l-0.27-0.38l-0.09,0l0-0.19l-0.18-0.1l-0.28-0.19l-0.19,0L505.6,292.02z M512.24,301.21l0,0.1l0,0.09l0.09,0l0-0.09
			l0.09,0l0-0.1l-0.09,0L512.24,301.21z M512.42,301.31l0,0.09l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0
			l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0L512.42,301.31z M513.16,301.51l0.09,0L513.16,301.51 M513.35,301.51l-0.09,0L513.35,301.51
			 M513.35,301.51l0.09,0L513.35,301.51 M512.98,301.5l0-0.09L512.98,301.5 M512.14,301.3l-0.09,0l0-0.09l0,0.09L512.14,301.3"></path>
	</g>
	<g id="elda">
		<path class="st3 fase_1" d="M505.73,289.33l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l0,0.09l-0.37,0.09l-0.09,0.09l0,0.09
			l0.18,0.65l0.18,0.47l0,0.09l0.09,0.37l0.09,0.19l0.09,0.09l-0.19,0.18l-0.93,0.26l-0.28,0.09l-0.09,0.09l0,0.1l-0.09,0
			l-0.09,0.09l-0.1,0l-0.09,0l-0.09,0.09l-0.09,0l-0.19,0l-0.09,0l-0.19,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09
			l-0.37,0.27l-0.09,0l0,0.09l-0.19,0.09l-0.19,0.09l-0.28,0.09L501.3,294l-0.28,0.27l-0.09,0l-0.19,0.18l-0.09,0l0,0.09l-0.19,0
			l-0.09,0.09l-0.09,0l-0.19,0l-0.28,0.09l-0.09,0l-0.47,0.27l-0.28-0.19l-0.28,0l-0.09,0l-0.56,0.18l-0.09,0l-0.19,0l-0.19,0.09
			l-0.09,0l-0.47,0.18l-0.09,0l-0.1,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0
			l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.1,0.18l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09-0.09l-0.09-0.09l0,0.09
			l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.1,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.1l-0.09,0
			l-0.19,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.1l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09-0.09
			l0,0.09l-0.09,0l-0.19,0l0-0.09l0.09,0l0.1-0.28l0.1-0.37l0.09,0l0-0.09l0.09-0.09l0-0.28l0.1-0.18l-0.09-0.09l-0.27-0.66
			l-0.18-0.47l-0.09-0.37l-0.27-0.84l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.19,0.09l-0.09,0.09l-0.09,0l-0.92-0.57l-0.18-0.1
			l-0.56-0.19l-0.46-0.2l-0.74-0.2l-0.46-0.01l-0.65-0.01l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09
			l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l0-0.09l0.09,0l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09
			l0-0.1l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09-0.09
			l-0.09,0l-0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.1-0.09l0.09-0.09l0.09-0.09l0.09,0l0-0.09
			l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.09-0.1l0.09-0.09l0.09-0.09l0.09-0.09l0-0.09l0.09,0l0.09-0.09
			l0.09-0.09l0-0.09l0.09,0l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09
			l0.1-0.09l0-0.09l0.09,0l0-0.09l0.09-0.09l-0.09-0.09l-0.09-0.19l0-0.09l-0.1-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09
			l-0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.1-0.09l0-0.09l0-0.09l0.09,0
			l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0
			l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.1l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0
			l0-0.09l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09l0-0.09l-0.1-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.09,0
			l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0.09,0l0.19,0l0-0.09l0.09,0l0.09,0l0.19,0l0.09-0.09
			l0.46,0.1l0.28,0.1l0.37,0.01l0.09,0l0.37-0.09l0.28,0l0.19-0.18l0.09-0.1l0.19-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.09-0.09
			l0.09-0.09l0.19-0.18l0.09-0.09l0.09-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.19-0.18l0.1-0.18l0-0.09l0.09,0
			l-0.18-0.1l0-0.09l-0.18-0.1l-0.09-0.19l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.19l-0.09-0.09l0-0.09l-0.09,0l0-0.09
			l-0.09-0.09l-0.09,0l0-0.09l0-0.19l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l0-0.09l0-0.09l-0.09-0.1l0-0.09l-0.09-0.09l0-0.19
			l-0.09-0.09l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09-0.09
			l0.09,0l0.28,0l0.1-0.09l0.19,0l0-0.09l0.09,0l0.09,0l0.28-0.19l0.19-0.09l0.09,0l0.37,0.1l0.19,0l-0.1,0.28l0,0.09l0.18,0.28
			l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.19-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.19,0l0.09,0
			l0.09,0l0.19,0l0.28,0l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0
			l0,0.09l0.09,0.09l0.09,0.19l0.09,0.19l0.09,0l0,0.09l0,0.09l0,0.19l0.09,0.19l0.09,0l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.09,0
			l0-0.09l0.09,0l0.09,0l0.19-0.09l0.09,0l0,0.1l0.09,0l0,0.09l0.09-0.09l0.09,0l0.09,0l0.19-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09
			l0.09-0.09l0.09-0.09l0.09-0.09l0.1,0.09l0.09,0l0.19-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0.09,0
			l0-0.09l0-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.19,0l0.09,0.09l0,0.09l0.09,0.09l0,0.09l0.18,0.28l0,0.09l0.09,0.09l0.09,0.28
			l0.09,0.28l0.1,0l0,0.09l0,0.09l0.09,0.09l0.09,0.19l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0.09,0.09l0.18,0.37
			l0.09,0.19l0.09,0.1l0.09,0.19l0.09,0l0.09,0.28l0.09,0.09l0.09,0.09l0.09,0.19l0,0.09l0.09,0.09l0,0.09l0.18,0.37l0,0.09
			l0.18,0.37l-0.19,0.19l-0.19,0.18l-0.09,0.09l-0.1,0.28l-0.1,0.18l-0.1,0.18l-0.28,0.09l-0.28,0.18l-0.09,0.09l-0.1,0.18
			l-0.09,0.09l-0.19,0.37l0,0.19l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.19l0,0.09l-0.09,0.09l-0.1,0.28l0,0.09l-0.09,0
			l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0
			l-0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.19,0.09l-0.09,0l0,0.1l0.09,0.09l0.82,0.94l0.09,0.09l0,0.09l0.19,0l0.37,0.19
			l0.09,0.09l0.19,0.1l0.18,0.1l0.18,0.1l0.18,0.1l0.46,0.38l0.27,0.28l0.28,0.1l0.18,0.1l0.09,0.09l0.28,0.19l0.1,0l0.37,0.19
			l0.27,0.28l0.09,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0.28l0,0.1l0,0.09l0,0.09l0,0.09l0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09
			l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.19,0.09l-0.19,0.09
			l-0.1,0.09l-0.28,0.09l-0.75,0.36l0.72,1.4l0.09,0.09l0.09,0.09l0.28,0.28l0.03,0l0.06,0l0.18,0.19l0.27,0.29l0.09,0.09l0.19,0
			l0.46,0.19L505.73,289.33z"></path>
	</g>
	<g id="marina_baixa">
		<path id="vila_joiosa" class="st3 fase_1" d="M525.77,271.26l-0.01,0.37l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.1l0,0.09l0,0.19
			l-0.1,0.18l0.09,0.46l0,0.19l-0.09,0l-0.09,0l0,0.19l-0.09,0l-0.28-0.1l-0.09,0l-0.18-0.1l-0.37-0.01l-0.09,0.09l-0.09,0
			l-0.09-0.09l0,0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.18-0.1l-0.18-0.1l-0.37-0.1l-0.09-0.09l-0.09,0l-0.1,0l-0.09-0.09l-0.09,0
			l-0.09,0l-0.18-0.1l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09
			l-0.09,0l-0.09,0l-0.94,0.36l-0.09,0l-0.37,0.18l-0.09,0.09l-0.09,0.09l-0.47,0.27l0,0.28l0,0.09l0,0.09l-0.01,0.47l0.09,0
			l0.09,0.09l0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.19,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09
			l-0.09,0.09l0,0.09l0.09,0.19l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l-0.09,0.1l0,0.09l-0.09,0.09l0,0.09l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09-0.09l-0.09,0l0,0.19l-0.1,0.09l0,0.09l0,0.09l0,0.09l0.1,0.09
			l0,0.09l-0.1,0.09l-0.09,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0l-0.09,0.09
			l0,0.09l-0.09,0l0,0.1l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.19,0.09l-0.09,0.09l-0.09,0l0,0.09
			l0,0.09l0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.1,0.09l-0.19,0.18l0,0.09l-0.09,0.09l-0.09,0.09
			l-0.19,0.09l-0.09,0l0,0.09l-0.09,0l0.09,0.09l0.09,0.19l0.09,0.1l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0.09l0,0.09
			l0,0.09l-0.09,0l0,0.09l0.09,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0
			l0,0.09l0.09,0l0.1-0.09l0.09-0.09l0.18,0.1l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0
			l0,0.09l0.09,0.19l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.1l0.09,0.09l0.09,0l0,0.09l0.09,0l0,0.09
			l0.1,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l-0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0.09l0-0.09l0.09,0l0,0.09l0,0.09l0,0.09
			l0.09,0l0,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09
			l0,0.09l0,0.09l0,0.1l0.09,0.09l0,0.09l0.19-0.09l0.09-0.09l0.09-0.1l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.1-0.09l0.09,0
			l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09,0l0.19-0.09l0.09,0l0.09,0l0.09,0
			l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.1,0l0.09,0l0-0.09l0.09,0l0.09,0
			l0,0.09l0.09,0l0.09-0.09l0-0.09l0.09,0l0.19-0.09l0.19-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0.09,0
			l0-0.09l0.09,0l0-0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0-0.1l0.1,0l0,0.1l0.09,0l0-0.1l0.09,0
			l0-0.09l-0.09,0l0-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0
			l0.09,0l0,0.09l0.09,0l0.09,0l0.09-0.09l0.09,0.09l0.09-0.09l0.09,0.09l0.09,0l0.09,0l0.19,0l0.09,0l0,0.09l0,0.09l0,0.09l0.1,0
			l0,0.09l0.09,0l0.09,0l0.09,0l0,0.09l0,0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09
			l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0-0.09l-0.09,0l0-0.09l0.09,0
			l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0-0.1l0.09,0l0-0.09l0.1-0.09l-0.1,0l0-0.09
			l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l0-0.19l0-0.09l0-0.09l0.09-0.09l0.1-0.18l0-0.09
			l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.1,0l0.09-0.1l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.09
			l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.19-0.09l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.19,0
			l0.09-0.09l0.09,0l0-0.09l0.19,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0-0.09l0.09,0l0-0.09l0.1-0.09l-0.1,0l-0.09-0.09l0-0.09
			l-0.55-0.29l-0.18-0.19l-0.37-0.19l-1.01-0.57l-0.1,0.18l-0.18-0.1l-0.19,0l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l-0.09-0.09
			l0-0.09l-0.09-0.09l0-0.09l0-0.19l0-0.09l0-0.09l0-0.19l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09
			l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l0-0.1
			l-0.09,0l-0.09-0.09l0-0.09l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l0,0.09
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.1,0.09l-0.09,0
			l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.18-0.19l-0.19,0l-0.28,0L525.77,271.26z
			 M529.55,278.83l0,0.09l0.09,0L529.55,278.83z M522.99,283.28l-0.09,0l0,0.09L522.99,283.28z M530.96,277.37l-0.09,0.09l0.09,0
			L530.96,277.37z M528.76,281.69l0.09,0L528.76,281.69z M526.51,283.42l-0.09,0.09l0.09,0l0,0.09l0.09-0.09L526.51,283.42z"></path>
	</g>
	<g id="general_alicante">
		<path id="general_alinate_3_" class="st3 fase_0" d="M513.71,285.88c0-0.26-0.25-0.39-0.12-0.65c-0.38-0.13-0.9-0.27-1.02-0.4
			c-0.86-1.28-2.12-1.44-3.38-1.35c0.02,0,0.03,0,0.05,0.01c-0.02,0-0.03-0.01-0.05-0.01c-0.03,0-0.05,0-0.08,0.01
			c0.03,0,0.05,0,0.08-0.01c-0.05-0.01-0.11-0.01-0.16-0.01l0.05,0.06l0,0.09l0,0.09l0,0.19l0,0.09l-0.1,0.18l0,0.09l-0.1,0.19
			l-0.1,0.28l0,0.09l-0.56,0.18l-0.09,0.09l-0.28,0.09l-0.19,0.09l-0.19,0l-0.09,0l-0.74-0.1l-0.18-0.1l-0.09-0.09l0-0.09l0-0.09
			l-0.1,0l0-0.09l-0.09,0l-0.09,0l0-0.19l-0.09-0.28l-0.09-0.28l0-0.09l-0.09,0l0,0.09l-0.09,0.09l-0.19,0.09l-0.09,0l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l0,0.1l0,0.09l0,0.09l0,0.09l0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09
			l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.19,0.09l-0.19,0.09l-0.1,0.09l-0.28,0.09
			l-0.75,0.36l0.72,1.4l0.09,0.09l0.09,0.09l0.28,0.28l0.09,0l0.18,0.19l0.27,0.29l0.09,0.09l0.19,0l0.46,0.19l0.37,0.28l0,0.09
			l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l0,0.09l-0.37,0.09l-0.09,0.09l0,0.09l0.18,0.65l0.18,0.47l0,0.09l0.09,0.37
			l0.09,0.19l0.09,0.09l0.55,0.1l0.19,0l0.28,0.19l0.18,0.1l0,0.19l0.09,0l0.27,0.38l0.09,0.09l0.64,0.57l0.27,0.28l0.09,0.09
			l0.18,0.1l0.1,0.09l0.09,0l0,0.09l0.28-0.09l0.19,0l0.19,0l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.18,0.28l0.09,0.09
			l0.09,0.28l0,0.09l0.09,0l0,0.09l0,0.1l0.09,0.37l0.18,0.28l0.09,0l0.09,0l0-0.09l0-0.19l0-0.09l0-0.09l0-0.09l0-0.09l0-0.1
			l0-0.09l0-0.09l-0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0
			l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.1-0.09l0-0.09l0-0.09l0.09-0.09l0-0.1l0.09,0l0-0.09l0.09-0.09l0-0.09
			l0.09-0.09l0.09,0l0.19,0l0,0.09l0.19,0l0.19-0.09l0.09-0.09l-0.09,0l-0.09-0.09l0.09-0.09l0.19-0.09l0.09,0.09l0,0.09l0,0.09
			l-0.19,0.28l0.28-0.18l0-0.09l-0.09-0.28l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0.09,0l0-0.09
			l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0-0.09l0.1,0l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0
			l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0.09,0l0.09,0l0-0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0-0.09
			l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.09l0.09-0.09l0-0.09l-0.09-0.1
			l0.09-0.09l-0.09-0.09l0.09-0.09l0-0.09l-0.09-0.09l0.1-0.18l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.19l0.09-0.09l0-0.09
			l0-0.09l0-0.19l0.09-0.1l0-0.09l0-0.19l0.09,0l0.09,0l0.1,0l0-0.09l0.09,0l0.09-0.09l0-0.09l0-0.09l-0.09-0.09l0.09,0l0-0.09
			l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.03
			c-0.03-0.02-0.06-0.04-0.09-0.06l-0.01,0l0,0C515.54,286.4,514.78,285.7,513.71,285.88z"></path>
	</g>
	<g id="alicante-sant_joan">
		<path id="sant_joan_3_" class="st3 fase_0" d="M519.35,284.53l0-0.1l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09
			l-0.09-0.09l0,0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09,0l0,0.09
			l-0.09-0.09l0-0.09l-0.09,0l-0.09-0.09l0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.1-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0
			l-0.09-0.09l-0.09-0.1l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.19l0-0.09l-0.09,0l-0.09,0l-0.09,0
			l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.18-0.1l-0.09,0.09l-0.1,0.09l-0.09,0l0-0.09l-0.09,0
			l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l-0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09
			l-0.09-0.09l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.1l-0.09-0.19l-0.09-0.09l0.09,0l0-0.09l0.09,0l0.19-0.09
			l0.09-0.09l0.09-0.09l0-0.09l0.19-0.18l0.1-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09-0.09l0.09,0l-0.09-0.09l0-0.09l0-0.09
			l0.09,0l0.09-0.09l0.19-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0.09-0.09l0-0.1l0.09,0l0-0.09l0.09-0.09l0.09,0
			l0-0.09l-0.09,0l0-0.09l-0.09,0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0
			l-0.09-0.09l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.1,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09
			l0-0.09l-0.09-0.09l-0.09-0.09l-0.19,0l-0.09,0l0-0.09l-0.09-0.09l0.09,0l0-0.09l-0.09,0l-0.09,0l-0.18-0.1l0-0.09l-0.09,0.09
			l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l0-0.09l-0.09,0l-0.1,0.28l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.1,0l-0.19,0
			l0-0.09l-0.09-0.09l-0.09-0.09l-0.09-0.19l-0.09,0l0-0.19l-0.18-0.19l-0.09,0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0.09l0,0.19l0,0.19l0,0.09l0,0.09l0,0.09l-0.09,0
			l-0.1,0.18l0,0.09l-0.09,0.09l-0.19,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.1,0.09l-0.19,0.19l-0.09,0l0,0.09l-0.09,0
			l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.19,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0
			l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.19,0.09l-0.1,0l-0.09,0l-0.19,0l-0.09,0l-0.09,0.09l0,0.09
			l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0.09,0.09l0.1,0l0.09,0l0.09,0l-0.09,0.09l0,0.09l0.09,0.19l0,0.09l-0.09,0l0,0.09l0,0.09
			l-0.09,0.09l0,0.1l0.09,0l0.09,0.28l0.09,0l0,0.09l0,0.09l0.09,0.19l0,0.19l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09
			l0.09,0l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09-0.09l0,0.09l0,0.09l-0.1,0.19l0,0.09l0,0.09l0,0.09l-0.19,0.28
			l-0.09,0.09l-0.1,0.18l-0.09,0.09l0,0.09l0,0.09l0.04,0.04c0.05,0,0.11,0,0.16,0.01c1.26-0.1,2.52,0.07,3.38,1.35
			c0.13,0.13,0.64,0.27,1.02,0.4c-0.13,0.26,0.12,0.39,0.12,0.65c1.07-0.18,1.84,0.52,2.64,1.1l0.09-0.09l0-0.09l0.09,0l0-0.09
			l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09-0.1l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0
			l0.09,0l0-0.09l0.09-0.09l0.1-0.09l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0
			l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0-0.09L519.35,284.53z"></path>
	</g>
	<g id="torrevieja">
		<path id="torrevieja_3_" class="st3 fase_1" d="M492.71,310.3l0.81,1.31l0.18,0.19l0.45,1.21l0.09,0.09l0.82,1.03l0.36,0.38l1.46,1.6
			l0.09,0.09l0.09,0l0.09,0.09l0,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0.09l0.18,0.1
			l0,0.09l0.46,0.1l0.09,0l0.18,0.1l0.09,0l0.1,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0.09,0
			l0.09,0.1l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0.37-0.18l0-0.09l0-0.1l0-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l0.1,0l0-0.09
			l0-0.09l-0.1,0l0-0.09l0.1-0.09l0-0.09l0-0.09l0-0.09l0-0.1l0-0.19l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09
			l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0
			l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.1l0-0.09l0.09,0l0-0.09l-0.09,0
			l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.1l0.09,0l0-0.09l0.1,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0
			l0.09,0l0,0.09l0.09-0.09l0.09,0.09l0,0.09l0,0.09l0.1-0.18l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09-0.09l0-0.09
			l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l-0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09
			l0.09,0l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.1l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l0.09,0
			l0-0.09l-0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l-0.09-0.03l0-0.07l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.19
			l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.1l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.19l0-0.09l0-0.09l0-0.09l-0.28,0l-0.09,0l-0.09,0l-0.19,0l-0.18-0.56
			l-0.09-0.19l-0.37-0.01l-0.28,0l-0.09,0.09l-0.19,0.09l-0.19,0.09l-0.38,0.09l-0.09,0l-0.09,0l-0.56-0.01l-1.47-1.32l-0.09,0
			l0-0.09l-0.09,0l-0.05,0c-0.85,0.82-1.5,2.02-1.96,3.1c-0.13,0.38-1.04,0.37-1.56,0.75c-0.52,0.64-0.16,1.8-0.94,2.43
			c-0.91,0.63-1.66-0.8-2.57-0.3c-1.03,0.37-1.18,1.51-1.32,2.54c0.04-0.04,0.08-0.07,0.13-0.11c-0.04,0.03-0.09,0.07-0.13,0.11
			c0,0.01,0,0.01,0,0.02c0-0.01,0-0.01,0-0.02c-0.01,0.01-0.02,0.02-0.03,0.03l0.05,0.14L492.71,310.3z"></path>
	</g>
	<g id="alcoi">
		<path class="st3 fase_1" d="M523.56,269.55l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.1l-0.09,0l0,0.09l-0.1,0l-0.09,0
			l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0.09
			l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.18,0.37l0.09,0l0.09,0.09l0,0.09l0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.1
			l-0.09,0.09l0.09,0l0,0.09l0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0.09,0.09l-0.18-0.1l-0.09,0l-0.09,0l0-0.09l-0.09,0
			l-0.09,0l-0.09,0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.94,0.36l-0.09,0l-0.37,0.18
			l-0.09,0.09l-0.09,0.09l-0.47,0.27l0,0.28l0,0.09l0,0.09l-0.01,0.47l0.09,0l0.09,0.09l0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09
			l-0.19,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l0.09,0.19l0,0.09l0.09,0.09l0,0.09l0,0.09
			l0.09,0.09l0,0.09l-0.09,0.1l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0.09l-0.09,0
			l-0.09-0.09l-0.09,0l0,0.19l-0.1,0.09l0,0.09l0,0.09l0,0.09l0.1,0.09l0,0.09l-0.1,0.09l-0.09,0.09l-0.09,0l-0.09,0.09l0,0.09
			l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0-0.09l-0.09,0l0-0.09l-0.09,0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09-0.09l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09
			l-0.1,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l0-0.09l-0.09-0.09l-0.09-0.09l-0.19,0l-0.09,0l0-0.09l-0.09-0.09l0.09,0l0-0.09
			l-0.09,0l-0.09,0l-0.18-0.1l0-0.09l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l0-0.09l-0.09,0l-0.1,0.28l-0.09,0
			l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.1,0l-0.19,0l0-0.09l-0.09-0.09l-0.09-0.09l-0.09-0.19l-0.09,0l0-0.19l-0.18-0.19
			l-0.09,0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0
			l-0.09,0.09l0,0.19l0,0.19l0,0.09l0,0.09l0,0.09l-0.09,0l-0.1,0.18l0,0.09l-0.09,0.09l-0.19,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0
			l0,0.09l-0.1,0.09l-0.19,0.19l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.19,0.09
			l-0.09,0l0,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.19,0.09
			l-0.1,0l-0.09,0l-0.19,0l-0.09,0l-0.09,0.09l0,0.09l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0.09,0.09l0.1,0l0.09,0l0.09,0l-0.09,0.09
			l0,0.09l0.09,0.19l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.1l0.09,0l0.09,0.28l0.09,0l0,0.09l0,0.09l0.09,0.19l0,0.19l0,0.09
			l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09-0.09l0,0.09l0,0.09
			l-0.1,0.19l0,0.09l0,0.09l0,0.09l-0.19,0.27l-0.09,0.09l-0.1,0.18l-0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.19
			l0,0.09l-0.1,0.18l0,0.09l-0.1,0.19l-0.1,0.28l0,0.09l-0.56,0.18l-0.09,0.09l-0.28,0.09l-0.19,0.09l-0.19,0l-0.09,0l-0.74-0.1
			l-0.18-0.1l-0.09-0.09l0-0.09l0-0.09l-0.1,0l0-0.09l-0.09,0l-0.09,0l0-0.19l-0.09-0.28l-0.09-0.28l0-0.09l-0.09,0l0,0.09
			l-0.09,0.09l-0.19,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09-0.28l0-0.09l0-0.09l-0.09-0.09l-0.09-0.09
			l-0.27-0.28l-0.37-0.19l-0.1,0l-0.27-0.19l-0.09-0.09l-0.18-0.1l-0.28-0.1l-0.27-0.28l-0.46-0.38l-0.18-0.1l-0.18-0.1l-0.18-0.1
			l-0.19-0.1l-0.09-0.09l-0.37-0.19l-0.19,0l0-0.09l-0.09-0.09l-0.82-0.94l-0.09-0.09l0-0.1l0.09,0l0.19-0.09l0-0.09l0.09,0
			l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09-0.09
			l0.09-0.09l0.09,0l0-0.09l0.1-0.28l0.09-0.09l0-0.09l0-0.19l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.19l0.19-0.37l0.09-0.09
			l0.1-0.18l0.09-0.09l0.28-0.18l0.28-0.09l0.1-0.18l0.1-0.18l0.1-0.28l0.09-0.09l0.19-0.18l0.19-0.19l-0.18-0.37l0-0.09l-0.18-0.37
			l0-0.09l-0.09-0.09l0-0.09l-0.09-0.19l-0.09-0.09l-0.09-0.09l-0.09-0.28l-0.09,0l-0.09-0.19l-0.09-0.1l-0.09-0.19l-0.18-0.37
			l-0.09-0.09l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0L502,272.3l-0.09-0.09l0-0.09l0-0.09l-0.1,0l-0.09-0.28
			l-0.09-0.28l-0.09-0.09l0-0.09l-0.18-0.28l0-0.09l-0.09-0.09l0-0.09l-0.09-0.09l-0.09-0.19l0.09,0l0.09,0l0.09,0.09l0.09,0
			l0.09-0.09l0.09,0.09l-0.09,0.09l-0.09,0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0.1-0.09l0.09,0l0.18,0.1l0.09,0l0,0.09l0.18,0.28
			l0,0.09l0.09,0l0.09,0.19l0,0.09l0,0.1l0,0.09l0.09-0.09l0.28-0.09l0.09,0l0-0.09l0.28-0.18l0.09,0.09l0,0.09l0.09,0.09l0.09,0.1
			l0,0.09l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.1,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0
			l0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09
			l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0,0.1l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.46,0.01
			l0.09-0.09l0-0.09l0.1,0l0.09-0.09l0.09-0.09l0.09-0.1l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0
			l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.19-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.19,0l0.09-0.09l0.09,0l0.09,0l0-0.09
			l0.1,0l0.09,0l0.09,0l0-0.09l0.09-0.09l0.19-0.18l0.09-0.09l0.1-0.18l0-0.09l0.09,0l0-0.09l0.09,0l0-0.1l0.09,0l0.09-0.09l0.09,0
			l0.09,0l0-0.09l0.09,0.09l0-0.19l0.09,0l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0-0.28l0-0.19l0-0.09
			l-0.28,0l-0.28,0l-0.09-0.09l-0.09-0.09l0-0.09l0-0.09l-0.18-0.19l-0.09,0l-0.09,0l0,0.09l-0.19,0l-0.09,0.09l0,0.09l-0.09,0
			l-0.19,0.09l-0.19,0.09l-0.09,0l-0.1,0.09l0,0.09l-0.09,0.09l-0.18-0.1l-0.09,0l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09
			l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.1l-0.09,0l0-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.19-0.09l0.09,0
			l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.19,0l0.09-0.09l0.19-0.09l0.09,0l0.19,0l0.19-0.09l0.09,0l0.09,0
			l0.19-0.09l0.09-0.09l0.09,0l0.19-0.09l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.19,0l0.09,0l0.09,0l0.18,0.1l0.37,0.01
			l0.28,0l0.37,0.01l0.18,0.1l0.19-0.18l0.28-0.09l0-0.09l0.1-0.28l0.28-0.09l0.19-0.18l0.19,0l0,0.09l0.09-0.09l0-0.09l0-0.09
			l0.09,0l0.1-0.09l0.19-0.09l0.09,0l0.19-0.09l0.46-0.09l0.09-0.09l0.19,0l0.09-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0.09,0.09
			l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.18,0.1l0-0.09l0.09,0.09l0.09,0l0.19-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0
			l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.09,0l-0.09-0.09l0.09,0l0.09,0l0-0.1l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09
			l0.09,0l-0.09-0.09l0.09-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.19,0l0.09,0l0-0.09
			l0.1-0.09l0.19-0.09l0-0.09l0-0.09l0.09,0l0.1-0.18l0-0.09l0.09-0.09l0-0.09l0.09,0l0.37-0.09l0,0.09l0.09,0.09l0.27,0.28
			l0.09,0.09l0.18,0.1l0.09,0.09l0.09,0l0.18,0.19l0.09,0l0.46,0.19l0,0.09l-0.57,0.92l-0.09,0.09l-0.1,0.18l-0.09,0.09l-0.09,0
			l-0.19,0.09l-0.19,0.09l-0.56,0.36l0,0.09l0.37,0.1l0.09,0.19l0.09,0.09l0.09,0.19l0.09,0.09l0.09,0.1l0.09,0.09l0,0.28
			l-0.09,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.19l0.09,0l0,0.09l0,0.09l-0.19,0.28l-0.09,0.09l0,0.09l0,0.09l0.09,0.09
			l0,0.09l0.09,0l0,0.1l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.1-0.19l0.09,0l0-0.09l0.1,0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0
			l0.09-0.09l0.09,0l0-0.09l0.09,0l0.19,0l0.19,0l0.46,0.1l0.83,0.01l0.19,0l0.28,0.1l0.19,0L523.56,269.55z"></path>
	</g>
	<g id="orihuela">
		<path id="orihuela_3_" class="st3 fase_1" d="M493.93,307.49c0.91-0.5,1.66,0.93,2.57,0.3c0.78-0.63,0.41-1.8,0.94-2.43
			c0.52-0.38,1.42-0.37,1.56-0.75c0.46-1.07,1.11-2.27,1.96-3.1l-0.21,0l-0.02,0l0-0.09l-0.09,0l0-0.09l-0.37-0.19l0-0.1l-0.09,0
			l-0.28-0.1l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.19,0.09l-0.1,0l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09
			l-0.09,0.09l-0.09,0l-0.19,0.09l0.09,0.19l-0.46,0.09l-0.09,0l-0.28-0.1l0.28-0.09l0.09-0.09l0.28-0.09l-0.09-0.09l0-0.09
			l-0.09-0.09l0-0.09l0-0.09l0-0.28l-0.09,0l0-0.09l0-0.19l0-0.09l-0.36-0.75l0-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09-0.19l-0.09,0
			l0-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09,0l0-0.1l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.1,0.09l0-0.19l-0.09,0
			l0-0.09l-0.09-0.09l-0.09-0.19l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09
			l-0.09-0.09l-0.09-0.19l-0.09-0.09l0-0.1l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.09l-0.18-0.19l-0.09-0.09l-0.09-0.09l0,0.09l-0.09,0
			l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.1,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.1l-0.09,0l-0.19,0
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.1l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09-0.09l0,0.09
			l-0.09,0l-0.19,0l0-0.09l0.09,0l0.1-0.28l0.1-0.37l0.09,0l0-0.09l0.09-0.09l0-0.28l0.1-0.18l-0.09-0.09l-0.27-0.66l-0.18-0.47
			l-0.09-0.37l-0.27-0.84l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.19,0.09l-0.09,0.09l-0.09,0l0.17,0.74l0.36,0.57l0.18,0.28
			l0.09,0.28l0.08,0.56l-0.01,0.46l-0.01,0.56l-0.19,0.28l-0.1,0.28l-0.1,0.37l-0.2,0.74l-0.2,0.83l0,0.19l0,0.28l-0.09,0.09
			l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l-0.19,0.56l-0.19,0.27l-0.57,0.83
			l-0.1,0.18l-0.09,0.09l-0.38,0.36l0,0.09l0.09,0.38l0.17,0.74l0,0.28l0,0.09l0,0.09l-0.01,0.56l0.18,0.37l0,0.09l0.09,0.19l0,0.09
			l0,0.19l0.09,0.19l0.09,0.09l0,0.09l0.09,0.19l0.09,0.19l0.18,0.19l0.09,0.09l0.09,0.09l0,0.1l0.09,0l0.09,0.09l0.09,0l0,0.09
			l0.09,0l0.09,0.19l0.09,0.19l0.18,0.19l0.18,0.28l0,0.09l0.18,0.19l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0l0.1,0.09
			l0.09,0.09l0.18,0.38l0.09,0.19l0.13,0.33c0.01-0.01,0.02-0.02,0.03-0.03C492.75,309.01,492.9,307.86,493.93,307.49z"></path>
	</g>
	<g id="la_ribera">
		<path id="alzira_3_" class="st3 fase_0" d="M503.91,242.96c-0.07,0.01-0.13,0.02-0.2,0.02c-0.02,0.01-0.05,0.02-0.07,0.03l-0.09,0.18
			l-0.1,0.37l-0.19,0.18l-0.09,0.09l-0.1,0.28l-0.47,0.18l-0.19,0.09l-0.01,0.37l-0.01,0.65l-0.2,0.46l0,0.09l-0.19,0.18l0,0.09
			l0,0.1l0.09,0.09l0.09-0.09l0.1,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0.09l0,0.19l0.09,0.09l0,0.09l0,0.09
			l-0.09-0.09l-0.09,0l-0.19,0l-0.19,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.09
			l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.1l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09
			l-0.09,0l0-0.09l-0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0,0.19l0.09-0.09l0.55,0.38l0.18,0.28l0.18,0.19l0.09,0.28l0.18,0.19
			l0.09,0.09l0.09,0.09l0.09,0.09l0.36,0.38l0.18,0.19l0.27,0.28l0.18,0.28l0.09,0.19l0.18,0.38l-0.1,0.37l0,0.09l0.09,0.19l0.09,0
			l0.09,0.19l0.09,0.19l0.09,0l0.27,0.28l0.27,0.28l0.09,0.09l0.18,0.19l0.18,0.19l0,0.28l0.09,0.19l0,0.09l-0.09,0.09l0.18,0.19
			l0.09,0.09l0,0.19l-0.09,0.09l0,0.19l0.18,0.1l0.19,0l0.18,0.1l0.09,0l0.09,0.19l0.18,0.1l0.09,0.09l0,0.09l-0.09,0l0,0.1l-0.09,0
			l0,0.09l0.19-0.09l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0-0.1l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09-0.09
			l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0
			l0.28-0.09l0.09-0.09l0.19-0.09l0.09-0.09l-0.09,0l-0.09,0l0,0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.1l0.09,0l0.09,0l0,0.1
			l0.09,0l0.1,0l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.19l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.09-0.09l0,0.09l0.09,0
			l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0,0.09
			l0.09,0l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0.1,0l0.09-0.09l0.09-0.09l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0
			l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0l0.18,0.1l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0
			l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.09l0.1,0l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09-0.09l0.09,0
			l0,0.09l0.09,0l0.19,0l0,0.09l0.19,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l-0.92-0.2
			l0,0.19l-0.09-0.09l-0.09-0.09l-0.18-0.28l-0.46-0.19l-0.37-0.01l0.1-0.28l0-0.09l0.01-0.46l0-0.09l0-0.09l0.09,0l0.1,0l0.09,0.09
			l0.19,0l0.19,0l0.09,0.09l0.18,0.19l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.19-0.09
			l0.18,0.1l-0.09-0.37l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0.1-0.18l0.19-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.28,0.19l0.09,0
			l0.37,0.01l0.37,0.28l0.37,0.01l-0.09-0.28l0-0.09l0.09,0l0-0.19l0.1-0.18l0.09-0.09l0.1-0.37l0.09,0l0.28-0.18l0.09,0l0.09-0.09
			l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0
			l0-0.09l0.09,0l0-0.09l0.09,0l0.28-0.18l0.18,0.19l0.28,0.1l0,0.09l0.37,0.19l0.28,0.19l0.28,0.1l0,0.09l0.19,0l0,0.09l0.09-0.09
			l0.09,0l0.19,0l0-0.09l0.09,0l0.09,0l-0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09
			l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09-0.1l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09
			l-0.09-0.09l0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0.09-0.09l0-0.09l0-0.09l0-0.1
			l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0,0.09l0.09,0l0-0.09l0-0.09l0.09,0.09l0.09,0l0-0.09
			l0.09,0l0,0.09l0.09,0l0-0.09l0-0.09l0.09,0l-0.09-0.09l0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l-0.09-0.28l-0.09,0
			l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09-0.1l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09-0.09l-0.09-0.19l0-0.09
			l-0.09,0l0-0.09l-0.09-0.09l0-0.09l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l-0.1,0l0-0.09l-0.09-0.19l0-0.09l-0.09,0l0-0.09l0-0.09
			l-0.09-0.09l0-0.1l-0.09-0.09l0-0.09l0-0.09l-0.09-0.19l-0.09-0.09l0-0.09l-0.09-0.19l-0.09,0l0-0.09l-0.09-0.09l0-0.09l0-0.09
			l-0.09-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.18-0.1l-0.09,0l-0.09-0.09l-0.09,0l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.27-0.29
			l0-0.09l-0.09-0.09l0-0.09l-0.19-0.1l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.28l-0.09,0l0-0.09l-0.09,0l-0.19,0.09
			l-0.09,0l-0.09-0.09l0-0.09l-0.09-0.09l-0.1,0.18l0-0.09l-0.46,0.09l-0.1,0.18l-0.09,0l-0.09,0l-0.19,0.28l-0.09,0l-0.1,0.09
			l-0.09-0.09l0-0.09l-0.19,0l-0.19,0.09l0-0.09l-0.09-0.09l-0.09,0l0.09-0.09l0.09,0l-0.27-0.28l-0.09,0l0,0.09l-0.19,0l-0.09,0
			l0-0.19l0-0.09l0-0.09l-0.09,0l-0.09-0.19l-0.09,0l-0.19,0.09l-0.09,0l-0.09-0.09l-0.19,0l-0.09-0.09l-0.09,0l-0.19,0l-0.18-0.1
			l0,0.09l0,0.09l0,0.09l0,0.19l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09
			l-0.09,0.09l-0.09,0l-0.37,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.37-0.01l-0.28,0l-0.19,0l-0.09,0l-0.28,0l-0.19,0l-0.09,0
			l-0.09,0l-0.18-0.1l-0.09,0l-0.09,0l-0.27-0.28l0-0.09l-0.09,0l-0.27-0.28l-0.09,0l-0.18-0.1l0-0.09l-0.09,0l-0.18-0.1l-0.56-0.1
			l-0.03-0.03c-1.72,0.85-3.63,1.56-5.41,2.25C503.77,242.97,503.84,242.97,503.91,242.96z"></path>
	</g>
	<g id="requena">
		<path class="st3 fase_1" d="M498.34,243.85l0,0.09l-0.09,0l-0.09,0.09l0,0.1l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0
			l0,0.09l-0.09,0l0,0.09l-0.1,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.19,0.28l-0.19,0.09l-0.1,0.28l-0.19,0.18l-0.09,0.09l-0.09,0
			l-0.09,0l-0.09,0.09l-0.93,0.26l-0.01,0.47l0.18,0.19l0.09,0.37l0,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0l-0.19,0.09l0.36,0.65
			l0.09,0.19l0,0.09l0.09,0.09l0.09,0l0.27,0.37l-0.28,0.09l-0.19,0.09l-0.28,0l-0.19,0l-0.09,0l-0.09,0l-0.09,0l-0.19,0l-0.09,0
			l-0.1,0l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0
			l-0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0.09l0.09,0l0,0.09l0.18,0.1l-0.01,0.65l-0.09,0.09l0,0.09l0,0.19l0.09,0.37l0,0.09
			l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.19l0,0.09l0.09,0l0,0.09l0.09,0.1l0.27,0.28l0,0.09l0,0.09l0,0.09l-0.09,0l0.09,0.09
			l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09
			l0,0.09l0,0.19l0,0.1l0,0.09l0,0.09l0.09,0.19l0,0.09l-0.09,0.09l0.09,0.09l0.09,0.19l0,0.09l0.09,0.28l0.09,0l0,0.09l0.09,0.09
			l0,0.09l0,0.09l0.09,0.37l0.1,0.09l0.09,0l0,0.19l0.09,0l0.09,0.19l0,0.09l0.09,0.09l-0.09,0l-0.09,0l0.09,0.09l-0.09,0l0,0.09
			l-0.09,0l0,0.09l-0.09,0l0,0.09l0.09,0l-0.09,0.09l0,0.09l0,0.09l-0.1,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09
			l0,0.09l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.1l-0.09,0l-0.09,0.09l0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.19l-0.09,0l0,0.09l-0.09,0
			l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.09
			l-0.09,0.09l-0.09,0l-0.09,0.09l-0.19,0.18l-0.09,0l-0.56,0.18l-0.09,0l-0.09,0l-0.09,0.09l-0.19,0l-0.28,0.09l-0.19,0l-0.09,0
			l-0.09-0.1l-0.09-0.09l-0.18-0.1l-0.19,0l-0.28-0.1l-0.09,0l-0.09,0.09l-0.09,0l-0.1,0.09l-0.1,0.19l-0.09,0.09l0,0.09l-0.19,0.09
			l-0.19,0l-0.09,0.09l-0.09,0l-0.09,0l-0.28,0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.46,0.09l-0.28,0l-0.28,0.09l-0.09,0l-0.28,0
			l-0.37,0.09l-0.09,0l-0.19,0l-0.19,0.09l-0.09,0l-0.28,0.09l-0.28,0l-0.09,0l-0.19,0l-0.19,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0
			l-0.09,0l-0.09,0.09l-0.37-0.1l0-0.09l-0.09-0.09l-0.18-0.37l-0.09,0l0-0.19l0-0.09l-0.09,0l0-0.1l0-0.09l-0.09-0.19l-0.09-0.09
			l0-0.09l-0.18-0.19l-0.36-0.38l-0.27-0.47l-0.18-0.19l-0.09-0.09l0-0.09l-0.09-0.09l-0.09-0.19l-0.09,0l-0.09-0.09l-0.18-0.1
			l0-0.09l-0.09-0.28l-0.09,0l-0.09-0.28l-0.18-0.28l-0.09-0.09l-0.45-0.66l-0.09,0l-0.19-0.1l-0.27-0.19l-0.27-0.19l-0.18-0.28
			l-0.09,0l-0.28-0.1l-0.18-0.19l-0.09-0.37l0.09,0l-0.09-0.19l0.1-0.55l0-0.09l-0.09-0.38l0.38-0.46l0.02-1.2l0.65-0.36l0.1-0.37
			l0-0.19l0.19-0.46l0.1-0.28l0.11-0.74l0.66-0.55l0.19-0.18l0.09-0.09l0.09-0.09l0.19-0.37l0-0.09l0.09-0.09l0-0.28l0.09-0.09
			l0-0.1l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0.19-0.37l0.1-0.65l0-0.19l0.09-0.1l0.01-0.37l0-0.09
			l-0.09-0.19l0-0.09l0-0.09l0-0.09l0-0.09l0-0.28l0-0.19l0-0.09l0-0.28l0-0.09l0.09,0l0-0.09l0-0.19l0.01-0.37l0.29-0.74l0-0.09
			l0.1-0.28l0-0.09l0.09-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.1l-0.09,0l-0.1-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09-0.09l0-0.09l-0.09,0l-0.09-0.09l0.09-0.09
			l-0.09,0l0-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.09
			l-0.09,0l-0.1-0.09l0-0.09l-0.09,0l-0.09,0l-0.28-0.1l-0.09-0.09l-0.09,0.09l-0.09,0l0-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09,0
			l0,0.09l0,0.19l-0.09,0l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0
			l0-0.09l-0.09-0.09l0-0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.1,0l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0
			l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l0.09-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0
			l-0.09,0l-0.09-0.09l0-0.09l-0.09,0l0,0.09l-0.09,0l0-0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0
			l0,0.09l-0.09,0l-0.09,0l0,0.09l0.09,0l0.09,0l0,0.09l0,0.09l-0.09,0l0-0.09l-0.09,0.09l0,0.09l-0.09,0l0-0.09l-0.1,0l0-0.09
			l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l-0.09-0.19l0-0.09l-0.09,0l-0.09,0l0-0.09
			l0-0.09l-0.09,0l0-0.1l-0.09,0l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09
			l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0.09
			l-0.1,0l-0.09,0l-0.09,0l-0.18-0.1l-0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.1-0.09
			l0-0.09l-0.1,0l0-0.09l-0.09,0l-0.09,0l-0.1,0.18l-0.09,0.09l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.09l-0.09-0.09l-0.09,0l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.28l0,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0-0.09
			l-0.09,0l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l-0.09,0l-0.09,0l0,0.09l0.09,0.09l0,0.09l-0.09-0.09
			l-0.09,0l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l-0.19,0l-0.19,0l0-0.09l0-0.09l0.1,0
			l0.09,0l0-0.09l0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0.09-0.09l0.09,0l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.1,0
			l-0.09-0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0.09l0.09,0
			l0.09,0.09l0,0.09l0.09,0l0.1,0l0-0.09l0-0.09l-0.1,0l-0.09,0l0-0.09l0-0.09l0.09,0l0.1,0l0-0.09l-0.1,0l0-0.09l0.1-0.1l0-0.09
			l0-0.09l-0.1-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l0,0.09l0,0.09l0,0.1l-0.09,0.09l0,0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0
			l-0.09-0.09l0-0.1l0-0.09l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09-0.09
			l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l0.09,0l-0.09-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09
			l-0.09,0.09l0,0.09l0,0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0.09,0l0.09-0.09l0-0.09
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l0-0.09l-0.1,0l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0.09l0-0.09l-0.09,0
			l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.1l0.09,0l0-0.09l-0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0
			l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l-0.09,0l-0.09-0.09l0,0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.09l0.09,0
			l0.09-0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.09l0.09-0.1
			l0-0.09l0.1,0l0.09,0l0.09,0l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.1,0l-0.09,0l0-0.09l0-0.09l0.09-0.09l0.1,0l0.09,0l0-0.09
			l-0.09-0.09l-0.1,0l-0.09,0l0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0.09,0.09l0.1,0.09l0.09,0.09l0.09,0l0-0.09l0.09-0.09l0-0.09
			l0-0.09l0.09-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l-0.09-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0
			l0-0.1l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0.09,0l0.09,0l0-0.09
			l0-0.09l-0.09-0.09l-0.09-0.09l0-0.09l0.09-0.09l0.09-0.09l-0.09-0.09l0-0.09l-0.09,0.09l-0.1,0l-0.09,0.09l-0.09-0.09l-0.09-0.09
			l-0.09,0l-0.09,0l0-0.09l0.09,0l0-0.09l0.09,0.09l0.09-0.09l0.09,0l0.09,0l0.1,0l0.09,0l0.09,0l0-0.09l-0.09-0.09l0.09-0.1l0-0.09
			l0-0.09l-0.09-0.09l0-0.09l0.28-0.09l0.28-0.09l0.19,0l0.09-0.09l0.19,0l0.09,0l0.19-0.09l0.09,0l0.28-0.27l0-0.09l0.09,0l0-0.09
			l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.1-0.28l0.1-0.37l0.2-0.65l0.19-0.37l0.19-0.55l0.38-0.83l0.29-0.55l0.19-0.09l0.38-0.46
			l0.09,0l0-0.09l0.09,0l0.09-0.09l0.19-0.18l0.38-0.28l0.66-0.64l0.09,0.09l0.18,0.1l0.09,0.09l0.65,0.19l0.09,0l0.09,0.09
			l0.28,0.19l0.19,0.1l0.18,0.1l0.09,0l0.28,0.1l0.19,0l0.37-0.09l0.19,0l0.09-0.09l0.47-0.27l0.19-0.09l0.19-0.27l0.1-0.28
			l0.28-0.46l-0.27-0.47l-0.09-0.28l-0.09-0.19l0-0.09l0.11-0.65l-0.1,0l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09
			l0.1-0.09l0.1-0.18l0.09,0l0-0.09l0.1-0.28l0.09-0.09l0.47-0.73l0.28-0.37l0.09-0.09l0.19-0.09l0.28-0.46l0.1-0.19l0.19-0.46
			l0.09-0.09l0.1-0.18l0-0.09l0-0.09l-0.09-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.19l0-0.19l0.1-0.18l0.28,0.1l0,0.28
			l0.09,0l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.18,0.1l0.09,0l0.18,0.1l0,0.09l0.09,0.09
			l0.09,0.09l0.09,0.19l0.09,0.09l0,0.09l0.18,0.38l0.09,0.19l0.09,0.09l0,0.09l0.28,0.1l0.09,0.09l0.28,0.19l-0.09,0.09l-0.09,0.09
			l-0.1,0.09l-0.1,0.18l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.28l0,0.1l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.19
			l0.27,0.47l0.09,0.46l0,0.28l0,0.09l-0.09,0.09l0,0.28l0,0.28l0,0.09l-0.01,0.37l0,0.28l0,0.09l0,0.09l-0.1,0.28l0,0.09l-0.09,0
			l0.09,0.09l0.19,0l0,0.09l0,0.09l0,0.1l0,0.09l0,0.09l0.09,0l0,0.09l0.1,0l0.09,0.09l0.18,0.1l0.09,0.09l0.09,0.09l0.09,0.09
			l0.18,0.19l0.18,0.19l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0,0.09l0.18,0.19l0.09,0l0.09,0.09l0.09,0.09
			l0,0.09l0.18,0.19l0.09,0.19l0.1,0l0,0.09l0.09,0.09l0.09,0.09l0.27,0.19l0.09,0l0.09,0l0.09,0l0.09,0l0.18,0.1l0.09,0l0.18,0.1
			l0.09,0l0.09,0l0.18,0.1l0.09,0.09l0.09,0l0.09,0.09l0.09,0.09l0.09,0.09l0.09,0.09l0.1,0.09l0.09,0l0.18,0.19l0.18,0.19
			l0.09,0.09l0.09,0l0,0.1l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0.09-0.09l0.1-0.28l0-0.19l0.09-0.09l0-0.19
			l0.1-0.18l0.19-0.18l0-0.09l0.1-0.18l0.09,0l0.09-0.09l0.09-0.09l0-0.09l0.1-0.18l0.1,0l0.19-0.09l0.1-0.18l0.19,0l0-0.09l0.09,0
			l0.09-0.1l0.09,0l0.09,0l0.09-0.09l0.45,0.66l0.09,0l0,0.09l0.09,0l0,0.09l0.45,0.56l0.18,0.37l0,0.28l0,0.19l0.19,0.19l0,0.09
			l-0.1,0.09l0.1,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09
			l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l-0.09,0l-0.19,0l-0.09,0l-0.19,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0
			l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.1,0.09l-0.09,0.09l-0.09,0l0,0.09l0.09,0.09l0,0.19l0,0.09l0.09,0.09l-0.09,0
			l-0.09-0.09l-0.09,0l-0.19,0l-0.09,0l-0.09,0l-0.09-0.09l-0.28-0.1l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.37-0.1l-0.09,0
			l-0.09,0l-0.09,0l0.09,0.09l0,0.09l0,0.19l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l-0.1,0.18l0,0.09l0.09,0l0,0.09l0,0.19l0,0.09
			l0,0.09l0,0.09l0.09,0.19l0,0.09l0,0.19l-0.09,0.09l0,0.09l0.18,0.38l0.09,0.19l0.27,0.74l0,0.09l0.09,0.46l-0.01,0.37l-0.1,0.28
			l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l-0.1,0.28l-0.1,0.28l-0.1,0.37l0,0.1l0,0.19l0,0.09
			l0.09,0.28l0.08,1.02l-0.1,0.37l0.09,0.09l-0.1,0.65l0,0.09l-0.19,0.65l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09
			l0,0.09l0,0.09l-0.09,0l0,0.1l0.09,0.09l0,0.09l0,0.09l0,0.19l0.09,0l0.19,0l0.09,0l0,0.09l0.18,0.1l0.09,0l0,0.09l0.09,0l0.09,0
			l0,0.09l0.09,0l0.09,0l0.09,0.09l0.18,0.1l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0.09l0.1,0l0.18,0.1l0.09,0.09
			l0.09,0l0.09,0.09l0.09,0l0.09,0l0.55,0.19l0.28,0l0,0.09l0.18,0.47l0.09,0.28l0,0.09l0.09,0.09l0.09,0.19l0,0.19l-0.1,0.28
			l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09-0.09l0-0.09l0.09-0.09l0.09,0l0.1,0.09l0.09,0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0
			l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0.09,0l0.09,0
			l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.18,0.1
			l0.1,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09L498.34,243.85z"></path>
	</g>
	<g id="as_castellon">
		<path id="castellon" class="st3 fase_0" d="M540.37,193.9c-2.86-2.85-3.07,0.29-3.98-0.8c1.02-1.02,0.49-2.73,0.97-3.84
			c-0.49-1.3-0.82-1.11-0.09-2.34c-0.68-0.81-0.48-0.57-0.49-1.34c-0.15-0.77-1.22-0.41-1.47-0.76c-0.01-0.52-0.71-0.36-0.72-1.18
			c-0.3-0.74-1.08-0.3-0.26-1.16c-0.11-0.43,0.19-0.62,0.29-0.92c-0.23-0.23-0.8-0.07-0.83-0.57c-0.15,0.09-0.33-0.73-0.08-0.83
			c-0.11-0.47-0.38-0.91-0.54-1.4c-0.32-0.25,0.55-0.84,0.39-1.2c0.06,0.06,0.13-0.04,0.1-0.09c-0.24-0.16,0.07-0.17,0.01-0.37
			c-0.03-0.09,0.09-0.22,0.19-0.18c-0.1-0.02-0.15-0.25,0-0.19c-0.56-1.1-0.94,0.22-1.5,0.63c-0.24-0.46-0.97-0.03-1.39-0.11
			c-1.49-0.11-0.28-0.04-1.22,1.19c-0.3-0.36-0.16,0.59-0.29,0.74c-0.54,0.55-0.65,1.44-1.05,2.12c-0.23,0.2,0.38,2.63-0.22,2.03
			c-0.57-1.05-0.26-2.55-1.16-3.26c-0.22-0.16,0.22-0.48-0.18-0.47c-0.03-0.18-0.05-0.44-0.27-0.47c0.18-0.89-1.46-0.89-1.57-0.3
			c-0.6-0.83-1.53,1.3-1.88,1.74c-0.81,0.65-2.42,0.26-3.43-0.14c0.27-0.2-0.18,0.02-0.09-0.19c-0.13,0.18-0.21,0.21-0.38,0.27
			c0.04-0.68-0.18,0.11-0.28-0.1c-0.1,0.08-0.38,0.05-0.37,0.27c-0.15-0.05-0.2,0.01-0.46-0.01c-0.03,0.12,0.15,0.08,0.09,0.28
			c-0.04-0.08-0.29-0.17-0.09,0c-0.11,0.06-0.21-0.21-0.19,0c-0.02,0.01-0.14-0.2-0.09,0c-0.1,0.02-0.32-0.07-0.28,0.09
			c-0.44,0.14-0.61,0.21-1.03,0.36c-0.13,0.2-0.02,0.42-0.29,0.55c0.32,0.37-0.2-0.01-0.1,0.18c0.83,1.48,2.6,2.48,2.73,3.85
			c-0.17-0.02-0.27,0.05-0.1,0.18c-0.19,0.09-0.38-0.23-0.37-0.01c-0.26-0.04-0.23,0.48-0.47,0.18c-0.32,0.08-0.07,0.29-0.47,0.18
			c-0.15,0.18-0.17,0.48-0.38,0.55c-0.01,0.09-0.09,0.18-0.19,0.18c-0.23-0.03,0.05,0.27-0.28,0.18c0,0.34-0.2-0.08-0.19,0.18
			c-0.24,0.01,0.03,0.09,0,0.19c-0.24,0.01,0.03,0.09,0,0.19c-0.05,0.27-0.12,0.5-0.28,0.18c-0.18-0.06,0.19,0.51-0.1,0.46
			c-0.16,0.01-0.07,0.56-0.47,0.27c0.1-0.2-0.36,0.16-0.28,0.27c0.37,0.05-0.17,0.44-0.09,0.09c0-0.25-0.09,0.16-0.19,0.09
			c-0.01-0.09-0.08-0.18-0.18-0.19c0.04,0.15-0.14,0.03-0.1,0.18c-0.14,0.04-0.04-0.13-0.18-0.1c0.05,0.09-0.04,0.1-0.09,0.09
			c0.17,0.09-0.05,0.59-0.1,0.28c-0.15-0.03-0.1,0.13,0,0.19c-0.21-0.07-0.45,0.23-0.1,0.18c-0.07,0.29,0.32,0.2,0.36,0.47
			c0.02,0.09-0.06,0.22,0.09,0.19c0.33,1.71-1.58,0.09-2.96,2.07c-0.13,0.25-1.2,0.25-1.25,0.56c0.35,0.13-1.6-0.46-1.39-0.21
			c0.37,0.25-0.96-0.87-0.92-0.38c-0.09,1.42-0.93,3.03-1.16,4.38c0.03,0.2,1.04,0.56,2.39,0.45c2.11-0.17,2.99-0.3,3.4,0.38
			c0.34,0.57,1.28,1.64,2.84,0.44c0.68-0.53,0.44,0.54,1.08,1.45c0.49,0.69,0.17,1.81,0.98,2.02c0.35-0.24-0.06,0.49,0.53,0.39
			c0.16,0.18,0.33,0.2,0.27,0.47c0.91,0.29,1.73,1,2.76,1.25c0.46,0.16,0.61,0.06,0.74,0.57c-0.07,0.4,0.2-0.06,0.37,0.01
			c0.43-0.17,0.89-0.52,1.49-0.35c0.07,0.3,0.26,0.54,0.08,0.84c0.68,0.64,1.55,0.92,0.35,1.58c-0.22,0.41,0.71,0.22,0.83,0.57
			c0.62,0.2,0.61-0.22,1.2,0.39c1.35-0.02,1.72,1.66,3.13,1.81c1.55,1.22,0.66-2.27,1.62-2.76c0.34-0.32,0.47-1.71,0.96-2.49
			c0.09-1.04,0.98-1.34,1.51-1.93c0.97-0.16,1.5-0.12,2.05-0.9c0.14-0.17,0.06-0.63,0.47-0.64c0,0.25,0.26-0.03,0.19-0.18
			c-0.28,0.02,0.09-0.45-0.09-0.38c0.21-0.42,0.27-0.82,0.48-1.2c0.33-1.25,0.88-2.88,2.19-3.77
			C539.54,194.07,540.68,194.64,540.37,193.9z"></path>
	</g>
	<g id="gandia">
		<path id="gandia_3_" class="st3 fase_1" d="M520.12,251.13c-0.52,0.31-0.74,0.5-0.85,1.01c-0.45,0.43-1.51,0.95-1.24,1.84
			c-0.59,0.07-1.29-0.78-1.76-0.21c-0.97,0.31-2.38,0.19-2.61,0.33c-1.14,1.89,3.63,2.89,4.78,3.51c0.1,0.36-0.15,0.89,0.26,1.02
			c0.04,0.35,0.58-0.45,0.65,0.01c0.18,0.17-0.01,0.78-0.2,0.93c0.34,0.3-0.29,0.47-0.01,0.74c1.03-0.4-3.1,3.42-3.97,4.86
			c-0.7,1.16,0.72,1.45,2.97-0.42c0.19-0.16,0.72-1.16,1.49-0.53c1.1,0.8,2.49,1.16,3.79,0.7c-0.08-1.13,1.16-0.76,1.51-1.56
			c0.25,2.85,1.25,0.49,2.68,0.78c0.12,0.04,0.32-0.08,0.1-0.09c0.07-0.09,0.06-0.28,0.19-0.28c0.09-0.25,0.08,0.27,0.47-0.27
			c0.41-0.15,0.37,0.11,0.46,0.47c0.51,0.2,0.76,0.91,1.29,0.95c-0.02-0.1,0.53-0.36,0.19-0.46c0.42-0.69-0.46-0.48-0.36-1.12
			c-1.87-1.47-3.44-3.54-4.75-5.17c-0.02-0.27-0.02-0.22,0.01-0.46c-0.18,0.18-0.62,0.27-0.19,0c-0.35-0.53-0.69-1.33-1.18-1.87
			c-0.59-1.29-1.08-2.21-1.53-3.46C522.4,251.87,520.85,252.1,520.12,251.13z M525.22,257.7c0.02-0.16-0.03-0.04-0.09,0
			C525.13,257.7,525.22,257.7,525.22,257.7z"></path>
	</g>
	<g id="dr__peset">
		<path id="valencia_dr_peset" class="st3 fase_0" d="M517.36,237.54l0.09,0.28l0.09,0.19l0.09,0.38l0,0.19l-0.09-0.19l-0.09,0
			l-0.09-0.28l-0.09,0l-0.19,0.09l0.09,0.28l-0.09,0l-0.09-0.19l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0
			l-0.09,0.09l-0.1,0.09l0,0.09l-0.28-0.1l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0.09l0,0.09
			l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.1l0.09,0l0,0.09l-0.09,0
			l0,0.19l0.09-0.09l0,0.19l0.09,0.19l0.09,0l0,0.09l0,0.09l0,0.19l0.09,0l0.19,0l0-0.09l0.09,0l0.27,0.28l-0.09,0l-0.09,0.09
			l0.09,0l0.09,0.09l0,0.09l0.19-0.09l0.19,0l0,0.09l0.09,0.09l0.1-0.09l0.09,0l0.19-0.28l0.09,0l0.09,0l0.1-0.18l0.46-0.09l0,0.09
			l0.1-0.18l0.09,0.09l0,0.09l0.09,0.09l0.09,0l0.19-0.09l0.09,0l0,0.09l0.09,0l0,0.28l0,0.09l0,0.09l0.09,0l0.09,0l0.09,0
			l0.09,0.09l0.19,0.1l0,0.09l0.09,0.09l0,0.09l0.27,0.29l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.18,0.1l0.09,0
			l0.09,0l0.09,0l-0.09-0.09l0-0.09l-0.09-0.09l-0.18-0.28l0-0.09l0-0.1l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l0-0.09
			l-0.09-0.28l-0.09,0l-0.18-0.1l0.09,0l0.18,0.1l0-0.09l-0.09-0.09l0-0.09l-0.09-0.19l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09
			l-0.09-0.09l0-0.09l0-0.09l-0.09-0.19l0-0.09l-0.1-0.09l0-0.09l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09
			l-0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09-0.19l0-0.09l0-0.09l-0.09-0.09l0-0.1l0-0.09l0-0.09l-0.09,0l0-0.09
			l0-0.19l-0.09-0.09l0-0.09l0-0.09l0-0.09l-0.08,0c-0.18,0.02-0.37,0.07-0.54,0.14l0.06,0.13L517.36,237.54z"></path>
	</g>
	<g id="arnau_de_vilanova">
		<path class="st3 fase_0" d="M513.7,233.96c0.88,0.1-2.25-1.36-1.57-0.69c-1.15,0.7-1.85-0.62-2.69,0.12c-2.97,0.07-1.73-0.09-3.03-2.33
			c-0.82-1.21-3.3-0.59-4.52-1.64c-1.63,0.4-1.26-0.15-2.77-0.51c-0.43-0.61-1.6,0.96-2.33,0.43c-1.25-0.88-2.65-0.03-3.53,0.23
			c-1.06-1.13,1.44-0.88,1.13-1.47c-0.28-0.03-0.56-0.13-0.74-0.29c-0.21-0.05-0.28-0.21-0.46-0.28c0.29-0.18-0.19-0.33-0.08-0.56
			c-1.19-3.49-2.48-1.36-3.44,0.32c-0.93-0.65-1.96-1.56-3.22-1.81c-1.51-1.95-3.48-2.16-2.62-5.05c-1-2.58,0.85-2.62,0.05-3.24
			c-0.71-0.36-0.7-1.61-1.55-1.79c-0.02-0.54-0.93-1.08-0.34-1.77c0.36-0.95,0.23-2.84,0.24-3.71c-0.11-0.33-0.35-0.3-0.46-0.56
			c0.23-0.12-0.09-0.61,0.2-1.02c1.26-0.07,2.62-2.02,3.35-0.6c-0.25-0.32,0.37-0.91,0.47-0.46c0.22-0.43,0.27-0.4,0.56,0.01
			c0.93-0.76,1.45-0.78,2.88-0.33c0.88-0.65,1.55-0.29,2.32,0.03c0.6,0.25,1.27,0.97,1.75,0.67c0.18-0.41,0.09,0.03,0.28,0
			c0.02,0.63,0.4,0.62,0.45,1.03c0.22,1.07-0.04,0.38-0.29,1.01c0.1,0.45,0.02,1.22,0.45,1.21c-0.18,1.24-0.97,1.18,0.52,2.33
			c1.74,0.47,0.63,0.3,2.33-0.8c2.07-1.64,2.37,0.39,3.5,1.63c0.1,0.4,0.4,0.14,0.27,0.75c1.04,1.22,1.15-0.61,1.69-1.46
			c1.49-0.18,1.88,1.63,2.09,3c-0.11,0.88-0.67,2.27,0.62,2.33c2.86,0.85,1.29-0.88,2.91-2.28c1.01-1.01,1.85,0.41,2.02,1.24
			c0.59,1.33,1.77,2.93,3.3,2.46c0.95-0.41,1.85,1.42,1.17,2.24c0.42,1.36,0.56,2.38,1.24,3.64c0.45,0.27-1,0.42-1.22,0.82
			c-0.34,1.54-0.23,3.8-0.8,4.81c-0.46,0.09,0.77,2.24,0.33,1.95"></path>
	</g>
	<g id="sagunto">
		<path class="st3 fase_0" d="M523.09,219.72l0,0.09L523,219.9l0,0.09l-0.1,0.18l0,0.09l-0.1,0.18l0,0.09l0,0.19l-0.09,0.09l0,0.09l0,0.09
			l0,0.09l0,0.19l-0.1,0.18l0,0.09l0,0.09l0,0.09l0,0.09l-0.1,0.18l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09
			l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.18-0.1l-0.1,0.28l-0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l-0.09,0
			l-0.09,0.09l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.1,0l0,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0.09
			l0,0.09l-0.09,0l-0.09,0.09l-0.1,0.18l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.19,0.09l-0.73-0.48
			l-0.37-0.38l-0.09,0l-0.09,0l-0.09,0l-0.28-0.1l-0.74-0.29l-0.65-0.1l-0.47,0.27l0,0.09l-0.47,0.18l-0.56,0.27l0-0.09l-0.09-0.09
			l0-0.09l-0.09-0.19l-0.09-0.09l0-0.09l-0.09-0.09l-0.09-0.19l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09
			l0-0.09l0-0.19l-0.09-0.09l0-0.09l-0.09-0.09l-0.09-0.19l0.1-0.28l0.19-0.28l0-0.09l0-0.09l0.09-0.09l-0.09-0.19l-0.09-0.09
			l-0.18-0.19l-0.09-0.19l-0.09-0.19l-0.09-0.37l-0.19-0.28l-0.19,0l-0.09,0l-0.09,0.09l-0.37-0.01l-0.46-0.01l-0.28,0l-0.09,0
			l0,0.09l-0.09,0l-0.09,0.09l-0.19,0l-0.19,0l-0.09-0.28l-0.18-0.19l-0.09-0.09l-0.09-0.09l0-0.09l-0.09-0.09l-0.18-0.1l-0.09-0.09
			l-0.09-0.09l-0.27-0.29l-0.27-0.28l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09-0.09
			l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.1l-0.09-0.09l-0.1,0l0-0.09l0-0.09
			l-0.09-0.09l-0.18-0.28l-0.09-0.09l-0.19,0l-0.19,0l-0.37,0.09l-0.37,0.27l-0.75,0.92l0,0.19l-0.2,0.65l-0.1,0.37l-0.1,0.37
			l-0.46-0.01l-0.19,0l-0.65-0.11l-0.09,0l-0.37-0.1l-0.19,0l-0.28-0.19l-0.46-0.19l0.2-0.83l0.01-0.93l0.1-0.18l-0.26-1.02l0-0.28
			l-0.09-0.37l0-0.19l-0.09-0.28l0-0.19l-0.28-0.1l-0.28-0.1l-0.46-0.38l-0.09,0l-0.18-0.1l-0.19-0.19l-0.09,0.09l-0.09,0.09
			l-0.19,0.37l0,0.19l-0.28,0.18l0,0.09l0,0.28l-0.09,0.09l0.09,0.1l-0.09,0.09l-0.1,0.37l-0.09,0l-0.19,0.09l-0.09,0.09l-0.09,0
			l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.1l0-0.09l0.09,0
			l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.19l-0.1,0l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0
			l-0.09-0.19l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.36-0.66l-0.18-0.1l-0.55-0.38l-0.09-0.09l-0.19,0
			l0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.19l0-0.09l-0.1-0.09l-0.09-0.09l-0.18-0.19l-0.09,0l-0.09,0l-0.09-0.09l-0.09-0.09
			l-0.09,0l0-0.09l-0.28-0.19l-0.09-0.19l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l-0.09-0.19l0-0.09l-0.18-0.1l0.09-0.09l0.09-0.09
			l0-0.28l0-0.09l-0.09-0.09l0-0.09l-0.09-0.19l-0.09-0.09l-0.18-0.19l0.09-0.1l0-0.09l0-0.09l0.19-0.09l0.09-0.09l0-0.09l0.09,0
			l0-0.09l0-0.09l-0.09-0.19l0-0.09l-0.18-0.1l0.19-0.18l0-0.19l0.09-0.09l0-0.09l0.09,0l0.09,0l0.28-0.18l0,0.09l0.09,0l0.09-0.09
			l0.09,0l0.09,0l0.19-0.18l0-0.09l0.09-0.1l0.09,0l0.09,0l0.1-0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0
			l0-0.09l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l-0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.09,0l0-0.09l0.09,0
			l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0.09-0.09l0-0.1l0-0.09l0.09,0l0-0.09l0-0.09
			l0-0.09l0-0.09l-0.09-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09-0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0.09,0l0-0.09l0.09,0
			l0.19,0l0.09,0l0.19-0.09l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0,0.09l0.09-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09
			l0.09,0l0.19-0.09l0.09,0l0.19,0l0.28,0.1l0.09,0l0.09,0l0.1,0l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09-0.09l0.19-0.09l0.09,0
			l0.37,0.01l0.37,0.01l0.55,0.1l0.28,0.1l0.19,0l0,0.09l0.09,0l0,0.09l0.09,0.19l0.09,0.09l0.09,0l0.09,0l0.27,0.47l0.18,0.1
			l0.55,0.1l0.09,0.19l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.65,0.01l0,0.09l0.28,0l0.09,0l0.37-0.09
			l0.37,0.19l0.28,0.1l0.28,0.1l0.1,0l0.09,0l0-0.09l0.18,0.1l0.09,0l0.28,0l0.09-0.09l0.18,0.1l0.09,0.19l0,0.09l0.09,0l0,0.09
			l0.09,0l0,0.09l0.09,0l-0.09,0.09l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0.09l0,0.19l0.09,0l0,0.09l0,0.1l0.09,0l0,0.09
			l0.18,0.1l0.09,0.09l0.18,0.19l0,0.09l0.1,0.09l0.09,0l0.09,0l-0.1,0.18l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0.19l0.09,0
			l0.09,0.09l0.18,0.1l0.18,0.19l0.09,0l0.09,0.09l0.09,0l0.09,0.09l0.18,0.19l0.09,0.1l0,0.09l-0.09,0l-0.09,0.09l0,0.09l0.09,0.19
			l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.19,0.1l0.09,0l0.09,0l0,0.09l0,0.09l0,0.19
			l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l-0.28,0.37l0.19,0l0.09,0.19l0.09,0l0.09,0.19
			l0.09,0.09l0,0.09l0.09,0l0,0.19l0.09,0l0.09-0.09l0.09,0.09l0.09,0.09l0.09-0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0.09
			l0.09,0.09l0-0.09l0.1,0l0.18,0.1l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l-0.09,0l0.09,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0.09,0
			l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.19,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.28l-0.28,0.28l-0.1,0.65
			l-0.28,0.37l-0.1,0.18l-0.09,0.09l-0.1,0.28l0.09,0l0.09,0l0.09,0.47l0,0.19l-0.09,0.09l-0.1,0.37l0,0.09l0.28,0.19l0.09,0.09
			l0.18,0.19l0.18,0.28l0.18,0.19l0.18,0.19l0.28,0.19l0.28,0.19l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09
			L519.6,218l0,0.09l0.09,0.19l0,0.09l0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0l0.09,0l0.09,0.09l0.09,0l0.19,0l0-0.09
			l0.09,0l0.09,0.09l0.09,0.09l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0.1,0.09l0.09,0.09l0.09,0l0.09,0.1l1.01,0.39l0.18,0.1
			L523.09,219.72z"></path>
	</g>
	<g id="la_plana">
		<path class="st3 fase_0" d="M530.11,209.71l-0.09,0.09l-0.1,0.18l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l0,0.09
			l-0.09,0.09l-0.09,0.09l-0.09,0.09l-0.09,0l0,0.1l-0.09,0.09l0,0.09l-0.09,0l-0.1,0.18l-0.09,0.09l-0.09,0.09l-0.19,0.18
			l-0.19,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09-0.09l0,0.09l-0.09,0l0,0.09
			l0.18,0.1l0,0.09l-0.09-0.09l-0.18-0.1l-0.09,0l-0.09,0.09l0,0.09l-0.09,0.09l-0.19,0.28l-0.1,0.18l0,0.09l-0.09,0.09l-0.09,0.09
			l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09
			l-0.1,0.09l-0.09,0.09l0,0.09l-0.1,0.18l0,0.1l-0.09,0.09l0,0.09l-0.09,0.09l-0.1,0.18l-0.09,0.09l0,0.09l-0.1,0.18l0,0.09
			l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09
			l-0.09,0l0,0.09l-0.09,0l0,0.1l0,0.09l-0.09,0l0,0.19l-0.09,0l-0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09
			l-0.09,0l0,0.09l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.1,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0.1l0,0.19
			l-0.09,0l0,0.09l-0.1,0.18l0,0.09l-0.28-0.1l-0.18-0.1l-1.01-0.39l-0.09-0.1l-0.09,0l-0.09-0.09l-0.1-0.09l-0.09,0.09l-0.09-0.09
			l-0.09,0l-0.09,0l-0.09-0.09l-0.09-0.09l-0.09,0l0,0.09l-0.19,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l-0.09,0
			l-0.09-0.09l-0.09,0l0-0.09l-0.09-0.19l0-0.09l0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09
			l-0.28-0.19l-0.28-0.19l-0.18-0.19l-0.18-0.19l-0.18-0.28l-0.18-0.19l-0.09-0.09l-0.28-0.19l0-0.09l0.1-0.37l0.09-0.09l0-0.19
			l-0.09-0.47l-0.09,0l-0.09,0l0.1-0.28l0.09-0.09l0.1-0.18l0.28-0.37l0.1-0.65l0.28-0.28l0-0.28l-0.09,0l0-0.09l-0.09,0l0-0.09
			l-0.19,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l0-0.09l-0.09-0.09l0.09,0
			l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.18-0.1l-0.1,0l0,0.09l-0.09-0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09
			l-0.09,0.09l-0.09-0.09l-0.09-0.09l-0.09,0.09l-0.09,0l0-0.19l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.19l-0.09,0l-0.09-0.19l-0.19,0
			l0.28-0.37l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.09l0-0.19l0-0.09l0-0.09l-0.09,0
			l-0.09,0l-0.19-0.1l-0.09,0l-0.09-0.09l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.19l0-0.09l0.09-0.09
			l0.09,0l0-0.09l-0.09-0.1l-0.18-0.19l-0.09-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.18-0.19l-0.18-0.1l-0.09-0.09l-0.09,0l-0.09-0.19
			l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09l0.1-0.18l-0.09,0l-0.09,0l-0.1-0.09l0-0.09l-0.18-0.19l-0.09-0.09l-0.18-0.1l0-0.09
			l-0.09,0l0-0.1l0-0.09l-0.09,0l0-0.19l-0.09-0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l-0.09,0
			l0-0.09l-0.09,0l0-0.09l-0.09-0.19l-0.18-0.1l-0.09,0.09l-0.28,0l-0.09,0l-0.18-0.1l0,0.09l-0.09,0l-0.1,0l-0.28-0.1l-0.28-0.1
			l-0.37-0.19l-0.37,0.09l-0.09,0l-0.28,0l0-0.09l-0.65-0.01l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09
			l-0.09-0.19l-0.55-0.1l-0.18-0.1l-0.27-0.47l-0.09,0l-0.09,0l-0.09-0.09l-0.09-0.19l0-0.09l-0.09,0l0-0.09l-0.19,0l-0.28-0.1
			l-0.55-0.1l-0.37-0.01l-0.37-0.01l0.19-0.28l0.09-0.09l0.09-0.09l0.19-0.18l0.09-0.09l0-0.1l0.09-0.09l0.09,0l0.09,0l0.09,0
			l0.09,0l0-0.09l0-0.09l0-0.09l0.09,0.09l0,0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l0.09-0.09l0.1,0l0,0.09l0.09,0.09l-0.09,0
			l0,0.09l0,0.09l0.09,0l0.09,0.09l0.09-0.09l0.09,0l0.09,0l-0.09-0.09l-0.09-0.37l-0.09-0.19l0-0.09l0-0.09l-0.09,0l0-0.09
			l-0.09-0.09l0-0.09l0-0.09l-0.1-0.09l0-0.19l-0.09-0.19l0-0.19l0.48-0.74l0.1-0.18l0.09-0.09l0.09-0.09l0-0.09l0.09,0l0.09,0.09
			l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09
			l0-0.09l0-0.09l0-0.09l0-0.1l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l-0.09-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.28-0.37l0.1-0.09l0-0.09l0-0.09l0-0.19
			l0.09-0.09l0.09-0.09l-0.09-0.28l0-0.19l0.09-0.09l0-0.19l0.2-0.83l0.1-0.65l0.09-0.09l0-0.28l0.09,0l0.09,0l0,0.09l0.28,0.19
			l0,0.09l0.18,0.19l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.46,0.19l0.28-0.09l0.37,0.28l0-0.09l0.09-0.09l0.09-0.09l0.09,0
			l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0,0.09l0,0.09l0.09,0.09l0.09,0l0,0.09l0.09-0.09
			l0.28,0l0.09,0l0.28-0.18l0.1-0.09l0.28-0.18l0.28,0l0.56-0.36l1.03-0.63l0,0.09l0.09,0l0.1,0l0,0.09l0.09,0l0,0.09l0.09,0.09
			l0.09,0.19l0.09,0.19l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0.64,0.38l0,0.09l0.09,0.09l0.18,0.19l0.46,0.2l0.74,0.57l0,0.09
			l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l0,0.09l-0.1,0l0,0.09
			l-0.09-0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l0,0.1l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l0-0.09
			l-0.09,0.09l-0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l-0.09,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.09
			l0,0.09l-0.09,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0.1l0.09,0l0,0.09l0.09,0l0.09,0.09
			l0.09,0l0.09,0l0,0.09l0,0.09l0.09,0.09l0.09,0.09l0.1,0l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0
			l0,0.09l0.09,0.09l0,0.09l0.09,0.19l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.1l-0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09
			l0.09,0.09l-0.09,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0.09l-0.09,0.09l0,0.09l0,0.09l0.09,0.09l0.19,0l0.19-0.09
			l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0.1l0.09,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09
			l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.19l0.1,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0,0.09
			l0.09,0l0.18,0.1l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0l0.18,0.1l0.09,0l0,0.09l0.19,0l0.09,0
			l0.09,0l0.09,0.09l0.1,0l0.09,0.09l0.09-0.09l0.09,0l0.09,0.09l0,0.28l0.09,0.09l0,0.09l0,0.09l0.28-0.18l0.09,0l0.09,0l0-0.09
			l0.09,0l0.09,0l0-0.1l0.09,0l0.09,0l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0.19-0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0.1,0
			l0,0.09l0.09,0.09l0,0.1l0,0.09l0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0.09
			l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0.09l0.18,0.19l0,0.19l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0.09l-0.19,0.09l-0.1,0.19
			l-0.09,0l-0.1,0.18l0.09,0l0.09,0l0.18,0.1l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0.09l0.09,0l0.09,0l0.09,0.09
			l0.09,0l0.09,0l0-0.09l0.09,0l0.09-0.09l0,0.09l0,0.09l0.09,0l0.19,0l0.1,0.09l0.18,0.19l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0
			l0.09,0l0.09,0l0.09,0.09l0.18,0.1l0,0.09l0.09,0.09l0.09,0.09l0.09,0l0.09,0.09l0.09,0.19l0.09,0l0.09,0.19l0.09,0.19l0.09,0
			l0,0.1l0.09,0.09l0.09,0l0.18,0.1l0.1,0l0.09,0l0.09,0.09l0.09,0l0.18,0.1l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0.09l0.09,0
			l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0l0,0.09L530.11,209.71z"></path>
	</g>
	<g id="vinaros">
		<path class="st3 fase_1" d="M550.45,177.25l0,0.09l0,0.1l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09
			l-0.09,0l0.09,0.09l-0.09,0l0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.1,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0
			l0,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.1l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0
			l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09
			l-0.09,0l0,0.19l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.1l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l0,0.09
			l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.1,0.09l0,0.19l0,0.09l0,0.09l0,0.09l-0.09,0.09l0-0.09l-0.09,0l0,0.09l0,0.09l-0.09,0
			l0,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.1l-0.09,0l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.1,0.18
			l-0.1,0.28l0,0.19l-0.1,0.18l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.19l0,0.1l0,0.09l0.09,0l0.09,0l0,0.09
			l-0.09,0.09l-0.09-0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09
			l0,0.09l-0.09,0l0.09,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.1,0l0,0.09l0,0.09l-0.09,0l0,0.09
			l-0.09,0l-0.09,0.09l-0.09,0.09l0,0.1l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0.09l-0.09,0.09l0,0.09l0,0.09
			l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0
			l0,0.09l0,0.09l0,0.09l0,0.1l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09
			l-0.09-0.09l0,0.09l0,0.09l-0.09,0l-0.09,0l-0.1,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0
			l0,0.09l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0
			l-0.09,0.1l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0
			l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0.1
			l-0.09,0l-0.09-0.19l0-0.09l-0.19-0.19l-0.09,0l-0.09-0.09l-0.18-0.1l-0.18-0.1l-0.18-0.19l-0.46-0.28l-0.09,0l0-0.09l-0.09,0
			l-0.09-0.09l-0.28-0.1l-0.19,0l-0.09-0.09l-0.09,0l-0.1,0l-0.28,0.18l-0.37,0.09l-0.1,0.18l-0.19,0.37l-0.19,0.09l-0.28-0.1
			l-0.09,0l-0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.38-0.92l0.19-0.37l0.1-0.46l-0.09-0.37l0-0.28l-0.09-0.09
			l-0.09-0.37l0.1-0.19l0.19-0.18l0-0.09l0.1-0.18l-0.09-0.28l-0.09-0.19l-0.09-0.19l-0.36-0.84l0.19-0.28l0.38-0.55l-0.64-0.84
			l0.19-0.46l-0.09-0.1l0-0.19l-0.46-0.28l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.1,0l-0.09,0l-0.09-0.09l0,0.09l-0.09,0
			l0-0.09l-0.09,0l-0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0,0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09
			l-0.09-0.09l-0.18-0.37l0-0.19l-0.27-0.38l-0.09,0l-0.18-0.1l-0.18-0.19l0.28-0.09l-0.09-0.28l0.28,0l0-0.09l0-0.09l0-0.19l0-0.09
			l0-0.09l0.1-0.18l0.09,0l0-0.09l0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0
			l-0.09-0.1l0-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09,0l0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
			l0.09,0l0-0.09l0-0.09l-0.09-0.09l-0.1-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09-0.1l0-0.09l-0.09-0.09l0-0.09
			l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0.1-0.18l0-0.09l0.1-0.18l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09
			l0.1,0l0-0.09l-0.1-0.09l0-0.09l0.1,0l0-0.09l0-0.1l0-0.09l0.09-0.09l0.09,0l-0.09-0.09l0-0.09l0.09,0l-0.09-0.09l0-0.09l0-0.09
			l-0.09-0.09l-0.28,0l-0.09-0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0.09,0.09
			l-0.09,0l0,0.09l-0.09,0l-0.09,0.1l-0.09,0.09l-0.09,0.09l-0.09-0.19l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09
			l-0.09,0l-0.18-0.1l-0.19,0.09l-0.09,0l-0.09,0l-0.28,0l-0.09,0l-0.09-0.09l-0.19,0.09l-0.09,0l-0.19,0l0.09,0.09l0.09,0.09
			l-0.1,0.28l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0-0.09l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09-0.09l-0.09,0
			l-0.01,0.37l0,0.19l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.1l-0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09
			l-0.09,0l0,0.09l-0.1,0.18l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09
			l-0.09,0.09l0,0.09l-0.09,0.1l-0.09,0.09l0,0.09l0.09,0.09l0,0.09l0,0.19l0,0.09l0,0.09l0,0.09l0.09,0.37l-0.09,0l0,0.09l0,0.09
			l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.1l-0.09,0l-0.09,0l-0.09-0.1l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09
			l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09
			l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.1l0-0.09l-0.18-0.1l-0.09-0.19l0-0.09l0-0.09l-0.09-0.19l0-0.09l-0.09,0l0-0.09l-0.09-0.09
			l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09
			l-0.09-0.1l-0.09,0l0-0.09l0-0.09l0-0.09l-0.1-0.09l0-0.09l0-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.19,0l-0.28,0
			l0-0.09l-0.09,0l0,0.09l-0.28,0.09l-0.09,0.09l-0.09,0l0,0.19l-0.09-0.09l-0.09-0.19l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09
			l0,0.09l-0.09,0.09l-0.09,0.09l-0.38,0.37l-0.47,0.64l-0.1,0.28l-0.09,0.09l0,0.09l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0
			l-0.09,0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.19,0.09l-0.09,0
			l-0.19,0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09-0.09l-0.19,0l-0.09-0.09l-0.19,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.19,0
			l-0.28-0.1l-0.09,0l-0.09,0l0-0.09l0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.1,0.09l0-0.09l-0.09,0l0,0.09l0.09,0l0,0.09l-0.09,0
			l-0.09,0l0,0.09l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.09l0-0.09
			l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.1l-0.09,0l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09
			l-0.09,0l-0.09,0l0,0.09l-0.19,0.09l-0.09,0l0-0.09l-0.09-0.09l0-0.19l-0.09-0.09l0-0.09l0-0.19l0.09-0.09l0-0.19l0-0.09l0-0.19
			l0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.1l0-0.09l-0.09-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09
			l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l-0.09-0.09l-0.09-0.09l0-0.09
			l0-0.09l0-0.09l0.09,0l0.09-0.09l0-0.09l0-0.1l-0.09-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l0.09,0l0-0.09l0-0.09l0-0.09
			l-0.09,0l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09-0.09l-0.09,0.09
			l-0.09,0l-0.09,0l-0.1,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l0-0.19l-0.09,0l-0.09,0l-0.18-0.1l-0.09,0l-0.09-0.09l-0.09,0l0,0.09l-0.09,0
			l-0.09,0l-0.09,0l-0.09,0l-0.1,0l0-0.09l-0.09-0.09l-0.09,0l-0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0.09,0l0-0.09l0-0.09
			l0-0.09l0-0.09l0.1-0.18l0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.28-0.19l0-0.19
			l-0.09,0l0-0.19l0.09-0.1l0.09-0.09l0.09,0l0.09,0l0.09-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0.47-0.18l0.09-0.09l0.09,0l0.09,0
			l0.19-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09-0.09l0.09,0.09l0.09,0.09l0.09-0.09l0,0.09l0.09,0l0,0.09l-0.09,0.09
			l0.09,0.09l0,0.19l0.09,0.09l0.09,0.09l0,0.1l0.09,0l0.28,0.19l0-0.09l0-0.09l0-0.1l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0
			l0-0.09l0-0.09l0.09,0l0-0.09l0.09-0.09l0,0.09l0.09,0l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0.09,0
			l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.1-0.09l0.19,0l0.09,0l0.09,0l0.09,0l0.09,0
			l-0.09-0.37l0.19-0.56l-0.09,0l0-0.09l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0.09-0.09
			l0-0.19l0.09-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.1l0-0.19l0-0.09l0.09-0.09l0-0.09l0-0.09
			l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0.09-0.09l0-0.19l0.09,0l0-0.28l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09
			l0.09-0.09l-0.09-0.09l0-0.09l0.09,0l0.09,0l0.09-0.09l0-0.1l0.28,0l0.09,0l0.09-0.09l0.28,0l0.19-0.09l0.09,0.09l0.09,0l0-0.09
			l0.09-0.09l0.09,0l0.37,0.1l0.28,0l0.09,0.09l0,0.09l-0.09,0.1l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09
			l0.09,0.09l0.09,0l0.09,0l0.09,0.19l0,0.09l0,0.09l0.09,0l0,0.09l0.1,0l0.09,0.19l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0
			l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0.19l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.1l0.09,0l0.09,0
			l0.09,0l0,0.09l0.18,0.19l0.46,0.19l0.18,0.1l0.09,0l0.28,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0.18,0.1l0.09,0l0.09,0
			l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0.09,0l0.09-0.09l0.09,0.09l0.1,0l0-0.09l0.09,0.09l0.09,0l0.09,0
			l0.09-0.09l0.09,0.09l0.09,0l0.19,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0,0.09l0,0.09l0,0.09l0.09,0.09l0.09,0.09l0.09,0
			l0,0.09l0.09,0l0.09,0.09l0,0.09l-0.1,0.18l0,0.09l0,0.09l-0.09,0.09l0,0.09l0.09,0l0.09,0.09l-0.09,0l0,0.09l0,0.09l0.09,0.1
			l0.09,0l0.28,0.1l0.18,0.1l1.23-1.28l1.59-0.62l0,0.09l0.19,0l0.28,0.19l0.09,0l0.28,0.1l0.28,0l0.09,0l0.09,0l0.09,0l0.09-0.09
			l0.09,0l0.09,0l0.09-0.09l0.09,0l0.1-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.37,0.19l0.09,0l0.09,0l0.09,0
			l0-0.09l0.09,0l0-0.09l0.09,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09-0.09l0.09,0l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.09,0
			l0-0.09l0.09-0.1l0.09,0l0.1,0l0.09,0l0.09,0l0,0.19l0,0.19l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0.18,0.28l0.09,0.09
			l0.09,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.19,0l0.09-0.09l0.09,0l0.09,0
			l0.09,0.09l0,0.09l0.09-0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.1,0.09l0,0.09l0.09,0l0.09,0l0.09,0.09l0.09,0.09
			l0.09,0l0.18,0.1l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09L541,169.6l0,0.09l-0.09,0l-0.09,0
			l-0.09,0.09l-0.1,0.09l0,0.09l0,0.19l-0.19,0.18l0,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.19l-0.09,0l0,0.09l-0.09,0l0,0.09
			l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.1,0l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0.09,0.09l0.28,0.1l0.09,0.09
			l0.19-0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.18,0.1l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0
			l0.09,0l0.09,0.09l0.1,0l0.09,0l0.18,0.1l0.19,0l0.09,0l0.09,0l0.19,0l0.09,0l0.09,0.1l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0
			l0.09,0l0.09,0l0-0.1l0.09,0l0.09,0l0.09,0.1l0.09,0l0.09-0.1l0.09,0l0.1,0.1l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09
			l0.09,0.09l0,0.09l0.09,0l-0.09,0.09l0,0.09l0.09,0l0.09,0.09l0.28,0.1l0.09,0.09l0,0.09l0.09,0.09l0.09,0l0.09,0l0,0.09l0.09,0
			l0.09,0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0.09l0.09,0l0,0.09
			l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0.09,0.19l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0.09,0.09l0,0.09l0.09,0l0.18,0.1
			l0.1,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0.09l0.28,0.1l0.28,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0
			l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0l0,0.09l0.19,0l0.09,0l0,0.09l0.09,0l0.09,0L550.45,177.25
			z"></path>
	</g>
	<g id="rincon_de_ademuz">
		<path class="st3 fase_1" d="M477.76,205.11c-2.09-0.39-2.62,0.51-3.31-1.98c-0.1-0.83-1.45-1.99-0.89-2.52c0.81-0.38-1.45-1-1.37-1.32
			c-0.78-0.67-0.09-1.51,0.66-0.82c0.53-0.67,1.46,0.62,1.76,0.03c1.21-0.5,0.73,0.76,1.3,0.39c0.2-0.43,1.84-0.71,1.41-1.37
			c0.01-1.19-0.47-2.26,0.88-2.77c-0.05-0.21,0.3,0.15,0.09,0.19c0.06,1.3,0.75,2.33,1.72,3.28c0.24-0.36,0.41-0.03,0.18,0.19
			c0.1,0.02-0.04,0.41,0.08,0.65c-0.06,0.1-0.32,0.3-0.1,0.28c0.17,0.51,0.62,0.45,1.11,0.3c0.93,0.91,2.75-0.71,3.24,0.51
			c0.31-0.05,0.1,0.49,0.46,0.47c1.04,0.33,2.06,1.17,2.29,2.17c-0.32,1.26-1.97,1.94-3.1,2.09c-0.21,0.44-1.1-0.01-1.21,0.54
			c-0.22-0.05-0.36,0.18-0.56,0.18c-0.08-0.03-0.32-0.21-0.28,0C480.47,206.04,479.31,205.63,477.76,205.11z"></path>
	</g>
	<g id="comunidad_valenciana_p">
		<g id="alicante_p">
			<path class="st7" d="M541.08,270.65l0-0.09l-0.19,0l-0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.1l-0.09,0l0-0.09
				l0-0.09l0-0.09l0-0.09l-0.09,0l0,0.09l0,0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0
				l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.09-0.09l0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09
				l-0.1-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.1l0-0.09l0.09,0l0.1,0l0.09,0l0-0.09l0.09,0.09l0.09-0.09l0.09,0
				l0.09-0.09l-0.09-0.09l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.1,0l-0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0
				l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.19,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0
				l-0.09,0l-0.09-0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.1,0l0,0.09l-0.09,0l-0.09,0l-0.09,0
				l0-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.09-0.1l0-0.09l-0.09-0.09l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.19,0l0-0.09l-0.09,0
				l0-0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.1,0l-0.09,0
				l-0.09-0.09l-0.09,0l-0.19,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.19,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.19,0l-0.09,0
				l-0.19,0l0-0.09l-0.19,0l-0.18-0.1l-0.09-0.09l-0.1,0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.19,0
				l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.18-0.1l-0.09-0.09l-0.09-0.09l-0.18-0.1l-0.09-0.09l-0.09-0.1
				l-0.09-0.09l-0.1,0l0,0.09l0,0.1l0,0.09l0.1,0l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0.09l-0.09,0.09l-0.1,0.18
				l0,0.09l0.09,0l0,0.09l0,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.1,0l-0.09,0l-0.09,0l0-0.09l0-0.09
				l-0.09-0.09l0-0.09l-0.09-0.09l-0.19,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l-0.09-0.19l-0.09,0l0-0.19l-0.09,0
				l0-0.19l0-0.1l-0.09,0l0-0.09l-0.09,0l0,0.09l-0.09,0l-0.19,0l0,0.1l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l0-0.09l-0.09,0l0,0.09
				l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.09l-0.09,0l-0.1,0l0-0.09l-0.09,0l-0.19,0l-0.19,0l-0.09,0.09l-0.28,0.09
				l-0.28,0.18l-0.09,0.09l-0.19,0l-0.09,0.09l-0.09,0.09l-0.28,0.09l-0.28,0.18l-0.09-0.09l-0.19-0.1l0-0.28l0-0.09l0-0.19
				l-0.09-0.28l-0.18-0.28l0-0.28l-0.09,0.09l0,0.19l-0.19,0.18l-0.09,0l-0.09,0l-0.19,0l-0.19,0.09l-0.09,0l-0.19,0.18l0,0.09
				l-0.28,0.18l-0.1,0.18l0.09,0.28l-0.09,0.09l-0.28,0.09l-0.75,0.17l-0.19,0l-0.19,0l-0.28-0.19l-0.09,0l-0.09-0.09l-0.09,0
				l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.19,0l-0.46-0.19l-0.09,0l-0.18-0.19l-0.09,0l-0.09-0.09l-0.18-0.1
				l-0.09-0.09l-0.27-0.28l-0.09-0.09l0-0.09l-0.37,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l-0.1,0.18l-0.09,0l0,0.09l0,0.09
				l-0.19,0.09l-0.1,0.09l0,0.09l-0.09,0l-0.19,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.19,0l-0.09,0l0,0.09
				l-0.09,0.09l0.09,0.09l-0.09,0l0,0.19l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.1l-0.09,0l-0.09,0l0.09,0.09l-0.09,0l-0.09,0
				l-0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.19,0.09l-0.09,0l-0.09-0.09l0,0.09
				l-0.18-0.1l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.09,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l-0.19,0l-0.09,0.09
				l-0.46,0.09l-0.19,0.09l-0.09,0l-0.19,0.09l-0.1,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0-0.09l-0.19,0l-0.19,0.18l-0.28,0.09
				l-0.1,0.28l0,0.09l-0.28,0.09l-0.19,0.18l-0.18-0.1l-0.37-0.01l-0.28,0l-0.37-0.01l-0.18-0.1l-0.19,0l-0.19,0l-0.09,0l-0.09-0.09
				l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.19,0.09l-0.09,0l-0.09,0.09l-0.19,0.09l-0.09,0l-0.09,0l-0.19,0.09l-0.19,0l-0.09,0
				l-0.19,0.09l-0.09,0.09l-0.19,0l-0.09,0l-0.09,0l-0.19,0l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.19,0.09l-0.09,0.09l0,0.09l0,0.09
				l-0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0,0.1l0.09,0l0,0.09
				l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0.09,0
				l0.18,0.1l0.09-0.09l0-0.09l0.1-0.09l0.09,0l0.37-0.18l0.09,0l0-0.09l0.09-0.09l0.19,0l0-0.09l0.19,0l0.18,0.19l0,0.09l0,0.09
				l0.09,0.09l0.09,0.09l0.28,0l0.28,0l0,0.09l0,0.19l0,0.28l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09,0
				l-0.09,0l0,0.19l-0.09-0.09l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l0,0.1l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.1,0.18
				l-0.09,0.09l-0.19,0.18l-0.09,0.09l0,0.09l-0.09,0l-0.09,0l-0.1,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.19,0l0,0.09l-0.09,0
				l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.19,0.09l-0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0
				l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.1l-0.19,0.18l-0.1,0l0,0.09l-0.09,0.09l-0.46-0.01l0-0.09l-0.09-0.09l0-0.09
				l-0.09,0l0-0.09l-0.09,0l0-0.1l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.19l-0.09-0.09l0-0.09
				l0-0.09l-0.09-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09-0.09l-0.09,0l-0.19,0l-0.09,0l-0.09,0
				l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.1,0l-0.09,0l0,0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l0-0.09l-0.09-0.1l-0.09-0.09
				l0-0.09l-0.09-0.09l-0.28,0.18l0,0.09l-0.09,0l-0.28,0.09l-0.09,0.09l0-0.09l0-0.1l0-0.09l-0.09-0.19l-0.09,0l0-0.09l-0.18-0.28
				l0-0.09l-0.09,0l-0.18-0.1l-0.09,0l-0.1,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l0.09-0.09l0.09-0.09l-0.09-0.09l-0.09,0.09
				l-0.09,0l-0.09-0.09l-0.19,0l0.09,0.19l0.09,0.09v0l-0.09-0.09l-0.19,0l-0.19,0l-0.09,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0
				l-0.09,0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l-0.19,0.09l-0.09,0l-0.1-0.09l-0.09,0.09l-0.09,0.09
				l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.19,0.09l-0.19,0l-0.09,0.09l0-0.09l-0.09,0l0-0.1l-0.09,0l-0.19,0.09l-0.19,0
				l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09-0.19l0-0.19l0-0.09l0-0.09l-0.09,0l-0.09-0.19l-0.09-0.19
				l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09
				l-0.28,0l-0.19,0l-0.09,0l-0.09,0l-0.19,0l-0.09,0l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.19,0l-0.19,0.09l-0.09,0l-0.09,0l-0.09,0
				l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.18-0.28l0-0.09l0.1-0.28l-0.19,0l-0.37-0.1l-0.09,0l-0.19,0.09l-0.28,0.19l-0.09,0
				l-0.09,0l0,0.09l-0.19,0l-0.1,0.09l-0.28,0l-0.09,0l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09
				l0,0.09l0.09,0.09l0,0.09l0.09,0.19l0.09,0.09l0,0.19l0.09,0.09l0,0.09l0.09,0.1l0,0.09l0,0.09l0.09,0.09l0.09,0l0,0.09l0.09,0
				l0,0.09l0,0.19l0,0.09l0.09,0l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.19l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.09
				l0.09,0.19l0.18,0.1l0,0.09l0.18,0.1l-0.09,0l0,0.09l-0.1,0.18l-0.19,0.18l0,0.09l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0.09
				l-0.09,0.09l-0.09,0.09l-0.19,0.18l-0.09,0.09l-0.19,0.18l-0.09,0.09l-0.09,0.09l-0.19,0.09l-0.09,0.1l-0.19,0.18l-0.28,0
				l-0.37,0.09l-0.09,0l-0.37-0.01l-0.28-0.1l-0.46-0.1l-0.09,0.09l-0.19,0l-0.09,0l-0.09,0l0,0.09l-0.19,0l-0.09,0l0,0.09l0.09,0
				l0,0.09l0,0.09l0,0.19l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0.1,0.09l0,0.19
				l0,0.09l0,0.09l0,0.1l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.19l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09
				l0.09,0l0,0.19l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.1l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09
				l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.28l-0.09,0l0,0.09l0,0.09
				l0,0.09l0,0.1l0,0.09l-0.09,0l0,0.09l0,0.09l-0.1,0.09l0,0.09l-0.19,0.18l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0.09,0l0,0.09
				l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0,0.09l0.1,0.09l0,0.09l0.09,0.19l0.09,0.09l-0.09,0.09l0,0.09l-0.09,0
				l0,0.09l-0.1,0.09l0,0.09l-0.09,0l0,0.09l-0.19,0.18l0,0.09l-0.09,0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0
				l0,0.09l-0.09,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.19,0.18l-0.09,0.1l-0.09,0.09l-0.09,0.09l-0.19,0.18l-0.09,0l0,0.09
				l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.1,0.09l-0.09,0l-0.19,0.18l-0.19,0l-0.09,0.09l-0.09,0l0,0.09l0.19,0
				l0.09,0.09l0.09,0l0.09,0.09l0.19,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.1l0,0.09
				l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.19l0,0.09l0,0.19l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.19l0,0.09l0,0.09l0,0.19
				l0,0.09l0,0.1l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.19l0,0.09l0,0.19l0.09,0l0,0.09l0,0.09
				l0,0.09l0,0.09l0,0.09l0.65,0.01l0.46,0.01l0.74,0.2l0.46,0.2l0.56,0.19l0.18,0.1l0.92,0.57l0.17,0.74l0.36,0.57l0.18,0.28
				l0.09,0.28l0.08,0.56l-0.01,0.46l-0.01,0.56l-0.19,0.28l-0.1,0.28l-0.1,0.37l-0.2,0.74l-0.2,0.83l0,0.19l0,0.28l-0.09,0.09
				l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.19l-0.09,0l0,0.09l-0.19,0.56l-0.19,0.28l-0.57,0.83l-0.1,0.18
				l-0.09,0.09l-0.38,0.36l0,0.09l0.09,0.38l0.17,0.74l0,0.28l0,0.19l-0.01,0.56l0.18,0.37l0,0.09l0.09,0.19l0,0.09l0,0.19
				l0.09,0.19l0.09,0.09l0,0.09l0.09,0.19l0.09,0.19l0.18,0.19l0.09,0.09l0.09,0.09l0,0.1l0.09,0l0.09,0.09l0.09,0l0,0.09l0.09,0
				l0.18,0.37l0.18,0.19l0.18,0.28l0,0.09l0.18,0.19l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0l0.1,0.09l0.09,0.09l0.18,0.38
				l0.09,0.19l0.13,0.33l0.05,0.14l0.09,0.09l0.81,1.31l0.18,0.19l0.45,1.21l0.09,0.09l0.82,1.03l0.36,0.38l1.46,1.6l0.09,0.09
				l0.09,0l0.09,0.09l0,0.09l0.09,0.09l0,0.09l0.09,0l0.09,0.09l0.18,0.19l0,0.09l0.09,0l0.09,0.09l0.18,0.1l0,0.09l0.46,0.1l0.09,0
				l0.18,0.1l0.09,0l0.1,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0.09,0l0.09,0.1l0.09,0.09l0.09,0
				l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0l0.37-0.18l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
				l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l0.1,0l0-0.09l0-0.09l-0.1,0l0-0.09l0.1-0.09
				l0-0.09l0-0.09l0-0.09l0-0.1l0-0.19l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.19,0
				l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.09
				l0-0.09l0-0.09l0-0.19l0-0.09l0-0.09l0.09,0l0-0.09l0-0.1l0-0.09l0.09,0l0-0.09l-0.09,0l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09,0
				l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0-0.19l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0.09,0l0-0.09l0-0.09
				l0.09-0.09l0-0.09l0-0.1l0.09,0l0-0.09l0.1,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0,0.09l0.09-0.09l0.09,0.09l0,0.09
				l0,0.09l0.1-0.18l0-0.09l0.09,0l0.09,0l0-0.09l0.19,0l0.09,0l0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0
				l0-0.09l0-0.09l0-0.19l0.09,0l-0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.19,0l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09
				l-0.09,0l0-0.1l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.09l0.09,0l0-0.09l-0.09,0l0-0.09l0.09-0.09l0-0.19l0.09,0l-0.09-0.03
				l0-0.07l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.19l0-0.09l0-0.1l0-0.19l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09
				l0.09,0l0-0.19l0-0.09l0-0.28l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.1l0.09-0.09l0-0.09l0-0.09l0-0.28l0-0.09
				l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.19l0-0.09l0-0.09l0-0.09l0-0.1l0-0.09l0.09,0l0-0.09
				l0-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.19l0.1,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0.09,0
				l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0-0.09l0-0.1l0.09,0l0-0.19l0.09,0l0-0.09l0.19-0.18l0.09,0l0-0.09l0.09-0.09
				l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.09-0.09l0.19,0l0.19,0l0.09,0l0.09,0l0.09-0.09l0.28,0l0.28,0l0.09,0l0.09,0l0.09,0
				l-0.09,0.09l-0.1,0.18l0.19-0.18l0.09,0l0.09-0.09l0.19,0l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.19,0
				l0.19-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.1,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.1
				l0-0.19l0-0.09l0-0.19l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.19l0-0.09l0-0.09l-0.09-0.09l0-0.09l-0.1-0.09l0-0.19
				l0-0.09l0-0.09l0-0.1l0-0.09l0-0.19l0-0.19l-0.09-0.09l0-0.09l0-0.19l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.19l0-0.09l0-0.19
				l0-0.09l0-0.09l0-0.19l0-0.1l0-0.09l0-0.09l-0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0
				l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.28l0-0.09l0.09-0.09l0-0.09l0-0.09l0.1-0.09l0-0.09l0-0.09l0.09-0.09l0-0.1l0.09,0l0-0.09
				l0.09-0.09l0-0.09l0.09-0.09l0.09,0l0.19,0l0,0.09l0.19,0l0.19-0.09l0.09-0.09l-0.09,0l-0.09-0.09l0.09-0.09l0.19-0.09l0.09,0.09
				l0,0.09l0,0.09l-0.19,0.27l0.28-0.18l0-0.09l-0.09-0.28l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09-0.09
				l0.09,0l0-0.09l0.09,0l0.19-0.18l0.09,0l0-0.09l0.1,0l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0.09l0.09,0
				l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0.09,0l0.09,0l0-0.09l0.09,0.09l0.09,0l0.19,0l0.09,0l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0
				l0.09,0l0.09,0l0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l-0.09-0.09l0.09-0.09l0-0.09l-0.09-0.1
				l0.09-0.09l-0.09-0.09l0.09-0.09l0-0.09l-0.09-0.09l0.1-0.18l0-0.09l0-0.09l0-0.19l0-0.19l0-0.09l0-0.19l0.09-0.09l0-0.19l0-0.19
				l0.09-0.1l0-0.09l0-0.19l0.09,0l0.09,0l0.1,0l0-0.09l0.09,0l0.09-0.09l0-0.09l0-0.09l-0.09-0.09l0.09,0l0-0.09l0-0.19l0.09-0.09
				l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.03c-0.03-0.02-0.06-0.04-0.09-0.06l-0.01,0l0,0
				l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09-0.1l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0
				l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0.1-0.09l0.09-0.09l0.09-0.09l0.09,0l0.09-0.09l0.19-0.18l0.09,0
				l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09,0l0.19-0.09l0.09-0.09
				l0.09-0.1l0.09,0l0.09-0.09l0.09,0l0.09,0l0.09,0l0.1-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0.09-0.09l0.09,0
				l0.09,0l0.09-0.09l0.19,0l0.19-0.09l0.09,0l0.09,0l0.09,0l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0
				l0.09-0.09l0.09-0.09l0.1,0l0.09,0l0-0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09-0.09l0-0.09l0.09,0l0.19-0.09l0.19-0.09l0.09,0
				l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0.09l0.09,0l0.19,0l0-0.09l0.09-0.09
				l0-0.09l0.09-0.09l0-0.1l0.1,0l0,0.1l0.09,0l0-0.1l0.09,0l0-0.09l-0.09,0l0-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.09,0
				l0.09-0.09l0.19,0l0.09,0l0.19,0l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0.09-0.09l0.09,0.09l0.09-0.09l0.09,0.09l0.09,0l0.09,0
				l0.19,0l0.09,0l0,0.09l0,0.19l0.1,0l0,0.09l0.09,0l0.09,0l0.09,0l0,0.09l0,0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0.09,0
				l0-0.09l0.09,0l0-0.19l0.09-0.09l0.19,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0-0.09
				l-0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0-0.1l0.09,0l0-0.09
				l0.1-0.09l-0.1,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.19,0l-0.09-0.09l0-0.09l0-0.19l0-0.19l0.09-0.09l0.1-0.18
				l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.1,0l0.09-0.1l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0
				l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.19-0.09l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.09,0l0-0.09
				l0.09-0.09l0.19,0l0.09-0.09l0.09,0l0-0.09l0.19,0l0.19,0l0.09,0l0.09,0.09l0-0.09l0.09,0l0-0.09l0.1-0.09l0,0.09l0.09,0l0.09,0
				l0,0.09l0.09,0l0.19,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0.09l0-0.09l0-0.09l0-0.09l0.09,0
				l0-0.09l0-0.09l0.19,0l0.09-0.09l0.09-0.1l0.09,0l0-0.09l0.09,0l0.09,0l0.19,0l0.09,0l0.19,0l0.09,0l0.09,0l0,0.09l0.1-0.09
				l0,0.09l0.09,0l0.09,0.1l-0.09,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l-0.09,0
				l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.1l0-0.09l-0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0.09,0l0-0.09l-0.09,0
				l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l-0.09-0.09l0.19,0l0-0.09l0.09,0l0-0.09
				l0-0.09l0.19,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.1
				l0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.1,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09
				l0.09,0.09l0,0.19l0,0.09l0.09,0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19
				l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l-0.09-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.19,0l0-0.09l0.09,0l0.09,0
				l0-0.09l0.09,0l0-0.1l0.09,0l0-0.19l0.09,0l0.1,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.19,0l0.09,0l0-0.09l0-0.09
				l-0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0.09-0.09l0.09,0l0,0.09l0,0.09l0,0.09l0.09,0
				l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l-0.09,0l0-0.09
				L541.08,270.65z"></path>
		</g>
		<g id="valencia_p">
			<path class="st7" d="M530.32,264.46c0.42-0.69-0.46-0.48-0.36-1.12c-1.87-1.47-3.44-3.54-4.75-5.17
				c-0.02-0.27-0.02-0.22,0.01-0.46c-0.18,0.18-0.62,0.27-0.19,0c-0.35-0.53-0.69-1.33-1.18-1.87c-0.59-1.29-1.08-2.21-1.53-3.46
				c0.02-0.14-0.1-0.22-0.27-0.29l0-0.09l0.09,0l0.09,0l-0.09-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09
				l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09-0.1l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09
				l0-0.19l-0.09-0.09l0.09-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0.09-0.09l0-0.09l0-0.09
				l0-0.1l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0,0.09l0.09,0l0-0.09l0-0.09l0.09,0.09l0.09,0
				l0-0.09l0.09,0l0,0.09l0.09,0l0-0.19l0.09,0l-0.09-0.09l0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l-0.09-0.28l-0.09,0
				l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09-0.1l0-0.09l-0.09,0l0-0.19l-0.09-0.09l-0.09-0.19l0-0.09l-0.09,0
				l0-0.09l-0.09-0.09l0-0.09l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l-0.1,0l0-0.09l-0.09-0.19l0-0.09l-0.09,0l0-0.19l-0.09-0.09
				l0-0.1l-0.09-0.09l0-0.19l-0.09-0.19l-0.09-0.09l0-0.09l-0.09-0.19l-0.09,0l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09
				l-0.09-0.09l0-0.09l-0.09-0.09l-0.18-0.28l0-0.09l0-0.1l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l0-0.09l-0.09-0.28
				l-0.09,0l-0.18-0.1l0.09,0l0.18,0.1l0-0.09l-0.09-0.09l0-0.09l-0.09-0.19l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09-0.09l0-0.19
				l-0.09-0.19l0-0.09l-0.1-0.09l0-0.09l-0.09-0.19l0-0.09l-0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09-0.09l0-0.09
				l0-0.09l0-0.09l-0.09,0l0-0.19l-0.09-0.19l0-0.09l0-0.09l-0.09-0.09l0-0.1l0-0.19l-0.09,0l0-0.09l0-0.19l-0.09-0.09l0-0.09
				l0-0.09l0-0.09l-0.08,0c-0.18,0.02-0.37,0.07-0.54,0.14l0.06,0.13l0.09,0.09l0.09,0.28l0.09,0.19l0.09,0.38l0,0.19l-0.09-0.19
				l-0.39-1.23l0.59-0.07l-0.09-0.29l0-0.19l-0.09-0.1l0-0.09l0-0.09l0-0.09l0-0.09l-0.09-0.1l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09
				l0-0.09l0-0.09l0.09,0l0-0.09l0-0.28l-0.09,0l0-0.19l0-0.09l0-0.19l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0l0-0.09
				l0-0.19l0.19,0l-0.09-0.1l-0.19,0l0-0.09l0.09,0l0.09,0l0-0.1l-0.19,0l0.09-0.09l-0.09-0.1l0,0.09l-0.09,0l-0.09-0.1l0-0.09
				l0.09-0.09l0.09,0l0.09,0l0,0.09l0,0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.19l0-0.19l0-0.09l-0.09-0.1l0-0.19l0-0.09
				l0-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.08l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0-0.19l0-0.09l0-0.1l0-0.09l0-0.19l0-0.09
				l0.1-0.18l0-0.09l0-0.19l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09
				l0.09-0.09l0-0.1l0.09-0.09l0-0.09l0.09-0.09l0.1-0.18l0.09,0l0-0.09l0-0.09l0.1,0l0-0.09l-0.1,0l0-0.09l-0.37-0.1l0-0.09
				l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.18-0.1l-0.09,0l0-0.09l0.09-0.09l0.09-0.09l0.09,0l0.09,0l0.09,0.09l0.09,0l0.09,0.09
				l0.09,0l0.09,0.09l0.09,0l0.1,0l0,0.09l0.09,0l0.09,0l0.09,0.09l0-0.09l0.09,0l0-0.19l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0
				l0-0.09l0-0.09l0.09-0.09l0-0.09l0.09-0.1l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.09
				l0-0.09l0-0.09l0.1-0.18l0-0.19l0.09-0.09l0.1-0.18l0-0.09l0.09-0.1l0-0.09l-0.09,0.09l-0.09,0l-0.19,0.09l-0.73-0.48l-0.37-0.38
				l-0.09,0l-0.09,0l-0.09,0l-0.28-0.1l-0.74-0.29l-0.65-0.1l-0.47,0.27l0,0.09l-0.47,0.18l-0.56,0.27l0.27,0.56l0,0.09l0.18,0.28
				l0,0.09l0.09,0l0,0.28l-0.22,0c0.2-0.09,0.3-0.17,0.13-0.28c-0.68-1.26-0.82-2.28-1.24-3.64c0.68-0.83-0.22-2.65-1.17-2.24
				c-1.53,0.47-2.71-1.13-3.3-2.46c-0.17-0.83-1.01-2.25-2.02-1.24c-1.62,1.4-0.05,3.13-2.91,2.28c-1.29-0.06-0.73-1.45-0.62-2.33
				c-0.21-1.37-0.6-3.18-2.09-3c-0.54,0.85-0.65,2.68-1.69,1.46c0.13-0.61-0.16-0.35-0.27-0.75c-1.13-1.23-1.43-3.27-3.5-1.63
				c-1.7,1.1-0.59,1.27-2.33,0.8c-1.49-1.15-0.7-1.09-0.52-2.33c-0.42,0-0.34-0.77-0.45-1.21c0.25-0.63,0.52,0.05,0.29-1.01
				c-0.05-0.41-0.43-0.4-0.45-1.03c-0.19,0.03-0.1-0.42-0.28,0c-0.48,0.29-1.15-0.42-1.75-0.67c-0.77-0.33-1.44-0.69-2.32-0.03
				c-1.44-0.45-1.95-0.43-2.88,0.33c-0.28-0.41-0.33-0.44-0.56-0.01c-0.1-0.46-0.72,0.13-0.47,0.46c-0.73-1.42-2.09,0.53-3.35,0.6
				c-0.3,0.41,0.03,0.9-0.2,1.02c0.1,0.27,0.35,0.23,0.46,0.56c-0.01,0.87,0.12,2.76-0.24,3.71c-0.27,0.32-0.22,0.6-0.09,0.87
				l-0.11-0.04l-0.1,0.18l0,0.19l0,0.19l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l0.09,0.09l0,0.09l0,0.09l-0.1,0.18l-0.09,0.09
				l-0.19,0.46l-0.1,0.19l-0.28,0.46l-0.19,0.09l-0.09,0.09l-0.28,0.37l-0.47,0.73l-0.09,0.09l-0.1,0.28l0,0.09l-0.09,0l-0.1,0.18
				l-0.1,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.09l0.1,0l-0.11,0.65l0,0.09l0.09,0.19l0.09,0.28l0.27,0.47
				l-0.28,0.46l-0.1,0.28l-0.19,0.27l-0.19,0.09l-0.47,0.27l-0.09,0.09l-0.19,0l-0.37,0.09l-0.19,0l-0.28-0.1l-0.09,0l-0.18-0.1
				l-0.19-0.1l-0.27-0.19l-0.09-0.09l-0.09,0l-0.65-0.19l-0.09-0.09l-0.18-0.1l-0.09-0.09l-0.66,0.64l-0.38,0.28l-0.19,0.18
				l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l-0.38,0.46l-0.19,0.09l-0.29,0.55l-0.38,0.83l-0.19,0.55l-0.19,0.37l-0.2,0.65l-0.1,0.37
				l-0.1,0.28l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.28,0.27l-0.09,0l-0.19,0.09l-0.09,0l-0.19,0
				l-0.09,0.09l-0.19,0l-0.28,0.09l-0.28,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l-0.09,0.1l0.09,0.09l0,0.09l-0.09,0l-0.09,0l-0.1,0
				l-0.09,0l-0.09,0l-0.09,0.09l-0.09-0.09l0,0.09l-0.09,0l0,0.09l0.09,0l0.09,0l0.09,0.09l0.09,0.09l0.09-0.09l0.1,0l0.09-0.09
				l0,0.09l0.09,0.09l-0.09,0.09l-0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l0.09,0.09l0,0.09
				l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0.09,0l0.09,0.09l0,0.09l0.09,0l0,0.1l-0.19,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09
				l0.09,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0l-0.09,0.09l0,0.19l-0.09,0.09l0,0.09l-0.09,0l-0.09-0.09l-0.1-0.09
				l-0.09-0.09l0,0.09l0,0.09l0.09,0l0,0.09l-0.09,0.09l0.09,0l0.1,0l0.09,0.09l0,0.09l-0.09,0l-0.1,0l-0.09,0.09l0,0.09l0,0.09
				l0.09,0l0.1,0l0.19,0l0,0.09l0,0.09l-0.09,0l-0.09,0l-0.1,0l0,0.09l-0.09,0.1l0,0.09l0,0.09l0,0.19l0,0.09l0,0.09l0,0.09l0,0.09
				l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0.09l-0.09,0l0,0.19l0,0.09l0.19,0l0,0.09l0.09,0l0-0.09
				l0.09,0.09l0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0.09l0,0.09l-0.09,0
				l-0.09,0.1l-0.09,0l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0.09-0.09l0-0.28l0.09,0l0.1,0l0,0.09l0,0.09l0.09,0.09
				l0.09,0l0.19,0l0.09,0l0,0.09l-0.09,0.09l-0.09,0l0,0.09l-0.09,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0-0.19
				l0.09-0.09l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0,0.19l-0.09,0l-0.09,0l-0.09,0l0,0.09l0,0.09l0,0.09l0.09,0.09l-0.09,0
				l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09
				l0.09,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l0,0.09l0,0.1l0.09,0.09
				l0.09,0l0.09,0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0-0.1l0-0.09l0-0.09l0.09-0.09l0.09,0l0.09,0l0.09,0l0.1,0.09l0,0.19l-0.1,0.1
				l0,0.09l0.1,0l0,0.09l-0.1,0l-0.09,0l0,0.09l0,0.09l0.09,0l0.1,0l0,0.09l0,0.09l-0.1,0l-0.09,0l0-0.09l-0.09-0.09l-0.09,0
				l-0.09-0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0.09l0.1,0
				l0.09,0l0.09,0l0,0.19l-0.09,0l-0.09,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.1,0l0,0.09
				l0,0.09l0.19,0l0.19,0l0.09,0l0-0.09l0-0.09l0-0.19l0.09,0l0-0.09l0.09-0.09l0-0.09l0.19,0l0.09,0.09l0-0.09l-0.09-0.09l0-0.09
				l0.19,0l0.09,0.09l0,0.19l0,0.09l-0.09,0.09l0,0.09l-0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0-0.09
				l0-0.28l0-0.09l0-0.09l-0.09,0l0-0.19l0-0.09l0.09,0l0.19,0l0.09,0l0.09,0l0.09,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0.09,0
				l0.09-0.09l0.1-0.18l0.09,0l0.09,0l0,0.09l0.1,0l0,0.09l-0.1,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l0,0.09
				l-0.09,0l0,0.09l0.09,0l0.18,0.1l0.19,0l0.1,0l0.09-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0,0.19l0,0.09
				l0,0.09l-0.09,0l0,0.09l0,0.09l0.09-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0,0.09l0.09-0.09l0.09,0l0.09,0l0,0.09l0,0.09l0.09,0
				l0,0.1l0.09,0l0,0.19l0.09,0l0.09,0l0,0.09l0.09,0.19l0,0.09l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09-0.09l0.09,0l0,0.09l0.09,0
				l0,0.19l0.1,0l0,0.09l0.09,0l0-0.09l0.09-0.09l0,0.09l0.09,0l0-0.09l0-0.09l-0.19,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0
				l0,0.09l0.09,0l0.19,0l0-0.09l0-0.09l0.09,0l0,0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.19,0
				l0,0.09l-0.09,0.09l0,0.09l0.09,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0.19,0l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0.1,0
				l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0l0.19,0l0-0.09l0.09,0l0,0.09l0.19,0l0.18,0.19
				l0,0.09l0.19,0l0-0.19l0-0.09l0.09,0l0,0.09l0.09,0.09l0,0.19l0.09,0l0.09-0.09l0.09,0.09l0.28,0.1l0.09,0l0.09,0l0,0.09
				l0.1,0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.19,0l0,0.09l0.09,0.09l0,0.09l0.09,0
				l-0.09,0.09l0.09,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0l0-0.09l0.19,0l0.09,0l0.19,0l0.09,0l0.09,0l0.09,0l0.09,0.09l0.19,0
				l0.09,0l0.1,0.09l0.09,0l0,0.1l0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l-0.1,0.28l0,0.09l-0.29,0.74l-0.01,0.37l0,0.19l0,0.09
				l-0.09,0l0,0.09l0,0.28l0,0.09l0,0.19l0,0.28l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.19l0,0.09l-0.01,0.37l-0.09,0.1l0,0.19
				l-0.1,0.65l-0.19,0.37l-0.09,0.09l-0.09,0.09l0,0.09l0,0.09l-0.1,0.18l0,0.09l0,0.09l0,0.09l0,0.1l-0.09,0.09l0,0.28l-0.09,0.09
				l0,0.09l-0.19,0.37l-0.09,0.09l-0.09,0.09l-0.19,0.18l-0.66,0.55l-0.11,0.74l-0.1,0.28l-0.19,0.46l0,0.19l-0.1,0.37l-0.65,0.36
				l-0.02,1.2l-0.38,0.46l0.09,0.38l0,0.09l-0.1,0.55l0.09,0.19l-0.09,0l0.09,0.37l0.18,0.19l0.28,0.1l0.09,0l0.18,0.28l0.28,0.19
				l0.27,0.19l0.19,0.1l0.09,0l0.45,0.66l0.09,0.09l0.18,0.28l0.09,0.28l0.09,0l0.09,0.28l0,0.09l0.18,0.1l0.09,0.09l0.09,0
				l0.09,0.19l0.09,0.09l0,0.09l0.09,0.09l0.18,0.19l0.27,0.47l0.36,0.38l0.18,0.19l0,0.09l0.09,0.09l0.09,0.19l0,0.09l0,0.1l0.09,0
				l0,0.09l0,0.19l0.09,0l0.18,0.37l0.09,0.09l0,0.09l0.37,0.1l0.09-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.19,0l0.19,0l0.09,0
				l0.28,0l0.28-0.09l0.09,0l0.19-0.09l0.19,0l0.09,0l0.37-0.09l0.28,0l0.09,0l0.28-0.09l0.28,0l0.46-0.09l0.09,0l0.09,0.09l0.09,0
				l0.28-0.09l0.09,0l0.09,0l0.09-0.09l0.19,0l0.19-0.09l0-0.09l0.09-0.09l0.1-0.19l0.1-0.09l0.09,0l0.09-0.09l0.09,0l0.28,0.1
				l0.19,0l0.18,0.1l0.09,0.09l0.09,0.1l0.09,0l0.19,0l0.28-0.09l0.19,0l0.09,0.28l0,0.28l0,0.09l0,0.19l0,0.09l0.09,0l0,0.19
				l0,0.09l0.18,0.19l0,0.09l0.09,0.19l0.09,0.19l0.27,0.47l0.09,0l0.55,0.38l-0.1,0.37l-0.19,0.28l-0.01,0.46l0,0.19l0.09,0.19
				l-0.19,0.37l-0.01,0.37l0.09,0.37l0.09,0.46l0,0.19l0.27,0.37l-0.1,0.37l-0.19,0.18l0,0.28l0,0.19l-0.1,0.46l0.27,0.37l0,0.19
				l0.37,0.1l0.19,0l-0.1,0.28l0,0.09l0.18,0.28l0.09,0l0.09,0.09l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.19-0.09l0.09,0l0.09,0
				l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.19,0l0.09,0l0.09,0l0.19,0l0.28,0l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0,0.09
				l0.09,0l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0.19l0.09,0.19l0.09,0l0,0.09l0,0.09l0,0.19l0.09,0.19
				l0.09,0l0.09,0l0.19-0.18l0.19,0l0-0.09l0.09,0l0.09,0l0.19-0.09l0.09,0l0,0.1l0.09,0l0,0.09l0.09-0.09l0.09,0l0.09,0l0.19-0.09
				l0.09,0l0-0.09l0.09-0.09l0-0.09l0.19-0.18l0.09-0.09l0.1,0.09l0.09,0l0.19-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.19,0l0.09,0.09
				l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09-0.09l0.09,0l0.09,0l0.19,0l-0.09-0.19l0.19,0l0.09,0.09l0.09,0l0.09-0.09l0.09,0.09
				l-0.09,0.09l-0.09,0.09l0.09,0.09l0.09,0l0.09,0l0.09,0l0.1-0.09l0.09,0l0.18,0.1l0.09,0l0,0.09l0.18,0.28l0,0.09l0.09,0
				l0.09,0.19l0,0.09l0,0.1l0,0.09l0.09-0.09l0.28-0.09l0.09,0l0-0.09l0.28-0.18l0.09,0.09l0,0.09l0.09,0.09l0.09,0.1l0,0.09
				l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09,0l0.1,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.09,0l0.19,0l0.09,0l0.09,0.09
				l0.09,0l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0.09l0,0.19l0,0.09l-0.09,0l0,0.09
				l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0,0.1l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.46,0.01l0.09-0.09l0-0.09
				l0.1,0l0.19-0.18l0.09-0.1l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09
				l0.09-0.09l0.19-0.09l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.19,0l0.09-0.09l0.09,0l0.09,0l0-0.09l0.1,0l0.09,0l0.09,0
				l0-0.09l0.09-0.09l0.19-0.18l0.09-0.09l0.1-0.18l0-0.09l0.09,0l0-0.09l0.09,0l0-0.1l0.09,0l0.09-0.09l0.09,0l0.09,0l0-0.09
				l0.09,0.09l0-0.19l0.09,0l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0-0.28l0-0.19l0-0.09l-0.28,0l-0.28,0
				l-0.09-0.09l-0.09-0.09l0-0.09l0-0.09l-0.18-0.19l-0.19,0l0,0.09l-0.19,0l-0.09,0.09l0,0.09l-0.09,0l-0.37,0.18l-0.09,0
				l-0.1,0.09l0,0.09l-0.09,0.09l-0.18-0.1l-0.09,0l-0.09-0.09l-0.09-0.09l0-0.09l-0.09,0l0-0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0
				l-0.09,0l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.1l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l-0.09,0l0-0.09
				l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.19-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.19,0l0.09,0
				l0.09,0l0.19,0l0.09-0.09l0.19-0.09l0.09,0l0.19,0l0.19-0.09l0.09,0l0.09,0l0.19-0.09l0.09-0.09l0.09,0l0.19-0.09l0.09,0l0.09,0
				l0.09,0.09l0.09,0l0.09,0.09l0.09,0l0.19,0l0.19,0l0.18,0.1l0.37,0.01l0.28,0l0.37,0.01l0.18,0.1l0.19-0.18l0.28-0.09l0-0.09
				l0.1-0.28l0.28-0.09l0.19-0.18l0.19,0l0,0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0.1-0.09l0.19-0.09l0.09,0l0.19-0.09l0.46-0.09
				l0.09-0.09l0.19,0l0.09-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09l0.09,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0.09,0l0.18,0.1l0-0.09
				l0.09,0.09l0.09,0l0.19-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.05c0.14-0.06,0.3-0.13,0.46-0.22l0.01,0l0,0
				c0.39-0.21,0.83-0.51,1.31-0.91l0.09,0l0-0.09l0.1-0.09l0.19-0.09l0-0.09l0-0.06c0.01-0.01,0.02-0.02,0.03-0.03l0.06,0l0.05-0.1
				c0.25-0.19,0.58-0.28,0.97,0.03c1.1,0.8,2.49,1.16,3.79,0.7c-0.08-1.13,1.16-0.76,1.51-1.56c0.25,2.85,1.25,0.49,2.68,0.78
				c0.12,0.04,0.32-0.08,0.1-0.09c0.07-0.09,0.06-0.28,0.19-0.27c0.09-0.25,0.08,0.27,0.47-0.27c0.41-0.15,0.37,0.11,0.46,0.47
				c0.51,0.2,0.76,0.91,1.29,0.95C530.1,264.82,530.65,264.56,530.32,264.46z M473.56,200.61c-0.56,0.53,0.79,1.69,0.89,2.52
				c0.7,2.49,1.22,1.59,3.31,1.98c1.54,0.52,2.71,0.93,4.35,0.45c-0.04-0.21,0.2-0.03,0.28,0c0.2,0,0.34-0.23,0.56-0.18
				c0.11-0.55,1.01-0.1,1.21-0.54c1.12-0.15,2.77-0.82,3.1-2.09c-0.22-1-1.25-1.84-2.29-2.17c-0.35,0.02-0.15-0.52-0.46-0.47
				c-0.5-1.22-2.31,0.4-3.24-0.51c-0.48,0.16-0.94,0.21-1.11-0.3c-0.23,0.03,0.03-0.18,0.1-0.28c-0.12-0.24,0.02-0.63-0.08-0.65
				c0.23-0.22,0.06-0.55-0.18-0.19c-0.97-0.95-1.66-1.98-1.72-3.27c0.21-0.04-0.14-0.4-0.09-0.19c-1.35,0.52-0.87,1.59-0.88,2.77
				c0.43,0.66-1.21,0.93-1.41,1.37c-0.57,0.37-0.09-0.89-1.3-0.39c-0.3,0.6-1.23-0.69-1.76-0.03c-0.76-0.69-1.45,0.15-0.66,0.82
				C472.12,199.61,474.37,200.23,473.56,200.61z"></path>
		</g>
		<g id="castellon_p">
			<path class="st7" d="M550.35,177.25l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.19,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0
				l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.28,0l-0.28-0.1l-0.09-0.09l0-0.09l-0.09,0
				l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.1,0l-0.18-0.1l-0.09,0l0-0.09l-0.09-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09
				l-0.09-0.09l0-0.09l-0.09-0.19l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l-0.09,0l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09
				l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l-0.09-0.09
				l-0.28-0.1l-0.09-0.09l-0.09,0l0-0.09l0.09-0.09l-0.09,0l0-0.09l-0.09-0.09l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09
				l-0.1-0.1l-0.09,0l-0.09,0.1l-0.09,0l-0.09-0.1l-0.09,0l-0.09,0l0,0.1l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0
				l-0.09,0l-0.09-0.1l-0.09,0l-0.19,0l-0.09,0l-0.09,0l-0.19,0l-0.18-0.1l-0.09,0l-0.1,0l-0.09-0.09l-0.09,0l-0.09,0l0-0.09
				l-0.09,0l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.18-0.1l0-0.09l-0.09,0l0-0.09l-0.09,0l-0.19,0.09
				l-0.09-0.09l-0.28-0.1l-0.09-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l-0.09,0l-0.1,0l0-0.09l-0.09-0.09l-0.09-0.09
				l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.19l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0.19-0.18l0-0.19l0-0.09
				l0.1-0.09l0.09-0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09
				l-0.18-0.1l-0.09,0l-0.09-0.09l-0.09-0.09l-0.09,0l-0.09,0l0-0.09l-0.1-0.09l0-0.09l-0.09,0l0-0.09l-0.09,0l0-0.09l-0.09-0.09
				l-0.09,0.09l0-0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.19,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09-0.09l-0.09,0
				l-0.09,0l0-0.09l0-0.09l-0.09-0.09l-0.09-0.09l-0.18-0.28l0-0.09l0-0.09l0-0.09l-0.09-0.09l0-0.09l0-0.09l0-0.19l0-0.19l-0.09,0
				l-0.09,0l-0.1,0l-0.09,0l-0.09,0.1l0,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l0-0.09l-0.09,0
				l0-0.09l0-0.09l-0.09,0l-0.09-0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.37-0.19l-0.09,0.09l-0.09,0.09l-0.09,0.09
				l0,0.09l0,0.09l-0.09,0l-0.1,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l-0.28,0l-0.28-0.1
				l-0.09,0l-0.27-0.19l-0.19,0l0-0.09l-1.59,0.62l-1.23,1.28l-0.18-0.1l-0.28-0.1l-0.09,0l-0.09-0.1l0-0.09l0-0.09l0.09,0
				l-0.09-0.09l-0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.1-0.18l0-0.09l-0.09-0.09l-0.09,0l0-0.09l-0.09,0l-0.09-0.09l-0.09-0.09
				l0-0.09l0-0.09l0-0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.19,0l-0.09,0l-0.09-0.09l-0.09,0.09l-0.09,0l-0.09,0
				l-0.09-0.09l0,0.09l-0.1,0l-0.09-0.09l-0.09,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0
				l-0.09,0l-0.18-0.1l-0.09,0l-0.09,0l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0l-0.28,0l-0.09,0l-0.18-0.1l-0.46-0.19l-0.18-0.19l0-0.09
				l-0.09,0l-0.09,0l-0.09,0l-0.09-0.1l-0.09,0l0-0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09-0.19l-0.09,0
				l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0l-0.09-0.19l-0.1,0l0-0.09l-0.09,0l0-0.09l0-0.09
				l-0.09-0.19l-0.09,0l-0.09,0l-0.09-0.09l0-0.09l0-0.09l-0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.1l0-0.09l-0.09-0.09
				l-0.28,0l-0.37-0.1l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09-0.09l-0.19,0.09l-0.28,0l-0.09,0.09l-0.09,0l-0.28,0l0,0.1
				l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l0.09,0.09l-0.09,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.28
				l-0.09,0l0,0.19l-0.09,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.19l-0.09,0.1l0,0.09
				l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0.09,0l-0.09,0.09l0,0.19l-0.09,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0.09l0,0.09
				l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0,0.09l0.09,0l-0.19,0.56l0.09,0.37l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.19,0l-0.1,0.09
				l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0l0,0.09l-0.09,0.09
				l0,0.09l-0.09,0l-0.09,0.09l-0.09,0.09l-0.09,0l-0.09,0l0-0.09l-0.09,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0
				l0,0.09l-0.09,0l-0.09,0l0,0.1l0,0.09l0,0.09l-0.28-0.19l-0.09,0l0-0.1l-0.09-0.09l-0.09-0.09l0-0.19l-0.09-0.09l0.09-0.09
				l0-0.09l-0.09,0l0-0.09l-0.09,0.09l-0.09-0.09l-0.09-0.09l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0
				l-0.19,0.09l-0.09,0l-0.09,0l-0.09,0.09l-0.47,0.18l0,0.09l-0.09,0l0,0.09l0.09,0l-0.09,0.09l-0.09,0l-0.09,0l-0.09,0.09
				l-0.09,0.1l0,0.19l0.09,0l0,0.19l0.28,0.19l0,0.09l0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l-0.09,0
				l-0.1,0.18l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.19l0,0.09l0,0.09l0,0.09l0.09,0.09l0.09,0l0.09,0.09l0,0.09l0.1,0
				l0.09,0l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0l0.09,0.09l0.09,0l0.18,0.1l0.09,0l0.09,0l0,0.19l0,0.09l0.09,0l0.09,0l0.09,0l0.09,0
				l0.09,0l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09,0l0.1-0.09l0.09,0l0.09,0
				l0.09-0.09l0.09,0.09l0-0.09l0.09,0l0.09,0l0.09,0l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0.09,0l0,0.09
				l0,0.09l0,0.09l-0.09,0l0.09,0.09l0,0.09l0,0.09l0,0.09l-0.09,0.09l0.09,0.09l0,0.1l0,0.09l-0.09,0.09l-0.09,0l0,0.09l0,0.09
				l0,0.09l0.09,0.09l0.09,0.09l0,0.09l0,0.09l-0.09,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09
				l0,0.09l0,0.09l0,0.09l-0.09,0l0.09,0.09l0,0.09l0,0.1l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0
				l0,0.19l0,0.09l0,0.19l-0.09,0.09l0,0.19l0,0.09l0.09,0.09l0,0.19l0.09,0.09l0,0.09l0.09,0l0.19-0.09l0-0.09l0.09,0l0.09,0
				l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0.09l0.09,0l0,0.1l0,0.09l0,0.09l0.09,0l0,0.09l0,0.09l-0.09,0.09l-0.09,0.09l0,0.09
				l0,0.09l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09c-0.02-0.3-0.2,0.27-0.28,0.1
				c-0.1,0.08-0.38,0.05-0.37,0.27c-0.15-0.05-0.2,0.01-0.46-0.01c-0.03,0.12,0.15,0.08,0.09,0.28c-0.04-0.08-0.29-0.17-0.09,0
				c-0.11,0.06-0.21-0.21-0.19,0c-0.02,0.01-0.14-0.2-0.09,0c-0.1,0.02-0.32-0.07-0.28,0.09c-0.44,0.14-0.61,0.21-1.03,0.36
				c-0.13,0.2-0.02,0.42-0.29,0.55c0.32,0.37-0.2-0.01-0.1,0.18c0.83,1.48,2.6,2.48,2.73,3.85c-0.17-0.02-0.27,0.05-0.1,0.18
				c-0.19,0.09-0.38-0.23-0.37-0.01c-0.26-0.04-0.23,0.48-0.47,0.18c-0.32,0.08-0.07,0.29-0.47,0.18c-0.15,0.18-0.17,0.48-0.38,0.55
				c-0.01,0.09-0.09,0.18-0.19,0.18c-0.23-0.03,0.05,0.27-0.28,0.18c0,0.34-0.2-0.08-0.19,0.18c-0.24,0.01,0.03,0.09,0,0.19
				c-0.24,0.01,0.03,0.09,0,0.19c-0.05,0.27-0.12,0.5-0.28,0.18c-0.18-0.06,0.19,0.51-0.1,0.46c-0.16,0.01-0.07,0.56-0.47,0.27
				c0.1-0.2-0.36,0.16-0.28,0.27c0.37,0.05-0.17,0.44-0.09,0.09c0-0.25-0.09,0.16-0.19,0.09c-0.01-0.09-0.08-0.18-0.18-0.19
				c0.04,0.15-0.14,0.03-0.1,0.18c-0.14,0.04-0.04-0.13-0.18-0.1c0.05,0.09-0.04,0.1-0.09,0.09c0.17,0.09-0.05,0.59-0.1,0.28
				c-0.15-0.03-0.1,0.13,0,0.19c-0.21-0.07-0.45,0.23-0.1,0.18c-0.07,0.29,0.32,0.2,0.36,0.47c0.02,0.09-0.06,0.22,0.09,0.19
				c0.33,1.71-1.58,0.09-2.96,2.07c-0.04,0.07-0.16,0.12-0.31,0.17l0-0.06l-0.09,0l0,0.09c0,0-0.01,0-0.01,0l-0.08,0l0,0.02
				c-0.34,0.08-0.72,0.16-0.75,0.34l-0.01,0.01c-0.24-0.06-1.26-0.34-1.38-0.26l0-0.04l-0.08,0c0,0-0.01-0.01-0.01-0.01l0-0.08
				l-0.09,0l-0.18-0.19l0-0.09l-0.28-0.19l0-0.09l-0.09,0l-0.09,0l0,0.26c-0.06,0-0.1,0.03-0.09,0.11l-0.1,0.65l-0.2,0.83l0,0.01
				c-0.02,0.08-0.05,0.16-0.07,0.24l-0.03,0.03l0,0.06c-0.15,0.46-0.31,0.93-0.45,1.38l-0.03,0.04l0,0.06
				c-0.12,0.37-0.21,0.73-0.27,1.08c0.01,0.07,0.14,0.15,0.35,0.23l0,0.01l-0.09,0.09l-0.09,0.09l0,0.09l0,0.09l-0.09,0l-0.09,0
				l0,0.09l0,0.1l0,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0.09l-0.09-0.09l-0.09,0l-0.09,0l-0.09,0
				l0,0.09l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l-0.09-0.09l-0.09,0l0,0.09l-0.09,0.09l-0.09,0.09l-0.1,0.18l-0.48,0.74l0,0.19
				l0.09,0.19l0,0.19l0.1,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0.19l0.09,0.37l0.09,0.09l-0.09,0l-0.09,0
				l-0.09,0.09l-0.09-0.09l-0.09,0l0-0.09l0-0.09l0.09,0l-0.09-0.09l0-0.09l-0.1,0l-0.09,0.09l-0.09,0l0,0.09l0,0.09l0,0.09l-0.09,0
				l0-0.09l-0.09-0.09l0,0.09l0,0.09l0,0.09l-0.09,0l-0.09,0l-0.09,0l-0.09,0l-0.09,0.09l0,0.1l-0.09,0.09l-0.19,0.18l-0.09,0.09
				l-0.09,0.09l-0.19,0.28l-0.09,0l-0.19,0.09l-0.09,0.09l-0.09,0l-0.09,0.09l-0.09,0l-0.09,0.09l-0.1,0l-0.09,0l-0.09,0l-0.28-0.1
				l-0.19,0l-0.09,0l-0.19,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l0-0.09l-0.09,0l-0.09,0
				l-0.09-0.09l-0.09,0l-0.09,0l-0.19,0.09l-0.09,0l-0.19,0l-0.09,0l0,0.09l-0.09,0l-0.09-0.09l0-0.09l-0.09-0.09l0-0.09l-0.09,0
				l-0.09,0.09l-0.09,0.09l-0.09,0.09l-0.09,0l0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.1l-0.09,0.09l0,0.09
				l0.09,0l0,0.09l0,0.09l-0.09,0l0,0.09l0,0.09l0,0.19l0,0.09l0,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0
				l0.09,0.09l-0.09,0l0,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l-0.09,0.09l0,0.09l-0.09,0l0,0.09l-0.09,0l-0.09,0l-0.09,0
				l0-0.09l-0.09,0l-0.09,0l-0.1,0.09l-0.09,0l-0.09,0l-0.09,0.1l0,0.09l-0.19,0.18l-0.09,0l-0.09,0l-0.09,0.09l-0.09,0l0-0.09
				l-0.28,0.18l-0.09,0l-0.09,0l0,0.09l-0.09,0.09l0,0.19l-0.19,0.18l0.18,0.1l0,0.09l0.09,0.19l0,0.09l0,0.09l-0.09,0l0,0.09
				l-0.09,0.09l-0.19,0.09l0,0.09l0,0.09l-0.09,0.1l0.18,0.19l0.09,0.09l0.09,0.19l0,0.09l0.09,0.09l0,0.09l0,0.28l-0.09,0.09
				l-0.09,0.09l0.18,0.1l0,0.09l0.09,0.19l0.09,0l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.19l0.27,0.19l0,0.09l0.09,0l0.09,0.09
				l0.09,0.09l0.09,0l0.09,0l0.18,0.19l0.09,0.09l0.1,0.09l0,0.09l0,0.19l0.09,0l0,0.09l0.09,0l0.09,0l-0.09,0.09l0.19,0l0.09,0.09
				l0.55,0.38l0.18,0.1l0.36,0.66l0.09,0.09l0,0.09l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.19l0.09,0l0,0.09l0.09,0.09
				l0.09,0.09l0.1,0l0,0.19l0.09,0l0,0.09l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09,0.09l-0.09,0l0,0.09l0,0.1l0,0.09l0,0.09l0.09,0
				l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0,0.09l0.09,0l0.09,0l0,0.09l0.09,0l0.09-0.09l0.19-0.09l0.09,0l0.1-0.37l0.09-0.09
				l-0.09-0.1l0.09-0.09l0-0.28l0-0.09l0.28-0.18l0-0.19l0.19-0.37l0.09-0.09l0.09-0.09l0.19,0.19l0.18,0.1l0.09,0l0.46,0.38
				l0.28,0.1l0.28,0.1l0,0.19l0.09,0.28l0,0.19l0.09,0.37l0,0.28l0.26,1.02l-0.1,0.18l-0.01,0.93l-0.2,0.83l0.46,0.19l0.28,0.19
				l0.19,0l0.37,0.1l0.09,0l0.65,0.11l0.19,0l0.46,0.01l0.1-0.37l0.1-0.37l0.2-0.65l0-0.19l0.75-0.92l0.37-0.27l0.37-0.09l0.19,0
				l0.19,0l0.09,0.09l0.18,0.28l0.09,0.09l0,0.09l0,0.09l0.1,0l0.09,0.09l0,0.1l0.09,0l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09
				l0.09,0.09l0,0.09l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0.09,0l0,0.09l0.09,0.09l0,0.09l0.09,0.09l0,0.09l0.09,0.09
				l0.27,0.28l0.27,0.29l0.09,0.09l0.09,0.09l0.18,0.1l0.09,0.09l0,0.09l0.09,0.09l0.09,0.09l0.18,0.19l0.09,0.28l0.19,0l0.19,0
				l0.09-0.09l0.09,0l0-0.09l0.09,0l0.28,0l0.46,0.01l0.37,0.01l0.09-0.09l0.09,0l0.19,0l0.19,0.28l0.09,0.37l0.09,0.19l0.09,0.19
				l0.18,0.19l0.09,0.09l0.09,0.19l-0.09,0.09l0,0.09l0,0.09l-0.19,0.27l-0.1,0.28l0.09,0.19l0.09,0.09l0,0.09l0.09,0.09l0,0.19
				l0,0.09l0,0.09l0,0.19l0,0.09l0,0.09l0,0.09l0.09,0.09l0,0.09l0,0.09l0,0.09l0.09,0.19l0.09,0.09l0,0.09l0.09,0.09l0.09,0.19
				l0,0.09l0.09,0.09l0,0.09l0.56-0.27l0.47-0.18l0-0.09l0.47-0.27l0.65,0.1l0.74,0.29l0.28,0.1l0.09,0l0.09,0l0.09,0l0.37,0.38
				l0.73,0.48l0.19-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09-0.09l0.1-0.18l0.09-0.09l0.09,0l0-0.09
				l0.09-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0.1,0l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0.09-0.09l0.09-0.09l0.09-0.09l0.09,0
				l0-0.09l0-0.09l-0.09,0l0-0.09l0.09,0l0.1-0.28l0.18,0.1l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09
				l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.09l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.19l0-0.09l0-0.09l0-0.09
				l0.09-0.09l0-0.19l0-0.09l0.1-0.18l0-0.09L523,220l0-0.09l0.09-0.09l0-0.09l0-0.09l0.1-0.18l0-0.09l0.09,0l0-0.19l0.09-0.1
				l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.1-0.09l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0
				l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0.09,0l0-0.19l0.09,0l0-0.09l0-0.1l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09
				l0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.1-0.18l0-0.09l0.09-0.09
				l0.1-0.18l0.09-0.09l0-0.09l0.09-0.09l0-0.1l0.1-0.18l0-0.09l0.09-0.09l0.1-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09
				l0.09-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09-0.09l0.09-0.09l0-0.09l0.1-0.18l0.19-0.28
				l0.09-0.09l0-0.09l0.09-0.09l0.09,0l0.18,0.1l0.09,0.09l0-0.09l-0.18-0.1l0-0.09l0.09,0l0-0.09l0.09,0.09l0-0.09l0.09-0.09
				l0-0.09l0.09,0l0-0.09l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0.19-0.09l0.19-0.18l0.09-0.09l0.09-0.09l0.1-0.18l0.09,0l0-0.09
				l0.09-0.09l0-0.1l0.09,0l0.09-0.09l0.09-0.09l0.09-0.09l0-0.09l0.09-0.09l0.09,0l0-0.09l0.09,0l0.09-0.09l0.09,0l0-0.09l0.1-0.18
				l0.09-0.09l-0.09,0l0-0.09l-0.09,0l0-0.04c0.9,0.17,0.32-2.58,1.16-3.01c0.34-0.32,0.47-1.71,0.96-2.49
				c0.09-1.04,0.98-1.34,1.51-1.93c0.97-0.16,1.5-0.12,2.05-0.9c0.14-0.17,0.06-0.63,0.47-0.64c0,0.25,0.26-0.03,0.19-0.18
				c-0.28,0.02,0.09-0.45-0.09-0.38c0.21-0.42,0.27-0.82,0.48-1.2c0.33-1.25,0.88-2.88,2.19-3.77c0.61-0.9,1.57-0.56,1.55-0.99
				l0.04,0.09l0.09,0l0.09-0.1l0-0.09l0.09,0l0.09,0l0-0.09l-0.09,0l0-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l0.09,0
				l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0
				l0.09-0.1l0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0.09,0l0-0.09l0.09,0
				l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.1,0l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0.09l0-0.09l0.09,0
				l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.1l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0
				l0-0.09l0.09,0l0.09,0l0.09-0.09l0.09,0l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0-0.09l0-0.09l0.09-0.09
				l0.09-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0-0.1l0.09-0.09l0.09-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09
				l0.1,0l0-0.09l0.09,0l0.09-0.09l0.09-0.09l0.09-0.09l0-0.09l0.09,0l-0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09
				l0.09,0l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0.09,0.09l0.09-0.09l0-0.09l-0.09,0l-0.09,0l0-0.09l0-0.1l0-0.19
				l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0-0.09l0.1-0.18l0-0.19l0.1-0.28l0.1-0.18l0-0.09l0-0.09l0.09-0.09l0-0.09
				l0.09,0l0-0.1l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0,0.09l0.09-0.09l0-0.09
				l0-0.09l0-0.09l0-0.19l0.1-0.09l0-0.09l0.09-0.09l0-0.09l0.09-0.09l0-0.09l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.1
				l0.09,0l0-0.09l0.09-0.09l0-0.09l0.09,0l0-0.09l0-0.19l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0.09-0.09l0.09-0.09l0.09,0l0-0.09
				l0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l0-0.1l0-0.09l0-0.09
				l0.09,0l0-0.09l0-0.09l0.09,0l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0-0.09l0.1,0l0-0.09l0-0.09l0.09,0
				l0-0.09l0.09,0l-0.09-0.09l0.09,0l-0.09-0.09l0.09,0l0-0.09l0-0.09l0.09,0l0-0.09l0.09,0l0-0.09l0-0.09l0-0.09l0.09,0l0-0.09
				l0-0.1l0-0.09L550.35,177.25z"></path>
		</g>
	</g>
</g>
<g id="castillas_y_leon">
	<g id="leon">
		<g id="san_emiliano">
			<path class="st3 fase_0" d="M296.46,48.37l0.09-0.17l0.37-0.19l0-0.16l-0.11-0.03l-0.14-0.24l0.05-0.31l-0.13-0.13l-0.7-0.17l-0.18,0.08
				l-0.43,0.02l-0.15,0.1L294.84,47l-0.54-0.62l-0.01-0.13l-0.51,0.11l-0.15-0.08l-0.35,0.09l-0.12-0.32l0.02-0.23l0.18-0.25
				l-0.79,0.22l-0.32-0.14l0.06-0.19l-0.27-0.35l-0.22-0.06l-0.01-0.11l-0.24,0.01l-0.1-0.27l-0.39-0.11l-0.54-0.3l-0.18-0.56
				l-0.37-0.66l0.1-0.2l-0.12-0.31l0.15-0.41l-0.11,0.1l-0.06-0.04l-0.11-0.42l-0.26,0.04l-0.35-0.19l-0.29,0.43l-0.31,0.06
				l-0.65,0.2l-0.32-0.15l-0.38-0.3l-0.5-0.13l-0.1-0.13l-0.41,0.18l-0.2-0.05l-0.59-0.54l0,0l-0.12,0.35l-0.28,0.12l0.06,0.38
				l-0.19,0.59l0.18,0.26l0.02,0.31l-0.23,0.23l-0.06,0.18l-0.18-0.07l-0.19,0.14l-0.26,0l-0.32,0.28l-0.66,0.18l-0.48-0.66
				l-0.44-0.13l-0.13,0.14l-0.2-0.51l-0.15-0.03l-0.3-0.26l-0.31-0.09l-0.33,0.36l-0.79-0.21l-0.52,2.03l-0.47-0.29l-0.17,0.05
				l-0.14-0.02c-0.01,0.04-0.01,0.09-0.02,0.13c-0.01,0.02-0.01,0.07-0.02,0.1c0,0.11,0,0.22,0.01,0.33
				c0.01,0.02,0.01,0.03,0.01,0.06c0.01,0.04,0.01,0.09,0.01,0.11c0.03,0.15,0.06,0.29,0.09,0.44c0.12,0.5,0.28,1,0.4,1.5
				c0.14,0.58,0.15,1.17,0.17,1.76c0.17,0.23,0.28,0.51,0.31,0.8c0.02,0.21,0.04,0.42,0.06,0.63c0,0,0,0,0,0
				c-0.01-0.09,0-0.05,0,0.01c0.03,0.08,0.05,0.17,0.08,0.25l0.58-0.35l0.27-0.08l0.23,0.29l0.21-0.17l0.01,0.14l0.35,0.37
				l-0.01,0.28l0.19,0.1l0.29-0.08l0.28,0.21l0.08-0.32l0.15-0.21l0.44-0.17l0.08-0.14l0.16-0.04l0.22-0.2l0.2-0.07l0.28,0.04
				l-0.03,0.49l0.4,0.1l0.08,0.15l0.23,0.09l0.24-0.18l0.47,0.35l0.61,0.13l0.31,0.19l0.57-0.05l0.22,0.12l0.68,0.07l0.54,0.27
				l0.32,0.07l0.41-0.05l0.14,0.28l-0.05,0.06l0.15,0.11l1.55,0.02l0.14-0.76l1.09,0.1l0.46-0.06l-0.02-0.15l0.25-0.13l0.02-0.13
				l0.25,0.01l0.04-0.13l0.13-0.03l0.15-0.16l0.46,0.14l0.5-0.24l0.75-0.04l0.13,0.31l0.66,0.01l0.05-0.06l0.07,0.24l0.05-0.17
				l-0.06-0.09l0.74-0.55l0.06-0.39l-0.09-0.09l0.04-0.25l-0.05-0.09l-1.19,0.2l-0.09-0.36l0.24-0.21l-0.04-0.07l0.26-0.34
				l-0.04-0.09l0.14-0.13l-0.05-0.14l0.46-0.23L296.46,48.37z"></path>
		</g>
		<g id="armunia">
			<path id="path233" class="st3 fase_0" d="M299.49,65.85l-0.41,0.31l0.1,1.02l-0.02,0.66l-0.12,0.49l0.04,0.24l0.43,0.45l-0.04,0.5
				l0.07,0.42l-0.39,0.37l0.19,0.37l-0.01,0.36l0.27-0.06l-0.08,0.21l0.3-0.19l0.22,0.18l0.13,0.44l0.28,0.03l-0.11,0.25l-0.67,0.21
				l-0.1,0.15l-0.4,0.1l0.04,0.25l-0.9,0.52l-0.23,0.22l0.14,0.3l0.51-0.21l0.19,1.42l0.6,0.19l0.29,0.11l0.28-0.04l1-0.25
				l0.21-0.02l0.07,0.07l0.41-0.16l0.12-0.07l0.04-0.32l0.17-0.29l0.35,0.27l-0.07,0.06l0.24,0.4l0.29,0.25l0.6-0.01l-0.17,0.74
				l0.16-0.12l0.56,0.58l0.06-0.05l-0.1-0.44l0.3-0.09l0.3,0.05l0.1-0.5l0.22-0.1l0.24,0.12l0.3,0.34l-0.02,0.26l0.06,0l0.19,0.33
				l0.32-0.01l0.16,0.13l-0.01,0.34l-0.34,0.07l-0.15,0.25l-0.22-0.06l-0.06,0.04l0.07,0.05l-0.16,0.02l-0.25,0.2l-0.14-0.02
				l-0.38,0.13l-0.01,0.28l-0.12,0.35l0.25,0.56l0.24,0.02l0.14-0.1l0.27,0.49l0.31-0.21l0.25-0.01l0.46,0.16l0.08,0.12l0.22,0.03
				l0.17-0.19l-0.29-1.5l0.47-0.52l-0.02-0.3l0.07-0.45l-0.08-1.08l0.18,0.03l0.12-0.08l0.15-0.35l0.74-0.36l0.04-0.02l-0.12-0.18
				l0.19-0.49l-0.34-0.48l-0.02-0.43l0.09-0.21l-0.14-0.15l0.09-0.15l-0.09-0.18l0.1-0.03l0.04-0.13l-0.56-0.55l-0.18,0l-0.14-0.16
				l-0.01-0.16l0.14-0.11l-0.25-0.39l-0.03-0.39l-0.44-0.93l-0.08,0.1l-0.56,0.21l-0.2,0.19l-0.12-0.13l0.08-0.79l-0.19-0.11l0-0.13
				l-0.06-0.03l-0.02,0.14l-0.21,0.03l-0.12-0.2l-0.26,0.07l-0.3,0.14l0.03,0.15l-0.06,0.02l-0.4-0.41l-0.13-0.59l-0.09-0.01
				l-0.09-0.17l-0.43-0.12l-0.28-0.31l-0.2-0.06l-0.11-0.12l-0.78-0.24l-0.37-0.21l-0.84-0.12l-0.69-0.39l-0.36-0.11L299.49,65.85
				L299.49,65.85z"></path>
		</g>
		<g id="la_magdalena">
			<path id="path105" class="st3 fase_0" d="M283.65,50.16l-0.2,0.07l-0.22,0.2l-0.16,0.04L283,50.62l-0.44,0.17L282.41,51l-0.08,0.32
				l-0.28-0.21l-0.29,0.08l-0.19-0.1l0.01-0.28l-0.35-0.37l-0.01-0.14l-0.21,0.17l-0.23-0.29l-0.27,0.08l-0.64,0.39l-0.98,0.02
				l-0.16,0.58l-0.35,0.19l-0.31,0l-0.41-0.18l-0.27,0.1l-0.45-0.1l-0.15,0.15l-0.05,0.32l-0.11,0.07l0.35,0.06l0.1,0.47l0.38,0.52
				l0.02,0.24l0.12,0.07l-0.21,0.54l0.18,0.33l-0.17,0.65l-0.53,0.2l-0.17-0.04l-0.29,0.42l0.35,0.15l-0.01,0.16l0.5,0.53l0.36-0.03
				l0.06-0.09l0.34-0.11l0.55,0.11l0.31,0.2l0.19,0.44l-0.34,0.37l-0.05,0.23l0.1,0.23l0.11,0.16l0.34-0.21l0.11,0.02l0.17,0.15
				l0.36,0.13l0.09,0.17l0.29,0.16l0.11,0.44l0.57,0.05l1.17,0.37l0.55-0.49l0.22-0.08l0.94,0.41l-0.18,0.08l0.17,0.51l0.27,0.18
				l0.08,0.16l-0.16,0.29l0.66,0.08l0.12,0.35l0.14,0.13l0.26,0l0.07,0.13l0.22,0.11l0.31,0.02l0.61-0.01l0.22-0.08l0.6,0.22
				l0.04-0.07l0.37-0.08l0.07-0.12l0.33,0.03l0.54-0.12l0.22,0.13l0.65,0.03l0.06,0.1l0.33,0.16l1.13,0.17l1.37,0.41l0.04,0.27
				l0.2,0.21l-0.2,0.47l-0.13,0.6l0.7-0.36l0.35,0.11l1.12,0.04l0.64,0.45l0.27,0.07l0.02,0.28l0.41,0.12l0.05-0.05l0.28-0.13
				l-0.22-0.67l0.08-0.01l0.16-0.56l0.42,0.09l0.39,0.19l0.26-0.06l0.26,0.31l0.2,0.08l0.23,0.24l0.34,0.64l0.1,0.46l0.3,0.55
				l0.71,0.26l-0.07-0.13l0.01-0.44L298.96,64l0.22-1.08l0.27-0.3l0.74,0.16l0.18-1.15l-0.43-1.43l-0.09-0.59l0.25-0.71l-0.04-1.09
				l-0.11-0.41l0.38-0.31l0.24-0.28l-0.27-0.38l0.02-0.58l0.26-0.44l-0.01-0.36l-0.16-0.51l-0.17-0.16l-0.59-0.14l-0.59-0.47
				l-0.12-0.56l-0.33-0.17l-0.18-0.23l-0.82-0.2l-0.21,0.09l-0.15-0.08l-0.04,0.05l-0.22-0.06l0.12-0.13L297,52.35l-0.51-0.15
				l0.04-0.25l-0.13-0.21l-0.47-0.38l-0.07-0.24l-0.05,0.06l-0.66-0.01l-0.13-0.31l-0.75,0.04l-0.5,0.24l-0.46-0.14l-0.15,0.16
				l-0.13,0.03l-0.04,0.13l-0.25-0.01l-0.02,0.13l-0.25,0.13l0.02,0.15l-0.46,0.06l-1.09-0.1l-0.14,0.76l-1.55-0.02l-0.15-0.11
				l0.05-0.06l-0.14-0.28L288.57,52l-0.32-0.07l-0.54-0.27l-0.68-0.07l-0.22-0.12l-0.57,0.05l-0.31-0.19l-0.61-0.13l-0.47-0.35
				l-0.24,0.18l-0.23-0.09l-0.08-0.15l-0.4-0.1l0.03-0.49L283.65,50.16L283.65,50.16z"></path>
		</g>
		<g id="cuenca_bernesga">
			<path id="path35" class="st3 fase_0" d="M301.4,42.19l-0.2,0.02l-0.31,0.25l-0.64,0.3l-0.18,0.18l-0.29,0.02l-0.05,0.27l-0.18,0.17
				L299.48,44l-0.18,0.14l-0.15,0l-0.18,0.19l-0.11,0.63l-0.32,0.34l-0.02,0.35L298.24,46l0.21,0.66l-0.06,0.14l-0.11,0.1
				l-0.59,0.12l-0.43-0.05l-0.17,0.11l-0.21,0.02l-0.16,0.15l-0.05,0.31l0.14,0.24l0.11,0.03l0,0.16l-0.37,0.19l-0.09,0.17
				l-0.16-0.01l-0.46,0.23l0.05,0.14l-0.14,0.13l0.04,0.09l-0.26,0.34l0.04,0.07l-0.24,0.21l0.09,0.36l1.19-0.2l0.05,0.09
				l-0.04,0.25l0.09,0.09l-0.06,0.39l-0.74,0.55l0.06,0.09l-0.05,0.17l0.47,0.38l0.13,0.21l-0.04,0.25l0.51,0.15l0.11,0.12
				l-0.12,0.13l0.22,0.06l0.04-0.05l0.15,0.08l0.21-0.09l0.82,0.2l0.18,0.23l0.33,0.17l0.12,0.56l0.59,0.47l0.59,0.14l0.17,0.16
				l0.16,0.51l0.01,0.36l-0.26,0.44l-0.02,0.58l0.27,0.38l-0.24,0.28l0.29-0.11l0.36,0.08l0.48-0.23l0.59,0.52l0.17-0.03l0.17,0.21
				l0.21,0.01l0-0.06l0.47-0.19l0.25,0.08l0.26-0.39l-0.04,0.24l0.64,0.2l0.37,0.45l0.34-0.03l0.94-0.32l0.46-0.02l0.2-0.21
				l0.03-0.2l0.58-0.08l0.49-0.77l0.39-0.39l0.44,0.19l0.75-0.1l0.8,0.07l0.18-0.2l0.32-0.11l0.06-0.2l0.24-0.24l0.05-0.15
				l-0.12-0.95l-0.19-0.08l-0.15,0.03l-0.05-0.27l0.17-0.07l0.04-0.51l-0.1-0.48l-0.14-0.19l-0.02-0.39l-0.3-0.19l-0.03-0.62
				l-0.18-0.63l0.17-0.46l0.36,0.02l0.1-0.24l-0.12-0.56l0.08-0.48l0.25-0.41l0.31-0.91l-1.51-0.48l0.12-0.23l-0.05-0.31l0.09-0.36
				l0.23-0.43l-0.1-0.28l-0.03-0.41l-0.15-0.1l-0.35-0.51l0.11-0.52l-0.5-0.39l-0.14,0.08l-0.42-0.01l-0.34,0.08l-0.07,0.17
				l-0.41,0.23l-0.64-0.12l-0.08-0.11l-0.49-0.01l-0.18-0.51l-0.72-0.11l-0.12,0.04l-0.35-0.14l-0.21,0.11l-0.16-0.07l-0.58,0.07
				l-0.37-0.06l-0.08-0.17l-0.18,0.03l-0.28-0.37l-0.37-0.05l-0.04-0.09l-0.18-0.03l-0.3-0.27L302,42.45L301.4,42.19L301.4,42.19z"></path>
		</g>
		<g id="bonar">
			<path id="path19" class="st3 fase_0" d="M314.33,40.16l-0.35,0.34l-0.22,0.06l-0.3,2.04l-0.18,0.09l-0.22-0.01l-0.22-0.11l-0.2,0.04
				l-0.25-0.16l-0.69-0.03l-0.58-0.15l-0.75,0.26l-0.36-0.27l-0.12,0.03l-0.09,0.07l-0.01,0.62l-0.26,0.57l-0.11,0.05l-0.14,0.5
				l-0.11,0.52l0.35,0.51l0.15,0.1l0.03,0.41l0.1,0.28l-0.23,0.43l-0.09,0.36l0.05,0.31l-0.12,0.23l1.51,0.48l-0.31,0.91l-0.25,0.41
				l-0.08,0.48l0.12,0.56l-0.1,0.24l-0.36-0.02l-0.17,0.46l0.18,0.63l0.03,0.62l0.3,0.19l0.02,0.39l0.14,0.19l0.1,0.48l-0.04,0.51
				l-0.17,0.07l0.05,0.27l0.15-0.03l0.19,0.08l0.12,0.95l0.67,0.15l0.22,0.21l0,0.2l0.17,0.21l0.1-0.04l0.37,0.15l0.12-0.1
				l0.26,0.07l0.05,0.28l0.2,0.06l0.15,0.16l0.3,0.64l0.19-0.41l0.32-0.13l0.44,0.22l0.37,0.07l-0.34,0.26l-0.55,0.16l0.06,0.1
				l0.55,0.24l0.15,0.26l0.7,0.68l-0.72,1.32l0.05,0.07l0.09-0.18l0.42-0.18l0.24-0.04l0.17,0.08l0.42,0.01l0.1-0.2l0.25,0.06
				l0.45-0.06l0.47-0.16l0.82-0.94l0.23-0.72l-0.26-0.02l-0.35,0.23l-0.2-0.03l-0.23-0.38l-0.27-0.16l0.02-0.27l-0.61-0.62
				l-0.23-0.55l-0.34-0.41l0.24-0.47l-0.01-0.24l0.38,0.01l0.64,0.38l1.23,0.1l0.37,0.1l0.05-0.62l0.32,0.24l0.2,0.04l0.22-0.13
				l0.07-0.31l0.2-0.12l0.5-0.06l0.33-0.18l0.22-0.49l0.48-0.51L321.06,53l-0.25-0.29l-0.11-0.64l0.25-0.65l-0.18-0.44l0.07-0.22
				l-0.31-0.14l-0.11-0.36l0.25-0.62l0.21-0.25l0.02,0.11l0.32,0.05l0.3-0.11l0.36-0.33l0.78-0.5l0.09-0.47l-0.26-0.3l-0.12-0.02
				l-0.13-0.36l0.3-0.46l0.05-0.21l-0.46-0.24L322.06,46l-0.34-0.05l-0.03-0.07l0.48-0.26l0.07-0.12l-0.13-0.42l-0.15,0l-0.08-0.34
				l0.04-0.24l0.23-0.24l-0.16-0.16l0.01-0.1l-0.86-0.18l-0.58-0.49l-0.3-0.1l-0.01-0.17l-0.14-0.21l-0.27-0.17l0.21-0.73l-0.23-0.1
				l-0.19-0.24l-0.22-0.11l-0.01-0.43l0.14-0.33l0.17-0.09l-0.18-0.14l-1.14,0.05l-0.39,0.23l-0.16,0.32l-0.6-0.1l-0.59-0.27
				l-0.23,0.08l-0.13,0.18l-0.66-0.14l-0.19-0.1l0.1-0.42l-0.24-0.14L314.33,40.16L314.33,40.16z"></path>
		</g>
		<g id="cistierna">
			<path id="path39" class="st3 fase_0" d="M324.47,44.43l-0.18,0.08l-0.11,0.17l-0.32,0.16l-0.14,0.25l-0.35-0.35l-0.71,0.08
				l-0.16,0.13l-0.4,0.11l0.13,0.42l-0.07,0.12l-0.48,0.26l0.03,0.07l0.34,0.05l0.05,0.55l0.46,0.24L322.52,47l-0.3,0.46l0.13,0.36
				l0.12,0.02l0.26,0.3l-0.09,0.47l-0.78,0.5l-0.36,0.33l-0.3,0.11l-0.32-0.05l-0.02-0.11l-0.21,0.25l-0.25,0.62l0.11,0.36
				l0.31,0.14l-0.07,0.22l0.18,0.44l-0.25,0.65l0.11,0.64l0.25,0.29l0.24,0.13l-0.48,0.51l-0.22,0.49l-0.33,0.18l-0.5,0.06
				l-0.2,0.12l-0.07,0.31l-0.22,0.13l-0.2-0.04l-0.32-0.24l-0.05,0.62l-0.37-0.1l-1.23-0.1l-0.64-0.38l-0.38-0.01l0.01,0.24
				l-0.24,0.47l0.34,0.41l0.23,0.55l0.61,0.62l-0.02,0.27l0.27,0.16l0.23,0.38l0.2,0.03l0.35-0.23l0.26,0.02l-0.23,0.72l0.01,0.38
				l0.39,0.73l0.23,0.23l0.15,0.03l0.14-0.09l0.19,0.2l0.22,0l0.31-0.15l0.27-0.39l0.48-0.12l0.28-0.33l0.17-0.03l0.34,0.09
				l0.29-0.24l0.23-0.02l0.1,0.03l-0.24,0.52l0.03,0.48l-0.48,1.03l0.79,0.6l0.38,0.03l0.2-0.54l0.16-0.08l0.18-0.44l0.12-0.1
				l0.02-0.76l0.22-0.28l0.14-0.09l0.65-0.01l0.83,0.35l0.09-0.04l0.11,0.5l-0.27,0.43l0.02,0.53l-0.2,0.45l-0.33,1.34l-0.09,0.13
				l0.08-0.05l0.09,0.11l0.11,0.91l0.48,0.62l-0.26,1.22l0.16,0.3l0.15,0.03l0.57,0.63l0.94,0.42l0.02-0.16l0.4-0.11l0.32,0.08
				l0.61,0.46l0.37,0.07l0.75-0.58l-0.13-0.43l-0.03-0.12l0.11-0.27l0.33-0.48l0.48-0.43l-0.01-0.21l0.14-0.26l0.52-0.28l0.23-0.29
				l0.33-0.86l0.64-0.55l0.23-0.52l0.99-1.57l-0.2-0.1l-0.1-0.73l-0.4-0.14l-0.32-0.41l0.12-0.48l-0.08-0.17l0.28,0.05l0.14-0.21
				l0.18,0.1l0.44-0.9l0.02-0.28l0.26-0.27l0.05,0.08l0.17-0.03l0.06-0.08l-0.03-0.78l-0.1-0.13l0.01-0.16l-0.16-0.22l0.24-0.43
				l0.04-0.53l0.16-0.28l0.14-0.48l-0.5-0.56l-0.01-0.31l0.11-0.31l-0.18-0.25l0.07-0.31l-0.04-0.26l0.1-0.24l-0.14-0.18l-0.03-0.27
				l-0.41,0.06l-0.21-0.14l-0.11-0.51l0.04-0.14l0.31-0.15l-0.21-0.43l-0.2-0.12l-0.25-0.01l-0.04-0.13l-0.32-0.17l0.02-0.14
				l-0.43-0.17l-0.02,0.08l-0.45,0.1l-0.15,0.03l-0.24-0.06l-0.27,0.07l-0.3-0.04l-0.57,0.13l-0.45-0.12l-0.14-0.13l-0.35,0.45
				l-0.7,0.06l-0.16-0.04l0.06-0.06l-0.34-0.26l-0.21,0.01l0.08-0.38l-0.22-0.43l-0.06-0.7l-0.64-0.12l-0.63,0.09l-0.32-0.09
				l-0.2-0.44l-0.19-0.04l0-0.23l-0.09-0.1l0.08-0.19l-0.04-0.21l0.09-0.18l0.07,0.03l0.19-0.23l-0.03-0.11l-0.23-0.2l-0.06-0.35
				l0.02-0.16l0.49-0.4l0.02-0.29l-0.33,0.01l-0.38-0.12L324.47,44.43L324.47,44.43z"></path>
		</g>
		<g id="astorga_ii">
			<path id="path89" class="st3 fase_0" d="M283.94,59.93l-0.02,0.55l-0.17,0.14l-0.58,0.17l-0.02,0.08l-0.32-0.03l-0.42,0.26l-0.1,0.21
				l-0.74,0.22l-0.73-0.31l-0.14,0.13l0.06,0.11l-0.07,0.12l-0.42,0.17l-0.08,0.55l0.31,0.18l0.44,0.03l-0.13,0.17l-0.03,0.34
				l-0.13,0.19l0.02,0.26l-0.37,0l-0.47,0.12l-0.23,0.52l-0.31,0.13l-0.21,1.09l-0.7,0.33l0.04,0.53l-0.7,0.26l-0.34-0.09l-0.2,0.17
				l-0.23,0.03l-0.1,0.22l-0.56,0.24l0.75,0.46l0.03-0.04l0.73,0.51l0.02,0.22l0.34,0.26l0.25,0.46l0.52,0.17l-0.32,0.29l-0.49,0.08
				l-0.39,0l-0.52,0.14l-0.27-0.11l-0.17,0.64l-0.14,0.17l0.18,0.21l-0.17,0.02l-0.2-0.08l0.15,0.24l-0.02,0.22l-0.2,0.2l0.16,0.25
				l-0.37-0.02l-0.02,0.17l0.09,0.15l-0.46,0.4l-0.33-0.3l-0.64-0.06l-0.38,0.15l-0.54-0.33l-0.18,0.64l-0.34-0.03l-0.33,0.27
				l-0.2,0.01l-0.78,0.34l-0.09,0.6l0.02,0.15l0.34,0.28l-0.03,0.31l-0.78,1.39l-0.39-0.19l-0.15,0.03l-0.24,0.15l-0.23-0.01
				l-0.31,0.11l-0.75,0.46l-0.59,0.13l-0.07,0.2l0.06,0.15l0.15,0.09l0.06,0.32l0.77,0.37l-0.14,0.37l-0.12,0.09l-0.06,0.45
				l0.12,0.19l0.18,0.1l0.13,0.45l0.14,0.21l0.48,0.11l0.83,0.65l0.9,0.19l1.73,1.24l0.05,0.22l0.21,0.11l0.22,0.01l0.23,0.25
				l0.21,0.12l0.16,0l0.57,0.33l0.14,0.14l1.37,0.35l0.25,0.45l0.31,0.31l0.47,0.18l0.32,0.01l0.21,0.14l1.26,0.66l1.02,0.15
				l0.55-1.1l0.54-0.16l-0.22-0.16l0.18-0.3l0.14-0.88l0.11-0.18l-0.15,0.13l-0.41-0.16l0.16-0.86l0.41-0.72l-0.23-0.19l-0.24,0.01
				l-0.22-0.34l0.3-0.15l0.33,0.02l0.22-0.42l0.31-0.37l0.04-0.15l0.41,0.18l0.72,0.4l0.52,0.17l1.44,0.43l1.52,0.09l0.17,0.03
				l0.35-0.36l0.53,0.17l0.03-0.34l-0.11-0.01l0.33-1l-0.09-0.27l0.2-0.44l-0.08-0.12l0.03-0.11l0.27-0.12l0.44-0.37l0.56,0.75
				l0.5-0.32l-0.03-0.1l-0.19-0.24l0.05-0.1l0.29-0.09l0.01-0.52l0.08-0.1l-0.1-0.19l-0.42-0.27l-0.58-0.74l-0.13-0.29l0.37-0.06
				l-0.05-0.34l0.15-0.63l0.08-0.02l-0.21-0.74l0.11-0.18L289.85,73l0.36-0.37l-0.5-0.32l-0.02-0.18l-0.41-0.31l-0.22-0.48
				l-0.53-0.34l0.26-0.34l0.4-0.21l-0.1-0.41l0.14-0.92l0.39-0.31l0.68-0.99l0.73-0.48l0.2-0.22l0.21-0.01l0.69-0.35l0.28,0.12
				l0.07-0.23l-0.06-0.49l-0.1-0.64l0.09-0.14l-0.33-0.62l-0.24,0.13l-0.05,0.11l-0.14-1.04l-0.56-0.82l-0.14-0.4l0.3,0.1l0.34-0.02
				l0.74-0.36l0.2-0.47l-0.2-0.21l-0.04-0.27l-1.37-0.41l-1.13-0.17l-0.33-0.16l-0.06-0.1l-0.65-0.03l-0.22-0.13l-0.54,0.12
				l-0.33-0.03l-0.07,0.12l-0.37,0.08l-0.04,0.07l-0.6-0.22l-0.22,0.08l-0.61,0.01l-0.31-0.02l-0.22-0.11l-0.07-0.13l-0.26,0
				l-0.14-0.13l-0.12-0.35L283.94,59.93L283.94,59.93z"></path>
		</g>
		<g id="fabero">
			<path id="path53" class="st3 fase_0" d="M262.92,49.25l-0.32,0.09l-0.41-0.04l-0.15,0.1l-0.27-0.09l-0.96,0.12l-0.2,0.18l-0.68,0.21
				l-0.4-0.13l-0.48,0.53l-0.25,0.44l-0.35,0.29l-0.3,0.6L258,51.48l-0.54,0.06l0.01-0.21l0.22-0.19l-0.01-0.25l-0.1-0.2l-0.77-0.32
				l-0.46-0.3l-0.43-0.06l-0.51-0.2l-0.96,0.04l0.16,0.43l0.44,0.45l0.22,0.5l0,0.47l0.07,0.16l0.45,0.33l-0.2,0.19l-0.63-0.34
				l-0.08-0.2l-0.12-0.04l0.27-0.21l-1.52,0.44l0.48,1.36l0.26,0.53l0.2,0.29l0.28,0.06l-0.11,0.23l0.15,0.26l-0.45,0.25l-0.12,0.44
				l0.17,0.09l0.07,0.19l0.26,0.25l0.06,0.23l0.36,0.26l0.6,0.01l0.31,0.7l0.12,0.05l-0.05,0.28l0.21,0.5l-0.12,0.03l0.1,0.18
				l0.36,0.2l0.3-0.06l0.86,0.1l0.2,0.06l0.09,0.12l-0.07,0.15l0.05,0.22l-0.18,0.22l0.39,0.5l0.7,1.6l0.23,0.28l0.03,0.13
				l-0.16,0.13l-0.12,0.74l0.54,0.51h0l0.42-0.59l1.01-0.99l0.6,0.04l0.22-0.09l0.22,0.04l0.71-0.49l0.44-0.05l0.16,0.13l0.48-0.11
				l0.58,0.12l0.22-0.08l-0.54-0.19l-0.16-0.21l-0.03-0.38l-0.52-0.57l0.21-0.42l0.28-0.15l0.42-0.37l0.31-0.12l0.09,0.08l0.29-0.01
				l0.81-0.38l0-0.18l0.12,0.01l0.27-0.26l0.03-0.23l0.14-0.52l-0.07-0.19l-0.15-0.14l-0.26-0.09l-0.38,0.02l0.09-0.46l-0.54-0.02
				l-0.19-0.21l-0.27-0.05l0.17-0.31l-0.24-0.34l-0.83-1.05l0.66-0.25l0.06-0.17l0.22-0.16l0.34-0.63l0-0.18l0.91-1.52l0.17-0.91
				l-0.25-0.69l-0.09-0.12l-0.33-0.08l-0.25,0l-0.19,0.09l-0.31-0.1l-0.35,0.13l-0.26-0.45l-0.63,0.27l-0.15-0.2L262.92,49.25
				L262.92,49.25z"></path>
		</g>
		<g id="puente_domingo_florez">
			<path id="path257" class="st3 fase_0" d="M262.13,70.42l-0.53,0.67l-0.52,0.29l0.1,0.39l-1.23-0.22l-0.36-0.34l-0.5-0.29l-0.41-0.11
				l-0.72,0.07l-0.38,0.49l-0.22-0.14l0.05-0.16l-0.38-0.16l-0.63,0.47l0.06,0.17l-0.25,0.16l-0.3-0.11l0,0.18l-0.1,0.08l-0.33-0.16
				l-0.16-0.23l-0.16-0.03l-0.17,0.1l-0.11,0.37l0.27,0.33l-0.13,0.26l0.03,0.06l0.26-0.04l0.16,0.11l-0.11,0.28l0.25,0.18
				l0.18,0.45l0.25,0.24l-0.12,0.44l-0.44,0.39l0.09,0.87l-0.06,0.24l0.29,0.34l-0.41,0.06l0.16,0.24l-0.32,0.25l-0.04,0.36
				l-0.06,0.09l-0.27-0.04l-0.22,0.18l0.4,0.56l0.41,0.19l0.11,0.34l0.12-0.04l0.33,0.09l0.3-0.09l0.01,0.14l0.24,0.09l0.32-0.03
				l0.56,0.2l0.21,0.31l0.1,0.43l0.35-0.13l0.33,0.25l0.63,0.2l-0.21,0.77l0.03,0.26l-0.1,0.63l0.15,0.18l0.12,0.38l0.84,0.56
				l0.14,0.18l0.39,0.13l0.13,0.16l0.16,0.01l0.23-0.22l0.25,0.28l0.25,0.02l0.07-0.15l0.26-0.03l0.3-0.22l0.67-0.72l0.5-0.08
				l0.26-0.26l0.21-0.63l-0.16-0.11l0.23-0.55l-0.38-0.32l0.17-0.25l-0.06-0.19l0.09-0.37l-0.05-0.14l0.55-0.29l0.18-0.28l0.46-0.32
				l0.08-0.12l0.02-0.34l0.33-0.58l0.39-0.23l-0.23-0.57l-0.42-0.51l-0.42-0.28l-0.14-0.22l-0.48-0.28l0.04-0.24l-0.42-0.48
				l-0.01-0.21l-0.15,0.16l-0.33,0.05l-0.29-0.62l-0.03-0.48l-0.18,0.28l-0.27,0.05l-0.24-0.06l-0.2-0.2l-0.01-0.21l0.21-0.29
				l0.02-0.88l0.51-0.78L262.13,70.42L262.13,70.42z"></path>
		</g>
		<g id="villafranca_bierzo">
			<path id="path135" class="st3 fase_0" d="M254.2,55.43l-0.35,0.2l-0.26,0.33l0.02,0.09l0.17,0.09l0.02,0.25l-0.14,0.31l-0.34,0.25
				l-0.17,0.01l-0.31,0.25l-0.13,0.02l-0.25,0.29l-0.04,0.28l0.07,0.09l-0.13,0l-0.2,0.3l-1.27,1.01l-0.46-0.03l-0.39-0.76
				l-0.12,0.03l-0.34,0.4l0.08,0.28l-0.03,0.19l-0.25,0.04l0.15,0.46l-0.15,0.24l-0.44,0.34l-0.26-0.43L248.53,60l-0.21-0.07
				l-0.15,0.17l-0.32,0.01l-0.44,0.52l0.07,0.2l-0.21-0.12l-0.16,0.11l-0.31,0.56l-0.34,0.29l0.45,0.58l0.09,0.45l0.41,0.34
				l0.04,0.15l0.21,0.1l-0.08,0.19l0.19,0.18l-0.31,0.16l-0.04,0.15l0.5,0.15l0.19,0.34l-0.35,0.06l-0.19-0.18l-0.25,0.12l-0.2,0.19
				l-0.11-0.01l-0.26,0.19l-0.16,0l-0.21,0.18l-0.04,0.36l0.09,0.2l-0.05,0.53l0.07,0.44l-0.19,0.34l0.23,0.45l0.39,0.11l0.03,0.17
				l-0.45,0.39l-0.02,0.14l-0.11,0.1l-0.25,0.05l-0.1,0.39l0.05,0.25l-0.28,0.38l-0.25-0.09l0.17,0.18l0.01,0.15l-0.27,0.55
				l0.14,0.09l0.12-0.09l0.11,0.02l0.08,0.44l-0.36,0.6l-0.03,0.27l-0.09,0.15l-0.21,0.12l0.13,0.08l0.56,0.01l0.32-0.11l0.25,0.05
				l0.53,0.18l0.16,0.35l0.37-0.2l0.61-0.13l0.14-0.1l0.01,0.2l0.14,0.13l0.74,0.34l0.22-0.04l0.32-1.01l0.22-0.27l1.02-0.08
				l0.11-0.07l0.29,0.06l0.42,0.5l-0.07,0.09l0.13,0.07l0.28-0.04l0.37,0.21l0.83,0.25l0.68,0.56l0.46,0.1l0.32-0.1l0.15-0.36
				l0.14,0.12l0.13-0.26l-0.27-0.33l0.11-0.37l0.17-0.1l0.16,0.03l0.16,0.23l0.33,0.16l0.1-0.08l0-0.18l0.3,0.11l0.25-0.16
				l-0.06-0.17l0.63-0.47l0.38,0.16l-0.11-0.45l0.08-0.26l0.31-0.09l0.75-0.64l0.03-0.16l-0.32-0.24l-0.26-0.7l0.56-0.6l0.48,0.21
				l0.06-0.58l0.17-0.04l0.06-0.1l-0.02-0.22l0.08-0.14l-0.17-0.32l-0.5,0.26l-0.1,0.32l-0.33-0.73L258,66.43l0.08-0.51l0.42-0.23
				l-0.17-0.1l-0.24-0.43l-0.29-0.26l-0.08-0.17l0.25-0.82l0.25-0.01l0.22-0.26l1.08-0.51l-0.54-0.51l0.12-0.74l0.16-0.13
				l-0.03-0.13L259,61.32l-0.7-1.6l-0.39-0.5l0.18-0.22l-0.05-0.22l0.07-0.15l-0.09-0.12l-0.2-0.06l-0.86-0.1l-0.3,0.06l-0.36-0.2
				l-0.1-0.18l0.12-0.03l-0.21-0.5l0.05-0.28l-0.12-0.05l-0.31-0.7l-0.6-0.01l-0.36-0.26l-0.06-0.23l-0.26-0.25l-0.07-0.19
				L254.2,55.43L254.2,55.43z"></path>
		</g>
		<g id="valencia_de_d__juan">
			<path id="path353" class="st3 fase_0" d="M309.39,75.58l-0.02,0.19l-0.21,0.16l-0.11-0.03l-0.23,0.13l-0.48-0.01l0.01,0.15l-0.1,0.03
				l-0.22-0.07l-0.07,0.16l-0.12,0.01l0.06,0.19l-0.14-0.06l-0.1,0.12l-0.25-0.19l-0.34,0.04l-0.11,0.11l0.02,0.3l-0.47,0.52
				l0.29,1.5l-0.17,0.19L306.42,79l-0.08-0.12l-0.46-0.16l-0.25,0.01l-0.31,0.21l-0.27-0.49l-0.14,0.1l-0.24-0.02l-0.23,0.21
				l-0.67,0.3l-0.6,0.67l-0.22,0.36l-0.1,0.14l0.13,0.41l-0.21,0.15l0.01,0.22l0.27,0.12l0.07,0.47l0.5,0.31l0.01,0.1l-0.11,0.05
				l0.03,0.08l-0.21,0.35l0.04,0.2l0.15,0.01l0.02,0.42l-0.11,0.16l0.01,0.17l-0.18,0.22l0.47,0.29l-0.01,0.49l0.24,0.18l0.24,0.33
				l0.48,0.34l-0.13,0.19l-0.05,0.47l0.36,0.53l-0.04,0.19l-0.22,0.18L304.59,88l0.09,0.1l-0.14,0.05l-0.16,0.21l-0.12,0.67
				l0.09,0.03l-0.26,0.46l0.01,0.37l0.09,0.1l0.16-0.07l0.32,0.1l0.35-0.06l1.12,0.44l-0.05,1.32l1.55-0.27l0.05-0.37l0.35-0.9
				l0.79,0.08l-0.04-0.38l0.09-0.2l0.17-0.17l0.3,0.03l0.04,0.29l0.19,0.4l0.64,0.66l0.61,0.06l0.29-0.15l0.07-0.03l0.19,0.18
				l0.63-0.11l0.36-0.25l0.1-0.17l0.37,0.18l0.28,0l0.01,0.06l0.13-0.02l0.31,0.21l0.24,0.04l-0.01-0.47l0.08-0.21l-0.29-0.28
				l-0.09-0.37l0.07-0.75l0.28-0.46l-0.34-0.4l-0.02-0.68l0.04-0.19l0.29-0.21l0.21,0.16l0.3-0.02l0.29-0.12l0.34,0.32l0.15-0.07
				l0.14,0.04l0.35-0.6l-0.17-0.1l0.03-0.75l-0.22-0.23l-0.05-0.44l0.07-0.46l0.33-0.1l0.35-0.24l0.33,0.37l0.56-0.24l-0.27-0.21
				l0.04-0.53l-0.14-0.14l-0.21-0.06l-0.06-0.25l-0.13,0.05l-0.03-0.43l-0.19-0.26l0.09-0.16l-0.21-0.26l0.7-0.31l-0.03-0.06
				l0.22-0.15l-0.01-0.06l-0.48-0.5l-0.22-0.74l-0.44,0.19l-0.23-0.11l-0.08-0.14l-0.15-0.65l-0.39,0.09l-0.13-0.57l-0.25-0.22
				l0.06-0.17l-0.12-0.11l-0.23,0.06l-0.35-0.05l-0.18,0.11l-0.1-0.17l-0.44-0.04l0.05,0.16l-0.18,0.42l-0.27,0.01l-0.11,0.11
				l-0.54,0.04l-0.07,0.15l-0.28,0.07l-0.05,0.31l-0.43-0.01l-0.29-0.24l-0.48,0.28l-0.09-0.2l0.11-0.17l-0.19-0.16l0.22-0.67
				l-0.16,0.02l0.03-0.15l0.25-0.07l-0.04-0.08l-0.17-0.13l-0.08,0.1l-0.27-0.07l-0.05,0.05l-0.14-0.13l-0.4-0.48l0.13-0.27
				l-0.03-0.35l0.21-0.2l-0.03-0.1l0.11-0.12l0.02-0.12l-0.09-0.09l0.08-0.02l0.08-0.45l-0.13-0.09l-0.04-0.32L309.39,75.58
				L309.39,75.58z"></path>
		</g>
		<g id="mansilla_mulas">
			<path id="path91" class="st3 fase_0" d="M318.11,58.3l-0.82,0.94l0.14,0.62l-0.15,0.27l-0.04,0.35l-0.15,0.21l0.03,0.27l-0.16,0.42
				l-0.44,0.95l-0.19,0.08l-0.12,0.45l-0.14,0.2l-0.43-0.14l0.13,0.24l-0.03,0.77l-0.69,0.86l-0.31,0.17l-0.34,0.49l0.37,0.47
				l0.02,0.17l-0.12-0.02l-0.32,0.49l-0.73,0.62l-0.4-0.08l-0.26-0.14l-0.87,0.83l0,0.16l-0.47,0.49l-0.18,0.47l-0.5,0.36
				l-0.23,0.37l0.05,0.16l0.25,0.1l-0.04,0.13l-0.09,0.7l0.06,0.02l-0.1,0.28l-0.1,0.09l-0.07-0.06l-0.23,0.12l0,0.14l-0.1,0.09
				l-0.21,0.52l0.02,0.66l-0.34,0.34l-0.68,0.15l-0.03,0.25l-0.23,0.17l-0.03,0.21l-0.67,0.54l-0.04,0.02l-0.74,0.36l-0.15,0.35
				l-0.12,0.08l-0.18-0.03l0.08,1.08l-0.07,0.45l0.11-0.11l0.34-0.04l0.25,0.19l0.1-0.12l0.14,0.06l-0.06-0.19l0.12-0.01l0.07-0.16
				l0.22,0.07l0.1-0.03l-0.01-0.15l0.48,0.01l0.23-0.13l0.11,0.03l0.21-0.16l0.02-0.19l0.19,0.02l0.04,0.32l0.13,0.09l-0.08,0.45
				l-0.08,0.02l0.09,0.09l-0.02,0.12l-0.11,0.12l0.03,0.1l-0.21,0.2l0.03,0.35l-0.13,0.27l0.4,0.48l0.14,0.13l0.05-0.05l0.27,0.07
				l0.08-0.1l0.17,0.13l0.04,0.08l-0.25,0.07l-0.03,0.15l0.16-0.02l-0.22,0.67l0.19,0.16l-0.11,0.17l0.09,0.2l0.48-0.28l0.29,0.24
				l0.43,0.01l0.05-0.31l0.28-0.07l0.07-0.15l0.54-0.04l0.11-0.11l0.27-0.01l0.18-0.42l-0.05-0.16l0.44,0.04l0.1,0.17l0.18-0.11
				l0.35,0.05l0.23-0.06l0.12,0.11l-0.06,0.17l0.25,0.22l0.13,0.57l0.39-0.09l0.15,0.65l0.08,0.14l0.23,0.11l0.44-0.19l0.22,0.74
				l0.48,0.5l0.01,0.06l-0.22,0.15l0.03,0.06l-0.7,0.31l0.21,0.26l-0.09,0.16l0.19,0.26l0.03,0.43l0.13-0.05l0.06,0.25l0.21,0.06
				l0.14,0.14l-0.04,0.53l0.27,0.21l0.19,0.13l0.24-0.04l0.07-0.37l0.17,0l0.16,0.1l0.06-0.03l-0.02-0.13l0.14-0.04l0.12-0.15
				l0.6,0.08l0.17-0.35l0.11-0.13l0.11,0.12l0.69-0.38l-0.18-0.33l-0.18,0.03l-0.35,0.24l-0.22-0.36l-0.08-0.64l0.39-0.38l0.04-0.7
				l-0.15-0.3l0.09-0.1l-0.24-0.03l-0.31-0.27l0.33-1.17l0.08-0.46l0.33,0.25l0.37,0.1l0.12-0.28l0.22-0.03l0.19,0.21l0.01-0.23
				l0.03-0.15l0.29-0.09l-0.04-0.27l0.13,0.05l0.02-0.05l0.43,0.18l0.41,0.01l0.08-0.02l0.1-0.24l0.09,0.02l0.02-0.22l0.13-0.22
				l-0.11-1.02l0.11-0.26l-0.48-0.35l0.23-0.1l0.26,0.19l0.21,0.33l0.89,0.24l0.62-0.24l0.25-0.01l0.18-0.31l0.1-0.27l-0.04-0.77
				l-0.39-0.57l-0.05,0.08l-0.69-0.02l-0.84-0.54l0.17-0.65l-0.46-1.02l-1.01-0.98l0.99-0.98l0.39-0.71l0.47-0.36l0.27-0.38
				l0.05,0.1l0.76-0.38l0.32,0.19l0.25-0.07l-0.12-0.7l-0.15-0.28l0.04-0.73l-0.07-0.14l0.07-0.48l0.18-0.25l0.02-0.27l0.29-0.38
				l0.22-0.22l0.26-1.22l-0.48-0.62l-0.11-0.91l-0.09-0.11l-0.08,0.05l0.09-0.13l0.33-1.34l0.2-0.45l-0.02-0.53l0.27-0.43l-0.11-0.5
				l-0.09,0.04l-0.83-0.35l-0.65,0.01l-0.14,0.09l-0.22,0.28l-0.02,0.76l-0.12,0.1l-0.18,0.44l-0.16,0.08l-0.2,0.54l-0.38-0.03
				l-0.79-0.6l0.48-1.03l-0.03-0.48l0.24-0.52l-0.1-0.03l-0.23,0.02l-0.29,0.24l-0.34-0.09l-0.17,0.03l-0.28,0.33l-0.48,0.12
				l-0.27,0.39l-0.31,0.15l-0.22,0l-0.19-0.2l-0.14,0.09l-0.15-0.03l-0.23-0.23l-0.39-0.73L318.11,58.3L318.11,58.3z"></path>
		</g>
		<g id="sahagun_campos">
			<path id="path181" class="st3 fase_0" d="M333.43,60.06l-0.56,0.3l-0.99,1.57l-0.23,0.52l-0.64,0.55l-0.33,0.86l-0.23,0.29
				l-0.52,0.28l-0.14,0.26l0.01,0.21l-0.48,0.43L329,65.82l-0.11,0.27l0.03,0.12l0.13,0.43l-0.75,0.58l-0.37-0.07l-0.61-0.46
				L327,66.62l-0.4,0.11l-0.02,0.16l-0.94-0.42l-0.57-0.63l-0.15-0.03l-0.16-0.3l-0.22,0.22l-0.29,0.38l-0.02,0.27l-0.18,0.25
				l-0.07,0.48l0.07,0.14l-0.04,0.73l0.15,0.28l0.1,0.11l0.03,0.61l-0.21,0.15l-0.33-0.1l-0.04-0.19l-0.76,0.38l-0.05-0.1
				l-0.27,0.38l-0.47,0.36l-0.39,0.71l-0.99,0.98l1.01,0.98l0.46,1.02l-0.17,0.65l0.84,0.54l0.69,0.02l0.05-0.08l0.39,0.57
				l0.04,0.77l-0.1,0.27l-0.18,0.31l-0.25,0.01l-0.62,0.24l-0.89-0.24l-0.21-0.33l-0.26-0.19l-0.23,0.1l0.48,0.35l-0.11,0.26
				l0.11,1.02L321.69,78l-0.02,0.22l-0.09-0.02l-0.1,0.24l-0.08,0.02l-0.41-0.01l-0.43-0.18l-0.02,0.05l-0.13-0.05l0.04,0.27
				l-0.29,0.09l-0.03,0.15l-0.01,0.23l-0.19-0.21l-0.22,0.03l-0.12,0.28l-0.37-0.1l-0.33-0.25l-0.08,0.46l-0.33,1.17l0.31,0.27
				l0.24,0.03l-0.09,0.1l0.15,0.3l-0.04,0.7l-0.39,0.38l0.08,0.64l0.22,0.36l0.35-0.24l0.18-0.03l0.18,0.33l-0.69,0.38l-0.11-0.12
				l-0.11,0.13l0.11,0.4l-0.09,0.2l0.34,0.56l0.24,0.06l0.24-0.12l0.13-0.28l-0.05-0.11l0.25-0.12l-0.05,0.65l0.12,0.06l0.24-0.69
				l0.09-0.14l0.15,0l0.66,0.71l0.1,0.22l0.61-0.2l-0.03-0.21l0.22-0.16l0.56,0l0.33-0.15l0-0.2l0.48-0.21l0.73-1.05l0.12-0.04
				l0.15-0.32l0.23-0.11l0.07-0.11l0.1,0l0.03,0.06l0.61,0.07l0.27,0.63l0.04,0.28l0.15-0.08l0.18,0.11l-0.04,0.1l0.14,0.25
				l0.01,0.33l0.13,0.06l0.05-0.07l0.19,0.16l0.16-0.09l-0.24,0.9l0.54,0.2l0.39-0.24l0.14-0.3l-0.03-0.21l0.14-0.31l0.08-0.64
				l-0.19-0.2l0.13-0.23l1.47,0.43l0.3-0.21l0.2,0.07l0.13-0.11l0.53,0.28l0.13-0.18l0.68-0.27l0.11-0.6l0.11-0.13l0.04-0.3
				l-0.08-0.13l0.12-0.28l0.15-0.19l0.01-0.13l0.06-0.31l-0.07-0.59l-0.42,0l-0.05,0.13l-1.13,0.35l-0.08-0.01l0.03-0.25l0.26-0.35
				l-0.02-0.35l0.24-0.66l-0.03-0.35l0.1-0.02l0.05-0.13l0.17-0.13l0.13,0.19l0.67-0.02l-0.03-0.07l0.09-0.09l-0.12-0.12l0.41-0.04
				l0.12-0.14l0.89,0.17l0.33-0.27l-0.32-0.31l0.12-0.25l-0.24-0.23l0.07-0.19l-0.09-0.53l0.35-0.32l0.04-0.23l-0.15-0.21l-0.05-0.3
				l-0.57-0.12l0.12-0.06l0.3-0.21l0.32-0.42l0.23-0.08l0.3-1.91l0.19-1.22l-0.1-0.01l-0.04-0.5l0.13-1.01l-0.89-0.22l-0.19-0.19
				l-0.05-0.28l0.02-0.56l0.51-0.95l0.05-0.35l0-0.55l-0.15-0.24v-0.22l0.41-1.11l-0.15-0.63l0.05-0.45l0.04-0.63l-0.17-0.73
				l0.4-1.27l0.18-0.14l0.25-0.46l0.34-0.21l-0.49-0.25l-0.04-0.3l-0.26-0.28L333.43,60.06L333.43,60.06z"></path>
		</g>
		<g id="bembibre">
			<path id="path73" class="st3 fase_0" d="M276.39,55.24l-0.17,0.08l-0.08,0.23l-0.39,0.36l-0.22,0.06l-0.42,0.28l-0.31-0.01
				l-0.14,0.11l-0.28-0.09l-0.35,0.08l-0.46,0.33l-1.11,0.42l-0.19,0.21l-0.09,0.24l0.34,0.59l-0.84,0.31l-0.24,0.33l-0.08,0.36
				l-0.31,0.01l-0.86,1.6l-0.3,0.17l-0.26,0.65l-0.2,0.09l0.11,0.25L269.15,62l-0.35,0.24l-0.07,0.28l0.1,0.03l0.3,0.83l-0.16,0.15
				l0,0.17l-0.1,0.18l-0.33,0.37l0.6,0.24l0.41,0.32l0.09,0.56l-0.07,0.87l-0.16,0.2l-0.17,0.68l-0.7,0.51l-0.46,0.14l-0.55,0.06
				l-0.13,0.73l0.1,0.27l0.63-0.26l0.24-0.02l0.36,0.11l0.22-0.12l0.23,0.24l0.2,0.05l0.02,0.15l0.02-0.07l0.36-0.17l0.12,0.28
				l0.34,0.22l0.91,0.21l0.44,0.18l0.1-0.05l-0.04-0.14l0.36,0.09l0.27,0.34l1,0.89l0.22,0.59l0.37,0.13l0.54,0.33l0.38-0.15
				l0.64,0.06l0.33,0.3l0.46-0.4l-0.09-0.15l0.02-0.17l0.37,0.02l-0.16-0.25l0.2-0.2l0.02-0.22l-0.15-0.24l0.2,0.08l0.17-0.02
				l-0.18-0.21l0.14-0.17l0.17-0.64l0.27,0.11l0.52-0.14l0.39,0l0.49-0.08l0.32-0.29l-0.52-0.17l-0.25-0.46l-0.34-0.26l-0.02-0.22
				l-0.73-0.51l-0.03,0.04l-0.75-0.46l0.56-0.24l0.1-0.22l0.23-0.03l0.2-0.17l0.34,0.09l0.7-0.26l-0.04-0.53l0.7-0.33l0.21-1.09
				l0.31-0.13l0.23-0.52l0.47-0.12l0.37,0l-0.02-0.26l0.13-0.19l0.03-0.34l0.13-0.17l-0.44-0.03l-0.31-0.18l0.08-0.55l0.42-0.17
				l0.07-0.12l-0.06-0.11l0.14-0.13l0.73,0.31l0.74-0.22l0.1-0.21l0.42-0.26l0.32,0.03l0.02-0.08l0.58-0.17l0.17-0.14l0.02-0.55
				l0.16-0.29l-0.08-0.16l-0.27-0.18l-0.17-0.51l0.18-0.08l-0.94-0.41l-0.22,0.08l-0.55,0.49l-1.17-0.37l-0.57-0.05l-0.11-0.44
				l-0.29-0.16l-0.09-0.17l-0.36-0.13l-0.17-0.15l-0.11-0.02l-0.34,0.21l-0.11-0.16l-0.1-0.23l0.05-0.23l0.34-0.37l-0.19-0.44
				l-0.31-0.2L278,55.85l-0.34,0.11l-0.06,0.09l-0.36,0.03l-0.5-0.53l0.01-0.16L276.39,55.24L276.39,55.24z"></path>
		</g>
		<g id="baneza_ii">
			<path id="path309" class="st3 fase_0" d="M294.42,75.24l-0.51,0.04l-0.02,0.27l-0.13,0.49l0.15,0.5l0.02,0.89l-0.12,0.26l-0.06,0.07
				l-0.17-0.09l-0.05-0.18l-0.13,0.02l0.07,0.22l-0.38,0.04l0.15-0.21l-0.32-0.08l-0.08-0.31l-0.17,0.11l-0.23-0.17l0.04-0.11
				l-0.15-0.01l-0.22-0.37l-0.49-0.14l-0.84-0.07l-0.08,0.1l-0.01,0.52l-0.29,0.09l-0.05,0.1l0.19,0.24l0.03,0.1l-0.5,0.32
				l-0.56-0.75l-0.44,0.37l-0.27,0.12l-0.03,0.11l0.08,0.12l-0.2,0.44l0.09,0.27l-0.33,1l0.11,0.01l-0.03,0.34l-0.53-0.17
				l-0.35,0.36l-0.17-0.03l-1.52-0.09l-1.44-0.43l-0.52-0.17l-0.72-0.4l-0.41-0.18l-0.04,0.15l-0.31,0.37l-0.22,0.42l-0.33-0.02
				l-0.3,0.15l0.22,0.34l0.24-0.01l0.23,0.19l-0.41,0.72l-0.16,0.86l0.41,0.16l0.15-0.13l-0.11,0.18l-0.14,0.88l-0.18,0.3l0.22,0.16
				l-0.54,0.16l-0.55,1.1l-1.02-0.15l-1.26-0.66l-0.33,0.54l-0.97-0.37l-0.33,0.4l-0.11,0.21l-0.1,0.69l-0.23,0.37l0.21,0.69
				l0.05,0.78l0.19,0.31l-0.13,0.1l-0.87,0.23l0.23,0.2l0.16,0.58l0.19,0.46l0.17,0.07l0.02,0.23l0.64-0.05l1.05,0.32l-0.09,0.32
				l0.48-0.03l0.66,0.3l0.29,0.59l0.26,0.16l1.7-0.54l0.41-0.41l0.25,0.24l0.43,0.13l0.25-0.32l0.51,0.31l0.78-0.17l0.38,0.06
				l0.27-0.04l0.49,0.14l0.08-0.15l0.33,0.09l1.57,0.14l0.49,0.27l0.38,0.12l0.7,0.38l0.63,0.71l0.54,0.16l0.17,0.17l0.13,0.02
				l0.05-0.61l0.17-0.37l0.5,0.04l0.17,0.1l0.2-0.07l0.61,0.01l0.22-0.18l0.35-0.65l0.28,0.33l0.39,0.25l-0.15,0.37l0.73,0.99
				l0.34,0.22l0.55,0.28l0.16-0.1l0.11-0.33l0.24-0.03l0.81-0.38l0.23-0.22l0.08,0.04l-0.07,0.11l0.04,0.17l0.29-0.16l0.28,0.02
				l0.13-0.33l0.32-0.1l0.41-0.04l0.42,0.09l0.17-0.12l0.17,0.66l-0.04,0.16l0.19,0.29l0.17,0.14l0.24-0.02l0.8-0.07l0.84-0.31
				l0.81,0.02l-0.12-0.66l-0.24-0.17l-0.06-0.14l0.25-0.19l0.05,0.08l0.49-0.1l0.09,0.05l-0.01-0.14l0.3-0.36l0.58,0.24l0.38-0.55
				l0.05,0.03l0.4-0.55l0.16-0.1l-0.09-0.1l-0.01-0.37l0.26-0.46l-0.09-0.03l-0.78-0.06l-0.51-0.54l-0.18-0.08l-0.1-0.49l-0.31,0.43
				l0.11,0.18l-0.04,0.42l-0.17-0.01L302.26,89l-0.36,0.25l-0.23-0.02l-0.51-0.95l-0.1-0.49l-0.19-0.36l-0.35-0.02l-0.57,0.13
				l-0.51,0.28l-0.16-0.06l0.1-0.9l-0.23-0.8l-0.35-0.05l-0.66-0.64l0.14-0.29l-0.33-0.63l0.03-0.23l-0.2-0.04l-0.12-0.22
				l-0.22,0.16l-0.54,0.01l-0.47-0.01l0.51-0.43l-0.06-0.41l-0.37-0.92l0.1-0.28l0.37-0.12l-0.05-0.33l0.11-0.04l-0.3-0.18
				l-0.01-0.29l-0.21-0.52l-0.03-0.02l-0.02-0.12l0.09,0l-0.02-0.19l-0.2-0.12l-0.14,0.05l0.13-0.29l-0.1-0.18v-0.39l-0.16-0.27
				l0.08-0.11l0-0.24l-0.19-0.14l0.14-0.54l-0.28-0.76l-0.24-0.15l-0.34,0.32l-0.2-0.91l-0.14-0.02l-0.02,0.1l-0.1-0.01l-0.19-0.1
				l-0.03-0.15l0.2,0l-0.06-0.16l0.12-0.25l-0.33-0.05l0.17-0.21l-0.27-0.25L294.42,75.24L294.42,75.24z"></path>
		</g>
		<g id="sta__maria_paramo">
			<path id="path93" class="st3 fase_0" d="M297.87,72.31l-0.49,0.13l0.19,0.33l-0.41,0.14l-0.25,0.19l-0.19,0.02l0.05,0.46l-0.29,0.06
				l-0.06-0.06l-0.4,0.14l0.11,0.29l-0.07,0.01l0.1,0.67l-0.28,0.1l-0.23-0.15l-0.43,0.07l-0.19-0.24l-0.19-0.01l-0.21-0.14
				l-0.33,0.01l-0.59,0.03l-0.02,0.34l0.2,0.85l0.02-0.27l0.51-0.04l-0.03,0.2l0.27,0.25l-0.17,0.21l0.33,0.05l-0.12,0.25l0.06,0.16
				l-0.2,0l0.03,0.15l0.19,0.1l0.1,0.01l0.02-0.1l0.14,0.02l0.2,0.91l0.34-0.32l0.24,0.15l0.28,0.76l-0.14,0.54l0.19,0.14l0,0.24
				l-0.08,0.11l0.16,0.27v0.39l0.1,0.18l-0.13,0.29l0.14-0.05l0.2,0.12l0.02,0.19l-0.09,0l0.02,0.12l0.03,0.02l0.21,0.52l0.01,0.29
				l0.3,0.18l-0.11,0.04l0.05,0.33l-0.37,0.12l-0.1,0.28l0.37,0.92l0.06,0.41l-0.51,0.43l0.47,0.01l0.54-0.01l0.22-0.16l0.12,0.22
				l0.2,0.04l-0.03,0.23l0.33,0.63l-0.14,0.29l0.66,0.64l0.35,0.05l0.23,0.8l-0.1,0.9l0.16,0.06l0.51-0.28l0.57-0.13l0.35,0.02
				l0.19,0.36l0.1,0.49l0.51,0.95l0.23,0.02l0.36-0.25l0.04-0.11l0.17,0.01l0.04-0.42l-0.11-0.18l0.31-0.43l0.1,0.49l0.18,0.08
				l0.51,0.54l0.78,0.06l0.12-0.67l0.16-0.21l0.14-0.05l-0.09-0.1l0.06-1.15l0.22-0.18l0.04-0.19l-0.36-0.53l0.05-0.47l0.13-0.19
				l-0.48-0.34L304,84.61l-0.24-0.18l0.01-0.49l-0.47-0.29l0.18-0.22l-0.01-0.17l0.11-0.16l-0.02-0.42l-0.15-0.01l-0.04-0.2
				l0.21-0.35l-0.03-0.08l0.11-0.05l-0.01-0.1l-0.5-0.31l-0.07-0.47L302.78,81l-0.01-0.22l0.21-0.15l-0.13-0.41l0.1-0.14l0.22-0.36
				l0.6-0.67l0.67-0.3l0.23-0.21l-0.25-0.56l0.12-0.35l0.01-0.28l0.38-0.13l0.14,0.02l0.25-0.2l0.16-0.02l-0.07-0.05l0.06-0.04
				l0.22,0.06l0.15-0.25l0.34-0.07l0.01-0.34l-0.16-0.13l-0.32,0.01l-0.19-0.33l-0.06,0l0.02-0.26l-0.3-0.34l-0.24-0.12l-0.22,0.1
				l-0.1,0.5l-0.3-0.05l-0.3,0.09l0.1,0.44l-0.06,0.05l-0.56-0.58l-0.16,0.12l0.17-0.74l-0.6,0.01l-0.29-0.25l-0.24-0.4l0.07-0.06
				l-0.35-0.27l-0.17,0.29l-0.04,0.32l-0.12,0.07l-0.41,0.16l-0.07-0.07l-0.21,0.02l-1,0.25l-0.28,0.04l-0.29-0.11l-0.6-0.19
				l-0.19-1.42l-0.51,0.21l-0.14-0.3l0.23-0.22l-0.11-0.31L297.87,72.31L297.87,72.31z"></path>
		</g>
		<g id="ponferrada_ii">
			<path class="st3 fase_0" d="M266.45,67.49l0.07,0.01c0.12-0.35,0.27-0.7,0.45-1.02c0.46-0.83,1.16-1.47,1.86-2.11l-0.28-0.11l-0.7-0.27
				l-0.12,0.73l-0.13,0.08l-0.09-0.1l-0.39-0.12l-0.47-0.39l0.05-1.59l-0.09-0.42l-1.01-0.53l-0.27-0.33l-0.75-0.23v0L264,62.11
				l0.14,0.49l0.1,1.43l-0.57,0.26l-0.06,0.11l-0.65,0.42l-0.74,0.68l-0.18,0.26l0.24,0.16l0.06,0.17l-0.03,0.11l0.57,0.23
				l-0.13,0.25l-0.54,0.29l-0.61,0.89v2.44l2.05-0.17l1.11,0.06L266.45,67.49z"></path>
		</g>
		<g id="cacabelos">
			<path id="path163" class="st3 fase_0" d="M263.14,60.99l-0.44,0.05l-0.71,0.49l-0.22-0.04l-0.22,0.09l-0.6-0.04l-1.01,0.99
				l-0.42,0.59h0l-1.08,0.51l-0.22,0.26l-0.25,0.01l-0.25,0.82l0.08,0.17l0.29,0.26l0.24,0.43l0.17,0.1l-0.42,0.23L258,66.43
				l0.19,0.16l0.33,0.73l0.1-0.32l0.5-0.26l0.17,0.32l-0.08,0.14l0.02,0.22l-0.06,0.1l-0.17,0.04l-0.06,0.58l-0.48-0.21l-0.56,0.6
				l0.26,0.7l0.32,0.24l-0.03,0.16l-0.75,0.64l-0.31,0.09l-0.08,0.26l0.11,0.45l-0.05,0.16l0.22,0.14l0.38-0.49l0.72-0.07l0.41,0.11
				l0.5,0.29l0.36,0.34l0.28-0.47l-0.36-1.07l0.34-0.04l0-0.22l0.21-0.45l0.91,0.06l0.31,0.11l0.5-0.06l0.34-0.14l-0.09-0.48
				l-0.09-0.1l0.05-0.47l-0.31-0.19l0.17-0.09l0.53-0.97l0.01-0.3l0.13-0.25l-0.57-0.23l0.03-0.11l-0.06-0.17l-0.24-0.16l0.18-0.26
				l0.74-0.68l0.65-0.42l0.06-0.11l0.57-0.26l-0.1-1.43L264,62.11l0.58-1.04l-0.22,0.08l-0.58-0.12l-0.48,0.11L263.14,60.99
				L263.14,60.99z"></path>
		</g>
		<g id="jose_aguado_ii">
			<path id="path167" class="st3 fase_0" d="M317.3,59.23l-0.47,0.16l-0.45,0.06l-0.25-0.06l-0.1,0.2l-0.42-0.01l-0.17-0.08l-0.24,0.04
				l-0.42,0.18l-0.09,0.18l-0.17,0.44l-0.01,0.27l0.12,0.6l-0.2,0.3l-0.14,0.04l-0.24,0.53l-0.36-0.73l-0.27-0.09l-0.8,0.24
				l-0.36-0.25l0.09-0.4l-0.17,0.03l-0.92-0.31l-0.52-0.35l-0.53-0.19L310,60.2l-0.1-0.03l-0.15,0.1l-0.2,1.3l-0.03,0l0.07,0.55
				l-0.19,0.9l-0.28,0.62l-0.09,0.05l0.24,0.09l0.02,0.27l-0.41,0.38l-0.46,0.14l-0.17,0.67l-0.21,0.3l0.01,0.52l-0.22,0.26
				l-0.2,0.56l-0.01,0.77l-0.16,0.55l0.16,0.5l-1.14,0.41l0.44,0.93l0.03,0.39l0.25,0.39l-0.14,0.11l0.01,0.16l0.14,0.16l0.18,0
				l0.56,0.55l-0.04,0.13l-0.1,0.03l0.09,0.18l-0.09,0.15l0.14,0.15l-0.09,0.21l0.02,0.43l0.34,0.48l-0.19,0.49l0.12,0.18l0.67-0.54
				l0.03-0.21l0.23-0.17l0.03-0.25l0.68-0.15l0.34-0.34l-0.02-0.66l0.21-0.52l0.1-0.09l0-0.14l0.23-0.12l0.07,0.06l0.1-0.09
				l0.1-0.28l-0.06-0.02l0.09-0.7l0.04-0.13l-0.25-0.1l-0.05-0.16l0.23-0.37l0.5-0.36l0.18-0.47l0.47-0.49l0-0.16l0.87-0.83
				l0.26,0.14l0.4,0.08l0.73-0.62l0.32-0.49l0.12,0.02l-0.02-0.17l-0.37-0.47l0.34-0.49l0.31-0.17l0.69-0.86l0.03-0.77l-0.13-0.24
				l0.43,0.14l0.14-0.2l0.12-0.45l0.19-0.08l0.44-0.95l0.16-0.42l-0.03-0.27l0.15-0.21l0.04-0.35l0.15-0.27L317.3,59.23L317.3,59.23
				z"></path>
		</g>
		<g id="ribera_del_orbigo">
			<path id="path191" class="st3 fase_0" d="M296.06,62.34l-0.16,0.56l-0.08,0.01l0.22,0.67l-0.28,0.13l-0.05,0.05l-0.41-0.12
				l-0.02-0.28l-0.27-0.07l-0.64-0.45l-1.12-0.04l-0.35-0.11l-0.7,0.36l0.13-0.6l-0.74,0.36l-0.34,0.02l-0.3-0.1l0.14,0.4l0.56,0.82
				l0.14,1.04l0.05-0.11l0.24-0.13l0.33,0.62l-0.09,0.14l0.1,0.64l0.06,0.49l-0.07,0.23l-0.28-0.12l-0.69,0.35l-0.21,0.01l-0.2,0.22
				l-0.73,0.48l-0.68,0.99l-0.39,0.31l-0.14,0.92l0.1,0.41l-0.4,0.21l-0.26,0.34l0.53,0.34l0.22,0.48l0.41,0.31l0.02,0.18l0.5,0.32
				L289.85,73l0.16-0.06l-0.11,0.18l0.21,0.74l-0.08,0.02l-0.15,0.63l0.05,0.34l-0.37,0.06l0.13,0.29l0.58,0.74l0.42,0.27l0.1,0.19
				l0.84,0.07l0.49,0.14l0.22,0.37l0.15,0.01l-0.04,0.11l0.23,0.17l0.17-0.11l0.08,0.31l0.32,0.08l-0.15,0.21l0.38-0.04l-0.07-0.22
				l0.13-0.02l0.05,0.18l0.17,0.09l0.06-0.07l0.12-0.26l-0.02-0.89l-0.15-0.5l0.13-0.49l-0.2-0.85l0.02-0.34l0.59-0.03l0.33-0.01
				l0.21,0.14l0.19,0.01l0.19,0.24l0.43-0.07l0.23,0.15l0.28-0.1l-0.1-0.67l0.07-0.01l-0.11-0.29l0.4-0.14l0.06,0.06l0.29-0.06
				l-0.05-0.46l0.19-0.02l0.25-0.19l0.41-0.14l-0.19-0.33l0.49-0.13l0.35,0.52l0.11,0.31l0.9-0.52l-0.04-0.25l0.4-0.1l0.1-0.15
				l0.67-0.21l0.11-0.25l-0.28-0.03l-0.13-0.44l-0.22-0.18l-0.3,0.19l0.08-0.21l-0.27,0.06l0.01-0.36l-0.19-0.37l0.39-0.37
				l-0.07-0.42l0.04-0.5l-0.43-0.45l-0.04-0.24l0.12-0.49l0.02-0.66l-0.1-1.02l0.41-0.31l0.16,0.05l-0.34-0.52l-0.04-0.28
				l-0.71-0.26l-0.3-0.55l-0.1-0.46l-0.34-0.64l-0.23-0.24l-0.2-0.08l-0.26-0.31l-0.26,0.06l-0.39-0.19L296.06,62.34L296.06,62.34z"></path>
		</g>
		<g id="san_andres_rabanedo">
			<path class="st3 fase_0" d="M314.51,60.34l-0.01,0.27l0.12,0.6l-0.2,0.3l-0.14,0.04l-0.24,0.53l-0.36-0.73l-0.27-0.09l-0.8,0.24
				l-0.36-0.25l0.09-0.4l-0.17,0.03l-0.92-0.31l-0.52-0.35l-0.53-0.19L310,60.2l-0.1-0.03l-0.15,0.1l-0.2,1.3l-0.03,0l0.07,0.55
				l-0.19,0.9l-0.28,0.62l-0.09,0.05l0.24,0.09l0.02,0.27l-0.41,0.38l-0.46,0.14l-0.17,0.67l-0.21,0.3l0.01,0.52l-0.22,0.26
				l-0.2,0.56l-0.19-0.08l-0.16,0.08l-0.33-0.24l0.03-0.2l-0.14-0.31l-0.03-0.39l-0.22,0.08l-0.81-0.34l-0.3-0.27l-0.24-0.63
				l-0.43,0.13l0.01,0.35l0,0l0.12,0.74l0.35,0.6l0,0.24l0.23,0.41l-1.82,0.53l-0.43-0.12l-0.28-0.31l-0.2-0.06l-0.11-0.12
				l-0.78-0.24l-0.37-0.21l-0.84-0.12l-0.69-0.39l-0.36-0.11l-0.34-0.52l-0.04-0.28l-0.07-0.13l0.01-0.44L298.96,64l0.22-1.08
				l0.27-0.3l0.74,0.16l0.18-1.15l-0.43-1.43l-0.09-0.59l0.25-0.71l-0.04-1.09l-0.11-0.41l0.38-0.31l0.29-0.11l0.36,0.08l0.48-0.23
				l0.59,0.52l0.17-0.03l0.17,0.21l0.21,0.01l0-0.06l0.47-0.19l0.25,0.08l0.26-0.39l-0.04,0.24l0.64,0.2l0.37,0.45l0.34-0.03
				l0.94-0.32l0.46-0.02l0.2-0.21l0.03-0.2l0.58-0.08l0.49-0.77l0.39-0.39l0.44,0.19l0.75-0.1l0.8,0.07l0.18-0.2l0.32-0.11l0.06-0.2
				l0.24-0.24l0.05-0.15l0.67,0.15l0.22,0.21l0,0.2l0.17,0.21l0.1-0.04l0.37,0.15l0.12-0.1l0.26,0.07l0.05,0.28l0.2,0.06l0.15,0.16
				l0.3,0.64l0.19-0.41l0.32-0.13l0.44,0.22l0.37,0.07l-0.34,0.26l-0.55,0.16l0.06,0.1l0.55,0.24l0.15,0.26l0.7,0.68l-0.72,1.32
				l0.05,0.07L314.51,60.34z"></path>
		</g>
		<g id="truchas">
			<path id="path317" class="st3 fase_1" d="M265.6,76.89l-0.39,0.23l-0.33,0.58l-0.02,0.34l-0.08,0.12l-0.46,0.32l-0.18,0.28
				l-0.55,0.29l0.05,0.14l-0.09,0.37l0.06,0.19l-0.17,0.25l0.38,0.32l-0.23,0.55l0.16,0.11l-0.21,0.63l-0.26,0.26l-0.5,0.08
				l-0.67,0.72l-0.3,0.22l-0.26,0.03l-0.07,0.15l-0.25-0.02l-0.25-0.28l-0.23,0.22l-0.16-0.01l-0.13-0.16l-0.39-0.13l-0.14-0.18
				l-0.84-0.56l-0.23-0.17l0.13,0.14l0.01,0.4l-0.28,0.42l0.08,0.25l-0.46,0.53l-0.11,0.32l-0.43,0.64l0.05,0.16l-0.09,0.56
				l-0.17,0.15l-0.12,0.03l-0.07-0.07l-0.34,0.1l0.74,0.43l0.28,0.37l0.65-0.08l0.33,0.25l0.07,0.21l-0.1,0.08l0,0.12l0.09,0.21
				l0.23,0.08l0.09-0.04l0.1,0.2l-0.03,0.15l0.16,0.19l0.33-0.04l0.31,0.25l0.19-0.02l0.4-0.27l0.6,0.01l0.14,0.11l0.48-0.07
				l0.18,0.04l0.28-0.21l0.38-0.02l0.35,0.29l0.26-0.01l0.19,0.15l0.1,0.27l0.12,0.06l0.25-0.14l0.13,0.1l0.27,0.03l0.14-0.16
				l0.24-0.08l0.56,0.17l0.65-0.12l0.09,0.05l1.05,0.19l0.15-0.03l0.2,0.17l0.09-0.02l0.35,0.2l0.17,0.16l-0.02,0.18l0.09,0.08
				l-0.16,0.26l0.34,0.03l0.09,0.1l0.22-0.02l0.06-0.09l0.49,0.1l0.06-0.07l0.32-0.5l0.3-0.12l0.06-0.1l0.71,0.19l0.17-0.26
				l0.41-0.19l0.2,0.16l0.51,0.17l0.48-0.11l0.04-0.16l0.64-0.13l0.04,0.09l1.35,0.05l0.22-0.25l0.5,0.28l0.24-0.2l0.11,0.06
				l0.34-0.23l0.87-0.23l0.13-0.1l-0.19-0.31l-0.05-0.78l-0.21-0.69l0.23-0.37l0.1-0.69l0.11-0.21l0.33-0.4l0.97,0.37l0.33-0.54
				l-0.21-0.14l-0.32-0.01l-0.47-0.18l-0.31-0.31l-0.25-0.45l-1.37-0.35l-0.14-0.14l-0.57-0.33l-0.16,0l-0.21-0.12l-0.23-0.25
				l-0.22-0.01l-0.21-0.11l-0.05-0.22l-1.73-1.24l-0.9-0.19l-0.83-0.65l-0.48-0.11l-0.14-0.21l-0.13-0.45l-0.18-0.1l-0.12-0.19
				l-0.45-0.07l-0.37,0.15l0.06-0.24l-0.57-0.17l-0.2-0.3l-0.36-0.2l-0.32,0.03l-0.37-0.15l-0.63,0.12l-0.06-0.2L265.6,76.89
				L265.6,76.89z"></path>
		</g>
		<g id="valderas">
			<path id="path445" class="st3 fase_0" d="M309.07,89.51l-0.17,0.17l-0.09,0.2l0.04,0.38l-0.79-0.08l-0.35,0.9l-0.05,0.37l-1.55,0.27
				l0.05-1.32l-1.12-0.44l-0.35,0.06l-0.32-0.1l-0.16,0.07l-0.16,0.1l-0.4,0.55l-0.05-0.03l-0.38,0.55l-0.58-0.24l-0.3,0.36
				l0.01,0.14l-0.02,0.23l0.84,0.25l0.29,0.54l0.67,0.72l0.09,0.2l-0.19,0.66l-0.57,0.41l0.05,0.07l0.12,0.13l0.38-0.12l0.75,0.22
				l0.08-0.06l0.02,0.06l0.03-0.05l0.26,0.06l-0.02,0.14l0.14,0.03l0.1-0.14l0.12,0.07l-0.01,0.09l-0.03,0.17l0.12,0.07l0.12,0.28
				l-0.22,0.71l0.09,0.1l0.26-0.41l0.03-0.32l0.07-0.06l0.12,0.05l0.2,0.36l0.2-0.07l0.17,0.18l0.13-0.25l0.19-0.16l0.13-0.36
				l0.05-0.47l0.17-0.38l-0.01-0.33l0.1-0.3l-0.06-0.07l0.66-0.26l0.25,0.39h0l0.11,0.34l0,0.01l-0.02,0.32l0.38,0.41l0.3,0.56
				l-0.05,0.21l-0.2,0.27l0.47,0.45l0.01,0.19v0l0.38,0.36l-0.25,0.3l0.18,0.18l0.02,0.22v0l0.6,0.66l0.04,0.39l-0.27,0.1
				l-0.01,0.07l0,0l-0.01,0.2l0,0l-0.2,0.16l-0.17,0.04l0,0l0.06,0.23l0.09,0.36l0.89,0.37l-0.01,0.12v0l0.09,0.04l0.19-0.05
				l0.45,0.21h0l0.27,0.24l0.19-0.84l0.1-1.04l-0.21-0.35l-0.27-0.16l-0.19,0.07l0.02-0.16l1.05-0.92l0.05,0.08l0.33-0.07l0.58-0.22
				l0.54-0.4l0.33,0.03l0.08-0.09l-0.05-0.19l0.18-0.25l-0.13-0.9l0.36,0.14l0.17,0.19l0.32,0.16l1.26,0.03l0.79-0.41l0.07,0.07
				l0.13-0.26l0.08-0.73l-0.02-0.15l-0.1-0.03l0.02-0.12l0.33,0.14l0.66,0.08l0.24-0.15l0.43-0.1l0.14-0.25l-0.18-0.29l-0.01-0.3
				l-0.11-0.12l0-0.23l-0.1-0.13l0.3-0.28l-0.35-0.47l-0.42-0.16l-0.3,0.01l0.07,0.13l-0.14-0.05l-0.18,0.21l-0.15,0.03l-0.17,0.19
				l-0.51,0.21l-0.2,0.19l-0.3,0.09l-0.59-0.26l-0.04-0.21l-0.14,0.17l-0.25,0.12l-0.16,0l-0.09,0.09l-0.19-0.07l-0.1,0.04
				l0.04,0.07l-0.11,0.05l0.04,0.1l-0.15,0.12l-0.08-0.09l-0.15,0.09l-0.11-0.02l-0.18-0.25l-0.1,0v0l0.1,0l-0.01-0.07l0.21-0.22
				l-0.1-0.17l0-0.29l0.15-0.18l-0.39-1.11l-0.24-0.04l-0.31-0.21l-0.13,0.02l-0.01-0.06l-0.28,0l-0.37-0.18l-0.1,0.17l-0.36,0.25
				l-0.63,0.11l-0.19-0.18l-0.07,0.03l-0.29,0.15l-0.61-0.06l-0.64-0.66l-0.19-0.4l-0.04-0.29L309.07,89.51z"></path>
		</g>
		<g id="la_baneza_i">
			<path class="st3 fase_0" d="M296.36,86.15l-0.06,0.08l-0.05,0.32l-0.04,0.09l-0.08,0.05l-0.07,0.11l-0.05,0.41L296,87.28l-0.13,0.05
				l0,0.11l0.02,0.2l-0.1,0.13l0.03,0.02l-0.05,0.07l-0.12,0.04l-0.06,0.06l-0.05-0.03l-0.03,0.02l-0.09-0.03l-0.07,0.01l-0.1-0.09
				l-0.02,0.08l-0.12,0.13l-0.02,0.07l0.01,0.03l0.11,0.05l0.02,0.09l-0.02,0.19l-0.03,0.08l-0.18,0.02l-0.17-0.04l-0.07,0.02
				l-0.14-0.2l-0.13-0.12l-0.15-0.07l0.03-0.06l0.03-0.15l0.05-0.01l0.08-0.08l0.01-0.1l-0.05-0.11l0.04-0.08l-0.19,0.01l-0.2,0.17
				l-0.27-0.01l-0.18,0.04l-0.22-0.21l-0.08,0.05l-0.08,0.01l-0.05,0.08l-0.1,0.07l-0.21,0.08l-0.12,0.03l-0.11-0.05l-0.05,0.06
				l-0.08,0.04L292.56,88l-0.14,0.07l-0.1-0.06l-0.06-0.09l-0.09-0.07l-0.03,0.01l-0.07-0.07l-0.11,0l0.01,0.06l-0.12,0.07
				l-0.08,0.08l-0.17,0.09l-0.29-0.12l0.03-0.15l0.1-0.27l-0.1-0.16l-0.06-0.17l-0.09-0.08L291,87.14l-0.07,0.04l-0.24,0.01
				l-0.01-0.35l-0.18-0.35l0.03-0.15l0.13-0.09l0.04,0.02l0.12-0.04l0.05-0.06l-0.06-0.04l0.11-0.17l0.23-0.04l0.34-0.01l0.04-0.04
				l0.15-0.01l-0.04-0.13l0.03-0.28l-0.03-0.05l-0.22-0.07l-0.06-0.11l-0.06-0.05l-0.04-0.21l-0.19-0.19l0.02-0.05l0.09-0.06
				l-0.01-0.03l0.05-0.02l0.13-0.06l0.11,0.07l0.08-0.01l0.24,0.05l0.04-0.09l0.3,0.13l0-0.08l0.09-0.07l-0.08-0.03l-0.11-0.09
				l-0.2-0.05l0.05-0.19l0.04-0.06l0-0.26l0.23-0.29l0.11-0.22l0.07-0.13l0.11-0.45l0.08-0.13l-0.16-0.13l0.04-0.2l-0.05-0.2
				l0.01-0.04l-0.03-0.01l0.01-0.08l-0.05-0.05l-0.03-0.13l0.03-0.17l-0.09-0.15l-0.1-0.07l0-0.13l-0.06-0.06l-0.01-0.05l0.02-0.02
				l-0.03-0.15l0.04-0.13l0.04-0.18l-0.03-0.15l0.11-0.31l0.04-0.12l-0.01-0.11l-0.09-0.34l0.04-0.19l-0.07-0.07l0-0.05l0.03-0.04
				l0.02,0.04l0.04,0.01l0.04-0.02l0.06,0.03l0.06-0.08l0.08,0.02l0.03-0.06l-0.05-0.14l0.02-0.08l0.03,0.08l0.17,0.09l0.17,0.17
				l0.14,0.01l0.08,0.04l0.1,0.03l0.05,0.2l0.08,0.06l0.05,0.01l0.05,0.13l-0.01,0.12l0.07,0.12l0.11,0.03l0,0.06l0.05,0.03
				l-0.01,0.07l0.08,0.13l0.19,0.13l0.19,0.02l0.11-0.05l0.05-0.08l0.05-0.02l0.02,0.08l0.01,0.41l-0.03,0.13l0.05,0.03l0.02,0.16
				l-0.03,0.14l-0.05,0.07l0.05,0.08l0.11,0.07l0.01,0.05l-0.05,0.12l0,0.25l0.05,0.1l-0.01,0.1l-0.14,0.24l0.01,0.23l0.14,0.23
				l0.05,0.03l0.13,0.01l0.01,0.04l-0.01,0.06l-0.13,0.22l0.02,0.02l0.22,0l0.03,0.09l-0.01,0.06l-0.07,0.08l-0.04-0.03l-0.03,0.04
				l-0.03,0.1l-0.17-0.06l-0.04,0.03l-0.14-0.03l0.01,0.14l-0.08-0.01l-0.25,0.06l0,0.12l-0.02,0.04l0.02-0.01l0.04,0.03l-0.03-0.08
				l0.04-0.01l0.05-0.09l0.11,0l0,0.03l-0.08,0.07l-0.02,0.08l0,0.22l0.06,0.12l0,0.03l0.02,0.01l0.01,0.05l-0.02,0.05l0.03,0.05
				l0.13,0.09l-0.01,0.12l-0.04-0.02l-0.06,0.02l-0.1,0.13l0.16,0.19l0.08-0.09l0.05,0.03l0.08-0.01l0.03,0.02l0,0.12l0.16,0.15
				l0.18-0.05l0.17-0.09l0.07-0.07l0,0.04l0.05,0.05l0.24,0l0.02,0.23l0.03,0.03l-0.07,0.07l-0.04,0.08l-0.08,0.09l-0.01,0.15
				l-0.01,0.07l0.02,0.05l-0.03,0.12l-0.14,0.18l-0.01,0.06l0.04,0.04l-0.01,0.13l0.02,0.01l0.1-0.09l0.13-0.06l0.36-0.14l0.22-0.06
				l0.06,0.02l0.1-0.02l0.21-0.1l0.08-0.01L296.36,86.15z"></path>
		</g>
		<g id="astorga_i">
			<path class="st3 fase_0" d="M290.26,75.94l-0.58-0.74l-0.13-0.29l0.37-0.06l-0.05-0.34l0.15-0.63l0.08-0.02l-0.21-0.74l0.11-0.18
				L289.85,73l0.36-0.37l-0.5-0.32l-0.02-0.18l-0.41-0.31l-0.13-0.29c-0.91-0.17-2.38,0.01-3.18-0.41
				c-0.95-0.51-0.29-1.18-1.22-1.33c-0.01,0-0.37-0.33-0.44-0.33c0,0.01-0.61-0.33-1.05-0.33c-0.7,0-0.61,0.33-0.61,0.33
				c-0.05,0.18-0.26,0.57,0.22,0.67c0.32,0.06,0.25,0.15,0.3,0.41c0.02,0.04,0.03,0.09,0.05,0.13c0.02,0.06,0.03,0.12,0.03,0.17
				c0.08,0.25,0.2,0.49,0.39,0.74c0.65,0.88-0.38,0.77-1.22,0.87c-1.03,0.13-2.1,1.31-2.44,0.22c-0.14-0.45-1.39-0.66-1.22-1
				c-0.33,0.11-0.74,1.25-1,1.44c-0.8,0.61,0.72,0.22,0.73,1.11c0,0.02-0.07-0.02-0.07,0c0.45,0.53,1.7,0.22,2.33,0.33
				c1.11,0.21,1.62,0.81,2.75,0.98c1.1,0.16,1.82,0.16,2.8,0.69c0.06,0.03,0.79,0.53,1.11,0.67c0.11-0.14,1.08-0.46,1.22-0.55
				c0.8-0.54,1.17,0.02,1.98,0.44c0.03,0.01,0.05,0.03,0.07,0.05l0.01-0.32l0.08-0.1l-0.1-0.19L290.26,75.94z"></path>
		</g>
		<g id="toreno">
			<path class="st3 fase_0" d="M277.37,53.68l0.18,0.33l-0.17,0.65l-0.53,0.2l-0.17-0.04l-0.29,0.42l-0.17,0.08l-0.08,0.23l-0.39,0.36
				l-0.22,0.06l-0.41,0.28l-0.31-0.01l-0.14,0.11l-0.28-0.09l-0.35,0.08l-0.46,0.33l-0.51,0.19h0l-0.6,0.23l-0.19,0.21l-0.09,0.24
				l0.34,0.59l-0.84,0.31l-0.24,0.33l-0.08,0.36l-0.31,0.01l-0.86,1.6l-0.3,0.17l-0.26,0.65l-0.2,0.09l0.11,0.25L269.15,62
				l-0.35,0.24l-0.07,0.28l0.1,0.03l0.3,0.83l-0.16,0.15l0,0.17l-0.1,0.18l-0.33,0.37l-0.7-0.27l-0.12,0.73l-0.13,0.08l-0.09-0.1
				l-0.39-0.12l-0.47-0.39l0.05-1.59l-0.09-0.42l-1.01-0.53l-0.27-0.33l-0.75-0.23l-0.54-0.19l-0.16-0.21l-0.03-0.38l-0.52-0.57
				l0.21-0.42l0.28-0.15l0.42-0.37l0.31-0.12l0.09,0.08l0.29-0.01l0.81-0.38l0-0.18l0.12,0.01l0.27-0.26l0.03-0.23l0.14-0.52
				l-0.07-0.19l-0.15-0.14l-0.26-0.09l-0.38,0.02l0.09-0.46l-0.54-0.02l-0.19-0.21l-0.27-0.05l0.17-0.31l-0.24-0.34l-0.83-1.05
				l0.66-0.25l0.06-0.17l0.22-0.16l0.34-0.63l0-0.18l0.91-1.52l0.17-0.91l-0.25-0.69l1.13,0.12l0.12-0.39l0.15-0.13l0.47-0.08
				l0.12-0.12l0.37-0.15l0.57,0.39l0.27,0.07l0.15-0.47l0.36-0.27l1.05-0.08l0.36-0.21l0.38-0.02l0,0l0.36,0.08l0.32,0.2l0.38-0.07
				l0.75,0.04l0.21,0.15l0.18,0.03l-0.01,0.2l0.39,0.19l0.21-0.19l0.43,0.04l-0.03,0.33l0.14,0.46l-0.01,0.58l-0.9,0.36l0.31,0.29
				l0.29,0.11l0.38,0.04l0.53-0.12l0.23,0.08l0.01,0.26l0.47-0.01l0.14,0.09l0.39,0.03l0.17,0.14l0.35,0.06l0.1,0.47l0.38,0.52
				l0.02,0.24l0.12,0.07L277.37,53.68z"></path>
		</g>
		<g id="villablino">
			<path class="st3 fase_0" d="M279.03,44.5l-0.07,0.02l-0.69-0.11l-0.21-0.09l-0.17-0.23l-0.03-0.19l-0.15-0.06l-0.47-0.55l-0.21,0.1
				l-0.2,0.4l-0.65-0.11l-0.1-0.13l-0.52,0l-0.13-0.6l-0.14-0.21l-0.2,0.03l-0.1-0.18l-0.3-0.03l-0.11-0.18l-0.28-0.19l-0.05-0.16
				l-0.18,0.12l0.06,0.17l-0.1,0.21l-0.38,0.21l0.02,0.28l-0.67,0.03l-0.28,0.12l0.08,0.3l0.14,0.08l0.04,0.29l-0.14,0.49
				l-0.32,0.02l-0.23-0.12l-0.13,0.08l-0.04,0.44l0.07,0.38l0.12,0.11l-0.71,0.64l-0.1,0l-0.39-0.29l-0.38,0.58l-0.51-0.08
				l-0.28-0.15l-0.2-0.29l-0.26-0.01l-0.06,0.12l0.03,0.52l0.23,0.15l-0.09,0.36l0.07,0.15l0.18,0.09l0.25-0.08l0.43,0.09l0.38-0.05
				l0.35,0.15l-0.32,0.91l0.34,0.41l0,0l0,0l-0.38,0.02l-0.36,0.21l-1.05,0.08l-0.36,0.27l-0.15,0.47l-0.27-0.07l-0.57-0.39
				l-0.37,0.15l-0.07,0.06c-0.01,0.5-0.06,1-0.14,1.52c-0.1,0.63-0.25,1.25-0.41,1.87c0.3-0.02,0.59-0.05,0.88-0.1
				c0.72-0.14,1.26,0.23,1.48,0.74c0.43-0.09,0.86-0.16,1.31-0.13c0.78,0.06,1.47,0.43,2.23,0.6c0,0,0,0.01,0,0.01
				c0.09,0.03,0.18,0.06,0.28,0.1c0.01,0.01,0.02,0.01,0.03,0.02c0.19,0.09,0.33,0.25,0.43,0.42c0.32,0.13,0.63,0.29,0.93,0.5
				c0.22,0.15,0.44,0.32,0.68,0.44c0.04,0.02,0.06,0.03,0.07,0.04c0.01,0,0.03,0,0.08,0.01c-0.29-0.05,0.3-0.03,0.14,0
				c0.19-0.04,0.37-0.03,0.55,0l0.01-0.03l0.17-0.08l0.29-0.42l0.17,0.04l0.53-0.2l0.17-0.65l-0.18-0.33l0.21-0.54l-0.12-0.07
				l-0.02-0.24l-0.38-0.52l-0.1-0.47l-0.24-0.12l0.05-0.32l0.15-0.15l0.45,0.1l0.27-0.1l0.41,0.18l0.31,0l0.35-0.19l0.16-0.58
				l0.98-0.02l0.09-0.05C279.58,48.57,279.25,46.54,279.03,44.5z"></path>
		</g>
		<g id="ponferrada_iv">
			<path class="st3 fase_0" d="M265.01,64.38c-0.29-0.23-0.55-0.47-0.8-0.73l0.03,0.37l-0.57,0.26l-0.06,0.11l-0.65,0.42l-0.74,0.68
				l-0.18,0.26l0.24,0.16l0.06,0.17l-0.03,0.11l0.57,0.23l-0.13,0.25l-0.54,0.29l-0.61,0.89v2.44l2.05-0.17l0.5,0.03
				c-0.22-1.32-0.22-2.68,0.2-3.97C264.53,65.58,264.78,64.98,265.01,64.38z"></path>
		</g>
		<g id="ponferrada_i">
			<path class="st3 fase_0" d="M273.48,71.35l-0.22-0.59l-1-0.89L272,69.54l-0.36-0.09l0.04,0.14l-0.1,0.05l-0.44-0.18l-0.91-0.21
				l-0.34-0.22l-0.12-0.28l-0.36,0.17l-0.02,0.07l-0.02-0.15l-0.2-0.05l-0.23-0.24l-0.22,0.12l-0.36-0.11l-0.24,0.02l-0.63,0.26
				l-0.1-0.27l0.13-0.73l0.55-0.06l0.46-0.14l0.7-0.51l0.17-0.68l0.16-0.2l0.07-0.87l-0.09-0.56l-0.41-0.32l-0.6-0.24l-0.7-0.27
				l-0.12,0.73l-0.13,0.08l-0.09-0.1l-0.17-0.05c-0.09,0.17-0.2,0.34-0.3,0.5c-0.29,1.11-0.7,2.2-1.23,3.2
				c-0.51,0.96-1.24,1.89-2.24,2.3c0.04,0.12,0.07,0.24,0.08,0.38c0.05,0.94-0.05,1.88-0.26,2.79c0,0.4,0.03,0.79,0.11,1.19
				c0,0.01,0,0.03,0,0.04l0.1-0.19l0.01-0.1l0.17,0.12l0.03-0.1l0.07-0.09l0.09,0.11l-0.04,0.24l0.48,0.28l0.14,0.22l0.42,0.28
				l0.42,0.51l0.23,0.57l0.63,0.25l0.06,0.2l0.63-0.12l0.37,0.15l0.32-0.03l0.36,0.2l0.2,0.3l0.57,0.17l-0.06,0.24l0.37-0.15
				l0.45,0.07l0.06-0.45l0.12-0.09l0.14-0.37l-0.77-0.37l-0.06-0.32l-0.15-0.09l-0.06-0.15l0.07-0.2l0.59-0.13l0.75-0.46l0.31-0.11
				l0.23,0.01l0.24-0.15l0.15-0.03l0.39,0.19l0.78-1.39l0.03-0.31l-0.34-0.28l-0.02-0.15l0.09-0.6l0.78-0.34l0.2-0.01l0.33-0.27
				l0.34,0.03l0.18-0.64L273.48,71.35z"></path>
		</g>
		<g id="ponferrada_iii">
			<path class="st3 fase_0" d="M263.8,74.77l0.07-0.09l0.04,0.04c0.11-0.53,0.22-1.06,0.27-1.56c0.06-0.73,0.04-1.48,0.07-2.22
				c0.01-0.35,0.02-0.7,0.03-1.05c-0.69-0.1-1.36-0.3-2.03-0.52l-0.09,0.04l-0.5,0.06l-0.31-0.11l-0.91-0.06l-0.21,0.45l0,0.22
				l-0.34,0.04l0.36,1.07l-0.28,0.47l0.37,0.07l0.04,0.1l0.07,0.06l0.14,0.03l0.07,0.08L260.5,72l0.03,0.09l-0.07,0.21l0.02,0.26
				l-0.03,0.01l0.13,0.08l-0.29,0.02l-0.07-0.02l-0.21,0.07l0.09,0.26l0.16,0.22l0.08,0.2l-0.03,0.36l0.08,0.13L260.4,74l-0.1,0.28
				l0.12,0.05l0.18-0.01l0.11,0.03l0.15,0.07l0.03,0.19l0.25,0.63l0.37,0.13l0.18-0.03l0,0.02l0.23,0.09l0.97-0.18l0.16,0.05
				l0.12-0.03l0.06-0.11l0.11-0.05l0.1,0.01l0.16-0.3l0.01-0.1l0.17,0.12L263.8,74.77z"></path>
		</g>
		<g id="matallana_de_torio">
			<path class="st3 fase_1" d="M310.52,54.06l-0.15,0.03l-0.05-0.27l0.17-0.07l0.04-0.51l-0.1-0.48l-0.14-0.19l-0.02-0.39l-0.3-0.19
				l-0.03-0.62l-0.18-0.63l0.17-0.46l0.36,0.02l0.1-0.24l-0.12-0.56l0.08-0.48l0.25-0.41l0.31-0.91l-1.51-0.48l0.12-0.23l-0.05-0.31
				l0.09-0.36l0.23-0.43l-0.1-0.28l-0.03-0.41l-0.15-0.1l-0.35-0.51l0.11-0.52l-0.5-0.39l-0.14,0.08l-0.42-0.01l-0.34,0.08
				l-0.07,0.17l-0.41,0.23l-0.64-0.12l-0.08-0.11l-0.49-0.01l-0.18-0.51l-0.72-0.11l-0.12,0.04l-0.35-0.14l-0.21,0.11l-0.16-0.07
				l-0.58,0.07l-0.37-0.06l-0.08-0.17l-0.18,0.03l-0.28-0.37l-0.37-0.05l-0.04-0.09l-0.18-0.03l-0.26-0.24
				c0.42,1.41,1.27,4.24,1.27,4.24l0.18,1.49c0,0,0.57,0.73,0.72,1.14c0.12,0.31,0.5,1.52,0.5,1.52l1.33,1.55l0.78,2.16
				c0,0,0.3,0.39,0.49,0.59c0.41,0.42,0.82,0.7,1.16,0.88l0.64-0.09l0.8,0.07l0.18-0.2l0.32-0.11l0.06-0.2l0.24-0.24l0.05-0.15
				l-0.12-0.95L310.52,54.06z"></path>
		</g>
		<g id="ribera_del_esla">
			<path class="st3 fase_0" d="M324.45,62.75l-0.09-0.11l-0.08,0.05l0.09-0.13l0.33-1.34l0.2-0.45l-0.02-0.53l0.27-0.43l-0.11-0.5
				l-0.09,0.04l-0.83-0.35l-0.65,0.01l-0.14,0.09l-0.22,0.28l-0.02,0.76l-0.12,0.1l-0.18,0.44l-0.16,0.08l-0.2,0.54l-0.38-0.03
				l-0.79-0.6l0.48-1.03l-0.03-0.48l0.24-0.52l-0.1-0.03l-0.23,0.02l-0.29,0.24l-0.34-0.09l-0.17,0.03l-0.28,0.33l-0.48,0.12
				l-0.27,0.39l-0.31,0.15l-0.22,0l-0.19-0.2l-0.14,0.09l-0.15-0.03l-0.23-0.23l-0.39-0.73l-0.01-0.38l0,0l-0.82,0.94l0.14,0.62
				l-0.15,0.27l-0.04,0.35l-0.15,0.21l0.03,0.27l-0.16,0.42l-0.44,0.95l-0.19,0.08l-0.12,0.45l-0.14,0.2l-0.43-0.14l0.13,0.24
				l-0.03,0.77l-0.69,0.86l-0.31,0.17l-0.34,0.49l0.37,0.47l0.02,0.17l-0.12-0.02l-0.32,0.49l-0.73,0.62l-0.15-0.03
				c1.81,1.79,3.48,3.72,5.45,5.33c0.7-0.4,1.41-0.79,2.11-1.19l0.73-0.73l0.39-0.71l0.47-0.36l0.27-0.38l0.05,0.1l0.76-0.38
				l0.32,0.19l0.25-0.07l-0.12-0.7l-0.15-0.28l0.04-0.73l-0.07-0.14l0.07-0.48l0.18-0.25l0.02-0.27l0.29-0.38l0.22-0.22l0.26-1.22
				l-0.48-0.62L324.45,62.75z"></path>
		</g>
		<g id="trobajo_del_camino">
			<path class="st3 fase_0" d="M299.05,68.34l0.04,0.24l0.43,0.45l-0.04,0.5l0.07,0.42l-0.39,0.37l0.19,0.37l-0.01,0.36l0.09-0.02
				c0.49-0.36,1.39-0.15,2.01-0.15c0.21,0,0.44,0.68,0.67,0.54c0.3-0.19,0.58-1.24,0.75-1.5c0.37-0.58,0.62-1.32,1.16-1.64
				l-0.12-0.53l-0.09-0.01l-0.09-0.17l-0.43-0.12l-0.28-0.31l-0.2-0.06l-0.11-0.12l-0.78-0.24l-0.37-0.21l-0.84-0.12l-0.69-0.39
				l-0.36-0.11l-0.16-0.05v0l-0.41,0.31l0.1,1.02l-0.02,0.66L299.05,68.34z"></path>
		</g>
		<g id="eras_de_renueva">
			<path class="st3 fase_0" d="M314.68,59.91l-0.05-0.07l0.72-1.32l-0.7-0.68l-0.15-0.26l-0.55-0.24l-0.06-0.1l0.55-0.16l0.34-0.26
				l-0.37-0.07l-0.44-0.22l-0.32,0.13l-0.19,0.41l-0.3-0.64L313,56.28l-0.2-0.06l-0.05-0.28l-0.26-0.07l-0.12,0.1L312,55.82
				l-0.1,0.04l-0.17-0.21l0-0.2l-0.22-0.21l-0.67-0.15l-0.05,0.15l-0.24,0.24l-0.06,0.2l-0.32,0.11l-0.18,0.2l-0.8-0.07l-0.75,0.1
				l-0.44-0.19l-0.39,0.39L307.11,57l-0.58,0.08l-0.03,0.2l-0.2,0.21l-0.46,0.02l-0.94,0.32l-0.34,0.03l-0.37-0.45l-0.2-0.06
				c-0.08,1.01-0.32,2.01-0.62,2.99c-0.23,0.74-0.49,1.49-0.67,2.25c0.08,0.02,0.15,0.02,0.23,0.05c0.71,0.24,1.26,0.79,1.6,1.45
				c0.11,0.21,0.19,0.44,0.28,0.67l0-0.04l0.43-0.13l0.24,0.63l0.3,0.27l0.81,0.34l0.22-0.08l0.03,0.39l0.14,0.31l-0.03,0.2
				l0.33,0.24l0.16-0.08l0.19,0.08l0.2-0.56l0.22-0.26l-0.01-0.52l0.21-0.3l0.17-0.67l0.46-0.14l0.41-0.38l-0.02-0.27l-0.24-0.09
				l0.09-0.05l0.28-0.62l0.19-0.9l-0.07-0.55l0.03,0l0.2-1.3l0.15-0.1l0.1,0.03l0.18-0.15l0.53,0.19l0.52,0.35l0.92,0.31l0.17-0.03
				l-0.09,0.4l0.36,0.25l0.8-0.24l0.27,0.09l0.36,0.73l0.24-0.53l0.14-0.04l0.2-0.3l-0.12-0.6l0.01-0.27L314.68,59.91z"></path>
		</g>
		<g id="riano">
			<path id="path13" class="st3 fase_1" d="M333.58,32.43l-0.56,0.03l-0.21,0.06l-0.28,0.5l-0.52,0.14l-0.4,0l-0.39,0.3l0,0.26
				l-0.48,0.11l-0.29,0.43l0.03,0.11l-0.35,0.1l-0.71,0.81l-0.01,0.66l-0.19-0.17l-0.65-0.3l-0.14-0.17l-0.4,0.08l-0.41-0.06
				l-0.5,0.1l-0.04,0.17l-0.22-0.07l-0.16,0.07l-0.49-0.11l-0.09,0.34l0.08,0.14l-0.09,0.66l-0.18,0.12l-0.19,0.28l0.07,0.26
				l-0.37,0.28l0.18,0.48l-0.03,0.24l-0.33,0.74l0.15,0.44l-0.25,0.31l-0.14-0.02l-0.62,0.08l-0.82-0.12l-0.29-0.66l-0.23,0.22
				l-0.17,0.05l-0.1-0.06l-0.12,0.1l-0.19,0.37l-0.3,0.13l-0.53,0.01l-0.11,0.24l-0.31,0.14l-0.09,0.14l-0.23-0.12l-0.12,0.3
				l-0.42,0.17l-0.23-0.1l-0.11,0.07l-0.28-0.09l-0.17,0.09l-0.14,0.33l0.01,0.43l0.22,0.11l0.19,0.24l0.23,0.1l-0.21,0.73
				l0.27,0.17l0.14,0.21l0.01,0.17l0.3,0.1l0.58,0.49l0.86,0.18l-0.01,0.1l0.16,0.16l-0.23,0.24l-0.04,0.24l0.08,0.34l0.15,0
				l0.4-0.11l0.16-0.13l0.71-0.08l0.35,0.35l0.14-0.25l0.32-0.16l0.11-0.17l0.18-0.08l0.25,0.02l0.38,0.12l0.33-0.01l-0.02,0.29
				l-0.49,0.4l-0.02,0.16l0.06,0.35l0.23,0.2l0.03,0.11l-0.19,0.23l-0.07-0.03l-0.09,0.18l0.04,0.21l-0.08,0.19l0.09,0.1l0,0.23
				l0.19,0.04l0.2,0.44l0.32,0.09l0.63-0.09l0.64,0.12l0.06,0.7l0.22,0.43l-0.08,0.38l0.21-0.01l0.34,0.26l-0.06,0.06l0.16,0.04
				l0.7-0.06l0.35-0.45l0.14,0.13l0.45,0.12l0.57-0.13l0.3,0.04l0.27-0.07l0.24,0.06l0.15-0.03l0.45-0.1l0.02-0.08l0.43,0.17
				l-0.02,0.14l0.32,0.17l0.04,0.13l0.25,0.01l0.2,0.12l0.21,0.43l-0.31,0.15l-0.04,0.14l0.11,0.51l0.21,0.14l0.41-0.06l0.03,0.27
				l0.14,0.18l-0.1,0.24l0.04,0.26l-0.07,0.31l0.18,0.25l0.72-0.15l0.49,0.17l0.12,0.12l0.2-0.56L335,51.8l-0.04-0.3l0.16-0.43
				l0.38-0.31l0.31,0.12l0.09-0.21l-0.07-0.4l0.3-0.28l0.08-0.27l0.35-0.29l0.24-0.09l0.03-0.18l0.16-0.16l-0.07-0.31l0.23-0.31
				l0.1-0.41l-0.16-0.38l0.05-0.28l-0.08-0.32l0.47-0.33l0.03-0.69l0.27-0.32l0.22-0.05l0.07-0.24l0.57-0.19l0.4-0.75l0.73-0.16
				l0.07-0.14l-0.25-0.38l0.18-0.67l0.19-0.34l-0.4-0.5l-0.23,0.03l-0.13-0.07l-0.53,0.09l-0.02-1.02L338.45,41L338,40.75
				l-0.16-0.02l-0.34-0.21l-0.16,0l-0.27-0.22l-0.38-0.05l-0.02-0.18l-0.17-0.18l-0.46-0.18l-0.09-0.33L335.92,39l-0.38-0.28
				l-0.02-0.13l-0.23-0.11l0.35-0.4l-0.05-0.68l0.21-0.06l0.14-0.48l0.02-0.12l-0.24-0.22l0.07-0.24l-0.52-0.33l0.1-0.22l0.34-0.2
				l0.07-0.48l-0.55-0.35l-0.17-0.42l-0.2-0.08l0.06-0.15l-0.1-0.2l0.12-0.18l-0.02-0.15l-0.29-0.34l-0.26-0.27l-0.34-0.13
				L333.58,32.43L333.58,32.43z"></path>
		</g>
		<g id="la_palomera">
			<path class="st3 fase_0" d="M305.3,66.64l0.23,0.41l-0.02,0.01c0.09,0.1,0.18,0.19,0.27,0.29c0.12-0.12,0.25-0.24,0.37-0.35
				c0.15-0.13,0.29-0.27,0.44-0.4c0.11-0.1,0.23-0.19,0.34-0.29l-0.08-0.19l-0.03-0.39l-0.22,0.08l-0.81-0.34l-0.3-0.27l-0.24-0.63
				l-0.44,0.13l0.01,0.35l0,0l0.12,0.74l0.35,0.6L305.3,66.64z"></path>
		</g>
		<g id="crucero">
			<path class="st3 fase_0" d="M306.97,68.52c-0.19-0.17-0.39-0.34-0.57-0.51c-0.19-0.17-0.37-0.35-0.55-0.53
				c-0.13-0.13-0.25-0.27-0.37-0.41l-1.76,0.52l0.09,0.17l0.09,0.01l0.13,0.59l0.4,0.41l0.06-0.02l-0.03-0.15l0.3-0.14l0.26-0.07
				l0.12,0.2l0.21-0.03l0.02-0.14l0.06,0.03l0,0.13l0.19,0.11l-0.08,0.79l0.12,0.13l0.2-0.19l0.56-0.21l0.08-0.1l0.81-0.29
				C307.2,68.71,307.08,68.62,306.97,68.52z"></path>
		</g>
		<g id="jose_aguado_i">
			<path class="st3 fase_0" d="M307.63,67.64l0.01-0.77l-0.19-0.08l-0.16,0.08l-0.33-0.24l0.03-0.2l-0.14-0.31l-0.03-0.39l-0.22,0.08
				l-0.05-0.02c-0.06,0.05-0.12,0.1-0.18,0.15c-0.14,0.19-0.29,0.39-0.43,0.58c-0.14,0.18-0.27,0.37-0.41,0.55
				c0.15,0.17,0.31,0.34,0.47,0.5c0.18,0.18,0.36,0.36,0.55,0.53c0.19,0.17,0.38,0.34,0.58,0.51c0.07,0.06,0.14,0.12,0.21,0.18
				l0.3-0.11l-0.16-0.5L307.63,67.64z"></path>
		</g>
		<g id="condesa">
			<path class="st3 fase_0" d="M306.97,68.52c-0.19-0.17-0.39-0.34-0.57-0.51c-0.19-0.17-0.37-0.35-0.55-0.53
				c-0.13-0.13-0.25-0.27-0.37-0.41l-0.07,0.65l-0.03,0.68l0.06,0.03l0,0.13l0.19,0.11l-0.08,0.79l0.12,0.13l0.2-0.19l0.56-0.21
				l0.08-0.1l0.81-0.29C307.2,68.71,307.08,68.62,306.97,68.52z"></path>
		</g>
	</g>
	<g id="salamanca">
		<g id="pedrosillo_el_ralo">
			<path id="path948" class="st3 fase_0" d="M306.39,141.09l-0.25,0.04l-0.68,0.75l-0.25,0.8l-0.02,0.24l0.21,0.48l0.11,0.6l-0.22,0.25
				l-0.31,0.05l-0.24,0.26l-0.03,0.43l-0.2,0.13l-0.22-0.13l-0.11-0.06l-0.15,0.05l-0.01,0.25l-0.41,0.01l-0.78,0.19l0.08,0.77
				l0.2,0.3l0.02,0.24l0.03,0.11l0.12-0.05l-0.01,0.19l0.16-0.05l-0.01,0.18l0.09,0.25l0.03,0.26l0.32,0.01l0.05,0.23l0.11,0
				l-0.08,0.2l0.18,0.25l0.23,0.03l0.09-0.07l0.16,0.11l0.14-0.03l0.09,0.18l0.12,0l0.16,0.23l-0.07,0.14l-0.1,0.03l-0.13,0.53
				l-0.33,0.12l-0.08-0.12l-0.17-0.03l-0.02,0.09l-0.17-0.12l-0.08,0.12l-0.13-0.05l0.04-0.18l-0.25-0.03l-0.02,0.24l-0.07-0.04
				l-0.13,0.11l0.05,0.1l-0.08,0.16l0.03,0.14l0.16,0.07l0.03,0.17l0.35,0.02l0.19-0.13l0.24,0.03l0.2-0.11l0.08-0.04l0.07-0.07
				l0.15,0.11l-0.09,0.09l0.06,0.07l0.26-0.31l0.28,0.36l0.22,0.03l0.27-0.07l0.18,0.16l0.08,0.24l0.26,0.27l0.05,0.21l0.16,0.2
				l-0.08,0.16l-0.06,0.11l0.28,0.55l0.58,0.24l0.2,0.04l0.17-0.11l0.24-0.33l0.29-0.11l0.17,0.02l0.27-0.27l0.27-0.16l0.14,0
				l0.18-0.15l0.41,0.04l0.12-0.08l0.28,0.06l0.1-0.12l0.42-0.19l0.4,0.2l0.14,0.18l0.04,0.35l0.5-0.4l0.09-0.19l-0.03-0.48
				l0.13-0.37l0.29-0.3l0.05-0.22l-0.04-0.11l-0.42-0.31l0.04-0.06l-0.09-0.26l0.24-0.17l-0.13-0.18l0.12-0.01l0.07-0.13l-0.05-0.28
				l0.26-0.42l0.1-0.14l0.08,0.02l0.03-0.11l0.14-0.01l0.07,0.14l0.09-0.05l0.03-0.13l0.19-0.08l0-0.13l0.09-0.08l0.17-0.02
				l0.31-0.28l0.23-0.1l0.04,0.1l0.17-0.16l-0.1-0.27l0.1-0.06l-0.02-0.12l-0.15-0.1l0.06-0.13l-0.1-0.15l0.08-0.06l-0.1-0.12
				l0.01-0.22l1.08-0.35l0.17,0.4l0.2,0l0.28,0.54l0.33-0.06l0.09,0.09l0.06,0.09l-0.12,0.17l0.12,0.04l0.46-0.09l0.79-0.38
				l0.11-0.43l0.12-0.65l0.06-0.04l-0.1-0.16l0.06-0.09l-0.29-0.57l0.28-0.4l-0.02-0.17l0.17-0.24l-0.03-0.25l-0.33,0.04l-0.19-0.23
				l-0.23,0.12l-0.04-0.06l-0.75,0.37l-0.18-0.05L316,143l-0.07-0.15l-0.36-0.22l0.1-0.09l-0.17-0.22l-0.09-0.16l0.05-0.46
				l-0.06-0.12l-0.19,0.09l-0.55,0.35l-0.38,0.12l-0.84-0.36l-0.66,0.99l0.19,0.49l-0.05,0.04l-0.14-0.53l-0.43-0.11l-0.33,0.01
				l-2.04,0.49l0.07,0.14l-0.02,0.38l0.23,0.57l-0.14,0.24l-0.16,0.1l-0.11,0.09l-0.04,0.16l-0.14,0.07l-0.16-0.5l0.01-0.91
				l0.1-0.39l-0.46,0.08l-0.4-0.16l-0.27-0.26l-0.05-0.31l-0.14-0.21l-0.29-0.05l-0.72-0.51l-0.48-0.46l-0.31-0.08L306.39,141.09
				L306.39,141.09z"></path>
		</g>
		<g id="alba_de_tormes">
			<path id="path1025" class="st3 fase_0" d="M311.94,157.29l-0.17,0.02l-0.42,0.83l-0.51,0.57l-0.27-0.12l-0.42,0.2l-0.05-0.17
				l-0.36-0.24l-0.42,0.27l-0.08,0.23l-0.06,1.06l-0.13,0.25l-0.28,0.01l-0.17,0.1l-0.01-0.09l-0.66-0.28l-0.14,0.01l-0.19,0.32
				l-0.59,0.21l-0.27-0.19l0.11-0.41l0.2-0.33l-0.1,0.01l-0.09-0.14l-0.24,0.02l-0.01-0.11l-0.48-0.26l-0.01-0.16l-0.09,0.06
				l-0.18-0.14l-0.04,0.14l-0.11,0.05l-0.05-0.12l-0.17,0.04l-0.16-0.12l-0.13,0.04l-0.01,0.25l-0.16-0.05l-0.21,0.06l0-0.1
				l-0.31-0.11l-0.47,0.51l-0.06,0.22l-0.14,0.13l-0.1,0l0.04,0.28l0.06,0.45l-0.12,0.21l0.31,0.23l-0.12,0.03l0.09,0.22l-0.08,0.2
				l0.07,0.14l0.14-0.03l-0.11,0.2l0.06,0.18l0.39,0.38l-0.12,0.29l-0.15-0.01l-0.41,0.27l-0.04-0.16l-0.42-0.14l-0.05-0.16
				l-0.24-0.13l-0.28,0.29l-0.26-0.06l-0.14,0.15l-0.35,0.86l0.06,0.12l-0.05,0.16l0.19,0.17l0.12,0.04l0.01,0.14l0.31,0.13
				l0.04,0.23l-0.08,0.15l0.09,0.27l0.14,0.54l-0.02,0.24l-0.1,0.05l-0.02,0.22l-0.18-0.03l0.03,0.09l-0.17,0.12l-0.02,0.12
				l0.14,0.34l-0.02,0.2l0.14-0.01l0.05,0.12l-0.54-0.04l-0.09,0.17l-0.23,0.01l-0.45-0.13l-0.04-0.12l-0.13,0.13l-0.05,0.35
				l-0.09,0.02l0.04,0.14l-0.36-0.02l0.03,0.32l-0.16-0.03l-0.09-0.15l-0.38-0.24l-0.17-0.01l-0.09-0.11l-0.25-0.07l-0.58-0.04
				l-0.07,0.09l-0.18,0.01l-0.05,0.1l-0.08-0.01l-0.2,0.41l-0.14,0.09l0.1,1.04l-0.19,0.63l-0.03,0.07l0.31,0.35l0.48,0.26
				l0.47,0.49l0.13,0.35l0.83-0.14l0.42-0.16l0.15-0.39l0.21,0.39l0.3,0.12l0.14-0.04l0.12,0.35l0.17,0.05l0.26-0.27l0.5,0.25
				l0.26,0.13l0.11,0.18l0.17,0.28l0.15-0.27l0.27-0.01l0.18-0.26l0.05-0.65l0.15-0.2l0.05-0.26l0.1-0.06l0.17,0.15l0.36-0.13
				l0.11,0.08l-0.08-0.24l0.12-0.02l0.14-0.27l0.34,0l0.01,0.08l-0.23,0.22l-0.07,0.24l-0.01,0.65l0.17,0.36l-0.01,0.34l0.14,0.2
				l0.4,0.28l-0.04,0.35l-0.11-0.06l-0.18,0.05l-0.3,0.38l0.48,0.56l0.23-0.26l0.16,0.1l0.24-0.02l0.1,0.06l0,0.37l0.49,0.45
				l0.54-0.16l0.49-0.27l0.21-0.2l0.01,0.12l0.16,0.14l0.7-0.01l0.06,0.68l0.1,0.08l0.1,0.47l-0.03,0.45l0.48-0.09l0.09,0.09
				l0.76,0.06l0.1,0.04l0,0.11l0.68-0.11l0.06-0.33l0.13-0.06l0.08-0.16l0.09,0.06l0.21-0.11l0.17-0.26l0.21-0.09l0.06-0.42
				l0.34-0.24l0.09-0.03l0.14,0.08l0.26-0.03l0.21-0.25l0.18-0.25l0.24-0.05l0.09-0.12l0.06,0.07l0.15-0.35l-0.1-0.19l0.35-0.44
				l-0.21-0.36l0.51-0.18l-0.81-0.78l-0.05-0.32l0.06-0.12l0.2-0.01l0.12-0.34l0.37-0.17l-0.07-0.35l-0.21-0.45l0.1-0.11l-0.15-0.05
				l-0.09-0.16l-0.17-0.25l0.09-0.41l0.54-0.19l0.36-0.27l0.23-0.02l0.42,0.16l0.28-0.02l0.35-0.21l-0.08-0.15l0.05-0.27l-0.08-0.2
				l-0.19-0.14l-0.01-0.53l-0.11-0.3l-0.42-0.14l-0.16-0.26l0.25-0.25l0.68-1.09l0.02-0.31l-0.15-0.32l-0.22-0.11l-0.02-0.16
				l0.07-0.09l-0.34-0.28l-0.06-0.32l0.05-0.06l-0.33-0.23l-0.3-0.15l-0.23-0.36l-0.18,0.09l-0.15-0.08l-0.43-0.66l-0.14,0.04
				l-0.14-0.25l-0.17-0.02l0.02-0.51l0.2-0.21l0.08-0.11l0.08,0.03l0.07,0.06l0.24-0.01l0.29-0.21l0.04,0.09l0.19,0.05l0.15-0.11
				l0.23-0.15l0.19,0.04l0.33-0.41l0.06,0.07l0.32-0.21l0.06,0.09l0.15-0.09l0.1-0.2l-0.09-0.27l-0.12,0.04l-0.02-0.11l-0.15-0.1
				l0.03-0.21l-0.17,0.18l-0.13-0.17l-0.18-0.02l-0.15,0.08l-0.18-0.05l-0.02-0.08l-0.12,0.01l-0.02,0.08l-0.16-0.04l-0.09-0.03
				l0.05-0.13l-0.24,0.07l0-0.16l-0.13-0.03l-0.03-0.14l-0.16,0.15l-0.25-0.21l-0.44,0.32l-0.09-0.11l-0.09,0.04l-0.1-0.09l0-0.12
				l-0.37-0.39l-0.03-0.17l-0.18,0l-0.28-0.21l-0.06,0.13L311.94,157.29z"></path>
		</g>
		<g id="guijuelo">
			<path id="path971" class="st3 fase_0" d="M304.9,169.58l-0.14,0.27l-0.12,0.02l0.08,0.24l-0.11-0.08l-0.36,0.13l-0.17-0.15l-0.1,0.06
				l-0.05,0.26l-0.15,0.2l-0.05,0.65l-0.18,0.26l-0.27,0.01l-0.15,0.27l-0.17-0.28l-0.21,0.08l-0.21-0.17l0.05-0.22l-0.5-0.25
				l-0.26,0.27l-0.17-0.05l-0.12-0.35l-0.14,0.04l-0.3-0.12l-0.21-0.39l-0.15,0.39l-0.42,0.16l-0.83,0.14l0.12,0.33l0.26,0.2
				l0.05,0.17l-0.48,0.37l0.01,0.24l-0.9-0.38l-0.11,0.27l-0.71-0.16l-0.24,0.04l-0.32-0.19l-0.37,0.18l-0.15,0.06l0.02,0.1
				l-0.28,0.19l-0.05,0.14l0.56,0.57l0.13,0.63l0.17,0.14l0.18,0.33l0.66,0.22l0.07,0.15l-0.1,0.82l0.12,0.38l-0.45,0.02l-0.11,0.13
				l-1,0.04l-0.69,0.12l-0.32,0.5l0.17,0.11l-0.15,0.17l-0.37,0.13l-0.13-0.06l-0.39,0.25l-0.1,0.45l0.55,1.03l0.02,1.03l0.7-0.03
				l0.2-0.11l0.54,0.04l0.26,0.24l0.18,0.49l0.3,0.47l-0.29,0.8l-0.08,0.44l0.86,0.36l0.52-0.26l0.24-0.25l0.36-0.21l-0.02-0.18
				l0.32,0.01l0.22,0.2l0.08-0.04l0.27,0.22l0.17,0.27l0.29,0.18l0.41-0.21l0.07-0.18l0.23-0.11l0.15-0.19l0.33,0.15l0.36-0.08
				l0.62-0.23l0.31-0.2l0.15-0.25l0.24-0.02l0.24-0.14l0.64,0.63l0.53-0.12l0.81,0.04l0.6-0.49l0.57-0.04l-0.13,0.23l0.16,0.33
				l-0.04,0.29l-0.23,0.23l-0.16,0.04l-0.1,0.45l-0.09,0.19l0.46,0.21l0.38,0.36l0.4,0.58l0.2-0.07l0.51,0.1l0.55-0.07l0.1-0.23
				l0.06-0.56l-0.07-0.27l-0.34-0.16l-0.02-0.07l0.07-0.22l0.37-0.38l0.06-0.25l0.31,0.26l0.22-0.02l0.27,0.1l0.08-0.05l0.15,0.09
				l0.18-0.18l0.35-0.12l0.16-0.2l-0.09-0.06l0.29-0.37l-0.05-0.59l-0.01-0.33l0.39-0.16l0.08-0.18l0.13-1.21l0.19-0.34l0.24-0.16
				l0.13-0.26l0.14-0.95l0.17-0.25l-0.26-0.56l-0.24,0.04l-0.62,0.3l-0.29,0.07l-0.18-0.05l-0.67,0.19l-1.08,0.43l-0.38,0.17
				l-0.29,0.26l-0.07-0.04l0.02-0.38l-0.12-0.11l0.02-0.19l0.42-0.52l0.09-0.36l-0.06-0.15l0.03-0.21l0.02-0.43l0.25-0.27l0.13-0.25
				l0.22-0.2l-0.1-0.08l-0.06-0.68l-0.7,0.01l-0.16-0.14l-0.01-0.12l-0.21,0.2l-0.49,0.27l-0.54,0.16l-0.49-0.45l0-0.37l-0.1-0.06
				l-0.24,0.02l-0.16-0.1l-0.23,0.26l-0.48-0.56l0.3-0.38l0.18-0.05l0.11,0.06l0.04-0.35l-0.4-0.28l-0.14-0.2l0.01-0.34l-0.17-0.36
				l0.01-0.65l0.07-0.24l0.23-0.22l-0.01-0.08L304.9,169.58z"></path>
		</g>
		<g id="aldeadavila_ribera">
			<path id="path1059" class="st3 fase_0" d="M264.84,139.02l-0.13,0.1l-0.13,0.48l-0.22-0.02l-0.08-0.18l-0.15-0.07l-0.44,0.11
				l-0.41,0.27l-0.57-0.3l-0.15,0.04l-0.2,0.64l-0.48,0.26l-0.24,0.37l-0.4,0.28l-0.21,0.36l-0.26,0.19l0.14,0.21l-0.01,0.2
				l-0.17,0.38l-0.22,0.01l-0.03,0.07l0.1,0.16l-0.06,0.31l-0.19,0.22l-0.14,0.02l-0.11,0.29l-0.42,0.36l-0.37,0.18l-0.4,0.55
				l-0.03,0.16l-0.29,0.22l-0.11,0.18l-0.74,0.41l-0.02,0.09l-0.1,0.15l0,0.34l0.67,1.12l-0.1,0.24l-0.26,0.24l-0.05,0.2l-0.32,0.33
				l-0.08,0.28l0.06,0.14l-0.09,0.16l-0.19,0.17l-0.02,0.18l-0.28,0.21l-0.14,0.35l-0.76,0.69l0.08,0.26l-0.05,0.14l0.07,0.11
				l0.69,0.35l0.37,0.38l0.3-0.03l0.43-0.18l0.46,0.04l-0.03-0.17l0.07-0.04l0.12,0.16l0.67-0.25l0.21,0.11l0.67-0.41l0.06-0.14
				l0.67,0.49l0.3,0.15l0.23,0.09l0.15,0.17l0.22-0.03l0.14,0.25l0.34,0.04l0.25,0.34l0.13-0.1l0.06,0.17l0.45,0.52l0.16-0.09
				l0.16,0.3l0.18,0.11l0.34-0.02l0.23,0.03l0.05-0.11l0.23,0.03l0.13-0.16l0.16,0.07l0.29-0.1l0.46-0.02l0.55-0.94l-0.16-0.07
				l-0.04-0.11l-0.18-0.97l0.08-0.47l0.21-0.55l0.49-0.42l0.04-0.28l-0.09-0.16l0-0.43l-0.3,0.07l-0.2-0.08l-0.13-0.27l0-0.5
				l-0.1-0.53l-0.18-0.29l-0.31-0.08l-0.19-0.29l-0.26-0.17l0.34-0.83l-0.04-0.18l-0.3-0.13l0.14-0.11l0.02-0.3l-0.08-0.09l0.36-0.8
				l0.22-0.55l0.09-0.02l0.25-0.31l0.06-0.28l0.14-0.01l0.1-0.14l-0.09-0.3l0.04-0.26l-0.14-0.1l0.04-0.22l-0.17-0.18l0.07-0.19
				l-0.1-0.14l0.08-0.11l-0.15-0.08l-0.07-0.54l0.07-0.05l-0.04-0.12l0.15-0.34l-0.41-0.02l-0.15-0.13l-0.04-0.49l-0.07-0.15
				L264.84,139.02z"></path>
		</g>
		<g id="vitigudino">
			<path id="path1051" class="st3 fase_0" d="M269.36,136.93l-0.42,0.3l-0.06,0.23l0.52,0.4l0.01,0.11l-0.69,0.47l-0.21,0.06l-0.43,0.05
				l-0.17-0.49l-0.24-0.14l-0.35,0.5l-0.69,0.51l-0.16,0.62l-0.64,0.16l-0.15,0.2l-0.15,0.34l0.04,0.12l-0.07,0.05l0.07,0.54
				l0.15,0.08l-0.08,0.11l0.1,0.14l-0.07,0.19l0.17,0.18l-0.04,0.22l0.14,0.1l-0.04,0.26l0.09,0.3l-0.1,0.14l-0.14,0.01l-0.06,0.28
				l-0.25,0.31l-0.09,0.02l-0.22,0.55l-0.36,0.8l0.08,0.09l-0.02,0.3l-0.14,0.11l0.3,0.13l0.04,0.18l-0.34,0.83l0.26,0.17l0.19,0.29
				l0.31,0.08l0.18,0.29l0.1,0.53l0,0.5l0.13,0.27l0.2,0.08l0.3-0.07l0,0.43l0.09,0.16l-0.04,0.28l-0.49,0.42l-0.21,0.55l-0.08,0.47
				l0.18,0.97l0.04,0.11l0.16,0.07l-0.55,0.94l0.12-0.01l0.19,0.47l0.21,0.12l0.34,0.36l0.08,0.18l-0.03,0.13l0.24,0.11l0.21,0.36
				l-0.2,0.86l0.13,0.2l0.37,0.24l-1.21,1.62l-0.29,0.46l-0.13,0.37l0.16,0.01l0.42,0.48l0.63,0.42l-0.02,0.18l0.54,0.35l0.36,0.37
				l0.2,0.15l0.83,0.28l0.55,0.44l0.08,0.26l0.49,0.3l0.05,0.09l-0.1,0.11l-0.03,0.59l0.09,0.84l0.31-0.03l0.22-0.23l0.75-0.45
				l0.54-0.27l0.24-0.01l-0.1-0.12l-0.09-0.52l-0.18-0.12l-0.16-0.65l0.55,0.47l0.35,0.48l0.33,0.28l0.57,0.14l0.35-0.83l0.34-0.19
				l0.41-0.48l-0.33-0.27l-0.15-0.32l0-0.17l-0.36-0.68l0.45-0.33l0.17-0.47l0.42-0.5l0.11-0.25l-0.13-0.4l0.11-0.21l0.22-0.2
				l0.39-0.12l0.02-0.25l0.16-0.34l0.28-0.39l0.31-0.18l0.03,0.1l0.4,0.01l0.09,0.05l0,0.19l0.72,0.47l0.9,0.09l0.28,0.52l0.6-0.13
				l0.06,0.09l0.67,0.3l0.4,0.03l0.21-0.15l0.35-0.47l0.19-0.09l0.42-0.41l0.15,0.04l0.3-0.13l0.19-0.28l0.26-0.17l0.53-1.12
				l-0.3-0.14l-0.15-0.21l-0.39-0.11l-0.12-0.13l-0.1-0.16l-0.45-0.01l-0.01-0.51l-0.11-0.31l0.05-0.27l-0.14-0.35l0.02-0.26
				l0.18-0.38l-0.1-0.24l0.65-0.4l0.19-0.03l0.09-0.37l0.18-0.23l0.3-0.75l-0.33-0.44l0.66-0.89l-0.12-0.14l0.03-0.07l0.18-0.09
				l0.08-0.79l-0.34,0.06l0.01-0.36l0.17-0.51l0.22,0.09l0.22-0.12l0.08-0.43l0.1-0.13l-0.2-0.11l-0.2,0.05l-0.27-0.05l-0.85-0.41
				l-0.1,0.26l0.03,0.65l-0.58,0.26l-0.23,0.22l-1.46-0.44l-0.63-0.03l-0.74-0.95l-0.2-0.52l-0.19,0.18l-1.18-0.1l-0.21-0.31
				l-0.57-0.49l-0.66-0.09l-0.48,0.03l-0.01-0.44l-0.31-0.69l-0.02-0.5l0.74-0.01l0.56-0.43l0.53-0.28l-0.03-0.15l0.11-0.42
				l0.56-0.4l0.2-0.32l-0.04-0.28l0.2-0.46l-0.02-0.53l-0.42-0.14l-0.1-0.14l-0.41-0.19l-0.25-0.4l-0.3,0.14l-0.24-0.06l-0.16-0.12
				l-0.51-0.06l-0.05-0.15l-0.21-0.17l-0.4-0.16l-0.09-0.16l-0.25-0.09l-0.42,0.03l-0.35,0.37l-0.35,0.01l-0.59,0.15l0.03-0.27
				l-0.12-0.1l-0.19,0.01l-0.09,0.15l-0.62-0.23l-0.25,0l-0.36-0.44l-0.45,0.32l-0.49-0.07l-0.06-0.04L269.36,136.93z"></path>
		</g>
		<g id="lumbrales">
			<path id="path1207" class="st3 fase_1" d="M251.73,150.66l-0.61,0.48l-0.05,0.67l0.07,0.16l0.22,0.18l0.11,0.44l0.5,0.21l0.22,0.54
				l0.48,0.42l0.2,0.52l0.43,0.57l0.3,0.16l0.35,0.47l0.1,0.34l0.18,0.4l-0.05,0.23l0.24,0.36l-0.02,1.2l0.46,0.7l0.33,0.24
				l0.28,0.02l0.13,0.19l0.41-0.1l0.1,0.33l0.11,0.4l-0.02,0.23l0.12,0.23l0.1,0.23l0.51,0.26l0.5,0.5l0.43,0.02l0.23,0.11
				l0.29,0.09l0.14,0.61l0.24,0.18l0.15,0.02l0.14,0.38l-0.03,0.34l0.21,0.36l0.34,0.08l0,0.18l0.15,0.1l-0.02,0.2l0.24,0.39
				l0.56,0.39l0.57,0.06l0.33-0.14l0.16-0.23l0.14-0.05l0.07,0.23l0.11-0.07l0.26,0.07l0.09,0.34l0.37,0.13l0.1,0.32l0.53,0.13
				l0.18-0.09l0.2-0.4l0.19,0.05l0.6,0.93l0.17,0.1l0.04,0.45l0.15,0.27l-0.07,0.29l0.29,0.25l0.2-0.12l0.16,0.03l0.24-0.31
				l-0.53-0.32l-0.2-0.21l-0.21-0.56l-0.64-0.65l0.04-0.48l-0.07-0.05l-0.35,0.05l-0.43-0.15l-0.03-0.26l0.27-0.32l0.31-0.06
				l0.05-0.21l0.19-0.25l0.04,0.03l0.67,0.81l0.17,0.08l0.54,0.08l0.61-0.1l0.26,0.1l0.25-0.59l-0.25-0.06l-0.15-0.23l0.38,0.16
				l0.13-0.03l0.07-0.21l-0.11-0.2l0.11-0.11l0.07-0.31l-0.41-0.3l-0.08-0.22l0.3,0.11l0.2-0.19l0.39,0.35l0.25,0.09l0.3-0.05
				l0.1,0.46l1.04,0.99l0.79,0.22l0.16-1.14l-0.09-0.84l0.03-0.59l0.1-0.11l-0.05-0.09l-0.49-0.3l-0.08-0.26l-0.55-0.44l-0.83-0.28
				l-0.2-0.15l-0.36-0.37l-0.54-0.35l0.02-0.18l-0.63-0.42l-0.42-0.48l-0.16-0.01l0.13-0.37l0.29-0.46l1.21-1.62l-0.37-0.24
				l-0.13-0.2l0.2-0.86l-0.21-0.36l-0.24-0.11l0.03-0.13l-0.08-0.18l-0.34-0.36l-0.21-0.12l-0.19-0.47l-0.12,0.01l-0.46,0.02
				l-0.29,0.1l-0.16-0.07l-0.13,0.16l-0.23-0.03l-0.05,0.11l-0.23-0.03l-0.34,0.02l-0.18-0.11l-0.16-0.3l-0.16,0.09l-0.45-0.52
				l-0.06-0.17l-0.13,0.1l-0.25-0.34l-0.34-0.04l-0.14-0.25l-0.22,0.03l-0.15-0.17l-0.23-0.09l-0.3-0.15l-0.67-0.49l-0.06,0.14
				l-0.67,0.41l-0.21-0.11l-0.67,0.25l-0.12-0.16l-0.07,0.04l0.03,0.17l-0.46-0.04l-0.43,0.18l-0.3,0.03l-0.37-0.38l-0.69-0.35
				l-0.07-0.11l-1.03,0.42l-0.78,0.09l-0.54,0.01l-0.52-0.09L251.73,150.66z M267.14,163.1l-0.38,0.74l0.32,0.31l0.22-0.1l0.15-0.3
				l-0.07-0.49L267.14,163.1z"></path>
		</g>
		<g id="ciudad_rodrigo">
			<path id="path979" class="st3 fase_0" d="M266.73,162.22l-0.2,0.19l-0.3-0.11l0.08,0.22l0.41,0.3l-0.07,0.31l-0.11,0.11l0.11,0.2
				l-0.07,0.21l-0.13,0.03l-0.38-0.16l0.15,0.24l0.24,0.05l-0.25,0.6l-0.26-0.1l-0.61,0.1l-0.54-0.08l-0.17-0.08l-0.7-0.85
				l-0.19,0.25l-0.05,0.21l-0.31,0.06l-0.27,0.32l0.03,0.26l0.43,0.15l0.35-0.05l0.07,0.05l-0.04,0.48l0.64,0.65l0.21,0.56l0.2,0.21
				l0.53,0.32l-0.24,0.31l-0.16-0.03l-0.2,0.12l-0.29-0.25l0.07-0.29l-0.15-0.27l-0.04-0.45l-0.17-0.1l-0.6-0.93l-0.19-0.05
				l-0.2,0.4l-0.18,0.09l-0.53-0.13l-0.1-0.32l-0.37-0.13l-0.09-0.34l-0.26-0.07l-0.11,0.07l0.04,1.22l-0.1,0.4l0.16,0.1l0.07,0.48
				l-0.08,0.43l-0.16,0.22l0.13,0.23l0.32,0.22l0.02,0.16l-0.14,0.36l0.01,0.74l0.14,0.3l-0.04,0.29l-0.41,0.73l0.02,0.67l0.43,0.7
				l0.15,0.09l0.39,0.03l0.04,0.11l-0.03,0.17l-0.37,0.66l0.06,0.07l0.66,0.01l0.1,0.26l-0.03,0.16l-0.21,0.24l-0.13-0.1l-0.1,0.11
				l-0.08,0.31l-0.5-0.19l-0.13,0.09l-0.42-0.1l0.04,0.42l-0.25-0.02l-0.73,0.19l0.01,0.37l-0.15,0.34l-0.04,0.54l0.18,0.18
				l0.32,0.11l0.23,0.18l0.04,0.14l-0.11,0.31l-0.24,0.36l0.19,0.16l0.58-0.24l0.23,0.13l0.14-0.15l0.22,0.02l0.14,0.2l0.4,0.15
				l0.11,0.14l0.14,0l0.04,0.38l0.11,0.12l-0.09,0.18l0.05,0.32l0.18,0.08l0.18,0.37l-0.06,0.55l-0.13,0.26l-0.28,0.18l0.15,0.28
				l-0.09,0.04l0.03,0.09l0.39,0.38l0.38,0.01l0.45,0.46l-0.47,0.68l0.36,0.25l0.16,0l0.34,0.26l0.11,0.45l0.2,0.25l0.15-0.11
				l0.4-0.16l0.05-0.21l0.23-0.1l0.14-0.19l0.09,0.03l0.29-0.37l0.66-0.54l0.26-0.04l0.16-0.31l0.29-0.05l0.18-0.12l0.12-0.33
				l0.59-0.3l0.1-0.23l0.52-0.3l0.09,0.12l0.17-0.02l0.22,0.79l-0.01,0.23l0.09,0.02l-0.06,0.08l0.07,0.14l-0.08,0.06l0.02,0.08
				l0.13,0.06l-0.25,0.11l-0.11,0.17l0.03,0.08l0.26-0.11l0.07,0.12l0.08-0.01l0.16,0.37l0.08,0.04l0.11-0.1l0.15,0.15l0.25-0.35
				l0.45-0.2l0.31-0.32l0.13,0.16l0.11,0.46l0.77,0.81l0.45-0.15l-0.03,0.36l0.29,1.23l0.43,0.06l0.44-0.18l0.79-0.79l0.05-0.31
				l0.4-0.25l0.38-0.01l0.12-0.15l1.13-0.08l0.27-0.3l0.51-0.05l0.17-0.13l0.1-0.36l0.15-0.18l0.4,0.05l0.61-0.85l0.35-0.04
				l0.3,0.12l0.12-0.03l0.36,0.12l0.43-0.01l0.11,0.08l0.41-0.29l0.18-0.59l-0.05-0.17l0.25-0.15l0.12-0.29l0.23-0.21l0.04-0.18
				l-0.34-0.49l-0.07,0.04l-0.33,0.1l-0.24-0.05l-0.14-0.13l-0.62-0.15l-0.31,0.03l-0.2,0.49l-0.22,0.01l-0.57-0.24l0.07-0.15
				l-0.08-0.62l-0.25-0.63l-0.69-0.3l-0.33-0.26l-0.15,0.05l-0.17-0.08l-0.12,0.11l-0.3,0.05l-0.32,0.27l-0.1,0.04l-0.23-0.11
				l-0.2,0.01l0.17-0.24l0.01-0.26l-0.45-0.13l0.19-0.78l-0.14-0.23l0.07-0.26l-0.39-0.54l-0.19-0.16l0.25-0.3l0.16-0.46l-0.4-0.43
				l-0.18-0.37l0-0.18l-0.14-0.28l-0.13-0.09l-0.05-0.39l-0.1-0.05l-0.5,0l-0.11-0.17l0.05-0.32l-0.15-0.07l-0.02-0.45l-0.28-0.74
				l-0.23-0.27l0.59,0.25l0.31,0.33l0.26-0.38l0.08,0.02l0.07-0.1l0-0.19l-0.27-0.31l-0.09-0.28l0.2-1.5l0.26-0.31l-0.36-0.48
				l-0.04-0.22l0.19-0.29l-0.03-0.09l-0.27-0.19l-0.16-0.16l-0.04-0.16l-0.15-0.09l-0.11,0.01l-0.22,0.29l-0.16,0.01l-0.27-0.12
				l-0.09-0.23l0.36-0.8l-0.13,0l-0.16-0.15l-0.7,0.02l-0.1-0.09l-0.23-0.53l-0.17,0.14l-1.35,0.6l-0.43,0.31l-0.5-0.38l-0.11-0.23
				l-0.79-0.22l-1.04-0.99l-0.1-0.46l-0.3,0.05l-0.25-0.09L266.73,162.22z M267.14,163.1l0.23,0.15l0.07,0.49l-0.15,0.3l-0.22,0.1
				l-0.32-0.31L267.14,163.1z M267.48,163.91l0.18,0.03l0.2,0.17l0.14,0.42l0.43,0.4l-0.18,0.49l-0.29-0.05l-0.31-0.39l-0.23-0.14
				L267.48,163.91z"></path>
		</g>
		<g id="la_alberca">
			<path id="path981" class="st3 fase_0" d="M283.74,172.4l-0.15,0.44l-0.41,0.35l-0.3,0.04l-0.5-0.13l-0.13,0.97l0.19,0.38l-0.02,0.23
				l0.21,0.04l0.03,0.17l-0.17-0.06l-0.09,0.06l-0.42-0.05l0.03-0.19l-0.11-0.33l-0.15-0.01l-0.34,0.18l-0.33,0l-0.73,0.46
				l-0.34,0.08l-0.38,0.04l-0.27-0.2l-0.17,0.18l-0.11,0.59l-0.21,0.37l-0.13,0.03l-0.35,0.37l-0.45,0.24l0.69,0.3l0.25,0.63
				l0.08,0.62l-0.07,0.15l0.57,0.24l0.22-0.01l0.2-0.49l0.31-0.03l0.62,0.15l0.14,0.13l0.24,0.05l0.33-0.1l0.07-0.04l0.34,0.49
				l-0.04,0.18l-0.23,0.21l-0.12,0.29l-0.25,0.15l0.05,0.17l-0.18,0.59l-0.41,0.29l0.3,0.5l0.14,0.04l0.04,0.27l0.25,0.22l0.45,0.22
				l0.47,0.06l0.21,0.24l-0.14,0.25l0.36,0.63l0.6-0.04l0.49-0.31l0.16,0.06l0.25-0.46l0.45-0.41l0.39,0.02l0.33-0.14l0.4-0.05
				l0.02-0.17l-0.03-0.18l0.45-0.22l0.64-0.45l0.03-0.22l-0.1-0.16l0.53-0.23l0.08-0.15l0.38,0.05l0.12-0.23l0.07-0.5l-0.08-0.16
				l0.07-0.19l0.24-0.03l0-0.11l-0.27-0.36l-0.42-0.02l-0.15-0.19l0.26-0.35l0.45-0.04l0.13-0.17l-0.26-0.07l-0.32-0.23l-0.08-0.26
				l-0.25-0.24l-0.28-0.15l-0.12,0.08l-0.33-0.42l-0.2-0.11l-0.77,1.05l-0.09-0.07l-0.41,0.09l-0.71-0.61l-0.31-1l0.03-0.42
				l-0.41-0.42l0.26-0.17l-0.4-1.03l0.31-0.06l0.1-0.28l0.09,0.04l0.11-0.18l0.06-0.28l-0.18-0.4L283.74,172.4z M282.68,174.97
				l0.18,0.23l-0.12-0.02L282.68,174.97z"></path>
		</g>
		<g id="linares_de_riofrio">
			<path id="path965" class="st3 fase_0" d="M298.89,165.87l-0.17,0.13l-0.13-0.08l-0.1,0.21l-0.33,0.05l-0.09,0.23l-0.26,0.23
				l-0.34-0.04l-0.11,0.1l-0.25,0.09l-0.14-0.06l-0.11,0.08l-0.37,0.63l-0.25,0.11l-0.03,0.19l0.7,0.36l-0.15,0.22l-0.18-0.01
				l-0.15,0.09l-0.22-0.06l-0.01,0.15l-0.21,0.12l-0.04,0.23l-0.16-0.03l0.03,0.5l0.14,0.07l0.05,0.25l-0.03,0.14l-0.11,0.08
				l-0.09,0.35l-0.15-0.13l-0.23-0.05l-0.12-0.24l-0.18-0.14l-0.55-0.03l-0.23-0.11l0.01,0.25l0.19,0.35l-0.08,0.29l-0.19,0.17
				l0.06,0.26l-0.85,0.25l-0.11,0.36l0.08,0.39l-0.12,0.19l0,0.4l-0.24,0.25l0.02,0.31l0.17,0.28l-0.18,0.05l-0.11,0.17l-0.18-0.23
				l-0.44-0.25l-0.13-0.02l-0.31,0.22l-0.49-0.51l-0.05-0.29l-0.16-0.18l-0.39-0.08l0.03-0.26l-0.28,0.03l0.01,0.14l-0.22,0.43
				l-0.61,0.49l-0.26,0.44l-0.09,0.28l-0.01,0.86l0.37,0.64l-1.34-0.19l0.13,1.18l-0.09,0.25l0.11,0.24l0.2,0.13l-0.09,0.08
				l0.07,0.16l0.25,0.15l0.12,0.21l-0.03,0.34l0.28,0.28l0.21,0.06l-0.21,0.44l0.23-0.01l0.26,0.22l0.53,0.26l0.66,0.19l-0.21,0.3
				l0.3,0.15l-0.22,0.32l0.16,0.16l0.24,0.38l0.03,0.28l0.4,0.55l0.88-0.52l0.33-0.29l1.16-0.65l0.29,0.03l0.79-0.22l-0.02-1.03
				l-0.55-1.03l0.1-0.45l0.39-0.25l0.13,0.06l0.37-0.13l0.15-0.17l-0.17-0.11l0.32-0.5l0.69-0.12l1-0.04l0.11-0.13l0.45-0.02
				l-0.12-0.38l0.1-0.82l-0.07-0.15l-0.66-0.22l-0.18-0.33l-0.17-0.14l-0.13-0.63l-0.56-0.57l0.05-0.14l0.28-0.19l-0.02-0.1
				l0.15-0.06l0.37-0.18l0.32,0.19l0.24-0.04l0.71,0.16l0.11-0.27l0.9,0.38l-0.01-0.24l0.48-0.37l-0.05-0.17l-0.26-0.2l-0.12-0.33
				l-0.13-0.35l-0.47-0.49l-0.48-0.26l-0.31-0.35l0.03-0.07l0.19-0.63l-0.1-1.04l0.14-0.09l0.2-0.41l0.08,0.01l0.05-0.1l0.18-0.01
				l0.07-0.09l0.58,0.04l-0.13-0.66l-0.15-0.16l0.09-0.22l-0.18-0.1L298.89,165.87z"></path>
		</g>
		<g id="tamames">
			<path id="path967" class="st3 fase_0" d="M293.4,165.92l-0.38,0.66l-0.09,0.68l0.15,0.26l0.43-0.07l0.22,0.11l0.09-0.17l0.33-0.06
				l0.07-0.13l-0.04-1.02l-0.26,0.04l-0.24-0.23L293.4,165.92z M291.8,166.06l-0.24,0.2l-0.02,0.21l-0.33,0.39l-0.3-0.03l-0.27-0.16
				l-0.25,0.33l-0.26-0.1l-0.78,0.09l-0.13-0.1l-0.23,0.17l-0.16-0.16l-0.71-0.06l-0.82,0.58l-0.1,0.31l-0.09-0.04l-0.3,0.59
				l-0.37,0.27l-0.17,0.01l-0.02,0.51l-0.08,0.11l-0.23,0.03l-0.34,0.42l-1.07-0.26l0.34-0.27l0-0.4l-0.23-0.03l-0.35-0.25
				l-0.09-0.31l-0.12-0.14l-0.32-0.01l-0.02-0.55l-0.51-0.31l0.06-0.19l-0.11-0.07l-0.21-0.48l-0.2,0.18l0.09,0.36l-0.08,0.07
				l-0.05,0.49l-0.57,0.62l-0.51-0.56l-0.81-0.23l-0.08,0.01l-0.06,0.15l-0.07-0.05l-0.14,0.12l-0.08,0.21l-0.46-0.21l-0.18,0.07
				l-0.25-0.03l-0.29,0.44l-0.27,0.01l-0.62,0.32l-0.11,0.13l-0.46,0.01l-0.19,0.43l-0.2,0.27l-0.53-0.05l-0.41,0.14l-0.55,0.64
				l-0.39-0.06l-0.59-0.26l-0.02-0.12l-0.16,0.06l0,0.19l-0.07,0.1l-0.08-0.02l-0.26,0.38l-0.31-0.33l-0.59-0.25l0.23,0.27
				l0.28,0.74l0.02,0.45l0.15,0.07l-0.05,0.32l0.11,0.17l0.5,0l0.1,0.05l0.05,0.39l0.13,0.09l0.14,0.28l0,0.18l0.18,0.37l0.4,0.43
				l-0.16,0.46l-0.25,0.3l0.19,0.16l0.39,0.54l-0.07,0.26l0.14,0.23l-0.19,0.78l0.45,0.13l-0.01,0.26l-0.17,0.24l0.2-0.01l0.23,0.11
				l0.1-0.04l0.32-0.27l0.3-0.05l0.12-0.11l0.17,0.08l0.15-0.05l0.33,0.26l0.45-0.24l0.35-0.37l0.13-0.03l0.21-0.37l0.11-0.59
				l0.17-0.18l0.27,0.2l0.38-0.04l0.34-0.08l0.73-0.46l0.33,0l0.34-0.18l0.15,0.01l0.11,0.33l-0.03,0.19l0.42,0.05l0.09-0.06
				l0.17,0.06l-0.03-0.17l-0.21-0.04l0.02-0.23l-0.19-0.38l0.13-0.97l0.5,0.13l0.3-0.04l0.41-0.35l0.15-0.44l0.42,0.13l0.18,0.4
				l-0.06,0.28l-0.11,0.18l-0.09-0.04l-0.1,0.28l-0.31,0.06l0.4,1.03l-0.26,0.17l0.41,0.42l-0.03,0.42l0.31,1l0.71,0.61l0.41-0.09
				l0.09,0.07l0.77-1.05l-1.44-1.05l-0.2-0.34l0.25-0.06l0.45,0.27l0.37,0.11l0.43,0.01l0.24,0.16l0.41,0.12l0.73,0.87l0.92,0.53
				l0.09-0.08l-0.2-0.13l-0.11-0.24l0.09-0.25l-0.13-1.18l1.34,0.19l-0.37-0.64l0.01-0.86l0.09-0.28l0.26-0.44l0.61-0.49l0.22-0.43
				l-0.01-0.14l0.28-0.03l-0.03,0.26l0.39,0.08l0.16,0.18l0.05,0.29l0.49,0.51l0.31-0.22l0.13,0.02l0.44,0.25l0.18,0.23l0.11-0.17
				l0.18-0.05l-0.17-0.28l-0.02-0.31l0.24-0.25l0-0.4l0.12-0.19l-0.08-0.39l0.11-0.36l0.85-0.25l-0.06-0.26l0.19-0.17l0.08-0.29
				l-0.19-0.35l-0.01-0.25l0.23,0.11l0.55,0.03l0.18,0.14l0.12,0.24l0.23,0.05l0.15,0.13l0.09-0.35l0.11-0.08l0.03-0.14l-0.05-0.25
				l-0.14-0.07l-0.03-0.5l-0.47,0l-0.43-0.13l-0.29-0.46l0.12-0.2l-0.07-0.11l-0.41,0.24l-0.1-0.1l-0.41,0.05l-0.16,0.15l-0.45-0.2
				l-0.15-0.33l-0.24-0.17l0.05-0.09l-0.17-0.17l-0.27-0.73L291.8,166.06z"></path>
		</g>
		<g id="miranda_del_castanar">
			<path id="path1599" class="st3 fase_1" d="M285.11,174.81l-0.25,0.06l0.2,0.34l1.44,1.05l0.2,0.11l0.33,0.42l0.12-0.08l0.28,0.15
				l0.25,0.24l0.08,0.26l0.32,0.23l0.26,0.07l-0.13,0.17l-0.45,0.04l-0.26,0.35l0.15,0.19l0.42,0.02l0.27,0.36l0,0.11l-0.24,0.03
				l-0.07,0.19l0.08,0.16l-0.07,0.5l-0.12,0.23l-0.38-0.05l-0.08,0.15l-0.53,0.23l0.1,0.16l-0.03,0.22l-0.64,0.45l-0.45,0.22
				l0.03,0.18l-0.02,0.17l-0.4,0.05l-0.33,0.14l-0.39-0.02l-0.45,0.41l-0.25,0.46l-0.04,0.34l-0.09,0.01l-0.24,0.31l-0.18,0.46
				l0.43,0.08l0.04,0.18l0.44,0.1l0.12-0.06l0.27,0.1l0.19,0.1l0.04,0.17l0.17-0.04l0.11,0.09l0.09,0.22l0.32-0.02l0.05,0.14
				l0.23,0.08l0.13,0.15l-0.08,0.08l-0.12-0.07l-0.16,0.03l-0.08,0.26l-0.3-0.07l-0.07,0.13l0.06,0.08l0.46,0.04l-0.09,0.29
				l-0.38,0.12l-0.08,0.1l0,0.16l0.21,0.11l-0.05,0.53l-0.14,0.13l-0.27-0.05l-0.18,0.28l-0.21-0.1l-0.14,0.09l1.54,0.5l0.63,0.32
				l0.26-0.08l0.37,0.04l0.36,0.14l0.47-0.04l0.29-0.45l0.08-0.48l0.04-0.17l-0.09-0.16l0.1-0.16l-0.03-0.31l-0.17-0.14l0.01-0.28
				l-0.22-0.18l-0.04-0.23l-0.21-0.13l0.01-0.18l-0.15-0.2l-0.05-0.3l0.11-0.28l0.22-0.03l0.1,0.07l0.82-0.46l-0.41-0.79l0.45-0.57
				l0.11-0.49l0.53-0.16l0.52-0.54l-0.1-0.16l0.05-0.46l0.02-0.25l0.11,0.05l0.56-0.18l-0.01-0.18l0.09-0.13l-0.1-0.25l0.27,0.08
				l-0.16-0.16l0.22-0.32l-0.3-0.15l0.21-0.3l-0.66-0.19l-0.53-0.26l-0.26-0.22l-0.23,0.01l0.21-0.44l-0.21-0.06l-0.28-0.28
				l0.03-0.34l-0.12-0.21l-0.25-0.15l-0.07-0.16l-0.92-0.53l-0.73-0.87l-0.41-0.12l-0.24-0.16l-0.43-0.01l-0.37-0.11L285.11,174.81z
				"></path>
		</g>
		<g id="cantalapiedra">
			<path id="path1111" class="st3 fase_0" d="M319.03,142.02l-0.11,0.12l-0.35,0.02l-0.18-0.11l-0.13,0.07l0.05,0.13l-0.14-0.09
				l-0.36,0.06l-0.23,0.03l-0.14,0.45l0.06,0.04l-0.05,0.14l0.12,0.1l0.03,0.25l-0.17,0.24l0.02,0.17l-0.28,0.4l0.29,0.57
				l-0.06,0.09l0.1,0.16l-0.06,0.04l-0.12,0.65l-0.11,0.43l0.12-0.05l0.31,0.3l0.25,0.06l-0.16,0.35l-0.05,0.11l0.28,0.63l0.09,0.31
				l-0.11,0.06l0.28,0.07l0.08,0.21l-0.16,0.07l-0.07,0.15l0.18-0.06l0.09,0.12l-0.16,0.09l0.15,0.1l-0.07,0.19l0.09,0.06
				l-0.06,0.08l-0.14,0.24l-0.11,0l-0.01,0.11l-0.42,0.05l0.03,0.24l-0.25,0.16l0.07,0.16l-0.14,0.17l-0.08,0.31l0.11,0.04
				l-0.05,0.15l-0.09,0.01l0.05,0.15l0.25,0.07l0.09-0.22l0.1,0.02l-0.19,0.47l0.07,0.18l0.18,0.05l-0.03,0.2l0.44,0.51l0.14-0.07
				l0.18,0.3l0.06,0.44l0.49,0.11l0.34,0.34l0.26,0.12l0.28-0.09l0.15-0.22l0.34-0.16l0.25,0.07l0.1,0.52l0.13-0.02l0.29,0.13
				l0.37,0.03l0.07-0.05l0.01-0.4l0.33-0.04l0.18-0.19l0.05-0.19l0.42-0.21l-0.06-0.11l0.16-0.24l0.34-0.15l0.22-0.32l0.01-0.44
				l0.27-0.33l-0.02-0.18l-0.21-0.17l0.05-0.16l-0.12-0.07l0.06-0.33l-0.09-0.06l0-0.22l-0.15-0.24l0.11-0.84l-0.08-0.14l0.09-0.1
				l-0.05-0.08l0.11-0.37l0.21-0.27l0.12,0.02l0.04-0.16l0.42-0.3l0.17-1.43l-0.14-0.09l-0.16-0.13l-0.17-0.27l0.01-0.15l-0.36-0.19
				l-0.15-0.19l-0.52-0.29l-0.18-0.19l-0.34-0.06l0-0.11l-0.14,0.01l-0.03-0.08l0.08-0.04l-0.12-0.11l-0.08,0.07l-0.14-0.04
				l-0.25,0.25l-0.21-0.06l-0.17-0.18l-0.27,0.08l-0.03-0.1l-0.42,0.39l-0.17-0.14l0.21-0.29l-0.2-0.16l-0.09-0.32l0.04-0.09
				l-0.08-0.01l0.03-0.16l-0.14-0.1l0.03-0.18l-0.07-0.17l-0.39-0.22l-0.15-0.17l-0.25,0.05l0.03-0.11L319.03,142.02z"></path>
		</g>
		<g id="penaranda">
			<path id="path991" class="st3 fase_0" d="M317.2,150.47l-0.26,0.14l0,0.13l-0.15-0.05l-0.21,0.12l0.11,0.09l-0.05,0.11l-0.48,0.22
				l-0.18,0.41l-0.34,0.26l0.12,0.08l0.13,0.65l0.16,0.23l0.2,0.05l-0.4,0.5l-0.02,0.27l0.1,0.31l-0.13,0.1l0.16,0.2l-0.6,0.34
				l-0.09,0.34l-0.09,0.07l-0.82,0.08l0.06,0.15l-0.24,0.22l0.08,0.08l-0.36,0.43l-0.02,0.19l-1.03,0.81l-0.5,0.02l-0.12,0.21
				l-0.08,0.07l0.28,0.21l0.18,0l0.03,0.17l0.37,0.39l0,0.12l0.1,0.09l0.09-0.04l0.09,0.11l0.44-0.32l0.25,0.21l0.16-0.15l0.03,0.14
				l0.13,0.03l0,0.16l0.24-0.07l-0.05,0.13l0.09,0.03l0.16,0.04l0.02-0.08l0.12-0.01l0.02,0.08l0.18,0.05l0.15-0.08l0.18,0.02
				l0.13,0.17l0.17-0.18l-0.03,0.21l0.15,0.1l0.02,0.11l0.12-0.04l0.09,0.27l-0.1,0.2l-0.15,0.09l-0.06-0.09l-0.32,0.21l-0.06-0.07
				l-0.33,0.41l-0.19-0.04l-0.23,0.15l-0.15,0.11l-0.19-0.05l-0.04-0.09l-0.29,0.21l-0.24,0.01l-0.07-0.06l-0.08-0.03l-0.08,0.11
				l-0.2,0.21l-0.02,0.51l0.17,0.02l0.14,0.25l0.14-0.04l0.43,0.66l0.15,0.08l0.18-0.09l0.23,0.36l0.3,0.15l0.33,0.23l-0.05,0.06
				l0.06,0.32l0.34,0.28l-0.07,0.09l0.02,0.16l0.22,0.11l0.15,0.32l-0.02,0.31l-0.68,1.09l-0.25,0.25l0.16,0.26l0.42,0.14l0.11,0.3
				l0.01,0.53l0.19,0.14l0.08,0.2l-0.05,0.27l0.08,0.15l-0.35,0.21l-0.28,0.02l-0.42-0.16l-0.23,0.02l-0.36,0.27l-0.54,0.19
				l-0.09,0.41l0.17,0.25l0.09,0.16l0.15,0.05l-0.1,0.11l0.21,0.45l0.07,0.35l0.15,0.08l0.63-0.41l0.3,0.01l0.18-0.13l0.21-0.29
				l-0.04-0.16l-0.17-0.1l0.28-0.1l0.06-0.22l0.27,0l0.01,0.13l0.35-0.03l0.22-0.12l0.6-0.24l0.13-0.22l0.42-0.32l0.09-0.19
				l0.46-0.25l0.25-0.22l0.09-0.2l0.14,0.11l-0.05-0.28l0.21-0.18l0.72-0.31l0.19-0.25l0.53-0.25l-0.1-0.69l-0.08-0.12l0.42-0.33
				l0.98-0.51l0.25-0.41l-0.09-0.06l0.12-0.42l0.44-0.32l0.03-0.13l0.18-0.03l-0.01-0.21l0.12-0.13l0.05-0.2l-0.18-0.21l0.03-0.08
				l-0.11-0.08l0-0.18l-0.1,0.01l-0.07-0.12l-0.03-0.19l0.14-0.17l0.1-0.59l-0.04-0.15l0.11-0.29l-0.07-0.17l-0.02-0.16l0.27-0.09
				l0.19-0.21l0.02-0.12l0.12-0.06l-0.02-0.23l0.38-0.32l0.13-0.19l-0.11-0.05l0.03-0.09l0.2-0.09l-0.04-0.06l0.2-0.2l0.12,0.01
				l0.16-0.12l0.07,0.07l0.24-0.07l-0.06-0.12l0.18-0.41l-0.08-0.07l0.14-0.07l-0.08-0.14l0.05-0.06l-0.2-0.29l0.09-0.12l-0.11-0.07
				l0.1-0.09l-0.05-0.08l-0.2,0.03l-0.03-0.07l0.14-0.08l-0.27-0.57l0.07-0.1l-0.01-0.17l-0.2-0.01l0.02-0.3l0.26-0.11l0.18,0.1
				l0.32-0.22l0-0.11l0.16,0.01l-0.06-0.17l0.34-0.74l0.06-0.07l0-0.34l-0.13-0.26l0.4-0.38l-0.09-0.1l0-0.45l-0.36-0.4l0.03-0.14
				l-0.1,0l0.03-0.27l-0.13-0.07l0.06-0.18l-0.17-0.02l0.05-0.14l-1-0.36l-0.27-0.17l-0.12,0.12l-0.27,0.33l-0.01,0.44l-0.22,0.32
				l-0.34,0.15l-0.16,0.24l0.06,0.11l-0.42,0.21l-0.05,0.19l-0.18,0.19l-0.33,0.04l-0.01,0.4l-0.07,0.05l-0.37-0.03l-0.29-0.13
				l-0.13,0.02l-0.1-0.52l-0.25-0.07l-0.34,0.16l-0.15,0.22l-0.28,0.09l-0.26-0.12l-0.34-0.34l-0.49-0.11l-0.06-0.44l-0.18-0.3
				l-0.14,0.07l-0.44-0.51l0.03-0.2l-0.18-0.05l-0.07-0.18l0.19-0.47l-0.1-0.02l-0.09,0.22l-0.25-0.07L317.2,150.47z"></path>
		</g>
		<g id="fuente_de_san_esteban">
			<path id="path993" class="st3 fase_0" d="M275.87,155.55l-0.31,0.18l-0.28,0.39l-0.16,0.34l-0.02,0.25l-0.39,0.12l-0.22,0.2
				l-0.11,0.21l0.13,0.4l-0.11,0.25l-0.42,0.5l-0.17,0.47l-0.45,0.33l0.36,0.68l0,0.17l0.15,0.32l0.33,0.27l-0.41,0.48l-0.34,0.19
				l-0.35,0.83l-0.57-0.14l-0.33-0.28l-0.35-0.48l-0.55-0.47l0.16,0.65l0.18,0.12l0.09,0.52l0.1,0.12l-0.24,0.01l-0.54,0.27
				l-0.75,0.45l-0.22,0.23l-0.31,0.03l-0.16,1.14l0.11,0.23l0.5,0.38l0.43-0.31l1.35-0.6l0.17-0.14l0.23,0.53l0.1,0.09l0.7-0.02
				l0.16,0.15l0.13,0l-0.36,0.8l0.09,0.23l0.27,0.12l0.16-0.01l0.22-0.29l0.11-0.01l0.15,0.09l0.04,0.16l0.16,0.16l0.27,0.19
				l0.03,0.09l-0.19,0.29l0.04,0.22l0.36,0.48l-0.26,0.31l-0.2,1.5l0.09,0.28l0.27,0.31l0.16-0.06l0.02,0.12l0.59,0.26l0.39,0.06
				l0.55-0.64l0.41-0.14l0.53,0.05l0.2-0.27l0.19-0.43l0.46-0.01l0.11-0.13l0.62-0.32l0.27-0.01l0.29-0.44l0.25,0.03l0.18-0.07
				l0.46,0.21l0.08-0.21l0.14-0.12l0.07,0.05l0.06-0.15l0.08-0.01l0.81,0.23l0.51,0.56l0.57-0.62l0.05-0.49l0.08-0.07l-0.09-0.36
				l0.2-0.18l0.21,0.48l0.11,0.07l-0.06,0.19l0.51,0.31l0.02,0.55l0.32,0.01l0.12,0.14l0.09,0.31l0.35,0.25l0.23,0.03l0,0.4
				l-0.34,0.27l1.07,0.26l0.34-0.42l0.23-0.03l0.08-0.11l0.02-0.51l0.17-0.01l0.37-0.27l0.3-0.59l-0.04-0.64l0.16-0.13l0.06-0.26
				l-0.16-0.78l-0.18-0.24l0.28-0.09l-0.33-0.56l0.01-0.47l0.27-0.35l-0.09-0.12l0.02-0.49l-0.64-0.03l-0.27-0.09l-0.24-0.19
				l-0.07-0.1l-0.22,0.1l-0.52,0.48l-0.37,0.06l-0.53-0.24l0.23-0.46l-0.3-0.34l-0.37-0.05l-0.34-0.27l-0.29-0.23l-0.15,0.1
				l-0.67,0.01l0.02-0.17l-0.1,0.02l-0.04-0.31l-0.07-0.03l-0.26,0.14l-0.23,0l0-0.16l0.14-0.24l-0.08-0.17l0.51-0.45l0.44,0
				l0.34-0.24l-0.09-0.13l-0.48-0.15l-0.22-0.3l0.06-0.27l0.41-0.8l-0.12-0.16l-0.38-0.11l-0.08-0.17l-0.43-0.09l-0.29,0.34
				l-0.04,0.34l-0.38-0.07l-0.16-0.06l-0.06-0.24l-0.24-0.24l0.06-0.18l-0.44-0.39l-0.45,0.12l-0.08-0.13l-0.2,0.08l-0.15-0.09
				l0.14-0.11l0.21-0.65l-0.67-0.3l-0.06-0.09l-0.6,0.13l-0.28-0.52l-0.9-0.09l-0.72-0.47l0-0.19l-0.09-0.05l-0.4-0.01
				L275.87,155.55z M285.59,159.9l0.02,0.24l0.2,0.1l0.13,0.4l0.17,0.22l-0.01,0.24l0.19,0.14l-0.04,0.3l0.26-0.09l0.07-0.19
				l-0.18-0.17l0.09-0.09l-0.03-0.4l-0.27-0.37L285.59,159.9z M283.25,160.97l-0.27,0.18l0.32,0.18l0.08-0.12l0.03-0.08l0.04-0.1
				L283.25,160.97z M267.48,163.91l-0.05,0.92l0.23,0.14l0.31,0.39l0.29,0.05l0.18-0.49l-0.43-0.4l-0.14-0.42l-0.2-0.17
				L267.48,163.91z"></path>
		</g>
		<g id="matilla_canos">
			<path id="path995" class="st3 fase_0" d="M287.17,153.92l-0.56,0.03l-0.24,0.62l-0.18,0.28l-0.18,0.26l0.04,0.09l0.14,0.03l0.58,0.49
				l-0.21,0.21l-0.02,0.24l0.07,0.1l-0.13,0.08l0.03,0.09l0.11-0.01l0.01,0.44l0.19,0.12l-0.24,0.33l-0.13,0.36l-0.44,0.08
				l-0.27-0.12l-0.22-0.23l-0.02-0.31l0.14-0.3l-0.09-0.24l-0.13-0.02l0.03-0.19l-0.32-0.35l-0.09,0.28l-0.32,0.15l-0.08,0.33
				l-0.43,0l-0.19,0.35l-0.71,0.15l-0.33,0.17l-0.25-0.08l-0.31-0.47l-0.25,0.05l0.02-0.17l0.22-0.27l0.03-0.17l-0.12-0.11
				l-0.42-0.03l-0.24-0.1l-0.3,0.13l-0.15-0.04l-0.42,0.41l-0.19,0.09l-0.35,0.47l-0.21,0.15l-0.4-0.03l-0.21,0.65l-0.14,0.11
				l0.15,0.09l0.2-0.08l0.08,0.13l0.45-0.12l0.44,0.39l-0.06,0.18l0.24,0.24l0.06,0.24l0.16,0.06l0.38,0.07l0.04-0.34l0.29-0.34
				l0.43,0.09l0.08,0.17l0.38,0.11l0.12,0.16l-0.41,0.8l-0.06,0.27l0.22,0.3l0.48,0.15l0.09,0.13l-0.34,0.24l-0.44,0l-0.51,0.45
				l0.08,0.17l-0.14,0.24l0,0.16l0.23,0l0.26-0.14l0.07,0.03l0.04,0.31l0.1-0.02l-0.02,0.17l0.67-0.01l0.15-0.1l0.29,0.23l0.34,0.27
				l0.37,0.05l0.3,0.34l-0.23,0.46l0.53,0.24l0.37-0.06l0.52-0.48l0.22-0.1l0.07,0.1l0.24,0.19l0.27,0.09l0.64,0.03l-0.02,0.49
				l0.09,0.12l-0.27,0.35l-0.01,0.47l0.33,0.56l-0.28,0.09l0.18,0.24l0.16,0.78l-0.06,0.26l-0.16,0.13l0.04,0.64l0.09,0.04l0.1-0.31
				l0.82-0.58l0.71,0.06l0.16,0.16l0.23-0.17l0.13,0.1l0.78-0.09l0.26,0.1l0.25-0.33l0.27,0.16l0.3,0.03l0.33-0.39l0.02-0.21
				l0.24-0.2l0.53,0.48l0.27,0.73l0.17,0.17l-0.05,0.09l0.24,0.17l0.15,0.33l0.45,0.2l0.16-0.15l0.41-0.05l0.1,0.1l0.41-0.24
				l0.07,0.11l-0.12,0.2l0.29,0.46l0.43,0.13l0.47,0l0.16,0.03l0.04-0.23l0.21-0.12l0.01-0.15l0.22,0.06l0.15-0.09l0.18,0.01
				l0.15-0.22l-0.7-0.36l0.03-0.19l0.25-0.11l0.37-0.63l0.11-0.08l0.14,0.06l0.25-0.09l0.11-0.1l0.04-0.36l-0.25-0.18l0.22-0.14
				l-0.05-0.18l-0.1,0.15l-0.05-0.07l0.07-0.11l-0.27-0.1l0-0.24l-0.08-0.08l0.02-0.08l0.26,0.03l-0.02-0.11l-0.08,0l0-0.12
				l0.11-0.08l-0.04-0.28l-0.07,0.03l-0.12-0.16l-0.35,0.07l-0.14-0.1l-0.06,0.1l-0.21-0.06l-0.17,0.4l-0.1,0.09l-0.21,0l-0.12-0.05
				l0.05-0.42l-0.22,0.02l-0.13-0.12l0.23-0.32l0.06-0.1l-0.17-0.73l-0.22-0.27l0.05-0.28l-0.2-0.37l-0.23,0.15l-0.12-0.02
				l0.01-0.43l0.18-0.16l-0.11-0.3l-0.16-0.32l-0.17-0.15l-0.15-0.43l0.22-0.1l-0.24-0.46l0.18-0.17l-0.05-0.16l0.12-0.15l0.1,0.06
				l-0.02-0.18l0.14-0.19l0.2,0.12l0.16-0.21l-0.02-0.16l0.09-0.2l-0.37-0.53l-0.03-0.34l0.15-0.13l0.08-0.13l-0.12-0.2l-0.13-0.03
				l-0.49,0.19l-0.41-0.29l-0.13,0.08l-0.05-0.15l-0.43,0.23l-0.51-0.65l0.09,0.31l-0.25-0.07l-0.18-0.53l-0.03,0.15l-0.12-0.06
				l-0.07-0.02l0.1-0.29l-0.11-0.09l-0.11-0.33l-0.24,0.24l-0.26-0.21l-0.31-0.07l-0.21-0.17l0-0.21l-0.17-0.09l-0.18,0.03
				l-0.3-0.26l-0.44-0.03l0-0.15l-0.43,0.09l-0.81-0.53l-0.02-0.24l-0.63-0.07l0.08-0.18l-0.18-0.11l-0.29,0.08l0.04,0.1l-0.22,0.04
				L287.17,153.92z M285.59,159.9l0.62,0.31l0.27,0.37l0.03,0.4l-0.09,0.09l0.18,0.17l-0.07,0.19l-0.26,0.09l0.04-0.3l-0.19-0.14
				l0.01-0.24l-0.17-0.22l-0.13-0.4l-0.2-0.1L285.59,159.9z M283.25,160.97l0.21,0.05l-0.04,0.1l-0.03,0.08l-0.08,0.12l-0.32-0.18
				L283.25,160.97z M293.4,165.92l0.29,0.06l0.24,0.23l0.26-0.04l0.04,1.02l-0.07,0.13l-0.33,0.06l-0.09,0.17l-0.22-0.11l-0.43,0.07
				l-0.15-0.26l0.09-0.68L293.4,165.92z"></path>
		</g>
		<g id="calzada_valdunciel">
			<path id="path1003" class="st3 fase_0" d="M302.44,139.54l-0.2,0.08l-0.46,0.67l-0.9-0.14l-1.36,0.36l-0.66,0.1l-0.2,0.11l-0.37,0.43
				l-1.03-0.01l-0.21-0.19l-0.34-0.07l-0.18-0.2l-1.07,0.22l-0.35-0.1l-0.28-0.15l-0.21,0.02l-0.22-0.21l-0.14,0.03l-0.19-0.25
				l-0.35-0.25l-0.54,0.08l-0.26-0.15l-0.43,0.34l-0.21,0.08l0.01,0.15l-0.33,0.53l0.14,0.71l0.07,0.93l0.09,0.2l-0.05,0.07
				l0.09,0.22l-0.1,0.22l-0.26-0.08l-0.19,0.22l-0.24,0.31l0.08,0.35l-0.04,0.1l-0.2-0.06l0,0.11l0.11,0.28l-0.41,0.05l-0.15,0.15
				l0.11,0.19l-0.23,0.19l-0.09,0.33l0.06,0.04l0.01,0.22l0.25,0.13l0.08-0.06l0.12,0.06l0.13-0.05l0.2,0.34l-0.29,0.19l-0.01,0.15
				l0.02,0.37l0.13,0.23h0.15l0.13-0.23l0.4,0.02l0.04,0.12l0.2,0.04l-0.01,0.26l0.29,0.01l0.11,0.17l0.13-0.12l0.04,0.15l0.49-0.08
				l0.07,0.11l-0.03,0.11l0.13,0.12l0.09-0.11l0.06,0.06l0.23-0.11l0.1-0.06l0.79,0.18l0.45-0.09l0.6,0.16l0.29-0.01l0.11-0.04
				l0.25,0.09l0,0.16l0.18,0.06l0.07,0.02l0.2,0.17l0.5,0.04l0.12,0.16l-0.07,0.21l0.13-0.02l0.44,0.15l0.26-0.01l0.24,0.16
				l0.36-0.02l0.1,0.14l0.26-0.13l0.1,0.12l0.01,0.15l0.27-0.05l0.07,0.04l-0.04,0.13l0.08,0.04l0.15-0.23l0.2,0.09l0.29-0.04
				l0.11-0.15l0.1-0.05l-0.02,0.19l0.09,0.05l0.15-0.02l0.07,0.08l0.12-0.12l0.27-0.05l0.06,0.08l0.16-0.27l0.1-0.05l0.14,0.11
				l0.15-0.02l0.08,0.07L302,149.2l0.09,0l0.06,0.15l0.12-0.09l0.08,0.05l-0.01-0.17l0.15-0.08l0.22-0.26l0.22-0.03l0-0.18l0.1-0.05
				l0.3,0.08l-0.06,0.15l0.36-0.16l0.19-0.33l0.1,0.03l0.06-0.25l0.08-0.2l-0.11,0l-0.05-0.23l-0.32-0.01l-0.03-0.26l-0.09-0.25
				l0.01-0.18l-0.16,0.05l0.01-0.19l-0.12,0.05l-0.03-0.11l-0.02-0.24l-0.2-0.3l-0.08-0.77l0.78-0.19l0.41-0.01l0.01-0.25l0.15-0.05
				l0.11,0.06l0.22,0.13l0.2-0.13l0.03-0.43l0.24-0.26l0.31-0.05l0.22-0.25l-0.11-0.6l-0.21-0.48l0.02-0.24l0.25-0.8l-0.68-0.3
				l-0.24-0.31l-0.37-0.09l-0.35,0.06l-0.25-0.94l-0.13-0.01l-0.78-0.72L302.44,139.54z"></path>
		</g>
		<g id="fuenteguinaldo">
			<path id="path1017" class="st3 fase_0" d="M261.97,176.74l-0.14,0.15l-0.23-0.13l-0.58,0.24l-0.03,0.33l-0.47,0.8l-0.03,0.3
				l0.15,0.27l-0.01,0.11l-0.33,0.03l-0.25,0.24l-0.16,0.51l-0.31,0.36l-0.15,0.45l0.64,0.49l-0.25,0.21l-0.2,0.06l-0.47,0.43
				l-0.12,0.22l-0.18-0.13l-0.08,0.06l-0.21,0.31l-0.46-0.16l-0.23,0.28l-0.48,0.05l-0.18-0.25l-0.58-0.21l-0.33-0.24l-0.22-0.03
				l0.15-0.51l-0.04-0.25l-0.25-0.39l-0.36-0.09l-0.11,0.24l-0.51,0.45l-0.58,1.23l0.19,0.88l0.15,0.73l0.27,0.78l0.48,0.6l0.4,0.29
				l0.16,0.28l-0.03,0.09l0.38,0.31l0.31,0.37l0.35,0.11l0.14,0.28l-0.34,0.23l-0.16,0.23l0.1,1.2l-0.13,0.27l-0.31,0.18l-0.64-0.04
				l-0.11,0.13l-0.14,0.02l-0.07,0.17l-0.29,0.08l-0.14,0.22l-0.16-0.01l-0.28,0.36l-0.55,0.41l-0.13,0.28l0.07,0.93l-0.13,0.39
				l0.74,0.87l0.28-0.06l0.11,0.38l0.11,0.05l0.48-0.03l-0.02,0.13l0.08,0.06l0.43,0.01l0.16,0.15l0.27-0.03l-0.08-0.4l0.83,0.02
				l0.1-0.06l0.08-0.47l-0.03-0.87l0.31-1.74l0.37-0.42l0.1-0.26l-0.13-0.3l0.1-0.67l0.88,0.1l-0.04-0.22l0.2-0.51l0.32-0.29
				l0.37,0.03l0.22-0.01l0.27,0.5l0.13-0.08l0-0.17l0.16,0.05l0.24-0.23l0.26-0.08l0.12,0.04l0.21-0.36l0.19-0.29l-0.06-0.07
				l0.02-0.37l-0.08-0.24l0.14-0.08l0.23-0.5l0.33-0.28l0.04-0.27l0.23-0.04l0.43-0.27l0.15,0l0.29-0.24l0.21-0.04l0.43-0.44
				l-0.03-0.14l0.6-0.47l0.14-0.34l0.3-0.15l-0.2-0.25l-0.11-0.45l-0.34-0.26l-0.16,0l-0.36-0.25l0.47-0.68l-0.45-0.46l-0.38-0.01
				l-0.39-0.38l-0.03-0.09l0.09-0.04l-0.15-0.28l0.28-0.18l0.13-0.26l0.06-0.55l-0.18-0.37l-0.18-0.08l-0.05-0.32l0.09-0.18
				l-0.11-0.12l-0.04-0.38l-0.14,0l-0.11-0.14l-0.4-0.15l-0.14-0.2L261.97,176.74z"></path>
		</g>
		<g id="bejar">
			<path id="path1669" class="st3 fase_0" d="M296.08,179.53l-0.2,0.11l-0.7,0.03l-0.79,0.22l-0.29-0.03l-1.16,0.65l-0.33,0.29
				l-0.88,0.52l-0.4-0.55l-0.03-0.28l-0.24-0.38l-0.27-0.08l0.1,0.25l-0.09,0.13l0.01,0.18l-0.56,0.18l-0.11-0.05l-0.02,0.25
				l-0.05,0.46l0.1,0.16l-0.52,0.54l-0.53,0.16l-0.11,0.49l-0.45,0.57l0.41,0.79l-0.82,0.46l-0.1-0.07l-0.22,0.03l-0.11,0.28
				l0.05,0.3l0.15,0.2l-0.01,0.18l0.21,0.13l0.04,0.23l0.22,0.18l-0.01,0.28l0.17,0.14l0.03,0.31l-0.1,0.16l0.09,0.16l-0.04,0.17
				l-0.08,0.48l-0.29,0.45l-0.14,0.95l0.13,0.33l0.2,0.17l0.21,0.39l0.57,0.38l0.54,0.11l0.13,0.2l0.29,0.19l0.14-0.1l0.94,0.38
				l0.16,0.18l0.69,0.01l0.15,0.17l0.13-0.5l0.23-0.16l0.49-0.82l0.15-0.35l-0.06-0.34l0.38-0.45l0.41,0l0.26-0.26l0.4-0.16
				l0.11-0.22l0.17,0.1l-0.11,0.27l0.27,0.18l0.9-0.87l0.09-0.2l0.23-0.16l0.34,0.05l0.44-0.17l0.14,0.25l0.48,1.94l-0.3,0.32
				l-0.16,0.34l-0.38,0.14l0.39,0.59l1.14,0.32l0.29-0.09l0.39-0.25l0.16-0.1l0.28-0.28l0.35-0.52l-0.01-0.35l0.6-0.8l0.26-0.26
				l0.72-0.41l0.16-0.32l0.53-0.26l-0.54-0.37l0.05-0.13l-0.08-0.18l0.01-0.82l0.34-0.56l-0.26-0.47l-0.11-1.09l-0.17-0.2l0.03-0.14
				l0.3-0.13l0.36-0.15l0.17-0.26l0.58-0.26l0.52-0.51l0.12-0.28l0.27-0.49l0.31-0.12l0.15-0.16l-0.64-0.63l-0.24,0.14l-0.24,0.02
				l-0.15,0.25l-0.31,0.2l-0.62,0.23l-0.36,0.08l-0.33-0.15l-0.15,0.19l-0.23,0.11l-0.07,0.18l-0.41,0.21l-0.29-0.18l-0.17-0.27
				l-0.27-0.22l-0.08,0.04l-0.22-0.2l-0.32-0.01l0.02,0.18l-0.36,0.21l-0.24,0.25l-0.52,0.26l-0.86-0.36l0.08-0.44l0.29-0.8
				l-0.3-0.47l-0.18-0.49l-0.26-0.24L296.08,179.53z"></path>
		</g>
		<g id="santa_marta_de_tormes">
			<path id="path1033" class="st3 fase_0" d="M310.71,153.07l-0.48,0.43l-0.19,0.24l-0.04,0.22l-0.09,0.05l-0.12,0.31l-0.12,0.04
				l-0.32-0.16l-0.93-0.07l-0.24-0.04l-0.21,0.16l-0.12-0.02l-0.29-0.07l-0.33-0.28l-0.41,0.13l-0.14,0.14l-0.23,0l-0.01-0.08
				l-0.99-0.12l-0.33,0.18l-0.84,0.19l-0.15,0.18l-0.33,0.76l-0.21,0.07l-0.34-0.44l-0.52,0.28l0.03,0.22l0.23,0.17l0.35-0.12
				l0.14,0.02l-0.07,0.07l0.11,0.1l-0.09,0.06l0.11,0.16l0.13,0.02l0,0.32l0.53,0.41l0.08,0.14l0.07,0.12l0.15,0.07l0.04,0.12
				l0.19-0.09l0.12,0.1l-0.1,0.12l0.17,0.2l-0.03,0.15l-0.13,0l0.07,0.25l-0.21,0.14l0.01,0.35l-0.21,0.08l-0.03,0.3l-0.11-0.06
				l-0.05,0.1l0.3,0.2l0.31,0.11l0,0.1l0.21-0.06l0.16,0.05l0.01-0.25l0.13-0.04l0.16,0.12l0.17-0.04l0.05,0.12l0.11-0.05l0.04-0.14
				l0.18,0.14l0.09-0.06l0.01,0.16l0.48,0.26l0.01,0.11l0.24-0.02l0.09,0.14l0.1-0.01l-0.2,0.33l-0.11,0.41l0.27,0.19l0.59-0.21
				l0.19-0.32l0.14-0.01l0.66,0.28l0.01,0.09l0.17-0.1l0.28-0.01l0.13-0.25l0.06-1.06l0.08-0.23l0.42-0.27l0.36,0.24l0.05,0.17
				l0.42-0.2l0.27,0.12l0.51-0.57l0.42-0.83l0.17-0.02l0.16,0.14l0.06-0.13l0.08-0.07L312,157l-0.08-0.07l-0.02-0.25l-0.1-0.09
				l-0.05-0.22l-0.11-0.08l-0.11,0.04l-0.04-0.1l-0.26-0.16l-0.01-0.14l-0.03-0.13l0.22-0.52l-0.07-0.49l-0.3-0.35l0.06-0.32
				l-0.16-0.46l-0.02-0.45L310.71,153.07z"></path>
		</g>
		<g id="ledesma">
			<path id="path1057" class="st3 fase_0" d="M277.68,138.93l-0.14,0.13l-0.44,0l0.02,0.53l-0.2,0.46l0.04,0.28l-0.2,0.32l-0.56,0.4
				l-0.11,0.42l0.03,0.15l-0.53,0.28l-0.56,0.43l-0.74,0.01l0.02,0.5l0.31,0.69l0.01,0.44l0.48-0.03l0.66,0.09l0.57,0.49l0.21,0.31
				l1.18,0.1l0.19-0.18l0.2,0.52l0.74,0.95l0.63,0.03l1.46,0.44l0.23-0.22l0.58-0.26l-0.03-0.65l0.1-0.26l0.85,0.41l0.27,0.05
				l0.2-0.05l0.2,0.11l-0.1,0.13l-0.08,0.43l-0.22,0.12l-0.22-0.09l-0.17,0.51l-0.01,0.36l0.34-0.06l-0.08,0.79l-0.18,0.09
				l-0.03,0.07l0.12,0.14l-0.66,0.89l0.33,0.44l-0.3,0.75l-0.18,0.23l-0.09,0.37l-0.19,0.03l-0.65,0.4l0.1,0.24l-0.18,0.38
				l-0.02,0.26l0.14,0.35l-0.05,0.27l0.11,0.31l0.01,0.51l0.45,0.01l0.1,0.16l0.12,0.13l0.39,0.11l0.15,0.21l0.3,0.14l-0.53,1.12
				l-0.26,0.17l-0.19,0.28l0.24,0.1l0.42,0.03l0.12,0.11l-0.03,0.17l-0.22,0.27l-0.02,0.17l0.25-0.05l0.31,0.47l0.25,0.08l0.33-0.17
				l0.71-0.15l0.19-0.35l0.43,0l0.08-0.33l0.32-0.15l0.09-0.28l0.32,0.35l-0.03,0.19l0.13,0.02l0.09,0.24l-0.14,0.3l0.02,0.31
				l0.22,0.23l0.27,0.12l0.44-0.08l0.13-0.36l0.24-0.33l-0.19-0.12l-0.01-0.44l-0.11,0.01l-0.03-0.09l0.13-0.08l-0.07-0.1l0.02-0.24
				l0.21-0.21l-0.58-0.49l-0.14-0.03l-0.04-0.09l0.18-0.26l0.18-0.28l0.24-0.62l0.56-0.03l0.42,0.37l0.22-0.04l-0.04-0.1l0.29-0.08
				l0.18,0.11l-0.08,0.18l0.63,0.07l0.02,0.24l0.81,0.53l0.43-0.09l0.03-0.25l0.32-0.21l0.11,0.04l0.27-0.09l0.12-0.13l0.23,0.1
				l0.4-0.71l0.45-0.47l-0.24-0.43l-0.09-0.02l-0.06-0.24l-0.12,0.03l-0.26-0.18l0.09-0.16l-0.18-0.07l0.12-0.21l0.45-0.12
				l0.15-0.12l-0.22-0.45l0.15-0.1l0.16,0.05l0.19-0.19l-0.22-0.13l-0.01-0.4l-0.26-0.26l0.19-0.16l0.34,0.07l0-0.2l-0.19-0.03
				l0.08-0.28l-0.1-0.05l0.04-0.13l0.1-0.02l0-0.1l-0.17-0.39l0.39-0.35l0.04-0.15l-0.13-0.21l-0.15-0.01l-0.15,0.14l-0.15-0.05
				l-0.5,0.12l0-0.37l-0.13-0.29l0.06-0.37l0.05-0.15l0.33-0.15l0-0.14h-0.15l-0.13-0.23l-0.02-0.37l0.01-0.15l0.29-0.19l-0.2-0.34
				l-0.13,0.05l-0.12-0.06l-0.08,0.06l-0.25-0.13l-0.01-0.22l-0.06-0.04l0.09-0.33l-0.1,0.03l-0.09-0.19l-0.31-0.13l0.01-0.16
				l-0.15-0.37l-0.09-0.05l-0.13-0.3l0.02-0.35l-0.11-0.2l-0.21-0.11l-0.24,0.07l-0.71,1.09l-0.29,0.06l-0.49-0.17l-0.26-0.61
				l0.03-0.68l0.38-0.6l0.29-0.19l0.09-0.36l-0.63-0.14l-0.63-0.28l-0.27-0.04l-0.48-0.31l-0.14-0.34l-0.15,0.18l-0.15,0.05
				l-0.19,0.29l-0.75-0.06l-0.3,0.12l-0.38-0.12l-0.48,0.48l-0.13,0.04l-0.08,0.34l-0.33,0.34l-0.37-0.4l-0.84-0.53l-0.57-0.06
				l-0.52-0.28l-0.07-0.06l-0.32-0.27l-0.12,0.01l-0.85-0.42l-0.24-0.27l-0.08-0.22l-0.77-0.3l-0.6-0.61l-0.04-0.1L277.68,138.93z"></path>
		</g>
		<g id="villoria">
			<path id="path1139" class="st3 fase_0" d="M314.82,145.19l-1.08,0.35l-0.01,0.22l0.1,0.12l-0.08,0.06l0.1,0.15l-0.06,0.13l0.15,0.1
				l0.02,0.12l-0.1,0.06l0.1,0.27l-0.17,0.16l-0.04-0.1l-0.23,0.1l-0.31,0.28l-0.17,0.02l-0.09,0.08l0,0.13l-0.19,0.08l-0.03,0.13
				l-0.09,0.05l-0.07-0.14l-0.14,0.01l-0.03,0.11l-0.08-0.02l-0.1,0.14l-0.26,0.42l0.05,0.28l-0.07,0.13l-0.12,0.01l0.13,0.18
				l-0.24,0.17l0.09,0.26l-0.04,0.06l0.42,0.31l0.04,0.11l-0.05,0.22l-0.29,0.3l-0.13,0.37l0.03,0.48l-0.09,0.19l-0.5,0.4
				l-0.08,0.02l-0.06,0.68l-0.16,0.4l-0.15,0.14l-0.02,0.16l0.22,0.13l0.02,0.45l0.16,0.46l-0.06,0.32l0.3,0.35l0.07,0.49
				l-0.22,0.52l0.03,0.13l0.01,0.14l0.26,0.16l0.04,0.1l0.11-0.04l0.11,0.08l0.05,0.22l0.1,0.09l0.02,0.25L312,157l0.23,0.23
				l0.12-0.21l0.5-0.02l1.03-0.81l0.02-0.19l0.36-0.43l-0.08-0.08l0.24-0.22l-0.06-0.15l0.82-0.08l0.09-0.07l0.09-0.34l0.6-0.34
				l-0.16-0.2l0.13-0.1l-0.1-0.31l0.02-0.27l0.4-0.5l-0.2-0.05l-0.16-0.23l-0.13-0.65l-0.12-0.08l0.34-0.26l0.18-0.41l0.48-0.22
				l0.05-0.11l-0.11-0.09l0.21-0.12l0.15,0.05l0-0.13l0.26-0.14l0.09-0.01l0.05-0.15l-0.11-0.04l0.08-0.31l0.14-0.17l-0.07-0.16
				l0.25-0.16l-0.03-0.24l0.42-0.05l0.01-0.11l0.11,0l0.14-0.24l0.06-0.08l-0.09-0.06l0.07-0.19l-0.15-0.1l0.16-0.09l-0.09-0.12
				l-0.18,0.06l0.07-0.15l0.16-0.07l-0.08-0.21l-0.28-0.07l0.11-0.06l-0.09-0.31l-0.28-0.63l0.05-0.11l0.16-0.35l-0.25-0.06
				l-0.31-0.3l-0.12,0.05l-0.79,0.38l-0.46,0.09l-0.12-0.04l0.12-0.17l-0.06-0.09l-0.09-0.09l-0.33,0.06l-0.28-0.54l-0.2,0
				L314.82,145.19z"></path>
		</g>
		<g id="periurbana_norte">
			<path id="path1155" class="st3 fase_0" d="M291.68,146.91l-0.13,0.23l0,0.14l-0.33,0.15l-0.05,0.15l-0.06,0.37l0.13,0.29l0,0.37
				l0.5-0.12l0.15,0.05l0.15-0.14l0.22-0.38l0.19-0.01l0.11,0.15l-0.02,0.43l0.2,0.3l0.54,0.14l0.16,0.22l0.19-0.04l0.08-0.15
				l0.33-0.24l0.17-0.02l0.81,0.4l0.47,0.18l0.06,0.1l-0.12,0.23l-0.21,0.03l-0.24,0.49l0.25-0.08l0.27,0.2l0.26-0.35l0.06-0.22
				l0.16,0.12l0.18-0.09l0.16-0.03l0.24,0.18l0.07,0.06l0.25,0.07l0.11,0.13l0.19,0.27l0.39-0.24l0.22-0.02l0.23,0.13l0.1,0.06
				l0.26,0.22l0.16,0.03l0.36,0.14l0.45,0.18l0.28,0.38l0.16-0.04l0.06,0.08l-0.08,0.21l-0.14,0.03l0,0.12l-0.04,0.34l0.16,0.59
				l-0.17,0.48l0.18,0.05l0.24,0.01l0.21,0.26l0.04,0.33l-0.07,0.59l0.27-0.24l-0.02-0.21l0.15-0.08l0.33,0.15l0.23-0.13l0.03-0.18
				l0.46-0.02l0.16-0.27l0.06-0.04l0.11,0.06l0.12-0.26l0.18-0.04l0.1,0.06l0.16-0.12l0.14,0.01l0.6,0.45l0.38,0.04l0.34,0.21
				l0.03,0.26l-0.08,0.12l0.12,0.26l-0.82,0.49l0.34,0.44l0.21-0.07l0.33-0.76l0.15-0.18l0.84-0.19l0.33-0.18l0.99,0.12l0.01,0.08
				l0.23,0l0.14-0.14l0.41-0.13l0.33,0.28l0.29,0.07l0.12,0.02l0.21-0.16l0.24,0.04l0.93,0.07l0.32,0.16l0.12-0.04l0.12-0.31
				l0.09-0.05l0.04-0.22l0.19-0.24l0.48-0.43l0.02-0.16l0.15-0.14l0.16-0.4l0.06-0.68l0.08-0.02l-0.04-0.35l-0.14-0.18l-0.4-0.2
				l-0.42,0.19l-0.1,0.12l-0.28-0.06l-0.12,0.08l-0.41-0.04l-0.18,0.15l-0.14,0l-0.27,0.16l-0.27,0.27l-0.17-0.02l-0.29,0.11
				l-0.24,0.33l-0.17,0.11l-0.2-0.04l-0.58-0.24l-0.28-0.55l0.06-0.11l0.08-0.16l-0.16-0.2l-0.05-0.21l-0.26-0.27l-0.08-0.24
				l-0.18-0.16l-0.27,0.07l-0.22-0.03l-0.28-0.36l-0.26,0.31l-0.06-0.07l0.09-0.09l-0.15-0.11l-0.07,0.07l-0.08,0.04l-0.2,0.11
				l-0.24-0.03l-0.19,0.13l-0.35-0.02l-0.03-0.17l-0.16-0.07l-0.03-0.14l0.08-0.16l-0.05-0.1l0.13-0.11l0.07,0.04l0.02-0.24
				l0.25,0.03l-0.04,0.18l0.13,0.05l0.08-0.12l0.17,0.12l0.02-0.09l0.17,0.03l0.08,0.12l0.33-0.12l0.13-0.53l0.1-0.03l0.07-0.14
				l-0.16-0.23l-0.12,0l-0.09-0.18l-0.14,0.03l-0.16-0.11l-0.09,0.07l-0.23-0.03l-0.18-0.25l-0.06,0.25l-0.1-0.03l-0.19,0.33
				l-0.36,0.16l0.06-0.15l-0.3-0.08l-0.1,0.05l0,0.18l-0.22,0.03l-0.22,0.26l-0.15,0.08l0.01,0.17l-0.08-0.05l-0.12,0.09l-0.06-0.15
				l-0.09,0l0.03-0.13l-0.08-0.07l-0.15,0.02l-0.14-0.11l-0.1,0.05l-0.16,0.27l-0.06-0.08l-0.27,0.05l-0.12,0.12l-0.07-0.08
				l-0.15,0.02l-0.09-0.05l0.02-0.19l-0.1,0.05l-0.11,0.15l-0.29,0.04l-0.2-0.09l-0.15,0.23l-0.08-0.04l0.04-0.13l-0.07-0.04
				l-0.27,0.05l-0.01-0.15l-0.1-0.12l-0.26,0.13l-0.1-0.14l-0.36,0.02l-0.24-0.16l-0.26,0.01l-0.44-0.15l-0.13,0.02l0.07-0.21
				l-0.12-0.16l-0.5-0.04l-0.2-0.17l-0.07-0.02l-0.18-0.06l0-0.16l-0.25-0.09l-0.11,0.04l-0.29,0.01l-0.6-0.16l-0.45,0.09
				l-0.79-0.18l-0.1,0.06l-0.23,0.11l-0.06-0.06l-0.09,0.11l-0.13-0.12l0.03-0.11l-0.07-0.11l-0.49,0.08l-0.04-0.15l-0.13,0.12
				l-0.11-0.17l-0.29-0.01l0.01-0.26l-0.2-0.04l-0.04-0.12L291.68,146.91z"></path>
		</g>
		<g id="garrido_norte">
			<path class="st3 fase_0" d="M304.06,154.42l-0.12-0.26l0.08-0.12l-0.03-0.26l-0.34-0.21h0l-0.38-0.04l-0.6-0.45l-0.14-0.01l-0.16,0.12
				l-0.1-0.06l-0.18,0.04l-0.12,0.26l-0.11-0.06l-0.06,0.04l-0.15,0.27l-0.46,0.02l-0.03,0.18l-0.23,0.13l-0.33-0.15l-0.15,0.08
				l0.02,0.21l-0.15,0.13c0.07,0.18,0.13,0.37,0.18,0.55c0.04,0.13,0.07,0.26,0.12,0.38c0.02,0.06,0.05,0.11,0.08,0.17
				c0.02,0.05,0.05,0.1,0.07,0.16c0.06,0.16,0.12,0.33,0.2,0.49l0.04-0.03l0.07,0.22l0.1-0.28l-0.09-0.19l0.14-0.08l0.06,0.1
				l0.04,0.28l0.25,0.07l-0.02,0.09l0.17,0.01l0,0.34l0.02,0.06c0.02,0.01,0.04,0.02,0.07,0.02c0.07,0.02,0.14,0.03,0.22,0.03
				l-0.08-0.19l0.42-0.21l0.02-0.13l0.02-0.1l0.12-0.07l0.07-0.39l0.14-0.14l-0.03-0.22l0.52-0.28L304.06,154.42z M301.63,156.03
				l-0.27-0.29l0.27-0.21l0.15,0.21L301.63,156.03z"></path>
		</g>
		<g id="garrido_sur">
			<path class="st3 fase_0" d="M304.02,154.04l-0.03-0.26l-0.34-0.21h0l-0.24-0.02c-0.02,0.01-0.05,0.03-0.07,0.04
				c-0.24,0.13-0.48,0.24-0.73,0.35c-0.25,0.11-0.51,0.23-0.75,0.35c-0.26,0.13-0.51,0.28-0.75,0.42c-0.12,0.07-0.25,0.15-0.37,0.22
				c-0.06,0.03-0.11,0.07-0.16,0.1c0.02,0.05,0.03,0.11,0.05,0.16c0.02,0.06,0.05,0.11,0.08,0.17c0.02,0.05,0.05,0.1,0.07,0.16
				c0.06,0.16,0.12,0.33,0.2,0.49l0.04-0.03l0.07,0.22l0.1-0.28l-0.09-0.19l0.14-0.08l0.06,0.1l0.04,0.28l0.25,0.07l-0.02,0.09
				l0.17,0.01l0,0.34l0.02,0.06c0.02,0.01,0.04,0.02,0.07,0.02c0.07,0.02,0.14,0.03,0.22,0.03l-0.08-0.19l0.42-0.21l0.02-0.13
				l0.02-0.1l0.12-0.07l0.07-0.39l0.14-0.14l-0.03-0.22l0.52-0.28l0.82-0.49l-0.12-0.26L304.02,154.04z M301.63,156.03l-0.27-0.29
				l0.27-0.21l0.15,0.21L301.63,156.03z"></path>
		</g>
		<g id="periurbano_sur">
			<path class="st3 fase_0" d="M304.9,157.42l-0.17-0.2l0.1-0.12l-0.12-0.1l-0.19,0.09l-0.04-0.12l-0.15-0.07l-0.07-0.12l-0.08-0.14
				l-0.53-0.41l0-0.32l-0.13-0.02l-0.11-0.16l0.09-0.06l-0.11-0.1l0.07-0.07l-0.14-0.02l-0.35,0.12l-0.23-0.17l-0.14,0.14
				l-0.07,0.39l-0.12,0.07l-0.02,0.11l0.08,0.08l-0.01,0.15l-0.12,0.07l0.03-0.17l-0.42,0.21l0.12,0.3l-0.11,0.32l-0.19,0.14
				l-0.15,0.01l-0.03-0.11l-0.08-0.1l0.04-0.12l0.11,0.11l0.14-0.23l-0.08-0.25l0-0.34l-0.17-0.01l0.02-0.09l-0.25-0.07l-0.04-0.28
				l-0.06-0.1l-0.14,0.08l0.09,0.19l-0.1,0.28l-0.07-0.22l-0.37,0.23l-0.01-0.32l-0.25,0.1l-0.6,0.49l-0.18-1.01l0.23-0.19
				l-0.11-0.03l-0.02-0.08l0.06-0.26l-0.24-0.14l0.09-0.19l0.12-0.14l0.28,0.1l0.08-0.16l0.11,0l0.07-0.59l-0.04-0.33l-0.21-0.26
				l-0.24-0.01l-0.18-0.05l0.17-0.48l-0.16-0.59l0.04-0.34l0-0.12l0.14-0.03l0.08-0.21l-0.06-0.08l-0.16,0.04l-0.28-0.38l-0.45-0.18
				l-0.36-0.14l-0.16-0.03l-0.26-0.22l-0.1-0.06l-0.23-0.13l-0.22,0.02l-0.39,0.24l-0.19-0.27l-0.11-0.13l-0.22,0.05l-0.03-0.12
				l-0.07-0.06l-0.24-0.18l-0.16,0.03l-0.18,0.09l-0.16-0.12l-0.06,0.22l-0.26,0.35l-0.27-0.2l-0.25,0.08l0.24-0.49l0.21-0.03
				l0.12-0.23l-0.06-0.1l-0.47-0.18l-0.81-0.4l-0.17,0.02l-0.33,0.24l-0.08,0.15l-0.19,0.04l-0.16-0.22l-0.54-0.14l-0.2-0.3
				l0.02-0.43l-0.11-0.15l-0.19,0.01l-0.22,0.38l0.15,0.01l0.13,0.21l-0.04,0.15l-0.39,0.35l0.17,0.39l0,0.1l-0.1,0.02l-0.04,0.13
				l0.1,0.05l-0.08,0.28l0.19,0.03l0,0.2l-0.34-0.07l-0.19,0.16l0.26,0.26l0.01,0.4l0.22,0.13l-0.19,0.19l-0.16-0.05l-0.15,0.1
				l0.22,0.45l-0.15,0.12l-0.45,0.12l-0.12,0.21l0.18,0.07l-0.09,0.16l0.26,0.18l0.12-0.03l0.06,0.24l0.09,0.02l0.24,0.43
				l-0.45,0.47l-0.4,0.71l-0.23-0.1l-0.12,0.13l-0.27,0.09l-0.11-0.04l-0.32,0.21l-0.03,0.25l0,0.15l0.44,0.03l0.3,0.26l0.18-0.03
				l0.17,0.09l0,0.21l0.21,0.17l0.31,0.07l0.26,0.21l0.24-0.24l0.11,0.33l0.11,0.09l-0.1,0.29l0.07,0.02l0.12,0.06l0.03-0.15
				l0.18,0.53l0.25,0.07l-0.09-0.31l0.51,0.65l0.43-0.23l0.05,0.15l0.13-0.08l0.41,0.29l0.49-0.19l0.13,0.03l0.12,0.2l-0.08,0.13
				l-0.15,0.13l0.03,0.34l0.37,0.53l-0.09,0.2l0.02,0.16l-0.16,0.21l-0.2-0.12l-0.14,0.19l0.02,0.18l-0.1-0.06l-0.12,0.15l0.05,0.16
				l-0.18,0.17l0.24,0.46l-0.22,0.1l0.15,0.43l0.17,0.15l0.16,0.32l0.34-0.16v0l-0.34,0.16l0.11,0.3l-0.18,0.16l-0.01,0.43
				l0.12,0.02l0.23-0.15l0.2,0.37l-0.05,0.28l0.22,0.27l0.17,0.73l-0.06,0.1l-0.23,0.32l0.13,0.12l0.22-0.02l-0.05,0.42l0.12,0.05
				l0.21,0l0.1-0.09l0.17-0.4l0.21,0.06l0.06-0.1l0.14,0.1l0.35-0.07l0.12,0.16l0.07-0.03l0.04,0.28l-0.11,0.08l0,0.12l0.08,0
				l0.02,0.11l-0.26-0.03l-0.02,0.08l0.08,0.08l0,0.24l0.27,0.1l-0.07,0.11l0.05,0.07l0.11-0.15l0.05,0.18l-0.22,0.14l0.25,0.18
				l-0.04,0.36l0.34,0.04l0.26-0.23l0.09-0.23l0.33-0.05l0.1-0.21l0.13,0.08l0.17-0.13l0.27,0.11l0.18,0.1l-0.09,0.22l0.15,0.16
				l0.13,0.66l0.25,0.07l0.09,0.11l0.17,0.01l0.38,0.24l0.09,0.15l0.16,0.03l-0.03-0.32l0.36,0.02l-0.04-0.14l0.09-0.02l0.05-0.35
				l0.13-0.13l0.04,0.12l0.45,0.13l0.23-0.01l0.09-0.17l0.54,0.04l-0.05-0.12l-0.14,0.01l0.02-0.2l-0.14-0.34l0.02-0.12l0.17-0.12
				l-0.03-0.09l0.18,0.03l0.02-0.22l0.1-0.05l0.02-0.24l-0.14-0.54l-0.09-0.27l0.08-0.15l-0.04-0.23l-0.31-0.13l-0.01-0.14
				l-0.12-0.04l-0.19-0.17l0.05-0.16l-0.06-0.12l0.35-0.86l0.14-0.15l0.26,0.06l0.28-0.29l0.24,0.13l0.05,0.16l0.42,0.14l0.04,0.16
				l0.41-0.27l0.15,0.01l0.12-0.29l-0.39-0.38l-0.06-0.18l0.11-0.2l-0.14,0.03l-0.07-0.14l0.08-0.2l-0.09-0.22l0.12-0.03l-0.31-0.23
				l0.12-0.21l-0.06-0.45l-0.04-0.28l0.1,0l0.14-0.13l0.06-0.22l0.47-0.51l-0.3-0.2l0.05-0.1l0.11,0.06l0.03-0.3l0.21-0.08
				l-0.01-0.35l0.21-0.14l-0.07-0.25l0.13,0L304.9,157.42z M301.78,155.73l-0.14-0.21l-0.26,0.23l0.26,0.29L301.78,155.73z"></path>
		</g>
		<g id="fuentes_de_onoro">
			<path id="path1385" class="st3 fase_0" d="M256.31,160.24l-0.07,0.25l0.14,0.42l-0.08,0.17l-0.23-0.05l-0.19,0.22l-0.16-0.08
				l-0.12,0.06l-0.12-0.04l-0.06-0.11l-0.08,0.12l0,0.15l0.2,0.2l-0.13,0.57l0.28,1l0.02,0.32l-0.12,0.36l-0.33,0.93l0.09,0.44
				l-0.1,0.52l-0.11,0.38l0.09,0.44l-0.07,0.07l0.04,0.14l0.19,0.17l0.01,0.39l0.29,0.22l0.26,0.44l-0.09,0.49l0.1,0.4l0.15,0.15
				l-0.02,0.23l0.08,0.03l-0.03,0.12l0.15,0.58l-0.12,0.6l0.47,0.38l-0.23,0.39l-0.34,1.33l-0.21,0.39l-0.32,0.66l-0.68,0.85
				l-0.01,0.78l-0.13,0.18l-0.16,0.57l0.58,0.15l0.31,0.26l0.61,0.2l0.09-0.03l0.04,0.23l0.14,0.01l-0.18,0.54l0.09,0.58l0.24,0.37
				l0.08,0.59l-0.63,0.37l-0.17,0.19l-0.13,0.37l0.03,0.2l-0.11,0.16l0.36,0.09l0.25,0.39l0.04,0.25l-0.15,0.51l0.22,0.03l0.33,0.24
				l0.58,0.21l0.18,0.25l0.48-0.05l0.23-0.28l0.46,0.16l0.21-0.31l0.08-0.06l0.18,0.13l0.12-0.22l0.47-0.43l0.2-0.06l0.25-0.21
				l-0.64-0.49l0.15-0.45l0.31-0.36l0.16-0.51l0.25-0.24l0.33-0.03l0.01-0.11l-0.15-0.27l0.03-0.3l0.47-0.8l0.03-0.33l-0.19-0.16
				l0.24-0.36l0.11-0.31l-0.04-0.14l-0.23-0.18l-0.32-0.11l-0.18-0.18l0.04-0.54l0.15-0.34l-0.01-0.37l0.73-0.19l0.25,0.02
				l-0.04-0.42l0.42,0.1l0.13-0.09l0.5,0.19l0.08-0.31l0.1-0.11l0.13,0.1l0.21-0.24l0.03-0.16l-0.1-0.26l-0.66-0.01l-0.06-0.07
				l0.37-0.66l0.03-0.17l-0.04-0.11l-0.39-0.03l-0.15-0.09l-0.43-0.7l-0.02-0.67l0.41-0.73l0.04-0.29l-0.14-0.3l-0.01-0.74
				l0.14-0.36l-0.02-0.16l-0.32-0.22l-0.13-0.23l0.16-0.22l0.08-0.43l-0.07-0.48l-0.16-0.1l0.1-0.4l-0.04-1.22l-0.07-0.23
				l-0.14,0.05l-0.16,0.23l-0.33,0.14l-0.57-0.06l-0.56-0.39l-0.24-0.39l0.02-0.2l-0.15-0.1l0-0.18l-0.34-0.08l-0.21-0.36l0.03-0.34
				l-0.14-0.38l-0.15-0.02l-0.24-0.18l-0.14-0.61l-0.29-0.09l-0.23-0.11l-0.43-0.02l-0.5-0.5l-0.51-0.26L256.31,160.24z"></path>
		</g>
		<g id="miguel_armijo">
			<path id="path989" class="st3 fase_0" d="M300.57,156.34l0.01-0.26l-0.11-0.23l-0.23-0.06l-0.17,0.07l-0.29-0.78l-0.4-0.42l0.23-0.17
				l0.5,0.03l0.15-0.32l0.18-0.02l0.06,0.12l0.22-0.03l0.11,0.09l-0.05,0.36l0.29,0.22l0.16,0.32l-0.07,0.06l-0.11,0.11l0.05,0.08
				l-0.26,0.28l-0.12,0.55L300.57,156.34z"></path>
		</g>
		<g id="elena_ginel_diez">
			<path id="path1027" class="st3 fase_0" d="M296.71,156.01l0.15,0.03l0.06-0.09l0.16,0.1l0.14-0.03l0.07-0.14l0.2,0.12l0.37-0.32
				l0.3,0.05l0.01,0.19l0.34-0.09l0.01-0.21l0.09-0.08l-0.19-0.14l-0.05-0.14l0.06-0.27l-0.14-0.07l0.1-0.34l0.08-0.02l0.01,0.16
				l0.25,0.02l0.1,0.16l0.13-0.12l0.24,0.24l0.12,0.16l-0.06,0.15l0.28,0.2l0.28-0.2l0.26,0.08l0.12,0.54l0.08,0.02l-0.06,0.33
				l-0.27,0.15l-0.28,0l-0.17,0.1l-0.01,0.3l-0.23,0.09l0.26,0.03l-0.06,0.23l0.25,0.11l0.01,0.1l-0.26,0.03l-0.24,0.13l-0.03,0.15
				l-0.13-0.04l-0.23,0.08l-0.28-0.14l-0.17-0.32l0.06-0.06l-0.05-0.13l-0.25-0.03l-0.14-0.12l-0.19,0.07l0.01,0.17l-0.09,0.13
				l-0.33-0.15l-0.19,0.08l-0.33-0.78l-0.12,0.07l-0.07-0.08l0.11-0.15l-0.12-0.07L296.71,156.01z"></path>
		</g>
		<g id="san_jose">
			<path id="path1027" class="st3 fase_0" d="M299.37,157.71l0.15-0.01l0.04-0.1l0.18,0.06l0.13-0.06l0.03-0.15l0.22,0.06l0.27-0.4
				l0.3-0.03l0.05,0.18l0.31-0.17l-0.04-0.21l0.07-0.1l-0.21-0.08l-0.08-0.13l-0.01-0.28l-0.15-0.04l0.02-0.35l0.07-0.04l0.05,0.15
				l0.25-0.04l0.14,0.13l0.09-0.14l0.29,0.17l0.15,0.13l-0.02,0.16l0.32,0.13l0.22-0.26l0.27,0.01l0.25,0.5l0.08,0l0.03,0.34
				l-0.22,0.22l-0.27,0.07l-0.14,0.14l0.07,0.3l-0.2,0.14l0.26-0.03l0,0.23l0.27,0.04l0.04,0.1l-0.24,0.1l-0.2,0.19l0.01,0.15
				l-0.14-0.01l-0.21,0.14l-0.3-0.07l-0.25-0.26l0.05-0.07l-0.08-0.12l-0.25,0.03l-0.17-0.08l-0.17,0.12l0.05,0.16l-0.05,0.15
				l-0.36-0.06l-0.16,0.13l-0.51-0.67l-0.1,0.1l-0.09-0.06l0.07-0.17l-0.13-0.04L299.37,157.71z"></path>
		</g>
		<g id="robleda">
			<path id="path1657" class="st3 fase_1" d="M269.21,179.53l-0.52,0.3l-0.1,0.23l-0.59,0.3l-0.12,0.33l-0.18,0.12l-0.29,0.05
				l-0.16,0.31l-0.26,0.04l-0.66,0.54l-0.29,0.37l-0.09-0.03l-0.14,0.19l-0.23,0.1l-0.05,0.21l-0.4,0.16l-0.15,0.11l-0.3,0.15
				l-0.14,0.34l-0.6,0.47l0.03,0.14l-0.43,0.44l-0.21,0.04l-0.29,0.24l-0.15,0l-0.43,0.27l-0.23,0.04l-0.04,0.27l-0.33,0.28
				l-0.23,0.5l-0.14,0.08l0.08,0.24l-0.02,0.37l0.06,0.07l-0.19,0.29l-0.21,0.36l-0.12-0.04l-0.26,0.08l-0.24,0.23l-0.16-0.05
				l0,0.17l-0.13,0.08l-0.27-0.5l-0.22,0.01l-0.37-0.03l-0.32,0.29l-0.2,0.51l0.04,0.22l-0.88-0.1l-0.1,0.67l0.13,0.3l-0.1,0.26
				l-0.37,0.42l-0.31,1.74l0.03,0.87l-0.08,0.47l-0.1,0.06l0.14,0.19l0.91,0.35l0-0.39l0.15-0.19l0.42-0.34l0.25-0.05l0.34-0.42
				l0.34-0.23l0.27,0.12l0.07,0.35l0.27,0.51l0.53,0.45l0.07-0.21l0.43-0.23l0-0.38l0.24-0.28l1.29-0.31l0.22,0.23l0.38-0.1
				l0.42,0.03l0.7-0.3l0.42,0.05l0.14-0.2l0.51-0.27l0.09-0.18l0.51-0.43l-0.14-0.27l0.14-0.45l-0.03-0.12l-0.32-0.44l0.18-0.78
				l0.98-0.98l0.81-0.38l0.93-0.42l0.31-0.07l0.45-0.37l0.51-0.14l0.48,0.1l0.48-0.1l-0.11-0.28l0.06-0.19l0.62-0.53l0.22-0.42
				l0.45-0.15l0.67,0.16l0.33-0.09l0.29,0.03l0.57-0.37l-0.05-0.25l0.06-0.2l-0.18-0.11l0.01-0.27l-0.44,0.18l-0.43-0.06l-0.29-1.23
				l0.03-0.36l-0.45,0.15l-0.77-0.81l-0.11-0.46l-0.13-0.16l-0.31,0.32l-0.45,0.2l-0.25,0.35l-0.15-0.15l-0.11,0.1l-0.08-0.04
				l-0.16-0.37l-0.08,0.01l-0.07-0.12l-0.26,0.11l-0.03-0.08l0.11-0.17l0.25-0.11l-0.13-0.06l-0.02-0.08l0.08-0.06l-0.07-0.14
				l0.06-0.08l-0.09-0.02l0.01-0.23l-0.22-0.79l-0.17,0.02L269.21,179.53z"></path>
		</g>
	</g>
	<g id="zamora">
		<g id="villarrin">
			<path id="path464" class="st3 fase_0" d="M305.51,102.33l-0.85,0.12l-0.05,0.12l-0.02,0.61l-0.11,0.14l-0.07-0.06l-0.08,0.07
				l-0.51,0.45l0.04,0.18l-0.5-0.21l-0.18,0.32l0.06,0.14l-0.1,0.05l0.41,0.3l-0.22,0.37l-0.56-0.51l0.17,0.48l0.11,0.99l0.06,0.12
				l-0.3,0.41l-0.69,0.66l0.12,0.13l-0.49,0.39l-0.42,0.2l-0.91,0.06l-0.84,0.33l-0.23,0.02l-0.76-0.2l-0.48-0.54l-0.56-0.12
				l-0.05,0.23l-0.32,0.34l-0.09,0.3l0.06,0.82l-0.38,0.87l-0.32,0.53l-0.04-0.03l-0.46,0.4l-0.14,0.31l0.02,0.48l-0.14,0.64
				l0.2,0.26l0.59,0.26l0.28,0.18l0.11,0.28l-0.06,0.1l-0.24,0.04l-0.1,0.28l0.46,0.17l-0.01,0.32l-0.35,0.98l-0.14,0.07l-0.06-0.06
				l0.01-0.34l-0.1-0.25l-0.13-0.1l-0.13,0.01l-0.22,0.17l-0.31,0.02l-0.11,0.17l-0.03,0.5l-0.17,0.03l-0.44-0.25l-0.19,0.05
				l-0.13,0.51l0.1,0.22l0.9,0.28l1.48-0.03l0.74,0.18l0.17,0.41l0.04,0.44l0.16-0.16l0.35-0.16l0.02,0.16l0.18,0.22l-0.16,0.31
				l0.15,0.2l0.25-0.01l0.27,0.12l1.02-0.02l0.2-0.14l0.14-0.32l-0.2-0.31l0.56-0.31l0.54-0.01l0.17,0l0.35,0.35l0.48,0.16
				l0.46-0.05l0.32-0.06l0.47-0.48l0.01-0.22l0.37-0.28l-0.01-0.36l0.3-0.6l0.41-0.16l0.58,0.01l0.37,0.17l0.15,0.2l0.13-0.01
				l0.08,0.14l0.14-0.16l0.05,0.09l0.1-0.11l0.64,0.17l0.05,0.35l0.32,0.26l0.06,0.39l0.21,0.35l0.22,0.02l0.01-0.18l0.25-0.36
				l0.26,0l0.73-0.38l-0.19-0.36l0.38,0.01l0.24-0.04l0.38,0.16l0.13-0.16l0.26,0.14l0.07-0.07l0.17,0.05l0.05-0.06l0.56,0.13
				l0.45,0.3l0.52,0.07l0.46-0.04l0.57-0.2l0.45-0.43l0.3-0.48l0-0.25l0.17-0.21l0.13-0.45l-0.01-0.44l-0.61-1.32l0-0.27l-0.6-0.15
				l-1.01,0.01l-0.17,0.06l-0.52-0.09l-0.17,0.26l0.01,0.15l-0.41,0.36l-0.34,0.55l-0.12,0.08l-0.57,0.86l-0.38-0.04l-0.26,0.1
				l-0.21-0.45l0.15-0.65l-0.49-0.33l-0.36-0.56l-1.1-0.77l-0.59-0.3l-0.06-0.66l0.52-0.62l0.27-0.32l0.14-0.43l0.04-0.37l0.35-0.28
				l0.39-0.17l0.17-0.15l0.1-0.19l-1.15-1.27l1.02-0.99l0.32-0.29l-0.59-0.48l0.03-0.07l-0.27-0.34l0.01-0.75l-0.1,0.08l-0.62,0.01
				l-0.08-0.15l0.07,0l-0.38-0.47l-0.69-0.33L305.51,102.33L305.51,102.33z"></path>
		</g>
		<g id="alta_sanabria">
			<path id="path558" class="st3 fase_0" d="M256.9,85.34l-0.27,0.25l0.05,0.15l-0.12,0.23l-0.23,0.08l-0.07,0.11l0.05,0.22l0.28,0.37
				l0,0.17l0.16,0.33l0.16,0.14l-0.31,0.19l-0.08,0.17l-0.23-0.51l-0.55-0.13l-0.56-0.28l-0.33-0.02l-0.23-0.1l-0.53,0.06
				l-0.45,0.22l-0.08-0.03l-0.69,0.51l-0.26,0.38l0.18,0.29l-0.5,0.66l-0.4,0.07l-0.15-0.06l-0.95,0.64l-0.21,0.28l0.1,0.29
				l-0.23,1.14l0.08,0.21l-0.48,0.66l-0.36,0.05l-0.32,0.2l-0.33,0.07l-0.2-0.05l-0.14,0.16l0.03,0.26l-0.07,0.07l0.08,0.17
				l-0.02,0.23l-0.1,0.05l-0.02,0.11l0.09,0.11l-0.06,0.16l-0.13-0.01l-0.02-0.1l-0.12,0.04l-0.2,0.25l0.03,0.28l0.07,0.07
				l-0.11,0.11l0.04,0.15l-0.04,0.05l-0.38-0.06l-0.18,0.21l-0.13-0.1l-0.55,0.48l-0.02,0.14l0.12,0.44l0.36,0.13l0.38,0.44
				l0.73,0.07l0.81-0.2l0.07,0.86l0.34,0.29l0.14,0.35l-0.21,0.18l-0.13,0.46l-0.27,0.34l-0.13,0.48l-0.31,0.17l0.02,0.12
				l-0.16,0.28l-0.05,0.2l0.05,0.18l0.28,0.52l0.2-0.06l0.84,0.25l0.3,0.82l0.25,0.48l0.1,0.04l0.29-0.13l0.39,0.12l0.51,0.01
				l0.81,0.24l0.28-0.03l0.45-0.2l0.2-0.18l1.02,0.28l0.35-0.16l0.23,0.08l0.36-0.13l0.44-1.28l-0.05-1.2l0.24,0.09l0.28-0.73
				l-0.03-0.31l-0.21-0.36l0.09-0.96l-0.21-0.53l-0.42-0.59l-0.24-0.68l0.56-0.19l0.19,0.06l0.76-0.04l-0.36-0.21l0.09-0.03
				l-0.05-0.7l0.19-0.56l-0.09-0.25l0.35-0.3l-0.19-0.21l-0.37-0.07l-0.19-0.16l-0.2-0.53l0.05-0.48l0.81-0.92l-0.23-0.75l0.15-0.35
				l0.6-0.59l0.43,0.11l0-0.56l0.21-0.29l0.27-0.18l0.97-0.68l0.11-0.11l0.03-0.15l-0.1-0.2l-0.09,0.04l-0.23-0.08l-0.09-0.21
				l0-0.12l0.1-0.08l-0.07-0.21l-0.33-0.25l-0.65,0.08l-0.28-0.37l-0.74-0.43L256.9,85.34L256.9,85.34z"></path>
		</g>
		<g id="puebla_de_sanabria">
			<path id="path476" class="st3 fase_0" d="M262.93,87.32l-0.38,0.02l-0.28,0.21l-0.18-0.04l-0.48,0.07l-0.14-0.11l-0.6-0.01l-0.4,0.27
				l-0.19,0.02l-0.31-0.25l-0.33,0.04l-0.16-0.19l-0.11,0.11l-0.97,0.68l-0.27,0.18l-0.21,0.29l0,0.56l-0.43-0.11l-0.6,0.59
				l-0.15,0.35l0.23,0.75l-0.81,0.92l-0.05,0.48l0.2,0.53l0.19,0.16l0.37,0.07l0.19,0.21l-0.35,0.3l0.09,0.25l-0.19,0.56l0.05,0.7
				l-0.09,0.03l0.36,0.21l-0.76,0.04l-0.19-0.06l-0.56,0.19l0.24,0.68l0.42,0.59l0.21,0.53l-0.09,0.96l0.21,0.36l0.03,0.31
				l-0.28,0.73l0.32,0.12l0.49-0.02l0.3,0.07l0.39,0.2l0.07,0.5l0.66,1.67l0.98,0.34l0.48-0.09l0,0.06l0.62,0.23l2.58-0.47
				l0.42,0.12l0.33-0.07l0.15-0.29l0.29-0.12l0.28,0.04l0.42-0.79l0.02-0.26l0.44,0.33l0.07-0.26l0.26-0.36l0.63-0.04l0.5,0.32
				l0.38,0.03l-0.04-0.75l0.08-0.11l0-0.38L267.5,99l0.27-0.33l0.23-0.24l0.28-0.64l-0.12-0.57l0.18-0.45l-0.1-0.54l0.13-0.63
				l-0.22,0.01l-0.52-0.26l0.1-0.4l0.24-0.3l0.35-0.79l0.11,0.65l0.24,0.69l0.22,0.35l-0.03,0.13l0.46,0.06l0.53,0.21l0.52-0.19
				l-0.07-0.28l-0.19-0.13l-0.51-1.43l0.44,0.28l0.06-0.5l0.29-0.18l0.07-0.15l0.68,0.16l-0.17-0.23l0.03-0.15l0.15-0.15l-0.06-0.5
				l0.11-0.19l-0.04-0.27l0.21-0.52l0.15-0.12l-0.06-0.46l-0.66-0.61l-1.17-0.68l-0.16-0.55l-0.06,0.07l-0.49-0.1l-0.06,0.09
				l-0.22,0.02l-0.09-0.1l-0.34-0.03l0.16-0.26l-0.09-0.08l0.02-0.18l-0.17-0.16l-0.35-0.2l-0.09,0.02l-0.2-0.17l-0.15,0.03
				l-1.05-0.19l-0.09-0.05l-0.65,0.12l-0.56-0.16l-0.24,0.08l-0.14,0.16l-0.27-0.03l-0.13-0.1l-0.25,0.14l-0.12-0.06l-0.1-0.27
				l-0.19-0.15l-0.26,0.01L262.93,87.32L262.93,87.32z"></path>
		</g>
		<g id="mombuey">
			<path id="path484" class="st3 fase_0" d="M276.13,87.9l-0.34,0.23l-0.11-0.06l-0.24,0.2l-0.5-0.28l-0.22,0.25l-1.35-0.05l-0.04-0.09
				l-0.64,0.13l-0.04,0.16l-0.48,0.11l-0.51-0.17l-0.2-0.16l-0.41,0.19l-0.17,0.26l-0.71-0.19l-0.06,0.1l-0.3,0.12l-0.32,0.5
				l0.16,0.55l1.17,0.68l0.66,0.61l0.06,0.46l-0.15,0.12l-0.21,0.52l0.04,0.27l-0.11,0.19l0.06,0.5L271,93.19l-0.03,0.15l0.17,0.23
				l-0.68-0.16l-0.07,0.15l-0.29,0.18l-0.06,0.5l-0.44-0.28l0.51,1.43l0.19,0.13l0.07,0.28l-0.52,0.19l-0.53-0.21l-0.46-0.06
				l0.03-0.13l-0.22-0.35l-0.24-0.69l-0.11-0.65l-0.35,0.79l-0.24,0.3l-0.1,0.4l0.52,0.26l0.22-0.01l-0.13,0.63l0.1,0.54l-0.18,0.45
				l0.12,0.57L268,98.42l-0.23,0.24L267.5,99l-0.23,0.52l0,0.38l-0.08,0.11l0.04,0.75l-0.38-0.03l-0.5-0.32l-0.63,0.04l-0.26,0.36
				l-0.07,0.26l0.15,0.06l1,0.87l0.13,0.46l-0.01,0.28l0.28,0.3l1.42,0.08l0.67-0.05l0.88,0.1l-0.06-0.17l0.04-0.26l0.89,0.24
				l0.42,0.32l0.45,0l0.07-0.13l0.18-0.04l0.19,0.06l0.25-0.08l0.19,0.16l0.54-0.86l0.29,0.19l0.26,0l-0.02-1.01l0.03-0.08
				l0.28-0.09l0.61,0.81l0.36,0.33l0.14,0.07l0.11-0.13l0.45,0.27l0.05,0.09l-0.1,1.52l0.4,0.21l0.62,0.01l0.81,0.43l0.22-0.08
				l0.11-0.15l0.74,0.09l0.11-0.12l0.56-1.29l-0.47-0.12l0.12-0.75l0.11-0.11l0.54-1.25l0.08-0.34l0.03-0.3l-0.61,0.05l-0.18-0.09
				l0.06-0.26l0.4-0.46l0.39,0.12l0.13-0.21l0.16-0.01l0.17,0.21l0.18,0.06l0.12-0.19l0.29,0.23l0.2,0.29l0.14,0.06l0.59-0.04
				l0.56,0.2l0.12-0.03l0.16-0.24l0.52,0.01l-0.08-0.37l0.08-0.09l-0.39-0.2l-0.55,0.06l-0.31-0.19l0,0.08l-0.94-0.26l0.02-0.42
				l0.15-0.15l-0.26-0.44l-0.14-0.06l-0.16-0.35l0.09-0.09l0.32-0.9l-0.65-0.4l-0.27-0.05l-0.33-0.4l0.07-0.08l0.01-0.87l-0.21-1.02
				l0.11-0.07l0.36-0.25l0.33-0.81l-0.04-0.31l0.06-0.18l0.25-0.1l0.2-0.53l-0.42-0.81l-0.26-0.16l-0.29-0.59l-0.66-0.3l-0.48,0.03
				l0.09-0.32l-1.05-0.32l-0.64,0.05l-0.02-0.23l-0.17-0.07l-0.19-0.46l-0.16-0.58L276.13,87.9L276.13,87.9z"></path>
		</g>
		<g id="carbajales_de_alba">
			<path id="path704" class="st3 fase_1" d="M287.49,110.87l-0.07,0.46l0.16,0.42l-0.27,0.54l-0.01,0.59l-0.06,0.1l0.21,0.19l-0.59,0
				l-0.67,0.27l-0.36-0.23l-0.49-0.02l-0.45-0.32l-0.1,0.22l-0.65,0.59l-0.27,0.57l0.08,0.14l-0.15,0.33l-0.45,0.47l-0.05,0.3
				l0.23,0.26l0.34,0.21l-0.15-0.01l-0.08,0.11l-0.02,0.29l0.76,0.9l0.11,0.33l0.25,0.38l1.27,0.33l0.12,0.11l0.1,0.31l-0.07,0.12
				l0.01,0.36l0.18,0.41l-0.18,0.91l0.42-0.07l0.11-0.11l0.33,0.17l0.09-0.03l0.62,0.39l0.27,0.03l0.11,0.18l0.28,0.06l0.12,0.15
				l0.2-0.04l0.07,0.16l0.53,0.05l0.15-0.06l0.09-0.28l0.38-0.41l0.64-0.27l0.35-0.19l0.27,0.37l0.15,0.06l0.28-0.02l0.27-0.22
				l0.16-0.27l-0.35-0.5l-0.03-0.17l0.15-0.63l0.36-0.08l0.48-0.77l0.25-0.27l0.04-0.05l0.27-0.07l0.22,0.04l0.62,0.42l0.37-0.12
				l0.08-0.22l-0.36-0.52l-0.23-1.26l0.2-0.21l0.29-0.1l0.13-0.51l0.19-0.05l0.44,0.25l0.17-0.03l0.03-0.5l-1.31-0.54l-0.08,0.18
				l-0.81-0.22l-0.01-0.11l-0.32-0.19l-0.11,0.1l-0.69-0.2l0.08-0.2l0.16,0.03l0.25-0.1l0.18-0.2l0.22-0.43l0.03-0.46l0.19-0.26
				l-1.01-0.46l-0.93-0.61l-0.54,0.48l-0.89,0.06l-1.44-0.39L287.49,110.87L287.49,110.87z"></path>
		</g>
		<g id="benavente_sur">
			<path id="path586" class="st3 fase_0" d="M302.24,91.37l-0.49,0.1l-0.05-0.08l-0.25,0.19l0.06,0.14l0.24,0.17l0.12,0.66l0.21-0.02
				l0.22,0.16l0,0.37l0.09,0.85l-0.64,0.43l0.27,0.61l-0.29,1.62l0.07,0.05l0.04,0.32l0,0.32l-0.07,0.12l0.23,0.04l0.58,0.28
				l0.02,0.15l0.24,0l0.19,0.18l0.18,0l0.31-0.16l0.34,0.01l0.02,0.17l0.1,0.09l0.54,0.22l-0.03,0.12l-0.38,0.47l-0.37,0.01
				l-0.04,0.12l0.06,0.22l-0.35,0.36l0.17,0.14l-0.06,0.19l-0.35,0.15l-0.15-0.24l-0.15,0.03l-0.3-0.21l-0.23-0.08l-0.36,0.02
				l-0.29,0.22l0.25,0.22l0.01,0.15l-0.17,0.23l0,0.12l-1.2,0.41l-0.14-0.05l-0.11,0.1l-0.08-0.02l-0.28-0.1l-0.07,0.01l0.01,0.14
				l-0.07,0.04l-0.21,0.64l-0.25,0.11l-0.21,0.44l-0.31-0.13l-0.41-0.46l-0.39,0.39l-0.39-0.08l-0.13,0.05l0.04,0.15l-0.15-0.03
				l-0.14,0.06l-0.02,0.15l-0.25-0.02l-0.42,0.16l-0.07-0.29l0.82-0.67l0.06-0.16l-0.67-1.05l-0.27,0.15l-0.29-0.03l-0.17,0.07
				l-0.16,0.26l-0.38,0l0.03,0.15l-0.08,0.04l0.06,1.09l-0.17,0.24l0.34,0.48l0.05,0.18l-0.94,0.49l-0.38-0.05l-0.27,0.22
				l-0.34,0.09l-0.68,0.97l-0.46,0.33l0.04,0.14l0.57-0.14l0.45,0.15l1.3,0.36l0.38,0.19l0.41,0.09l1.68,0.39l0.29-0.05l0.04,0.18
				l0.34,0.46l0.1,0.4l-0.08,0.29l0.48,0.54l0.76,0.2l0.23-0.02l0.84-0.33l0.91-0.06l0.42-0.2l0.49-0.39l-0.12-0.13l0.69-0.66
				l0.3-0.41l-0.06-0.12l-0.11-0.99l-0.17-0.48l0.56,0.51l0.22-0.37l-0.41-0.3l0.1-0.05l-0.06-0.14l0.18-0.32l0.5,0.21l-0.04-0.18
				l0.51-0.45l0.08-0.07l0.07,0.06l0.11-0.14l0.02-0.61l0.05-0.12l0.04-0.63l0.63-0.25l0.27,0.04l1.04-0.36l-0.01-0.4l0.1-0.1
				l0.18-0.14l0.69,0.28l0.38-0.17l-0.11-0.27l0.21-0.16l0.28-0.4l0.09,0.05l0.27-0.37l0.34-0.02l0.44-0.27l0.17-0.04l0.2-0.16
				l0.03-0.28l0.27-0.1l-0.04-0.39l-0.6-0.66l-0.02-0.22l-0.18-0.18l0.25-0.3l-0.38-0.36l-0.01-0.19l-0.47-0.45l0.2-0.27l0.05-0.21
				l-0.3-0.56l-0.38-0.41l0.02-0.32l-0.12-0.35l-0.25-0.39l-0.66,0.26l0.06,0.07l-0.1,0.3l0.01,0.34l-0.17,0.38l-0.05,0.47
				l-0.13,0.36l-0.19,0.16l-0.13,0.25l-0.17-0.18l-0.2,0.07l-0.2-0.36l-0.12-0.05l-0.07,0.06l-0.03,0.32l-0.26,0.4l-0.09-0.1
				l0.22-0.71l-0.12-0.28l-0.12-0.07l0.03-0.17l0.01-0.09l-0.12-0.07l-0.1,0.14l-0.14-0.03l0.02-0.14l-0.26-0.06l-0.03,0.05
				l-0.02-0.06l-0.08,0.06l-0.75-0.22l-0.38,0.12l-0.12-0.13l-0.05-0.07l0.57-0.41l0.19-0.66l-0.09-0.2l-0.67-0.72l-0.29-0.54
				l-0.84-0.25l0.02-0.23L302.24,91.37L302.24,91.37z"></path>
		</g>
		<g id="tabara">
			<path id="path458" class="st3 fase_1" d="M279.85,99.73l-0.16,0.01l-0.13,0.21l-0.39-0.12l-0.4,0.46l-0.06,0.26l0.18,0.09l0.61-0.05
				l-0.03,0.3l-0.08,0.34l-0.54,1.25l-0.11,0.11l-0.12,0.75l0.47,0.12l-0.56,1.29l-0.11,0.12l-0.74-0.09l-0.11,0.15l-0.22,0.08
				l0.42,0.28l0.16,0.19l-0.17,0.33l-0.01,0.43l-0.22,0.87l0.03,0.15l0.27-0.01l0.08,0.21l0.27,0.26l0.11,0l0.6,0.42l0.31-0.22
				l0.43-0.13l0.15-0.15l0.27,0.27l0.55,0.84l-0.24,0.58l-0.21,0.31l-0.12,0.06l-0.04,0.25l0.12,0l0.46,0.45l0.2,0.08l0.19-0.03
				l0.56-0.22l0.49,0.04l0.5,0.19l1.67,0.99l0.78,1.21l-0.09,0.2l0.45,0.32l0.49,0.02l0.36,0.23l0.67-0.27l0.59,0l-0.21-0.19
				l0.06-0.1l0.01-0.59l0.27-0.54l-0.16-0.42l0.07-0.46l0.88,0.42l1.44,0.39l0.89-0.06l0.54-0.48l0.93,0.61l1.01,0.46l-0.19,0.26
				l-0.03,0.46l-0.22,0.43l-0.18,0.2l-0.25,0.1l-0.16-0.03l-0.08,0.2l0.69,0.2l0.11-0.1l0.31,0.19l0.01,0.11l0.81,0.22l0.08-0.18
				l1.31,0.54l0.11-0.17l0.31-0.02l0.22-0.17l0.13-0.01l0.13,0.1l0.1,0.25l-0.01,0.34l0.06,0.06l0.14-0.07l0.35-0.98l0.01-0.32
				l-0.46-0.17l0.1-0.28l0.24-0.04l0.06-0.1l-0.11-0.28l-0.28-0.18l-0.59-0.26l-0.2-0.26l0.14-0.64l-0.02-0.48l0.14-0.31l0.46-0.4
				l0.04,0.03l0.32-0.53l0.38-0.87l-0.06-0.82l0.09-0.3l0.32-0.34l0.05-0.23l0.56,0.12l0.08-0.29l-0.1-0.4l-0.34-0.46l-0.04-0.18
				l-0.29,0.05l-1.68-0.39l-0.41-0.09l-0.38-0.19l-1.3-0.36l-0.45-0.15l-0.57,0.14l-0.04-0.14l0.46-0.33l0.68-0.97l0.34-0.09
				l0.27-0.22l0.38,0.05l0.94-0.49l-0.05-0.18l-0.34-0.48l0.17-0.24l-0.06-1.09l-0.83-0.11l-0.17,0.13l-0.23-0.05l-0.4,0.06
				l-0.06-0.09l-0.15-0.03l-1.68,0.62l0.04-0.09l-0.14,0.05l-0.36-0.4l-0.1,0.01l-0.1,0.23l-0.38,0.48l-0.69,0.23l0.4,2.58
				l-0.09,0.09l-0.17-0.11l-1.01-0.16l-0.03-0.2l-0.19-0.13l0.03-0.9l-0.29-0.52l-0.14-0.08l-0.33,0.16l-0.22-0.06l-0.45,0.44
				l-0.28,0.16l-1.86-0.49l-0.81,0.04l-0.1-0.03l-0.35-1.04l-1.03-0.98l-0.44-0.2l-0.44-0.31l-0.16,0.24l-0.12,0.03l-0.56-0.2
				l-0.59,0.04l-0.14-0.06l-0.2-0.29l-0.29-0.23L280.2,100l-0.18-0.06L279.85,99.73L279.85,99.73z"></path>
		</g>
		<g id="villalpando">
			<path id="path588" class="st3 fase_1" d="M313.71,96.74l-0.54,0.4l-0.58,0.22l-0.33,0.07l-0.05-0.08l-1.05,0.91l-0.02,0.16l0.19-0.07
				l0.27,0.16l0.21,0.35l-0.1,1.04l-0.19,0.84l-0.27-0.24l-0.45-0.21l-0.19,0.05l-0.09-0.04l0.01-0.12l-0.89-0.37l-0.15-0.59
				l-0.44,0.27l-0.34,0.02l-0.27,0.37l-0.09-0.05l-0.28,0.4l-0.21,0.16l0.11,0.27l-0.38,0.17l-0.69-0.28l-0.18,0.14l-0.1,0.1
				l0.01,0.4l-1.04,0.36l-0.27-0.04l-0.63,0.25l-0.04,0.63l0.85-0.12l0.28,0.55l0.69,0.33l0.38,0.47l-0.07,0l0.08,0.15l0.62-0.01
				l0.1-0.08l-0.01,0.75l0.27,0.34l-0.03,0.07l0.59,0.48l-0.32,0.29l-1.02,0.99l1.15,1.27l-0.1,0.19l-0.17,0.15l-0.39,0.17
				l-0.35,0.28l-0.04,0.37l-0.14,0.43l-0.27,0.32l-0.52,0.62l0.06,0.66l0.59,0.3l1.1,0.77l0.36,0.56l0.49,0.33l-0.15,0.65l0.21,0.45
				l0.26-0.1l0.38,0.04l0.57-0.86l0.12-0.08l0.34-0.55l0.41-0.36l-0.01-0.15l0.17-0.26l0.52,0.09l0.17-0.06l1.01-0.01l0.6,0.15
				l-0.03-0.16l0.86-0.93l0.84-0.16l0.32-0.45l0.32-0.01l0.29-0.03l0.15,0.09l0.09-0.11l0.16-0.04l0.1-0.15l1.39-1.06l0.11-0.51
				l-0.42-0.26l-0.14,0.08l-0.18-0.07l-0.14,0.11l-0.32-0.07l-0.05,0.06l-0.76-0.18l-0.12-0.11l0.1-0.23l0.29-0.24l-0.07-0.06
				l0.62-0.92l-0.12-0.33l0.17-0.17l-0.25-0.84l0.04-0.57l0.4-0.74l0.19-0.22l0.09-0.47l-0.21-0.12l-0.45-0.55l-0.44-0.28l-0.58,0
				l0.02-0.14l0.13-0.13l1-0.55l0.29-0.46l0.45-0.36l-0.02-0.16l0.15-0.17l0.5-0.32l0.39-0.1l0.35,0.08l0-0.07l-0.24-0.16
				l-0.82,0.31l-0.91-0.66l0.04-0.09l-0.29-0.54l-0.16,0.01l-0.18-0.21l-0.22-0.66l-0.09-0.08l-0.95-0.12l-0.78-0.58l-0.33-0.34
				l0-0.1L313.71,96.74L313.71,96.74z"></path>
		</g>
		<g id="santibanez_vidrial">
			<path id="path490" class="st3 fase_1" d="M282.3,90.09l-0.41,0.41l-1.7,0.54l0.42,0.81l-0.2,0.53l-0.25,0.1l-0.06,0.18l0.04,0.31
				l-0.33,0.81l-0.36,0.25l-0.11,0.07l0.21,1.02L279.54,96l-0.07,0.08l0.33,0.4l0.27,0.05l0.65,0.4l0.78-0.34l3.17-2l0.26-0.34
				l0.21-0.11l0.26,0.12l0.04,0.82l-0.11,0.18l0,0.19l0.24,0.32l0.63,0.31l0.13,0.15l0.14,0.01l0.2,0.29l0.35,0.35l0.11-0.05
				l0.3,0.13l0.02,0.25l0.25,0.13l0.04,0.13l0.59-0.11l0.77,0.05l0.01,0.44l0.8,0.26l0.44-0.01l-0.02,0.16l0.5-0.09l0.28,0.02
				l1.42-0.08l0.27-0.51l-0.09-0.15l0.6-0.54l0.01-0.37l0.15-0.16l0.08-0.3l0.34-0.45l0.18-0.44l-0.66-0.25l-0.6-0.38l-0.6-0.34
				l-0.02-0.11l-0.61-0.49l-0.05-0.24l0.25-0.08l0.33,0.1l1.32,0.71l0.26-0.11l0.17-0.53l0.44-0.35l0.21-0.36l-0.34-0.22l-0.73-0.99
				l0.15-0.37l-0.39-0.25l-0.28-0.33l-0.35,0.65l-0.22,0.18l-0.61-0.01l-0.2,0.07l-0.17-0.1l-0.5-0.04l-0.17,0.37l-0.05,0.61
				l-0.13-0.02l-0.17-0.17L289.82,92l-0.63-0.71l-0.7-0.38l-0.38-0.12l-0.49-0.27l-1.57-0.14l-0.33-0.09l-0.08,0.15l-0.49-0.14
				l-0.27,0.04l-0.38-0.06l-0.78,0.17l-0.51-0.31l-0.25,0.32l-0.43-0.13L282.3,90.09L282.3,90.09z"></path>
		</g>
		<g id="camarzana_de_tera">
			<path id="path544" class="st3 fase_0" d="M285.15,94.14l-0.21,0.11l-0.26,0.34l-3.17,2l-0.78,0.34l-0.32,0.9l-0.09,0.09l0.16,0.35
				l0.14,0.06l0.26,0.44l-0.15,0.15l-0.02,0.42l0.94,0.26l0-0.08l0.31,0.19l0.55-0.06l0.39,0.2l-0.08,0.09l0.08,0.37l-0.52-0.01
				l0.44,0.31l0.44,0.2l1.03,0.98l0.35,1.04l0.1,0.03l0.81-0.04l1.86,0.49l0.28-0.16l0.45-0.44l0.22,0.06l0.33-0.16l0.14,0.08
				l0.29,0.52l-0.03,0.9l0.19,0.13l0.03,0.2l1.01,0.17l0.17,0.11l0.09-0.09l-0.4-2.58l0.69-0.23l0.38-0.48l0.1-0.24l0.1-0.01
				l0.36,0.4l0.14-0.05l-0.04,0.09l1.68-0.62l0.15,0.03l0.06,0.09l0.4-0.06l0.23,0.05l0.17-0.13l0.83,0.11l0.08-0.04l-0.03-0.15
				l0.38,0l0.16-0.26l0.17-0.07l0.29,0.03l0.27-0.15l-0.18-0.15l-0.1-0.24l-0.16-0.06l-0.27-0.02l-0.63-0.69l-0.37-0.25l-1.13-0.2
				l0.62,0.01l-0.02-0.25l-0.28-0.08l-0.06,0.06l-0.55-0.09l-0.01,0.05l-0.17-0.09l-0.09-0.16l-0.8-0.06l-1.42,0.08l-0.28-0.02
				l-0.5,0.09l0.02-0.16l-0.44,0.01l-0.8-0.26l-0.01-0.44l-0.77-0.05l-0.59,0.11l-0.04-0.13l-0.25-0.13l-0.02-0.25l-0.3-0.13
				l-0.11,0.05l-0.35-0.35l-0.2-0.29l-0.14-0.01l-0.13-0.15l-0.63-0.31l-0.24-0.32l0-0.19l0.11-0.18l-0.04-0.82L285.15,94.14
				L285.15,94.14z"></path>
		</g>
		<g id="benavente_norte">
			<path class="st3 fase_0" d="M303.97,98.13l-0.1-0.09l-0.02-0.17l-0.34-0.01l-0.31,0.16l-0.18,0l-0.19-0.18l-0.24,0l-0.02-0.15
				l-0.58-0.28l-0.23-0.04l0.07-0.12l0-0.32l-0.04-0.32l-0.07-0.05l0,0l0.29-1.62l-0.27-0.61l0.64-0.43l-0.09-0.85l0-0.37
				l-0.22-0.16l-0.21,0.02l-0.81-0.02l-0.84,0.31l-0.8,0.07l-0.24,0.02l-0.17-0.14l-0.19-0.29l0.04-0.16l-0.17-0.66v0l-0.17,0.12
				l-0.42-0.09l-0.41,0.04l-0.32,0.1l-0.13,0.33l-0.28-0.02l-0.29,0.16l-0.04-0.17l0.07-0.11l-0.08-0.04l-0.23,0.22l-0.81,0.39
				l-0.24,0.03l-0.11,0.33l-0.16,0.1l-0.55-0.28l-0.21,0.36l-0.44,0.35l-0.17,0.53l-0.26,0.11l-1.32-0.71l-0.33-0.1l-0.25,0.08
				l0.05,0.24l0.61,0.49l0.02,0.11l0.6,0.34l0.6,0.38l0.66,0.25l-0.18,0.44l-0.34,0.45l-0.08,0.3l-0.15,0.16l-0.01,0.37l-0.6,0.54
				l0.09,0.15l-0.27,0.51l0.8,0.06l0.09,0.16l0.17,0.09l0.01-0.05l0.55,0.09l0.06-0.06l0.28,0.08l0.02,0.25l-0.62-0.01l1.13,0.2
				l0.37,0.25l0.63,0.69l0.27,0.02l0.16,0.06l0.1,0.24l0.18,0.15l0.67,1.05l-0.06,0.16l-0.82,0.67l0.07,0.29l0.42-0.16l0.25,0.02
				l0.02-0.15l0.14-0.06l0.15,0.03l-0.04-0.15l0.13-0.05l0.39,0.08l0.39-0.39l0.41,0.46l0.31,0.13l0.21-0.44l0.25-0.11l0.21-0.64
				l0.07-0.04l-0.01-0.14l0.07-0.01l0,0l0.28,0.1l0.08,0.02l0.11-0.1l0.14,0.05l1.2-0.41l0-0.12l0.17-0.23l-0.01-0.15l-0.25-0.22
				l0.29-0.22l0.36-0.02l0.23,0.08l0.3,0.21l0.15-0.03l0.15,0.24l0.35-0.15l0.06-0.19l-0.17-0.14l0.35-0.36l-0.06-0.22l0.04-0.12
				l0.37-0.01l0.38-0.47l0.03-0.12L303.97,98.13z"></path>
		</g>
		<g id="toro">
			<path id="path730" class="st3 fase_0" d="M314.47,113.55l-0.18,0.13l-0.06,0.3l-0.15,0.08l-0.13,0.45l-0.17,0.21l0,0.25l-0.3,0.48
				l-0.45,0.43l-0.57,0.2l-0.46,0.04l-0.52-0.07l-0.45-0.3l-0.56-0.13l-0.05,0.06l-0.17-0.05l-0.07,0.07l-0.26-0.14l-0.13,0.16
				l-0.38-0.16l-0.24,0.04l-0.38-0.01l0.19,0.36l-0.73,0.38l-0.26,0l-0.25,0.36l-0.01,0.18l-0.22-0.02l0.01,0.03l0.3,0.21
				l-0.17,0.87l0.02,0.52l0.62,0.47l0.23,0.16l0.15-0.01l-0.03,0.15l0.1-0.01l0,0.24l0.09,0.23l0.19,0.06l-0.13,0.28l0.16,0.06
				l-0.08,0.28l0.36,0.18l0.19,0.04l0.19,0.18l-0.03,0.13l0.43,0.27l-0.2,0.06l-0.04,0.08l0.11,0.04l0.09,0.62l-0.23,0.15
				l-0.35-0.01l-0.03-0.06l-0.22,0.13l-0.45,0.04l-0.44-0.29l-0.16,0.14l-0.22,0l-0.2-0.48l-0.46,0.55l0.14,0.11l-0.5,1.37
				l-0.15,0.83l0.06,0.16l-0.18,0.41l-0.04,0.39l0.16,0.46l-0.07,0.52l-0.34,0.32l0.12,0.22l0.19,0.24l-0.06,0.16l0.08,0.14
				l0.2,0.01l0.19,0.24l0.2-0.03l0.06,0.23l0.21-0.02l-0.16,0.47l0.03,0.17l0.13-0.04l-0.13,0.23l0.11,0.33l0.12,0.08l-0.06,0.14
				l0.18,0.22l-0.15,0.01l-0.08,0.14l-0.07,0.01l0.01,0.09l0.2,0.3l0.08-0.01l0.07,0.55l0.17,0.07l-0.11,0.34l0.17,0.15l-0.07,0.06
				l0.2,0.52l-0.07,0.21l0.39,0.53l0.29-0.48l0.17,0.09l0.26,0.34l0.34,0.28l0.03,0.31l-0.2,0.24l0.11,0.14l0.14,0.03l0.51-0.41
				l0.51,0.27l0.79,0.07l0.33-0.37l-0.14-0.27l0.02-0.26l-0.07-0.17l0.37-0.37l-0.03-0.11l0.15-0.33l0.02-0.38l0.45,0.01l0.35,0.23
				l0.2-0.24l0.22,0.14l0.19-0.12l0.38,0.19l0.01,0.84l0.12,0.02l0.43,0.35l0.89-0.3l0.39,0.05l0.47-0.77l0.04-0.3l0.38-0.39
				l0.15-0.29l-0.44-0.37l-0.85-0.26l0.24-0.57l0.02-0.42l0.23-0.17l-0.2-0.16l0.3-0.28l-0.2-0.23l-0.22-0.01l-0.1-0.08l-0.29-0.97
				l0.11-0.11l0.2,0.27l0.09,0.06l0.12-0.04l-0.01-0.11l0.34-0.12l0.52-0.55l0.48-0.53l0.25-0.07l0.09,0.04l0.55-0.3l0.62,0.14
				l0.32-0.32l0.5-0.54l0.32,0.11l0.2-0.58l-0.17-0.13l-0.05-0.23l-0.21-0.29l-0.33-0.25l-0.01-0.09l-0.39-0.02l0.02-0.06
				l-0.29-0.19l-0.35,0.08l-0.06-0.13l0.06-0.35l-0.32-0.16l0.05-0.11l-0.16-0.24l-0.4-0.31l0.49-0.55l0.52-0.29l0.05-0.16
				l-0.17-0.05l-0.05-0.13l0.06-0.26l-0.28-0.21l-0.61-0.21l-0.54,0.03l-0.11-0.18l-0.47,0.04l-0.35-0.27l0.19-0.49l-0.04-0.27
				l0.07-0.22l0.16-0.14l-0.21-0.36l0.05-0.31l-0.02-0.07l0.07-0.65l-0.05-0.41l-0.18-0.62l-0.41-0.27l-0.1-0.66l-0.2-0.16
				l-0.5,0.13l0.08-0.93l-0.22-0.63L314.47,113.55L314.47,113.55z"></path>
		</g>
		<g id="zamora_norte">
			<path id="path736" class="st3 fase_0" d="M304.67,115.02l-0.41,0.16l-0.3,0.6l0.01,0.36l-0.37,0.28l-0.01,0.22l-0.47,0.48l-0.32,0.06
				l-0.46,0.05l-0.48-0.16l-0.35-0.35l-0.17,0l-0.54,0.01l-0.56,0.31l0.2,0.31l-0.14,0.32l-0.2,0.14l-1.02,0.02l-0.27-0.12
				l-0.25,0.01l-0.15-0.2l0.16-0.31L298.4,117l-0.02-0.16l-0.35,0.16l-0.16,0.16l-0.04-0.44l-0.17-0.41l-0.74-0.18l-1.48,0.03
				l-0.9-0.28l-0.1-0.22l-0.29,0.1l-0.2,0.21l0.23,1.26l0.36,0.52l-0.08,0.22l-0.37,0.12l-0.62-0.42l-0.22-0.04l-0.27,0.07
				l-0.04,0.05l-0.25,0.27l-0.48,0.77l-0.36,0.08l-0.15,0.63l0.03,0.17l0.35,0.5l-0.16,0.27l-0.27,0.22l-0.28,0.02l-0.15-0.06
				l-0.27-0.37l-0.35,0.19l-0.64,0.27l-0.38,0.41l-0.09,0.28l-0.15,0.06l-0.53-0.05l-0.07-0.16l-0.2,0.04l-0.12-0.15l-0.28-0.06
				l-0.11-0.18l-0.27-0.03l-0.62-0.39l-0.09,0.03l-0.33-0.17l-0.11,0.11l-0.42,0.07l-0.84,0.27l0.09,0.42l0.34,0.52l-0.29,0.1
				l-0.41,0.47l-0.17,0.03l-0.18,0.38l-0.28,0.3l-0.08,1.54l0.15,0.21l0.36,0.1l0.23-0.09l0.17,0.07l0.15,0.22l-0.05,0.73l-0.3,0.53
				l0.08,0.14l0.18,0.06l0.05-0.19l0.18-0.14l0.41,0.65l0.47,0.35l0.24,0.23l0.21,0.01l0.59,0.28l0.61,0.41l0.64,0.1l0.67-0.23
				l0.59,0.33l0.6,0.07l0.39,0.29l0.27,0.34l0.25,0.14l0.28,0.3l0.26,0.12l0.23,0.01l-0.03-0.3l0.13-0.17l0.36,0.03l0.51-0.24
				l0.12-0.23l-0.3-0.63l0.14-0.22l0.41-0.14l0.04-0.17l-0.13-0.32l-0.06-0.77l0.22-0.65l0.38-0.65l0.25-0.29l-0.06-0.18l-0.29-0.23
				l-0.31-0.39l-0.18,0.29l-0.11,0.03l-0.11-0.06l-0.12,0.04l-0.07-0.09l-0.43,0.06l-0.36-0.06l-0.01,0.06l-0.2,0.05l-0.3-0.09
				l-0.12-0.16l-0.2-0.02l0-0.16l-0.44-0.41l-0.04-0.13l-0.4-0.08l-0.08,0.05l-0.23-0.22l-0.52-0.12l0.36-0.72l-0.03-0.19l0.4-0.16
				l0.5-0.03l1.07,0.15l0.29,0.09l0.21,0.18l0.26,0.04l0.34-0.33l0.16,0.01l0.35,0.35l0.29,0.03l0.12,0.05l0.12,0.4l0.19,0.29
				l0.03,0.32l0.26,0.15l0.52-0.13l0.3-0.29l0.5-0.18l0.22,0.04l0.39,0.63l0.08,0.5l0.5,0.22l0.73-0.47l0.34,0l0.1,0.37l0.23-0.07
				l0.38,0.22l0.24,0l0.08-0.11l0.17,0l0-0.14l0.34-0.1l0.1-0.43l0.38,0.18l0.26-0.26l0.2,0.19l0.21-0.14l0.11,0.06l0.08-0.18
				l0.15-0.05l0.33,0.13l0.54-0.58l0.3-0.04l-0.06,0.49l0.1,0.43l0.29,0.65l0.22,0.11l0.46-0.03l0.14,0.07l-0.01,0.27l-0.11,0.14
				l-0.06-0.02l-0.06,0.17l-0.32,0.19l0.02,0.16l0.43,0.23l0.18-0.11l0.26,0.09l0.3,0.24l0.3,0.09l0.26-0.02l0.23-0.32l0.54-0.25
				l0.2-0.35l0.25-0.42l0.19,0.09l0.31-0.48l0.4-0.01l0.15-0.83l0.5-1.37l-0.14-0.11l0.46-0.55l0.2,0.48l0.22,0l0.16-0.14l0.44,0.29
				l0.45-0.04l0.22-0.13l0.03,0.06l0.35,0.01l0.23-0.15l-0.09-0.62l-0.11-0.04l0.04-0.08l0.2-0.06l-0.43-0.27l0.03-0.13l-0.19-0.18
				l-0.19-0.04l-0.36-0.18l0.08-0.28l-0.16-0.06l0.13-0.28l-0.19-0.06l-0.09-0.23l0-0.24l-0.1,0.01l0.03-0.15l-0.15,0.01l-0.23-0.16
				l-0.62-0.47l-0.02-0.52l0.17-0.87l-0.3-0.21l-0.01-0.03l-0.21-0.35l-0.06-0.39l-0.32-0.26l-0.05-0.35l-0.64-0.17l-0.1,0.11
				l-0.05-0.09l-0.14,0.16l-0.08-0.14l-0.13,0.01l-0.15-0.2l-0.37-0.17L304.67,115.02L304.67,115.02z"></path>
		</g>
		<g id="bermillo_de_sayago">
			<path id="path786" class="st3 fase_0" d="M282.35,121.49l-0.59,0.02l-0.11,0.22l-0.41,0.06l-0.12,0.06l-0.54,0.39l-0.35,0.4
				l-0.25,0.52l-0.01,0.41l-0.4,0.48l-0.36,0.63l-0.18,0.11l-0.11,0.19l-0.51,0.16l0.22,0.71l-0.1,0.13l-0.17,0.1l-0.31-0.09
				l-0.08,0.04l-0.07,0.17l0.24,0.41l-0.34,0.01l-0.13,0.15l-0.39,0.2l-0.1,0.15l0.03,0.44l-0.09,0.2l-0.34,0.27l-0.36,0.5
				l-0.06,0.12l0.05,0.16l0.4,0.54l-0.17,0.3l-0.39,0.55l-0.28,0.21l-0.52,0.04l-0.16,0.21l0,0.33l0.5,0.24l0.12,0.23l0.07,0.32
				l-0.1,0.24l-0.27-0.06l-0.74,0.4l-0.38,0.08l-0.34-0.23l-0.22-0.36l0.02-0.25l-0.2,0.08l-0.26,0.26l-0.12-0.02l-0.38,0.14
				l-0.16,0.41l-0.01,0.37l0.26,0.38l0.29-0.02l0.14,0.12l-0.1,0.25l-0.31,0.21l-0.27,0.18l-0.57,0l-0.36,0.13l-0.42,0.71l0.34,0.41
				l-0.57,0.32l-0.12,0.15l-0.14,0.31l0.07,0.35l-0.57,0.4l-0.29,0.02l-0.43-0.17l-0.42,0.36l0.03,0.32l0.06,0.04l0.49,0.07
				l0.45-0.32l0.36,0.44l0.25,0l0.62,0.23l0.09-0.15l0.19-0.01l0.12,0.1l-0.03,0.27l0.59-0.15l0.35-0.01l0.35-0.37l0.42-0.03
				l0.25,0.09l0.09,0.16l0.4,0.16l0.21,0.17l0.05,0.15l0.51,0.06l0.16,0.12l0.24,0.06l0.3-0.14l0.25,0.4l0.41,0.19l0.1,0.14
				l0.42,0.14l0.44,0l0.14-0.13l0.25,0.08l0.04,0.1l0.6,0.61l0.72,0.22l0.05,0.08l0.08,0.22l0.24,0.27l0.85,0.42l0.12-0.01
				l0.32,0.27l0.07,0.06l0.52,0.28l0.57,0.06l0.84,0.53l0.37,0.4l0.33-0.34l0.08-0.34l0.13-0.04l0.48-0.48l0.38,0.12l0.3-0.12
				l0.75,0.06l0.19-0.29l0.15-0.05l0.15-0.18l0.14,0.34l0.48,0.31l0.27,0.04l0.63,0.28l0.63,0.14l-0.09,0.36l-0.29,0.19l-0.37,0.6
				l-0.03,0.68l0.26,0.61l0.49,0.17l0.29-0.06l0.71-1.09l0.24-0.07l0.21,0.11l0.11,0.2l-0.02,0.35l0.13,0.3l0.09,0.05l0.15,0.37
				l-0.01,0.16l0.31,0.13l0.09,0.19l0.08-0.01l0.25-0.21l-0.11-0.19l0.15-0.15l0.41-0.05l-0.11-0.28l0-0.11l0.2,0.05l0.04-0.1
				l-0.08-0.35l0.24-0.31l0.19-0.22l0.26,0.08l0.1-0.22l-0.09-0.22l0.05-0.07l-0.09-0.2l-0.07-0.93l-0.14-0.71l0.33-0.53l-0.01-0.15
				l0.21-0.08l0.43-0.34l-0.24-0.4l-0.24-0.92l-0.09-0.11l-0.03,0.06l-0.23-0.03l-0.53-0.29l-0.18,0.02l-0.11,0.29l-0.24,0.28
				l-0.86,0.01l-0.44,0.11l-0.1-0.18l0.02-0.29l0.13-0.07l0.08-0.27l-0.06-0.09l0.03-1.02l0.66,0.18l0.06-0.22l0.89-0.85l-0.21-0.57
				l0.25-0.02l1.14-0.84l0.72-0.41l0.04-0.38l0.01-0.07l-0.63,0.05l-0.11,0.12l-0.08-0.03l-0.26-0.52l-0.2-0.08l-0.38-0.61
				l-0.15,0.13l-0.78,0.31l-0.15,0.14l-0.49,0.05l-0.02-0.48l-0.53-0.36l-0.81-0.03l-0.02-0.23l-0.08-0.06l0.22-0.08l0.15-0.57
				l0.16-0.12l0.3-0.76l0.04-0.5l0.22-0.12l0.06-0.13l-0.11-0.49l0.14-0.78l-0.05-0.13l0.11-0.09l-0.01-0.18l-0.59-0.33l-0.67,0.23
				l-0.64-0.1l-0.61-0.41l-0.59-0.28l-0.21-0.01l-0.24-0.23l-0.47-0.35l-0.41-0.65l-0.18,0.14l-0.05,0.19l-0.18-0.06l-0.08-0.14
				l0.3-0.53l0.05-0.73l-0.15-0.22l-0.17-0.07l-0.23,0.09l-0.36-0.1l-0.15-0.21l0.08-1.54l-0.28-0.14l-0.5-0.11l-0.14-0.32l-0.7-0.2
				l-0.09-0.51L282.35,121.49L282.35,121.49z"></path>
		</g>
		<g id="corrales_del_vino">
			<path id="path818" class="st3 fase_1" d="M302.37,130.56l-0.09,0.43l-0.37,0.24l-0.04-0.17l-0.63-0.01l-0.71,0.1l-0.32,0.31
				l-0.41,0.25l-0.01,0.08l-0.16-0.15l-0.52,0.19l-0.42-0.09l-0.13,0.18l-0.44,0.24l-0.13,0.25l-0.31,0.27l-0.54,0.28l-0.24-0.12
				l-0.06-0.11l-0.04,0.22l0.13,0.14l0.01,0.14l-0.25,0.35l-0.2-0.03l-0.14,0.36l-0.11-0.1l-0.13,0.16l-0.4,0.13l-0.33,0.21
				l-0.32-0.38l-0.12-0.04l-0.11-0.53l-0.68,0.56l-0.24,0.04l-0.51-0.1l-0.04,0.38l-0.72,0.41l-1.14,0.84l-0.25,0.02l0.21,0.57
				l-0.89,0.85l-0.06,0.22l-0.66-0.18l-0.03,1.02l0.06,0.09l-0.08,0.27l-0.13,0.07l-0.02,0.29l0.1,0.18l0.44-0.11l0.86-0.01
				l0.24-0.28l0.11-0.29l0.18-0.02l0.53,0.29l0.23,0.03l0.03-0.06l0.09,0.11l0.24,0.92l0.24,0.4l0.26,0.15l0.54-0.08l0.35,0.25
				l0.19,0.25l0.14-0.03l0.22,0.21l0.2-0.02l0.28,0.15l0.35,0.1l1.07-0.22l0.18,0.2l0.34,0.07l0.21,0.19l1.03,0.01l0.37-0.43
				l0.2-0.11l0.66-0.1l1.36-0.36l0.9,0.13l0.46-0.67l-0.02-1.21l0.41-0.92l0.37-0.09l0.12-0.12l0.06-0.24l0.38-0.27l0.33-0.12
				l0.95-1.46l0.19-0.7l0.36-0.28l-0.01-0.07l0.07,0.03l0.15-0.22l0.15,0.06l0.27-0.2l0.1,0.03l0.4-0.45l0.07-0.2l-0.01-0.27
				l-0.47-0.59l-0.25-0.13l-0.24,0.01l0.05-0.53l-0.29-0.2l-1.05,1.07l-0.29,0.14l0.05-0.24l-0.62-0.79l-0.37-0.7l-0.22-0.19
				L302.37,130.56L302.37,130.56z"></path>
		</g>
		<g id="la_guarena">
			<path id="path820" class="st3 fase_0" d="M313.17,131.08l-0.2,0.24l-0.35-0.23l-0.45-0.01l-0.02,0.38l-0.15,0.33l0.03,0.11
				l-0.37,0.37l0.07,0.17l-0.02,0.26l0.14,0.27l-0.33,0.37l-0.79-0.07l-0.51-0.27l-0.51,0.41l-0.14-0.03l-0.11-0.14l0.2-0.24
				l-0.03-0.31l-0.34-0.28l-0.26-0.34l-0.17-0.09l-0.29,0.48l0.25,0.04l0.16,0.13l0.15,1.06l0,0.74l-0.03,0.31l-0.26,0.3l-0.43,0.27
				l-0.12-0.41l-0.21-0.32l-0.37-0.35l-0.21-0.31l-0.32-0.07l-0.6-0.65l-0.07,0.2l-0.4,0.45l-0.1-0.03l-0.27,0.2l-0.15-0.06
				l-0.15,0.22l-0.07-0.03l0.01,0.07l-0.36,0.28l-0.19,0.7l-0.94,1.46l-0.33,0.12l-0.38,0.27l-0.06,0.24L303,137.4l-0.37,0.09
				l-0.41,0.92l0.02,1.21l0.2-0.08l0.24,0.01l0.78,0.72l0.13,0.01l0.25,0.94l0.35-0.06l0.37,0.09l0.24,0.31l0.68,0.3l0.68-0.75
				l0.25-0.04l0.1,0.1l0.31,0.08l0.48,0.46l0.72,0.51l0.29,0.05l0.14,0.21l0.05,0.31l0.27,0.26l0.4,0.16l0.46-0.08l-0.1,0.39
				l-0.01,0.91l0.16,0.5l0.14-0.07l0.04-0.16l0.15-0.05l-0.04-0.04l0.12-0.14l0.04,0.03l0.14-0.24l-0.23-0.57l0.02-0.38l-0.07-0.14
				l2.04-0.49l0.33-0.01l0.43,0.11l0.66-0.99l0.84,0.36l0.38-0.12l0.55-0.35l0.19-0.09l0.06,0.12l-0.05,0.46l0.09,0.16l0.42-0.54
				l0.59-0.42l0.17-0.25l0.05-0.17l-0.06-0.09l0.13-0.04l0.03-0.15l0.12-0.09l0.08,0.13l0.37-0.12l-0.18,0.01l0.1-0.18l-0.09-0.03
				l-0.03-0.1l0.07-0.23l0.09-0.02l-0.09-0.06l0.07-0.17l-0.29-1l0.03-0.19l0.4-0.34l-0.2-0.08l0.07-0.64l-0.09-0.53l-0.26-0.43
				l0.02-0.43l-0.13-0.17l0.12-0.24l-0.08-0.12l-0.42-0.12l-0.25-0.32l-0.09-0.76l-0.35-0.96l-0.01-0.35l-0.05-0.31l0.06-0.59
				l-0.39-0.05l-0.89,0.3l-0.43-0.35l-0.12-0.02l-0.01-0.84l-0.38-0.19l-0.19,0.12L313.17,131.08L313.17,131.08z"></path>
		</g>
		<g id="zamora_sur">
			<path class="st3 fase_0" d="M290.02,122.09L290.02,122.09L290.02,122.09L290.02,122.09z M294.47,124.91l-0.19-0.08l-0.25,0.29
				l-0.38,0.65l-0.22,0.65l0.06,0.77l0.13,0.32l-0.04,0.17l-0.41,0.14l-0.14,0.22l0.3,0.63l-0.12,0.23l-0.51,0.24l-0.36-0.03
				l-0.13,0.17l0.03,0.3l-0.23-0.01l-0.26-0.12l-0.28-0.3L291.2,129l-0.27-0.34l-0.39-0.29l-0.6-0.07l0.01,0.18l-0.11,0.09
				l0.05,0.13l-0.14,0.78l0.11,0.49l-0.06,0.13l-0.22,0.12l-0.04,0.5l-0.3,0.76l-0.16,0.12l-0.15,0.57l-0.22,0.08l0.08,0.06
				l0.02,0.23l0.81,0.03l0.53,0.36l0.02,0.48l0.49-0.05l0.15-0.14l0.78-0.31l0.15-0.13l0.38,0.61l0.2,0.08l0.26,0.52l0.08,0.03
				l0.11-0.12l0.63-0.05l-0.01,0.07l0.51,0.1l0.24-0.04l0.68-0.56l0.11,0.53l0.12,0.04l0.32,0.38l0.33-0.21l0.4-0.13l0.13-0.16
				l0.11,0.1l0.14-0.36l0.2,0.03l0.25-0.35l-0.01-0.14l-0.13-0.14l0.04-0.22l0.06,0.11l0.24,0.12l0.54-0.28l0.31-0.27l0.13-0.25
				l0.44-0.24l0.13-0.18l0.42,0.09l0.52-0.19l0.16,0.15l0.01-0.08l0.41-0.25l0.32-0.31l0.71-0.1l0.63,0.01l0.04,0.17l0.37-0.24
				l0.09-0.43l0.5,0.2l0.22,0.19l0.37,0.7l0.62,0.79l-0.05,0.24l0.29-0.14l1.05-1.07l0.29,0.2l-0.05,0.53l0.24-0.01l0.25,0.13
				l0.47,0.59l0.01,0.27l0.6,0.65l0.32,0.07l0.21,0.31l0.37,0.35l0.21,0.32l0.12,0.41l0.43-0.27l0.26-0.3l0.03-0.31l0-0.74
				l-0.15-1.06l-0.16-0.13l-0.25-0.04l-0.39-0.53l0.07-0.21l-0.2-0.52l0.07-0.06l-0.17-0.15l0.11-0.34l-0.17-0.07l-0.07-0.55
				l-0.08,0.01l-0.2-0.3l-0.01-0.09l0.07-0.01l0.08-0.14l0.15-0.01l-0.18-0.22l0.06-0.14l-0.12-0.08l-0.11-0.33l0.13-0.23
				l-0.13,0.04l-0.03-0.17l0.16-0.47l-0.21,0.02l-0.06-0.23l-0.2,0.03l-0.19-0.24l-0.2-0.01l-0.08-0.14l0.06-0.16l-0.19-0.24
				l-0.12-0.22l0.34-0.32l0.07-0.52l-0.16-0.46l0.04-0.39l0.18-0.41l-0.06-0.16l-0.4,0.01l-0.31,0.48l-0.19-0.09l-0.25,0.42
				l-0.2,0.35l-0.54,0.25l-0.23,0.32l-0.26,0.02l-0.3-0.09l-0.3-0.24l-0.26-0.09l-0.18,0.11l-0.43-0.23l-0.02-0.16l0.32-0.19
				l0.06-0.17l0.06,0.02l0.11-0.14l0.01-0.27l-0.14-0.07l-0.46,0.03l-0.22-0.11l-0.29-0.65l-0.1-0.43l0.06-0.49l-0.3,0.04
				l-0.54,0.58l-0.33-0.13l-0.15,0.05l-0.08,0.18l-0.11-0.06l-0.21,0.14l-0.2-0.19l-0.26,0.26l-0.38-0.18l-0.1,0.43l-0.34,0.1
				l0,0.14l-0.17,0l-0.08,0.11l-0.24,0l-0.38-0.22l-0.23,0.07l-0.1-0.37l-0.34,0l-0.73,0.47l-0.5-0.22l-0.08-0.5l-0.39-0.63
				l-0.22-0.04l-0.5,0.18l-0.3,0.29l-0.52,0.13l-0.26-0.15l-0.03-0.33l-0.19-0.29l-0.12-0.4l-0.12-0.05l-0.29-0.03l-0.35-0.35
				l-0.16-0.01l-0.34,0.33l-0.26-0.04l-0.21-0.18l-0.29-0.09l-1.07-0.15l-0.5,0.03l-0.4,0.16l0.03,0.19l-0.36,0.72l0.52,0.12
				l0.23,0.22l0.08-0.05l0.4,0.08l0.04,0.13l0.44,0.41l0,0.16l0.2,0.02l0.12,0.16l0.3,0.09l0.2-0.05l0.01-0.06l0.36,0.06l0.43-0.06
				l0.07,0.09l0.12-0.04l0.11,0.06l0.11-0.03l0.18-0.29l0.31,0.39l0.29,0.23l0.06,0.18L294.47,124.91 M302.98,125.52l0-0.01
				L302.98,125.52L302.98,125.52z"></path>
		</g>
		<g id="alcanices">
			<path id="path610" class="st3 fase_0" d="M273.91,101.4l-0.28,0.09l-0.03,0.08l0.02,1.01l-0.26,0l-0.29-0.19l-0.54,0.86l-0.19-0.16
				l-0.25,0.08l-0.19-0.06l-0.18,0.04l-0.07,0.13l-0.45,0l-0.42-0.32l-0.89-0.24l-0.04,0.26l0.06,0.17l-0.88-0.1l-0.67,0.05
				l-1.42-0.08l-0.28-0.3l-0.15,0.07l0.08,0.11l-0.1,0.09l0.06,0.09l-0.37,0.73l0.04,0.15l-0.17,0.62l-0.19,0.09l-0.21,0.57
				l1.34,0.37l0.81,0.07l0.09,0.25l-0.16,0.31l-0.23,0.22l0.13,0.35l-0.19,0.2l-0.06,0.2l0.1,0.2l-0.25,0.43l0.15,0.38l-0.13,0.25
				l0.16,0.15l-0.3,0.12l0.08,0.22l-0.17,0.08l-0.05,0.06l0.05,0.26l-0.21,0.09l0.13,0.22l-0.15,0.04l0.01,0.19l-0.17,0.1l0.15,0.38
				l-0.14,0.43l0.13,0.08l-0.24,0.08l-0.02,0.24l-0.15,0.16l-0.06,0.23l0.01,0.43l-0.13,0.21l-0.27,0.09l-0.16,0.43l0.05,0.07
				l0.25,0.01l0.09,0.18l0.15,0.04l-0.09,0.18l0.09,0.5l0.15-0.08l0.1,0.04l-0.24,0.26l0.21,0.14l-0.16,0.04l0.22,0.21l-0.07,0.12
				l0.17,0.21l-0.03,0.21l0.16,0l0.02,0.14l-0.11,0.14l-0.18-0.2l-0.18,0.06l0.02,0.42l0.21,0.42l0.26,0.28l0.29,0.21l0.28,0.08
				l0.63,0.74l0.55,0.22l0.41-0.02l0.81-0.49l0.43-0.01l0.21-0.46l0.05-0.33l0.28-0.18l0.19,0.03l0.14,0.18l0.09-0.08l0.36,0.17
				l0.23-0.04l0.35,0.08l0.05-0.11l0.16-0.04l0.42,0.21l0.83,0.18l0.49-0.09l0.09-0.11l0.16,0.03l1.37,0.54l0.81,0.21l0.59,0.34
				l1.41,1.26l0.74,1.39l0.24,0.19l0.59,0.22l0.26,0.01l0.31,0.27l0.47,1.04l0.12-0.06l0.41-0.06l0.11-0.22l0.59-0.02l0.35,0.24
				l0.09,0.51l0.7,0.2l0.14,0.32l0.5,0.11l0.28,0.14l0.28-0.3l0.18-0.38l0.17-0.03l0.41-0.47l0.29-0.1l-0.34-0.52l-0.09-0.42
				l0.84-0.27l0.18-0.91l-0.18-0.41l-0.01-0.36l0.07-0.12l-0.1-0.31l-0.12-0.11l-1.27-0.33l-0.25-0.38l-0.11-0.33l-0.76-0.9
				l0.02-0.29l0.08-0.11l0.15,0.01l-0.34-0.21l-0.23-0.26l0.05-0.3l0.45-0.47l0.15-0.33l-0.08-0.14l0.27-0.57l0.65-0.59l0.1-0.22
				l0.09-0.2l-0.78-1.21l-1.67-0.99l-0.5-0.19l-0.49-0.04l-0.56,0.22l-0.19,0.03l-0.2-0.08l-0.46-0.45l-0.12,0l0.04-0.25l0.12-0.06
				l0.21-0.31l0.24-0.58l-0.55-0.84l-0.27-0.27l-0.15,0.15l-0.43,0.13l-0.31,0.22l-0.6-0.42l-0.11,0l-0.27-0.26l-0.08-0.21
				l-0.27,0.01l-0.03-0.15l0.22-0.87l0.01-0.43l0.17-0.33l-0.16-0.19l-0.42-0.28l-0.81-0.43l-0.62-0.01l-0.4-0.21l0.1-1.52
				l-0.05-0.09l-0.45-0.27l-0.11,0.13l-0.14-0.07l-0.36-0.33L273.91,101.4L273.91,101.4z"></path>
		</g>
	</g>
	<g id="soria">
		<g id="almazan">
			<path id="path3670" class="st3 fase_0" d="M433.17,121.3l-0.14,0.2l-0.25,0.06l-0.5,0.3l-0.04-0.4l-0.09-0.1l-0.36-0.01l-0.07,0.19
				l-0.14,0.09l-0.32,0.3l-0.56-0.18l-0.29,0.02l-0.65-0.26l-0.51,0.09l-0.07,0.16l-0.8,0.08l-0.46,0.11l-0.31,0.16l-0.87,0.72
				l-0.56,0.27l-0.52,0.18l-0.76-0.03l-1.13,0.19l-0.47,0.02l-0.66,0.88l-1.2-0.29l-0.68,0.53l-0.47,1.03l-1.17,0.29l-0.17,0.11
				l0.05,0.19l-0.08,0.12l-0.33,0l-0.12-0.1l-0.28,0.77l-0.2,0.84l0.26,0.26l0.59,0.18l0.45-0.19l0.88,0.27l0.32,0.31l-0.07,0.29
				l0.33,0.48l0.45,0.18l0.28,0.3l1.39,0.64l-0.07,0.3l-0.26,0.31l0.04,0.13l0.16,0.01l0.27,0.35l0.36-0.01l-0.67,0.88l-0.1,0.41
				l0.08,0.36l-0.04,0.68l-0.21,0.12l-0.12,0.02l-0.73,0.99l-0.84,0.47l-0.3,0.03l-0.57,0.34l0.18,1.79l-0.1,0.28l1.25,0.13
				l0.54-0.22l0.11,0.78l0.18,0.49l0.57,0.34l1.05,0.46l0.59-0.19l0.36,0.02l0.04,0.12l0.42,0.32l-0.06,0.44l0.29,0.46l0.16-0.35
				l0.05-0.11l0.28,0.23l1.11-0.27l0.1-0.23l0.88-0.51l0.44-0.1l1.71,0.39l0.42-0.01l0.31-0.5l0.69-0.57l0.21,0.12l0.8,0.03
				l0.67-0.18l0.2-0.18l0.44-0.15l0.23-0.19l0.19-0.32l-0.56-1l-0.09,0l0,0.1l-0.08,0.01l-0.3-0.46l-0.12,0.01l-0.24-0.29
				l-0.37-0.05l0.09-0.12l-0.16-0.29l0.18-0.16l-0.09-0.87l0.2-0.94l-0.13-0.41l0.27-0.58l-0.14-0.24l0.52,0.05l0.37-0.13l0.86,0.55
				l0.27,0.1l0.25-0.08l0.21,0.09l0.49-0.33l-0.12-0.19l0.62-0.64l0.4-0.2l0.17,0.42l0.3,0.3l0.54,0.1l-0.01-0.3l0.45-0.58
				l1.09-1.12l0.32-1.08l0.64-1.02l-0.85,0.33l-0.31,0.03l-0.34-0.15l-1.04-1.08l-0.58-0.82l0.05-0.09l0.27-0.16l0.13-0.34l0.28-0.4
				l-0.06-0.18l0.1-0.2l0.18-0.1l0.09-0.19l0.46-0.37l0.14-0.26l-0.38-0.68l-1.01-1l0.01-0.44l-0.14-0.15l-0.34,0l-0.18,0.13
				l-0.08-0.23l-0.14,0.04l-0.03-0.15l-0.39-0.2l-1.41,0.31l-0.46-0.39l-0.15-0.4l-0.02-0.41l-0.19-0.04l-0.21,0.15L433.17,121.3
				L433.17,121.3z"></path>
		</g>
		<g id="gomara">
			<path id="path3830" class="st3 fase_0" d="M442.48,112.16l-0.3,0.13l-0.62,0.7l-0.29-0.06l-0.67,0.76l0.47,1.1l-0.32,0.99l0.07,0.17
				l-0.13,0.67l-0.48,0.12l-0.66,0.38l-0.34,0.03l-0.22,0.55l-0.59,0.3l0.3,0.63l0.06,0.54l0.2,0.25l-0.21-0.05l-0.2,0.09l-0.2,0.06
				v0.08l-0.41,0.04l-0.2-0.24l-0.37-0.14l-0.04-0.22l-0.19-0.06l-0.62,0.12l-0.37,0.19l-0.19,0.26l-0.02,0.45l0.81,0.37l-0.18,0.14
				l-0.07,0.19l0.06,0.17l-0.26,0.39l0.24,0.19l-0.16,0.17l-0.21,0.07l0.04,0.09l-0.09,0.23l-0.09,0.14l-0.09-0.04l-0.06,0.18
				l0.39,0.2l0.03,0.15l0.14-0.04l0.08,0.23l0.18-0.13l0.34,0l0.14,0.15l-0.01,0.44l1.01,1l0.38,0.68l-0.14,0.26l-0.46,0.37
				l-0.09,0.19l-0.18,0.1l-0.1,0.2l0.06,0.18l-0.28,0.4l-0.13,0.34l-0.27,0.16l-0.05,0.09l0.58,0.82l1.04,1.08l0.34,0.15l0.31-0.03
				l0.85-0.33l0.72,0.52l1.33,0.57l-0.44,0.07l0.28,0.41l-0.13,0.51l-0.28,0.16l-0.03,0.13l-0.19,0.18l-0.09,0.03l-0.01-0.11
				l-0.24-0.15l-0.56,0.02l-0.41-0.14l-0.65,0.2l0.84,0.46l2.98-0.26l0.34,0.08l1.48-0.12l0.37,0.1l0.21,0.17l0.37,0.01l-0.04-0.21
				l0.09-0.1l0.09-0.67l-0.03-0.87l0.19-0.54l0.18-0.16l0.16-0.09l0.09-0.33l2.2,0.62l0.46,0.4l0.03,1.21l-0.36,0.39l0.05,0.44
				l0.16-0.02l0.17,0.16l0.09-0.03l0.11,0.14l0.17,0.02l0.35,0.01l-0.01-0.08l0.29-0.05l0.31-0.43l0.34-0.09l0.25-0.03l0.22-0.34
				l0.46,0.07l0.33-0.13l0.36-0.28l0.64,0.07l0.02-0.23l-0.07-0.06l-0.03-0.23l0.14-0.16l-0.23,0l-0.46-0.24l0.2-0.29l0-0.33
				l-0.15-0.52l0.04-0.57l-0.07-0.34l-0.68-1.02l-0.45-0.27l0.18-0.55l-0.09-0.4l0.15-0.64l-0.13-0.57l0.25-0.33l-0.04-0.07
				l0.23-0.64l-0.13-0.14l0.07-0.11l-0.2-0.39l-0.13,0.03l-0.21-0.25l0.05-0.14l-0.53-0.67l-0.05-0.25l0.12-0.46L451,120.5
				l0.36-0.37l-0.67-0.41l-0.51-0.42l-0.28-0.55l-0.27-0.39l0.49-0.63l-0.15-0.28l-0.1,0l-0.3-0.26l-0.51-0.25l0.15-0.27l-0.68-0.29
				l-0.17-0.22l-1-0.4l-0.96-0.66l-0.33-0.35l-0.39,0.03l-1.12-0.2l0.01-0.39l0.17-0.44l-1.15-1.01l-0.98-0.55L442.48,112.16
				L442.48,112.16z"></path>
		</g>
		<g id="san_esteban_gormaz">
			<path id="path3824" class="st3 fase_0" d="M398.25,111.8l-0.45,0.1l-0.1,0.49l-0.49,0.1l-0.05,0.44l-0.25,0.02l-0.05,0.07l-0.21-0.11
				l-0.31,0.15l-0.39,0.04l-0.09,0.22l0.11,0.25l-0.33,0.41l0.03,0.43l-0.08,0.18l0.04,0.58l0.1,0.32l-0.49,0.09l-0.6-0.06
				l-0.33,0.13l-0.05,0.33l-0.51,0.48l0.01,0.13l-0.31,0.45l-0.12,0.46l0.05,0.45l-0.1,0.2l-0.1-0.11l-0.33-0.01l-0.13,0.35
				l-0.09,0.02l-0.06,0.28l-0.16,0.25l0.04,0.24l0.16,0.16l-0.23,0.2l-0.07,0.36l-0.29,0.32l-0.38,0.87l-0.26,0.02l-0.12,0.02
				l0.01-0.18l-0.29-0.25l-0.17,0.13l-0.07-0.01l-0.01-0.15l-0.53,0.03l-0.45-0.07l-0.08,0.08l-0.09-0.03l-0.06-0.06l0.04-0.11
				l-0.25-0.17l-0.28-0.35l-0.23-0.15l-0.12,0.09l-0.32,0.13l-0.02,0.58l-0.2-0.04l-0.19,0.11l0.58,0.54l-0.22,0.48l-0.09,0.3
				l0.29,0.68l-0.07,0.4l0.14,0.18l0.41,0.19l0.24,0.31l0.68,0.55l0.68,0.28l0.22,0.17l0.62-0.37l0.25-0.49l0.08,0.12l0.17,0.12
				l0.38,0.44l0.41,0.3l0.32,0.11l0.15,0.22l-0.09,0.5l-0.15,0.31l0.1,0.81l0.43,0.36l0.4,0.01l0.53,0.28l0.25,0.27l0.28,0.08
				l0.46-0.04l0.04-0.13l0.1,0.12l-0.02,0.17l0.27-0.04l0.33,0.35l0.04-0.08l0.5-0.11l0.03-0.09l0.16,0.06l-0.35,0.42l-0.05,0.46
				l-0.24,0.83l0.19,0.31l-0.01,0.34l-0.2,0.17l-0.11,0.36l0.08,0.14l0,0.41l-0.14,0.11l0.38,0.18l0.49,0.82l0.94,0.43l0.13,0.28
				l0.59,0.15l0.29,0.45l0.36-0.04l0.11,0.6l0.86,0.62l0.48-0.01l-0.02,0.63l0.67,0.43l0.47,0.15l0.07,0.05l-0.02,0.13l0.15,0.06
				l0.16-0.07l0.31,0.09l0.15-0.07l0.4,0.22l0.31,0.04l0.24,0.2l0.35-0.62l0.05-0.4l0.23-0.34l0.19-0.52l0.64,0.47l0.13,0.18
				l0.5-1.38l0.41-0.38l0.57-0.1l0.39-0.38l0.02-0.19l-0.54-0.2l-0.84-0.57l-0.86,0.2l-0.04-0.13l0.2-0.21l0.16-0.51l0.27,0
				l0.4-0.59l0.02-0.27l-0.08-0.19l0.16-0.26l-0.06-0.51l-0.3-0.42l-0.19-0.06l0.23-0.22l-0.04-0.25l-0.23-0.35l-0.22,0l0-0.84
				l-0.19-0.5l0.17-0.41l0.29,0.08l0.29-0.3l0-0.22l-0.33-0.25l0.16-0.39l0.01-0.31l0.3-0.27l-0.12-0.15l-0.06-0.28l0.3-0.64
				l-0.03-0.16l0.25-0.36l-0.07-0.17l0.17-0.11l-0.04-0.22l-0.2,0.01l-0.18,0.12l-0.8-0.51l-0.31-0.1l-0.44,0.05l-0.45-0.49
				l-0.05-0.19l0.01-0.45l0.42-0.17l0.52-0.33l0.44-0.64l-0.08-0.08l0.26-1.44l-0.37-0.3l-0.15,0.12l-0.72-0.12l-0.12-0.03
				l-0.26-0.27l-0.84-0.21l-0.35-0.87l-0.06-0.69l0.19-0.33l-0.06-0.43l0.33-0.15l0.53-0.07l0.58-0.32l0.16,0.06l0.33-0.42
				l0.41-0.12l0.24-0.27l0-0.6l-0.09-0.25l-1.75-1.02l-0.78-0.68l-0.84,0.28l-1.01,0.91l-0.59,0.19l0.04-0.44l-0.29,0.07l0.04,0.13
				l-0.24-0.01l-0.17-0.13l-0.33-0.32l-0.17-0.55l-0.28-0.22L398.25,111.8L398.25,111.8z"></path>
		</g>
		<g id="pinares-covaleda">
			<path id="path3684" class="st3 fase_0" d="M419.74,92.29l-0.38,0.07l-0.3,0.51l-0.38,0.29l-0.07,0.42l0.12,0.18l0,0.18l-0.53,0.74
				l0.01,0.13l0.19,0.15l0.07,0.15l0.08,0.48l-0.14,0.23l0.02,0.67l-0.25,0.03l-0.34,0.52l-0.1,0l-0.35-0.24l-0.4,0.15l-0.27,0.19
				l-0.18,0l-0.49,0.3l-0.06,0.33l-0.16,0.19l-0.22,0.08l-0.07,0.15l-0.29,0.17l-0.29,0.13l-0.23-0.05l-0.07-0.36l-0.91-0.31
				l-0.75,0.26l-0.21,0.54l-0.53,0.76l-0.05,0.31l-0.57,1.19l-0.07,0.43l0.31,0.36l-0.19,0.78l-0.25,0.41l1.01,0.53l0.05,0.29
				l0.2,0.21l0.12,0.12l-0.01,0.18l0.68,0.18l0.12,0.29l0.92,0.14l0.29-0.08l0.09-0.14l0.33,0.5l0.42,1.04l0.24,0l1.01,0.3
				l0.56-0.02l0.49,0.09l0.69-0.15l0.18-0.17l0.13-0.37l0.23-0.33l0.14,0.32l-0.04,0.21l0.15,0.24l-0.52,0.53l0.3,0.14l0.14,0.16
				l1.4,0.01l0.49-0.12l0.13,0.25l0.23-0.08l0.02-0.11l-0.25-0.27l-0.07-0.13l0.35-0.31l0.48-0.09l0.04-0.21l-0.12-0.01l0.22-0.46
				l0.25-0.02l-0.24-0.09l-0.33-0.3l-0.14-0.62l0.1,0.02l-0.39-0.94l0.49-0.62l-0.16-0.14l-0.21-0.68l-0.5-0.87l-0.13-0.55
				l-0.4-0.85l-0.12-0.38l-0.04-1.1l-0.02-0.2l-0.28-0.47l0.03-0.33l-0.1-0.14l0.77-0.92l0.16-0.55l0.29-0.3l0.07-0.31l0.12-0.06
				l0.57-0.74l0.03-0.23l-0.11-0.32l0.07-0.27l0.11-0.06l-0.03-0.17l-0.63-0.54l-0.25-0.39l-0.4-0.1l-0.32,0.05l-0.19,0.33
				L419.74,92.29z"></path>
		</g>
		<g id="agreda">
			<path id="path3628" class="st3 fase_0" d="M456.53,100.81l-0.85,0.75l-0.53,0.25l-0.65,0.12l0.06,0.14l-0.25,0.13l-0.16,0.45
				l-0.14-0.06l-0.12-0.05l-0.33,0.25l-0.13-0.05l-0.11,0.19l-0.25,0.08l-0.54-0.11l-0.26,0.38l-0.84,0.06l-0.66-0.62l-0.12-0.42
				l-0.48-0.12l-0.12-0.22l-0.14,0.02l-0.08-0.21l-0.27-0.25l-0.1,0.27l-0.15,0.09l0.03,0.15l-0.09-0.01l-0.02,0.23l-0.32-0.36
				l-0.04-0.21l-0.24-0.04l-0.44-0.19l-0.57,0.19l-0.1,0.12l0.06,0.3l-0.2,0.29l0.33,0.31l0.12,0.2l0,0.21l-0.19-0.2l-0.2,0.12
				l-0.03-0.09l-0.32-0.13l-0.61,0.07l-0.46-0.14l0.09,0.22l-0.04,0.28l-0.71,0.45l-2.06-0.26l-0.58,0.23l-0.76-0.33l-0.49,0.43
				l-0.26-0.04l-0.36,0.08l-0.34-0.02l-0.08,1.32l0.11,0.56l-0.2,0.12l-0.19,0.44l0.62,0.08l0.02-0.24l1.37-0.46l0.15-0.24
				l0.09,0.03l0.12-0.14l0.26-0.02l0.06-0.13l0.17-0.02l0.16,0.06l0.7,1.47l0.2,0.06l0.06-0.07l0.11,0.03l0.18,0.25l0.75,0.8
				l0.87,0.62l0.56,0.09l0.18,0.87l0.17,0.28l0.94,0.65l0.23-0.4l0.62-0.28l0.31-0.01l0.33-0.13l0.1-0.15l0.23-0.06l-0.08-0.1
				l0.24-0.62l0.08-0.76l-0.12,0.03l0.11-0.26l-0.09-0.06l0.2-0.41l-0.14-0.19l0.66-0.05l0.14-0.14l0.37,0.02l0.37,0.41l0.71,0.4
				l0.22,0.3l-0.14,0.72l-0.82,0.83l0.01,0.11l1.62,0.77l0.01,0.28l0.34,0.55l0.11,0.37l0.43,0.16l0.41-0.32l0.71-1.03v0.62
				l0.72,0.55l0.27,0.04l0.05-0.07l1.72-0.14l-0.01-0.25l-0.58-0.47l-0.54-0.13l0.37-0.88l0.4-0.47l0.34-0.23l0.28-0.59l-0.09-0.92
				l0.02-0.2l0.11-0.15L458,106.4l-0.1-0.16l-0.39,0.11l-0.32-0.64l0-0.12l0.05-0.62l-0.75-1.1l-0.05-1.24l0.15-0.23l-0.07-0.49
				l0.08-0.71L456.53,100.81z"></path>
		</g>
		<g id="arcos_de_jalon">
			<path id="path3634" class="st3 fase_0" d="M440.01,128.94l-0.64,1.02l-0.32,1.08l-1.09,1.12l-0.45,0.58l0.01,0.3l-0.54-0.1l-0.3-0.3
				l-0.17-0.42l-0.4,0.2l-0.62,0.64l0.12,0.19l-0.49,0.33l-0.21-0.09l-0.25,0.08l-0.27-0.1l-0.86-0.55l-0.37,0.13l-0.52-0.05
				l0.14,0.24l-0.27,0.58l0.13,0.41l-0.2,0.94l0.09,0.87l-0.18,0.16l0.16,0.29l-0.09,0.12l0.37,0.05l0.24,0.29l0.12-0.01l0.3,0.46
				l0.08-0.01l0-0.1l0.09,0l0.56,1l-0.19,0.32l-0.23,0.19l-0.44,0.15l-0.2,0.18l-0.67,0.18l-0.8-0.03l-0.21-0.12l-0.69,0.57
				l-0.31,0.5l-0.42,0.01l-1.71-0.39l-0.44,0.1l-0.88,0.51l-0.1,0.23l0.04,0.08l0.04,0.5l-0.23,0.69l-0.31,0.33l-0.2,0.07l-0.58-0.1
				l-0.08,0.4l0.11,0.25l0.68-0.29l0.99,0.51l0.04,0.08l-0.22,0.72l-0.09,0.16l-0.36,0.26l-0.09,0.27l0.6,0.58l0.23,0.07l0.32-0.29
				l0.5-0.09l0.54-0.24l0.22-0.62l0.79,0.86l-0.02,0.51l0.23,0.29l0.11,0.68l0.21,0.07l0.67,0.03l0.07-0.05l0.21,0.23l0.44,1.28
				l0,0.63l0.81,0.02l0.06,0.15l0.23-0.13l0.09,0.22l-0.04,0.1l0.45-0.05l0,0.35l0.38,0.3l0.06,0.14l0.19-0.17l0.49-0.04l0.77-0.67
				l0.07-0.15l0.35-0.11l0.7-0.39l0.25,0.49l0.62,0.54l0.13,0.03l0.3,0.55l0.32-0.27l0.5-0.18l0.39-0.34l0.1,0.02l0.27,0.3
				l0.85-0.12l1.27-1.58l0.39,0.19l1.91,0.52l-0.02-0.15l0.34-0.47l0.97-0.39l0.3-0.76l0.16-0.08l0.58,0.82l0.32,0.29l0.48,0.27
				l0.42,0.13l0.23,0.2l0.19,0.24l-0.08,0.24l0.28,0.27l0.3,0.13l0.31,0.02l0.33-0.47l-0.38-0.34l-0.33-0.6l0.08-0.42l0.13-0.09
				l-0.09-0.43l0.14-0.36l0.35-0.33l0-0.19l-0.04-0.94l-1.04-0.88l-0.1-0.42l-1.09-0.23l-1.6-0.49l-0.11-1.85l-0.24-1.16l-0.15-1.32
				l-0.2-0.98l-0.41-0.26l0.03-0.41l0.16-0.29l0.07-0.54l-0.13-0.41l0.06-0.2l0.19-0.33l0.26-0.16l0.05-0.31l-0.24-0.56l-0.03-0.4
				l0.29-0.08l0.25,0.05l0.73-0.67l0.24-0.07l0.23-0.54l0.17-0.13l-0.14-0.12l0-0.33l-0.37-0.01l-0.21-0.17l-0.37-0.1l-1.48,0.12
				l-0.34-0.08l-2.98,0.26l-0.84-0.46l0.65-0.2l0.41,0.14l0.56-0.02l0.24,0.15l0.01,0.11l0.09-0.03l0.19-0.18l0.03-0.13l0.28-0.16
				l0.13-0.51l-0.28-0.41l0.44-0.07l-1.33-0.57L440.01,128.94z"></path>
		</g>
		<g id="s_pedro_manrique">
			<path id="path3672" class="st3 fase_1" d="M436.63,91.12l-0.12,0.1l-0.79,0.08l-0.22,0.11l-0.47-0.06l-0.38,0.3l-0.17-0.14
				l-0.66,0.03l-0.38,0.17l-0.47-0.08l-0.11,0.04l-0.21,0.23l-0.08,0.44l-0.2,0.15l-0.21,0.32l-0.53,0.05l-0.11,0.17l-0.4,0.25
				l-0.24-0.25l-0.23-0.11l-0.21,0.03l-0.3-0.16l-0.25,0.05l-0.07,0.2l0.01,0.44l-0.13,0.25l0.05,0.39l-0.13,0.17l-0.04,0.25
				l0.13,0.1l0.19,0.36l-0.01,0.29l0.61,0.35l0.35,0.78l0.21,0.26l0.15,0.03l0.12,0.43l0,0.28l0.44,0.45l0.36,0.21l0.64,0.47
				l0.08,0.44l0.19,0.31l0.26,0l0.27,0.15l0.3,0.39l0.18,0.09l0.33,0.06l0.3,0.17l0.36-0.06l1.47,0.79l0.13-0.02l0.38,0.66
				l0.82-0.15l0.24,0.03l0.43,0.29l0.23,0.03l0.28-0.02l0.39,0.28l0.4,0.69l0.07,0.12l0.1,0.16l0.05,0.15l0.4,0.41l0.11,0.25
				l0.34,0.02l0.36-0.08l0.26,0.04l0.49-0.43l0.76,0.33l0.58-0.23l2.06,0.26l0.71-0.45l0.04-0.28l-0.09-0.22l0.46,0.14l0.61-0.07
				l0.32,0.13l0.03,0.09l0.2-0.12l0.19,0.2l0-0.21l-0.12-0.2l-0.33-0.31l0.2-0.29l-0.06-0.3l0.1-0.12l-0.09-0.02l-0.07-0.16l-0.25,0
				l-0.2-0.1l-0.38,0.06l-0.39-0.14l-0.25-0.31l0.04-0.14l-0.05-0.46l-0.12-0.1l-0.02-0.51l-0.17-0.49l0.19-0.09l0.3,0l0.18-0.31
				l-0.31-0.47l-0.07-0.54l-0.48-0.25l-0.65-0.83l-0.23-0.03l0.03-0.15l-0.2-0.11L444.29,96l0.02-0.22l-0.14-0.36l0.42-0.12
				l0.38-0.29l0.27-0.02l0.34-0.34l0.01-0.39l-0.06-0.08l0.04-0.41l-0.38-0.32l-0.08,0.04l-0.38-0.11l-1.11-0.02l-0.93,0.12
				l-0.42-0.12l-0.31,0.24l-0.3,0.11l-0.27-0.18l-0.29-0.04l-0.85,0.8l-0.43-0.07l-0.49-1.07l0.05-0.32l-0.08-0.24l0.15-0.19
				l-0.13-0.5l-0.25-0.18l-0.79-0.28l-0.32-0.23l-0.14-0.04l-0.31,0.08l-0.21-0.13l-0.48,0.03L436.63,91.12L436.63,91.12z
				 M441.15,102.9l0.08,0.12l-0.1,0.22l-0.34,0.31l-0.11-0.06l-0.11,0.01l0.06-0.15l-0.05-0.16l0.28-0.24L441.15,102.9z"></path>
		</g>
		<g id="olvega">
			<path id="path3662" class="st3 fase_0" d="M450.68,106.58l-0.14,0.14l-0.66,0.05l0.14,0.19l-0.2,0.41l0.09,0.06l-0.11,0.26l0.12-0.03
				l-0.08,0.76l-0.24,0.62l0.08,0.1l-0.23,0.06l-0.1,0.15l-0.33,0.13l-0.31,0.01l-0.62,0.28l-0.23,0.4l-0.94-0.65l-0.17-0.28
				l-0.86,0.74l-0.36,0.55l-0.3,0.19l0.08,0.09l-0.3-0.04l-0.95,0.67l-0.21-0.09l0.05-0.15l-0.07-0.62l-0.3,0.61l-0.33-0.1
				l-0.1,0.13l-0.48-0.05l-0.32,0.35l-0.2,0.4l-0.01,0.32l0.06,0.05l0.3-0.13l0.14,0.02l0.98,0.55l1.15,1.01l-0.17,0.44l-0.01,0.39
				l1.12,0.2l0.39-0.03l0.33,0.35l0.96,0.66l1,0.4l0.17,0.22l0.68,0.29l-0.15,0.27l0.51,0.25l0.3,0.26l0.1,0l0.15,0.28l-0.49,0.63
				l0.27,0.39l0.28,0.55l0.51,0.42l0.67,0.41l0.34,0.22l0.27-0.15l0.46,0.04l1.41,0.35l0.19-0.44l0.17-0.11l0.02-0.4l0.23-0.26
				l0.31-0.28l0.1-0.33l0.77-0.46l0.2,0.16l0.61-1.27l0.99,0.05l0.22-0.11l0.65,0.11l0.28-0.2l-0.12-1.1l-0.12-0.1l0.42-0.52
				l0.27-0.15l0.04,0.07l0.28-0.09l-0.05-0.18l0.13-0.77l0.39-0.42l-0.36-0.2l-0.14-0.3l-0.41,0.08l-0.46-0.22l0.26-0.78l-0.75-0.8
				l-0.16-0.41l-1.72,0.14l-0.05,0.07l-0.27-0.04l-0.72-0.55v-0.62l-0.71,1.03l-0.41,0.32l-0.43-0.16l-0.11-0.37l-0.34-0.55
				l-0.01-0.28l-1.62-0.77l-0.01-0.11l0.82-0.83l0.14-0.72l-0.22-0.3l-0.71-0.4l-0.37-0.41L450.68,106.58L450.68,106.58z"></path>
		</g>
		<g id="soria_rural">
			<path class="st3 fase_0" d="M446.03,108.29l-0.87-0.62l-0.75-0.8l-0.18-0.25l-0.11-0.03l-0.06,0.07l-0.2-0.06l-0.7-1.47l-0.16-0.06
				l-0.17,0.02l-0.06,0.13l-0.26,0.02l-0.12,0.14l-0.09-0.03l-0.15,0.24l-1.37,0.46l-0.02,0.24l-0.62-0.08l0.19-0.44l0.2-0.12
				l-0.11-0.56l0.08-1.32l-0.11-0.25l-0.4-0.41l-0.05-0.15l-0.1-0.16l-0.07-0.12l-0.4-0.69l-0.39-0.28l-0.28,0.02l-0.23-0.03
				l-0.43-0.29l-0.24-0.03l-0.82,0.15l-0.38-0.66l-0.13,0.02l-1.47-0.79l-0.36,0.06l-0.3-0.17l-0.33-0.06l-0.18-0.09l-0.3-0.39
				l-0.27-0.15l-0.26,0l-0.19-0.31l-0.08-0.44l-0.64-0.47l-0.36-0.21l-0.44-0.45l0-0.28l-0.12-0.43l-0.15-0.03l-0.21-0.26
				l-0.35-0.78l-0.61-0.35l0,0l-0.71,0.73l-0.12,0.06l-0.17-0.04l-0.39,0.19l-0.12,0.32l0.03,0.36l-0.19,0.22l-0.38,1.12l-0.4,0.39
				l-0.19,0.56l-0.68-0.31l-0.53-0.05l-0.22-0.15l-0.38,0.05l-0.09-0.08l-0.95,0.15l-0.4-0.08l-0.08,0.13l-0.47,0.26l-0.55-0.18
				l-0.37-0.5l-0.39-0.24l-0.18,0.08l-0.41-0.11l-0.28,0.05l-0.77,0.34l-0.09-0.15l0.04,1.1l0.12,0.38l0.4,0.85l0.13,0.55l0.5,0.87
				l0.21,0.68l0.16,0.14l-0.49,0.62l0.39,0.94l-0.1-0.02l0.14,0.62l0.33,0.3l0.24,0.09l-0.25,0.02l-0.22,0.46l0.12,0.01l-0.04,0.21
				l-0.48,0.09l-0.35,0.31l0.07,0.13h0l0.25,0.27l-0.02,0.11l-0.23,0.08l-0.13-0.25l-0.49,0.12l0.25,0.2l0,0l-0.25-0.2l-1.4-0.01
				l-0.14-0.16l-0.3-0.14l0.52-0.53l-0.15-0.24l0.04-0.21l-0.14-0.32l-0.23,0.33l-0.13,0.37l-0.18,0.17l-0.69,0.15l-0.49-0.09
				l-0.56,0.02l-1.01-0.3l-0.24,0l-0.42-1.04l-0.33-0.5l-0.09,0.14l-0.29,0.08l-0.92-0.14l-0.12-0.29l-0.68-0.18l0.01-0.18
				l-0.12-0.12l-0.2-0.21l-0.05-0.29l-1.01-0.53l-0.47,0.06l-0.21-0.18l-0.44-0.06v0l-0.39,0.56l-0.13,0.43l0.14,0.29l-0.36,0.56
				l0.06,0.12l-0.07,0.5l0.6,0.23l0.3,1.08l0.29,0.43l0.19,0.58l0.11,0.15l0.39,0.1l-0.03,0.15l0.18,0.08l0.05,0.2l0.08,0.15
				l-0.03,0.11h0l-0.1,0.27l-0.16,0.01l-0.2,0.24l0.25,0.01l0.74,0.36l0.25,0.22l0.08,0.42l-0.16,0.11l-0.11,0.3l0.19,0.18
				l0.08,0.49l0.84,1.56l0.17,0.15l0.05,0.53l-0.01,0.54l-0.62-0.41l-0.21,0.34l0.64,0.49l0.14,0l1.1,0.31l0.27,0.69l1.13,0.04
				l0.29,0.58l0.18,0.14l0.21,0l0.2,0.51l-0.04,0.38l0.54,0.12l0.03-0.14l0.79,0.76l-0.04,0.27l0.22,0.55l-0.32,0.72l-0.03,0.53
				l-0.24,0.18l0.53,0.82l0.31,0.65l1.05-0.09l0.33-0.1l0.17-0.42l0.13,0l0.27,0.28l0.42,0.14l1.04,1l0.03,0.31l0.16,0.29l0.22,0.09
				l0.32,0.52l0.37-0.21l0.41,0.22l0.14,0.19l1.13-0.19l0.76,0.03l0.52-0.18l0.56-0.27l0.87-0.72l0.31-0.16l0.46-0.11l0.8-0.08
				l0.07-0.16l0.51-0.09l0.65,0.26l0.29-0.02l0.56,0.18l0.32-0.3l0.14-0.09l0.07-0.19l0.36,0.01l0.09,0.1l0.04,0.4l0.5-0.3
				l0.25-0.06l0.14-0.2l0.24,0.22l0.21-0.15l0.19,0.04l0.02,0.41l0.15,0.4l0.46,0.39l1.41-0.31l0.06-0.18l0.09,0.04l0.09-0.14
				l0.09-0.23l-0.04-0.09l0.21-0.07l0.16-0.17l-0.24-0.19l0.26-0.39l-0.06-0.17l0.07-0.19l0.18-0.14l-0.81-0.37l0.02-0.45l0.19-0.26
				l0.37-0.19l0.62-0.12l0.19,0.06l0.04,0.22l0.37,0.14l0.2,0.24l0.41-0.04v-0.08l0.2-0.06l0.2-0.09l0.21,0.05l-0.2-0.25l-0.06-0.54
				l-0.3-0.63l0.59-0.3l0.22-0.55l0.34-0.03l0.66-0.38l0.48-0.12l0.13-0.67l-0.07-0.17l0.32-0.99l-0.47-1.1l0.67-0.76l0.29,0.06
				l0.62-0.7l-0.06-0.05l0.01-0.32l0.2-0.4l0.32-0.35l0.48,0.05l0.1-0.13l0.33,0.1l0.3-0.61l0.07,0.62l-0.05,0.15l0.21,0.09
				l0.95-0.67l0.3,0.04l-0.08-0.09l0.3-0.19l0.36-0.55l0.86-0.74l-0.18-0.87L446.03,108.29z M422.72,105.68L422.72,105.68l-0.1,0.34
				l0,0L422.72,105.68z M440.68,103.49l0.11,0.07l0.35-0.32l0.09-0.22l-0.09-0.12v0l-0.27,0.05l-0.29,0.24l0.05,0.17l-0.07,0.16
				L440.68,103.49z"></path>
		</g>
		<g id="san_leonardo_yague">
			<path class="st3 fase_0" d="M413.59,113.71l0.01-0.54l-0.05-0.53l-0.17-0.15l-0.84-1.56l-0.08-0.49l-0.19-0.18l0.11-0.3l0.16-0.11
				l-0.08-0.42l-0.25-0.22l-0.74-0.36l-0.25-0.01l0.2-0.24l0.16-0.01l0.1-0.27l0.03-0.11l-0.08-0.15l-0.05-0.2l-0.18-0.08l0.03-0.15
				l-0.39-0.1l-0.11-0.15l-0.19-0.58l-0.29-0.43l-0.3-1.08l-0.6-0.23l-0.7-0.12l-0.16,0.18l-0.27-0.08l-0.02-0.12l-0.54-0.1
				l-0.2,0.43l-0.48,0.01l0,0l-0.39-0.29l0.05-0.25l-0.24-0.49l0-0.37l-0.38-0.22l0,0l0.2,0.62l-0.18,0.49l-0.15,0.2l-0.94-0.69
				l-0.83-0.49l-0.33,0.9l0.1,0.33l-0.04,0.21l-0.11,0.03l-0.17-0.12l-0.25-0.01l-0.03,0.19l-1.16,0.42l-0.21,0.29l-0.75-0.57
				l-0.96,0.84l0,0l-0.13-0.09l-0.41-0.27l-0.25,0.15l-0.24-0.07l-0.08-0.52l-0.15-0.18l-0.36-0.09l0,0l-0.24,0.4l0.23,0.37
				l0.2,0.09l-0.24,0.87l0.04,0.32l-0.21,0.18l0.01,0.75l0.17,0.07l0.14,0.23l-0.15,0.45l0.26-0.01l-0.04,0.12l0.1,0.21l0.06,0.47
				v0.3l-0.42,0.33l0.09,0.54l0.33,0.51l-0.34,0.35l-0.09,0.12l0.28,0.22l0.17,0.55l0.33,0.32l0.17,0.13l0.24,0.01l-0.04-0.13
				l0.29-0.07l-0.04,0.44l0.59-0.19l1.01-0.91l0.84-0.28l0.78,0.68l1.75,1.02l0.42-1.11l0.3-0.32l0.42-0.19l0.43-0.38l0.17,0.02
				l0.16,0.19l-0.05,0.09l0.15,0.11l0.09-0.02l0.25-0.11l0.09-0.13l0.6,0.09l0.01,0.32l-0.12,0.1l0.46,0.32l0.51,0.19l0.09,0.39
				l0.23-0.2l1-0.5l0.28,0.89l-0.34,1.18l0.64,0.72l0.35,1.23l0.67,0.53l-0.03-0.23l0.31-0.23l0.15-0.38l0.16-0.09l0.08-0.25
				l0.05-0.19l0.36-0.14l0.17-0.24l0.01-0.23l0.44-0.99l-0.14,0l-0.64-0.49l0.21-0.34L413.59,113.71z"></path>
		</g>
		<g id="el_burgo_de_osma">
			<path class="st3 fase_0" d="M418.43,118.35l-0.32,0.72l-0.03,0.53l-0.24,0.18l-0.7,0.36l-0.06,0.2l-0.95,0.55l-0.18,0l-0.27,0.34
				l-1.22,0.69L414,122l-0.68-0.07l-0.39,0.48l-0.02,0.25l0.09,0.22l-0.17,0.05l-0.78,0.7l0.11,0.04l0.13,0.44l0.74,0.83l0.07,0.55
				l-0.17,1.17l-0.34,0.11l-0.37-0.17l-0.26,0.09l0.14,1.05l-0.08,0.98l-0.65,0.42l0.15,0.87l-0.25,0.33l-0.08,0.31l0.1,0.1
				l-0.17,1.21l-0.19,0.35l-0.16,0.13l-0.2-0.54l-0.16-0.11l-0.35-0.04l-0.29,0.23l-0.4-0.14l-0.38-0.43l-1.41,0.62l-0.1,0.12
				l-0.3,0.88l-0.18,0.18l-0.54-0.2l-0.85-0.57l-0.86,0.2l-0.04-0.13l0.2-0.21l0.16-0.51l0.27,0.01l0.4-0.59l0.02-0.27l-0.08-0.19
				l0.16-0.26l-0.06-0.51l-0.3-0.42l-0.19-0.06l0.23-0.22l-0.04-0.25l-0.23-0.35l-0.22,0l0-0.84l-0.19-0.5l0.17-0.41l0.29,0.08
				l0.29-0.3l0-0.22l-0.33-0.25l0.16-0.39l0.01-0.31l0.3-0.27l-0.12-0.15l-0.06-0.28l0.3-0.64l-0.03-0.16l0.25-0.36l-0.07-0.17
				l0.17-0.11l-0.04-0.22l-0.2,0.01l-0.18,0.12l-0.8-0.51l-0.31-0.1l-0.44,0.05l-0.45-0.49l-0.05-0.19l0.01-0.45l0.42-0.17
				l0.52-0.33l0.44-0.64l-0.08-0.08l0.26-1.44l-0.37-0.3l-0.15,0.12l-0.72-0.12l-0.12-0.03l-0.26-0.27l-0.84-0.21l-0.35-0.87
				l-0.06-0.69l0.19-0.33l-0.06-0.43l0.33-0.15l0.53-0.07l0.58-0.32l0.16,0.06l0.33-0.42l0.41-0.12l0.24-0.27l0-0.6l-0.09-0.25
				l0.42-1.11l0.3-0.32l0.42-0.19l0.43-0.38l0,0l0.17,0.02l0.16,0.19l-0.05,0.09l0.15,0.11l0.09-0.02l0,0l0.25-0.11l0.09-0.13
				l0.6,0.09l0.01,0.32l-0.12,0.1l0.46,0.32l0.51,0.19l0.09,0.39l0,0l0.23-0.2l1-0.5l0.28,0.89l-0.34,1.18l0.64,0.72l0.35,1.23
				l0.67,0.53l-0.03-0.23l0.3-0.23l0.15-0.38l0.16-0.09l0.08-0.25l0.05-0.19l0.36-0.14l0.17-0.24l0.01-0.23l0.44-0.99l1.1,0.31
				l0.27,0.69l1.13,0.04l0.29,0.58l0.18,0.14l0.21,0l0.2,0.51l-0.04,0.38l0.54,0.12l0.03-0.14l0.79,0.76l-0.04,0.27L418.43,118.35z"></path>
		</g>
		<g id="berlanga_de_duero">
			<path id="path3882" class="st3 fase_0" d="M417.85,119.78l-0.7,0.36l-0.06,0.2l-0.95,0.55l-0.18,0l-0.27,0.34l-1.22,0.69L414,122
				l-0.68-0.07l-0.39,0.48l-0.02,0.25l0.09,0.22l-0.17,0.05l-0.78,0.7l0.11,0.04l0.13,0.44l0.74,0.83l0.07,0.55l-0.17,1.17
				l-0.34,0.11l-0.37-0.17l-0.26,0.09l0.14,1.05l-0.08,0.98l-0.65,0.42l0.15,0.87l-0.25,0.33l-0.08,0.31l0.1,0.1l-0.17,1.21
				l-0.19,0.35l-0.16,0.13l-0.2-0.54l-0.16-0.11l-0.35-0.04l-0.29,0.23l-0.4-0.14l-0.38-0.43l-1.41,0.62l-0.1,0.12l-0.3,0.88
				l-0.18,0.18l-0.02,0.19l-0.39,0.38l-0.57,0.1l-0.41,0.38l-0.5,1.38l-0.13-0.18l-0.64-0.47l-0.19,0.51l-0.23,0.34l-0.05,0.4
				l-0.35,0.62l0.67,0.18l0.29-0.01l0.46,0.22l0.2-0.15l0.66,0.16l0.34,0.42l0.09,0.01l0.16-0.22l0.16,0.03l0.26,0.2l0.42,0.1
				l0.45,0.23l0.05-0.17l0.29-0.02l0.73-0.23l0.71-0.42l0.27,0.2l0.22,0.05l1.08-0.34l0.48-0.06l0.05-0.07l0.65,0.03l0.25-0.26
				l0.16-0.46l-0.04-0.43l0.29,0.03l0.4-0.28l0.54-0.14l0.17-0.34l0.35,0.15l0.39-0.02l0.23,0.19l0.29,0.02l0.26,0.23l0.09,0.21
				l-0.25,0.78l0.29,0.36l0.07,0.24l0.02,0.33l-0.13,0.37l1.39,0.58l0.43,0.28l-0.02,0.09l0.76,0.01l0.39,0.12l0.47,0.03l0.28,0.2
				l-0.04-0.4l0.37-0.63l0.26-0.26l0.1-0.28l-0.18-1.79l0.57-0.34l0.3-0.03l0.84-0.47l0.73-0.99l0.12-0.02l0.21-0.12l0.04-0.68
				l-0.08-0.36l0.1-0.41l0.67-0.88l-0.36,0.01l-0.27-0.35l-0.16-0.01l-0.04-0.13l0.26-0.31l0.07-0.3l-1.39-0.64l-0.28-0.3
				l-0.45-0.18l-0.33-0.48l0.07-0.29l-0.32-0.31l-0.88-0.27l-0.45,0.19l-0.59-0.18l-0.26-0.26l0.2-0.84l0.28-0.77l0.12,0.1l0.33,0
				l0.08-0.12l-0.05-0.19l0.17-0.11l1.17-0.29l0.47-1.03l0.68-0.53l1.2,0.29l0.66-0.88l0.47-0.02l-0.14-0.19l-0.41-0.22l-0.37,0.21
				l-0.32-0.52l-0.22-0.09l-0.16-0.29l-0.03-0.31l-1.04-1l-0.42-0.14l-0.27-0.28l-0.13,0l-0.17,0.42l-0.33,0.1l-1.05,0.09
				l-0.31-0.65L417.85,119.78L417.85,119.78z"></path>
		</g>
	</g>
	<g id="avila">
		<g id="navarredonda_gredos">
			<path id="path2664" class="st3 fase_0" d="M327.75,180.33l-0.43,0.3l-1.07,0.22l-0.76,0.34l-0.17-0.13l-0.47-0.16l-0.4,0.1
				l-0.62-0.06l-0.22,0.51l-0.1,0.22l-0.12,0.02l-0.15,0.21l-0.76,0.14l-0.17-0.08l-0.4,0.09l-0.19-0.23l-0.24-0.12l-0.23,0.28
				l-0.21,0.1l-0.19-0.01l-0.94,0.83l-0.85,0.18l-0.35,0.04l-0.32-0.11l-0.37,0.1l-0.14,0.4l-0.5,0.42l-0.04,0.2l-0.13,0.12
				l0.33,0.49l0.02,0.18l-0.25,0.06l0.25,0.07l0.01,0.07l-0.16,0.47l-0.2,0.19l0.07,0.12l-0.02,0.15l-0.14,0.02l0.4,0.15l-0.14,0.15
				l0.26,0.54l-0.02,0.16l0.57,0.22l0.3-0.01l0.01,0.17l0.52,0.94l0.26,0.23l-0.07,0.11l0.21,0.18l0.3,0.03l-0.07,0.13l0.23,0.67
				l-0.28,0.41l-0.03,0.65l0.11,0.21l-0.15,0.47l0.14,0.25l-0.61,1.07l-0.38,0.23l0.24-0.01l0.41,0.31l0.21-0.18l0.64-0.3l0.1-0.27
				l0.61-0.21l0.34-0.27l0.25,0.04l0.42,0.38l0.07-0.36l0.21,0.01l0.07-0.19l0.23-0.09l0.29-0.32l0.27-0.19l0.26-0.03l0.29-0.27
				l0.17-0.02l0.2-0.2l0.36-0.07l0.31-0.14l0.22,0.08l0.25-0.09l0.26,0.05l-0.02-0.09l0.26-0.25l0.17,0.01l0.06-0.14l0.44-0.22
				l0.23-0.03l0.25-0.24l0.23,0.18l0.41-0.21l0.57-0.17l0.17-0.15l0.33-0.01l0.23-0.19l0.1-0.37l-0.04-0.48l-0.07-0.08l0.1-0.64
				l-0.09-0.08l-0.03-0.9l-0.2-0.99l0.01-0.24l-0.16-0.12l-0.08-0.32l-0.29-0.59l-0.24-0.17l0.18-0.04l-0.05-0.2l0.06,0.02
				l0.05-0.14l0.04-0.16l0.17-0.19l0.33-0.02l0.07-0.22l-0.1-0.08l-0.05-0.2l0.05-0.15l-0.19-0.55l0.04-0.1l-0.05-0.24l0.32-0.39
				l0.32-0.6l-0.76-0.13L327.75,180.33L327.75,180.33z"></path>
		</g>
		<g id="avila_rural">
			<path id="path2752" class="st3 fase_0" d="M343.51,155.52l-0.65,0.22l-0.49,0.53l0.08,0.25l0.18,0.08l0.05,0.12l0.01,0.02l-0.6,0.32
				l-0.19,0.21l-1.54,0.67l0.08,0.16l-0.38,0.17l-0.07-0.27l-0.13,0l-0.14,0.15l-0.35,0.01l0.04,0.07l-0.19-0.06l-0.16,0.14
				l0.03,0.07l-0.37,0.24l-0.09-0.11l0.31-0.43l-0.82-0.25l-0.21,0.36l0.21,0.37l-0.08,0.27l0.09,0.13l-0.28,0.02l0,0.11l-0.32-0.02
				l-0.1,0.15l-0.09-0.13l0.04,0.11l-0.08,0.05l0.08,0.12l0.05,0.1l-0.09,0.09l0.16,0.15l-0.14,0.7l0.1,0.19l0.15-0.03l0.22,0.25
				l-0.06,0.08l0.14,0.07l0.04,0.17l0.31,0.43l-0.11,0.09l0.04,0.11l0.2,0.05l-0.01,0.07l0.05-0.04l0.12,0.18l-0.21,0.19l0.1,0.01
				l-0.08,0.15l0.06,0.11l-0.24,0.16l0.1,0.2l-0.09-0.08l-0.04,0.2l-0.18-0.02l-0.12,0.27l-0.21,0.04l-0.03-0.15l-0.14-0.07
				l-0.15,0.18l0.08,0.17l-0.08,0.09l0.14,0.21l-0.07,0.13l-0.35,0.35l-0.11,0.09l0.04,0.14l-0.08-0.13l-0.05,0.04l0.09,0.1
				l-0.18,0.15l0.06,0.09l0.11-0.07l0.12,0.27l-0.13,0.02l0.16,0.08l-0.33-0.15l-0.01,0.21l0.08,0.26l-0.06,0.06l0.12,0.04
				l-0.06,0.22l0.05,0.06l-0.06,0.03l0.38,0.34l-0.02,0.11l0.36-0.1l0.11-0.02l0.1,0.28l0-0.11l0.42,0.15l0.14,0.15l-0.16,0.13
				l0.14,0.02l0,0.06l-0.11,0.07l-0.1,0.23v0.21l-0.18,0.12l-0.02,0.3l0.17,0.03l0.1,0.29l0.03-0.11l0.18,0.01l0.51-0.39l0.67,0.2
				l0.05,0.18l0.34,0.3l-0.17,0.33l0.15,0.41l0.16-0.01l0.39-0.22l0.07-0.13l0.09,0.06l0.25-0.08l0.02,0.23l0.14,0.09l0-0.24
				l0.13,0.11l0.03-0.07l0.21-0.02l0.09-0.16l0.19-0.09l-0.03-0.12l0.13-0.25l-0.15-0.03l-0.1-0.16l0.21-0.34l0-0.22l0.15-0.18
				l0.55,0.13l0.18-0.06l0,0.06l0.28-0.06l0.09-0.12l0.39,0l0.02-0.06l0.02,0.09l0.22,0.16l-0.22,0.33l0.16,0.11l-0.05,0.12
				l0.16,0.21l0.4-0.01l0.13-0.02l0.01,0.26l0.29-0.1l0.11,0.05l-0.02,0.18l0.16,0.11l-0.2,0.13l0.19-0.01l0.09-0.09l0.19-0.03
				l0.09,0.14l-0.18,0l-0.05,0.07l0.15,0.01l-0.04,0.18l-0.17,0.07l0.33,0.19l-0.04,0.1l0.08,0.04l-0.12,0.04l-0.04,0.26l0.13-0.01
				l-0.01,0.14l0.06-0.01l-0.02-0.09l0.17-0.05l0.26,0.23l0.07-0.04l0.08,0.16l-0.17,0l0.16,0.2l0.08-0.04l-0.02-0.1l0.25-0.11
				l0.19,0.03l-0.05,0.09l0.22,0l0.11-0.1l0.28,0.09l0.03,0.09l0.2-0.01l0.31-0.41l-0.02-0.1l0.64,0.32l-0.01-0.28l0.17,0l0.04,0.12
				l0.2,0.1l0.17-0.12l-0.16-0.4l0.24,0.15l0.06-0.05l0.11,0.21l0.21-0.06l0.03,0.07l0.13-0.08l0.12,0.04l0.07-0.1l0.4,0.12
				l0.1-0.03l-0.03-0.11l0.08-0.15l0.07,0.26l0.1-0.01l0.22-0.17l0.05-0.2l0.26,0.25l-0.02,0.11l0.18-0.16l0.15,0.14l0.37,0.67
				l0.13,0.51l-0.14,0.69l-0.29,0.37l-0.05,0.73l0.35,0.04l0.35,0.22l1.17-0.17l0.01-1.16l-0.12-1.09l-0.4-1.55l-0.11-0.11
				l-0.04-0.52l0.14-0.07l0.09-0.59l0.13-0.08l-0.15-0.35l-0.02-0.33l-0.31-0.4l-0.64-0.26l-0.22-0.23l-0.29-0.55l-0.27-0.3
				l0.05-0.4l-0.33-0.22l-0.03-0.29l-0.71-0.09l-0.7,0.3l-0.5,0l-0.21-0.36l-0.09-0.01l-0.2-0.28l-0.28-0.05l-0.23-0.17l-0.08,0.06
				l-0.14-0.05l-0.25,0.43l0.03,0.08l-0.12,0.29l0.03,0.33l-0.12,0.3l-0.36-0.16l-0.38,0.07l-0.21,0.22l-0.21-0.29l-0.26-0.01
				l-0.2-0.12l-0.06,0.04l0.16,0.37l-0.04,0.38l-0.5-0.9l-0.26-0.33l-0.01-0.23l-0.15-0.17l0.07-0.13l0.65-0.34l-0.24-0.21
				l-0.07-0.19l-0.06-0.23l0.11-0.1l0.69-0.26l0.32,0.03l0.19-0.39l0.19,0.13l0.27-0.6l0.15-0.09l0.18,0.02l-0.12-0.44l0.08-0.35
				l-0.17-0.04l-0.15,0.09l-0.17-0.13l0.03-0.44l-0.23-0.27l-0.14,0.04l-0.33-0.07l-0.03-0.09l-0.14-0.01l0.06-0.11l-0.24,0
				l0.03-0.54l-0.05,0.02l-0.18-0.57l-0.23,0.12l-0.33-0.1l0.07-0.33l0.2-0.01l0.03-0.52l0.12,0.06l0.12-0.14l0-0.27l0.11-0.01
				l-0.56-0.89l-0.45-0.41l-0.04-0.21L343.51,155.52L343.51,155.52z M342.24,157.37l0.14,0.23l0.13-0.04l0-0.15l0.23,0l0.04,0.45
				l-0.21,0.49l0.11,0.07l-0.01,0.24l-0.16,0.18l-0.02,0.19l-0.18,0.12l0.03,0.14l-0.28,0.46l0.03,0.16l-0.1,0.1l-0.01,0.31
				l-0.41-0.01l-0.07-0.07l0.02-0.17l-0.13,0.01l-0.19-0.21l-0.17,0.01l0-0.08l-0.17-0.07l0.03-0.19l-0.11-0.08l-0.1,0.08
				l-0.08-0.08l-0.25,0.29l-0.19,0.01l-0.07-0.03l0.02-0.18l-0.1-0.04l-0.04,0.06l-0.14-0.15l0.12-0.13l-0.16-0.42l0.53-0.44
				l0.16-0.1l0.29-0.03l0.13-0.22l0.17,0.04l0.12-0.23l0.29,0.18l0.09-0.19l0.15-0.05l0.02-0.28l0.12-0.16l0.17,0.05L342.24,157.37
				L342.24,157.37z"></path>
		</g>
		<g id="arevalo">
			<path id="path2688" class="st3 fase_0" d="M342.2,142.77l-0.1,0.07l-0.06,0.2l-0.72,0.33l-0.27,0.41l-0.5,0.53l-0.03,0.41v0l0,0
				l-0.28-0.08l-0.34,0.02l-0.07,0.08l-0.4-0.04l-0.73,0.4l-0.47-0.14l-0.3,0.08l-0.11,0.2l-0.29,0.22l-0.39,0.03l-0.31,0.31
				l0.05,0.16l-0.19,0.52l-0.08,0.47l0.16,0.24l-0.53,0.23l-1.12-0.08l-0.33,0.36l-0.17,0.22l-0.22,0.73l-0.11,0.78l-0.05,0.11
				l-0.36,0.23l-0.01,0.16l-0.09-0.02l-0.53,0.28l-0.12-0.01l-0.3-0.07l-0.77,0.1l-0.2,0.39l0.06,0.35l-0.2,0.17l-0.01,0.27
				l-0.17,0.05l0.1,0.1l-0.01,0.19l0.15,0.35l0.22,0.29l0.07,0.32l-0.03,0.45l0.08,0.06l0.16-0.03l0.06,0.11l1.18,0.03l0.38,0.35
				l0.26,0.05l0.25,0.27l-0.18,0.43l-0.23,0.32l0.02,0.22l-0.12,0.04l0.06,0.22l-0.29,0.18l-0.12-0.07l-0.76,0.27l-0.09,0.24
				l0.24,0.43l-0.03,0.07l-0.26,0.04l0.35,0.41l-0.06,0.16l0.16,0.08l-0.23,0.24l0.02,0.22l0.05,0.18l0.28,0.3l0.12-0.04l0.33,0.33
				l-0.06,0.2l1.06-0.49l0.11,0.07l0.4-0.49l0.36-0.13l0.05,0.06l0.43-0.13l0.36-0.08l0.34-0.51l0.41,0.25l0.9-0.15l0.05,0.08
				l0.02,0.91l0.82,0.25l-0.31,0.43l0.09,0.11l0.37-0.24l-0.03-0.07l0.16-0.14l0.19,0.06l-0.04-0.07l0.35-0.01l0.14-0.15l0.13,0
				l0.07,0.27l0.38-0.17l-0.08-0.16l1.54-0.67l0.19-0.21l0.6-0.32l-0.01-0.02l-0.05-0.12l-0.18-0.08l-0.08-0.25l0.49-0.53l0.65-0.22
				l0.43,0.07l0.04,0.21l0.45,0.41l0.56,0.89l-0.11,0.01l0,0.27l-0.12,0.14l-0.12-0.06l-0.03,0.52l-0.2,0.01l-0.07,0.33l0.33,0.1
				l0.23-0.12l0.18,0.57l0.05-0.02l-0.03,0.54l0.24,0l-0.06,0.11l0.14,0.01l0.03,0.09l0.33,0.07l0.14-0.04l0.23,0.27l-0.03,0.44
				l0.17,0.13l0.15-0.09l0.17,0.04l0.2-0.04l0-0.21l0.14-0.25l-0.13-0.24l0.28-0.56l-0.01-0.23l0.18-0.49l-0.05-0.42l0.16-0.25
				l-0.28-0.31v-0.18l0.04-1.27l-0.14-0.02l-0.2-0.27l0.28-0.32l0.15-0.32l0.82-0.61l0.16-0.02l0.07-0.42l0.24,0.04l0.03,0.1
				l0.43,0.04l0.18-0.35l-0.06-0.22l0.19-0.51l-0.08-0.08l0.06-0.15l-0.19-0.42l0.04-0.08l-0.72-0.36l-0.33,0.03l-0.64-0.4
				l-0.1,0.03l-0.14-0.28l-0.04-0.1l-0.39-0.64l-0.01-0.15l0.28-0.66l0.12-0.09l-0.08-0.07l0.07-0.36l-0.22,0l-0.29-0.71l0.15-0.29
				l-0.01-0.57l0.09-0.05l-0.18-0.39l0.13-0.27l-0.18-0.18l-0.23-0.65l-0.25-0.2l-0.68-0.66l-0.33-0.69l-0.11-0.09l-0.46-0.44
				l-0.21-0.58l-0.3,0.14l-0.35,0l-0.9-0.66L342.2,142.77z M346.88,155.32L346.88,155.32l-0.11,0.08L346.88,155.32z M342.24,157.37
				l-0.22,0.07l-0.17-0.05l-0.12,0.16l-0.02,0.28l-0.15,0.05l-0.09,0.19l-0.29-0.18l-0.12,0.23l-0.17-0.04l-0.13,0.22l-0.29,0.03
				l-0.16,0.1l-0.53,0.44l0.16,0.42l-0.12,0.13l0.14,0.15l0.04-0.06l0.1,0.04l-0.02,0.18l0.07,0.03l0.19-0.01l0.25-0.29l0.08,0.08
				l0.1-0.08l0.11,0.08l-0.03,0.19l0.17,0.07l0,0.08l0.17-0.01l0.19,0.21l0.13-0.01l-0.02,0.17l0.07,0.07l0.41,0.01l0.01-0.31
				l0.1-0.1l-0.03-0.16l0.28-0.46l-0.03-0.14l0.18-0.12l0.02-0.19l0.16-0.18l0.01-0.24l-0.11-0.07l0.21-0.49l-0.04-0.45l-0.23,0
				l0,0.15l-0.13,0.04L342.24,157.37z"></path>
		</g>
		<g id="san_pedro_arroyo">
			<path id="path2756" class="st3 fase_0" d="M336.71,156.75l-0.34,0.51l-0.36,0.08l-0.43,0.13l-0.05-0.06l-0.36,0.13l-0.4,0.49
				l-0.11-0.07l-1.06,0.49l-0.27,0.15l-0.07,0.38l-0.27,0.33l-0.01,0.23l-0.12,0.11l0,0.14l-0.12,0.09l-0.17,0.08l-0.06-0.18
				l-0.25,0.03l-0.39,0.2l-0.03,0.11l-0.23,0.08l-0.16-0.05l-0.1,0.58l0.16,0.2l0.15,0.01l0.06,0.33l-0.07,0.11l0.24,0.29
				l-0.41,0.28l-0.25,0.41l-0.24,0.13l-0.11-0.11l-0.31-0.09l-0.13-0.07l-0.15,0.06l-0.22-0.52l-0.2-0.05l-0.36-0.67l-0.15-0.03
				l-0.18,0.07l-0.29,0.12l-0.08,0.11l-0.36-0.27l-0.05-0.15l0.05-0.35l-0.11-0.07l-0.3,0.02l-0.02-0.48l-0.15-0.49l-0.16,0.1
				l-0.12-0.19l-0.25-0.09l0.1-0.15l-0.36,0.17l-0.04-0.08l-0.12,0l-0.09-0.15l-0.44,0.07l0.08,0.23l-0.11,0.01l-0.15,0.17
				l-0.29,0.08l0.03,0.11l-0.12-0.03l-0.07,0.15l-0.3,0.05l-0.06,0.2l-0.26,0.25l-0.08,0.18l0.12,0.54l-0.45,0.1l-0.08-0.33
				l-0.3,0.13l0.02-0.1l-0.05,0.02l-0.19-0.19l-0.33,0.07l-0.19-0.21l-0.27-0.08l0-0.13l-0.19-0.22l-0.1,0l-0.01-0.09l-0.09,0.02
				l-0.23-0.2l-0.06,0.05l0.07,0.17l-0.11,0.29l0.04,0.15l-0.1,0.59l-0.14,0.17l0.03,0.19l0.07,0.12l0.1-0.01l0,0.18l0.11,0.08
				l-0.03,0.08l0.18,0.21l-0.05,0.21l0.21,0.16l0.12-0.06l0.08,0.25l0.13,0l0.24,0.18l0.19,0l0.11,0.22l0.29-0.07l0.12,0.13
				l0.13-0.09l0.02,0.59l-0.17,0.1l0.22,0.15l0.72,0.19l0.13-0.19l0.11,0.01l0.12,0.14l0.45,0.12l0.08-0.13l0.11,0.02l0.24,0.33
				l0.19-0.06l0.15,0.21l0.23,0.07l0.42-0.23l-0.05,0.58l0.14,0.04l0.3-0.13l0.11,0.05l0.53-0.47l0.27,0.01l0.39,0.16l0.14,0.14
				l-0.05,0.48l0.11,0.2l0.17-0.08l0.05,0.17l0.18,0.09l0.17,0.22l0.02,0.15h0.45l0.06,0.17l0.09-0.04l0.21,0.23l0.37-0.13l0.3,0.09
				l-0.01,0.05l0.4-0.09l0.15,0.14l-0.16,0.5l0.52,0.48l-0.04,0.17l0.2,0.06l0.03,0.1l0.22,0.1l-0.17-0.12l0.12-0.08l-0.09-0.13
				l0.03-0.09l0.29-0.15l0.18,0.04l0.08,0.12l0.37-0.08l-0.01,0.13l0.18,0.02l0.38,0.65l0.11-0.09l0.29,0.11l0.25-0.2l0.41,0.14
				l-0.01,0.36l0.28-0.34l0.25,0.09l-0.08-0.11l0.16-0.16l-0.13-0.19l0.07-0.05l-0.04-0.14l0.13-0.05l-0.21-0.27l0.15-0.04
				l0.22,0.28l0.08-0.06l0.09,0.13l0.1-0.05l0.35,0.15l0.14-0.06l0.07-0.07l-0.09-0.29l0.05-0.08l0.06,0.1l0.2-0.26l-0.01-0.08
				l0.27-0.03l0.04-0.34l0.04-0.11l0.06,0.02l-0.1-0.27l0.13-0.26l-0.36,0.1l0.02-0.11l-0.38-0.34l0.06-0.03l-0.05-0.06l0.06-0.22
				l-0.12-0.04l0.06-0.06l-0.08-0.26l0.01-0.21l0.33,0.15l-0.16-0.08l0.13-0.02l-0.12-0.27l-0.11,0.07l-0.06-0.09l0.18-0.15
				l-0.09-0.1l0.05-0.04l0.08,0.13l-0.04-0.14l0.11-0.09l0.35-0.35l0.07-0.13l-0.14-0.21l0.08-0.09l-0.08-0.17l0.15-0.18l0.14,0.07
				l0.03,0.15l0.21-0.04l0.12-0.27l0.18,0.02l0.04-0.2l0.09,0.08l-0.1-0.2l0.24-0.16l-0.06-0.11l0.08-0.15l-0.1-0.01l0.21-0.19
				l-0.12-0.18l-0.05,0.04l0.01-0.07l-0.2-0.05l-0.04-0.11l0.11-0.09l-0.31-0.43l-0.04-0.17l-0.14-0.07l0.06-0.08l-0.22-0.25
				l-0.15,0.03l-0.1-0.19l0.14-0.7l-0.16-0.15l0.09-0.09l-0.05-0.1l-0.08-0.12l0.08-0.05l-0.04-0.11l0.09,0.13l0.1-0.15l0.32,0.02
				l0-0.11l0.28-0.02l-0.09-0.13l0.08-0.27l-0.21-0.37l0.21-0.36l-0.02-0.91l-0.05-0.08l-0.9,0.15L336.71,156.75L336.71,156.75z
				 M337.36,167.88l-0.37,0.08l-0.09,0.38l-0.1,0.08l0.04,0.21l0.21,0.1l-0.12,0.2l0.08,0.12l-0.1,0.09l-0.04,0.09l0.24,0.26
				l-0.03-0.27l0.13,0.16l0.34,0.1l0-0.15l-0.15-0.1l-0.01-0.13l0.03-0.07l0.08,0.05l0.03-0.21l0.23-0.09l-0.26-0.13l0.07-0.22
				l0.01-0.3L337.36,167.88L337.36,167.88z"></path>
		</g>
		<g id="munana">
			<path id="path2892" class="st3 fase_0" d="M332.01,170.91l-0.64,0.06l-0.07,0.12l-0.16,0.05l-1.04,0.17l0.16,0.09l-0.21,0.06
				l-0.17,0.23l-0.18-0.1L329.5,172l-0.63,0.02l0.09,0.1l-0.08,0.21l-0.2,0.16l-0.16,0.02l-0.18,0.24l-0.28,0l-0.01,0.09l-0.17,0.03
				l-0.09,0.13l0.07,0.09l-0.03,0.08l-0.23,0.16l-0.37,0.02l-0.73,0.23l-0.48-0.29l-0.6-0.63l-0.43,0.05l-0.23-0.1l-0.11-0.52
				l-0.32-0.82l-0.25-0.05l-0.23,0.08l-0.3,0.15l-0.16,0.48l-0.57,0.58l-1.16,0.36l-0.02,0.08l-0.1,0.32l-0.14,0.08l-0.5-0.03
				l-0.39,0.21l-0.4-0.09l-0.23-0.19l-0.23,0.08l-0.01,0.16l0.21,0.59l0.04,0.32l-0.54,0.51l0,0.33l0.11,0.16l1.02,0.59l0.03,0.38
				l0.74,0.27l0.36,0.05l0.16,0.19l0.63-0.09l0.22,0.22l0.23,0.77l-0.31,0.64l0.19,0.22l1.03-0.27l0.51,0.05l0.16,0.12l0.18-0.02
				l0.3,0.23l0.09,0.36l0.29,0.45l0.85,0.1l0.34,0.48l-0.18,0.7l1.07-0.22l0.43-0.3l0.33,0.49l0.76,0.13l0.21-1.07l0.53-0.61
				l0.23,0.26l0.14,0.03l0.14,0.28l0.5,0.03l0.42-0.16l0.1,0.05l0.35-0.04l1.59-0.03l0.09-0.48l0.21-0.2l0.13,0.01l0.37-0.36
				l0.67-0.31l0.04-0.35l0.09-0.04l-0.25-0.32l0.18-0.26l0.13-0.01l-0.13-0.05l-0.01-0.17l-0.26-0.14l-0.12-0.24l-0.13,0l-0.05-0.28
				l-0.15-0.04l0.02-0.15l0.13-0.04l0.04-0.28l0.08-0.04l0.03-0.35l0.39-0.34l-0.11-0.2l0.06-0.03l-0.03-0.11l-0.08-0.02l0-0.14
				l-0.15-0.01l-0.06,0.07l-0.18-0.05l-0.14-0.1l0.14-0.04l-0.05-0.42l0.07,0.4l0.07,0.03l-0.01-0.2l0.03,0.19l0.06-0.02l0.02-0.17
				l-0.09,0l-0.01-0.15l0.13-0.01l0.11-0.34l-0.12,0.03l-0.16-0.15l0.07-0.21l-0.41-0.13l-0.2-0.34l-0.4-0.42l0.13-0.33l-0.21-0.31
				l-0.28-0.09l-0.04-0.11l-0.37,0.17l-0.24-0.06l-0.07-0.19l-0.23,0.05l-0.2-0.2l-0.04-0.06l0.11-0.05l0.06-0.18l0.23-0.24
				L332.01,170.91L332.01,170.91z"></path>
		</g>
		<g id="piedrahita">
			<path id="path2654" class="st3 fase_0" d="M318.86,166.93l-0.09,0.2l-0.25,0.22l-0.46,0.25l-0.09,0.19l-0.42,0.32l-0.13,0.22
				l-0.6,0.24l-0.22,0.12l-0.35,0.03l-0.01-0.13l-0.27,0l-0.06,0.22l-0.28,0.1l0.17,0.1l0.04,0.16l-0.21,0.29l-0.18,0.13l-0.3-0.01
				l-0.63,0.41l-0.15-0.08l-0.37,0.17l-0.12,0.34l-0.2,0.01l-0.06,0.12l0.05,0.32l0.81,0.78l-0.51,0.18l0.21,0.36l-0.35,0.44
				l0.1,0.19l-0.15,0.35l-0.06-0.07l-0.09,0.12l-0.24,0.05l-0.18,0.25l-0.21,0.25l-0.26,0.03l-0.14-0.08l-0.09,0.03l-0.34,0.24
				l-0.06,0.42l-0.21,0.09l-0.17,0.26l-0.21,0.11l-0.09-0.06l-0.08,0.16l-0.13,0.06l-0.06,0.33l-0.68,0.11l0-0.11l-0.1-0.04
				l-0.76-0.06l-0.09-0.09l-0.48,0.09l0.03-0.45l-0.08-0.4l-0.2,0.39l-0.17-0.02l-0.25,0.27l-0.02,0.43l-0.03,0.21l0.06,0.15
				l-0.09,0.36l-0.42,0.52l-0.02,0.19l0.12,0.11l-0.02,0.38l0.07,0.04l0.29-0.26l0.38-0.17l1.08-0.43l0.67-0.19l0.18,0.04l0.29-0.07
				l0.62-0.3l0.28-0.05l0.22,0.57l-0.17,0.25l-0.14,0.95l-0.13,0.26l-0.24,0.16l-0.19,0.34l-0.13,1.21l-0.08,0.18l-0.39,0.16
				l0.01,0.33l0.05,0.59l-0.29,0.37l0.09,0.06l0.38,0.04l0.22-0.03l0.27-0.16l0.42-0.02l0.13,0.12l-0.29,0.46l0.21,0.52l0.15,0.14
				l0.05,0.28l0.69,0.45l0.31-0.37l0.22-0.04l0.14-0.23l-0.01-0.19l0.09,0.03l0.15,0.19l0.47,1.56l-0.22,0.71l0.33,0.17l0.59,0.11
				h0.34l0.21-0.12l1.08-0.32l0.49-0.24l0.35-0.28l0.13-0.12l0.04-0.2l0.5-0.42l0.14-0.4l0.37-0.1l0.32,0.11l0.35-0.04l0.85-0.18
				l0.94-0.83l0.19,0.01l0.21-0.1l0.23-0.28l0.24,0.12l0.19,0.23l0.4-0.09l0.17,0.08l0.76-0.14l0.15-0.21l0.12-0.02l0.1-0.22
				l0.22-0.51l0.62,0.06l0.4-0.1l0.47,0.16l0.17,0.13l0.76-0.34l0.18-0.7l-0.34-0.48l-0.85-0.1l-0.29-0.45l-0.09-0.36l-0.3-0.23
				l-0.18,0.02l-0.16-0.12l-0.51-0.05l-1.03,0.27l-0.19-0.22l0.31-0.64l-0.23-0.77l-0.22-0.22l-0.63,0.09l-0.16-0.19l-0.36-0.05
				l-0.74-0.27l-0.03-0.38l-1.02-0.59l-0.11-0.16l0-0.33l0.54-0.51l-0.04-0.32l-0.21-0.59l0.01-0.16l0.23-0.08l0-0.21l-0.03-0.26
				l-0.22-0.34l-0.2-0.14l-0.7-0.19l-0.1-0.14l0.25-0.22l0.03-0.57l0.49-0.53l0.03-0.17l0.13-0.22l0.35-0.1l-0.4-0.89l-0.05-0.55
				l-0.19-0.47l0.17-0.5l-0.28-0.24l-0.17-0.49L318.86,166.93L318.86,166.93z"></path>
		</g>
		<g id="barco_de_avila">
			<path id="path2668" class="st3 fase_0" d="M306.56,180.87l-0.57,0.04l-0.6,0.49l-0.81-0.04l-0.53,0.12l-0.15,0.16l-0.31,0.12
				l-0.27,0.49l-0.12,0.28l-0.52,0.51l-0.58,0.26l-0.17,0.26l-0.36,0.15l-0.3,0.13l-0.03,0.14l0.17,0.2l0.11,1.09l0.26,0.47
				l-0.34,0.56l-0.01,0.82l0.08,0.18l-0.05,0.13l0.54,0.37l-0.53,0.26l-0.16,0.32l-0.72,0.41l-0.26,0.26l-0.6,0.8l0.01,0.35
				l-0.35,0.52l0.13,0.09l0.25-0.04l0.43,0.11l0.5-0.01l0.2,0.12l0.31-0.17l0.4,0.44l1.06,0.74l-0.13,0.69l0.43,0.41l0.93,0.11
				l0.17,0.1l0.04,0.24l0.19,0.28l0.08,0.02l0.03,0.14l-0.21,0.51l0.36,0.45l0.47-0.07l0.3,0.08l0.93,0.48l0.19,0.2l0.47,0.11
				l0.15,0.18l0.28,0.07l0.19-0.04l0.22,0.1l0.49-0.12l0.37-0.27l0.39,0.06l0.03,0.1l0.08-0.06l0.36-0.25l0.69-0.17l0.11-0.88
				l0.64-0.16l0.51-0.35l0.4-1l0.7-0.16l0.71,0.26l0.66-0.4l0.59,0.05l0.26-0.32l0.36-0.12l0.23,0.06l0.16,0.09l0.64,0.09l0.31-0.18
				l0.31,0.27l0.34,0.13l0.07,0.51l0.19,0.2l0.42,0.08l0.38-0.36l0.16-0.04l0.04-0.13l0.28,0.06l0.38-0.23l0.61-1.07l-0.14-0.25
				l0.15-0.47l-0.11-0.21l0.03-0.65l0.28-0.41l-0.23-0.67l0.07-0.13l-0.3-0.03l-0.21-0.18l0.07-0.11l-0.26-0.23l-0.52-0.94
				l-0.01-0.17l-0.3,0.01l-0.57-0.22l0.02-0.16l-0.26-0.54l0.14-0.15l-0.4-0.15l0.14-0.02l0.02-0.15l-0.07-0.12l0.2-0.19l0.16-0.47
				l-0.01-0.07l-0.25-0.07l0.25-0.06l-0.02-0.18l-0.33-0.49l-0.35,0.28l-0.49,0.24l-1.08,0.32l-0.21,0.12h-0.34l-0.59-0.11
				l-0.33-0.17l0.22-0.71l-0.47-1.56l-0.15-0.19l-0.09-0.03l0.01,0.19l-0.14,0.23l-0.22,0.04l-0.31,0.37l-0.69-0.45l-0.05-0.28
				l-0.15-0.14l-0.21-0.52l0.29-0.46l-0.13-0.12l-0.42,0.02l-0.27,0.16l-0.22,0.03l-0.38-0.04l-0.16,0.2l-0.35,0.12l-0.18,0.18
				l-0.15-0.09l-0.08,0.05l-0.27-0.1l-0.22,0.02l-0.31-0.21l-0.07-0.04l0.02,0.24l-0.37,0.38l-0.07,0.22l0.02,0.07l0.34,0.16
				l0.07,0.27l-0.06,0.56l-0.1,0.23l-0.55,0.07l-0.51-0.1l-0.2,0.07l-0.4-0.58l-0.38-0.36l-0.43-0.2l0.06-0.21l0.1-0.45l0.16-0.04
				l0.23-0.23l0.04-0.29l-0.16-0.33L306.56,180.87L306.56,180.87z"></path>
		</g>
		<g id="madrigal_de_las_altas_torres">
			<path id="path2678" class="st3 fase_0" d="M328.71,143.93l0,0.11l-0.4,0.29l0.09,0.17l-1.13,0.61l-0.41,0.2l-0.33,0.01l-0.32-0.06
				l-0.31-0.19l-0.67-0.23l-0.09,0.24l-0.37,0.41l-0.06-0.07l-0.45,0.04l-0.04,0.19l-0.17,1.43l-0.42,0.3l-0.04,0.16l-0.12-0.02
				l-0.21,0.27l-0.11,0.37l0.05,0.08l-0.09,0.1l0.08,0.14l-0.11,0.84l0.15,0.24l0,0.22l0.09,0.06l-0.06,0.33l0.12,0.07l-0.05,0.16
				l0.21,0.17l0.02,0.18l0.12-0.12l0.27,0.17l1,0.36l-0.05,0.14l0.17,0.02l-0.06,0.18l0.13,0.07l-0.03,0.27l0.1,0l-0.03,0.14
				l0.36,0.4l0,0.45l0.09,0.1l-0.4,0.38l0.13,0.26l0,0.34l-0.06,0.07l0.11,0.09l0.07-0.11l0.12,0.1l0.37-0.18l0.23,0l0.23,0.1
				l0,0.06l0.23,0.01l0.56-0.32l0.54,0.36l0.36-0.33l0.18-0.31l0.14-0.08l0.07-0.2l-0.11-0.07l0.03-0.19l0.32-0.19l0.25,0.05
				l0.28,0.19l0.14-0.27l0.27-0.26l0.15-0.4l-0.04-0.09l1.09-0.25l0.21-0.25l0.19,0.11l0.24-0.1l0.17-0.05l0.01-0.27l0.2-0.17
				l-0.06-0.35l0.2-0.39l0.77-0.1l0.3,0.07l0.12,0.01l0.53-0.28l0.09,0.02l0.01-0.16l0.36-0.23l0.05-0.11l0.11-0.78l0.22-0.73
				l0.17-0.22l-0.34-0.21l-0.02-0.07l0.1-0.06l-0.17-0.17l-0.18-0.1l-0.08,0.04l-0.53-0.47l-0.31-0.07l-0.13-0.23l-0.16,0
				l-0.11-0.05l-0.49-0.24l-0.29-0.28l-0.33-0.14l-0.47-0.48l-0.29,0.09l-0.2-0.06l-0.07-0.52l-0.12-0.14l-0.39,0.06l-0.29-0.06
				l0.08-0.14l-0.21-0.06l-0.08-0.26l-0.6,0.1l-0.12-0.16l-0.14,0L328.71,143.93L328.71,143.93z"></path>
		</g>
		<g id="fontiveros">
			<path id="path2716" class="st3 fase_0" d="M331.11,151.44l-0.21,0.25l-1.09,0.25l0.04,0.09l-0.15,0.4l-0.27,0.26l-0.14,0.27
				l-0.28-0.19l-0.25-0.05l-0.32,0.19l-0.03,0.19l0.11,0.07l-0.07,0.2l-0.14,0.08l-0.18,0.31l-0.36,0.33l-0.54-0.36l-0.56,0.32
				l-0.23-0.01l0-0.06l-0.23-0.1l-0.23,0l-0.37,0.18l-0.12-0.1l-0.07,0.11l-0.11-0.09l-0.34,0.74l0.06,0.17l-0.16-0.01l0,0.11
				l-0.32,0.22l-0.18-0.1l-0.26,0.11l-0.02,0.3l0.2,0.01l0.01,0.17l-0.07,0.1l0.27,0.57l-0.14,0.08l0.03,0.07l0.2-0.03l0.05,0.08
				l-0.11,0.09l0.11,0.07l-0.09,0.12l0.2,0.29l-0.05,0.06l0.08,0.14l-0.14,0.07l0.08,0.07l-0.18,0.41l0.06,0.12l-0.24,0.07
				l-0.07-0.07l-0.16,0.12l-0.12-0.01l-0.2,0.2l0.04,0.06l-0.2,0.09l-0.03,0.09l0.11,0.05l-0.13,0.19l-0.38,0.32l0.02,0.23
				l-0.12,0.06l-0.02,0.12l-0.19,0.21l-0.27,0.09l0.02,0.16l0.06-0.05l0.23,0.2l0.09-0.02l0.01,0.09l0.1,0l0.19,0.22l0,0.13
				l0.27,0.08l0.19,0.21l0.33-0.07l0.19,0.19l0.05-0.02l-0.02,0.1l0.3-0.13l0.08,0.33l0.45-0.1l-0.12-0.54l0.08-0.18l0.26-0.25
				l0.06-0.2l0.3-0.05l0.07-0.15l0.12,0.03l-0.03-0.11l0.29-0.08l0.15-0.17l0.11-0.01l-0.08-0.23l0.44-0.07l0.09,0.15l0.12,0
				l0.04,0.08l0.36-0.17l-0.1,0.15l0.25,0.09l0.12,0.19l0.16-0.1l0.15,0.49l0.02,0.48l0.3-0.02l0.1,0.07l-0.05,0.35l0.05,0.15
				l0.36,0.27l0.08-0.11l0.29-0.12l0.18-0.07l0.15,0.03l0.36,0.67l0.2,0.05l0.22,0.52l0.15-0.06l0.13,0.07l0.31,0.09l0.11,0.11
				l0.24-0.13l0.25-0.41l0.41-0.28l-0.24-0.29l0.07-0.11l-0.06-0.33l-0.15-0.01l-0.16-0.2l0.1-0.58l0.16,0.05l0.23-0.08l0.03-0.11
				l0.39-0.2l0.25-0.03l0.06,0.18l0.17-0.08l0.12-0.09l0-0.14l0.12-0.11l0.01-0.23l0.27-0.33l0.07-0.38l0.27-0.15l0.06-0.2
				l-0.33-0.33l-0.12,0.04l-0.28-0.3l-0.05-0.18l-0.02-0.22l0.23-0.24l-0.16-0.08l0.06-0.16l-0.35-0.41l0.26-0.04l0.03-0.07
				l-0.24-0.43l0.09-0.24l0.76-0.27l0.12,0.07l0.29-0.18l-0.06-0.22l0.12-0.04l-0.02-0.22l0.23-0.32l0.18-0.43l-0.25-0.27
				l-0.26-0.05l-0.38-0.35l-1.18-0.03l-0.06-0.11l-0.16,0.03l-0.08-0.06l0.03-0.45l-0.07-0.32l-0.22-0.29l-0.15-0.35l0.01-0.19
				l-0.1-0.1l-0.24,0.1L331.11,151.44L331.11,151.44z"></path>
		</g>
		<g id="munico">
			<path id="path2806" class="st3 fase_1" d="M322.84,162.42l-0.12,0.13l0.01,0.21l-0.18,0.03l-0.03,0.13l-0.44,0.32l-0.12,0.42
				l0.09,0.06l-0.25,0.41l-0.98,0.52l-0.42,0.33l0.08,0.12l0.11,0.69l-0.53,0.25l-0.19,0.25l-0.72,0.31l-0.21,0.18l0.05,0.28
				l0.17,0.49l0.28,0.24l-0.17,0.5l0.19,0.47l0.05,0.55l0.4,0.89l-0.35,0.1l-0.13,0.22l-0.03,0.17l-0.49,0.53l-0.03,0.57l-0.25,0.22
				l0.1,0.14l0.7,0.19l0.2,0.14l0.22,0.34l0.03,0.26l0,0.21l0.23,0.19l0.4,0.09l0.39-0.21l0.5,0.03l0.14-0.08l0.1-0.32l0.02-0.08
				l1.16-0.36l0.57-0.58l0.16-0.48l0.3-0.15l0.23-0.08l0.25,0.05l0.32,0.82l0.11,0.52l0.23,0.1l0.43-0.05l0.6,0.63l0.48,0.29
				l0.73-0.23l0.37-0.02l0.23-0.16l0.03-0.08l-0.07-0.09l0.09-0.13l0.17-0.03l0.01-0.09l0.28,0l0.18-0.24l0.16-0.02l0.2-0.16
				l0.08-0.21l-0.09-0.1l0.63-0.02l0.19-0.41l0.18,0.1l0.17-0.23l0.21-0.06l-0.16-0.09l1.04-0.17l0.16-0.05l0.07-0.12l0.64-0.06
				l0.06,0l0.31-0.59l-0.1-0.05l0.08-0.12l-0.12-0.25l-0.32-0.36l-0.34-0.12l0.06-0.22l-0.07-0.13l-0.13-0.05l0.12-0.52l0.11,0.04
				l0.21-0.32l0.29-0.08l0.17-0.23l-0.03-0.23l0.04-0.17l-0.52-0.48l0.16-0.5l-0.15-0.14l-0.4,0.09l0.01-0.05l-0.3-0.09l-0.37,0.13
				l-0.21-0.23l-0.09,0.04l-0.06-0.17h-0.45l-0.02-0.15l-0.17-0.22l-0.18-0.09l-0.05-0.17l-0.17,0.08l-0.11-0.2l0.05-0.48
				l-0.14-0.14l-0.39-0.16l-0.27-0.01l-0.53,0.47l-0.11-0.05l-0.3,0.13l-0.14-0.04l0.05-0.58l-0.42,0.23l-0.23-0.07l-0.15-0.21
				l-0.19,0.06l-0.24-0.33l-0.11-0.02l-0.08,0.13l-0.45-0.12l-0.12-0.14l-0.11-0.01l-0.13,0.19c-0.36-0.05-0.65-0.14-0.94-0.34
				l0.17-0.1l-0.02-0.59l-0.13,0.09l-0.12-0.13l-0.29,0.07l-0.11-0.22l-0.19,0l-0.24-0.18l-0.13,0l-0.08-0.25l-0.12,0.06
				L322.84,162.42z"></path>
		</g>
		<g id="avila_estacion">
			<path id="path2856" class="st3 fase_0" d="M338.23,166.47l-0.16,0.13l0.14,0.02l0,0.06l-0.11,0.07l-0.1,0.23l0,0.21l-0.18,0.12
				l-0.02,0.3l0.17,0.03l0.1,0.29l0.03-0.11l0.18,0.01l0.51-0.39l0.67,0.2l0.05,0.18l0.34,0.3l-0.17,0.33l0.15,0.41l0.16-0.01
				l0.39-0.22l0.07-0.13l0.09,0.06l0.25-0.08l0.02,0.23l0.14,0.09l0-0.24l0.13,0.11l0.03-0.07l0.21-0.02l0.09-0.16l0.19-0.09
				l-0.03-0.12l0.13-0.25l-0.15-0.03l-0.1-0.16l0.21-0.34l0-0.22l0.15-0.18l0.55,0.13l0.18-0.06l0,0.06l0.28-0.06l0.09-0.12l0.39,0
				l0.02-0.06l0.02,0.09l0.22,0.16l-0.22,0.33l0.16,0.11l-0.05,0.12l0.16,0.21l0.4-0.01l0.13-0.02l0.01,0.26l0.29-0.1l0.11,0.05
				l-0.02,0.18l0.16,0.11l-0.2,0.13l0.19-0.01l0.09-0.09l0.19-0.03l0.09,0.14l-0.18,0l-0.05,0.07l0.15,0.01l-0.04,0.18l-0.17,0.07
				l0.33,0.19l-0.04,0.1l0.08,0.04l-0.12,0.04l-0.04,0.26l0.13-0.01l-0.01,0.14l0.06-0.01l-0.02-0.09l0.17-0.05l0.26,0.23l0.07-0.04
				l0.08,0.16l-0.17,0l0.16,0.2l0.08-0.04l-0.02-0.1l0.25-0.11l0.19,0.03l-0.05,0.09l0.22,0l0.11-0.1l0.28,0.09l0.03,0.09l0.2-0.01
				l0.31-0.41l-0.02-0.1l0.64,0.32l-0.01-0.28l0.17,0l0.04,0.12l0.2,0.1l0.17-0.12l-0.16-0.4l0.24,0.15l0.06-0.05l0.11,0.21
				l0.21-0.06l0.03,0.07l0.13-0.08l0.12,0.04l0.07-0.1l0.4,0.12l0.1-0.03l-0.03-0.11l0.08-0.15l0.07,0.26l0.1-0.01l0.22-0.17
				l0.05-0.2l0.26,0.25l-0.02,0.11l0.18-0.16l0.15,0.14l0.37,0.67l0.13,0.51l-0.14,0.69l-0.29,0.37l-0.05,0.73l-0.16,0.02
				l-0.38,0.79l-0.14-0.04l-0.35,0.06l-0.75-0.12l-0.12-0.08l0.13-0.32l0-0.31l-0.54-0.31l-0.28-0.4l-0.16-0.03l-0.06-0.1
				l-0.67-0.07l-0.22-0.21l-0.05-0.29l-0.27,0.13l0.12-0.27l-0.13-0.12l-0.24,0.02l-0.19,0.15l-0.04-0.05l-0.03,0.09l-0.14-0.05
				l-0.34,0.21l-0.14,0.2l-0.22,0.08l0.15,0.09l0.3,0.6l-0.03,0.07l-0.16,0.08l-0.68-0.15l-0.08-0.3l-0.15-0.12l-0.22,0.06
				l-0.01,0.46l-0.09,0.08l0.15,0.28l-0.2,0.14l-0.02-0.11l-0.15-0.11l-0.05,0.07l-0.13-0.01l0.13,0.11l-0.11,0.04l0.11,0.09
				l0.01,0.19l0.11,0.11l-0.24-0.08l-0.05-0.13l-0.19,0.08l-0.01,0.12l-0.11-0.06l-0.2,0.2l-0.06-0.07l-0.06,0.04l-0.02,0.32
				l0.2-0.09l0.1,0.2l-0.18,0.21l-0.1-0.23l-0.07,0l-0.1,0.29l-0.14,0.14l-0.05,0.41l-0.15-0.29l-0.17,0.08l-0.29-0.16l-0.19,0.01
				l-0.16-0.22l0.09-0.2l-0.24-0.1l-0.01,0.33l0.13,0.09l-0.09,0.2l-0.17,0.11l0.1,0.1l-0.12,0.24l0.19,0.1l-0.3,0.12l0.01,0.51
				l0.18,0.35l0.02,0.26l-0.11,0.22l-0.21,0.09l-0.3,0.09l-0.05-0.42l-0.4,0.23l-0.02-0.12l-0.15-0.15l-0.15-0.03l0.01-0.41
				l-0.3-0.13l-0.19,0.16l-0.34-0.08l0.06,0.19l-0.11,0.02l-0.06,0.13l-0.38-0.15l-0.18,0.15l-0.3,0.02l-0.11-0.13l0.21-0.12
				l0.07-0.17l0.33,0.05l0.27-0.26l-0.14-0.02l0-0.25l0.1-0.05l-0.17,0.04l-0.36-0.11l0.32-0.09l0.17,0.05l-0.21-0.1l0.06-0.15
				l-0.07,0.01l0.12-0.12l-0.06-0.12l0.24-0.01l-0.07-0.06l0.2-0.23l0.08-0.02l0,0.13l0.07,0.03l0.03-0.11l-0.07-0.07l0.15-0.09
				l-0.03-0.11l0.09-0.03l0.03,0.08l0.08-0.02l-0.01-0.08l0.19-0.02l0.03-0.24l0.06-0.03l0.03-0.61l0.25-0.09l0.05-0.27l0.19-0.21
				l-0.17-0.06l0.09-0.12l-0.2-0.06l0-0.05l-0.15,0.08l-0.41,0l-0.04-0.09l-0.18,0.08l-0.03-0.27l-0.13-0.08l-0.03,0.15l-0.15-0.13
				l-0.12,0l-0.01-0.06l0.11-0.03l-0.18-0.27l0.01-0.11l0.52,0.01l0.12,0.06l0.08-0.12l0.01-0.07l-0.41-0.04l-0.01-0.14h-0.19
				l0-0.16l-0.16-0.02l0.01-0.08l0.11,0l0.01-0.16l-0.06-0.03l0.03-0.08l0.07,0.03l0.07-0.22l0.16,0.06l0-0.08l0.09-0.03l-0.09-0.07
				l0.01-0.15l-0.1-0.08l-0.12,0.03l-0.13-0.14l0.03-0.07l-0.33-0.06l0.08-0.07l-0.07-0.04l0.03-0.19l-0.16-0.16l-0.11,0.13
				l-0.12-0.19l-0.31-0.09l-0.08,0.08l-0.09-0.07l0.16-0.3l-0.08-0.17l-0.71-0.04l0.01-0.3l-0.21-0.26l-0.37,0.08l-0.19,0.08
				l-0.03-0.17l0.06-0.08l-0.11-0.11l0.07-0.07l-0.09-0.29l0.05-0.08l0.06,0.1l0.2-0.26l-0.01-0.08l0.27-0.03l0.04-0.34l0.04-0.11
				l0.06,0.02l-0.1-0.27l0.13-0.26l0.11-0.02l0.1,0.28l0-0.11l0.42,0.15L338.23,166.47L338.23,166.47z"></path>
		</g>
		<g id="avila_rural">
			<path class="st3 fase_0" d="M356.45,177.77l0.16-0.33l0-0.36l-0.32,0.13l-0.74,0.01l-1.22-0.28l-1.39-0.03l-0.29-0.31l-0.44,0.04
				l-0.42,0.15l-0.1,0.18l-0.14-0.04l-0.03-0.27l-0.19-0.19l0.02-0.23l-0.08-0.15l-0.15-0.06l-0.1,0.16l-0.11-0.01l-0.03-0.15
				l0.1-0.26l-0.21-0.42l-0.22-0.1l-0.22,0.02l-0.11-0.2l-0.14,0.06l0.05-0.21l-0.14-0.38l0-0.99l0.4-0.38l-0.14-0.04l-0.35,0.06
				l-0.75-0.12l-0.12-0.08l0,0l-0.09,0.04l0,0l0.09-0.04l0.13-0.32l0-0.31l-0.54-0.31l-0.28-0.4l-0.16-0.03l-0.06-0.1l-0.67-0.07
				l-0.22-0.21l-0.05-0.29l-0.27,0.13l0.12-0.27l-0.13-0.12l-0.24,0.02l-0.19,0.15l-0.04-0.05l-0.03,0.09l-0.14-0.05l-0.34,0.21
				l-0.14,0.2l-0.22,0.08l0.15,0.09l0.3,0.6l-0.03,0.07l-0.16,0.08l-0.68-0.15l-0.08-0.3l-0.15-0.12l-0.22,0.06l-0.01,0.46
				l-0.09,0.08l0.15,0.28l-0.2,0.14l-0.02-0.11l-0.15-0.11l-0.05,0.07l-0.13-0.01l0.13,0.11l-0.11,0.04l0.11,0.09l0.01,0.19
				l0.11,0.11l-0.24-0.08l-0.05-0.13l-0.19,0.08l-0.01,0.12l-0.11-0.06l-0.2,0.2l-0.06-0.07l-0.06,0.04l-0.02,0.32l0.2-0.09l0.1,0.2
				l-0.18,0.21l-0.1-0.23l-0.07,0l-0.1,0.29l-0.14,0.14l-0.05,0.41l-0.15-0.29l-0.17,0.08l-0.29-0.16l-0.2,0.02l-0.16-0.22l0.09-0.2
				l-0.24-0.1l-0.01,0.33l0.13,0.09l-0.09,0.2l-0.17,0.11l0.1,0.1l-0.12,0.24l0.19,0.1l-0.3,0.12l0.01,0.51l0.18,0.35l0.02,0.26
				l-0.11,0.22l-0.21,0.09l-0.3,0.09l-0.05-0.42l-0.4,0.23l-0.02-0.12l-0.15-0.15l-0.15-0.03l0.01-0.41l-0.3-0.13l-0.19,0.16
				l-0.34-0.08l0.06,0.19l-0.11,0.02l-0.06,0.13l-0.38-0.15l-0.18,0.15l-0.3,0.02l-0.11-0.13l0.21-0.12l0.07-0.17l0.33,0.05
				l0.27-0.26l-0.14-0.02l0-0.25l0.1-0.05l-0.17,0.04l-0.36-0.11l0.32-0.09l0.17,0.05l-0.21-0.1l0.06-0.15l-0.07,0.01l0.12-0.12
				l-0.06-0.12l0.24-0.01l-0.07-0.06l0.2-0.23l0.08-0.02l0,0.13l0.07,0.03l0.03-0.11l-0.07-0.07l0.15-0.09l-0.03-0.11l0.09-0.03
				l0.03,0.08l0.08-0.02l-0.01-0.08l0.19-0.02l0.03-0.24l0.06-0.03l0.03-0.61l0.25-0.09l0.05-0.27l0.19-0.21l-0.18-0.06l0.09-0.12
				l-0.2-0.06l0-0.05l-0.15,0.08l-0.41,0l-0.04-0.09l-0.18,0.08l-0.03-0.27l-0.13-0.08l-0.03,0.15l-0.15-0.13l-0.12,0l-0.01-0.06
				l0.11-0.03l-0.18-0.27l0.01-0.11l0.52,0.01l0.12,0.06l0.08-0.12l0.01-0.07l-0.41-0.04l-0.01-0.14h-0.19l0-0.16l-0.16-0.02
				l0.01-0.08l0.11,0l0.01-0.16l-0.06-0.03l0.03-0.08l0.07,0.03l0.07-0.22l0.16,0.06l0-0.08l0.09-0.03l-0.09-0.07l0.01-0.15
				l-0.1-0.08l-0.12,0.03l-0.13-0.14l0.03-0.07l-0.33-0.06l0.08-0.07l-0.07-0.04l0.03-0.19l-0.16-0.16l-0.11,0.13l-0.12-0.19
				l-0.31-0.09l-0.08,0.08l-0.09-0.07l0.16-0.3l-0.08-0.17l-0.71-0.04l-0.07,0.22l0.26,0.13l-0.23,0.09l-0.03,0.21l-0.08-0.05
				l-0.03,0.07l0.01,0.13l0.15,0.1l0,0.15l-0.34-0.1l-0.13-0.16l0.03,0.27l-0.24-0.26l0.04-0.09l0.1-0.09l-0.08-0.12l0.12-0.2
				l-0.21-0.1l-0.04-0.21l0.1-0.08l0.09-0.38l-0.19,0.08l-0.03-0.17l0.06-0.08l-0.11-0.11l-0.14,0.06l-0.35-0.15l-0.1,0.05
				l-0.09-0.13l-0.08,0.06l-0.22-0.28l0,0l-0.15,0.04l0.21,0.27l-0.13,0.05l0.04,0.14l-0.07,0.05l0.13,0.19l-0.16,0.16l0.08,0.11
				l-0.25-0.09l-0.28,0.34l0.01-0.36l-0.41-0.14l-0.25,0.2l-0.29-0.11l-0.11,0.09l-0.38-0.65l-0.18-0.02l0.01-0.13l-0.37,0.08
				l-0.08-0.12l-0.18-0.04l-0.29,0.15l-0.03,0.09l0.09,0.13l-0.12,0.08l0.17,0.12l-0.22-0.1l-0.03-0.1l-0.2-0.06l0.03,0.23
				l-0.17,0.23l-0.29,0.08l-0.21,0.32l-0.11-0.04l-0.12,0.52l0.13,0.05l0.07,0.13l-0.06,0.22l0.34,0.12l0.32,0.36l0.12,0.25
				l-0.08,0.12l0.1,0.05l-0.31,0.59l-0.06,0l-0.05,0.18l-0.23,0.24l-0.06,0.18l-0.11,0.05l0.04,0.06l0.2,0.2l0.23-0.05l0.07,0.19
				l0.24,0.06l0.37-0.17l0.04,0.11l0.28,0.09l0.21,0.31l-0.13,0.33l0.4,0.42l0.2,0.34l0.41,0.13l-0.07,0.21l0.16,0.15l0.12-0.03
				l-0.11,0.34l-0.13,0.01l0.01,0.15l0.09,0l-0.02,0.17l-0.06,0.02l-0.03-0.19l0.01,0.2l-0.07-0.03l-0.07-0.4l0.05,0.42l-0.14,0.04
				l0.14,0.1l0.18,0.05l0.06-0.07l0.15,0.01l0,0.14l0.08,0.02l0.03,0.11l-0.06,0.03l0.11,0.2l-0.39,0.34l-0.03,0.35l-0.08,0.04
				l-0.04,0.28l-0.13,0.04l-0.02,0.15l0.15,0.04l0.05,0.28l0.13,0l0.12,0.24l0.26,0.14l0.01,0.17l0.13,0.05l-0.13,0.01l-0.18,0.26
				l0.25,0.32l-0.09,0.04l-0.04,0.35l-0.67,0.31l-0.37,0.36l-0.13-0.01l-0.21,0.2l-0.09,0.48l-1.59,0.03l-0.35,0.04l-0.1-0.05
				l-0.42,0.16l-0.5-0.03l-0.14-0.28l-0.14-0.03l-0.23-0.26l-0.53,0.61l-0.21,1.07l-0.32,0.6l-0.32,0.39l0.05,0.24l0.76-0.72
				l0.28-0.38l0.04-0.17l0.19-0.14l0.39-0.2l0.26,0.14l0.37-0.18l0.38,0.42l0.71-0.07l0.63-0.22l0.24,0.07l0.4-0.17l0.4-0.04
				l0.24,0.53l-0.41,0.92l0.38-0.35l0.16-0.33l0.03-0.29l-0.13-0.33l-0.15-0.12l0.68-0.38l0.79-0.24l0.64,0.17l0.33-0.05l0.13,0.08
				l0.05,0.17l0.4,0.07l0.18,0.23l0.2,0.01l0.54,0.33l0.28-0.06l0.13-0.07l0.02-0.13l0.23-0.04l0.38-0.4l1.54-0.51l0.41,0l0.27-0.14
				v0l-0.01,0.2l0.16,0.5l-0.04,0.09l0.21,0.18l0.26,0.68l-0.21,0.38l0.23,0.49l-0.09,0.54l0,0.2l0.07,0.03l0.32,0.31l0.71-0.05
				l0.6,0.27l0.16,0.19l-0.01,0.22l0.08,0.15l-0.04,0.15l0.18,0.07l0.23-0.15l0.26,0.04l0.05,0.15l-0.06,0.95l0.7-0.27l0.22,0.2
				l0.07,0.32l-0.07,0.52l-0.38,0.82l0.27,0.1l0.55,0.55l0.3,0.1l0.35,0.31l0.36,0.21l0.33-0.08l0.18,0.07l0.95-0.06l0.51-2.65
				l-0.63-1.41l0.24-0.36l-0.01-0.39l0.18-0.23l0.3-0.36l-0.02-0.37l0.36-0.12l0.1-0.13l-0.21-0.48l0.05-0.17l0.11-0.07l-0.05-0.39
				l0.17-0.03l-0.09-0.26l-0.12-0.03l-0.1-0.25l0.82-0.7l0.15-0.31l0.25-0.23l0.08-0.45l0.32-0.74l0.54,0.2l0.84-0.43l0.43-0.31
				l0.81,0.1l0.65-0.08l0.55,0.12l0.53-0.02l0.71,0.33l0.84,0.18l0.88,0.7l0.07-0.78L356.45,177.77z"></path>
		</g>
		<g id="las_navas_marques">
			<path id="path2886" class="st3 fase_0" d="M362.32,168.99l-0.16,0.12l-1.13,0.16l-0.68,0.25l-0.53-0.09l-0.24,0.08l-0.06-0.06
				l-0.23,0.19l-0.28,0.05l-0.32,0.2l-0.07,0.27l0.21,0.04l0.27,0.46l0.27,0.17l-0.04,0.56l-0.42-0.08l-1.36,0.11l-0.73,0.28
				l-0.34,0.01l-0.28,0.42l0.09,0.13l-1.3-0.02l-0.97,0.14l-0.22,0.09l-0.54-0.06l-0.5,0.05l-1.17,0.17l-0.35-0.22l-0.35-0.04
				l-0.16,0.02l-0.38,0.79l-0.4,0.38l0,0.99l0.14,0.38l-0.05,0.21l0.14-0.06l0.11,0.2l0.22-0.02l0.22,0.1l0.21,0.42l-0.1,0.26
				l0.03,0.15l0.11,0.01l0.1-0.16l0.15,0.06l0.08,0.15l-0.02,0.23l0.19,0.19l0.03,0.27l0.14,0.04l0.1-0.18l0.42-0.15l0.44-0.04
				l0.29,0.31l1.39,0.03l1.22,0.28l0.74-0.01l0.32-0.13l0.07-0.19l0.59-0.41l0.76-0.3l0.16-0.96l0.16-0.39l-0.07-0.31l0.17-0.31
				l0.01-0.17l-0.46-0.23l0.06-0.95l-0.05-0.11l0.42-0.14l0.28,0.08l0.06,0.35l0.36,0.92l0.26,0.11l0.54,0.06l1.1-0.14l0.9-0.45
				l0.67-0.16l0.61-0.38l-0.07-0.12l-0.23-0.01l-0.18-0.27l0.1-0.18l-0.05-0.21l0.15-0.29l0.01-0.24l-0.05-0.86l-0.18-0.16l0-0.37
				l0.07-0.32l0.47-0.55l-0.19-0.18l-0.29,0.04L362.32,168.99L362.32,168.99z"></path>
		</g>
		<g id="cebreros">
			<path id="path2988" class="st3 fase_0" d="M351.7,177.36l-0.43,0.31l-0.84,0.43l-0.54-0.2l-0.32,0.74l-0.08,0.45l-0.25,0.23
				l-0.15,0.31l-0.82,0.7l0.1,0.25l0.12,0.03l0.09,0.26l-0.17,0.03l0.05,0.39l-0.11,0.07l-0.05,0.17l0.21,0.48l-0.1,0.13l-0.36,0.12
				l0.02,0.37l-0.3,0.36l-0.18,0.23l0.01,0.39l-0.24,0.36l0.63,1.41l-0.51,2.65l0.14,0.11l0.33,0.08l0.48-0.23l0.47-0.15l0.54-0.39
				l0.11-0.34l0.32-0.34l0.19-0.14l0.23-0.05l0.28,0.11l-0.02,0.36l0.09,0.21l-0.09,0.55l0.13,0.29l0.18,0.14l0.23-0.01l0.14-0.23
				l0.31-0.06l0.04-0.08l0.37-0.16l0.05-0.22l-0.08-0.51l0.04-0.86l-0.12-0.31l0.06-0.15l0.17-0.08l-0.01-0.17l0.2-0.24l0.35-0.25
				l0.06-0.21l0.18,0l0.14-0.22l0.53-0.05l0.96,0.21l0.45-0.02l0.19-0.21l0.36-0.09l0.73,0.25l0.22-0.1l0.32,0l-0.22-0.16l0.09-0.27
				l0-0.31l-0.12-0.02l-0.09,0.07l-0.18-0.1l-0.06-0.13l-0.2-0.07l0.04-0.15l-0.06-0.32l0.35-0.19l-0.12-0.1l0.17-0.07l-0.06-0.22
				l0.38-0.04l0.1-0.11l-0.04-0.25l-0.21-0.06l0.02-0.16l0.05-0.05l0.19,0.12l0.07-0.03l-0.08-0.19l0.17-0.13l-0.12-0.1l0.07-0.13
				l-0.12-0.04l-0.11-0.34l0-1.32l-0.06-0.3l-0.11-0.02l0.18-0.55l-0.88-0.7l-0.84-0.18l-0.71-0.33l-0.53,0.02l-0.55-0.12
				l-0.65,0.08L351.7,177.36L351.7,177.36z"></path>
		</g>
		<g id="burgohondo">
			<path id="path3012" class="st3 fase_0" d="M340.62,179.63l-0.27,0.14l-0.41,0l-1.54,0.51l-0.38,0.4l-0.23,0.04l-0.02,0.13l-0.13,0.07
				l-0.28,0.06l-0.54-0.33l-0.2-0.01l-0.18-0.23l-0.4-0.07l-0.05-0.17l-0.13-0.07l-0.33,0.05l-0.64-0.17l-0.79,0.24l-0.68,0.38
				l0.15,0.12l0.13,0.33l-0.03,0.29l-0.16,0.33l-0.38,0.35l0.41-0.92l-0.24-0.53l-0.4,0.04l-0.4,0.17l-0.24-0.07l-0.63,0.22
				l-0.71,0.07l-0.38-0.42l-0.37,0.18l-0.26-0.14l-0.39,0.2l-0.19,0.14l-0.04,0.17l-0.28,0.38l-0.76,0.72l-0.04,0.1l0.19,0.55
				l-0.05,0.15l0.05,0.2l0.1,0.08l-0.07,0.22l-0.33,0.02l-0.17,0.19l-0.04,0.16l-0.05,0.14l-0.06-0.02l0.05,0.2l-0.18,0.04
				l0.24,0.17l0.29,0.59l0.08,0.32l0.16,0.12l-0.01,0.24l0.2,0.99l0.37-0.16l0.39,0.01l0.62,0.36l0.38,0.35l0.78,0.26l0.17,0.01
				l0.2-0.04l0.14,0.58l0.5,0.78l-0.05,0.09l0.04,1.19l0.04,0.09l0.26,0.08l0.48,0.13l0.22-0.41l0.28-0.18l0.67-0.13l0.19-0.33
				l0.32-0.21l0.38,0l0.18-0.22l0.48-0.13l0.33-0.34l0.27-0.09l0.57,0.34l0.21,0.07l0.14,0.15l0.17-0.14l0.44-0.13l0.12-0.23
				l0.22-0.21l0.38-0.2l0.31-0.25l0.42,0.02l0.8-0.13l0.25-0.35l0.21-0.15l0.12,0.06l0.3-0.07l0.13,0.08l0.21-0.04l0.32,0.08
				l0.53-0.17l1.32-0.26l0.34,0.05l0.43-0.08l0.38-0.82l0.07-0.52l-0.07-0.32l-0.22-0.2l-0.7,0.27l0.06-0.95l-0.05-0.15l-0.26-0.04
				l-0.23,0.15l-0.18-0.07l0.04-0.15l-0.08-0.15l0.01-0.22l-0.16-0.19l-0.6-0.27l-0.71,0.05l-0.32-0.31l-0.07-0.03l0-0.2l0.09-0.54
				l-0.23-0.49l0.21-0.38l-0.26-0.68l-0.21-0.18l0.04-0.09l-0.16-0.5L340.62,179.63L340.62,179.63z"></path>
		</g>
		<g id="mombeltran">
			<path id="path3088" class="st3 fase_0" d="M329.01,186.48l-0.37,0.16l0.03,0.9l0.09,0.08l-0.1,0.64l0.07,0.08l0.04,0.48l-0.1,0.37
				l-0.23,0.19l-0.33,0.01l-0.17,0.15l-0.57,0.17l-0.41,0.21l0.04,0.41l-0.18,0.28l-0.04,0.47l-0.12,0.22l0.07,0.29l0.23,0.27
				l-0.02,0.23l0.13,0.19l-0.03,0.45l0.08,0.15l-0.02,0.23l-0.11,0.13l0.11,0.7l0.3,0.55l-0.33,0.53l0.28,0.61l0.49,0.46l0.42,0.92
				l0.46,0.54l0.08,0.38l0.59-0.21l0.23,0l0.11,0.1l0.16,0.1l0.22-0.02l0.22,0.23l0.35-0.1l0.11-0.01l0.18,0.14l-0.18-0.63
				l-0.24-0.57l-0.14-0.77l0.05-0.26l-0.04-0.51l-0.06-0.09l0.1-0.26l0.34-0.25l0.44-0.06l0.27-0.25l-0.31-0.15l-0.43,0.12
				l-0.19-0.67l-0.16-0.26l0.03-0.33l0.13-0.2l0.59-0.43l0.21-0.71l-0.04-0.26l0.4-0.48l0.22-0.47l0.37-0.27l0.19,0.01l-0.04-0.18
				l-0.26-0.08l-0.04-0.09l-0.04-1.19l0.05-0.09l-0.5-0.78l-0.14-0.58l-0.2,0.04l-0.17-0.01l-0.78-0.26l-0.38-0.35l-0.62-0.36
				L329.01,186.48L329.01,186.48z M332.32,197.38l-0.23,0.14l-0.28,0.33l-0.4,0.13l-0.26,0.29l0.04,0.36l0.15,0.73l0.12,0.03
				l0.83-0.18l0-0.12l0.13-0.18l0.18-0.74l-0.23-0.52L332.32,197.38L332.32,197.38z"></path>
		</g>
		<g id="sotillo_de_adrada">
			<path id="path3092" class="st3 fase_0" d="M350.3,186.57l-0.23,0.05l-0.19,0.14l-0.32,0.34l-0.11,0.34l-0.54,0.39l-0.47,0.15
				l-0.01,0.36l0.23,0.13l0.22,0.38l0.1,0.44l0.35,0.29l0.27-0.08l0.28-0.2l0.22-0.03l0.53,0.08l0.31,0.05l0.38-0.07l0.07-0.7
				l-0.28-0.4l-0.23,0.01l-0.18-0.14l-0.13-0.29l0.09-0.55l-0.09-0.21l0.02-0.36L350.3,186.57L350.3,186.57z M344.21,186.81
				l-0.43,0.08l-0.34-0.05l-1.32,0.26l-0.53,0.17l-0.32-0.08l-0.21,0.04l-0.13-0.08l-0.3,0.07l-0.12-0.06l-0.21,0.15l-0.25,0.35
				l-0.8,0.13l-0.42-0.02l-0.31,0.25l-0.38,0.2l-0.22,0.21l-0.12,0.23l-0.44,0.13l-0.17,0.14l-0.04,0.29l0.08,0.38l-0.08,0.11
				l-0.15,0.86l0.24,0.42l-0.16,0.27l0.09,0.49l0.62,0.16l0.22,0.1l0.07,0.15l0.14,0.1l0.69,0.25l0.32-0.17l0.43-0.6l0.42,0.23
				l0.14,0.02l0.04-0.16l0.62-0.13l-0.06-0.26l0.63-0.08l0.16,0.85l-0.18,0.13l-0.17,0.02l0.03,0.34l0.36,0.18l0.23,0.23l-0.26,0.46
				l0.29,0.28l-0.12,0.35l0.14,0.58l-0.02,0.4l0.3,0.22l0.22-0.15l0.14,0.12l0.3,0.08l0.2,0.34l0.15,0.08l0.56,0.05l0.17-0.18
				l0.24-0.07l0.53,0.09l0.13-0.05l0.1-0.15l0.18-0.05l0.1-0.18l0.16,0l0.31-0.17l0.24-0.26l0.31-0.06l0.18-0.12l0.36,0.09l0.2-0.25
				l0.07-0.61l-0.05-0.13l-0.23-0.18l-0.13-0.94l0.52-0.39l0.02-0.08l0.07-0.56l-0.06-0.03l0.35-0.16l0.19-0.35l0.34-0.12l0.2-0.21
				l-0.35-1l-0.09-0.89l-0.18-0.13l0.06-0.03l-0.03-0.15l-0.17-0.31l0.05-0.15l-0.14-0.11l-0.95,0.06l-0.18-0.07l-0.33,0.08
				l-0.36-0.21l-0.35-0.31l-0.3-0.1l-0.55-0.55L344.21,186.81L344.21,186.81z"></path>
		</g>
		<g id="lanzahita">
			<path id="path3118" class="st3 fase_0" d="M336.29,188.34l-0.27,0.09l-0.33,0.34l-0.48,0.13l-0.18,0.22l-0.38,0l-0.32,0.21
				l-0.19,0.33l-0.67,0.13l-0.28,0.18l-0.22,0.41l-0.48-0.13l0.04,0.18l-0.19-0.01l-0.37,0.27l-0.22,0.47l-0.4,0.48l0.04,0.26
				l-0.21,0.71l-0.59,0.43l-0.13,0.2l-0.03,0.33l0.16,0.26l0.19,0.67l0.43-0.12l0.31,0.15l-0.27,0.25l-0.44,0.06l-0.34,0.25
				l-0.1,0.26l0.06,0.09l0.04,0.51l-0.05,0.26l0.14,0.77l0.24,0.57l0.18,0.63l0.2,0.11l0.26-0.29l0.4-0.13l0.28-0.33l0.23-0.14
				l0.58-0.3l0.55-0.51l0.49-0.32l0.25,0.04l0.12-0.4l0.53-0.4l0.1,0.09l0.42-0.3l0.37-0.05l0.2-0.13l0.59-0.59l-0.07-0.33
				l0.15-0.08l0.45-0.23l-0.1-0.3l-0.07,0.01l0.04-0.17l-0.09-0.31l-0.2-0.28l0.33-0.25l-0.05-0.25l0.17-0.21l-0.03-0.17l0.14-0.22
				l-0.09-0.49l0.16-0.27l-0.24-0.42l0.15-0.86l0.08-0.11l-0.08-0.38l0.04-0.29l-0.14-0.15l-0.21-0.07L336.29,188.34L336.29,188.34z
				"></path>
		</g>
		<g id="arenas_san_pedro">
			<path id="path3130" class="st3 fase_0" d="M326.72,189.74l-0.25,0.24l-0.23,0.03l-0.44,0.22l-0.06,0.14l-0.17-0.01l-0.26,0.25
				l0.02,0.09l-0.26-0.05l-0.25,0.09l-0.22-0.08l-0.31,0.14l-0.36,0.07l-0.2,0.2l-0.17,0.02l-0.29,0.27l-0.26,0.03l-0.27,0.19
				l-0.29,0.32l-0.23,0.09l-0.07,0.19l-0.21-0.01l-0.07,0.36l0,0l-0.42-0.38l-0.25-0.04l-0.34,0.27l-0.61,0.21l-0.1,0.27l-0.64,0.3
				l2.04,3.85l-0.14,0.19l0.23,0.44l-0.22,0.47l0.05,0.79l-0.07,0.2l0.02,0.36l0.11,0.59l0.47,1.19l-0.09,0.55l0.42,0.16l0.19-0.17
				l0.52-0.27l0.2,0.11l0.29-0.04l0.08-0.08l-0.01-0.2l0.63-0.4l0.78-0.79l0.16-0.44l0.27-0.02l0.15,0.21l0.08,0l0.12-0.13
				l0.04-0.51l0.3-0.04l-0.03-0.26l0.18-0.14l0.15-0.27l0.29-0.15l0.24,0.13l0.2-0.12l0.4,0.02l0.33-0.3l0.62-0.13l0.06,0.71
				l0.13,0.12l0.27,0.67l-0.03,0.51l-0.23,0.57l0.29,0.07l0.69-0.33l0.28,0.03l0.47-0.24l0.39-0.09l0.27,0.02l-0.18-0.27l0.14-0.01
				l-0.16-0.23l0.12-0.16l0.31,0.01l-0.15-0.73l-0.04-0.36l-0.2-0.11l-0.18-0.14l-0.11,0.01l-0.35,0.1l-0.22-0.23l-0.22,0.02
				l-0.16-0.1l-0.11-0.1l-0.23,0l-0.59,0.21l-0.08-0.38l-0.46-0.54l-0.42-0.92l-0.49-0.46l-0.28-0.61l0.33-0.53l-0.3-0.55l-0.11-0.7
				l0.11-0.13l0.02-0.23l-0.08-0.15l0.03-0.45l-0.13-0.19l0.02-0.23l-0.23-0.27l-0.07-0.29l0.12-0.22l0.04-0.47l0.18-0.28
				l-0.04-0.41L326.72,189.74z M322.96,196.85l0.36,0.29l-0.33,0.41l-0.14,0.31l-0.26,0.01l-0.3-0.21l-0.11-0.48l0.28-0.27
				L322.96,196.85z"></path>
		</g>
		<g id="candeleda">
			<path id="path3144" class="st3 fase_0" d="M315.37,192.32l-0.61,1.73l0,0.2l-0.18,0.28l-0.28,0.25l0.01,0.39l0.22,0.42l-0.17,0.22
				l0.07,0.17l0,0.68l0.08,0.08l0,0.4l-0.13,0.56l0.04,0.2l0.28,0.19l0,0.14l0.12,0.08l0.01,0.24l0.44,0.8l0,0.21l0.16,0.2l0.2,0.09
				l0.04,0.45l0.82,0.47l0.58-0.02l0.19-0.14l0.14,0.03l0.92-0.01l0.14-0.07l0.26,0.17l0.46-0.04l1.47,0.1l0.1,0.09l-0.05,0.68
				l0.16,0.17l0.04,0.21l0.19,0.15l0.4-0.04l0.29-0.24l0.13,0.01l0.09-0.55l-0.47-1.19l-0.11-0.59l-0.02-0.36l0.07-0.2l-0.05-0.79
				l0.22-0.47l-0.23-0.44l0.14-0.19l-2.04-3.85l-0.21,0.18l-0.41-0.31l-0.24,0.01l-0.28-0.06l-0.04,0.13l-0.16,0.04l-0.38,0.36
				l-0.42-0.08l-0.19-0.2l-0.07-0.51l-0.34-0.13l-0.31-0.27l-0.31,0.18l-0.64-0.09L315.37,192.32L315.37,192.32z M322.96,196.85
				l-0.49,0.06l-0.28,0.27l0.11,0.48l0.3,0.21l0.26-0.01l0.14-0.31l0.33-0.41L322.96,196.85L322.96,196.85z"></path>
		</g>
		<g id="avila_sur_oeste">
			<path class="st3 fase_0" d="M345.77,171.65c-0.07,0.01-0.14,0.02-0.22,0.02c-0.41,0-0.7-0.16-0.91-0.4c-0.65-0.04-1.29-0.05-1.94-0.05
				c-0.2,0.07-0.41,0.11-0.62,0.09c-0.11-0.01-0.21,0-0.32,0c-0.06,0.01-0.12,0.02-0.18,0.04c-0.06,0.02-0.13,0.04-0.19,0.06
				c-0.01,0-0.03,0.01-0.05,0.02c-0.01,0-0.01,0.01-0.02,0.01c-0.06,0.03-0.13,0.06-0.19,0.09c-0.11,0.06-0.23,0.12-0.34,0.18
				c-0.17,0.09-0.34,0.18-0.51,0.26l0.14,0l0.15-0.08l0,0.05l0.2,0.06l-0.09,0.12l0.17,0.06l-0.19,0.21l-0.05,0.27l-0.25,0.09
				l-0.03,0.61l-0.06,0.03l-0.02,0.24l-0.19,0.02l0.01,0.08l-0.08,0.02l-0.03-0.08l-0.09,0.03l0.03,0.11l-0.15,0.09l0.07,0.07
				l-0.03,0.11l-0.07-0.03l0-0.13l-0.08,0.02l-0.2,0.23l0.07,0.06l-0.24,0.01l0.06,0.12l-0.12,0.12l0.07-0.01l-0.06,0.15l0.21,0.1
				l-0.17-0.05l-0.32,0.09l0.36,0.11l0.17-0.04l-0.1,0.05l0,0.25l0.14,0.02l-0.27,0.26l-0.33-0.05l-0.07,0.17l-0.21,0.12l0.11,0.13
				l0.3-0.02l0.18-0.15l0.38,0.15l0.06-0.13l0.11-0.02l-0.06-0.19l0.34,0.08l0.19-0.16l0.3,0.13l-0.01,0.41l0.15,0.03l0.15,0.15
				l0.02,0.12l0.4-0.23l0.05,0.42l0.3-0.09l0.21-0.09l0.11-0.22l-0.02-0.26l-0.18-0.35l-0.01-0.51l0.3-0.12l-0.19-0.1l0.12-0.24
				l-0.1-0.1l0.17-0.11l0.09-0.2l-0.13-0.09l0.01-0.33l0.24,0.1l-0.09,0.2l0.16,0.22l0.19-0.01l0.29,0.16l0.17-0.08l0.15,0.29
				l0.05-0.41l0.14-0.14l0.1-0.29l0.07,0l0.1,0.23l0.18-0.21l-0.1-0.2l-0.2,0.09l0.02-0.32l0.06-0.04l0.06,0.07l0.2-0.2l0.11,0.06
				l0.01-0.12l0.19-0.08l0.05,0.13l0.24,0.08l-0.11-0.11l-0.01-0.19l-0.11-0.09l0.11-0.04l-0.13-0.11l0.13,0.01l0.05-0.07l0.15,0.11
				l0.02,0.11l0.2-0.14l-0.15-0.28l0.09-0.08l0.01-0.46l0.22-0.06l0.15,0.12l0.08,0.3l0.68,0.15l0.16-0.08l0.03-0.07L345.77,171.65z
				"></path>
		</g>
		<g id="avila_sur_este">
			<path class="st3 fase_0" d="M351.26,170.09l-0.37-0.67l-0.15-0.14l-0.18,0.16l0.02-0.11l-0.26-0.25l-0.05,0.2l-0.22,0.17l-0.1,0.01
				l-0.07-0.26l-0.08,0.15l0.03,0.11l-0.1,0.03l-0.4-0.12l-0.07,0.1l-0.12-0.04l-0.13,0.08l-0.03-0.07l-0.21,0.06l-0.11-0.21
				l-0.06,0.05l-0.24-0.15l0.16,0.4l-0.17,0.12l-0.2-0.1l-0.04-0.12l-0.17,0l0.01,0.28l-0.64-0.32l0.02,0.1l-0.31,0.41l-0.2,0.01
				l-0.03-0.09l-0.11-0.03c0.11,0.28,0.12,0.59,0.05,0.88l0.13-0.01l0.13,0.12l-0.12,0.27l0.27-0.13l0.05,0.29l0.22,0.21l0.67,0.07
				l0.06,0.1l0.16,0.03l0.28,0.4l0.54,0.31l0,0.31l-0.13,0.32l0.12,0.08l0.75,0.12l0.35-0.06l0.14,0.04l0.38-0.79l0.16-0.02
				l0.05-0.73l0.29-0.37l0.14-0.69L351.26,170.09z"></path>
		</g>
		<g id="avila_norte">
			<path class="st3 fase_0" d="M342.22,167.13l-0.39-0.09l-0.15,0.18l0,0.22l-0.21,0.34l0.1,0.16l0.15,0.03l-0.13,0.25l0.03,0.12
				l-0.19,0.09l-0.09,0.16l-0.21,0.02l-0.03,0.07l-0.13-0.11l0,0.24l-0.14-0.09l-0.02-0.23l-0.25,0.08l-0.09-0.06l-0.07,0.13
				l-0.39,0.22l-0.16,0.01l-0.15-0.41l0.17-0.33l-0.34-0.3l-0.05-0.18l-0.67-0.2l-0.51,0.39l-0.18-0.01l-0.03,0.11l-0.1-0.29
				l-0.17-0.03l0.02-0.3l0.18-0.12l0-0.21l0.1-0.23l0.11-0.07l0-0.06l-0.14-0.02l0.16-0.13l0,0l-0.14-0.15l-0.42-0.15l0,0.11
				l-0.1-0.28l-0.11,0.02l-0.13,0.26l0.1,0.27l-0.06-0.02l-0.04,0.11l-0.04,0.34l-0.27,0.03l0.01,0.08l-0.2,0.26l-0.06-0.1
				l-0.05,0.08l0.09,0.29l-0.07,0.07l0.11,0.11l-0.06,0.08l0.03,0.17l0.19-0.08l0.37-0.08l0.21,0.26l-0.01,0.3l0.71,0.04l0.08,0.17
				l-0.16,0.3l0.09,0.07l0.08-0.08l0.31,0.09l0.12,0.19l0.11-0.13l0.16,0.16l-0.03,0.19l0.07,0.04l-0.08,0.07l0.33,0.06l-0.03,0.07
				l0.13,0.14l0.12-0.03l0.1,0.08l-0.01,0.15l0.09,0.07l-0.09,0.03l0,0.08l-0.16-0.06l-0.07,0.22l-0.07-0.03l-0.03,0.08l0.06,0.03
				l-0.01,0.16l-0.11,0l-0.01,0.08l0.16,0.02l0,0.16h0.19l0.01,0.14l0.41,0.04l-0.01,0.07l-0.08,0.12l-0.12-0.06l-0.52-0.01
				l-0.01,0.11l0.18,0.27l-0.11,0.03l0.01,0.06l0.12,0l0.13,0.11c0.01,0,0.02,0,0.02,0l0.03-0.13l0.13,0.08l0.01,0.05
				c0.4,0,0.79,0.02,1.19,0.03c0.02-0.43,0.22-0.72,0.44-1.09c0.23-0.36,0.72-0.78,0.72-1C342.14,168.97,342.2,167.97,342.22,167.13
				z"></path>
		</g>
		<g id="villacastin">
			<path class="st3 fase_0" d="M350.27,163.98c0.02-0.03,0.04-0.06,0.06-0.09c0.01-0.01,0.01-0.02,0.02-0.03c0.03-0.13,0.08-0.34,0.14-0.5
				l-0.02-0.02l-0.01-0.23l-0.37-0.36l-0.03-0.32l-0.28-0.71h-0.2l-0.01,0l0-0.21l-0.25-0.31l-0.15,0.02l0.02-0.21l-0.09-0.11
				l0.14-0.49l0.29-0.24l0.36-0.14l0.1,0.04l0.34-0.24l-0.02-0.13l-0.56-0.66l-0.17-0.1l-0.45-0.02l-0.29,0.86l-0.69,0.28
				l-0.08-0.07l0.02-0.37l-0.14-0.14l-0.49-0.24l-0.04-0.14l-0.32-0.2l0.02,0.23l-0.28,0.56l0.13,0.23l-0.14,0.26v0.21l-0.2,0.04
				l-0.08,0.36l0.12,0.44l-0.19-0.02l-0.14,0.1l-0.27,0.6l-0.19-0.13l-0.2,0.38l-0.32-0.03l-0.69,0.26l-0.12,0.1l0.06,0.23l0.07,0.2
				l0.25,0.21l-0.66,0.34l-0.07,0.13l0.14,0.17l0.01,0.23l0.26,0.33l0.5,0.9l0.04-0.38l-0.16-0.37l0.07-0.04l0.2,0.13l0.26,0.01
				l0.21,0.29l0.21-0.22l0.38-0.07l0.36,0.16l0.13-0.3l-0.03-0.34l0.12-0.28l-0.03-0.08l0.25-0.43l0.14,0.04l0.08-0.06l0.23,0.17
				l0.29,0.05l0.2,0.28l0.09,0.01l0.21,0.37h0.49l0.71-0.3l0.7,0.09L350.27,163.98z"></path>
		</g>
	</g>
	<g id="burgos">
		<g id="valle_de_losa">
			<path id="path4019" class="st3 fase_1" d="M401.19,40.43l-0.18,0.16l-0.11,0.17l-0.02,0.26l-0.12,0.08l-0.3,0.15l-0.43,0.01
				l-0.45,0.39l-0.29,0.1l-0.2,0.17l-1.5,0.63l-0.23,0l-0.33-0.23l-0.09,0.05l-0.42-0.11l-0.21,0.02l-0.5,0.74l0.19,0.05l0.34,0.84
				l0.09,0.47l0.1,0.32l-0.13,0.25l-0.18-0.01l-0.35,0.27l-0.14-0.08l-0.12,0.17l-0.2,0.02l-0.69-0.12l0.04-0.46l-0.24-0.06
				l-0.1,0.22l-0.39-0.01l-0.2-0.04l-0.07-0.25l-0.25,0.17l0.22-0.53l-0.21-0.27l0.01-0.34l-0.27,0.01l-0.11,0.03l-0.01,0.21
				l-0.25,0.44l0.29,0.28l0.18,0.43l-0.1,0.3l-0.32,0.41l-0.08,0.03l0-0.15l-0.18-0.33l-0.26,0.08l-0.08-0.07l-0.22-0.41l0.22-0.41
				l-0.16-0.05l-0.17,0.41l-0.4,0.18l0.04,0.28l0.09,0.11l-0.13,0.11l-0.16,0.02l-0.04,0.09l-0.54-0.02l-0.54,0.17l0.15,0.16
				l-0.34,0.47l0.01,0.14l-0.23,0.1l-0.15,0.33l-0.12-0.03l-0.05,0.44l-0.3,0.1l-0.14,0.18l-0.02,0.05l0.3,0.11l0.14,0.23l-0.04,0.2
				l-0.25,0.15l-0.01,0.21l0.17,0.05l-0.19,0.17l0,0.46l-0.19,0.48l0.06,0.11l0.19,0.02l0,0.27l0.07,0.09l0.24,0.11l0.47,0.05
				l0.16,0.23l0.1,0.45l0.27,0l-0.03,0.51l0.08,0.15L390.68,52l0.34,0.32l0.39-0.24l-0.02-0.32l0.37,0.05l0.27-0.13l0.37,0.51
				l0.09-0.08l-0.28-0.49l0.26-0.4l0.01-0.7l-0.1-0.03l0.18-0.2l0.65-0.43l0.38-0.04l0.37,0.43l0.08,0.22l0.86-0.03l0.16-0.23
				l0-0.19l0.33,0.04l0.28,0.2l0.12-0.22l0.26,0l0.51,0.47l0.27,0.07l0.2-0.25l1.13,1.19l0.23-0.33l0.14,0.21l0.11,0.03l0.42-0.1
				l-0.03-0.78l1.16-0.57l0.11-0.63l0.23-0.41l0.19-0.08l-0.01-0.58l0.44-0.31l0.19,0.01l0.14,0.09v0.13l0.12-0.03l0.09,0.09
				l0.07-0.01l0-0.12l0.13-0.01l0.16,0.11l0.24-0.08l0.64,0.1l0.46,0.23l1,0.3l-0.12,0.11l-0.12,0.71l0.05,0.12l0.32,0.22l0.39,0.03
				l0.22,0.47l0.73-0.11l0.15-0.22l0.25,0.22l0.23,0.31l-0.04,0.06l0.17,0.01l0.58-0.11l0.25-0.16l0.36,0.12l1.27-0.26l0.4-0.11
				l0.11-0.11l0.07-0.45l0.38-0.43l0.11-0.04l0.19,0.09l0.25-0.45l0.26-0.05l0.14-0.18l-0.14-0.29l-0.24-0.15l-0.22-0.32l-0.63,0.02
				l-0.32-0.14l-0.07-0.14l0.04-0.18l-0.18-0.05l-0.53-0.4l-0.11,0l-0.17-0.21l0.24-0.26l0.08-0.21l-0.17,0l-0.48-0.65l-0.23-0.13
				l-0.1-0.12l0.04-0.23l-0.47-0.11l-0.22-0.16l-0.09,0.2l-0.39,0.17l-0.69-0.2l-0.58-0.03l-0.16-0.1l-0.21-0.01l-0.61,0.06
				l-0.99-0.45l-0.35-0.36l-0.15,0.03l-0.24-0.12l-0.14-0.19l-0.03-0.15l0.07,0l-0.15-0.13l0.06-0.09l-0.27-0.39l0.01-0.16l0.15-0.1
				l0.27-0.02l0-0.23l-0.17-0.41l-1.05-0.85L401.39,41L401.19,40.43L401.19,40.43z M395.03,50.6l-0.61,0.17l-0.14-0.03L394,51.17
				l-0.07-0.01l-0.11,0.15l-0.02,0.34l0.14,0.26l0.39,0.41l0.45,0.09l0.16-0.12l0.39,0.05l0.05-0.43l-0.32-0.17l-0.12,0.02
				l-0.17-0.31l0.1-0.25l0.21-0.22L395.03,50.6L395.03,50.6z M405.15,51.09l-0.5,0.81l-0.03,0.26l-0.24,0.14l-0.21,0.4l-0.44,0.27
				l-0.5-0.05l-0.71-0.66l-0.4,0.77l-0.26,0.2l-0.1,0.38l-0.52,0.3l0.67,0.6l0.49,0.48l0.85,0.46l0.15,0l0.38,0.29l0.45-0.53
				l-0.03-0.33l0.32-0.85l0.32-0.06l0.11-0.17l-0.05-0.22l0.14-0.26l0.2,0.04l0.08-0.33l0.31-0.33l-0.02-0.16l0.22-0.38l0.12-0.48
				l-0.33-0.09l0.03-0.26l-0.1-0.14l-0.16-0.11L405.15,51.09L405.15,51.09z"></path>
		</g>
		<g id="belorado">
			<path id="path4235" class="st3 fase_0" d="M404.8,69.86l-0.3,0.17l-0.67,0.18l-0.2,0.19l-0.2-0.03l-0.35,0.15l-0.26,0.21l-0.28,0.02
				l-0.73,0.47l-0.19-0.05l-0.24,0.09l-0.1,0.27l-0.41-0.08l-0.55,0.27l-0.44,0.77l0.31,0.63l-0.19,0.13l-0.31,0.05l-0.07-0.11
				l-0.09,0.17l-0.41,0.1l-0.12,0.17l-0.11,0.3l0.1,0.18l-0.19,0.41l-0.98-0.44l-0.48-0.59l0.03-0.16l-0.26-0.21l-0.09,0.1
				l-0.25-0.01l-0.46,0.2l-0.15,0.18l-0.06,0.28l-0.33,0.2l-0.2,0.32l0,0.44l-0.43,0.51l-0.24-0.16l-0.01-0.09l-0.19,0l-0.1-0.11
				l-0.1,0.72l0.05,0.43l0.41,0.93l-0.17,0.34l0.12,0.46l-0.03,0.23l0.37,0.64l0.09,0.55l0.65,0l0.27,0.42l0.02,0.22l-0.2,0.35
				l-0.25,0.12l0.01,0.46l0.07,0.05l0.78,0.21l0.26,0.06l0.17,0.24l0.11,0.41l0.47,0.44l-0.2,0.12l-0.09,0.27l0.07,0.22l-0.09,0.32
				l0.16,0.14l0.05,0.23l0.5,0.2l0.04,0.3l0.18,0.09l-0.03,0.2l-0.47,0.29l-0.11,0.15l0.71-0.08l0.32,0.14l0.42-0.05l0.14,0.1
				l0.32,0.54l0.69-0.19l0.13,0.27l0.26,0.19l0.25-0.04l0.53,0.28l0.09,0.36l0.31,0.24l0.91-0.55l0.64,0.4l0.27-0.09l0.11-0.13
				l0.43-0.12l0.02-0.09l0.04,0.11l0.59,0.06l0.55,0.35l0.64,0.05l0.41-0.67l0.15-0.05l-0.37-0.93l0.33-0.53L406.35,84l0.01-0.45
				l-0.31-0.99l0.04-0.16l-0.16-0.2l0.13-0.44l0.22-0.08l0.09-0.25l-0.18-0.08l-0.42-0.55l0.05-0.41l0.01-0.31l0.88-0.02l0.97-0.21
				l0.02-0.36l-0.3-0.36l-0.57-0.86l-0.41,0.09l-0.38-0.09l0.04-0.26l0.26-0.44l0.05-0.35l-0.13-0.18l-0.1-0.44l0.2-0.01l0.23-0.16
				l0.01-0.13l0.49,0.36l-0.02,0.44l-0.16,0.54l0.16,0.32l0.15-0.01l0.06,0.32l0.23,0.3l0.14,0.36l0.27,0.17l0.18-0.18l-0.12-0.63
				l-0.32-1.03l0.21-0.3l0.06-0.39l-0.24-0.42l0.23-0.48l0.47-0.6L408,74.5l-0.23-0.33l-0.13-0.48l-0.37-0.3l-0.36,0.11l-0.02-0.08
				l-0.26-0.16l-0.05,0.05l-0.04-0.45l-0.1,0l-0.03-0.1l0.06-0.19l0.57-0.41l0.16-0.4l0.06-0.49l-0.09-0.24l0.17-0.4l-0.19-0.24
				l-0.19,0.06l-0.11-0.05l0.06-0.08l-0.06-0.09l-0.21-0.16l-0.3,0.2l-0.13-0.09l-0.31,0.4l-0.4-0.19l-0.32-0.05L404.8,69.86z
				 M406.6,76.28l-0.07-0.04L406.6,76.28z"></path>
		</g>
		<g id="aranda_norte">
			<path id="path4855" class="st3 fase_0" d="M379.81,112.66l0.18,0.35l0.36,0.37l0.01,0.19l0.16,0.28l0.32,0.23l0.44-0.14l1.1,0.1
				l0.26-0.24l0.57-0.25l0.19-0.02l0.21,0.23l1.44,0.58l1.31-1.14l0.21,0.42l-0.03,0.58l-0.65,0.22l-0.21,0.57l-0.11,0.06
				l-0.21-0.25l0.1,0.56l0.1,0.37l0.38,0.06l0.27,0.46l0.63,0.36l0.17,0.01l-0.05,0.69l-1.17,0.52l-0.58-0.27l-0.58,0.05l-0.03,0.25
				l-0.7,0.04l-0.55-0.1l-0.02,0.2l-0.53,0.18l-0.13,0.12l-0.25-0.02l-0.2,0.18l-0.55,0.21l-0.45,0.04l-0.3,0.11l-0.35-0.13
				l-0.14-0.34l0.14-0.93l-0.1-0.24l-0.35-0.27l0.07-0.04l0.25,0l0.09-0.13l-0.04-0.62l0.15-0.65l-0.24-0.26l-0.47-0.23l-0.46,0.11
				l-0.36,0.22l-0.57,0.16l-0.12-0.46l-0.52-0.92l-0.45-0.62l0.39-0.09l0.13-0.1l0.28,0.04l0.42-0.17l-0.3-0.2l0.61-0.41l0.12,0
				l0.11,0.11l0.08,0.22l0.35-0.08L379.81,112.66L379.81,112.66z"></path>
		</g>
		<g id="burgos_rural_norte">
			<path class="st3 fase_0" d="M385.36,62.42L385.36,62.42L385.36,62.42L385.36,62.42z M401.58,86.29l-0.09-0.36l-0.53-0.28l-0.25,0.04
				l-0.26-0.19l-0.13-0.27l-0.69,0.19l-0.32-0.54l-0.14-0.1l-0.42,0.05l-0.32-0.14l-0.71,0.08l0.11-0.15l0.47-0.29l0.03-0.2
				l-0.18-0.09l-0.04-0.3l-0.5-0.2l-0.05-0.23l-0.16-0.14l0.09-0.32l-0.07-0.22l0.09-0.27l0.2-0.12l-0.47-0.44l-0.11-0.41
				l-0.17-0.24l-0.26-0.06l-0.78-0.21l-0.07-0.05l-0.01-0.46l0.25-0.12l0.2-0.35l-0.02-0.22l-0.27-0.42l-0.65,0l-0.09-0.55
				l-0.37-0.64l0.03-0.23l-0.12-0.46l0.17-0.34l-0.41-0.93l-0.05-0.43l0.1-0.72l-0.03-0.15l-0.71,0.33l0.1-0.62l-0.3,0.02
				l-0.18-0.21l-0.74,0.11l-0.43,0.23l-0.17-0.04l-0.05-0.17l0.14-0.31l-1.32-1.62l-0.14-0.37l-1.05,0.49l-0.33,0.34l-0.22-0.45
				l-0.23,0.14l-0.09-0.03l0,0l-0.01-0.49l-0.14-0.21l0.01-0.16l-0.2-0.52l-0.37-0.16l0.26-0.47l0.09-0.39l-0.04-0.42l-0.18-0.19
				l-0.01-0.13L388,69.45l-0.2-0.13l-0.29,0.1l-0.32-0.28l0.01-0.65l0.15-0.25l-0.09-0.52l0.13-0.08l0.03-0.22l-0.2-0.26l-0.19,0.08
				l0.04-0.27l-0.54-0.12l-1.37,0.32l-0.1-0.16l-0.01-0.34l-0.34-0.06l-0.27-0.66l0.2-0.04l-0.03,0.16l0.07-0.02l0.33-0.75
				l0.86-0.71l0.12-0.36l-0.03-0.42l-0.23-0.24l-0.57,0.14l0.16-0.56l0.06-0.73l-0.48,0.19l-0.28-0.18l-0.57,0.46l-0.16,0.53
				l-0.68,0.31l-0.08-0.13l-0.1,0.02l-0.26,0.22L382.3,64l-0.35-0.44l-0.22-0.05l-0.22-0.18l-0.1-0.32L381.16,63l-0.15,0.09
				l0.01,0.07l-0.41,0.35l0.04,0.21l-0.36,0.14l-0.13-0.04l0,0.13L380.08,64l0.29,1.23l-0.11,0.43l-0.3-0.21l-0.38,0.75l-0.16,0.05
				l-0.47-0.17l-0.18,0.08l-0.13-0.07l-0.11,0.04l0.02,0.18l-0.16,0.11l-0.11,0.07l-0.19,0.57l-0.08,0l0.08,0.15l-0.14,0.19
				l0.12,0.24l-0.08,0l-0.24,0.01l-0.13-0.18l-0.14,0.09l-0.31,0.03l-0.27,0.19l-0.47-0.17l-0.01,0.18l-0.15,0.59l0.14,0.25
				l-0.04,0.3l0.1,0.06l-0.14,0.06l-0.05,0.14l-0.36,0.33l0.04,0.56l-0.27,0.63l-0.49,0.44l-0.68,1.41l-0.17,0.21l-0.42,0.93
				l-0.31,0.35l0.03,0.2l-0.22,0.34l-0.21,0.09l-0.32-0.05l-0.4,0.45l-0.19,0.02l-0.16,0.22l-0.36,0.19l-0.02,0.15l0.11,0.22
				l-0.05,0.09l-0.19,0.03l-0.07,0.1l-0.08-0.03l-0.55,0.33l-0.17,0.33l0.14,0.37l0.33,0.28l-0.05,0.18l0.21,0.55l-0.09,0.34
				l0.11,0.13l-0.08,0.16l-0.26,0.17L370.73,79l-0.01,0.25l-0.2,0.53l0.14,0.7l0.03,0.08l0.09,0.22l0.53,0.16l0.3,0.22l0.14,0.81
				l0.59-0.42l0.15-0.21l0.44-0.13l0.71-0.07l0.41-0.44l0.42-0.06l0.06,0.22l-0.06-0.22l-0.42,0.06l-0.41,0.44l-0.71,0.07
				l-0.44,0.13l-0.15,0.21l-0.59,0.42l-0.19,0.15l0.23,0.22l-0.26,0.1l-0.29,0.28l-0.08,0.78l0.17,0.21l0.12,0.4l-0.13,0.21v0.15
				l-0.24,0.18l0.02,0.06l0.22,0.5l0.34,0.42l0.21,0.03l0.05-0.1l0.13-0.01l0.17-0.24l0.61-0.05l0.06,0.11l0.13,0.01l0.12,0.21
				l0.04,0.47l-0.07,0.41l-0.15,0.23l0.4,0.27l0.29-0.18l0.4,0.23l0.43,0.47l0.44-0.04l0.24,0.3l0.16,0.04l0.18-0.1l0.21,0.6
				l0.73-0.18l0.62,0.21l0.1-0.14l0.13-0.29l0.3-0.33l0.6-0.13l0.46,0.03l0.24,0.12l0.56-0.33l-0.13,0.32l0.07,0.12l0.64,0.31
				l-0.06,0l-0.05,0.21l0.46,0.3l0.05,0.45l0.2,0.19l0.02,0.34l-0.08,0.13l0.38,0.63l0.4,0.36l0.02,0.43l0.43,0.3l0.11,0.37
				l0.55,0.58l0.45,0.29l0.26,0.05l0.36-0.01l0.48-0.13l0.83-0.5l-0.02,0.16l0.23,0.32l0.35-0.19l0.25-0.29l0,0l-0.25,0.29
				l-0.35,0.19l0.05,0.3l0.24,0.19l-0.02,0.12l0.54-0.15l0.09,0.07l0.52-0.06l0.21-0.52l0.15-0.16l0.17,0l0.03,0.13l0.43,0.14
				l0.14-0.19l0.11,0.09l0.09-0.03l0.08-0.24l0.33,0.14l-0.35-0.21l0.05-0.44l0.21,0.09l0.52-0.3l0.46,0.03l-0.16-0.07l-0.07-0.18
				l0.2-0.13l-0.04-0.18l0.24-0.02l0.79,0.2l0.08-0.57l0.44-0.72l0.09-0.02l-0.42-0.27l-0.2-0.42l0.31-0.53l0.35-0.22l0.01-0.17
				l-0.37,0.26l-0.19-0.22l-0.13-0.03l0.1-0.02l0.04-0.16l0.03,0.08l0.18,0.04l0.1-0.07l-0.04-0.21l0.06-0.23l0.48,0.56l0.99,0.59
				l0.45,0.31l0.55,0.15l0.35,0.2l0.4,0.12l0.3-0.09l-0.05-0.23l0.1-0.18l0.39-0.46l0.39-0.28l0.49-0.78l0.53,0.51l0.98,0.21
				l0.62,0.5l0.08,0.2l1.26,0.44l0.42-0.15l0.05-0.35l0.2-0.18l0.05-0.29l0.4-0.35l0.19-0.1l0.56-0.03l0.43-0.68l0.5-0.16l0.27-0.61
				L401.58,86.29z M374.58,81.02L374.58,81.02l0.18,0.08l0,0L374.58,81.02z M376.1,81.42l-0.45,0.2l-0.34,0.27l-0.5-0.46l0,0
				l0.5,0.46l0.34-0.27L376.1,81.42l0.42-0.01l0,0L376.1,81.42z M378.49,81.67l-0.3-0.09l-0.16,0.2l-0.16-0.14l-0.27,0.08
				l-0.13-0.16l-0.11,0.04l-0.14-0.1l-0.16-0.2l-0.13,0.03l-0.01,0.07l-0.15,0.08l-0.13,0.04l-0.11,0.01l0,0l0.09-0.62l0.14-0.02
				l-0.04,0.15l0.22,0.17l-0.08,0.18l0.06,0.02l0.01-0.07l0.13-0.03l0.16,0.2l0.14,0.1l0.11-0.04l0.13,0.16l0.27-0.08l0.16,0.14
				l0.16-0.2L378.49,81.67l0.22,0.01l0,0L378.49,81.67z M378.82,81.98l0.09,0.25l0.18,0.23l0,0l-0.18-0.23L378.82,81.98z
				 M384.14,82.47l0-0.05l0,0L384.14,82.47L384.14,82.47z M386.64,77.17l-0.15,0.1l0.05,0.6l-0.55,0.43l-0.09,0.2l0.61,0.48
				l-0.5,0.13l-0.03,0.38l-0.3,0.27l0.03,0.13l0.09-0.03l0.16,0.16l0.32,0.09l-0.07,0.13l-0.16,0.19l-0.19-0.15l-0.23,0.4
				l-0.47-0.12l-0.09,0.17l-0.26,0.15l-0.45,0l0.05,0.63l0.27,0.2l-0.56,0.58l-0.16,0.07l0,0l-0.28-0.26l-0.23,0.36l-0.4-0.1
				l-0.18,0.04l-0.23,0.2l-0.08,0.26l-0.26,0.34l0.07,0.1l0,0.33l-0.52,0.02l-0.07-0.02l0.03-1.16l-0.55-0.24l-0.51-0.07l-0.16-0.26
				l-0.5,0.01l0.05,0.22l-0.14,0.32l-0.79,0.56l-0.12-0.15l-0.17-0.21l0,0l0.17,0.21l0.12,0.15l0.79-0.56l0.14-0.32l-0.05-0.22
				l-0.18-0.47l-0.5-0.5l0.05-0.39l0.16-0.24l-0.06-0.25l0.1-0.07l-0.02-0.07l-0.31-0.08l-0.18,0.31l-0.71-0.11l-0.29-0.15
				l-0.08-0.16l0.52-0.29l-0.29-0.16l0.51-0.05l0.31-0.14l0.14-0.16l0.07,0.23l0.28-0.19l0.09,0.07l0.16-0.12l0.04,0.05l0.72-0.46
				l0.08,0.02l0.06,0.29l0.36-0.01l-0.06-0.19l0.43,0.1l0.11-0.23l0.2,0l0.15-0.55l0.57-0.35l0.25,0.07l0.4-0.25l0.22-0.01l0.1,0.36
				l0.17,0.25l0.01,0.16l0.74,0.06l0.04,0.06l0.18,0l0.17-0.11l0.4,0.03l0.17-0.31l0.63-0.57l0.47-0.07L386.64,77.17l0.14-0.23l0,0
				L386.64,77.17z"></path>
		</g>
		<g id="burgos_rural_sur" class="active">
			<path class="st3 fase_0" d="M395.48,87.69c0.09-0.28,0.21-0.55,0.36-0.81c-0.28-0.12-0.53-0.33-0.7-0.59c-0.46-0.1-0.92-0.18-1.38-0.24
				c-0.4-0.05-0.76-0.22-1-0.49c-0.15-0.04-0.3-0.07-0.45-0.11c-1.12-0.31-1.19-1.74-0.31-2.35c0.06-0.04,0.12-0.07,0.18-0.11
				c-0.03-0.13-0.06-0.26-0.06-0.39c-0.6-0.21-1.22-0.35-1.8-0.59c-0.39-0.16-0.71-0.42-0.86-0.79c-0.35-0.05-0.69-0.11-1.03-0.21
				c-0.43-0.11-0.72-0.44-0.85-0.83c-0.54-0.31-1.07-0.66-1.57-1.05l0,0l-0.03,0.38l-0.3,0.27l0.03,0.13l0.09-0.03l0.16,0.16
				l0.32,0.09l-0.07,0.13l-0.16,0.19l-0.19-0.15l-0.23,0.4l-0.47-0.12l-0.09,0.17l-0.26,0.15l-0.45,0l0.05,0.63l0.27,0.2l-0.56,0.58
				l-0.16,0.07l0,0l-0.28-0.26l-0.23,0.36l-0.4-0.1l-0.18,0.04l-0.23,0.2l-0.08,0.26l-0.26,0.34l0.07,0.1l0,0.33l-0.52,0.02
				l-0.07-0.02l0.03-1.16l-0.55-0.24l-0.51-0.07l-0.16-0.26l-0.5,0.01l0.05,0.22l-0.14,0.32l-0.79,0.56l-0.12-0.15l-0.17-0.21
				l0.06-0.09l0.1-0.13l-0.18-0.23l-0.2-0.55l-0.22-0.01l-0.3-0.09l-0.16,0.2l-0.16-0.14l-0.27,0.08l-0.13-0.16l-0.11,0.04
				l-0.14-0.1l-0.16-0.2l-0.13,0.03l-0.01,0.07l-0.15,0.08l-0.13,0.04l-0.11,0.01l-0.02-0.11l-0.42,0.01l-0.45,0.2l-0.34,0.27
				l-0.5-0.46l-0.05-0.32l-0.18-0.08l-0.11-0.38l-0.42,0.06l-0.41,0.44l-0.71,0.07l-0.44,0.13l-0.15,0.21l-0.59,0.42l-0.19,0.15
				l0.23,0.22l-0.26,0.1l-0.29,0.28l-0.08,0.78l0.17,0.21l0.12,0.4l-0.13,0.21v0.15l-0.24,0.18l0.02,0.06l0.22,0.5l0.34,0.42
				l0.21,0.03l0.05-0.1l0.13-0.01l0.17-0.24l0.61-0.05l0.06,0.11l0.13,0.01l0.12,0.21l0.04,0.47l-0.07,0.41l-0.15,0.23l0.4,0.27
				l0.29-0.18l0.4,0.23l0.43,0.47l0.44-0.04l0.24,0.3l0.16,0.04l0.18-0.1l0.21,0.6l0.73-0.18l0.62,0.21l0.1-0.14l0.13-0.29l0.3-0.33
				l0.6-0.13l0.46,0.03l0.24,0.12l0.56-0.33l-0.13,0.32l0.07,0.12l0.64,0.31l-0.06,0l-0.05,0.21l0.46,0.3l0.05,0.45l0.2,0.19
				l0.02,0.34l-0.08,0.13l0.38,0.63l0.4,0.36l0.02,0.43l0.43,0.3l0.11,0.37l0.55,0.58l0.45,0.29l0.26,0.05l0.36-0.01l0.48-0.13
				l0.83-0.5l-0.02,0.16l0.23,0.32l0.35-0.19l0.25-0.29l0,0l-0.25,0.29l-0.35,0.19l0.05,0.3l0.24,0.19l-0.02,0.12l0.54-0.15
				l0.09,0.07l0.52-0.06l0.21-0.52l0.15-0.16l0.17,0l0.03,0.13l0.43,0.14l0.14-0.19l0.11,0.09l0.09-0.03l0.08-0.24l0.33,0.14
				l-0.35-0.21l0.05-0.44l0.21,0.09l0.52-0.3l0.46,0.03l-0.16-0.07l-0.07-0.18l0.2-0.13l-0.04-0.18l0.24-0.02l0.79,0.2l0.08-0.57
				l0.44-0.72l0.09-0.02l-0.42-0.27l-0.2-0.42l0.31-0.53l0.35-0.22l0.01-0.17l-0.37,0.26l-0.19-0.22l-0.13-0.03l0.1-0.02l0.04-0.16
				l0.03,0.08l0.18,0.04l0.1-0.07l-0.04-0.21l0.06-0.23l0.48,0.56l0.99,0.59l0.45,0.31l0.55,0.15l0.35,0.2l0.4,0.12l0.3-0.09
				l-0.05-0.23l0.1-0.18l0.39-0.46l0.39-0.28l0.49-0.78L395.48,87.69z"></path>
		</g>
		<g id="valle_de_mena">
			<path class="st3 fase_1" d="M404.27,41.55l-0.43-0.28L404.1,41l0.16-0.36l0.55-0.62l-0.31-0.72l-0.26,0.13l-0.17,0.22l-0.42-0.26
				l-0.29-0.04L402.94,39l-0.05-0.25l0.82-1.2l-0.21-0.2l0.2-0.39l0.54-0.37l0.29-0.02l-0.06-0.36l-0.22-0.31l-0.37-0.21l-0.21,0.04
				l-0.22-0.08L403,35.83l-0.27,0.32h-0.08l-0.46-0.09l-0.24-0.19l-0.12-0.27l-0.13,0.08l-0.07,0.2l-0.06,0.04l-0.16-0.1l-0.25,0.28
				l0.02-0.11l-0.17-0.22l-0.09,0l-0.04-0.13l-0.09-0.01l-0.17-0.36l-0.64-0.69h0l-0.12,0.18l-0.25-0.05l-0.22,0.11l-0.41-0.01
				l0.02,0.24l-0.8,0.4l-0.12,0.19l-0.21,0.11l-0.14,0.24l-0.42,0.07l-0.03,0.08l-0.55,0.22l-0.2-0.1l-0.14,0.04l-0.35,0.67
				l-0.38,0.12l-0.45,0.03l-0.04,0.14l-0.17,0.12c-0.06,0.45-0.14,0.9-0.23,1.34c-0.22,1.03-0.51,2.06-0.93,3.03c0,0,0.01,0,0.01,0
				c0.11,0.03,0.23,0.06,0.34,0.1c0.01,0,0.01,0,0.02,0.01c0.02,0.01,0.05,0.01,0.07,0.02c0.02,0,0.02,0.01,0.02,0.01
				c0.01,0,0.01,0,0.03,0.01c0.04,0.01,0.08,0.02,0.11,0.03c0.01,0,0.09,0.02,0.1,0.02c0.17,0.04,0.34,0.08,0.53,0.11
				c0.38,0.07,0.76,0.13,1.14,0.17c0.08,0.01,0.15,0.03,0.23,0.05l0.08-0.01l0.42,0.11l0.09-0.05l0.33,0.23l0.23,0l1.5-0.63
				l0.2-0.17l0.29-0.1l0.45-0.39l0.43-0.01l0.3-0.15l0.12-0.08l0.02-0.26l0.1-0.17l0.18-0.16L401.4,41l0.01,0.26l1.05,0.85
				l0.17,0.41l0,0.23l-0.27,0.02l-0.15,0.1l-0.01,0.16l0.27,0.39l-0.06,0.09l0.15,0.13l-0.07,0l0.03,0.15l0.14,0.19l0.24,0.12
				l0.15-0.03l0.28-0.19l0.03-0.19l0.7,0.11l0.38-0.02l0.06-0.13l-0.23-0.27l0.12-0.17l-0.03-0.6l0.13,0.02l-0.08-0.25l0.12-0.31
				l0.01-0.42l-0.03-0.08L404.27,41.55z"></path>
		</g>
		<g id="espinosa_monteros">
			<path class="st3 fase_1" d="M396.34,43.92L396,43.08l-0.19-0.05l0.43-0.64c-0.78-0.12-1.56-0.28-2.31-0.53
				c-0.04-0.01-0.08-0.03-0.12-0.05c-0.04,0.07-0.05,0.05-0.04-0.02c-0.03-0.01-0.07-0.02-0.09-0.04c-0.03-0.02-0.07-0.03-0.03-0.05
				c0.02-0.01,0.09,0,0.14,0.02c0.04-0.21,0.11-0.41,0.2-0.61c0.3-0.68,0.47-1.42,0.6-2.15c0.09-0.48,0.15-0.96,0.2-1.44l-0.28,0.19
				l-0.74,0.14l-0.13,0.15l-0.3,0.06l-0.17,0.06l-0.31-0.07l-0.16,0.1l-0.33-0.05l-0.13,0.07l-0.57-0.25h-0.87l-0.27,0.15
				l-0.61-0.18l-0.26,0l-0.9-0.34l-0.52-0.28l-0.71-0.07l-0.47-0.18l-0.52-0.09l-0.57,0.14l-0.16,0.12l-0.19-0.76l-0.17-0.31
				l-0.45-0.24l-0.3,0.06l-0.1-0.14l-0.3-0.03l-0.16-0.18l-0.12,0.01l-0.12-0.1l-0.29,0.18l0.08,0.37l-0.2,0.11l-0.05,0.19
				l-0.18,0.13l-0.05,0.28l-0.35,0.14l0,0.21l-0.31,0.47l-0.06,0.28l0.07,0.05l-0.45,0.52l-0.36,0.17l0.04,0.29l-0.07,0.1
				l-0.39,0.12l-0.2,0.3l0.43,0.18l0.56,0.76l1.05-0.34l0.39,0.07l0.38,0.2l0.59-0.04l0.28,0.06l0.36-0.09l0.5,0.35l0.18,0.27
				l-0.04,0.42l0.62,0.6l0.37,0.89l0.3,0.14l0.29-0.06l0.23,0.06l0.32-0.11l0.43,0.37l-0.22,0.75l0.35,0.68l0.01,0.2l-0.18,0.08
				l0.78,0.43l-0.05,0.16l0.26,0.19l0.07-0.14l0.3-0.12l0.02,0.14l-0.18,0.08l0.13,0.14l0.66-0.12l0.07,0.16l0.54-0.17l0.54,0.02
				l0.04-0.09l0.16-0.02l0.13-0.11l-0.09-0.11l-0.04-0.29l0.4-0.18l0.17-0.41l0.16,0.05l-0.22,0.41l0.22,0.41l0.08,0.07l0.26-0.08
				l0.18,0.33l0,0.15l0.08-0.03l0.32-0.41l0.1-0.3l-0.18-0.43l-0.29-0.28l0.25-0.44l0.01-0.21l0.11-0.03l0.27-0.01l-0.01,0.34
				l0.21,0.27l-0.22,0.53l0.25-0.17l0.07,0.25l0.2,0.04l0.39,0.01l0.1-0.22l0.24,0.06l-0.04,0.46l0.69,0.12l0.2-0.02l0.12-0.17
				l0.14,0.08l0.35-0.27l0.18,0.01l0.13-0.25l-0.1-0.32L396.34,43.92z"></path>
		</g>
		<g id="sedano">
			<path id="path4049" class="st3 fase_1" d="M378.53,51.65l-0.22,0.11l0.04,0.21l-0.91,0.23l-0.32-0.09l0.04,0.08l-0.32,0.14L376.47,53
				l0.01,0.19l0.7,0.87l-0.01,0.31l-0.12,0.13l0.14,0.46l0.01,0.47l-0.18,0.23l-0.04,0.32l-0.07,0.08l-0.23,0.01l-1.02,0.6
				l-0.37,0.06l-0.92-0.63l-0.34-0.34l-0.21,0.74l-0.08,0.12l-0.26,0.03l-0.08,0.08l0.03,0.51l-0.09,0.46l-0.63-0.09l-0.17-0.23
				l-0.74,0.38l0.09,0.3l-0.11,0.12l-0.12,0.38l0.06,0.31l0.25,0.38l-0.03,0.19l0.2,0.11l0.28,0.38l0.1,0.44l-0.05,0.54l-0.18,0.07
				l0.01,0.15l0.69,0.45l0.2,0.24l0,0.13l0.08-0.15l0.15-0.04l-0.04,0.38l0.07-0.06l0.18,0.2l-0.02,0.3l-0.15,0.03l0.02,0.35
				l0.29,0.4l0.22,0.15l0.04-0.1l0.12,0.04l0.54-0.44l0.35,0.08l0.12,0.24l-0.01,0.19l0.33,0.27l-0.17,0.13l0.03,0.5l0.4,0.17
				l0.84,0.35l-0.04,0.18l0.15,0.14l0.17,0.06l0.2,0.2l0.38,0.05l0.09,0.2l0.67,0.35l0.18,0.18l-0.05,0.16l0.16-0.11l-0.02-0.18
				l0.11-0.04l0.13,0.07l0.18-0.08l0.47,0.17l0.16-0.05l0.38-0.75l0.3,0.21l0.11-0.43L380.08,64l0.08-0.05l0-0.13l0.13,0.04
				l0.36-0.14l-0.04-0.21l0.41-0.35l-0.01-0.07l0.15-0.09l0.25,0.02l0.1,0.32l0.22,0.18l0.22,0.05l0.35,0.44l0.46-0.16l0.26-0.22
				l0.1-0.02l0.08,0.13l0.68-0.31l0.16-0.53l0.57-0.46l0.28,0.18l0.48-0.19l-0.01-0.14l0.29-0.03l0.74-1.14l0.41-0.43l0.15-0.39
				l0.53-0.23l0.19,0.03l0.02-0.15l0.35-0.54l-0.12-0.17l-0.1-0.53l-0.59-0.29l-0.12-0.3l-0.01-0.3l-0.15-0.11l-0.3-0.48l0.25-0.39
				l-0.22-0.16l-0.43,0.01l-0.68,0.27l-0.47-0.36l0.23-0.75l-0.04-0.25l-0.62-0.92l-0.64-0.5l-0.12-0.2l0.15-0.83l-0.07-0.49
				l0.18-0.18l0.01-0.61l-0.16,0.06l0,0.27l-0.23,0.18L383,52.06l0.09-0.15l-0.16-0.15l-0.7-0.03l-0.21,0.06l-0.41,0.04l-0.15,0.28
				l-0.21,0.05l0.1,0.33l-0.11,0.05l0.08,0.25l-1.05-1.05l-0.63,0.35l-0.23-0.13l-0.62-0.15L378.53,51.65L378.53,51.65z
				 M370.13,57.56l-0.16,0.13l-0.21-0.02l-0.12,0.03l-0.02,0.27l0.03,0.31l0.31,0.07l0.3-0.02l0.33-0.08l-0.11-0.55L370.13,57.56z"></path>
		</g>
		<g id="lerma">
			<path id="path4209" class="st3 fase_0" d="M379.47,87.26l-0.56,0.33l-0.24-0.12l-0.46-0.03l-0.6,0.13l-0.3,0.33l-0.13,0.29l0.13,0.16
				l-0.23,0.99l0.05,0.23l-0.15,0.22l-0.62,0.49l-0.24,0.06l-0.16-0.13l-0.18-0.48l-0.26,0.31L375.09,90l-0.11-0.16l-0.37-0.02
				l-0.06,0.28l0.18,0.25l0.13,0.8l0.13,0.07l-0.01,0.22l0.4-0.2l0.59,0.19l-0.98,0.59l-0.63,0.13L374,92.56l-0.02,0.46l-0.27-0.13
				l-0.32,0.14l-1.76,1.05l-0.35,0.2l-0.13,0.77l0.05,0.18l-0.16,0.22l-0.68,0.53l-0.14-0.21l-0.51-0.15l-0.21-0.22l0.04-0.09
				l-0.11-0.29l-0.39,0.11l-0.31-0.24l-0.06,0.09l-0.33,0.03l-0.29,0.17l-0.15,0.29l0.03,0.21l-0.11,0.58l0.07,0.02l0.27-0.14
				l0.31,0.21l0.84-0.11l0.46,0.14l0.13,0.21l0.31-0.09l0.25-0.25l0.09,0.44l0.16,0.11l0.21,0.42l-0.36,0.65l-0.13-0.15l-0.08,0.01
				l-0.38,0.97l-0.81,0.37l-0.37,0.9l-0.21,0.28l0.07,0.18l-0.14,0.23l-0.08,0.8l0.35,0l0.19-0.23l1.17-0.07l0.32-0.3l0.18,0.03
				l0.21-0.1l0.03-0.13l0.66-0.65l0.22,0.08l1.11-0.59l0.35-0.08l0.19-0.27l0.76,0.02l0.06,0.54l-0.16,0.79l0.12,0.61l-0.06,0.16
				l-0.8,0.08l-0.11-0.13l-0.33-0.11l-0.56,0.19l-0.38,0.57l0.06,0.1l-0.11,0.27l-0.2,0.22l-1.18,0.25l-0.1,0.2l0.03,0.73l0.1,0.68
				l0.04,0.39l0.62-0.16l0.19,0.04l0.42,0.3l0.24,0.04l1.05-0.04l0.32-0.1l0.44,0.09l0.39-0.03l0.07,0.11l0.34,0.11l0.29-0.03
				l0.37-0.16l0.26,1.01l0.48-0.07l0.4,0.29l0.54,0.23l-0.11,0.47l0.03,0.25l-0.09,0.07l0.07,0.51l0.09,0.4l0.27,0.3l0.08,0.47
				l0.13,0.25l-0.02,0.21l0.34,0.26l0.22,0.06l1.08,0.01l0.02,0.29l0.06,0.06l0.37,0.02l0.42-0.06l0.48-0.25l0.12,0.07l0.2-0.18
				l0.5-0.56l0.03,0.11l1.12-0.51l0.54,0.14l0.19-0.11l0.14-0.31l0.14-0.49l0.27,0.19l0.12,0l0.23-0.16l0.19,0.01l0.64-0.19
				l0.48,0.04l0.64,0.22l0.17-0.79l-0.08-0.13l0.16-0.46l-0.02-0.14l0.16-0.08l-0.02-0.26l0.31-0.42l-0.09-0.48l0.32-0.09l0.63-0.36
				l0.88,0.2l1.06,0.61l0.3-0.44l-0.03-0.43l0.34-0.07l0.1-0.53l0.29-0.21l0.29-0.66l-0.4-0.11l0.08-0.27l0.19-0.06l-0.55-0.12
				l-0.03-0.38l0.26-0.37l0.71-0.52l0.04-0.48l0.24-0.57l-0.04-0.22l0.22-0.66l0.15-0.1l-0.26-0.09l0.12-0.7l-0.17,0.03l-0.18-0.72
				l-0.4-0.64l-0.16,0l-0.22-0.37l-0.33,0.1l0.18-0.35l0.18-0.18l-0.04-1.22l-0.27-0.35l-0.15-0.01l0.19-0.37l-0.04-0.4l-0.27-0.2
				l-0.15,0.04l-0.15-0.07l-0.24-0.27l0.19-0.39l0.5-0.58l-0.94-0.33l-0.22,0.14l-0.17-0.13l-0.42,0.1l-0.33-0.14l-0.08,0.24
				l-0.09,0.03l-0.11-0.09l-0.14,0.19l-0.43-0.14l-0.03-0.13l-0.17,0l-0.15,0.17l-0.21,0.52l-0.52,0.06l-0.09-0.07l-0.54,0.15
				l0.02-0.12l-0.24-0.19l-0.05-0.3l-0.23-0.32l0.02-0.16l-0.83,0.5l-0.48,0.13l-0.36,0.01l-0.26-0.05l-0.45-0.29v0l-0.55-0.58
				l-0.11-0.37l-0.43-0.3l-0.02-0.43l-0.4-0.36l-0.38-0.63l0.08-0.13l-0.02-0.34l-0.2-0.19l-0.05-0.45l-0.46-0.29l0.05-0.21l0.06,0
				l-0.64-0.31l-0.07-0.12L379.47,87.26z"></path>
		</g>
		<g id="roa_de_duero">
			<path id="path4783" class="st3 fase_0" d="M370.54,104.35l-0.71,0.44l-0.36,0.39l-0.5,0.36l-0.63,0.25l-0.1,0.05l-0.4-0.05
				l-0.44-0.03l-0.15-0.15l-0.17,0.03l-0.26,0.13l-0.28,0l-0.37,0.19l0.18,0.36l0.14,0.1l0.07,0.73l-0.35,0.2l-0.11,0.32l0.03,0.42
				l1.27,1.4l0.4,0.11l0.24,0.63l0.2,0.07l-0.04,0.92l-0.08,0.23l-0.35,0.01l-0.25,1.48l-0.4,0.07l-0.14-0.09l-0.28,0.31l0.05,0.36
				l-0.07,0.14l0.33-0.02l0.37,0.29l0.24,0.09l0.07,0.15l0.37,0.31l0.12,0.24l0.53,0.77l0.07,0.91l-0.06,0.86l-0.04,0.1l-0.01,0.03
				l-0.07,0.68l-0.06,0.75l-0.18,0.26l-0.02,0.27l0.13,0.19l0.14,0.15l0.25-0.09l0.24-0.23l0.2,0.18l0.23-0.13l0.11,0.18l-0.21,0.22
				l-0.01,0.1l0.64,0.07l0.02,0.44l0.1,0.11l0.2,0.01l0,0.42l0.06,0.25l0.12,0.14l0.31-0.01l0.37-0.13l0.13,0.44l0.3,0.24l0.44,0.23
				l0.55,0.02l0.57,0.28l0.36,0.35l0.45,0.26l0.32,0.84l0.04,0.22l0.12-0.09l0.31-0.04l0.29,0.12l0.17,0.19l0.44,0.88l0.42,0.64
				l0.33,0.05l-0.32-1.28l1.16-0.24l0.38,0.14l0.03,0.09l-0.02-0.16l0.15-0.06l0.04-0.09l-0.07-0.27l-0.37-0.82l-0.04,0.06
				l-0.19-0.15l-0.09,0.03l-0.13-0.3l-0.17,0.28l-0.08-0.24l-0.1-0.04l0.02-0.25l0.15-0.12l-0.32-0.46l0.04-0.11l0.27-0.12
				l-0.12-0.08l0.03-0.14l-0.17,0.04l-0.39-0.29l-0.14-0.06l-0.03,0.07l-0.11-0.08l0.08-0.16l0.22-0.06l-0.02-0.22l0.32-0.34
				l0.04-0.18l0.1-0.08l0.1-0.1l0.2,0.04l-0.02-0.12l0.38-0.33l0.21-0.03l0.02,0.17l0.44-0.03l0.44,0.32l0.33-0.11l0.6-0.39
				l0.03-0.17l-0.53-0.41l-0.15,0.17l-0.06-0.06l0.02-0.21l-0.19,0.15l-0.04-0.17l-0.14,0.05l-0.03-0.11l-0.74-0.82l-0.04-0.15
				l0.12-0.39l-0.06-0.18l0.12-0.45l1.13-0.21l-0.35-0.48l-0.52-0.06l-0.47,0.55l-0.26-0.2l-0.26-0.34l-0.35-0.2l-0.53-0.09
				l-0.47-0.46l-0.08-0.26l0.3-0.21l0.04-0.55l-0.15-0.4l0.06-0.03l-0.21-0.14l-0.13-0.3l0.6-0.29l0.37,0.11l0.12-0.36l0.23-0.15
				l-0.03-0.5l0.21-0.21l0.11,0.01L377,112l0.1,0l0.03-0.22l0.49-0.33l0.16-0.24l0.23-0.16l0.1-0.5l0.47-0.23l0.05-0.3l0.49-0.12
				l-0.02-0.29l-1.08-0.01l-0.22-0.06l-0.34-0.26l0.02-0.21l-0.13-0.25l-0.08-0.47l-0.27-0.3l-0.09-0.4l-0.07-0.51l0.09-0.07
				l-0.03-0.25l0.11-0.47l-0.54-0.23l-0.4-0.29l-0.48,0.07l-0.26-1.01l-0.37,0.16l-0.29,0.03l-0.34-0.11l-0.07-0.11l-0.39,0.03
				l-0.44-0.09l-0.32,0.1l-1.05,0.04l-0.24-0.04l-0.42-0.3l-0.19-0.04l-0.62,0.16L370.54,104.35z M376.47,125.9l0.42,0.99l0.28-0.13
				l0.68-0.13l0.17,0.07l0.34-0.07l-0.26-0.3l-0.37-0.27l-0.26-0.1L376.47,125.9z"></path>
		</g>
		<g id="quintanar_sierra">
			<path id="path4679" class="st3 fase_1" d="M412.86,94.2l-0.21,0.01l-0.59,0.19l-0.14-0.01l-0.08,0.08l-0.77,0.01l-0.56-0.18
				l-0.43,0.23l-0.44-0.27l-0.36-0.05l-0.44,0.18l-0.37,0.82l-0.17,0.63l-0.74,0.47l0.01,0.28l0.14,0.23l0.31,0.09l0.02,0.17
				l-0.18,0.64l0.03,0.02l-0.16,0.05l-0.36-0.07l-1.06,0.3l-0.44,0.59l-0.43,0.38l-0.01,0.16l-0.18,0.3l-0.27,0.06l-0.44-0.08
				l-0.29,0.11l-0.06,0.14L404,99.76l-0.06,0.19l-0.18,0.12l-0.28-0.11l-0.02-0.07l-0.36,0.86l0.4,1.08l0.12,0.8l-0.03,0.73
				l0.76,0.38l-0.04,0.07l0.83,0.49l0.94,0.69l0.15-0.2l0.18-0.49l-0.2-0.62l0.38,0.22l0,0.37l0.24,0.49l-0.05,0.25l0.39,0.29
				l0.48-0.01l0.2-0.43l0.54,0.1l0.07-0.06l0.18,0.09l0.2-0.01l0.7,0.12l0.07-0.5l-0.06-0.12l0.36-0.56l-0.14-0.29l0.13-0.43
				l0.39-0.56l0.44,0.06l0.21,0.18l0.47-0.06l0.25-0.41l0.19-0.78l-0.31-0.36l0.07-0.43l0.57-1.19l0.05-0.31l0.53-0.76l0.21-0.54
				l0.75-0.26l-0.07-0.87l-0.21-0.55l-0.1-0.64l-0.16-0.29l-0.04-0.41l-0.18-0.37l0.05-0.32L412.86,94.2L412.86,94.2z"></path>
		</g>
		<g id="condado_de_trevino">
			<path id="path4087" class="st3 fase_1" d="M416.9,56.2l-0.26,1.08l-0.14-0.1l-0.34,0.53l0,0.09l0.22,0.18l-0.16,0.06l-0.04,0.19
				l0.08,0.31l-0.19,0.01l0.07,0.3l-0.31,0.08l-0.26,0.3l0.14,0.15l0.43,0.04l0.3,0.3l-0.34,0.11l0.15,0.2l0.3,0.03l0.24,0.16
				l0.38,0.13l0.15,0.54l0.58,0.12l0.17,0.24l0.45,0.24l0.44-0.05l0.25,0.71l0.32,0.49l-0.01,0.21l-0.2,0.13l0.21,0.08l0.11,0.14
				l0.77,0l0.45,0.15l0.11,0.05l0.35-0.08l0.34,0.09l0.14-0.1l-0.01-0.31l0.38-0.23l0.25-0.05l0.78,0.17l0.08,0.13l0.29-0.03
				l0.17,0.19l0.29-0.34l0.21,0.13l0.09,0.48l0.57,0.61l0.11,0.1l0.12-0.08l0.53,0.03l0.22,0.1l0.38-0.15l0.11,0.09l0.05-0.15
				l0.61-0.05l0.81,0.38l0.19-0.15l-0.02-0.05l0.16-0.07l0.2,0.02l0.17,0.22l-0.15,0.23l0.1,0.17l0.44-0.06l0.58-0.34l0.26-0.07
				v-0.43l-0.24-0.58l0.05-1.03l-0.55-0.19l-0.16,0.05l-0.36,0.96l-0.78-0.09l-0.23,0.13l-0.18-0.01l-0.07-0.04l0.05-0.27
				l-0.65-0.86l-0.39-0.3l-0.21-0.02l0.04-0.6l0.2-0.08l0.11-0.35l0.32,0.25l0.1-0.01l0.7-0.77l0.1-0.4l0.13-0.26l0.1,0.02
				l0.08-0.23l-0.06-0.31l0.21-0.33l0.05-0.38L428.12,58l-0.26-0.01l-0.09,0.21l-0.32,0.23l-0.58,0.24l-0.1-0.07l-0.03-0.41
				l-0.41-0.36l0-0.25l-0.27-0.01l-0.21,0.13l-0.14-0.13l-0.12,0.08l-0.14-0.06l-0.35,0.27l-0.1,0.04l-0.2-0.08l-0.15,0.09
				l-0.08-0.13l-0.24-0.02l-0.07-0.25l-0.56-0.13l-0.03,0.11l-0.45-0.39l-0.59,0.04l-0.05-0.15l-0.29-0.13l-0.35,0.04l-0.32-0.27
				l-0.38-0.14l-0.14-0.18l-0.49,0.12l-0.59-0.09l-0.37,0.08l-0.23-0.09l-0.25,0.07l-0.59-0.02l-0.67,0.09l-0.31,0.08l-0.36,0.3
				l-0.15-0.41L416.9,56.2L416.9,56.2z"></path>
		</g>
		<g id="briviesca">
			<path id="path4085" class="st3 fase_0" d="M390.4,56.33l-0.15,0.32l0.11,0.04l-0.19,0.57l-0.66-0.15l-1.58,0.01l-0.46-0.16
				l-0.14-0.27l-0.43,0.17l-0.25,0.39l0.3,0.48l0.15,0.11l0.01,0.3l0.12,0.3l0.59,0.29l0.1,0.53l0.12,0.17l-0.35,0.54l-0.02,0.15
				l-0.19-0.03l-0.53,0.23l-0.15,0.39l-0.41,0.43l-0.74,1.14l-0.29,0.03l0.01,0.14l-0.06,0.73l-0.16,0.56l0.57-0.14l0.23,0.24
				l0.03,0.42l-0.12,0.36L385,65.31l-0.33,0.75l-0.07,0.02l0.03-0.16l-0.2,0.04l0.27,0.66l0.34,0.06l0.01,0.34l0.1,0.16l1.37-0.32
				l0.54,0.12l-0.04,0.27l0.19-0.08l0.2,0.26l-0.03,0.22l-0.13,0.08l0.09,0.52l-0.15,0.25l-0.01,0.65l0.32,0.28l0.29-0.1l0.2,0.13
				l0.32,0.04l0.01,0.13l0.18,0.19l0.04,0.42l-0.09,0.39l-0.26,0.47l0.37,0.16l0.2,0.52l-0.01,0.16l0.14,0.21l0.01,0.49l0.09,0.03
				l0.23-0.14l0.22,0.45l0.33-0.34l1.05-0.49l0.14,0.37l1.32,1.62l-0.14,0.31l0.05,0.17l0.17,0.04l0.43-0.23l0.74-0.11l0.18,0.21
				l0.3-0.02l-0.1,0.62l0.71-0.33l0.03,0.15l0.1,0.11l0.19,0l0.01,0.09l0.24,0.16l0.43-0.51l0-0.44l0.2-0.32l0.33-0.2l0.06-0.28
				l0.15-0.18l0.46-0.2l0.25,0.01l0.09-0.1l0.26,0.21l-0.03,0.16l0.48,0.59l0.98,0.44L399,74.1l-0.1-0.18l0.11-0.3l0.12-0.17
				l0.41-0.1l0.09-0.17l0.07,0.11l0.31-0.05l0.19-0.13l-0.31-0.63l0.44-0.77l0.55-0.27l0.41,0.08l0.1-0.27l0.24-0.09l0.19,0.05
				l0.73-0.47l0.28-0.02l0.26-0.21l0.35-0.15l0.2,0.03l0.2-0.19l0.67-0.18l0.3-0.17l-0.02-0.29l-0.28-0.61l-0.48-0.65l0-0.14
				l-0.14-0.08l-0.06-0.26l0.02-0.88l-0.06-0.08l0.21-0.24l-0.24-0.13l-0.06-0.08l-0.12-0.24l0.32-0.56l-0.28-0.03l-0.16-0.28
				l-0.12-0.3l0.05-0.34l-0.2-0.64l0.2-0.4l0.25-0.19l-0.28-0.3l0.35-0.41l0.24-0.06l-0.14-0.3l-0.01-0.42l-0.17-0.51l-0.35,0.16
				l-0.12,0.25l-0.18-0.01l-0.06-0.03l-0.05-0.45l-0.21-0.29l-0.68,0.06l-0.01-0.14l-0.34-0.02l0.13-0.65l-0.16-0.01l-0.65,0.24
				l-0.04-0.08l-0.09,0.07l-0.78,0.04l-0.14,0.15l-0.26-0.37l-0.3-0.11l-0.21,0.24l0.02,0.29l-0.1,0.08l-0.08,0.02l-0.34-0.28
				l-0.42-0.17L398,60.05l-0.32-0.18l-0.66-0.14l0.08-0.15l-0.03-0.25l-0.02-0.34l-0.2-0.6l-0.46-0.38l-0.32-0.02l-0.26,0.25
				l-0.39-0.02l-0.66-0.06l-0.06-0.34l-0.29,0.02l-0.58-0.13l-0.24-0.28l0.19-0.36l-0.49-0.3l-0.34,0.12l-0.13,0.33l-0.41,0.11
				l-0.08,0.19l-0.55-0.37l-0.04-0.11l-0.43-0.14L390.4,56.33L390.4,56.33z"></path>
		</g>
		<g id="miranda_este">
			<path class="st3 fase_0" d="M415.95,64.66l-0.37,0l-0.15-0.08l-0.11-0.15l-0.59-0.33l-0.28-0.31l-0.19-0.69l0.14-0.09l-0.11-0.19
				l0.12-0.24l0.07-0.1l-0.12-0.16l-0.14-0.01l-0.1-0.21l0.05-0.39l-0.65-0.23l-0.13-0.18l-0.35,0.08l0.09-0.27l-0.1-0.19
				l-0.67-0.08v0l-0.38-0.04l-0.29,0.13l-0.43-0.04l-0.27,0.26l-0.26-0.27l-0.1-0.56l-0.2-0.12l-0.53-0.09l-0.13-0.28l0.2-0.22
				l-0.05-0.06l-0.37-0.01l0-0.36l-0.06-0.04l-0.62,0.22l-0.31-0.08l-0.1,0.15l0.1-0.15l-0.09-0.28l-0.23-0.26l-0.07-0.39l0.12-0.27
				l-0.04-0.21l-0.04-0.04l-0.33,0.11l-0.1,0.08l-0.15,0.1l-0.18-0.05l-0.5-0.34l-0.06,0.06l0.04,0.23l-0.13,0.26l-0.19-0.17
				l-0.08-0.21l-0.39-0.16l-0.37,0.2l-0.23,0.23l-0.64,0.06l-0.39,0.96l0.04,0.39l-0.19-0.05l-0.12,0.1l-0.11-0.07L404,59.8
				l-0.29,0.03l-0.04,0.16l0.14,0.51l-0.02,0.46l-0.3,0.2l-0.4-0.09l-0.24,0.29l0.05,0.45l0.06,0.03l0.18,0.01l0.12-0.25l0.35-0.16
				l0.17,0.51l0.01,0.42l0.14,0.3l-0.24,0.06l-0.35,0.41l0.28,0.3l-0.25,0.19l-0.2,0.4l0.2,0.64l-0.05,0.34l0.12,0.3l0.16,0.28
				l0.28,0.03l-0.32,0.56l0.12,0.24l0.06,0.08l0.24,0.13l-0.21,0.24l0.06,0.08l-0.02,0.88l0.06,0.26l0.14,0.08l0,0.14l0.48,0.65
				l0.28,0.61l0.02,0.29l0.32-0.09l0.46-0.4l0.38-0.14l0.07,0.04l-0.02-0.2l0.07-0.14l0.57-0.41l0.24-0.37l-0.16-0.44l0.28-0.08
				l0.45-0.28l0.33-0.01l-0.21-0.23l0.01-0.26l0.17-0.12l0-0.11l-0.2-0.21l-0.21-0.8l-0.31-0.12l-0.05-0.13l-0.28-0.09l0.22-0.19
				l-0.15-0.22l0.02-0.29l0.6,0.1l1.16-0.07l0.46,0.1l0.46,0.32l0.14-0.44l0.15-0.07l0.08,0.04l0.16-0.18l0,0l0.39,0.13l0.21-0.02
				l0.16,0.32l0.24,0.05l0.09-0.07l0.08-0.14l0.22-0.1l0.72,0.14l0.25-0.04l0.53,0.26l0.38,0.69l0.33,0.12l0.47-0.03l0-0.34
				l0.47-0.07l0.09,0.11l-0.14,0.24l0.48,0.03l-0.06-0.2l0.29-0.35l0.49-0.12l0.24-0.27L415.95,64.66z"></path>
		</g>
		<g id="villadiego">
			<path id="path4117" class="st3 fase_0" d="M362.13,59.76l-0.11,0.01l-0.41,0.47l-0.01,0.18l-0.55,0.19l-0.06,0.15l0.06,0.13
				l-0.14,0.22l-0.29,0.07l-0.46,0.31l-0.37,0.03l-0.11-0.07l0.05,0.23l-0.38,0.32l-0.28-0.33l-0.15,0l0.04-0.59l-0.1,0.03
				l-0.48,0.28l-0.12,0.21l-0.37,0.26l0.26,0.24l0.03,0.15l-0.16,0l0,0.23l0.21,0.21l0.36,0.12l0.01,0.33l0.16,0.57l-0.22,0.22
				l0.12-0.03l0.04,0.39l-0.16,0.11l0.11,0.33l0.15,0.08l0.13,0.21l-0.12,0.46l0.27,0.29l-0.41,0.05l-0.35,0.39l-0.41,0.2
				l-0.08,0.16l0.23,0.24l0.03,0.28l0.11,0.1l-0.03,0.11l0.15,0.14l-0.16,0.19l0.09,0.09l0.37-0.1l0.17,0.09l0.53,0.34l0.16,0.36
				l0.44,0.37l0.12,0.01l0.15,0.61l-0.08,0.42l0.02,0.44l0.17,0.04l0.28,0.31l0.16,0.06l0.12,0.24l0.18,0v0.32l0.25-0.05l0.01,0.19
				l0.09,0.07l0.17-0.08l0.14,0.23l0.38,0.15l0.53,0.77l0.29,0.06l-0.23,0.34l0.01,0.23l0.03,0.18l0.34,0.32l0.07,0.5l0.38,0.53
				L363.45,75l0.07,0.14l-0.03,0.25l0.13,0.04l-0.2,0.37l0.08,0.02l0.19,0.66l-0.03,0.21l0.14-0.07l0.59,0.13l0,0.34l0.31,0.03
				l0.83-0.07l0.05-0.09l0.63-0.09l0.29,0.61l-0.03,0.21l0.76,0.46l0.13,0.29l-0.13,0.24l-0.28,0.18l-0.18,0.5l-0.42,0.26
				l-0.46,0.61l-0.07,0.2l0.29,0.54l0.46,0.23l0.1,0.18l0.31,0.17l0.22,0.03l0.08,0.16l0.12,0.01l0.19-0.18l0.09-0.5l0.27-0.29
				l0.43-0.23l0.13,0.05l0.39-0.2l0.02,0.11l0.54,0.4l0.21,0.27l0.46-0.43l0.26-0.1l0.12-0.17l0.19-0.01l-0.14-0.7l0.2-0.53
				l0.01-0.25l0.32-0.14l0.26-0.17l0.08-0.16l-0.11-0.13l0.09-0.34l-0.21-0.55l0.05-0.18l-0.33-0.28l-0.14-0.37l0.17-0.33l0.55-0.33
				l0.08,0.03l0.07-0.1l0.19-0.03l0.05-0.09l-0.11-0.22l0.02-0.15l0.36-0.19l0.16-0.22l0.19-0.02l0.4-0.45l0.32,0.05l0.21-0.09
				l0.22-0.34l-0.03-0.2l0.31-0.35l0.42-0.93l0.17-0.21l0.68-1.41l0.49-0.44l0.27-0.63l-0.04-0.56l0.36-0.33l0.05-0.14l0.14-0.06
				l-0.1-0.06l0.04-0.3l-0.14-0.25l0.15-0.59l0,0l0.01-0.18l0.47,0.17l0.27-0.19l0.31-0.03l0.14-0.09l0.13,0.18l0.24-0.01l0.08,0
				l-0.12-0.24l0.14-0.19l-0.08-0.15l0.08,0l0.19-0.57l0.11-0.07l0.05-0.16l-0.18-0.18l-0.67-0.35l-0.09-0.2l-0.38-0.05l-0.2-0.2
				l-0.17-0.06l-0.15-0.14l0.04-0.18l-0.84-0.35l-0.23,0.25l-0.1,0.51l-0.3,0.33l0.05,0.17l-1.17,0.25l0,0L374,65.8l-0.51-0.51
				l-0.54-0.08l-0.09,0.09l-0.89-0.12l-0.29-0.11l-0.79,0l-0.04-0.28l-0.52,0.07l-0.05-0.43l0.08-0.14l-0.09-0.48l-0.52-0.21
				l0.02-0.11l-1.25-0.48l-0.27,0.03l-1.2-0.71l-0.46-0.16l-0.05-0.19l-0.16-0.12l-0.39-0.02l-0.86-0.36l0.02,0.13l-0.32,0.04
				l0.06-0.1l-0.05-0.07l-0.51-0.14l-0.03-0.12l-0.53-0.07l-0.29-0.13l-0.5-0.02l-0.5-0.34L362.17,60l0.05-0.08L362.13,59.76z"></path>
		</g>
		<g id="salas_los_infantes">
			<path class="st3 fase_0" d="M408.3,93.07l-0.45-0.89l-0.29-0.34l-0.3-0.14l-0.41,0.13l-0.1-0.11l-0.18-0.7l0.03-0.58l-0.4-0.43
				l0.13-0.11l-0.22-0.03l0.01-0.1l-0.3-0.24l-0.14,0.02l-0.1-0.16l-0.01-0.51l-0.4-0.27L405,88.28l0.02-0.08l0.41-0.19l0.22-0.27
				l0.3-0.12l0.18-0.29l0.07-0.54l-0.11-0.27l-0.64-0.05l-0.55-0.35l-0.59-0.06l-0.04-0.11l0,0l-0.02,0.09l-0.43,0.12l-0.11,0.13
				l-0.27,0.09l-0.64-0.4l-0.91,0.55l-0.27,0.61l-0.5,0.16l-0.43,0.68l-0.56,0.03l-0.19,0.1l-0.4,0.35l-0.05,0.29l-0.2,0.18
				l-0.05,0.35l-0.42,0.15l-1.26-0.44l-0.08-0.2l-0.62-0.5l-0.98-0.21l-0.53-0.51l-0.49,0.78l-0.39,0.28l-0.39,0.46l-0.1,0.18
				l0.05,0.23l-0.3,0.09l-0.4-0.12l-0.35-0.2l-0.55-0.15l-0.45-0.31L391,88.21l-0.48-0.56l-0.06,0.23l0.04,0.21l-0.1,0.07
				l-0.18-0.04l-0.03-0.08l-0.04,0.16l-0.09,0.02l0.13,0.03l0.19,0.22l0.37-0.26l-0.01,0.17l-0.35,0.22l-0.31,0.53l0.2,0.42
				l0.42,0.27l-0.09,0.02l-0.44,0.72l-0.08,0.57l-0.79-0.2l-0.24,0.02l0.04,0.18l-0.2,0.13l0.07,0.18l0.16,0.07l-0.46-0.03
				l-0.52,0.3l-0.21-0.09l-0.05,0.44l0.35,0.21l0.42-0.1l0.17,0.13l0.22-0.14l0.94,0.33l-0.5,0.58l-0.19,0.39l0.24,0.27l0.15,0.07
				l0.15-0.04l0.27,0.2l0.04,0.4l-0.19,0.37l0.15,0.01l0.27,0.35l0.04,1.22l-0.18,0.18l-0.18,0.35l0.33-0.1l0.22,0.37l0.16,0
				l0.4,0.64l0.18,0.72l0.17-0.03l-0.12,0.7l0.26,0.09l-0.15,0.1l-0.22,0.66l0.04,0.22l-0.24,0.57l-0.04,0.48l-0.71,0.52l-0.26,0.37
				l0.03,0.38l0.55,0.12l-0.19,0.06l-0.08,0.27l0.4,0.11l0.46,0.18l0.12,0.16l0.94-0.2l0.34,0.14l-0.22,0.65l0.68,0.3l0.32-0.12
				l0.34-0.39l0.72-0.13l0.42,0.3l0.71,0l0.46,0.26l0.09,0.3l0.57,0.25l0.14,0.01l0.63,0.32l0.41,0.25l0.62,0.22l0.24-0.4v0
				l0.36,0.09l0.15,0.18l0.08,0.52l0.24,0.07l0.25-0.15l0.41,0.27l0.25-0.86l0,0l-0.25,0.86l0.13,0.09l0.96-0.84l0.75,0.57
				l0.21-0.29l1.16-0.42l0.03-0.19l0.25,0.01l0.17,0.12l0.11-0.03l0.04-0.21l-0.1-0.33l0.33-0.9l0.04-0.07l-0.76-0.38l0.03-0.73
				l-0.12-0.8l-0.4-1.08l0.36-0.86l0-0.07l-0.2,0.14l0,0l0.2-0.14l0,0.07l0.02,0.07l0.28,0.11l0.18-0.12l0.06-0.19l0.19-0.11
				l0.06-0.14l0.29-0.11l0.44,0.08l0.27-0.06l0.18-0.3l0.01-0.16l0.43-0.38l0.44-0.59l1.06-0.3l0.36,0.07l0.16-0.05l-0.03-0.02
				l0.18-0.64l-0.02-0.17l-0.31-0.09l-0.14-0.23l-0.01-0.28l0.74-0.47l0.17-0.63l0.37-0.82L408.3,93.07z"></path>
		</g>
		<g id="melgar_fernamental">
			<path id="path4219" class="st3 fase_0" d="M358.79,67.75l-0.24,0.09l-0.23,0.21l-0.45,0.09l-0.2,0.2l-0.27-0.01l0.31,0.56l-0.22,0.07
				l-0.01,0.2l0.14,0.15l0.89,0.42l-0.05,0.1l0.06,0.39l-0.09,0.09l0.17,0.27l0.13,0.47l-0.18,0.29l-0.39,0.14l0.53,0.08l0.42-0.08
				l0.42,0.46l0.21,0.46l0.04,0.12l-0.18,0.7l0.23,0.4l-0.22,0.02l-0.16,0.16l0.09,0.13l0.19-0.06l0.08,0.23l-0.06,0.14l0.03,0.08
				l0.11-0.01l0.04,0.14l-0.36-0.02l-0.7,0.17l-0.29-0.09l-0.26,0l-0.1,0.09l-0.89-0.09l-0.01,0.09l-0.41-0.3l-0.24,0.77l-0.5,0.75
				l0.08,0.4l-0.07,0.17l0.07,0.2l0.11,0.06l0.08,0.29l-0.07,0.27l0.06,0.49l-0.1,0.24l0.38,0.59l0.65-0.19l0.49-0.25l0.47-0.05
				l0.24-0.15l-0.07,0.29l0.2,0.05l0.05,0.26l-0.1,0.31l-0.27-0.14l-0.02,0.25l0.17,0.25l0.01,0.18l-0.09-0.04l-0.07,0.11l0.06,0.14
				l0.2,0.14l0.19,0.08l0.19,0.25l-0.06,0.28l-0.15,0.21l0.01,0.22l0.1,0.06l-0.01,0.13l0.21,0.43l0.21,0.16l0,0.09l-0.08,0.02
				l0-0.1l-0.14-0.07l-0.12,0.13l0.06,0.19l0.09,0.01l-0.07,0.4l0.05,0.14l0.4-0.02l0.08,0.84l-0.11,0.21l0.25,0.31l-0.11,0.24
				l0.11,0.1l0.01,0.24l0.17,0.1l0.04,0.17l-0.13,0.52l0.51,0.18l0.11,0.35L360.47,86l-0.32,0.28v0.08l0.12,0.12l0.26,0.07
				l0.04,0.12l-0.21,0.66l0.03,0.48l0.43-0.01l0.22,0.09l0.13,0.32l0.26,0.2l0,0.25l0.6,0.62l0.09,0.25l-0.09,0.67l-0.25,0.36
				l-0.36,0.18l0.22,0.2l-0.11,0.19l-0.08,0.65l0.2,0.14l0.55,0.3l0.35-0.23l-0.11-0.48l0.1-0.57l0.34-0.2l-0.58-0.21l-0.17-0.16
				l0-0.31l0.54-1.37l1.16-1.1l0.15-0.05l0.36,0.07l0.12,0.06l0.6-0.31l1.17-0.88l0.04-0.32l0.21-0.27l-0.28-0.82l0.61-0.67
				l0.69-0.48l-0.01-0.6l0.02-0.09l0.36-0.01l0.35-0.45l0.34,0.05l0.16-0.13l0.26,0.06l0.1-0.11l0.05-0.56l0.21-0.32l0.18-0.08
				l0.14-0.48l-0.21-0.27l-0.54-0.4l-0.02-0.11l-0.39,0.2l-0.13-0.05l-0.43,0.23l-0.27,0.29l-0.09,0.5l-0.19,0.18l-0.12-0.01
				l-0.08-0.16l-0.22-0.03l-0.31-0.17l-0.1-0.18l-0.46-0.23l-0.29-0.54l0.07-0.2l0.46-0.61l0.42-0.26l0.18-0.5l0.28-0.18l0.13-0.24
				l-0.13-0.29l-0.76-0.46l0.03-0.21l-0.29-0.61l-0.63,0.09l-0.05,0.09l-0.83,0.07l-0.31-0.03l0-0.34l-0.59-0.13l-0.14,0.07
				l0.03-0.21l-0.19-0.66l-0.08-0.02l0.2-0.37l-0.13-0.04l0.03-0.25L363.45,75l0.02-0.28l-0.38-0.53l-0.07-0.5l-0.34-0.32
				l-0.03-0.18l-0.01-0.23l0.23-0.34l-0.29-0.06l-0.53-0.77l-0.38-0.15l-0.14-0.23l-0.17,0.08l-0.09-0.07l-0.01-0.19l-0.25,0.05
				v-0.32l-0.18,0l-0.12-0.24l-0.16-0.06l-0.28-0.31l-0.17-0.04l-0.02-0.44l0.08-0.42l-0.15-0.61l-0.12-0.01l-0.44-0.37l-0.16-0.36
				L358.79,67.75L358.79,67.75z"></path>
		</g>
		<g id="pampliega">
			<path id="path4423" class="st3 fase_1" d="M370.67,80.48l-0.19,0.01l-0.12,0.17l-0.26,0.1l-0.46,0.43l-0.14,0.48l-0.18,0.08
				l-0.21,0.32l-0.05,0.56l-0.1,0.11l-0.26-0.06l-0.16,0.13l-0.34-0.05l-0.35,0.45l-0.36,0.01l-0.02,0.09l0.01,0.6l-0.69,0.47
				l-0.61,0.67l0.28,0.82l-0.21,0.27l-0.04,0.32l-1.17,0.88l-0.6,0.31l-0.12-0.06l-0.36-0.07l-0.15,0.05l-1.16,1.1l-0.54,1.37
				l0,0.31l0.17,0.16l0.58,0.21l-0.34,0.2l-0.1,0.57l0.11,0.48l0.06,0.26l0.35-0.2l0.57-0.11l0.4,0.12l0.11,0.32L364,92.57l0.17,0
				l0.22-0.2l0.22,0.06l0.24-0.32l0.23,0.19l0.28,0.31l0.03,0.27l0.15,0.23l0.46,0.1l0.28,0.25l0.38-0.12l0.54,0.23l0.28,0.01
				l-0.17,0.52l-0.23,0.12l-0.01,0.17l-0.44,0.18l-0.69-0.22l-0.34-0.01l-0.04,1.37l0.17,0.2l0.08,0.29l0.23,0.16l0.32,0.57
				l0.18,0.11l0.66-0.37l0.27,0.21l0.37,0.04l0-0.4l0.08-0.23l-0.07-0.02l0.11-0.58l-0.03-0.21l0.15-0.29l0.29-0.17l0.33-0.03
				l0.06-0.09l0.31,0.24l0.39-0.11l0.11,0.29l-0.04,0.09l0.21,0.22l0.51,0.15l0.14,0.21l0.68-0.53l0.16-0.22l-0.05-0.18l0.13-0.77
				l0.35-0.2l1.76-1.05l0.32-0.14l0.27,0.13l0.02-0.46l0.35-0.39l0.63-0.13l0.98-0.59l-0.59-0.19l-0.4,0.2l0.01-0.22l-0.13-0.07
				l-0.13-0.8l-0.18-0.25l0.06-0.28l0.37,0.02l0.11,0.16l0.44,0.03l0.26-0.31l0.18,0.48l0.16,0.13l0.24-0.06l0.62-0.49l0.15-0.22
				l-0.05-0.23l0.23-0.99l-0.13-0.16l-0.1,0.14l-0.62-0.21l-0.73,0.18l-0.21-0.6l-0.18,0.1l-0.16-0.04l-0.24-0.3l-0.44,0.04
				l-0.43-0.47l-0.4-0.23l-0.29,0.18l-0.4-0.27l0.15-0.23l0.07-0.41l-0.04-0.47l-0.12-0.21l-0.13-0.01l-0.06-0.11l-0.61,0.05
				l-0.17,0.24l-0.13,0.01l-0.05,0.1l-0.21-0.03l-0.34-0.42l-0.22-0.5l-0.02-0.06l0.24-0.18v-0.15l0.13-0.21l-0.12-0.4l-0.17-0.21
				l0.08-0.78l0.29-0.28l0.26-0.1l-0.23-0.22l0.19-0.15l-0.14-0.81l-0.3-0.22l-0.53-0.16l-0.09-0.22L370.67,80.48z M368.4,93.25
				l0.21,0.54l0.11,0.28l0.05,0.56l-0.31,0.13l0.18-0.5L368.55,94l-0.23-0.16l0-0.29l-0.09-0.06L368.4,93.25z"></path>
		</g>
		<g id="huerta_de_rey">
			<path class="st3 fase_0" d="M399.09,109.76l-0.06-0.47l-0.04-0.24l-0.02-0.1l-0.04,0.12l-0.09,0.29l-0.21,0.1l0.11-0.23l0.04-0.11
				l-0.07-0.17l0.15-0.45l-0.14-0.23l-0.17-0.07l-0.01-0.75l0.21-0.18l-0.04-0.32l0.24-0.87l-0.2-0.09l-0.23-0.37l-0.62-0.22
				l-0.41-0.25l0,0l-0.63-0.32v0l-0.08,0.05v0l0.08-0.05l-0.14-0.01l-0.57-0.25l-0.09-0.3l-0.46-0.26l-0.71,0l-0.42-0.3l0,0
				l-0.72,0.13l-0.34,0.39l-0.32,0.12l-0.54,0.72l-0.3,0.56l0.02,0.47l0.23,0.51l-0.08,0.22l0.18,0.54l-0.37,0.38l-0.09,0.23
				l0.15,0.56l0.5,0.51l-1.28,0.96l0.17,0.18l-0.11,0.27l-0.72,0.29l0.41,0.89l-0.01,0.19l0.06,0.08l0.18-0.03l0.03,0.11l0.33,0.19
				l-0.09,0.35l-0.18,0.23l-0.11,0.54l0.66,0.52l0.35,0.16l0.34,0.92l0.35,0.4l-0.04,0.18l0.1,0.22l-0.36,0.29l-0.03,0.29l0.12,1.07
				l0.37,0.08l0.12-0.46l0.31-0.45l-0.01-0.13l0.51-0.48l0.05-0.33l0.33-0.13l0.6,0.06l0.49-0.09l-0.1-0.32l-0.04-0.58l0.08-0.18
				l-0.03-0.43l0.33-0.41l-0.11-0.25l0.09-0.22l0.39-0.04l0.31-0.15l0.21,0.11l0.05-0.07l0.25-0.02l0.05-0.44l0.49-0.1l0.1-0.49
				l0.45-0.1l0.37,0.1l0.13-0.11l0.34-0.35l-0.33-0.51l-0.09-0.54l0.42-0.33V109.76z"></path>
		</g>
		<g id="aranda_rural">
			<path class="st3 fase_0" d="M393.33,117.48l-0.37-0.08l-0.12-1.07l0.03-0.29l0.36-0.29l-0.1-0.22l0.04-0.18l-0.35-0.4l-0.34-0.92
				l-0.35-0.16l-0.66-0.52l0.11-0.54l0,0l0.18-0.23l0.09-0.35l-0.33-0.19l-0.03-0.11l-0.18,0.03l-0.06-0.08l0.01-0.19l-0.41-0.89
				l0.72-0.29l0.11-0.27l-0.17-0.18l1.28-0.96l-0.5-0.51l-0.15-0.56l0.09-0.23l0.37-0.38l-0.18-0.54l0.08-0.22l-0.23-0.51
				l-0.02-0.47l0.3-0.56l0.54-0.72l-0.68-0.3l0.22-0.65l-0.34-0.14l-0.94,0.2l-0.12-0.16l-0.46-0.18v0l-0.29,0.66l-0.29,0.21
				l-0.1,0.53l-0.34,0.07l0.03,0.43l-0.3,0.44l-1.06-0.61l-0.88-0.2l-0.63,0.36l-0.32,0.09l0.09,0.48l-0.31,0.42l0.02,0.26
				l-0.16,0.08l0.02,0.14l-0.16,0.46l0.08,0.13l-0.17,0.79l-0.64-0.22l-0.48-0.04l-0.64,0.19l-0.19-0.01l-0.23,0.16l-0.12,0
				l-0.27-0.19l-0.14,0.49l-0.14,0.31l-0.19,0.11l-0.54-0.14l-1.12,0.51l-0.03-0.11l-0.5,0.56l-0.2,0.18l-0.12-0.07l-0.48,0.25
				l-0.42,0.06l-0.37-0.02l-0.06-0.06l-0.49,0.12l-0.05,0.3l-0.47,0.23l-0.1,0.5l-0.23,0.16l-0.16,0.24l-0.49,0.33L377.1,112l-0.1,0
				l-0.01,0.1l-0.11-0.01l-0.21,0.21l0.03,0.5l-0.23,0.15l-0.12,0.36l-0.37-0.11l-0.6,0.29l0.13,0.3l0.21,0.14l-0.06,0.03l0.15,0.4
				l-0.04,0.55l-0.3,0.21l0.08,0.26l0.47,0.46l0.53,0.09l0.35,0.2l0.26,0.34l0.26,0.2l0.47-0.55l0.52,0.06l0.26,0.35
				c-0.04,0.06-0.1,0.11-0.13,0.17l-0.91,0.17l-0.12,0.45l0.06,0.18l-0.12,0.39l0.04,0.15l0.74,0.82l0.03,0.11l0.14-0.05l0.04,0.17
				l0.19-0.15l-0.02,0.21l0.06,0.06l0.15-0.17l0.53,0.41l-0.03,0.17l-0.6,0.39l-0.33,0.11l-0.44-0.32l-0.44,0.03l-0.02-0.17
				l-0.21,0.03l-0.38,0.33l0.02,0.12l-0.2-0.04l-0.1,0.1l-0.1,0.08l-0.04,0.18l-0.32,0.34l0.02,0.22l-0.22,0.06l-0.08,0.16
				l0.11,0.08l0.03-0.07l0.14,0.06l0.39,0.29l0.17-0.04l-0.03,0.14l0.12,0.08l-0.27,0.12l-0.04,0.11l0.32,0.46l-0.15,0.12
				l-0.02,0.25l0.1,0.04l0.08,0.24l0.17-0.28l0.13,0.3l0.09-0.03l0.19,0.15l0.04-0.06l0.37,0.82l0.07,0.27l-0.04,0.09l-0.15,0.06
				l0.02,0.16l-0.04,0.39l0.2,0.4l-0.12,0.24l-0.04,0.43l0.37,0.27l0.26,0.3l0.44-0.04l0.27,0.23v0l-0.05,0.63l0.07,0.56l0.49-0.07
				l0.55,0.02l0.19,0.55l0.42-0.37l-0.13-0.66l0.35-0.5l-0.22-0.25l0.04-0.12l0.56,0.55l0.49,0.21l0.11-0.09l0.13,0.13l0.33,0.03
				l0.31-0.18l0.22,0.23l0.76-0.59l0.24-0.11l0.1-0.28l0.05,0.03l0.17-0.16l0.22-0.05l0.35-0.17l0.14-0.3l0.25-0.15l0.38-0.57
				l0.85-1.06l0.16,0.01l0.03-0.22l0.35,0.18l-0.02-0.12l0.22,0.04l-0.19-0.22l0.27-0.67l-0.02-0.17l-0.22-0.16l0.04-0.39l0,0
				l0.06-0.18l-0.13-0.2l0.14-0.27l0.18-0.14l0.15,0.14l0.4-0.18l0.26,0.04l0.2,0.13l0.22-0.48l-0.58-0.54l0.19-0.1l0.2,0.04
				l0.04-0.53l-0.07-0.03l0.38-0.15l0.12-0.09l0.23,0.15l0.28,0.35l0.25,0.17l-0.04,0.11l0.06,0.06l0.09,0.03l0.08-0.08l0.45,0.07
				l0.53-0.03l0.01,0.15l0.07,0.01l0.17-0.13l0.29,0.25l-0.01,0.18l0.12-0.02l0.26-0.02l0.38-0.87l0.29-0.32l0.07-0.36l0.23-0.2
				l-0.16-0.16l-0.04-0.24l0.16-0.25l0.06-0.28l0.09-0.02l0.13-0.35l0.33,0.01l0.1,0.11l0.1-0.2L393.33,117.48z M386.97,117.31
				L386.97,117.31l-1.17,0.52l-0.58-0.27l-0.58,0.05l-0.03,0.25l-0.7,0.04l-0.55-0.1l-0.02,0.2l-0.53,0.18l-0.13,0.12l-0.25-0.02
				l-0.2,0.18l-0.55,0.21l-0.45,0.04l-0.3,0.11l-0.35-0.13l-0.14-0.34l0.14-0.93l-0.1-0.24l-0.35-0.27c0,0,0-0.01,0-0.01l0.06-0.03
				l-0.06,0.02c0,0,0,0,0,0l0.06-0.02l0.25,0l0.09-0.13l-0.04-0.62l0.15-0.65l-0.24-0.26l-0.47-0.23l-0.46,0.11l-0.36,0.22
				l-0.57,0.16l-0.12-0.46l-0.52-0.92l-0.45-0.63l0.39-0.09l0.13-0.1l0.28,0.04l0.42-0.17l-0.3-0.2l0.61-0.41l0.12,0l0.11,0.11
				l0.08,0.22l0.35-0.08l0.14-0.12l0.18,0.35l0.36,0.37l0.01,0.19l0.16,0.28l0.32,0.23l0.44-0.14l1.1,0.1l0.26-0.24l0.57-0.25
				l0.19-0.02l0.21,0.23l1.44,0.58l1.31-1.14l0.21,0.42l-0.03,0.58l-0.65,0.22l-0.21,0.57l-0.11,0.06l-0.21-0.25l0.1,0.56l0.1,0.37
				l0.38,0.06l0.27,0.46l0.63,0.36l0.17,0.01L386.97,117.31z"></path>
		</g>
		<g id="valle_tobalina">
			<path class="st3 fase_0" d="M404.71,57.85l0.34-1.01l0.2-0.27l-0.3-0.02l-0.38-0.33l-0.78-0.48l-0.38-0.29l-0.15,0l-0.85-0.46
				l-0.49-0.48l-0.46,0.05l-0.36-0.19l-2.3-2.44l-0.17-0.52l-0.11-0.03l-0.14-0.21l-0.23,0.33l-0.32-0.34
				c-0.15,0.59-0.13,1.37-0.87,1.83c-0.24,0.15,0.14,0.95,0,1.2c-0.15,0.27-0.71,0.1-0.88,0.36c-0.14,0.22,0.29,1.1,0.13,1.31
				c0.43,0.5,0.83,0.35,1.16,0.91c0.57,0.95,0.36,1.99,0.43,3.14l0.21,0.12l0.19,0.39l0.42,0.17l0.34,0.28l0.08-0.02l0.1-0.08
				l-0.02-0.29l0.21-0.24l0.3,0.11l0.26,0.37l0.14-0.15l0.78-0.04l0.09-0.07l0.04,0.08l0.65-0.24l0.16,0.01l-0.13,0.65l0.34,0.02
				l0.01,0.14l0.68-0.06l0.21,0.29l0.24-0.29l0.4,0.09l0.3-0.2l0.02-0.46l-0.14-0.51l0.04-0.16L404,59.8l0.15-0.23l0.11,0.07
				l0.12-0.1l0.19,0.05l-0.04-0.39l0.39-0.96l-0.68,0.07l0.24-0.33L404.71,57.85z"></path>
		</g>
		<g id="valle_valdebezana">
			<path class="st3 fase_0" d="M387.09,42.89l-0.3-0.14l-0.37-0.89l-0.62-0.6l0.04-0.42l-0.18-0.27l-0.5-0.35l-0.36,0.09l-0.28-0.06
				l-0.59,0.04l-0.38-0.2l-0.39-0.07l-1.05,0.34l-0.56-0.76l-0.43-0.18l0,0l-0.37,0.19l-0.41,0.12l-0.39,0.02l-0.32,0.17l-0.16,0.38
				l0.11,0.38l-0.14,0.08l-0.28,0.17l-0.33-0.12l-0.1,0.11l-0.45,0.13l-0.47-0.27l-0.21-0.01l-0.16-0.17l-0.19,0.05l-0.24,0.26
				l-0.23,0.02l-0.36,0.01l-0.37-0.11l-0.1-0.1l-0.15,0.23l0,0.16l0.19,0.33l0.58,0.39l-0.09,0.05l0,0.23l-0.35,0.39l-0.29,0.18
				l0.03,0.37l-0.08,0.11l-0.19-0.01l-0.23-0.22l-0.14,0.06l-0.08-0.09l-0.33,0.01l-0.12,0.07L374.18,43l-0.08,0.3l-0.67,1.09
				l-0.62,0.31l-0.84,0.25l-0.62,0.72l-0.54,1.28l-0.63,1.87l-0.05,0.16l0.13,0.27l0.41,0.4l0.19,0.36l0.77,0.02l0.33,0.1l0.26-0.04
				l0.39-0.24l-0.27,0.35l-0.05,0.22l0.16,0.24l0.29-0.26l0.12-0.6l0.21-0.03l0.27-0.38l0.16-0.07l0.66-0.04l0.1-0.24l-0.03-0.28
				l0.08-0.14l0.01-0.59l0.39,0.05l0.31-0.1l0.12-0.28l1.12,0.81l0.09,0.3l0.3,0.35l-0.01,0.03c0.23,0.08,0.4,0.21,0.55,0.38
				c0.07-0.03,0.14-0.06,0.21-0.07c0.37-0.09,0.75-0.14,1.13-0.11c0.29,0.02,0.58,0.06,0.87,0.07c0.22,0,0.41,0.06,0.59,0.15
				c0.18,0.02,0.36,0.06,0.53,0.08c0.56-0.19,1.1-0.41,1.63-0.68c0.83-0.88,1.8-1.64,2.68-2.5c0.88-0.87,1.51-1.97,2.12-3.08
				c0.01-0.05,0.02-0.11,0.03-0.16c0.03-0.13,0.08-0.24,0.13-0.37L387.09,42.89z"></path>
		</g>
		<g id="medina_de_pomar">
			<path class="st3 fase_0" d="M397.36,56.79c-0.33-0.56-0.73-0.41-1.16-0.91c0.16-0.21-0.27-1.09-0.13-1.31c0.17-0.26,0.73-0.08,0.88-0.36
				c0.14-0.25-0.24-1.05,0-1.2c0.75-0.46,0.73-1.23,0.87-1.83l0.32,0.34l-1.13-1.19l-0.2,0.25l-0.27-0.07l-0.51-0.47l-0.26,0
				l-0.12,0.22l-0.28-0.2l-0.33-0.04l0,0.19l-0.16,0.23l-0.86,0.03l-0.08-0.22l-0.37-0.43l0,0l-0.38,0.04l-0.65,0.43l-0.18,0.2
				l0.1,0.03l-0.01,0.7l-0.26,0.4l0.28,0.49l-0.09,0.08l-0.37-0.51l-0.27,0.13l-0.37-0.05l0.02,0.32l-0.39,0.24L390.68,52l0.02-0.24
				l-0.08-0.15l0.03-0.51l-0.27,0l-0.1-0.45l-0.16-0.23l-0.47-0.05l-0.23-0.07l-0.04,0.15l0,0l0.04-0.15l0.23,0.07l-0.24-0.11
				l-0.07-0.09l0-0.27l-0.19-0.02l-0.06-0.11l0.19-0.48l0-0.46l0.19-0.17l-0.17-0.05l0.01-0.21l0.25-0.15l0.04-0.2l-0.14-0.23
				l-0.3-0.11l0.02-0.05l0.14-0.18l0.3-0.1l0.05-0.44l0.12,0.03l0.15-0.33l0.23-0.1l-0.01-0.14l0.34-0.47l-0.15-0.16l-0.07-0.16
				l-0.66,0.12l-0.13-0.14l0.18-0.08l-0.02-0.14l-0.3,0.12l-0.07,0.14l-0.26-0.19l0.05-0.16l-0.78-0.43l0.18-0.08l-0.01-0.2
				l-0.35-0.68l0.22-0.75l-0.43-0.37l-0.32,0.11l-0.23-0.06l-0.27,0.06c-0.04,0.12-0.09,0.24-0.13,0.37
				c-0.01,0.05-0.02,0.11-0.03,0.16c-0.61,1.11-1.24,2.21-2.12,3.08c-0.88,0.86-1.85,1.62-2.68,2.5c-0.53,0.27-1.07,0.49-1.63,0.68
				c-0.18-0.03-0.35-0.06-0.53-0.08c-0.17-0.09-0.37-0.15-0.59-0.15c-0.29,0-0.58-0.04-0.87-0.07c-0.38-0.03-0.76,0.03-1.13,0.11
				c-0.07,0.02-0.15,0.04-0.21,0.07c-0.15-0.17-0.32-0.31-0.55-0.38l0.01-0.03l-0.15,0.52l-0.19,0.09l-0.23-0.04l-0.11-0.12
				l-0.27,0.05l-0.16,0.57l-0.12,0.16l0.06,0.05l-0.39,0.25l-0.42-0.12l-0.17-0.16l-0.31,0.09l-0.48-0.37l-0.21-0.27l-0.01,0.12
				l-0.1,0.07l0.14,0.18l0.32,0.18l0.25,0.9l-0.4,0.06l-0.25-0.17l-0.06-0.13l-0.25,0.15l-0.15-0.08l-0.16,0.08l0.88,0.23l0.2,0.25
				l-0.33,0.54l-0.31,0.13l-0.09,0.37l0.1,0.1l0.56-0.05l0.11,0.2l0.35-0.03l0.31,0.29l0.26-0.08l0.21-0.2l-0.22-0.35l0.01-0.38
				l0.11-0.22l-0.12-0.04l0.24-0.58l0.17-0.21l0.48,0.34l0.61,0.25l0.46,0.55l0.32-0.14l-0.04-0.08l0.32,0.09l0.91-0.23l-0.04-0.21
				l0.22-0.11l0.26,0.17l0.62,0.15l0.23,0.13l0.63-0.35l1.05,1.05l-0.08-0.25l0.11-0.05l-0.1-0.33l0.21-0.05l0.15-0.28l0.41-0.04
				l0.21-0.06l0.7,0.03l0.16,0.15L383,52.06l0.78,0.34l0.23-0.18l0-0.27l0.16-0.06l-0.01,0.61l-0.18,0.18l0.07,0.49l-0.15,0.83
				l0.12,0.2l0.64,0.5l0.62,0.92l0.04,0.25l-0.23,0.75l0.47,0.36l0.68-0.27l0.43-0.01l0.22,0.16l0.43-0.17l0.14,0.27l0.46,0.16
				l1.58-0.01l0.66,0.15l0.19-0.57l-0.11-0.04l0.15-0.32l0.91,0.55l0.43,0.14l0.04,0.11l0.55,0.37l0.08-0.19l0.21-0.11l0,0
				l-0.21,0.11l0.41-0.11l0.13-0.33l0.34-0.12l0.49,0.3l-0.19,0.36l0.24,0.28l0.58,0.13l0.29-0.02l0.06,0.34l0.66,0.06l0.39,0.02
				l0.26-0.25l0.32,0.02l0.46,0.38l0.2,0.6l0.02,0.34l0.03,0.25l-0.08,0.15l0.66,0.14l0.32,0.18l-0.21-0.12
				C397.72,58.78,397.93,57.74,397.36,56.79z M395.34,52.34l-0.39-0.05l-0.16,0.12l-0.45-0.09l-0.39-0.41l-0.14-0.26l0.02-0.34
				l0.11-0.15l0.07,0.01l0.28-0.44l0.14,0.03l0.61-0.17v0l0.06,0.39l-0.21,0.22l-0.1,0.25l0.17,0.31l0.12-0.02l0.32,0.17
				L395.34,52.34z"></path>
		</g>
		<g id="villarcayo">
			<path class="st3 fase_0" d="M390.63,56.47c-0.28-2.3-0.93-4.56-1.66-6.74c-0.15-0.45,0.02-0.92,0.34-1.25l0-0.09l0.25-0.15l0.04-0.2
				l-0.14-0.23l-0.3-0.11l0.02-0.05l0.14-0.18l0.3-0.1l0.05-0.44l0.12,0.03l0.15-0.33l0.23-0.1l-0.01-0.14l0.34-0.47l-0.15-0.16
				l-0.07-0.16l-0.66,0.12l-0.13-0.14l0.18-0.08l-0.02-0.14l-0.3,0.12l-0.07,0.14l-0.26-0.19l0.05-0.16l-0.78-0.43l0.18-0.08
				l-0.01-0.2l-0.35-0.68l0.22-0.75l-0.43-0.37l-0.32,0.11l-0.23-0.06l-0.27,0.06c-0.04,0.12-0.09,0.24-0.13,0.37
				c-0.01,0.05-0.02,0.11-0.03,0.16c-0.61,1.11-1.24,2.21-2.12,3.08c-0.88,0.86-1.85,1.62-2.68,2.5c-0.53,0.27-1.07,0.49-1.63,0.68
				c-0.18-0.03-0.35-0.06-0.53-0.08c-0.17-0.09-0.37-0.15-0.59-0.15c-0.29,0-0.58-0.04-0.87-0.07c-0.38-0.03-0.76,0.03-1.13,0.11
				c-0.07,0.02-0.15,0.04-0.21,0.07c-0.15-0.17-0.32-0.31-0.55-0.38l0.01-0.03l-0.15,0.52l-0.19,0.09l-0.23-0.04l-0.11-0.12
				l-0.27,0.05l-0.16,0.57l-0.12,0.16l0.06,0.05l-0.39,0.25l-0.42-0.12l-0.17-0.16l-0.31,0.09l-0.48-0.37l-0.21-0.27l-0.01,0.12
				l-0.1,0.07l0.14,0.18l0.32,0.18l0.25,0.9l-0.4,0.06l-0.25-0.17l-0.06-0.13l-0.25,0.15l-0.15-0.08l-0.16,0.08l0.88,0.23l0.2,0.25
				l-0.33,0.54l-0.31,0.13l-0.09,0.37l0.1,0.1l0.56-0.05l0.11,0.2l0.35-0.03l0.31,0.29l0.26-0.08l0.21-0.2l-0.22-0.35l0.01-0.38
				l0.11-0.22l-0.12-0.04l0.24-0.58l0.17-0.21l0.48,0.34l0.61,0.25l0.46,0.55l0.32-0.14l-0.04-0.08l0.32,0.09l0.91-0.23l-0.04-0.21
				l0.22-0.11l0.26,0.17l0.62,0.15l0.23,0.13l0.63-0.35l1.05,1.05l-0.08-0.25l0.11-0.05l-0.1-0.33l0.21-0.05l0.15-0.28l0.41-0.04
				l0.21-0.06l0.7,0.03l0.16,0.15L383,52.06l0.78,0.34l0.23-0.18l0-0.27l0.16-0.06l-0.01,0.61l-0.18,0.18l0.07,0.49l-0.15,0.83
				l0.12,0.2l0.64,0.5l0.62,0.92l0.04,0.25l-0.23,0.75l0.47,0.36l0.68-0.27l0.43-0.01l0.22,0.16l0.43-0.17l0.14,0.27l0.46,0.16
				l1.58-0.01l0.66,0.15l0.19-0.57l-0.11-0.04l0.15-0.32L390.63,56.47z"></path>
		</g>
		<g id="aguilar_de_campoo">
			<path class="st3 fase_0" d="M375.36,64.37l-0.03-0.5l0.17-0.13l-0.33-0.27l0.01-0.19l-0.12-0.24l-0.35-0.08l-0.54,0.44l-0.12-0.04
				l-0.04,0.1l-0.22-0.15l-0.29-0.4l-0.02-0.35l0.15-0.03l0.02-0.3l-0.18-0.2l-0.07,0.06l0.04-0.38l-0.15,0.04l-0.08,0.15l0-0.13
				l-0.2-0.24l-0.69-0.45l-0.01-0.15l0.18-0.07l0.05-0.54l-0.1-0.44l-0.28-0.38l-0.2-0.11l0.03-0.19l-0.25-0.38l-0.06-0.31
				l0.12-0.38l0.11-0.12l-0.4-0.02l-0.37,0.19l-0.53,0.03l-0.33,0.08l-0.3,0.02l-0.31-0.07l-0.03-0.31l0,0l0.01-0.13l0.01-0.15
				l0.06-0.01l0.06-0.01l0,0l0-0.02l0.15-0.9l-0.06-0.08l0,0l0.04-0.6l0,0v0L369.88,56l-0.16-0.52l0.03-0.77l0,0v0l-0.14-0.39
				l-0.3,0.06l-0.39,0.59l0,0l0,0l0.24,0.62l-0.37,0.08l0,0l-0.36,0.27l0.05,0.41l0.06,0.07l0.08,0.1l-0.06,0.11l-0.15,0.09
				l-0.17-0.23l-0.24,0.11l-0.26,0.48v0l0,0l0.01,0.25l0.01,0.12l0,0v0l0.15,0.34l-0.35,0.19l-0.69,0.13l-0.39-0.05l-0.16-0.58
				l-0.35-0.06l-0.05-0.31l-0.18-0.3l0.05-0.35l-0.59-0.18l-0.39,0l-0.6,0.23l-0.24-0.05c0,0,0,0.01-0.01,0.01c0,0,0,0,0,0
				c0,0.03-0.01,0.06-0.01,0.09c-0.04,0.3-0.12,0.58-0.24,0.86c-0.1,0.22-0.23,0.42-0.32,0.65c-0.1,0.25-0.19,0.5-0.31,0.74
				c-0.21,0.44-0.48,0.8-0.84,1.11L362.17,60l0.31,0.65l0.5,0.34l0.5,0.02l0.29,0.13l0.53,0.07l0.03,0.12l0.51,0.14l0.05,0.07
				l-0.06,0.1l0.32-0.04l-0.02-0.13l0.86,0.36l0.39,0.02l0.16,0.12l0.05,0.19l0.46,0.16l1.2,0.71l0.27-0.03l1.25,0.48l-0.02,0.11
				l0.52,0.21l0.09,0.48l-0.08,0.14l0.05,0.43l0.52-0.07l0.04,0.28l0.79,0l0.29,0.11l0.89,0.12l0.09-0.09l0.54,0.08L374,65.8
				l0.01,0.26l1.17-0.25l-0.05-0.17l0.3-0.33l0.1-0.51l0.23-0.25L375.36,64.37z"></path>
		</g>
		<g id="las_huelgas">
			<path class="st3 fase_0" d="M382.39,80.3c-0.11-0.22-0.21-0.45-0.32-0.67c-0.01,0.01-0.02,0.03-0.04,0.04
				c-0.16,0.15-0.34,0.27-0.54,0.37c-0.2,0.11-0.41,0.2-0.62,0.29c-0.09,0.04-0.19,0.08-0.28,0.13c-0.12,0.05-0.23,0.11-0.35,0.16
				c-0.26,0.12-0.53,0.22-0.8,0.31c-0.01,0-0.02,0.01-0.04,0.01l0,0.02l0.5,0.5l0.18,0.47l0.5-0.01l0.16,0.26l0.51,0.07l0.08,0.04
				c0.01-0.01,0.01-0.02,0.02-0.03c0.06-0.09,0.11-0.18,0.17-0.27c0.05-0.09,0.11-0.18,0.16-0.26c0.09-0.15,0.19-0.29,0.28-0.44
				c0.1-0.16,0.19-0.32,0.27-0.48c0.07-0.15,0.12-0.3,0.15-0.46C382.4,80.32,382.39,80.31,382.39,80.3z"></path>
		</g>
		<g id="los_cubos">
			<path class="st3 fase_0" d="M379.34,79.84l0.31,0.08l0.02,0.07l-0.1,0.07l0.06,0.25l-0.16,0.24l-0.04,0.3c0.01,0,0.03-0.01,0.04-0.01
				c0.27-0.1,0.54-0.2,0.8-0.32c0.11-0.05,0.21-0.1,0.31-0.15c0.1-0.04,0.2-0.09,0.29-0.13c0.21-0.09,0.42-0.19,0.62-0.3
				c0.19-0.1,0.37-0.22,0.51-0.38c0,0,0.04,0.02,0.04,0.02c0,0,0.01,0,0.01,0.01c-0.03-0.07-0.07-0.15-0.1-0.22
				c-0.13-0.27-0.26-0.54-0.39-0.8l-0.07,0.14l-0.43-0.1l0.06,0.19l-0.36,0.01l-0.06-0.29l-0.08-0.02l-0.72,0.46l-0.04-0.05
				l-0.16,0.12l-0.09-0.07l-0.28,0.19l-0.07-0.23l-0.14,0.16l-0.31,0.14l-0.51,0.05l0.29,0.16l-0.52,0.29l0.08,0.16l0.29,0.15
				l0.71,0.11L379.34,79.84z"></path>
		</g>
		<g id="san_agustin">
			<path class="st3 fase_0" d="M383.3,82.11c-0.16-0.3-0.32-0.6-0.47-0.9c-0.12-0.25-0.25-0.5-0.37-0.76c-0.03,0.15-0.09,0.29-0.16,0.43
				c-0.08,0.17-0.18,0.33-0.28,0.48c-0.09,0.15-0.19,0.29-0.28,0.44c-0.1,0.17-0.21,0.34-0.32,0.51l0.38,0.17l-0.03,1.16l0.07,0.02
				l0.52-0.02l0-0.33l-0.07-0.1l0.26-0.34l0.08-0.26l0.23-0.2l0.18-0.04l0.4,0.1l0.03-0.05C383.42,82.32,383.36,82.22,383.3,82.11z"></path>
		</g>
		<g id="jose_luis_santamaria">
			<path class="st3 fase_0" d="M382.38,79.95c0.26-0.08,0.52-0.17,0.78-0.27c0.26-0.1,0.51-0.2,0.77-0.3c0.25-0.1,0.51-0.21,0.76-0.31
				c0.25-0.1,0.51-0.2,0.77-0.3c0.18-0.07,0.36-0.13,0.53-0.19l-0.09-0.07l0.09-0.2l0.55-0.43l-0.05-0.6l-0.47,0.07l-0.63,0.57v0
				l-0.17,0.31l-0.4-0.03l-0.17,0.11l-0.18,0l-0.04-0.06l-0.74-0.06l-0.01-0.16l-0.17-0.25l-0.1-0.36l-0.22,0.01l-0.4,0.25
				l-0.25-0.07l-0.57,0.35l-0.15,0.55l-0.2,0l-0.01,0.03c0,0,0,0,0,0c0.02,0.04,0.04,0.07,0.06,0.11c0.15,0.3,0.3,0.6,0.44,0.91
				c0.07,0.15,0.14,0.29,0.21,0.44C382.33,79.97,382.36,79.96,382.38,79.95z"></path>
		</g>
		<g id="garcia_lorca">
			<path class="st3 fase_0" d="M386.01,79.13l0.5-0.13l-0.47-0.37c-0.2,0.07-0.41,0.14-0.61,0.22c-0.26,0.1-0.51,0.2-0.77,0.3
				c-0.25,0.1-0.51,0.21-0.76,0.31c-0.25,0.1-0.51,0.2-0.77,0.3c-0.26,0.09-0.52,0.18-0.78,0.27c0,0-0.01,0-0.01-0.01
				c-0.01,0-0.02-0.01-0.03-0.01c-0.01,0-0.02-0.01-0.02-0.01c0.05,0.12,0.11,0.23,0.16,0.35c0.01,0,0.02,0.01,0.02,0.01
				c0,0.01,0,0.02,0,0.02c0.01,0.03,0.03,0.06,0.04,0.09c0.14,0.3,0.29,0.61,0.44,0.91c0.15,0.3,0.31,0.6,0.47,0.9
				c0.02,0.04,0.05,0.08,0.07,0.12l0.16-0.26l0.28,0.26l0.17-0.07l0.56-0.58l-0.27-0.2l-0.05-0.63l0.45,0l0.26-0.15l0.09-0.17
				l0.47,0.12l0.23-0.4l0.19,0.15l0.16-0.19l0.07-0.13l-0.32-0.09l-0.16-0.16l-0.09,0.03l-0.03-0.13l0.3-0.27L386.01,79.13z"></path>
		</g>
	</g>
	<g id="palencia">
		<g id="baltanas">
			<path id="path2579" class="st3 fase_0" d="M361.45,98.37l-0.13,0.43l-0.26,0.11l-0.09,0.46l-0.23,0.39l-0.33,0.39l-0.2-0.05
				l-0.45,0.77l-0.07-0.07l-0.46,0.17l-0.82-0.39l-0.07,0.08l-0.24-0.11l-0.07-0.05l0.04-0.08l-0.2-0.09l-0.06,0.09l-0.22-0.2
				l-0.33-0.03l-0.11-0.32l-0.33-0.07l-0.19,0.18l-0.62-0.39l-0.19-0.4l-0.54,0.02l-0.09,0.07l-0.2,0.3l-0.05,0.37l-0.34,0.25
				l-0.17,0.07l-0.07-0.15l-0.38-0.19l-0.38,0.15l-0.42,0.33l-0.36-0.15l-0.49,0.02l-0.07,0.17l0.14,0.13l0.01,0.12l-0.13,0.09
				l0.38,0.18l0.77,0.94l0.71,0.6l0.07,0.06l0.37,0.02l0.07-0.11l0.12,0.13l-0.04,0.14l0.17,0.03l0.1-0.02l0.13-0.21l0.04,0.1
				l0.22-0.07l0.16,0.03l0.12,0.06l-0.08,0.18l0.1,0.27l0.85,0.57l0.16,0.27l0.37,0.35l0.22-0.01l0.01,0.14l0.3-0.18l0.17,0.05
				l0.06,0.44l0.11,0.05l0.17-0.06l0.01,0.15l0.14-0.12l0.06,0.09l0.74,0.09l0.49-0.06l0.05,0.35l0.12,0.18l-0.09,0.97l0.18,0.82
				l0.16,0.16l-0.11,0.13l-0.08,0.32l0.17,0.1l0.24,0.7l0.23-0.01l-0.28,0.13l-0.08,0.15l0.13,0.11l0.05,0.11l0.08,0.2l0.72,0.46
				l0.84-0.24l0.16,0.13l0.18-0.04l0.42-0.02l0.22,0.14l0.27-0.06l0.39,0.27l0.13-0.14l0.61-0.24l0.51-0.22l0.21,0.38l0.29,0.3
				l0.08,0.5l0.37,0.59l0.12-0.02l0.26-0.25l0.2,0.02l0.55,0.46l0.31,0.02l0.11-0.24l0.83,0.34l0.08-0.23l0.04-0.92l-0.2-0.07
				l-0.24-0.63l-0.4-0.11l-1.27-1.4l-0.06-0.44l0,0l0.06,0.44l-0.03-0.42l0.11-0.32l0.35-0.2l-0.07-0.74l-0.14-0.1l-0.18-0.36
				l0.37-0.18h0.28l0.25-0.13l0,0l0.17-0.03l0.15,0.14l0.44,0.03l0.38,0.07l0.12-0.07l0.08-0.54l-0.14-1.38l-0.21-0.74l-0.01-0.63
				l-0.05-0.28l0.13-0.31v-0.24l-0.42-0.51l0.07-0.23l-0.15,0.06l-0.59-0.63l-0.08-0.19l-0.3,0.1l-0.49-0.1l-0.07,0.11l-0.28,0.04
				l-0.3,0.49l-1.01-0.05l0.15-0.34l-0.29-0.48l-0.07-0.65l0.06-0.25l-0.37,0.07l-0.38-0.31l-0.21,0.05l-0.11-0.4l-0.26-0.06
				l-0.66,0.23L361.45,98.37z M373.39,99.17l-0.19,0.27l-0.35,0.08l-1.11,0.6l-0.22-0.08l-0.66,0.65l-0.03,0.13l-0.21,0.1
				l-0.18-0.03l-0.32,0.3l-1.17,0.07l0.19,1.36l-0.06,0.42l0.22,0.62l0.16,0.23l0.35-0.21l0.62,0.01l-0.03-0.73l0.1-0.2l1.18-0.25
				l0.2-0.22l0.11-0.27l-0.06-0.1l0.38-0.57l0.56-0.19l0.33,0.11l0.11,0.13l0.8-0.08l0.06-0.16l-0.12-0.6l0.16-0.79l-0.06-0.54
				L373.39,99.17z"></path>
		</g>
		<g id="torquemada" class="active">
			<path id="path2475" class="st3 fase_1" d="M361.64,91.91l-0.66,0.53l-0.12-0.01l-0.83,0.43l-0.34-0.06l-0.02-0.2l-0.13-0.14
				l-0.61,0.2l-0.12-0.08l-0.4,0.11l-0.46,0.65l-0.13,0.51l-0.44,0.06l-0.59-0.12l-0.15,0.03l-0.29,0.25l-0.18-0.14l-0.12-0.35
				l-0.54-0.03l-0.49,0.15l-0.34-0.51l-0.23-0.18l-0.26,0.24l0.35,0.27l0.19,0.25l0.25,0.15l-0.07,0.26l-0.41,0.07l-0.5-0.1
				l0.02,0.18l-0.14,0.28l-0.06,0.27l-0.04,0.76l0.14,0.18l0.04,0.32l0.16,0.17l-0.09,0.52l-0.19,0.28l0,0.12l-0.83,0.4l-0.84,0.22
				l0.04,0.28l-0.38,0.03l0.08,0.16l0.18,0.03l0.02,0.08l1.25,0.9l0.12,0.16l0.13,0.59l0.38-0.15l0.38,0.19l0.07,0.15l0.17-0.07
				l0.34-0.25l0.05-0.37l0.2-0.3l0.09-0.07l0.54-0.02l0.19,0.4l0.62,0.39l0.19-0.18l0.33,0.07l0.11,0.32l0.33,0.03l0.22,0.2
				l0.06-0.09l0.2,0.09l-0.04,0.08l0.07,0.05l0.24,0.11l0.07-0.08l0.82,0.39l0.46-0.17l0.07,0.07l0.45-0.77l0.2,0.05l0.33-0.39
				l0.23-0.39l0.09-0.46l0.26-0.11l0.13-0.43l0.9,0.23l0.66-0.23l0.26,0.06l0.11,0.4l0.21-0.05l0.38,0.31l0.37-0.07l-0.06,0.25
				l0.07,0.65l0.29,0.48l-0.15,0.34l1.01,0.05l0.3-0.49l0.28-0.04l0.07-0.11l0.49,0.1l0.3-0.1l0.08,0.19l0.59,0.63l0.15-0.06
				l-0.07,0.23l0.42,0.51v0.24l-0.13,0.31l0.05,0.28l0.01,0.63l0.21,0.74l0.14,1.38l-0.08,0.54l0.63-0.25l0.5-0.36l0.36-0.39
				l0.71-0.44l-0.1-0.68l-0.62-0.01l-0.35,0.21l-0.16-0.23l-0.22-0.62l0.06-0.42l-0.19-1.36l-0.19,0.23l-0.35,0l0.08-0.8l0.14-0.23
				l-0.07-0.18l0.21-0.29l0.37-0.9l0.81-0.37l0.38-0.97l0.08-0.01l0.13,0.15l0.36-0.65l-0.21-0.42l-0.16-0.11l-0.09-0.45l-0.25,0.25
				l-0.31,0.09l-0.13-0.21l-0.46-0.14l-0.84,0.11l-0.31-0.21l-0.27,0.14l-0.08,0.23l0,0.4l-0.37-0.04l-0.27-0.21l-0.66,0.37
				l-0.18-0.11l-0.32-0.57l-0.23-0.16l-0.08-0.29l-0.17-0.2l0.04-1.37l0.34,0.01l0.69,0.22l0.44-0.18l0.01-0.17l0.23-0.12l0.17-0.52
				l-0.28-0.01l-0.54-0.23l-0.38,0.12l-0.28-0.25l-0.46-0.1l-0.15-0.23l-0.03-0.27l-0.28-0.31l-0.23-0.19l-0.24,0.32l-0.22-0.06
				l-0.22,0.2l-0.17,0l0.02-0.21l-0.11-0.31l-0.4-0.12l-0.57,0.11l-0.35,0.2l-0.06-0.26l-0.35,0.23L361.64,91.91L361.64,91.91z"></path>
		</g>
		<g id="guardo">
			<path id="path2235" class="st3 fase_0" d="M339.91,44.26l-0.8,0.15l-0.4,0.75l-0.57,0.19l-0.07,0.24l-0.22,0.05l-0.27,0.32
				l-0.03,0.69l-0.47,0.33l0.08,0.32l-0.05,0.28l0.16,0.38l-0.1,0.41l-0.23,0.31l0.07,0.3l-0.16,0.16l-0.03,0.18l-0.24,0.09
				l-0.35,0.29l-0.08,0.27l-0.3,0.28l0.07,0.4l-0.09,0.21l-0.31-0.12l-0.38,0.31l-0.16,0.43l0.04,0.3l-0.09,0.38l-0.2,0.56
				l-0.12-0.12l-0.49-0.17l-0.72,0.15l-0.11,0.31l0.01,0.31l0.5,0.56l-0.14,0.48l-0.16,0.28l-0.04,0.53l-0.24,0.43l0.16,0.22
				l-0.01,0.16l0.1,0.13l0.03,0.78l-0.06,0.08l-0.17,0.03l-0.05-0.08l-0.26,0.27l-0.02,0.28l-0.44,0.9l-0.18-0.1l-0.14,0.21
				l-0.28-0.05l0.08,0.17l-0.12,0.48l0.32,0.41l0.4,0.14l0.1,0.73l0.2,0.1l0.56-0.3l0.24,0.05l0.26,0.28l0.04,0.3l0.49,0.25
				l-0.34,0.21l-0.25,0.46l-0.18,0.14l-0.4,1.27l0.17,0.73l1.38,0.01l0.07-0.44l0.86-0.02l0.48-0.14l0.64,0.03l0.55-0.31l0.49-0.05
				l0.67-0.5l-0.11-0.49l0.18-0.39l0.18-0.37l0.4-0.28l0.3,0.09l0.18,0.19l0.51-0.03l0.19-0.96l0.42-0.5l0.61-0.47l0.11,0.22
				l0.27,0.28l0.27,0.04l0.14,0.54l0.18,0.06l0.48,0.79l0.53-0.78l0.21-0.11l0.11,0.04l0.06-0.07l-0.06-0.13l0.63-0.41l0.29-0.38
				l0.48-0.11l0.28,0.07l0.24-0.05l-0.04-0.16h-0.72l-0.33-0.41l-0.82-0.1l0.01-0.11l-0.44-0.49l-0.06-0.05l-0.15,0.07l-0.52-0.47
				l-0.01-0.26l0.01-0.7l0.04-0.08l0.32-0.07l-0.11-0.41l0.12,0.02l0.25,0.25l0-0.27l0.13-0.34l-0.14-0.58l0.24-0.09l-0.01-0.11
				l0.14-0.12l-0.07-0.46l0.06-0.21l-0.13-0.15l0.14-0.84l-0.16-0.51l-0.2,0.4l-0.87-0.26l-0.58-0.06l-0.13-0.13l-0.25-0.46
				l0.01-0.12l0.13-0.14l-0.02-0.13l0.18-0.07l0.09-0.16l0-0.58l0.16-0.13l-0.15-0.2l0-0.32l-0.12-0.03l-0.16-0.23l-0.08-0.19
				l0.01-0.29l-0.19-0.19l-0.51-0.21l-0.07,0.07l-0.18-0.09l-0.62-0.49l0.32-0.43l0.07-0.22l-0.23-0.27l0.16-0.52l-0.15-0.09
				l0.16-0.26l0.38-0.16l0.07-0.16l0.12,0l-0.03-0.15l0.11-0.24l-0.08-0.11l-0.01-0.53l-0.91-0.93L339.91,44.26L339.91,44.26z"></path>
		</g>
		<g id="venta_de_banos">
			<path id="path2615" class="st3 fase_0" d="M351.08,100.98l-0.08,0.05l0.32,0.42l-0.04,0.19l0.32,0.04l0.13,0.1l0.14,0.05l0.01-0.12
				l-0.05-0.13l0.04-0.26L351.08,100.98L351.08,100.98z M351.06,101.83l-0.27,0.26l-0.06-0.09l-0.79,0.27l0.09,0.12l-0.34,0.34
				l-0.3,0.75l-0.1-0.55l-0.21,0.08l-0.23,0.29l-0.3,0.1l-0.09,0.15l-0.32-0.15l-0.15,0.08l-0.4-0.35l0.06-0.13l-0.16,0.01
				l-0.08-0.13l-0.71,0.08l-0.13,0.17l-0.07-0.07l0.03-0.11l-0.1-0.05l-0.64,0.08l-0.13,0.1l-0.45-0.13l-0.21-0.26l-0.59,0.5
				l-0.3,0.14l-0.17-0.03l0,0.32l-0.23,0.74l-0.06,0.06l-0.66-0.18l0.1,0.31l0.01,0.12l0.34,0.19l0.24-0.01l0.27,0.17l0.1,0.24
				l-0.06,0.24l0.11,0.18l-0.05,0.25l0.12,0.15l0.1,0l-0.13,0.27l0.26,0.26l0.1,0.23l0.24,0.15l0.58,0.68l0.52,0.1l0.36,0.77
				l0.65,0.53l-0.05-0.2l0.11-0.11l0.39,0.11l0.16-0.02l0.12-0.12l0.1-0.38l-0.19-0.15l0.04-0.12l0.8,0.17l0.34,0.25l0.36-0.01
				l0.12-0.16l0.14-0.03l0.01-0.17l-0.13-0.12l0.37-0.05l0.31-0.28l0.51,0.01l0.04-0.28l0.11,0.06l0.09-0.04l0.06,0.13l-0.06,0.46
				l-0.2,0.19l0.7,0.44l-0.19,0.06l-0.11-0.07l-0.43,0.13l-0.18-0.11l-0.15,0.01l-0.26,2.14l0.05,0.09l-0.36,0.39l0.67,0.1
				l1.23,0.56l0.3,0.05l0.19-0.02l0.84-0.51l0.71-0.81l0.36-0.11l0.13-0.11l0.46,0.04l-0.09,0.15l0.05,0.07l1.17,0.46l0.59,0.12
				l0.16-0.03l0.18,0.18l0.48,0.29l0.02-0.27l0.15-0.05l0.27-0.35l0.82,0.06l0.32-0.74l0.3-0.19l0.45-0.71l0.16-0.06l0.36,0.19
				l0.51-0.48L360.2,109l-0.13-0.11l0.08-0.15l0.28-0.13l-0.23,0.01l-0.24-0.7l-0.17-0.1l0.08-0.32l0.11-0.13l-0.16-0.16l-0.18-0.82
				l0.09-0.97l-0.12-0.18l-0.05-0.35l-0.49,0.06l-0.74-0.09l-0.06-0.09l-0.14,0.12l-0.01-0.15l-0.17,0.06l-0.11-0.05l-0.06-0.44
				l-0.17-0.05l-0.3,0.18l-0.01-0.14l-0.22,0.01l-0.37-0.35l-0.16-0.27l-0.85-0.57l-0.1-0.27l0.08-0.18l-0.12-0.06l-0.16-0.03
				l-0.22,0.07l-0.04-0.1l-0.13,0.21l-0.1,0.02l-0.17-0.03l-0.14-0.06l-0.32,0.05l-0.08-0.02l0.02-0.15l-0.07-0.06l-0.15,0.03
				l-0.18,0.19l0.01-0.23l-0.1-0.07l-0.34,0.22l-0.09-0.05l-0.32,0.19l-0.18-0.11l-0.16,0.09l-0.02-0.25l-0.09,0.21l-0.15,0.07
				l-0.26-0.19l-0.13-0.32l-0.09-0.06l-0.23,0.04l-0.27,0.19l-0.05-0.13l-0.25,0.04l-0.11-0.27L351.06,101.83L351.06,101.83z"></path>
		</g>
		<g id="villarramiel" class="active">
			<path id="path2461" class="st3 fase_0" d="M336.85,88.45l-1.02,0.66l-0.66,0.3l-0.42,0.32l-0.28,0.05l-0.16,0.22l-0.26,0.17
				l-0.12-0.12l-0.53-0.14l-0.37-0.2l-0.41,0.25l-0.29,0.48l-0.13,0.49l-0.09,0.3l0.03,0.76l0.16,0.53l-0.15,0.13l0.18,0.42
				l0.16,0.76l0.33,0.28l-0.23,0.03l0.55,0.67l-0.51,0l-0.44,0.26l-0.89,1.21l0.47,1.67l0.09,0.11l-0.61,0.31l-0.59,0.27l-0.19,0.46
				l-0.21,0.27l0.16,0.24l-0.12,0.62l-0.07,0.2l-0.28,0.2l-0.01,0.17l-0.44,0.22l-0.05,0.25l-0.29,0.54l-0.11,0.04l0.15,0.22
				l-0.14,0.28l0.3,0.49l0.3,0.18l0.12,0.2l0.01,0.45l0.13,0.18l0.02-0.1l0.27-0.09l-0.04-0.27l0.22-0.05l0.2-0.28l0.4-0.19
				l0.07-0.15l0.24-0.14l0.69,0.39l0.18,0.41l-0.08,0.05l0.02,0.09l0.07,0.07l0.46-0.15l0.03,0.09l0.16,0.03l0.48-0.68l0.46-0.28
				l0.58,0.02l0.2,0.07l0.06,0.76l0.19-0.04l0.16,0.17l0.13-0.03l-0.01,0.13l0.72,0.79l0.24,0.63l0,0.34l0.12,0.29l0.26,0.41
				l0.37,0.24l0.59,1.55l0.17,0.29l0.06,0.36l0.17,0.22l0.78-0.27l0.03-0.15l0.44,0l0.16-0.42l0.26-0.38l0.88-0.75l-0.24-0.39
				l0.16-0.38l0.18-0.12l0.2-0.13l0.72-0.74l0.61-0.2l0.33-0.38l-0.02-0.1l-0.4-1.06v-0.5l-0.17-0.53l0.35-0.17l0.26-0.39l0.39-0.07
				l-0.1-0.8l-0.06-0.15l-0.1-0.04l0.03-0.23l-0.11-0.19l0.23-0.49l0.22-0.11l-0.07-0.22l0.25-0.19l0.3-0.51l0.09,0.07l0.15-0.09
				l0.54,0.03l0.62-0.74l-0.42-0.25l-0.35-0.39l-0.33,0.19l-0.14-0.01l-0.3-0.31l0.07-0.15l-0.35-0.32l0.28-0.44l-0.51-0.21
				l-0.15,0.14l-0.26-0.24l-0.1,0.11l-0.1-0.08l-0.32,0.26l-0.28-0.65l-0.17-0.1l-0.28-0.4l-0.06-0.27l-0.42,0.34l-0.11,0.33
				l-0.14,0.05l-0.13,0.21l-0.09,0.17l-0.26,0.19l-0.26,0.09l-0.02,0.07l-0.09-0.02l-0.01,0.1l-0.12,0.03l-0.12,0.09l-0.09-0.05
				l-0.04,0.07l-0.34,0.06l-0.26-0.12l-0.46-0.14l-0.19-0.18l-0.3-0.05l-0.28-0.46l-0.06-0.18l-0.31-0.32l0.12-0.11l0.04-0.23
				l0.05-1.24l-0.06-0.43l0.12-0.1l0.1-0.45l-0.03-0.21l-0.2-0.34l-0.01-0.2l0.58-0.65l0.48-0.09l0.26,0.09l0.1-0.17l-0.5-0.28
				l-0.19,0.01l-0.28-0.37l-0.39-0.91l-0.1-0.46l-0.25-0.36L336.85,88.45L336.85,88.45z"></path>
		</g>
		<g id="carrion_de_los_condes">
			<path id="path2339" class="st3 fase_0" d="M345.55,70.35l-0.1,0.26l0.21,0.23l-0.22,0.61l-0.22,0.15l0.13,0.33l-0.07,0.13l-0.41-0.01
				l-0.19-0.39l-0.11-0.02L344.35,72l0.16,0.07l-0.12,0.14l-0.02,0.22l0.08,0.29l0.18,0.27l0.13,0.42l0.26,0.31l-1.23,0.37
				l0.16,0.26l0.09,0.45l0.09,0.01l0.23,0.25l0.04,0.39l-0.26,0.8l-0.71-0.44l-0.28-0.05l-0.29-0.22l-0.49,0.22l-0.06-0.12
				l-0.09,0.36l-0.21,0.27l0.17,0.09l0.08,0.38l-0.52-0.21l-0.48,0.11l-0.18,0.07l-0.37,0.47l-0.24,0.12l-0.54,0.28l-0.47,0.35
				l-0.43,0.77l-0.41,1.05l-0.53,0.32l-0.38-0.02l-0.25-0.58l-0.75,0.23l-0.44,0.03l0.02,0.11l-0.18,0.11v0.15l-0.09,0.09
				l-0.02,0.26l-0.43,0.17l-0.06,0.13l0.68,0.55l-0.15,0.04l-0.09,0.15l0.07,0.54l0.15,0.15l-0.01,0.27l0.19,0.26l0.1,0.01
				l0.34,0.58l0.05,0.43l-0.08,0.39l0.22-0.09l0.36,0.01l0.12,0.2l0,0.16l0.45,0.39l0.61,0.17l0.48,0.25l0.29-0.23l0.02-0.21
				l0.32-0.36l-0.08-0.1l0.2-0.45l-0.06-0.13l0.28-0.24l0-0.09l0.18-0.22l0.64-0.43l0.1-0.19l0.4-0.27l0.48,0.23l0.17,0.56l0.39,0.3
				l0.44,0.59l0.25,0.13l0.36,0.23l-0.06,0.05l0.08,0.18l-0.14,0.32l0.28,0.16l0.06,0.25l-0.06,0.51l0.28,0.47l0.03,0.31l0.75-0.26
				l0.37-0.31l0.26,0.31l-0.05,0.34l0.22,0.47l0.07,0.08l0.29,0.05l0.05,0.38l-1.09,0.4l-0.28,0.39l0.05,0.27l-0.22,0.79l0.33-0.23
				l0.06,0.49l0.13-0.02l0.3,0.29l0.03,0.08l-0.24,0.5l0.06,0.08l0.23,0.22l0.24,0.08l0.67-0.07l-0.05,0.13l0.28-0.07l0.12,0.08
				l0.22-0.18l0.49-0.49l-0.06-0.19l0.12-0.04l-0.08-0.48l-0.09-0.03l-0.17-0.57l0.07-0.17l-0.08-0.09l0.57-0.33l0.02-0.61
				l0.09-0.26l-0.05-0.13l0.34-0.39l-0.06-0.35l-0.19-0.38l0.31-0.25l0.14,0.06l0.08-0.2l-0.24-0.05l-0.06-0.15l0.14-0.65
				l-0.04-0.36l0.1-0.04l0.15,0.07l0.28-0.26l-0.41-1.14l0.04-0.23l0.15-0.18l0.79-0.48l0.34-0.41l0.78-0.1l0.56-0.59l0.19-0.08
				l0.28,0.03l0.4-0.69l-0.71-0.72l-0.38-0.03l-0.27-0.14l-0.77-0.09l-0.11-0.18l-0.45,0.08l-0.13-0.5l-0.25-0.16l-0.16-0.21
				l0.07-0.07l-0.04-0.15l-0.52-1.15l-0.02-0.19l0.47-0.75l-0.02-0.26l0.01-0.26l0.17-0.26l0.27-0.88l-0.12-0.47l-0.15-0.34
				l0.01-0.16l-0.28-0.45l-0.2,0.12l-0.27-0.19l-0.43-0.14l-0.03-0.49l-0.04-0.09l-0.33-0.11l-0.08-0.5l-0.33-0.35l-0.03-0.17
				l-0.29-0.36l-0.18-0.15l-0.08,0.01L345.55,70.35L345.55,70.35z M346.15,89.72l-0.06,0.14l-0.34,0.21l-0.21,0.13l-0.06-0.23
				L346.15,89.72z"></path>
		</g>
		<g id="herrera_de_pisuerga">
			<path id="path2271" class="st3 fase_0" d="M348.99,67.06l0.27,0.44l-0.09,0.35l0.01,0.69l0.13,0.05l0.28,0.35l-0.12,0.51l0.2,0.78
				l-0.04,0.41l0.23,0.3l-0.01,0.15l0.17,0.09l0.08,0.17l0.35,0.12l-0.14,0.64l0.09,0.01l0.22,0.71l0.16,0.09l0.32-0.09l0.46,0.06
				l0.19,0.51l0.43,0.4l0.43-0.9l-0.22-0.53l-0.05-0.23l0.13-0.11l-0.12-0.01l-0.06-0.47l-0.43-0.55l-0.11-0.68l0.11-0.39l0.27-0.22
				l0.27,0.06l0.26-0.14l0.2-0.2l-0.01-0.13l0.22-0.12l0.28,0.19l0.33-0.09l0.42,0.2l0.41,0l0.32,0.02l0.27-0.08l0.3-0.58l0.25-0.02
				l0.31,0.01l0.06,0.18l0.43,0.32l-0.33,0.35l0.04,0.92l-0.13,0.16l0.19,0.1l0.1,0.12l0.21-0.01l0.14,0.2l0.18-0.01l0.21,0.34
				l0.09-0.1l0.2-0.03l0.16,0.06l0.34-0.1l0.13,0.06l-0.11,0.4l-0.34,0.19l-0.08,0.55l-0.23,0.44l-0.52,0.25l-0.26,0l0.08,0.2
				l-0.15,0.16l0.12,0.57l0.4,0.17l0.41,0.3l0.01-0.09l0.89,0.09l0.1-0.09l0.26,0l0.29,0.09l0.7-0.17l0.36,0.02l-0.05-0.14
				l-0.11,0.01l-0.03-0.08l0.06-0.14l-0.08-0.23l-0.19,0.06l-0.09-0.13l0.16-0.16l0.22-0.02l-0.23-0.41l0.18-0.7l-0.04-0.12
				l-0.21-0.46l-0.42-0.46l-0.42,0.08l-0.53-0.08l0.39-0.14l0.18-0.29l-0.13-0.47l-0.17-0.27l0.09-0.09l-0.06-0.39l0.05-0.1
				l-0.89-0.42l-0.14-0.15l0.01-0.2l0.22-0.07l-0.31-0.56l0.27,0.01l0.2-0.2l0.45-0.09l0.23-0.21l0.24-0.09l-0.17-0.09l-0.37,0.1
				l-0.09-0.09l0.16-0.19l-0.15-0.14l0.03-0.11l-0.11-0.1l-0.03-0.28l-0.23-0.24l0.08-0.16l0.41-0.2l0.35-0.39l0.41-0.05l-0.27-0.29
				l0.12-0.46l-0.13-0.21l-0.15-0.08l-0.11-0.33l0.16-0.11l-0.09-0.44l0.15-0.14l-0.16-0.57l-0.01-0.33l-0.36-0.12l-0.21-0.21
				l0-0.23l0.16,0l-0.03-0.15l-0.26-0.24l0.37-0.26l0.12-0.21l0.48-0.28l-0.43-0.26l0.09-0.9l-0.33-0.21l-0.3,0.03l-0.19-0.11
				l-0.36,0.15l-0.75-0.57l0.2-0.19l-0.45-0.28l-0.2-0.01l-0.75,0.36l-0.53,0.11l-0.2-0.15L354.34,59l-0.13,0.22l-0.26,0.08
				l-0.31,0.3l-0.41-0.06l-0.26-0.21l-0.62-0.11l0.2-0.61l0.1-0.44l-0.25-0.63l-0.67-0.75l-0.6-0.19l-0.09,0.16L350.41,56l0.04-0.71
				l-0.48,0.16l-0.47,0.57l-0.11,0.47l-0.31,0.37l-0.12,0.37l-0.14,0.04l-0.02,0.17l-0.28,0.38l-0.12,0.02l-0.48,0.15l-0.42,0.61
				l-0.32,0.14l-0.17,0.62l-0.24,0.57l0.24,0.12l0.85-0.09l-0.02,0.61l-0.62,0.38l0.27,1.18l-0.37-0.13l-0.02,0.15l0.08,0.05
				l-0.09,0.1l-0.07,0.3l0.16,0.25l0.09,0.45l-0.09,0.49l0.1,0.18l0.13,0.51l0.12,0.24l0.15,0.1l0.08,0.29l-0.13,0.26l-0.1,0.03
				l0.33,0.57l0.16,0.06l0.19,0.23l0.51,0.57 M348.69,66.85l-0.29,0.35l0.04,0.11l0.09,0.37l0.16-0.13l0.3-0.5"></path>
		</g>
		<g id="saldana">
			<path id="path2535" class="st3 fase_0" d="M345.2,59.11l-0.48,0.11l-0.29,0.38l-0.63,0.41l0.06,0.13l-0.06,0.07l-0.11-0.04
				l-0.21,0.11l-0.53,0.78l-0.48-0.79l-0.18-0.06l-0.14-0.54l-0.27-0.04l-0.27-0.28l-0.11-0.22l-0.61,0.47l-0.42,0.5l-0.19,0.96
				l-0.51,0.03l-0.18-0.19l-0.3-0.09l-0.4,0.28l-0.18,0.37l0.01,0.01l-0.18,0.38l0.11,0.49l-0.01,0l-0.67,0.5l-0.49,0.05l-0.55,0.31
				l-0.64-0.03l-0.48,0.14l-0.86,0.02l-0.07,0.44l-1.38-0.01l-0.04,0.63l-0.05,0.45l0.15,0.63l-0.41,1.11v0.22l0.15,0.24l0,0.55
				l-0.05,0.35l-0.51,0.95l-0.02,0.56l0.05,0.28l0.19,0.19l0.89,0.22l-0.13,1.01l0.04,0.49l0.1,0.01l-0.19,1.22l-0.3,1.91l0.73-0.6
				l0.09,0.16l0.44,1.11l0.25,0.33l0.1,0.39l0.77,0.31l-0.21,0.79l0.11,0.04l0.12,0.88l0.22,0.35l0.04-0.1l1.01-0.41l0.32,0.48
				l0.03,0.52l0.16,0.43l0.25,0.58l0.38,0.02l0.53-0.32l0.41-1.05l0.43-0.77l0.47-0.35l0.54-0.28l0.24-0.12l0.37-0.47l0.18-0.07
				l0.48-0.11l0.52,0.21l-0.08-0.38l-0.17-0.09l0.21-0.27l0.09-0.36l0.06,0.12l0.49-0.22l0.29,0.22l0.28,0.05l0.71,0.44l0.26-0.8
				l-0.04-0.39l-0.23-0.25l-0.09-0.01l-0.09-0.45l-0.16-0.26l1.23-0.37l-0.26-0.31l-0.13-0.42l-0.18-0.27l-0.08-0.29l0.02-0.22
				l0.12-0.14l-0.16-0.07l0.22-0.36l0.11,0.02l0.19,0.39l0.41,0.01l0.07-0.13l-0.13-0.33l0.22-0.15l0.22-0.61l-0.21-0.23l0.1-0.26
				l0.41,0.54l0.08-0.01l0.18,0.15l0.29,0.36l0.03,0.17l0.33,0.35l0.08,0.5l0.33,0.11l0.04,0.09l0.03,0.49l0.43,0.14l0.27,0.19
				l0.2-0.12l0.29-0.35l0.41-0.28l0.06-0.25l0.23,0.07l0.22-0.08l0.06-0.34l0.16-0.19l-0.04-0.48l0.22-0.33l0.01-0.15l-0.23-0.3
				l0.04-0.41l-0.2-0.78l0.12-0.51l-0.28-0.35l-0.13-0.05l-0.01-0.69l0.09-0.35l-0.27-0.44l-0.3,0.5l-0.16,0.13l-0.09-0.37
				l-0.04-0.11l0.29-0.35l-0.51-0.57l-0.19-0.23l-0.16-0.06l-0.33-0.57l0.1-0.03l0.13-0.26l-0.08-0.29l-0.15-0.1l-0.12-0.24
				l-0.13-0.51l-0.1-0.18l0.09-0.49l-0.09-0.45L347,62.61l0.07-0.3l0.09-0.1l-0.08-0.05l0.02-0.15l0.37,0.13l-0.27-1.18l0.62-0.38
				l0.02-0.61l-0.85,0.09l-0.24-0.12l0.24-0.57l-0.33,0.25l-0.4-0.07l-0.16,0.05l-0.29-0.23l-0.1-0.24l-0.24,0.05L345.2,59.11z"></path>
		</g>
		<g id="cervera_pisuerga" class="active">
			<path id="path2243" class="st3 fase_0" d="M351.09,41.98l-0.08,0.03l-0.33,0.45l-0.22,0.56l0.03,0.14l-0.74-0.05l-0.08,0.07
				l-1.01-0.48l-1.52,1.5l-0.64-0.4l-0.66,0.06l-0.63-0.53l-0.23,0.28l-0.32,0.22l-0.4,0.14l-0.35,0.39l-0.09-0.03l-0.45-0.2
				l-0.88-0.03l-0.45-0.25l-0.24-0.02l-0.37-0.01l-0.38,0.09l-0.23,0.29l-0.21,0.13l-0.36-0.08l-0.1-0.19l-0.24,0.07l-0.01,0.14
				l-0.01,0.16l0.91,0.93l0.01,0.53l0.08,0.11l-0.11,0.24l0.03,0.15l-0.12,0l-0.07,0.16l-0.38,0.16l-0.16,0.26l0.15,0.09l-0.16,0.52
				l0.23,0.27l-0.07,0.22l-0.32,0.43l0.62,0.49l0.18,0.09l0.07-0.07l0.51,0.21l0.19,0.19l-0.01,0.29l0.08,0.19l0.16,0.23l0.12,0.03
				l0,0.32l0.15,0.2l-0.16,0.13l0,0.58l-0.09,0.16l-0.18,0.07l0.02,0.13l-0.13,0.14l-0.01,0.12l0.25,0.46l0.13,0.13l0.58,0.06
				l0.87,0.26l0.2-0.4l0.16,0.51l-0.14,0.84l0.13,0.15l-0.06,0.21l0.07,0.46l-0.14,0.12l0.01,0.11l-0.24,0.09l0.14,0.58l-0.13,0.34
				l0,0.27l-0.25-0.25l-0.12-0.02l0.11,0.41l-0.32,0.07l-0.04,0.08l-0.01,0.7l0.01,0.26l0.52,0.47l0.15-0.07l0.06,0.05l0.44,0.49
				l-0.01,0.11l0.82,0.1l0.33,0.41h0.72l0.04,0.16l0.1,0.24l0.29,0.23l0.16-0.05l0.4,0.07l0.33-0.25l0.17-0.62l0.32-0.14l0.42-0.61
				l0.48-0.15l0.12-0.02l0.28-0.38l0.02-0.17l0.14-0.04l0.12-0.37l0.31-0.37l0.11-0.47l0.47-0.57l0.48-0.16L350.41,56l0.63,0.77
				l0.09-0.16l0.6,0.19l0.67,0.75l0.25,0.63l-0.1,0.44l1.09-1.16l0.19-0.55l0.52-0.34l0.52-0.03l-2.33-1.73l0.7,0.03l0.49-0.59
				l0.62,0.42l0.61-0.36l0.29,0.14l0.04,0.35l0.44-0.1l0.47,0.07l0.14-0.27l0.15,0.06l-0.18-0.38l-0.14-0.05l-0.11-0.19l-0.32,0.09
				l0.06-0.34l-0.28-0.25l-0.01-0.11l-0.41-0.37l-0.11-0.25l-0.16-0.15l-0.31-0.11l0.01-0.37l0.48,0.08l0.43-0.19l-0.2-0.33
				l-0.05-0.29l0.44-0.23l-0.09-0.13l0.14-0.05l0.1,0.12l0.05-0.08l0.39,0.21l-0.03-0.33l0.54-0.72l-0.07-0.36l-0.17-0.17
				l-0.06-0.19l-0.04-0.31l-0.49-0.05l-0.75-0.28l-0.16-0.23l0.07-0.3l-0.13-0.32l-0.44-0.45l0.04-0.14l0.38-0.32l0.07-0.56
				l-0.07-0.16l0.18-0.23l-0.17-0.33l-0.21-0.19l-0.11-0.27l0.02-0.35l-0.48-0.56l-0.12-0.5l-0.13-0.09l-0.24-0.4l-0.24-0.18
				l-0.4-0.21l-0.34-0.04l-0.38-0.16l-0.43-0.55l-0.21-0.14l-0.37,0.08L351.09,41.98L351.09,41.98z"></path>
		</g>
		<g id="paredes_de_nava" class="active">
			<path id="path2239" class="st3 fase_0" d="M341.33,82.29l-0.4,0.27l-0.1,0.19l-0.64,0.43l-0.18,0.22l0,0.09l-0.28,0.24l0.06,0.13
				l-0.2,0.45l0.08,0.1l-0.32,0.36l-0.02,0.21l-0.29,0.23l-0.48-0.25l-0.61-0.17l-0.45-0.39l0-0.16l-0.12-0.2l-0.36-0.01l-0.22,0.09
				l-0.21,0.51l-0.18,0.88l0.08,0.93l0.16,0.58l-0.04,0.34l0.19,0.09l-0.11,0.53l0.17,0.16l0,0.31l-0.16,0.2l0.25,0.36l0.1,0.46
				l0.39,0.91l0.28,0.37l0.19-0.01l0.5,0.28l-0.1,0.17l-0.26-0.09l-0.48,0.09l-0.58,0.65l0.01,0.2l0.2,0.34l0.03,0.21l-0.1,0.45
				l-0.12,0.1l0.06,0.43l-0.05,1.24l-0.04,0.23l-0.12,0.11l0.31,0.32l0.06,0.18l0.28,0.46l0.3,0.05l0.19,0.18l0.46,0.14l0.26,0.12
				l0.34-0.06l0.04-0.07l0.09,0.05l0.12-0.09l0.12-0.03l0.01-0.1l0.09,0.02l0.02-0.07l0.26-0.09l0.26-0.19l0.09-0.17l0.13-0.21
				l0.14-0.05l0.11-0.33l0.41-0.34l0.06,0.27l0.28,0.4l0.17,0.1l0.28,0.65l0.32-0.26l0.1,0.08l0.1-0.11l-0.08-0.04l0.04-0.11l0.15,0
				l0.13,0l0.29-0.37l0.29,0.17l0.66-0.69l0.33-0.92l0.27-0.25l0.48-0.76l0.34-0.04l0.23-0.23l0.16,0.02l0.16,0.22l1.11,0.02
				l0.14-0.12l0.26,0.01l0.14-0.39l0.36-0.58l-0.14-0.31l0.35-1.09l-0.23-0.23l-0.09-0.07l-0.92,0.59l-0.22,0.18l-0.12-0.08
				l-0.28,0.07l0.05-0.13l-0.67,0.07l-0.24-0.08l-0.23-0.22l-0.06-0.08l0.24-0.5l-0.03-0.08l-0.3-0.29l-0.13,0.02l-0.06-0.49
				l-0.33,0.23l0.22-0.79l-0.05-0.27l0.28-0.39l1.09-0.4l-0.05-0.38l-0.29-0.05l-0.07-0.08l-0.22-0.47l0.05-0.34l-0.26-0.31
				l-0.37,0.31l-0.75,0.26l-0.03-0.31l-0.28-0.47l0.06-0.51l-0.06-0.25l-0.28-0.16l0.14-0.32l-0.08-0.18l0.06-0.05l-0.36-0.23
				l-0.25-0.13l-0.44-0.59l-0.39-0.3l-0.17-0.56L341.33,82.29L341.33,82.29z M346.15,89.72l-0.68,0.25l0.06,0.23l0.21-0.13
				l0.34-0.21L346.15,89.72z"></path>
		</g>
		<g id="aguilar_de_campoo">
			<path class="st3 fase_0" d="M362.02,59.77l0.06-0.01c0.04-0.13,0.1-0.38,0.21-0.46c0.4-0.29,0.74-0.64,0.93-1.1
				c0.12-0.3,0.25-0.6,0.39-0.89c0-0.01,0-0.02,0-0.02c0.01-0.05,0.01-0.11,0.02-0.16c0.02-0.16,0.06-0.31,0.11-0.46
				c0.02-0.06,0.05-0.11,0.07-0.17l-0.12,0.06l-0.96-1.04l-0.21-0.12l-0.16,0.03l-0.11-0.37l0.1-0.18l0.18-0.3l0.53-0.11l0.09-0.1
				l0.06-0.32l0.21-0.09l0.23,0.17l0.02,0.17l0.2,0.13l0.21,0l-0.02-0.07l0.15-0.17l0.01-0.32l0.35-0.37l0.1-0.56l-0.53-0.14
				l-0.08-0.13l-0.17-0.03l-0.15-0.19l0.19-0.15l-0.23-0.14l-0.79,0.65l-0.48,0.01l-0.18,0.23l-0.4,0.29l-0.15,0.23l-0.39,0
				l-0.4-0.31l-0.27-0.45l0.11-0.31l-0.02-0.55l-0.29-1.16l-0.01-0.35l0.25-0.17l0.09-0.24l-0.57-0.33l-0.1-0.85l0.02-1.15
				l-0.3-0.27l-0.66-0.31l-0.64-0.01l-0.46-0.2l-0.39,0.14l-0.19-0.02l-0.48-0.15l-1.29-0.13l-0.63-0.69l0,0l-0.18,0.23l0.07,0.16
				l-0.07,0.56l-0.38,0.32l-0.04,0.14l0.44,0.45l0.13,0.32l-0.07,0.3l0.16,0.23l0.75,0.28l0.49,0.05l0.04,0.31l0.06,0.19l0.17,0.17
				l0.07,0.36l-0.54,0.72l0.03,0.33l-0.39-0.21l-0.05,0.08l-0.1-0.12l-0.14,0.05l0.09,0.13l-0.44,0.23l0.05,0.29l0.2,0.33L355,52.15
				l-0.48-0.08l-0.01,0.37l0.31,0.11l0.16,0.15l0.11,0.25l0.41,0.37l0.01,0.11l0.28,0.25l-0.06,0.34l0.32-0.09l0.11,0.19l0.14,0.05
				l0.18,0.38l-0.15-0.06l-0.14,0.27l-0.47-0.07l-0.44,0.1l-0.04-0.35l-0.29-0.14l-0.61,0.36l-0.62-0.42l-0.49,0.59l-0.7-0.03
				l2.33,1.73l-0.52,0.03l-0.52,0.34l-0.19,0.55l-1.09,1.16l-0.2,0.61l0.62,0.11l0.26,0.21l0.41,0.06l0.31-0.3l0.26-0.08l0.13-0.22
				l0.33,0.08l0.2,0.15l0.53-0.11l0.75-0.36l0.2,0.01l0.45,0.28l-0.2,0.19l0.75,0.57l0.36-0.15l0.19,0.11l0.3-0.03l0.33,0.21
				l-0.09,0.9l0.43,0.26l0.1-0.03l-0.04,0.59l0.15,0l0.28,0.33l0.38-0.32l-0.05-0.23l0.11,0.07l0.37-0.03l0.46-0.31l0.29-0.07
				l0.14-0.22l-0.06-0.13l0.06-0.15l0.55-0.19l0.01-0.18L362.02,59.77L362.02,59.77z"></path>
		</g>
		<g id="osorno" class="active">
			<path id="path2331" class="st3 fase_0" d="M355.67,68.86l-0.25,0.02l-0.3,0.58l-0.27,0.08l-0.32-0.02l-0.41,0l-0.42-0.2l-0.33,0.09
				l-0.28-0.19l-0.22,0.12l0.01,0.13l-0.2,0.2l-0.26,0.14l-0.27-0.06l-0.27,0.22l-0.11,0.39l0.11,0.68l0.43,0.55l0.06,0.47
				l0.12,0.01l-0.13,0.11l0.05,0.23l0.22,0.53l-0.43,0.9l-0.43-0.4l-0.19-0.51l-0.46-0.06l-0.32,0.09l-0.16-0.09l-0.22-0.71
				l-0.09-0.01l0.14-0.64l-0.35-0.12l-0.08-0.17l-0.17-0.09l-0.22,0.33l0.04,0.48l-0.16,0.19l-0.06,0.34l-0.22,0.08L349,72.43
				l-0.06,0.25l-0.41,0.28l-0.29,0.35l0.28,0.45l-0.01,0.16l0.15,0.34l0.12,0.47l-0.27,0.88l-0.17,0.26l-0.01,0.26l0.02,0.26
				l-0.47,0.75l0.02,0.19l0.52,1.15l0.04,0.15l-0.07,0.07l0.16,0.21l0.25,0.16l0.13,0.5l0.45-0.08l0.11,0.18l0.77,0.09l0.27,0.14
				l0.38,0.03l0.71,0.72l0.13-0.07l0.58-0.16l0.56,0.09l0.45,0.18l0.47-0.08l0.23,0.13l0.21-0.01l1.04,0.52l0.05,0.15l0.32,0.18
				l0.21,0.22l0.37,0.17l0.72-0.03l0.12,0.09l0.42,0.72l0.33-0.29l0.39-0.18l0.7-0.11l0.18,0.05l0.07-0.4l-0.09-0.01l-0.06-0.19
				l0.17-0.16l-0.05-0.06l-0.21-0.43l0.01-0.13l-0.1-0.06l-0.01-0.22l0.15-0.21l0.06-0.29l-0.19-0.25l-0.19-0.08l-0.2-0.14
				l-0.06-0.14l0.07-0.11l0.09,0.04l-0.01-0.18l-0.17-0.25l0.02-0.25l0.27,0.14l0.1-0.31l-0.05-0.26l-0.2-0.05l0.07-0.29l-0.24,0.16
				l-0.47,0.05l-0.49,0.25l-0.65,0.19l-0.38-0.59l0.1-0.24l-0.06-0.49l0.07-0.27l-0.08-0.29l-0.11-0.06l-0.07-0.2l0.07-0.17
				l-0.08-0.4l0.5-0.75l0.24-0.77l-0.4-0.17l-0.12-0.57l0.15-0.16l-0.08-0.2l0.26,0l0.52-0.25l0.23-0.44l0.08-0.55l0.34-0.19
				l0.11-0.4l-0.13-0.06l-0.34,0.1l-0.16-0.06l-0.2,0.03l-0.09,0.1l-0.21-0.34l-0.18,0.01l-0.14-0.2l-0.21,0.01l-0.1-0.12l-0.19-0.1
				l0.13-0.16l-0.04-0.92l0.33-0.35l-0.43-0.32l-0.06-0.18L355.67,68.86L355.67,68.86z M352.82,72.87l0.09,0.16l-0.07,0.12
				L352.82,72.87L352.82,72.87z"></path>
		</g>
		<g id="villada">
			<path id="path2355" class="st3 fase_0" d="M334.05,74.18l-0.73,0.6l-0.23,0.08l-0.32,0.42l-0.3,0.21l-0.12,0.06l0.57,0.12l0.05,0.3
				l0.15,0.21l-0.04,0.23l-0.35,0.32l0.09,0.53l-0.07,0.19l0.24,0.23l-0.12,0.25l0.32,0.31l-0.33,0.27l-0.89-0.17l-0.12,0.14
				l-0.41,0.04l0.12,0.12l-0.09,0.08l-0.63,0.09l-0.13-0.19l-0.17,0.13l-0.05,0.13l-0.1,0.02l0.03,0.35l-0.24,0.66l0.02,0.35
				l-0.26,0.35l-0.03,0.25l0.08,0.01l1.13-0.35l0.05-0.13l0.42,0l0.07,0.59l-0.06,0.31l-0.01,0.13l-0.15,0.19l-0.12,0.28l0.08,0.13
				l-0.04,0.3l-0.11,0.13l-0.11,0.6l-0.68,0.27l-0.13,0.18l-0.53-0.28l-0.13,0.11l-0.2-0.07l-0.3,0.21l-0.31,1.21l0.05,0.27
				l-0.32,0.13l-0.09,0.27l0.4,0.62l-0.06,0.76l-0.58,0.44l-0.01,0.14l0.18,0.63l0.3,0.59l-0.07,0.07l0.09,0.2l-0.29,0.59l-0.5,0.28
				l0.23,0.43l-0.03,0.63l0.48-0.15l-0.01,0.16l0.27,0.28l0.25-0.02l0.13,0.14l0.17-0.07l0.03,0.16l0.24,0.03l0.29-0.13L329.99,91
				l0.08-0.1l0.57-0.27l0.03,0.23l0.13,0.1l0.06-0.11l0.68-0.06l0.68,0.14l0.13-0.49l0.29-0.48l0.41-0.25l0.37,0.2l0.53,0.14
				l0.12,0.12l0.26-0.17l0.16-0.22l0.28-0.05l0.42-0.32l0.66-0.3l1.02-0.66l0-0.31l-0.17-0.16l0.11-0.53l-0.19-0.09l0.04-0.34
				l-0.16-0.58l-0.08-0.93l0.18-0.88l0.21-0.51l0.08-0.39l-0.05-0.43l-0.34-0.58l-0.1-0.01l-0.19-0.26l0.01-0.27l-0.15-0.15
				l-0.07-0.54l0.09-0.15l0.15-0.04l-0.68-0.55l0.06-0.13l0.43-0.17l0.02-0.26l0.09-0.09v-0.15l0.18-0.11l-0.02-0.11l0.44-0.03
				l0.75-0.23l-0.16-0.43l-0.03-0.52l-0.32-0.48l-1.01,0.41l-0.04,0.1l-0.22-0.35l-0.12-0.88l-0.11-0.04l0.21-0.79l-0.77-0.31
				l-0.1-0.39l-0.25-0.33l-0.44-1.11L334.05,74.18L334.05,74.18z"></path>
		</g>
		<g id="palencia_rural">
			<path class="st3 fase_0" d="M353.95,96.13l-0.04-0.32l-0.14-0.18l0.04-0.76l0.06-0.27l0.14-0.28l-0.02-0.18l-0.46,0.07l-0.31-0.21
				l0.02-0.14l0.18-0.18l0.18,0.06l0.36-0.11l0.02-0.1l-0.27-0.45l-0.19,0.17l-0.07-0.08l-0.32,0.24l-0.16-0.32l-0.54,0.54
				l-0.59-0.09l-0.39-0.29l-0.4-0.61l0.01-0.5l-0.01-0.22l-0.07-0.09l-0.56,0.02l-0.31-0.22l-0.1,0.43l-0.21-0.1l-0.07-0.15
				l-0.52-0.14l-0.54,0.07l-0.88,0.37l-0.06-0.13l-0.36,0.58l-0.14,0.39l-0.26-0.01l-0.14,0.12l-1.11-0.02l-0.16-0.22l-0.16-0.02
				l-0.23,0.23l-0.34,0.04l-0.48,0.76l-0.27,0.25l-0.33,0.92l-0.66,0.69l-0.29-0.17l-0.29,0.37l0.12-0.01l0.06,0.24l0.51,0.21
				l-0.28,0.44l0.35,0.32l-0.07,0.15l0.3,0.31l0.14,0.01l0.33-0.19l0.35,0.39l0.42,0.25l-0.62,0.74l-0.54-0.03l-0.15,0.09
				l-0.09-0.07l-0.3,0.51l-0.25,0.19l0.07,0.22l-0.22,0.11l-0.23,0.49l0.11,0.19l-0.03,0.23l0.1,0.04l0.06,0.15l0.1,0.8l-0.39,0.07
				l-0.26,0.39l-0.35,0.17l0.17,0.53v0.5l0.4,1.06l0.35-0.03l0.46,0.2l-0.1-0.31l0.66,0.18l0.06-0.06l0.23-0.74l0-0.32l0,0
				l0.17,0.03l0.3-0.14l0.59-0.5l0.21,0.26l0.45,0.13l0.13-0.1l0.64-0.08l0.1,0.05l-0.03,0.11l0.07,0.07l0.13-0.17l0.71-0.08
				l0.14-0.23l-0.17-0.25l0.03-0.09l-0.14-0.3l0.5-0.65l0.12-0.09c-0.19-0.11,1.97-2.16,1.97-2.16l-0.2,0.49l0.12,0.11l0.04-0.22
				l0.27,0.05l0.06-0.15l0.08,0.01l0.15,0.42l0.12,0.11l-0.16,0.2l0.4,0.22l0.34,0.59l0.78,0.34l0.42-0.55l0.13-0.09l-0.01-0.12
				l-0.14-0.13l0.07-0.17l0.49-0.02l0.36,0.15l0.42-0.33l-0.13-0.59l-0.12-0.16l-1.25-0.9l-0.02-0.08l-0.18-0.03l-0.08-0.16
				l0.38-0.03l-0.04-0.28l0.84-0.22l0.83-0.4l0-0.12l0.19-0.28l0.09-0.52L353.95,96.13z"></path>
		</g>
		<g id="fromista" class="active">
			<path id="path2395" class="st3 fase_0" d="M352.32,80.43l-0.58,0.16l-0.13,0.07l-0.4,0.69l-0.28-0.03l-0.19,0.08l-0.56,0.59
				l-0.78,0.1l-0.34,0.41l-0.79,0.48l-0.15,0.18l-0.04,0.23l0.41,1.14l-0.28,0.26l-0.15-0.07l-0.1,0.04l0.04,0.36l-0.14,0.65
				l0.06,0.15l0.24,0.05l-0.08,0.2l-0.14-0.06l-0.31,0.25l0.19,0.38l0.06,0.35l-0.34,0.39l0.05,0.13l-0.09,0.26l-0.02,0.61
				l-0.57,0.33l0.08,0.09l-0.07,0.17l0.17,0.57l0.09,0.03l0.08,0.48l-0.12,0.04l0.06,0.19l-0.49,0.49l0.92-0.59l0.09,0.07l0.23,0.23
				l-0.35,1.09l0.14,0.31l0.06,0.13l0.88-0.37l0.54-0.07l0.52,0.14l0.07,0.15l0.21,0.1l0.1-0.43l0.31,0.22l0.56-0.02l0.07,0.09
				l0.01,0.22l-0.01,0.5l0.4,0.61l0.39,0.29l0.59,0.09l0.54-0.54l0.16,0.32l0.32-0.24l0.07,0.08l0.19-0.17l0.27,0.45l-0.02,0.1
				l-0.36,0.11l-0.18-0.06l-0.18,0.18l-0.02,0.14l0.31,0.21l0.46-0.07l0.5,0.1l0.41-0.07l0.07-0.26l-0.25-0.15l-0.19-0.25
				l-0.35-0.27l0.26-0.24l0.23,0.18l0.34,0.51l0.49-0.15l0.54,0.03l0.12,0.35l0.18,0.14l0.29-0.25l0.15-0.03l0.59,0.12l0.44-0.06
				l0.13-0.51l0.46-0.65l0.4-0.11l0.12,0.08l0.61-0.2l0.13,0.14l0.02,0.2l0.34,0.06l0.83-0.43l0.12,0.01l0.66-0.53l-0.2-0.14
				l0.08-0.65l0.11-0.19l-0.22-0.2l0.36-0.18l0.31-0.37l-0.03-0.15l0.07-0.51l-0.09-0.25l-0.6-0.61l0-0.25l-0.26-0.2l-0.13-0.32
				l-0.22-0.09l-0.43,0.01l-0.03-0.48l0.21-0.66l-0.04-0.12l-0.26-0.07l-0.12-0.12v-0.08l0.32-0.28l0.03-0.41l-0.11-0.35l-0.51-0.18
				l0.13-0.52l-0.04-0.17l-0.17-0.1l-0.01-0.24l-0.11-0.1l0.11-0.24l-0.25-0.31l0.11-0.21l-0.08-0.84l-0.41,0.02l-0.05-0.14
				l-0.18-0.05l-0.7,0.11l-0.39,0.18l-0.33,0.29l-0.42-0.72l-0.12-0.09l-0.72,0.03l-0.37-0.17l-0.21-0.22l-0.32-0.18l-0.05-0.15
				l-1.04-0.52l-0.21,0.01l-0.23-0.13l-0.47,0.08l-0.45-0.18L352.32,80.43L352.32,80.43z"></path>
		</g>
		<g id="pintor_oliva">
			<path class="st3 fase_0" d="M350.24,98.6l-1.17-0.65c-0.05,0.02-0.11,0.03-0.16,0.05c-0.75,0.26-1.55,0.4-2.28,0.72
				c-0.3,0.23-0.61,0.46-0.9,0.71c0.06,0.02,0.12,0.05,0.14,0.09c0.46,0.65,1.1,1.08,1.78,1.44l0-0.04l0.07-0.05l-0.07-0.29
				l1.33-0.34l0.6-0.26l0.12,0.07l0.06-0.32l-0.12-0.11l0-0.19l0.2-0.3l-0.05-0.28l0.04-0.08L350.24,98.6z"></path>
		</g>
		<g id="eras_del_bosque">
			<path class="st3 fase_0" d="M348.96,97.99l-0.09,0.02l0.05-0.38l-0.54-0.07l-0.05-0.5c-0.69,0.11-1.4,0.16-2.1,0.2l-0.02,0
				c-0.06,0.31-0.15,0.6-0.28,0.88l0.03,0.01l0.19,0.8l-0.03,0.11l0.05,0.06c0.91-0.43,1.82-0.89,2.83-0.97L348.96,97.99z"></path>
		</g>
		<g id="villamuriel_de_cerrato">
			<path id="path2585" class="st3 fase_0" d="M350.14,99.42l-0.06,0.15l-0.27-0.05l-0.04,0.22l-0.06,0.32l-0.12-0.07l-0.6,0.26
				l-1.33,0.34l0.07,0.29l-0.07,0.05l0,0.16l0.23,0.08l-0.03,0.18l-0.11,0.04l-0.5,0.65l0.14,0.3l-0.03,0.09l0.17,0.25l-0.14,0.23
				l0.08,0.13l0.16-0.01l-0.06,0.13l0.4,0.35l0.15-0.08l0.32,0.15l0.09-0.15l0.3-0.1l0.23-0.29l0.21-0.08l0.1,0.55l0.3-0.75
				l0.34-0.34l-0.09-0.12l0.79-0.27l0.06,0.09l0.27-0.26l-0.08,0.26l0.11,0.27l0.25-0.04l0.05,0.13l0.27-0.19l0.23-0.04l0.09,0.06
				l0.13,0.32l0.26,0.19l0.15-0.07l0.09-0.21l0.02,0.25l0.16-0.09l0.18,0.11l0.32-0.19l0.09,0.05l0.34-0.22l0.1,0.07l-0.01,0.23
				l0.18-0.19l0.15-0.03l-0.71-0.6l-0.77-0.94l-0.38-0.18l-0.42,0.55l-0.04,0.26l0.05,0.13l-0.01,0.12l-0.14-0.05l-0.13-0.1
				l-0.32-0.04l0.04-0.19l-0.32-0.42l0.08-0.05l-0.34-0.59l-0.4-0.22l0.16-0.2l-0.12-0.11l-0.15-0.42L350.14,99.42L350.14,99.42z"></path>
		</g>
	</g>
	<g id="valladolid">
		<g id="mayorga_de_campos">
			<path id="path85" class="st3 fase_0" d="M324.74,82.24l-0.07,0.11l-0.23,0.11l-0.15,0.32l-0.12,0.04l-0.73,1.05l-0.48,0.21l0,0.2
				l-0.33,0.15l-0.56,0l-0.22,0.16l0.03,0.21l-0.61,0.2l-0.1-0.22l-0.66-0.71l-0.15,0l-0.09,0.14l-0.24,0.69l-0.12-0.06l0.05-0.65
				l-0.25,0.12l0.05,0.11l-0.13,0.28l-0.24,0.12l-0.24-0.06l-0.34-0.56l0.09-0.2l-0.11-0.4l-0.17,0.35l-0.6-0.08l-0.12,0.15
				l-0.14,0.04l0.02,0.13l-0.06,0.03l-0.16-0.1l-0.17,0l-0.07,0.37l-0.24,0.04l-0.19-0.13l-0.56,0.24l-0.33-0.37l-0.35,0.24
				l-0.33,0.1v0l-0.07,0.46v0v0l0.05,0.44v0l0.22,0.23l-0.03,0.75v0l0.17,0.1l-0.35,0.6l-0.14-0.04l-0.15,0.07l-0.34-0.32
				l-0.29,0.12l-0.3,0.02l-0.21-0.16l-0.29,0.21l-0.04,0.19l0.02,0.68l0.34,0.4l-0.28,0.46l-0.07,0.75l0.09,0.37l0.29,0.28
				l-0.08,0.21l0.01,0.47l0.39,1.11l-0.15,0.18l0,0.29l0.1,0.17l-0.2,0.29l0.18,0.25l0.11,0.02l0.15-0.09l0.08,0.09l0.15-0.12
				l-0.04-0.1l0.11-0.05l-0.04-0.07l0.1-0.04l0.19,0.07l0.09-0.09l0.16,0l0.25-0.12l0.14-0.17l0.04,0.21l0.59,0.26l0.3-0.09
				l0.2-0.19l0.51-0.21l0.17-0.19l0.15-0.03l0.18-0.21l0.07-0.08l0.3-0.01l0.42,0.16l0.35,0.47l-0.3,0.28l0.1,0.13l0,0.23l0.11,0.12
				l0.01,0.3l0.18,0.29l-0.14,0.25l-0.43,0.1l-0.24,0.15l-0.66-0.08l-0.33-0.14l-0.02,0.12l0.1,0.03l0.02,0.15l-0.08,0.73
				l-0.13,0.26l-0.07-0.07l-0.79,0.41l-1.26-0.03l-0.32-0.16l-0.17-0.19l-0.36-0.14l0.13,0.9l-0.18,0.25l0.05,0.19l-0.08,0.09l0,0.1
				l0.33,0.34l0.78,0.58l0.95,0.12l0.09,0.08l1.48-0.77l0.36-0.5l0.19-0.07l0.11,0.01l0.22,0.36l0.19,0.15l0.09-0.03l0.22,0.09
				l0.07,0.12l-0.09,0.15l0.11,0.27l-0.01,0.35l0.11,0.39l-0.08,0.22l0.25,0.31l0.34,0.05l0.05,0.28l0.26-0.33l0.1,0.04l0.19-0.23
				l-0.01-0.11l0.13-0.05l0.31-0.47l0.64,0.09l0.65,0.43l0.42,0.11l0.31,0.01l0.13-0.19l0-0.11l-0.17-0.05l-0.12-0.2l0.04-0.07
				l-0.27-0.3l0.01-0.11l-0.11,0l0.12-0.27l-0.15-0.09l-0.05,0.1l-0.28-0.12l-0.12-0.22l-0.12,0.07l-0.03-0.21l0.08-0.22l0.32-0.4
				l0.17-0.36l-0.1-0.3l0.44-0.16l0.19-0.21l-0.16-0.56l0.34-0.39l0.2-0.03l0.46-0.59l0.09-0.28l0.19-0.16l0.07-0.19l-0.22-0.11
				l-0.8-1.03l-0.36-0.4l-0.22-0.43l-0.47-0.09l-0.24-0.17L321.26,91l0.3-0.31l0-0.13l0.15-0.19l0.38-0.26l0.25-1.47l-0.34-0.43
				l0.33-0.41l0.39-0.46l0.25,0.03l0.41-0.09l0.25,0.04l0.17-0.26l0.6-0.44l0.39-0.68l0.45-0.08l0.14-0.2l0.81,0.06l0.18-0.07
				l0.17,0l0.19,0.31l0.13-0.03l0.35-0.28l-0.02-0.28l-0.11-0.21l-0.54-0.2l0.24-0.9l-0.16,0.09L326.41,84l-0.05,0.07l-0.13-0.06
				l-0.01-0.33l-0.14-0.26l0.04-0.1l-0.18-0.11l-0.15,0.08l-0.04-0.28l-0.27-0.64l-0.64-0.14L324.74,82.24z"></path>
		</g>
		<g id="tudela">
			<path id="path1980" class="st3 fase_0" d="M352.61,115.03l-0.22,0.06l-0.51,0.47l-0.46,0.15l-0.12-0.04l-0.03,0.12l-0.61,0.51
				l-0.21-0.03l0,0.23l-0.2,0.36l-0.75,0.44l-1.47,0.03l-0.59,0.14l-0.2,0.24l-0.3-0.12l-0.24,0.23l-0.26,0.08l-0.2,0.39l0.06,0.15
				l-0.3-0.13l-0.06,0.24l-0.18,0.01l-0.4,0.46l-0.85,0.1l-0.07,0.07l-0.05,0.42l-0.48,0.23l0.33,0.08l0.12,0.25l0.15,0.21l0.1,0.38
				l0.38,0.6l0.18,0.13l-0.52,0.28l0.2,0.39l0.46,0.65l0.6,0.15l0.26-0.07l0.05,0.23l0.24,0.21l-0.34,0.94l0.12-0.02l0.1,0.12
				l0.34,0.57l0.24-0.04l0.42,0.08l0.97,0.39l0.72,0.07l0.66-0.24l0.22-0.17l0.04-1.05l0.59-1.04l0.1,0l0.16-0.3l0.1,0.11l0.16-0.04
				l0.16,0.15l-0.05,0.19l0.21,0.12l0.47-0.17l0.13,0.12l0.4-0.1l0.13,0.28l0.46-0.08l0.11,0.08l0.08,0.3l0.11-0.26l0.32,0.04
				l0.19-0.27l0.05,0.16l0.21,0.06l0.32-0.08l0.13-0.17l0.41,0l0.03-0.4l-0.43-0.36l0.31-0.04l0.41,0.33l0.34-0.12l0.28-0.33
				l0.01-0.2l0.14-0.06l0.12,0.14l0.08-0.05l0.49-0.67l0.17-0.04l0.06-0.23l-0.47-0.31l0.33-0.07l0.16,0.05l0.37-0.11l0.22,0.07
				l0.18-0.1l0.24-0.82l0.02-0.32l0.17-0.11l0.35-0.78l-0.38-0.25l-0.14,0.83l-0.18,0.19l-0.37-0.47l-0.06-0.32l0.12-0.24
				l-0.33-0.13l-0.24,0.2l-0.08-0.63l-0.11,0l0-0.08l0.08-0.16l0.09-0.02l0.05-0.28l-0.07-0.22l-0.19-0.1l0.06-0.34l-0.5-0.11
				l-0.29,0.12l-0.32,0.38l-0.25,0l-0.2,0.19l-0.12-0.1l-0.17,0.01l-0.16,0.17l-0.25-0.02l-0.11,0.09l-0.26-0.31l-0.45,0.03
				l0.05-0.18l-0.16-0.1l-0.05-0.33l-0.51-0.64l0.33-0.37l-0.03-0.07l-0.36-0.11l-0.12,0.04l-0.15-0.13L352.61,115.03L352.61,115.03
				z M343.56,122.2l-0.38,0.15l-0.43,0.04l-0.11,0.34l-0.14,0.05l-0.21-0.18l-0.22,0.19l0,0.55l0.26-0.07l0.36,0.12l0.26-0.02
				l0.04,0.07l1.03-0.65l-0.1-0.31l-0.11-0.23L343.56,122.2L343.56,122.2z"></path>
		</g>
		<g id="mota_del_marques">
			<path id="path1756" class="st3 fase_0" d="M315.12,110.78l-0.84,0.16l-0.86,0.93l0.03,0.16l0,0.27l0.61,1.32l0.01,0.44l0.15-0.08
				l0.06-0.3l0.18-0.13l0.16,0.03l0.22,0.63l-0.08,0.93l0.5-0.13l0.2,0.16l0.1,0.66l0.41,0.27l0.18,0.62l0.05,0.41l-0.06,0.65
				l-0.04,0.38l0.21,0.36l-0.16,0.14l-0.07,0.22l0.04,0.27l-0.19,0.5l0.35,0.27l0.47-0.04l0.12,0.18l0.54-0.03l0.61,0.21l0.28,0.21
				l-0.06,0.26l0.05,0.13l0.17,0.05l-0.05,0.16l-0.52,0.29l-0.49,0.55l0.4,0.31l0.16,0.24l-0.05,0.11l0.32,0.16l-0.06,0.35
				l0.06,0.13l0.35-0.08l0.27,0.25l0.39,0.02l0.27-0.46l0.4-0.43l0.62-0.53l0.59-0.24l0.1,0.01l0.49-0.26l0.3,0.19l0.07-0.13
				l0.11-0.03l-0.06-0.11l0.27-0.08l0.31,0.25l0.14,0.28l-0.03,0.13l0.1,0.02l0.01,0.1l0.2-0.11l0.01,0.52l0.18,0.3l0.24-0.41
				l0.53,0.3l0.26-0.13l0.05-0.24l0.08-0.09l0.05,0.08l0.17-0.27l0.16,0.05l0.23-0.17l0.07,0.04l0.19-0.16l0.57-0.14l0.19-0.14
				l0.08-0.28l0.49-0.23l0.55-0.17l0.02-0.19l-0.21-0.32l-0.17-0.01l-0.06,0.08l-0.59-0.39l-0.5-0.13l-0.26-0.35l-0.17-0.11
				l0.01-0.08l-0.14-0.03l-0.09-0.14l-0.4-0.13l-0.03-0.27l0.23-0.18l0.21,0.01l0.14-0.4l-0.01-0.27l0.06-0.17l-0.14-0.16l0-0.38
				l0.01-0.18l0.09-0.09l-0.2-0.3l0.17-0.13l0-0.28l0.34-0.68l-0.41-0.07l-0.64-0.34l-0.54-0.09l0.06-0.18l-0.26-0.14l0.27-0.96
				l-0.3-0.44l0.13,0.02l0.3-0.39l-0.7-0.31l-0.36-0.32l-0.1,0.08l-0.32,0.05l-0.26,0.19l-0.42-0.15l-0.05,0.09l-0.23-0.05
				l-1.18-0.87l-0.21,0.31l-0.22,0.13l-0.43,0.1l-0.23-0.06l-0.45-0.04l-0.43-0.17l-0.17,0.06l-0.16-0.04l-0.14,0.04l-0.2,0.24
				l-0.24,0.08l-0.09,0.15l-0.23,0.1l-0.74-0.67l-1.45-0.67L315.12,110.78L315.12,110.78z"></path>
		</g>
		<g id="medina_de_rioseco">
			<path id="path1782" class="st3 fase_0" d="M327.67,97.69l-0.3,0.06l-0.43,0.22l-0.44-0.04l-0.33-0.02l-0.51,0.2l-0.56-0.05
				l-0.24,0.44l-0.35,0.28l-0.16,0.31l-0.52,0.64l0.14,0.53l-0.03,0.36l-0.11,0.16l0.01,0.11l0.24-0.03l0.14,0.16l0.15-0.03
				l0.14,0.16l-0.13,0.16l0.37,0.26l-0.16,0.23l-0.02,0.21l0.1,0.2l-0.11,0.02l-0.02,0.3l0.16,0.19l-0.05,0.15l0.14,0.09l0.11,0.42
				l-0.01,0.3l0.33,0.56l-0.35,0.3l-0.26-0.06l-0.17-0.06l0,0.08l-0.43,0.1l-0.08-0.09l-0.24,0.04l-0.09,0.01l0,0.11l-0.22,0.19
				l-0.07,0.18l-0.66,0.39l0.11,0.52l0.28,0.34l0.22,0.67l0.22-0.01l0.16,0.4l-0.04,0.12l0.17,0.72l0.02,0.42l0.92,1.26l0.03,0.41
				l0.81,1.6l-0.45,0.36l-0.44,0.22l-0.56,0.41l-0.3,0.39l-0.13-0.02l0.3,0.44l-0.27,0.96l0.26,0.14l-0.06,0.18l0.54,0.09l0.64,0.34
				l0.41,0.07l0.25-0.25l0.27-0.53l0.28-0.46l1.07-0.72l0.04-0.17l0.25,0.02l0.54-0.07l0.38-0.24l0.1-0.1l0.12-0.48l0.64-0.57
				l0.35-0.64l0.53-0.21l0.27,0.53l0.14-0.1l0.2,0.21l0.14,0.03l0.15-0.16l0.53-0.06l0.12,0.29l-0.25,0.5l0.31,0.51l0.59,0.58
				l0.56,0.16l0.7-1.11l0.15-0.67l-0.17-0.1l-0.3-0.42l-0.36-0.23l-0.3-0.16l-0.38-0.71l-0.06-0.33l-0.19-0.1l0.09-0.09l0.37-0.1
				l0.27,0.18l0.84,0.11l0.48-0.36l0.01-0.18l0.66-0.15l0.39-0.22l0.01,0.28l-0.12,0.41l0.11,1.1l0.14,0.16l0.33-0.04l0.35,0
				l0.27-0.13l-0.07-0.25l0.05-0.28l0.28-0.32l0.22-0.12l0.36-0.43l0.49-0.34l0.11-0.17l-0.17-0.22l-0.06-0.36l-0.17-0.29
				l-0.59-1.55l-0.37-0.24l-0.26-0.41l-0.12-0.29l0-0.34l-0.24-0.63l-0.72-0.79l0.01-0.13l-0.13,0.03l-0.16-0.17l-0.19,0.04
				l-0.06-0.76l-0.2-0.07l-0.58-0.02l-0.46,0.28l-0.48,0.68l-0.16-0.03l-0.03-0.09l-0.46,0.15l-0.07-0.07l-0.02-0.09l0.08-0.06
				l-0.18-0.41l-0.69-0.39l-0.24,0.14l-0.07,0.15l-0.4,0.19l-0.2,0.28l-0.22,0.05l0.04,0.27l-0.27,0.09l-0.02,0.1l-0.13-0.18
				l-0.01-0.45l-0.12-0.2l-0.3-0.18l-0.3-0.49l0.14-0.28l-0.15-0.22l0.11-0.04l0.29-0.54l0.05-0.25l0.44-0.22l0.01-0.17l0.28-0.2
				l0.07-0.2l0.12-0.62l-0.16-0.24l-0.21-0.15l-0.37-0.06l-0.31-0.15l-0.12-0.02l-0.07,0.09l-0.14-0.05l-0.13-0.1l-0.2-0.21
				l-0.34,0.11l-0.28-0.24l0.14-0.18l0-0.15l-0.15-0.15l0.04-0.12l-0.07-0.09l-0.09-0.15L327.67,97.69L327.67,97.69z"></path>
		</g>
		<g id="cigales">
			<path id="path1778" class="st3 fase_0" d="M342.64,104.35l-0.35,0.03l0.02,0.1l-0.33,0.38l-0.61,0.2l-0.72,0.75l-0.2,0.13l-0.18,0.12
				l-0.16,0.38l0.24,0.39l-0.88,0.75l-0.26,0.38l-0.16,0.42l-0.44,0l-0.03,0.15l-0.78,0.27l-0.11,0.17l-0.49,0.34l-0.36,0.43
				l-0.22,0.12l-0.28,0.32l-0.05,0.28l0.07,0.25l-0.27,0.13l-0.35,0l0.34,0.59l0.76,1.7l-0.13,0.01l0.75,2.09l0.17,0.12l0.27,0.18
				l0.49-0.07l0.59,0.19l0.57-0.07l0.32-0.26l0.63,0.25l0.27-0.32l0.07,0.04l0.29-0.76l0.47-0.25l0.24,0.03l-0.02-0.27l0.26-0.23
				l0.11-0.05l0.24-0.67l0.7-0.83l0.28-0.52l0.68,0.61l0.34-0.29l0.08,0.04l0.06-0.12l0.37,0.14l-0.05,0.39l0.57,0.24l0.31,0.36
				l1.53,0.5l0.75,0.5l0.78-0.15l1.29-0.38l0.36-0.26l0.49-0.52l0.38-0.83l-1.23-0.56l-0.67-0.1l0.36-0.39l-0.05-0.09l0.26-2.14
				l0.15-0.01l0.18,0.11l0.43-0.13l0.11,0.07l0.19-0.06l-0.7-0.44l0.2-0.19l0.06-0.46l-0.06-0.13l-0.09,0.04l-0.11-0.06l-0.04,0.28
				l-0.51-0.01l-0.31,0.28l-0.37,0.05l0.13,0.12l-0.01,0.17l-0.14,0.03l-0.12,0.16l-0.36,0.01l-0.34-0.25l-0.8-0.17l-0.04,0.12
				l0.19,0.15l-0.1,0.38l-0.12,0.12l-0.16,0.02l-0.39-0.11l-0.11,0.11l0.05,0.2l-0.65-0.53l-0.36-0.78l-0.52-0.1l-0.58-0.68
				l-0.24-0.15l-0.1-0.23l-0.26-0.26l0.13-0.27l-0.1,0l-0.12-0.15l0.05-0.25l-0.11-0.18l0.06-0.24l-0.1-0.24l-0.27-0.17l-0.24,0.01
				l-0.34-0.19l-0.01-0.12L342.64,104.35L342.64,104.35z"></path>
		</g>
		<g id="villalon_de_campos">
			<path id="path1766" class="st3 fase_0" d="M327.72,83.03l-0.13,0.23l0.19,0.2l-0.08,0.64l-0.14,0.31l0.03,0.21l-0.14,0.3l-0.39,0.24
				l0.11,0.21l0.02,0.28l-0.35,0.28l-0.13,0.03l-0.19-0.31l-0.17,0l-0.18,0.07l-0.81-0.06l-0.14,0.2l-0.45,0.08l-0.39,0.68
				l-0.6,0.44l-0.17,0.26l-0.25-0.04l-0.41,0.09l-0.25-0.03l-0.39,0.46l-0.33,0.41l0.34,0.43l-0.25,1.47l-0.38,0.26l-0.15,0.19
				l0,0.13l-0.3,0.31l0.36,0.05l0.24,0.17l0.47,0.09l0.22,0.43l0.36,0.4l0.8,1.03l0.22,0.11l-0.07,0.19l-0.19,0.16l-0.09,0.28
				l-0.46,0.59l-0.2,0.03l-0.34,0.39l0.16,0.56l-0.19,0.21l-0.44,0.16l0.1,0.3l-0.17,0.36l-0.32,0.4l-0.08,0.22l0.03,0.21l0.12-0.07
				l0.12,0.22l0.28,0.12l0.05-0.1l0.15,0.09l-0.12,0.27l0.11,0l-0.01,0.11l0.27,0.3l-0.04,0.07l0.12,0.2l0.17,0.05l0,0.11l0.34,0.33
				l0.37,0.16l0.07,0.24l-0.04,0.25l0.2,0.08l0.52-0.64l0.16-0.31l0.35-0.28l0.24-0.44l0.56,0.05l0.51-0.2l0.33,0.02l0.44,0.04
				l0.43-0.22l0.3-0.06l0.29,0.04l0.09,0.15l0.07,0.09l-0.04,0.12l0.15,0.15l0,0.15l-0.14,0.18l0.28,0.24l0.34-0.11l0.2,0.21
				l0.13,0.1l0.14,0.05l0.07-0.09l0.12,0.02l0.31,0.15l0.37,0.06l0.21,0.15l0.21-0.27l0.19-0.46l0.59-0.27l0.61-0.31l-0.09-0.11
				l-0.47-1.67l0.89-1.21l0.44-0.26l0.51,0l-0.55-0.67l0.23-0.03l-0.33-0.28l-0.16-0.76l-0.18-0.42l0.15-0.13l-0.16-0.53l-0.03-0.76
				l0.09-0.3l-0.68-0.14l-0.68,0.06l-0.06,0.11l-0.13-0.1l-0.03-0.23l-0.56,0.27l-0.08,0.1l0.04,0.11l-0.29,0.13l-0.24-0.03
				l-0.03-0.16l-0.17,0.07l-0.13-0.14l-0.25,0.02l-0.27-0.28l0.01-0.16l-0.48,0.16l0.03-0.63l-0.23-0.43l0.5-0.28l0.29-0.59
				l-0.09-0.2l0.07-0.07l-0.3-0.59l-0.18-0.63l0.01-0.14l0.58-0.44l0.06-0.76l-0.4-0.62l0.09-0.28l0.32-0.13l-0.05-0.27l0.31-1.21
				L327.72,83.03L327.72,83.03z"></path>
		</g>
		<g id="esguevillas">
			<path id="path1792" class="st3 fase_0" d="M360.24,109.12l-0.51,0.48l-0.36-0.19l-0.16,0.06l-0.45,0.71l-0.3,0.19l-0.32,0.74
				l-0.82-0.06l-0.27,0.35l-0.15,0.05l-0.02,0.27l-0.48-0.29l-0.18-0.18l-0.16,0.03l-0.59-0.12l-1.17-0.46l-0.05-0.07l0.09-0.15
				l-0.46-0.04l-0.13,0.11l-0.36,0.11l-0.71,0.81l-0.84,0.51l-0.19,0.02l-0.3-0.05l-0.38,0.83l-0.49,0.52l-0.36,0.26l-1.29,0.38
				l-0.78,0.15l-0.04,0.33l0.07,0.23l-0.35,0.47l-0.05,0.42l0.22,0.47l0.11,1.33l1.47-0.03l0.75-0.44l0.2-0.36l0-0.23l0.21,0.03
				l0.61-0.51l0.03-0.12l0.12,0.04l0.46-0.15l0.51-0.47l0.22-0.06l-0.01,0.09l0.15,0.13l0.12-0.04l0.36,0.11l0.03,0.07l-0.33,0.37
				l0.51,0.64l0.05,0.33l0.16,0.1l-0.05,0.18l0.45-0.03l0.26,0.31l0.11-0.09l0.25,0.02l0.16-0.17l0.17-0.01l0.12,0.1l0.2-0.19
				l0.25,0l0.32-0.38l0.29-0.12l0.5,0.11l0.09-0.17l-0.26-0.26l-0.18-0.48l-0.28-0.09l-0.33-0.38l0.41-0.11l0.08,0.07l0.31-0.05
				l0.33-0.15l0.41,0.06l0.2,0.35l0.48,0.08l0.19-0.06l0.27,0.16l0.37-0.19l0.2-0.03l0.07,0.21l0.11-0.13l0.29-0.04l0.23-0.04
				l0.39-0.3l0.15,0.01l0.29-0.15l0.26-0.13l0.63-0.02l0.22-0.1l-0.1-0.32l0.52-0.5l0.23,0.03l0.21-0.19l0.71-0.36l0.4,0.46
				l0.28,0.17l0.21-0.21l0.42-0.07l0.34-0.2l0.3,0.69l1.48-0.42l0.07-0.14l-0.05-0.36l0.28-0.31l0.14,0.09l0.4-0.07l0.25-1.48
				l0.35-0.01l-0.83-0.34l-0.11,0.24l-0.31-0.02l-0.55-0.46l-0.2-0.02l-0.26,0.25l-0.12,0.02l-0.37-0.59l-0.08-0.5l-0.29-0.3
				l-0.21-0.38l-0.51,0.22l-0.61,0.24l-0.13,0.14l-0.39-0.27l-0.27,0.06l-0.22-0.14l-0.42,0.02l-0.18,0.04l-0.16-0.13l-0.84,0.24
				l-0.72-0.46L360.24,109.12L360.24,109.12z"></path>
		</g>
		<g id="valladolid_rural_i">
			<path class="st3 fase_0" d="M347.92,116l0.11,1.33l-0.59,0.14l-0.2,0.24l-0.3-0.12l-0.24,0.23l-0.26,0.08l-0.2,0.39l0.06,0.15l-0.3-0.13
				l-0.06,0.24l-0.18,0.01l-0.4,0.46l-0.85,0.1l-0.07,0.07l-0.05,0.42l-0.48,0.23v0l0.33,0.08l0.12,0.25l0.15,0.21l0.1,0.38
				l0.38,0.6l0.18,0.13l-0.52,0.28l0.2,0.39l-0.92,0.28l-0.11-0.23l-0.24-0.04l-0.38,0.15l-0.43,0.04l-0.11,0.34l0.02-0.31
				l-0.16-0.94l-0.28-0.7l-0.61,0.08l0.12-0.38l-0.15-0.5l0.07-0.23l0.52-0.33l0.11,0.01l0.06-0.12l0.41-0.11l0.08-0.1l0.45,0.03
				l0.29-0.05l-0.05-0.33l-0.28-0.46l-0.12-0.4l0.12-0.27l-0.01-0.21l-0.22-0.35l-0.05-0.26l0.18-0.05l0.12-0.3l0.28-0.13l0.06-0.09
				l-0.07-0.17l0.35-0.19l0.18,0.07l0.15-0.17l-0.06-0.2l-0.1-0.05l-0.21,0.09l-0.1,0.25l-0.14,0.06l-0.61-0.02l-0.18,0.15
				l-0.18,0.03l-0.3-0.08l-0.07,0.08l-0.41-0.07l0.03-0.09l-0.36-0.24l-0.06-0.18l0.05-0.29l0.41-0.02l0.14-0.07l0.06-0.14
				l-0.11-0.46l-0.16-0.22l-0.02-0.27l0.26-0.23l0.11-0.05l0.24-0.67l0.7-0.83l0.28-0.52v0l0.68,0.61l0.34-0.29l0.08,0.04l0.06-0.12
				l0.37,0.14l-0.05,0.39l0.57,0.24l0.31,0.36l1.53,0.5l0.75,0.5l-0.04,0.33l0.07,0.23l-0.35,0.47l-0.05,0.42L347.92,116z"></path>
		</g>
		<g id="villafrechos">
			<path id="path1796" class="st3 fase_0" d="M318.21,96.65l-0.19,0.07l-0.36,0.5l-1.48,0.77l0.22,0.66l0.18,0.21l0.16-0.01l0.29,0.54
				l-0.04,0.09l0.91,0.66l0.3-0.11l0.52-0.2l0.24,0.16l-0.35-0.01l-0.41,0.05l-0.48,0.36l-0.15,0.17l0.02,0.16l-0.45,0.36
				l-0.29,0.46l-1,0.55l-0.13,0.13l-0.02,0.14l0.58,0l0.44,0.28l0.45,0.55l0.21,0.12l-0.09,0.47L317.1,104l-0.4,0.74l-0.04,0.57
				l0.25,0.84l-0.17,0.17l0.12,0.33l-0.62,0.92l0.07,0.06l-0.29,0.24l-0.1,0.23l0.12,0.11l0.76,0.18l0.05-0.06l0.32,0.07l0.14-0.11
				l0.18,0.07l0.14-0.08l0.42,0.26l-0.11,0.51l-1.39,1.06l-0.1,0.15l-0.16,0.04l-0.09,0.11l-0.15-0.09l-0.29,0.03l-0.32,0.01
				l-0.32,0.45l-0.02,0.38l1.45,0.67l0.74,0.67l0.23-0.1l0.09-0.15l0.24-0.08l0.2-0.24l0.14-0.05l0.16,0.04l0.17-0.06l0.43,0.17
				l0.45,0.04l0.23,0.06l0.43-0.1l0.22-0.13l0.21-0.31l1.18,0.87l0.23,0.05l0.05-0.09l0.42,0.15l0.26-0.19l0.32-0.05l0.1-0.08
				l0.36,0.32l0.7,0.31l0.56-0.41l0.44-0.22l0.45-0.36l-0.81-1.6l-0.03-0.41l-0.92-1.26l-0.02-0.42l-0.17-0.72l0.04-0.12l-0.16-0.4
				l-0.22,0.01l-0.22-0.67l-0.28-0.34l-0.11-0.52l0.66-0.39l0.07-0.18l0.22-0.19l0.01-0.11l0.09-0.01l0.24-0.04l0.08,0.09l0.43-0.1
				l0-0.08l0.17,0.06l0.26,0.06l0.35-0.3l-0.33-0.56l0.01-0.3l-0.11-0.42l-0.14-0.09l0.05-0.15l-0.16-0.19l0.02-0.3l0.11-0.02
				l-0.1-0.2l0.02-0.21l0.16-0.23l-0.37-0.26l0.13-0.16l-0.14-0.16l-0.15,0.03l-0.14-0.16l-0.24,0.03l-0.01-0.11l0.11-0.16
				l0.03-0.36l-0.14-0.53l-0.2-0.08l0.04-0.25l-0.07-0.24l-0.37-0.16l-0.34-0.33l-0.13,0.19l-0.31-0.01l-0.42-0.11l-0.65-0.43
				l-0.64-0.09l-0.31,0.47l-0.13,0.05l0.01,0.11l-0.19,0.23l-0.1-0.04l-0.26,0.33l-0.05-0.28l-0.34-0.05l-0.25-0.31l0.08-0.22
				l-0.11-0.39l0.01-0.35l-0.11-0.27l0.09-0.15l-0.07-0.12l-0.22-0.09l-0.09,0.03l-0.19-0.15l-0.22-0.36L318.21,96.65L318.21,96.65z
				"></path>
		</g>
		<g id="olmedo">
			<path id="path1812" class="st3 fase_0" d="M340.48,130.32l-0.43,0.3l0,0.15l0.19,0.13l0.14,0.43l-0.22,0.09l-0.34-0.01l-0.05,0.24
				l-0.2,0.02l-0.12-0.19l-0.1,0.04l-0.21,0.38l-0.58,1.76l-0.21-0.05l-0.17-0.16l-0.54-0.02l-0.03,0.5l0.05,0.11l-0.29,0.54
				l0.24,0.63l-0.22,1.09l0.1,0.69l0.17,0.5l-0.12,0.33l-0.27,0.17l-0.05,0.58l0.29,0.49l0.3,0.23l0.12-0.04l0.54,0.62l0.38,0.09
				l0.38-0.01l0.24-0.16l0.05,0.33l-0.06,0.23l-0.32,0.55l-0.7,0.52l0.4,1l0.18,0.03l0.3,0.24l1.28-0.85l0.05-0.1l0.2,0.02
				l0.25,0.35l0,0.25l0.22,0.43l0.03,0.59l0.72-0.33l0.06-0.2l0.1-0.07l0.59,0.29l0.9,0.66l0.35,0l0.3-0.14l-0.17-0.75l-0.18-0.31
				l-0.08,0.04l-0.08-0.2l-0.41-0.35l0.17-0.44l-0.18-0.71l0.12-0.14l0.43-0.17l0.33-0.3l0.21-0.08l0.35-0.38l0.11-0.15l0.37-0.53
				l-0.03-0.1l0.22-0.44l0.44-0.34l0.53-0.18l0.04-0.07l-0.11-0.1l0.04-0.36l-0.2-0.24l-0.39-0.2l-0.13-0.24l-0.35-0.34l0.04-0.27
				l-0.17-0.3l0.34-0.4l-0.17-0.03l-0.46-0.4l-0.34-1.58l-0.17-0.27l0.13-0.2l-0.06-0.07l0.3-0.41l0-0.21l0.24-0.4l-0.01-0.17
				l-0.02-0.18l-0.19-0.07l-0.23-0.19l-0.6-0.31l-0.28,0.12l-0.19-0.12l-0.18,0.11l-0.31-0.12l-0.05-0.13l-0.23-0.06l-0.13,0.02
				l-0.08,0.11l-0.06-0.17l-0.12-0.09l-0.92,0.23l-0.62-0.29L340.48,130.32L340.48,130.32z"></path>
		</g>
		<g id="parque_alameda-covaresa">
			<path id="path1922" class="st3 fase_0" d="M341.62,114.26l-0.47,0.25l-0.29,0.76l-0.07-0.04l-0.27,0.32l-0.63-0.25l-0.32,0.26
				l-0.57,0.07l-0.59-0.19l-0.49,0.07l-0.27-0.18l-0.01,0.15l-0.41,0.12l-0.28,0.18l0.05,0.07l0.34-0.16l0.14,0.16l0.2-0.07
				l-0.03,0.12l0.27,0.07l-0.05,0.23l-0.16-0.02l0.03,0.1l-0.08,0.13l0.01,0.15l0.48,0.47l0.57-0.02l0.12,0.12l0.1-0.03l0.01,0.27
				l-0.23,0.19l0.23,0.21l-0.12,0.2l-0.04,0.19l0.06,0.06l-0.49,0.34l0.22,0.43l0.22,0.2l0.14,0.02l-0.32,0.21l-0.4,0.5l-0.71,0.23
				l0.38,0.84l-0.1,0.07l0.18,0.1l0.43-0.23l-0.41,0.87l-0.11,0.01l-0.09,0.18l-0.11,0.46l-0.65-0.19l-0.38,0.3l-0.25-0.09
				l-0.23,0.1l-0.74,0.04l-0.55,0.34l0.95,0.78l0.26,0.3l-0.04,0.32l0.79,0.22l0.03-0.08l0.5-0.04l0.18,0.04l0.11,0.13l0.13-0.06
				l0.09,0.08l0.2-0.07l-0.05-0.21l0.77-0.12l0.01-0.13l-0.16-0.1l-0.05,0.18l-0.06-0.09l0.11-0.09l-0.4-0.18l0.19-0.53l0.19,0.04
				l0.09-0.1l-0.11-0.03l-0.14-0.26l0.55-0.16l0.15-0.53l-0.06-0.41l0.06-0.31l0.25-0.41l0.68-0.21l0.19-0.18l0.19-0.04l0.62,0.29
				l0.54-0.13l0.12-0.38l-0.15-0.5l0.07-0.23l0.52-0.33l0.11,0.01l0.06-0.12l0.41-0.11l0.08-0.1l0.45,0.03l0.29-0.05l-0.05-0.33
				l-0.28-0.46l-0.12-0.4l0.12-0.27l-0.01-0.21l-0.22-0.35l-0.05-0.26l0.18-0.05l0.12-0.3l0.28-0.13l0.06-0.09l-0.07-0.17l0.35-0.19
				l0.18,0.07l0.15-0.17l-0.06-0.2l-0.1-0.05l-0.21,0.09l-0.1,0.25l-0.14,0.06l-0.61-0.02l-0.18,0.15l-0.18,0.03l-0.3-0.08
				l-0.07,0.08l-0.41-0.07l0.03-0.09l-0.36-0.24l-0.06-0.18l0.05-0.29l0.41-0.02l0.14-0.07l0.06-0.14l-0.11-0.46l-0.16-0.22
				L341.62,114.26L341.62,114.26z"></path>
		</g>
		<g id="tordesillas">
			<path id="path1938" class="st3 fase_0" d="M327.39,113.37l-0.04,0.17l-1.07,0.72l-0.28,0.46l-0.27,0.53l-0.25,0.25l-0.34,0.68l0,0.28
				l-0.17,0.13l0.2,0.3l-0.09,0.09l-0.01,0.18l0,0.38l0.14,0.16l-0.06,0.17l0.01,0.27l-0.14,0.4l-0.21-0.01l-0.23,0.18l0.03,0.27
				l0.4,0.13l0.09,0.14l0.14,0.03l-0.01,0.08l0.17,0.11l0.26,0.35l0.5,0.13l0.59,0.39l0.06-0.08l0.17,0.01l0.21,0.32l-0.02,0.19
				l-0.55,0.17l-0.49,0.23l-0.08,0.28l-0.19,0.14l-0.57,0.14l-0.19,0.16l-0.07-0.04l-0.23,0.17l-0.16-0.05l-0.17,0.27l-0.05-0.08
				l-0.08,0.09l-0.05,0.24l-0.26,0.13l-0.53-0.3l-0.24,0.41l-0.18-0.3l-0.01-0.52l-0.2,0.11l-0.01-0.1l-0.1-0.02l0.03-0.13
				l-0.14-0.28l-0.31-0.25l-0.27,0.08l0.06,0.11l-0.11,0.03l-0.07,0.13l-0.3-0.19l-0.49,0.26l-0.1-0.01l-0.59,0.24l-0.62,0.53
				l-0.4,0.43l-0.27,0.46l0.01,0.09l0.33,0.25l0.21,0.29l0.05,0.23l0.17,0.13l-0.2,0.58l-0.32-0.11l-0.5,0.54l-0.32,0.32l-0.62-0.14
				l-0.55,0.3l-0.09-0.04l-0.25,0.07l-0.48,0.53l-0.52,0.55l-0.34,0.12l0.01,0.11l-0.12,0.04l-0.09-0.06l-0.2-0.27l-0.11,0.11
				l0.29,0.97l0.1,0.08l0.22,0.01l0.2,0.23l0.3,0.33l0.55-0.1l0.09,0.13l0.1,0.75l0.48,0.33l0.37-0.45l0.3-0.62l0.49-0.74l0.71-0.8
				l0.21-0.08l0.54-0.09l0.82,0.06l0.17-0.22l0.33,0.23l-0.05,0.28l0.24,0.27l-0.1,0.2l0.04,0.44l-0.14-0.06l-0.5,0.13l0.21,0.68
				l0.01,0.81l0.27,0.4l0.15-0.48l0.41,0.09l0.14,0.23l0.01,0.29l0.14,0l-0.01,0.1l-0.16,0.14l0.05,0.23l0.22,0.04l0.27-0.09
				l0.14,0.08l0.03,0.22l0.36,0.07l0.21,0.29l0.28-0.76l0.15-0.12l0.1,0.05l0.25-0.09l-0.05-0.1l0.1-0.03l0.28,0.14l0.52,0
				l0.25-0.29l0.36,0.06l0.06-0.16l0.42-0.08l-0.09-0.29L326,129.6l0.12-0.7l-0.2-0.33l-0.03-0.23l0.81-0.7l0.32-0.14l0.69,0.61
				l0.36-0.05l0.23-0.34l0.18,0.24l0.56-0.08l0.97,0.02l0.24,0.09l0.42-0.01l0.9-0.25l-0.05-0.28l-0.33-0.33l-0.11-0.25l0.32-0.21
				l0.37-0.54l0.3,0.52l0.32,0.22l0.86-0.34l0.01-0.28l-0.13-0.44l0.08-0.02l0.03-0.13l0.52-0.19l0.11-0.31l0.54-0.74l0.53-0.48
				l0.05-0.24l-0.27-0.5l-0.05-0.24l-0.67,0.41l-0.25-0.28l-0.45-0.02l-0.32-0.2l-0.12-0.07l0.08-0.16v-0.6l0.34-0.31l0.08-0.22
				l-0.11-0.78l0.27-0.15l0.06,0.07l0.18-0.06l0.11-0.18l0.16-0.05l-0.05-0.12l0.06-0.12l-0.14-0.48l-0.1-0.13l-0.32,0.09
				l-0.17-0.14l-0.1,0.1l0.08,0.23l-0.34-0.14l-0.17-0.15l-0.12-0.52l0.09-0.18l0.12-0.42l0-0.23l-0.14,0.05l-0.03,0.22l-0.18,0.32
				l-0.61,0.27l-0.64-0.05l-0.42,0.18l-0.25,0.23l-0.34-0.01l-0.13,0.11l-0.34-0.17l-0.02-0.15l0.27-0.64l0.2,0.06l-0.06-0.37
				l0.33-0.4l0.23-0.46l0.32-0.3l0.33-0.09l0.16-0.19l-0.34-0.54l-0.27-0.05l-0.95,0.11l-0.26-0.18l-0.44,0.12l-0.16-0.01l-0.24-0.2
				l-0.19-0.5l-0.07-0.67l-0.43-0.48l-0.43-0.29l-0.08-0.64L327.39,113.37L327.39,113.37z"></path>
		</g>
		<g id="portillo">
			<path id="path2088" class="st3 fase_0" d="M344.82,122.19l-0.92,0.28l0.1,0.31l-1.03,0.65l-0.04-0.07l-0.26,0.02l-0.36-0.12
				l-0.26,0.07l0.04,0.23l0.31-0.01l0.04,0.1l-0.11,0.09l-0.32,0.61l-0.36,0.18l-0.47-0.07l-0.69,0.97l-1,0.57l-0.47,0.16l0.22,2.04
				l0.33,0.43l0.92,1.69l0.35,0.34l0.62,0.29l0.92-0.23l0.12,0.09l0.06,0.17l0.08-0.11l0.13-0.02l0.23,0.06l0.05,0.13l0.31,0.12
				l0.18-0.11l0.19,0.12l0.28-0.12l0.6,0.31l0.17-0.18l0.05-0.4l0.35-0.41l-0.03-0.23l0.25-0.07l-0.05-0.21l0.12,0.05l0.06-0.12
				l0.24-0.02l0.12,0.03l0.03,0.11l0.18-0.06l0.28,0.04l0.12,0.1l0.23-0.03l0.13,0.15l0.34-0.24l0.17,0.07l0-0.22l0.13-0.19
				l0.1-0.01l0.52,0.18l0.07-0.05l0.59,0.76l0.12,0.34l0.52-0.07l0.07-0.17l0.2,0.13l-0.01,0.08l0.42,0.04l0.09-0.15l0.59-0.06
				l0.14-0.1l-0.05-0.09l0.14-0.01l-0.05-0.12l0.2-0.12l0.13,0.05l0.08-0.05l0.02-0.13l0.11,0.03l0.03-0.08l0.24-0.05l0.11-0.11
				l0.18,0.03l0.23-0.08l-0.17-0.1l0.12-0.03l0.02-0.09l0.05-0.06l0.18,0.06l0.27-0.38l0.46-0.11l0.55,0.12l0.01,0.1l0.12,0
				l0.1-0.14l0.25,0.12l0.13-0.18l0.09-0.01l0.16,0.16l0.2-0.03l0.12,0.17l0.14-0.47l-0.04-0.28l-0.14-0.42l0.02-0.21l-0.51-0.33
				l-0.31,0.1l-0.58-0.51l-0.11,0.01l-0.54,0.51l-0.41-0.24l-0.04-0.08l0.09-0.13l0.33-0.62l0.42-0.41l0.07-0.37l0.15-0.26
				l0.23-0.18l-0.05-0.27l0.1-0.25l-0.15-0.41l-0.31-0.3l-0.24-0.29l0.03-0.25l-0.08-0.3l-0.11-0.08l-0.46,0.08l-0.13-0.28l-0.4,0.1
				l-0.13-0.12l-0.47,0.17l-0.21-0.12l0.05-0.19l-0.16-0.15l-0.16,0.04l-0.1-0.11l-0.16,0.3l-0.1,0l-0.59,1.04l-0.04,1.05
				l-0.22,0.17l-0.66,0.24l-0.72-0.07l-0.97-0.39l-0.42-0.08l-0.24,0.04l-0.34-0.57l-0.1-0.12l-0.12,0.02l0.34-0.94l-0.24-0.21
				l-0.05-0.23l-0.26,0.07l-0.6-0.15L344.82,122.19L344.82,122.19z"></path>
		</g>
		<g id="iscar">
			<path id="path2166" class="st3 fase_0" d="M347.61,129.52l-0.1,0.01l-0.13,0.19l0,0.22l-0.17-0.07l-0.34,0.24l-0.13-0.15l-0.23,0.03
				l-0.12-0.1l-0.28-0.04l-0.18,0.06l-0.03-0.11l-0.12-0.03l-0.24,0.02l-0.06,0.12l-0.12-0.05l0.05,0.21l-0.25,0.07l0.03,0.23
				l-0.35,0.41l-0.05,0.4l-0.17,0.18l0.23,0.19l0.19,0.07l0.02,0.18l0.01,0.17l-0.24,0.4l0,0.21l-0.3,0.41l0.06,0.07l-0.13,0.2
				l0.17,0.27l0.34,1.58l0.46,0.4l0.17,0.03l0,0l-0.34,0.4l0.17,0.3l-0.04,0.27l0.35,0.34l0.13,0.24l0.39,0.2l0.2,0.24l-0.04,0.36
				l0.11,0.1l0.33,0.1l0.22,0.22l1.19-0.6l0.51-0.04l0.97,0.16l0.28,0.42l0.25,0.04l0.07,0.09l0.33-1.27l0,0l0.15-0.23l-0.11-1.21
				l-0.16-0.26l0,0l0.11-0.58l-0.01-0.47l-0.14-0.51l-0.18-0.24l-0.16-0.47c0,0,0,0,0-0.06v0l0.28-0.15l0.4-0.05l0.25-0.1l0.38-0.43
				l-0.05-0.2l-0.21-0.13l0.17-0.05l0.14-0.51l-0.62-1.05l0,0.12l-0.09,0.07l-0.59,0.06l-0.08,0.09l0.04,0.07l-0.47-0.05l0.01-0.08
				l-0.2-0.13l-0.07,0.16l-0.52,0.07l-0.12-0.34l-0.59-0.76l-0.07,0.05L347.61,129.52L347.61,129.52z"></path>
		</g>
		<g id="serrada">
			<path id="path2106" class="st3 fase_0" d="M334.86,122.95l-0.04,0.06l-0.11,0.16l0.27,0.5l-0.05,0.24l-0.53,0.48l-0.54,0.74
				l-0.11,0.31l-0.52,0.19l-0.03,0.13l-0.08,0.02l0.13,0.44l-0.01,0.28l-0.86,0.34l-0.32-0.22l-0.3-0.52l-0.37,0.54l-0.32,0.21
				l0.11,0.25l0.33,0.33l0.05,0.28l-0.9,0.25l-0.42,0.01l0.17,0.56l0.18,0.18l0.22-0.03l0.13,0.07l0.41,0.42l-0.23,0.42l0.58,0.52
				l0.09,0.24l0.17,1.6l0.18,0.33l0.04-0.08l0.18,0.09l0.42-0.03l0.08,0.06l0.37-0.22l0.32-0.36l0.54-0.36l0.1-0.53l0.09-0.05
				l0.01-0.55l0.17-0.62l-0.1-0.14l0.6-0.33l0.26,0.36l0.22,0.12l0.08,0.71l-0.02,0.17l-0.11,0.09l-0.31,0.67l0.43-0.01l0.35,0.27
				l0.36-0.01l0.32-0.11l0.19,0.15l0.6,0.09l0.01,0.51l0.21,0.65l0.06,0.63l0.54,0.02l0.17,0.16l0.21,0.05l0.58-1.76l0.21-0.38
				l0.1-0.04l0.12,0.19l0.2-0.02l0.05-0.24l0.34,0.01l0.22-0.09l-0.14-0.43l-0.19-0.13l0-0.15l0.43-0.3l-0.92-1.69l-0.33-0.43
				l-0.22-2.04l-0.22,0.09l-0.36-0.12l-0.22-0.28l-0.26-0.6l-0.4-0.49l-0.33-0.11l-0.4-0.07l-0.79-0.22l0.04-0.32l-0.26-0.3
				L334.86,122.95L334.86,122.95z"></path>
		</g>
		<g id="alaejos">
			<path id="path2128" class="st3 fase_0" d="M321.22,126.79l-0.17,0.22l-0.82-0.06l-0.54,0.09l-0.21,0.08l-0.71,0.8l-0.49,0.74
				l-0.3,0.62l-0.37,0.45l-0.48-0.33l-0.1-0.75l-0.09-0.13l-0.55,0.1l-0.3-0.33l-0.3,0.28l0.2,0.16l-0.23,0.17l-0.02,0.42
				l-0.24,0.57l0.85,0.25l0.44,0.37l-0.15,0.29l-0.38,0.39l-0.04,0.3l-0.47,0.77l-0.06,0.59l0.05,0.32l0.01,0.35l0.35,0.96
				l0.09,0.76l0.25,0.32l0.42,0.12l0.08,0.12l-0.12,0.24l0.13,0.17l-0.02,0.43l0.26,0.43l0.09,0.53l-0.07,0.64l0.2,0.08l-0.4,0.34
				l-0.03,0.19l0.29,1l-0.07,0.17l-0.07,0.3l0.03,0.11l0.09,0.03l-0.09,0.22l-0.2,0.06l-0.08-0.13l-0.12,0.09l-0.03,0.14l-0.13,0.04
				l0.06,0.09l-0.05,0.17l-0.17,0.25l-0.59,0.42l-0.42,0.54l0.17,0.22l-0.1,0.09l0.36,0.22L316,143l-0.14,0.16l0.18,0.05l0.75-0.37
				l0.04,0.06l0.23-0.12l0.19,0.23l0.33-0.04l-0.12-0.1l0.05-0.14l-0.06-0.04l0.14-0.45l0.23-0.03l0.36-0.06l0.14,0.09l-0.05-0.13
				l0.13-0.07l0.18,0.11l0.35-0.02l0.11-0.12l0.21,0.09l-0.03,0.11l0.25-0.05l0.15,0.17l0.39,0.22l0.13-0.03l0.03-0.23l0.41-0.25
				l0.3-0.3l0.17,0.01l0.43-1.18l0.59-0.07l0.68,0.22l0.17-0.24l0.05-0.31l0.16-0.13l-0.01-0.11l0.3-0.19l-0.04-0.15l0.2-0.38
				l0.41-0.28l-0.3-0.47l-0.24-0.13l-0.07-0.38l-0.16-0.19l0.09-0.56l0-0.55l0.11-0.29l0.17-0.18l-0.11-0.01l-0.01-0.11l-0.53-0.39
				l-0.13-0.32l0.03-0.39l0.22-0.26l0.19,0.03l0.2-0.2l0.1,0.06l0.12-0.04l0.11-0.31l-0.05-0.33l-0.09-0.02l0.35-0.66l-0.26,0.13
				l-0.28-0.08l0.05-0.07l0.18,0.03l0.18-0.08l-0.11-0.39l0.03-0.31l-0.32-0.39l-0.24-0.72l-0.27-0.33l-0.03-0.22l-0.14-0.08
				l-0.27,0.09l-0.22-0.04l-0.05-0.23l0.16-0.14l0.01-0.1l-0.14,0l-0.01-0.29l-0.14-0.23l-0.41-0.09l-0.15,0.48l-0.27-0.4
				l-0.01-0.81l-0.21-0.68l0.5-0.13l0.14,0.06l-0.04-0.44l0.1-0.2l-0.24-0.27l0.05-0.28L321.22,126.79L321.22,126.79z"></path>
		</g>
		<g id="medina_campo_rural">
			<path class="st3 fase_0" d="M341.29,142.78l-0.22-0.43l0-0.25l-0.25-0.35l-0.2-0.02l-0.05,0.1l-1.28,0.85l-0.3-0.24l-0.18-0.03l-0.4-1
				l0.7-0.52l0.32-0.55l0.06-0.23l-0.05-0.33l-0.24,0.16l-0.38,0.01l-0.38-0.09l-0.54-0.62l-0.12,0.04l-0.3-0.23l-0.29-0.49
				l0.05-0.58l0.27-0.17l0.12-0.33l-0.17-0.5l-0.1-0.69l0.22-1.09l-0.24-0.63l0.29-0.54l-0.05-0.11l0.03-0.5l-0.06-0.63l-0.21-0.65
				l-0.01-0.51l-0.6-0.09l-0.19-0.15l-0.32,0.11l-0.36,0.01l-0.35-0.27v0l-0.43,0.01l0.31-0.66l0.11-0.09l0.02-0.17l-0.08-0.71
				l-0.22-0.12l-0.26-0.36l-0.6,0.33l0.1,0.14l-0.17,0.62l-0.01,0.55l-0.09,0.05l-0.1,0.53l-0.54,0.36l-0.32,0.36l-0.37,0.22
				l-0.08-0.06l-0.42,0.03l-0.18-0.09l-0.04,0.08l0,0l-0.18-0.33l-0.17-1.6l-0.09-0.24l-0.58-0.52l0.23-0.42l-0.41-0.42l-0.13-0.07
				l-0.22,0.03l-0.18-0.18l-0.17-0.56L330,127.9l-0.97-0.02l-0.56,0.08l-0.18-0.24l-0.23,0.34l-0.36,0.05l-0.69-0.61l0,0l-0.32,0.14
				l-0.81,0.7l0.03,0.23l0.2,0.33l-0.12,0.7l0.12,0.11l0.09,0.29l-0.42,0.08l-0.06,0.16l-0.36-0.06l-0.25,0.29l-0.52,0l-0.28-0.14
				l-0.1,0.03l0.05,0.1l-0.25,0.09l-0.1-0.05l-0.15,0.12l-0.28,0.76l-0.21-0.29l-0.36-0.07l0.27,0.33l0.24,0.72l0.32,0.39
				l-0.03,0.31l0.11,0.39l-0.18,0.08l-0.18-0.03l-0.05,0.07l0.28,0.08l0.26-0.13l-0.35,0.66l0.09,0.02l0.05,0.33l-0.11,0.31
				l-0.12,0.04l-0.1-0.06l-0.2,0.2l-0.19-0.03l-0.22,0.26l-0.03,0.39l0.13,0.32l0.53,0.39l0.01,0.11l0.11,0.01l-0.17,0.18
				l-0.11,0.29l0,0.55l-0.09,0.56l0.16,0.19l0.07,0.38l0.24,0.13l0.3,0.47l-0.41,0.28l-0.2,0.38l0.04,0.15l-0.3,0.19l0.01,0.11
				l-0.16,0.13l-0.05,0.31l-0.17,0.24l-0.68-0.22l-0.59,0.07l-0.43,1.18l-0.17-0.01l-0.3,0.3l-0.41,0.25l-0.03,0.23l-0.13,0.03
				l0.07,0.17l-0.03,0.18l0.14,0.1l-0.03,0.16l0.08,0.01l-0.04,0.09l0.09,0.32l0.2,0.16l-0.2,0.29l0.17,0.14l0.42-0.39l0.03,0.1
				l0.27-0.08l0.17,0.18l0.21,0.06l0.25-0.25l0.14,0.04l0.08-0.07l0.12,0.11l-0.08,0.04l0.03,0.08l0.14-0.01l0,0.11l0.34,0.06
				l0.18,0.19l0.52,0.29l0.15,0.19l0.36,0.2l-0.01,0.15l0.17,0.27l0.16,0.13l0.14,0.09l0.04-0.19l0.45-0.04l0.06,0.07l0.37-0.41
				l0.09-0.24l0.67,0.23l0.31,0.19l0.32,0.06l0.33-0.01l0.41-0.2l1.13-0.6l-0.09-0.17l0.4-0.29l0-0.11l0,0l0.12,0.12l0.14,0
				l0.12,0.16l0.6-0.1l0.08,0.26l0.21,0.06l-0.08,0.14l0.29,0.06l0.39-0.06l0.12,0.14l0.07,0.52l0.2,0.06l0.29-0.09l0.47,0.48
				l0.33,0.14l0.29,0.28l0.49,0.24l0.11,0.05l0.16,0l0.13,0.23l0.31,0.07l0.53,0.47l0.08-0.04l0.18,0.1l0.17,0.17l-0.1,0.06
				l0.36,0.28l0.33-0.36l1.12,0.09l0.53-0.23l-0.16-0.24l0.08-0.47l0.19-0.52l-0.05-0.16l0.31-0.31l0.39-0.03l0.29-0.22l0.11-0.2
				l0.3-0.08l0.47,0.14l0.74-0.4l0.4,0.04l0.07-0.08l0.34-0.02l0.28,0.08l0.03-0.41l0.5-0.54l0.27-0.41L341.29,142.78z
				 M335.11,131.26L335.11,131.26L335.11,131.26L335.11,131.26z"></path>
		</g>
		<g id="penafiel">
			<path id="path1944" class="st3 fase_0" d="M363.23,113.3l-0.71,0.36l-0.21,0.19l-0.23-0.03l-0.52,0.5l0.1,0.32l-0.22,0.1l-0.63,0.02
				l-0.26,0.13l-0.29,0.15l-0.15-0.01l-0.39,0.3l-0.23,0.04l-0.29,0.04l-0.11,0.13l-0.07-0.21l-0.2,0.03l-0.37,0.19l-0.27-0.16
				l-0.19,0.06l-0.48-0.08l-0.2-0.35l-0.41-0.06l-0.33,0.15l-0.31,0.05l-0.08-0.07l-0.41,0.11l0.33,0.38l0.28,0.09l0.18,0.48
				l0.26,0.26l-0.09,0.17l-0.06,0.34l0.19,0.1l0.07,0.22l-0.05,0.28l-0.09,0.02l-0.08,0.16l0,0.08l0.11,0l0.08,0.63l0.24-0.2
				l0.33,0.13l-0.12,0.24l0.06,0.32l0.37,0.47l0.18-0.19l0.14-0.83l0.38,0.25l-0.35,0.78l-0.17,0.11l-0.02,0.32l-0.24,0.82
				l-0.18,0.1l-0.22-0.07l-0.37,0.11l-0.16-0.05l-0.33,0.07l0.47,0.31l-0.06,0.23l-0.17,0.04l-0.49,0.67l-0.08,0.05l-0.12-0.14
				l-0.14,0.06l-0.01,0.2l-0.28,0.33l-0.34,0.12l-0.41-0.33l-0.31,0.04l0.43,0.36l-0.03,0.4l-0.41,0l-0.13,0.17l-0.32,0.08
				l-0.21-0.06l-0.05-0.16l-0.19,0.27l-0.32-0.04l-0.11,0.26l0.16,0.08l-0.06,0.09l0.13,0.18l-0.02,0.2l0.31,0.3l0.15,0.41
				l-0.1,0.25l0.05,0.27l-0.23,0.18l-0.15,0.26l-0.07,0.37l-0.42,0.41l-0.33,0.62l-0.09,0.13l0.04,0.08l0.41,0.24l0.54-0.51
				l0.11-0.01l0.58,0.51l0.31-0.1l0.51,0.33l-0.02,0.21l0.14,0.42l0.18,0.04l0.38-0.07l0.07,0.13l0.37-0.05l0.23,0.21l0.15,0.01
				l0.3-0.2l0.05-0.26l0.75-0.24l0.26,0.15l0.16-0.02l0.1-0.16l0.55,0.02l0.16-0.18l0.23,0.27l0.13,0.3l0.05-0.31l0.51-0.31
				l0.16,0.07l0.39,0.28l0.23-0.23l1.23-0.6l0.77-0.23l0.36-0.02l0.45-0.34l0.64,0.65l0.24-0.38l0.55-0.18l0.57-0.16l0.34-0.19
				l0.87-0.46l0.17,0.02l0.38,0.01l0.29-0.14l0.06-0.18l0.16-0.11l0.16-0.28l0.49-0.11l0.37-0.42l0.37-0.05l0.28,0.06l0.04,0.09
				l0.15-0.07l0.05,0.08l0.23-0.02l0.17-0.27l0.08-0.54l0-0.45l-0.01-0.89l0.34-0.48l0.19-0.52l0-0.48l-0.12-0.14l-0.06-0.25l0-0.42
				l-0.2-0.01l-0.1-0.11l-0.02-0.44l-0.64-0.07l0.01-0.1l0.18-0.12l0.07-0.05l-0.15-0.23l-0.23,0.13l-0.2-0.18l-0.24,0.23
				l-0.25,0.09l-0.14-0.15l-0.13-0.19l0.02-0.27l0.18-0.26l0.06-0.75l0.07-0.68l0.06-0.13l0.06-0.86l-0.07-0.91l-0.53-0.77
				l-0.12-0.24l-0.37-0.31l-0.07-0.14l-0.24-0.09L367,113.7l-0.33,0.02l-1.48,0.42l-0.3-0.69l-0.34,0.2l-0.42,0.07l-0.21,0.21
				l-0.28-0.17L363.23,113.3z"></path>
		</g>
		<g id="laguna_de_duero">
			<path class="st3 fase_0" d="M342.5,121.46l-0.28-0.7l-0.61,0.08l-0.54,0.13l-0.62-0.29l-0.19,0.04l-0.19,0.18l-0.68,0.21l-0.25,0.41
				l-0.06,0.31l0.06,0.41l-0.15,0.53l-0.55,0.16l0.14,0.26l0.11,0.03h0l-0.09,0.1l-0.19-0.04l-0.19,0.53l0.4,0.18l0.16,0.1
				l-0.01,0.13l-0.77,0.12l0.05,0.21l-0.2,0.07l-0.09-0.08l-0.13,0.06l-0.11-0.13l-0.18-0.04l-0.5,0.04l-0.03,0.08l0.4,0.07
				l0.33,0.11l0.4,0.49l0.26,0.6l0.22,0.28l0.36,0.12l0.22-0.09l0.47-0.16l1-0.57l0.69-0.97l0.47,0.07l0.36-0.18l0.32-0.61
				l0.11-0.09l-0.04-0.1l-0.31,0.01l-0.04-0.23l0-0.55l0,0l0.22-0.19l0.21,0.18l0.14-0.05l0.02-0.31L342.5,121.46z"></path>
		</g>
		<g id="pisuerga">
			<path class="st3 fase_0" d="M338.59,119.41l0.32-0.21l-0.14-0.02l-0.22-0.2l-0.22-0.43l-0.06,0.19l-0.45-0.18l-0.7,0.24l-0.14-0.15
				l-2.94-1.45l-0.36,0.21l-0.55,0.48l-0.09,0.2l-0.17,0.03l-0.01,0.23l0,0l-0.12,0.42l-0.09,0.18l0.12,0.52l0.17,0.15l0.34,0.14
				l-0.08-0.23l0.1-0.1l0.17,0.14l0.34-0.06l0.08,0.11l0.14,0.48l-0.06,0.12l0.05,0.12l-0.16,0.05l-0.11,0.18l-0.18,0.06l-0.06-0.07
				l-0.27,0.15l0.11,0.78l-0.08,0.22l-0.34,0.31v0.6l-0.08,0.16l0.12,0.07l0.32,0.2l0.45,0.02l0.25,0.28l0.67-0.41l0.16,0.09
				l0.04-0.06l0.55-0.34l0.74-0.04l0.23-0.1l0.25,0.09l0.38-0.3l0.65,0.19l0.11-0.46l0.09-0.18l0.11-0.01l0.41-0.87l-0.43,0.23
				l-0.18-0.1l0.1-0.07l-0.38-0.84l0.71-0.23L338.59,119.41z M332.91,118.31l0.14-0.22l0.03,0.06L332.91,118.31z"></path>
		</g>
		<g id="valladolid_rural_ii">
			<path class="st3 fase_0" d="M338.93,117.76l-0.23-0.21l0.23-0.19l-0.01-0.27l-0.1,0.03l-0.12-0.12l-0.57,0.02l-0.48-0.47l-0.01-0.15
				l0.08-0.13l-0.03-0.1l0.16,0.02l0.05-0.23l-0.27-0.07l0.03-0.12l-0.2,0.07l-0.14-0.16l-0.34,0.16l-0.05-0.08l0.28-0.18l0.41-0.12
				l0.01-0.15l-0.17-0.12l-0.75-2.09l0,0l0.13-0.01l-0.76-1.7l-0.34-0.59l-0.33,0.04l-0.14-0.16l-0.11-1.1l0.12-0.41l-0.01-0.28v0
				l-0.39,0.22l-0.66,0.15l-0.01,0.18l-0.48,0.36l-0.84-0.11l-0.27-0.18l-0.37,0.1l-0.09,0.09l0.19,0.1l0.06,0.33l0.38,0.71
				l0.3,0.16l0.36,0.23l0.3,0.42l0.17,0.1l-0.15,0.67l-0.7,1.11l-0.56-0.16v0l-0.59-0.58l-0.31-0.51l0.25-0.5l-0.12-0.29l-0.53,0.06
				l-0.15,0.16l-0.14-0.03l-0.2-0.21l-0.14,0.1l-0.27-0.53v0l-0.53,0.21l-0.35,0.64l-0.64,0.57l-0.12,0.48l-0.1,0.1l-0.38,0.24
				l-0.54,0.07l0.08,0.64l0.43,0.29l0.43,0.48l0.07,0.67l0.19,0.5l0.24,0.2l0.16,0.01l0.44-0.12l0.26,0.18l0.95-0.11l0.27,0.05
				l0.34,0.54l-0.16,0.19l-0.33,0.09l-0.32,0.3l-0.23,0.46l-0.33,0.4l0.06,0.37l-0.2-0.06l-0.27,0.64l0.02,0.15l0.34,0.17l0.13-0.11
				l0.34,0.01l0.25-0.23l0.42-0.18l0.64,0.05l0.61-0.27l0.18-0.32l0.03-0.22l0.14-0.05l0.01,0l-0.01,0.23l0,0l-0.11,0.38
				c0.95-0.11,1.91-0.12,2.87-0.14c0.41-0.01,0.81-0.04,1.22-0.08l0.14,0.14l0.14,0.15l0.7-0.24l0.45,0.18l0.06-0.19l0.49-0.34
				l-0.06-0.06l0.04-0.19L338.93,117.76z M332.88,118.11l0.16-0.03l0,0l0,0L332.88,118.11z"></path>
		</g>
		<g id="circunvalacion">
			<path class="st3 fase_0" d="M344.16,115.56l-0.1-0.05l-0.21,0.09l-0.1,0.25l-0.14,0.06l-0.61-0.02l-0.18,0.15l-0.18,0.03l-0.3-0.08
				l-0.07,0.08l-0.41-0.07l0.03-0.09l-0.36-0.24l-0.06-0.18l0.05-0.29l0.41-0.02l0.14-0.07l0.06-0.14l-0.11-0.46l-0.16-0.22
				l-0.24-0.03l0,0l-0.47,0.25l-0.29,0.76l-0.07-0.04l-0.27,0.32l-0.63-0.25l-0.32,0.26l-0.57,0.07l-0.59-0.19l-0.49,0.07
				l-0.27-0.18l-0.01,0.15l-0.41,0.12l-0.28,0.18l0.05,0.08l0.34-0.16l0.14,0.16l0.2-0.07l-0.03,0.12l0.27,0.07l-0.05,0.23
				l-0.16-0.02l0.03,0.1l-0.08,0.13l0.01,0.15l0.48,0.47l0.57-0.02l0.12,0.12l0.1-0.03l0.01,0.27l-0.23,0.19l0.23,0.21l-0.12,0.2
				l-0.04,0.19l0.06,0.06l-0.49,0.34l0.22,0.43l0.22,0.2l0.14,0.02l-0.32,0.21l-0.4,0.5l-0.71,0.23l0.31,0.69
				c0.13-0.05,0.26-0.08,0.4-0.08c0.45-0.02,0.84,0.23,1.07,0.59l0.13-0.21l0.68-0.21l0.19-0.18l0.19-0.04l0.62,0.29l0.54-0.13
				l0.12-0.38l-0.15-0.5l0.07-0.23l0.52-0.33l0.11,0.01l0.06-0.12l0.41-0.11l0.08-0.1l0.45,0.03l0.29-0.05l-0.05-0.33l-0.28-0.46
				l-0.12-0.4l0.12-0.27l-0.01-0.21l-0.22-0.35l-0.05-0.26l0.18-0.05l0.12-0.3l0.28-0.13l0.06-0.09l-0.07-0.17l0.35-0.19l0.18,0.07
				l0.15-0.17L344.16,115.56z"></path>
		</g>
		<g id="huerta_del_rey">
			<path class="st3 fase_0" d="M338.62,119.39l-0.02,0.01l-0.4,0.5l-0.8,0.25l0.15,0.98c0.22-0.07,0.43,0.04,0.57,0.05
				c0.35,0.04,0.82-0.05,1.15,0.15l0.13-0.21l0.68-0.21l0.13-0.13C339.64,120.36,339.11,119.89,338.62,119.39z"></path>
		</g>
	</g>
	<g id="segovia">
		<g id="villacastin">
			<path class="st3 fase_0" d="M360.56,164.25l-0.18-0.25l-0.11-0.04l-0.5,0.18l-0.1-0.18l0.16-0.1l-0.11-0.32l0.07-0.18l0.03-0.29
				l0.51-0.62l-0.38-0.49l-0.22-0.72l-0.07-0.04l-0.15,0.16l-0.12-0.01l-0.04-0.14l-0.15-0.03l-0.15,0.11l-0.1-0.08l-0.58,0.38
				l-0.12-0.07l-0.2,0.05l-0.17-0.14l-0.43,0.23l-0.03-0.13l-0.25-0.22l0,0.1l-0.13,0.08l-0.13-0.19l0,0.24l-0.09-0.02l-0.15,0.12
				l-0.5-0.38l0.14-0.01l-0.01-0.19l-0.04-0.11l0.24-0.11l-0.21-0.48l-0.05,0.05l-0.27-0.14l-0.78-0.06l-0.13-0.2l-0.32,0.02
				l-0.54-0.04l-0.44,0.67l-0.38,0.02l-0.44-0.36l-0.33-0.16l-0.14-0.01l-0.16,0.11l-0.13,0.23l-0.22,0.83l0.15,0.35l-0.04,0.2
				l0.05,0.09l-0.12,0.06l-0.4-0.09l-0.23,0.18l-0.44,0.7l-0.1,0.5l-0.11,0.2l-0.1,0.01l-0.01-0.01c-0.01,0.04-0.02,0.09-0.04,0.14
				c-0.07,0.21-0.16,0.41-0.26,0.61c0,0,0,0,0,0l0,0.01l0.33,0.22l-0.05,0.4l0.27,0.3l0.29,0.55l0.22,0.23l0.19,0.08l0.45,0.18
				l0.3,0.39l0.02,0.33l0.15,0.36l-0.13,0.08l-0.09,0.59l-0.14,0.07l0.04,0.52l0.11,0.11l0.4,1.55l0.12,1.09l-0.01,1.16l0.5-0.05
				l0.54,0.06l0.22-0.09l0.97-0.14l0.36-0.57l0.02-0.23l-0.08-0.07l0.09-0.37l-0.19-1.16l0.09-0.16l0.17-0.95l-0.07-0.28l0.74-0.84
				l0.59-0.23l1.12-0.98l0.5-0.53l0.15-0.41l0.08,0.1l0.28-0.22l0.44-0.61l0.39-0.03l0.13,0.17l0.15,0.06l0.23-0.29l0.45-0.16
				l0.12-0.19L360.56,164.25z"></path>
		</g>
		<g id="sepulveda">
			<path id="path3233" class="st3 fase_0" d="M378.79,126.6l-0.44,0.04l-0.34,0.07l-0.17-0.07l-0.68,0.13l-0.28,0.13l0.71,1.15
				l0.48,1.13l0.08,0.37l-0.02,0.09l-0.35,0.14l-0.47-0.13l-0.18,0.68l0.1,0.36l-0.09,0.46l-0.12,0.01l-0.49,0.44l0.04,0.08l0.21,0
				l0.03,0.09l-0.13,0.31l-0.15,0.06l0,0.13l-0.59,0.28l-0.13-0.08l-0.12,0.06l-0.91,0.81l-0.05,0.4L374.4,134l0.24,0.34l0.41,0.13
				l1.07,0.74l-0.43,0.02l-0.06-0.08l-0.02,0.16l-0.41,0.15l-0.04,0.11l-0.31,0.2l0.02,0.12l0.11-0.12l0.1,0l-0.11,0.11l0.1,0.14
				l-0.1,0.14l0.27-0.04l0.14,0.11l-0.01,0.45l-0.17,0l-0.07,0.08l0.12,0.06l0.21-0.07l-0.01,0.1l0.34-0.09l0.26,0.25l0.17-0.08
				l0.41-0.01l-0.08,0.17l0.22,0.56l-0.34,0.46l-0.22-0.09l-0.31,0.13l0.04,0.07l-0.1,0.11l0.06,0.11l0.04,0.07l0.09-0.07l0.02,0.08
				l0.18-0.06l0.04,0.11l0.17,0.08l0,0.12l0.13,0.05l0.06-0.05l0.36,0.1l0.27-0.09l0.07,0.06l0.64-0.43l0.81,1.54l-0.09,0.15
				l-0.13,0.27l-0.15,0.72l-0.42,0.44l-0.04,0.25l0.32,0.32l0.11,0.78l0.22,0.44l0.75-0.03l0.19,0.12l0.35,0.02l0.26,0.17l0.28,0.03
				l-0.02-0.3l0.02-0.6l0.4-0.59l0.31-0.07l0.32-0.38l0.31,0.3l0.31,0.13l0.37,0l0.19,0.11l0.08-0.03l0.05,0.11l0.26-0.43l0.35-0.06
				l0.31,0.08l0.05,0.32l0.51-0.12l0.15-0.26l0.71-0.49l-0.05-0.28l-0.16-0.15l-0.02-0.17l0.06-0.46l0.27-0.53l0.53-0.06l-0.15-0.16
				l-0.13-0.02l-0.17-0.22l0.41-0.27l-0.27-0.24l0.08-0.24l-0.22-0.21l-0.23-0.39l-0.68-0.16l0-0.67l0.13-0.24l0.49-0.26l-0.01-0.22
				l0.76-0.04l0.33,0.16l0.2,0.2l0.31,0.1l0.11-0.04l0.48,0.41l0.04,0.11l0.49,0.25l-0.18-0.33l0.18-0.25l0.14-0.07l-0.03-0.22
				l0.08-0.11l-0.12-0.06l-0.48-0.93l-0.1-0.05l-0.1-0.14l-0.08-0.51l-0.15-0.27l-0.57-0.28l-0.14-0.4l0.09-0.34l-0.11-0.16
				l0.51-0.3l0-0.28l0.23-0.29l0.19-0.12l0.54-0.25l0.33,0.2l0.12-0.34l-0.2-0.23l0.11-0.06l0.15-0.05l0.03-0.33l-0.04-0.48
				l-0.13-0.24l-0.24-0.17l-0.49-0.17l-0.58-0.15l-0.21,0.51l-0.38,0.1l-0.43,0.12l-0.27-0.12l-0.3-0.13l-0.33-0.02l-0.78-0.35
				l-0.33-0.03l-0.44-0.27l0.14-0.19l-0.11-0.07l-0.02-0.11l0.1-0.34l0.21-0.14l0.05,0.11l0.01-0.11l0.34-0.11l-0.06-0.34l0.11,0
				l0.04-0.08l-0.08-0.12l-0.35,0.25l-0.41-0.3l0.03-0.19l0.18-0.26l-0.09-0.08l0.02-0.21l-0.43-0.21l-0.22-0.23l-0.31,0.18
				l-0.33-0.03l-0.13-0.13l-0.11,0.09l-0.49-0.21l-0.56-0.55l-0.04,0.12l0.22,0.25l-0.35,0.5l0.13,0.66l-0.42,0.37l-0.19-0.55
				l-0.55-0.02l-0.49,0.07l-0.07-0.56l0.05-0.63L378.79,126.6L378.79,126.6z"></path>
		</g>
		<g id="navafria">
			<path id="path3183" class="st3 fase_0" d="M388.68,141.25l0.14,0.02l0.07,0.14l-0.05,0.28l-0.16,0.13l-0.11,0.06l-0.08-0.2
				l-0.17-0.03l-0.62,0.38l-0.79-0.49l-0.24,0.2l-0.28-0.04l-0.18-0.15l-0.35,0.13l-0.2,0l-0.15-0.04l-0.11-0.15l-0.26-0.06
				l-0.71,0.49l-0.15,0.26l-0.51,0.12l-0.05-0.32l-0.31-0.08l-0.35,0.06l-0.26,0.43l-0.05-0.11l-0.08,0.03l-0.19-0.11l-0.37,0
				l-0.31-0.13l-0.31-0.3l-0.32,0.38l-0.31,0.07l-0.4,0.59l-0.02,0.6l0.02,0.3l-0.28-0.03l-0.26-0.17l-0.35-0.02l-0.19-0.12
				l-0.75,0.03l-0.22-0.44l-0.11-0.78l-0.32-0.32l-0.38,0.09l-0.15-0.07l-0.18,0.23l-0.08-0.03l-0.17,0.08l-0.09-0.13l-0.21-0.01
				l-0.16,0.11l-0.04,0.31l-0.17,0.43l0.04,0.35l-0.08,0.08l-0.28,0.05l-0.81,0.43l0,0.36l-0.13,0.25l-0.3,0.28l-0.59,0.01
				l-0.2,0.31l0,0.3l0.2,0.2l0.19,0.39l0.22,0.05l0.05,0.18l-0.21,0.23l-0.05,0.23l-0.36,0.29l-0.53,0.62l-0.2,0.87l0.06-0.01
				l-0.16,0.58l-0.01,0.18l-0.11-0.09l0.03,0.14l-0.06-0.14l-0.14-0.02l-0.05-0.09l-0.47,0.07l-0.1,0.11l0.05,0.07l-0.15,0.15
				l-0.19,0.09l-0.14-0.1l-0.06,0.25l-0.09,0.01l-0.06-0.09l-0.09,0.23l-0.27,0.08l-0.77-0.46l-0.86-0.11l-0.04,0.05l-0.25,0.42
				l-0.3,0.28l0.03,0.05l0.35-0.13l0.38,0.24l0.63-0.05l0.3,0.24l0.18,0.52l0.37,0.31l0.14,0.22l0.09,0.38l0.09,0.09l-0.13,0.24
				l0.02,0.16l0.87,0.38l0.13,0.26l1.65,1.61l0.4-0.2l0.03-0.09l0.25-0.03l0.19,0.2l0.39-0.4l0.46-0.06l0.21-0.16l0.52-0.18
				l0.32,0.08l0.3-0.08l0.48-0.33l0.86-0.42l0.43-0.65l0.16-0.21l0.02-0.15l0.32-0.43l0.14-0.04l0.24-0.39l0.42-1.01l0.39-0.45
				l0.18-0.03l0.19-0.15l0.18-0.31l0.41-0.32l0.43-0.3l0.3-0.11l0.29,0.05l0.15-0.12l0.19-0.29l0.87-0.83l0.29-0.71l0.38-0.28
				l0.09-0.44l0.5-0.47l0.24-0.34l0.54-0.33l0.35,0.05l0.07-0.21l0.27-0.15l0.58,0.09l0.47-0.27l0.72-0.03l0.51-0.06l0.45,0.2
				l-0.13-0.31l0.02-0.59l-0.14-0.7l-0.31-0.37l0.3-0.32l0.11-0.45L388.68,141.25L388.68,141.25z"></path>
		</g>
		<g id="sacramenia">
			<path id="path3215" class="st3 fase_0" d="M371.22,121.25l-0.37,0.13l-0.31,0.01l0,0.48l-0.19,0.51l-0.34,0.48l0.01,0.89l0,0.45
				l-0.03,0.01l-0.06,0.52l-0.17,0.27l-0.23,0.02l-0.05-0.08l-0.15,0.07l-0.04-0.09l-0.28-0.06l-0.37,0.05l-0.37,0.42l-0.49,0.11
				l-0.16,0.28l-0.16,0.11l-0.06,0.18l-0.29,0.14l-0.38-0.01l0.11,0.32l0.18,0.08l0.77-0.03l0.42,0.35l0.41,0.03l0.16-0.1l0,0.31
				l0.1,0.03l-0.04,0.28l-0.1,0.12l1.53-0.02l0,0.77l-0.31,0.84l0,0.25l-0.07,0.04l0.01,0.26l0.31,0.18l0.39,0.03l0.22,0.25
				l0.25-0.14l0.28,0.03l0.08,0.06l-0.03,0.12l0.03,0.08l0.13,0l0.21-0.24l0.25-0.11l0.23,0.01l0.14,0.45l0.14,0.14l0.79,0.23
				l0.14,0.05l0.47,0.42l0.15-0.11l0.15,0.27l-0.02,0.06l0.59,0.56l0.2,0l0.7,0.49l0.12-0.06l0.13,0.08l0.59-0.28l0-0.13l0.15-0.06
				l0.13-0.31l-0.03-0.09l-0.21,0l-0.04-0.08l0.49-0.44l0.12-0.01l0.09-0.46l-0.1-0.36l0.18-0.68l0.47,0.13l0.35-0.14l0.02-0.09
				l-0.08-0.37l-0.48-1.13l-0.71-1.15l-0.42-0.99l-0.33-0.05l-0.42-0.64l-0.44-0.88l-0.17-0.19l-0.29-0.12l-0.31,0.04l-0.12,0.09
				l-0.04-0.22l-0.32-0.84l-0.45-0.26l-0.36-0.35l-0.57-0.28l-0.55-0.02l-0.44-0.23l-0.3-0.24L371.22,121.25L371.22,121.25z
				 M368.9,132.98l-1.44,1.85l-0.35,0.75l1.25,1.19l0.12,0.09l0.49-0.69l0.18-0.43l0.18-0.02l0.27-0.84l0.75-0.36l0.15-0.32
				l0.12-0.04l0.4-0.45l1.12,0.07l-0.24-0.22l-0.5-0.46l-0.47-0.06l-0.09,0.4l-0.41-0.01l-0.58-0.13l-0.7,0.03l-0.25-0.17
				L368.9,132.98L368.9,132.98z"></path>
		</g>
		<g id="fuentesauco">
			<path id="path3227" class="st3 fase_0" d="M366.57,126.13l-0.87,0.46l-0.34,0.19l-0.57,0.16l-0.55,0.18l-0.24,0.38l-0.64-0.65
				l-0.45,0.34l-0.36,0.02l-0.77,0.23l-1.23,0.6l0.54,0.46l0.38,0.73l0.08,0.38l-0.16,0.67l0.06,0.22l0.18,0.11l-0.08,0.08
				l-0.02,0.52l0.02,0.21l0.13,0.01l0.73-0.18l0.37,0.05l-0.04,0.32l0.11,0.14l0.05-0.05l0.01,0.21l0.26,0.37l0.02,0.3l-0.09,0.43
				l-0.17,0.46l0.1,0.19l-0.13,0.55l-0.18-0.01l-0.2,0.15l0.1,0.37l0.41,0.25l0.21,0.04l0.22,0.27l0.36,0.12l0.36,0.04l0.2-0.05
				l0.11-0.14l0.2-0.02l0.76,0.29l-0.05,0.11l0.33,0.37l0.26,0.1l0.4,0.46l0.59-1.15l0.13,0.13l0.35-0.75l1.44-1.85l-0.01-0.34
				l0.12-0.06l-0.16-0.89l0.05-0.26l0.43-0.21l-0.03-0.44l-0.31-0.19l0.22-0.72l-0.04-0.07l0.21-0.26l0.57-0.15l0-0.25l0.31-0.84
				l0-0.77l-1.53,0.02l0.1-0.12l0.04-0.28l-0.1-0.03l0-0.31l-0.16,0.1l-0.41-0.03l-0.42-0.35l-0.77,0.03l-0.18-0.08l-0.11-0.32
				L366.57,126.13L366.57,126.13z"></path>
		</g>
		<g id="cantalejo">
			<path id="path3167" class="st3 fase_0" d="M369.96,129.38l-0.57,0.15l-0.21,0.26l0.04,0.07l-0.22,0.72l0.31,0.19l0.03,0.44
				l-0.43,0.21l-0.05,0.26l0.16,0.89l-0.12,0.06l0.01,0.34l0.03,0.16l0.25,0.17l0.7-0.03l0.58,0.13l0.41,0.01l0.09-0.4l0.47,0.06
				l0.5,0.46l0.24,0.22l-1.12-0.07l-0.4,0.45l-0.12,0.04l-0.15,0.32l-0.75,0.36l-0.27,0.84l-0.18,0.02l-0.18,0.43l-0.49,0.69
				l-0.12-0.09l-0.13,0.32l0.18-0.03l-0.07,0.2l0.08,0.09l-0.08-0.01l0.18,0.3l-0.19,0.09l0.03,0.13l-0.09,0.11l0.07,0.07l0.31-0.08
				l0.1,0.09l0.2-0.03l0.24,0.15l0.09,0.49l-0.25,1.43l-0.22,0.04l-0.52-0.15l-0.08-0.09l-0.13-0.01l-0.04,0.09l-0.35,0.59
				l-0.42,0.37l-1.26,1.65l0.07,0.4l0.28,0.65l0.18,0.34l0.19,0.08l-0.21,0.36l0.21,0.28l-0.43,0.49l0.12,0.19l-0.06,0.33l0.23,0.2
				l0.19,0.39l-0.16,0.43l0.39,0.01l0.18,0.54l-0.2,0.06l-0.02,0.12l-0.48,0.36l-0.1,0.39l-0.18,0.2l0.26,0.21l0.18,0.01l0.2,0.4
				l0.66-0.18l0.26,0.12l0.21,0l0.35,0.31l0.19,0.32l0.27,0.1l0.03,0.21l0.23,0.06l0.3-0.28l0.25-0.42l0.04-0.05l0.86,0.11
				l0.77,0.46l0.27-0.08l0.09-0.23l0.06,0.09l0.09-0.01l0.06-0.25l0.14,0.1l0.19-0.09l0.15-0.15l-0.05-0.07l0.1-0.11l0.47-0.07
				l0.05,0.09l0.14,0.02l0.06,0.14l-0.03-0.14l0.11,0.09l0.01-0.18l0.16-0.58l-0.06,0.01l0.2-0.87l0.53-0.62l0.36-0.29l0.05-0.23
				l0.21-0.23l-0.05-0.18l-0.22-0.05l-0.19-0.39l-0.2-0.2l0-0.3l0.2-0.31l0.59-0.01l0.3-0.28l0.13-0.25l0-0.36l0.81-0.43l0.28-0.05
				l0.08-0.08l-0.04-0.35l0.17-0.43l0.04-0.31l0.16-0.11l0.21,0.01l0.09,0.13l0.17-0.08l0.08,0.03l0.18-0.23l0.15,0.07l0.38-0.09
				l0.04-0.25l0.42-0.44l0.15-0.72l0.13-0.27l0.09-0.15l-0.81-1.54l-0.64,0.43l-0.07-0.06l-0.27,0.09l-0.36-0.1l-0.06,0.05
				l-0.13-0.05l0-0.12l-0.17-0.08l-0.04-0.11l-0.18,0.06l-0.02-0.08l-0.09,0.07l-0.04-0.07l-0.06-0.11l0.1-0.11l-0.04-0.07
				l0.31-0.13l0.22,0.09l0.34-0.46l-0.22-0.56l0.08-0.17l-0.41,0.01l-0.17,0.08l-0.26-0.25l-0.34,0.09l0.01-0.1l-0.21,0.07
				l-0.12-0.06l0.07-0.08l0.17,0l0.01-0.45l-0.14-0.11l-0.27,0.04l0.1-0.14l-0.1-0.14l0.11-0.11l-0.1,0l-0.11,0.12l-0.02-0.12
				l0.31-0.2l0.04-0.11l0.41-0.15l0.02-0.16l0.06,0.08l0.43-0.02l-1.07-0.74l-0.41-0.13L374.4,134l0.32-0.28l0.05-0.4l0.91-0.81
				l-0.7-0.49l-0.2,0l-0.59-0.56l0.02-0.06l-0.15-0.27l-0.15,0.11l-0.47-0.42l-0.14-0.05l-0.79-0.23l-0.14-0.14l-0.14-0.45
				l-0.23-0.01l-0.25,0.11l-0.21,0.24l-0.13,0l-0.03-0.08l0.03-0.12l-0.08-0.06l-0.28-0.03l-0.25,0.14l-0.22-0.25l-0.39-0.03
				l-0.31-0.18l-0.01-0.26L369.96,129.38L369.96,129.38z"></path>
		</g>
		<g id="cuellar">
			<path id="path3173" class="st3 fase_0" d="M359.75,127.91l-0.51,0.31l-0.05,0.31l-0.13-0.3l-0.23-0.27l-0.16,0.18l-0.55-0.02
				l-0.1,0.16l-0.16,0.02l-0.26-0.15l-0.75,0.24l-0.05,0.26l-0.3,0.2l-0.15-0.01l-0.23-0.21l-0.37,0.05l-0.07-0.13l-0.38,0.07
				l-0.18-0.04l0.04,0.28l-0.14,0.47l-0.12-0.17l-0.2,0.03l-0.16-0.16l-0.09,0.01l-0.13,0.18l-0.25-0.12l-0.1,0.14l-0.12,0
				l-0.01-0.1l-0.55-0.12l-0.42,0.16l-0.04-0.05l-0.27,0.38l-0.18-0.06l-0.05,0.06l-0.02,0.09l-0.12,0.03l0.17,0.1l-0.23,0.08
				l-0.18-0.03l-0.11,0.11l-0.24,0.05l-0.03,0.08l-0.11-0.03l-0.02,0.13l-0.08,0.05l-0.13-0.05l-0.2,0.12l0.05,0.12l-0.14,0.01
				l0.62,1.05l-0.14,0.51l-0.17,0.05l0.21,0.13l0.05,0.2l-0.38,0.43l-0.25,0.1l-0.4,0.05l-0.28,0.15v0.06l0.16,0.47l0.18,0.24
				l0.14,0.51l0.01,0.47l-0.11,0.59l0.02,0.06l0.13,0.2l0.11,1.21l-0.15,0.23l-0.33,1.27l0.29,0.18l1.16,0.85l0.27,0.15l0.22,0
				l0.01,0.09l0.24,0.23l0.16,0.04l0.15,0.17l0.09,0.26l0.23-0.02l0.28,0.15l0.31-0.4l-0.05-0.29l0.29-0.69l0.3-0.11l0.24,0.08
				l-0.02,0.1l0.82,0.34l0.63,0.09l0.31,0.3l0.5,0.07l1.52,0.52l0.71-0.16l0.15-0.18l-0.03-0.13l0.08-0.13l0.23-0.04l0.07-0.2
				l0.36-0.2l0.19-0.25l0.36-0.18l0.46-0.29l0.02-0.5l0.3-0.22l0.46,0.01l0.32-0.19l0.62,0l0.94,0.21l0.36-0.42l-0.39-0.56
				l0.01-0.14l-0.13-0.03l-0.1-0.22l-0.1-0.05l-0.27,0.1l0.01-0.28l-0.1-0.16l-0.1-0.03l-0.11,0.1l-0.25-0.15l-0.09-0.21l0.37-0.25
				l0.92-0.3h0.31l-0.36-0.12l-0.22-0.27l-0.21-0.04l-0.41-0.25l-0.1-0.37l0.2-0.15l0.18,0.01l0.13-0.55l-0.1-0.19l0.17-0.46
				l0.09-0.43l-0.02-0.3l-0.26-0.37l-0.01-0.21l-0.05,0.05l-0.11-0.14l0.04-0.32l-0.37-0.05l-0.73,0.18l-0.13-0.01l-0.02-0.21
				l0.02-0.52l0.08-0.08l-0.18-0.11l-0.06-0.22l0.16-0.67l-0.08-0.38l-0.38-0.73l-0.54-0.46l-0.23,0.23l-0.39-0.28L359.75,127.91
				L359.75,127.91z"></path>
		</g>
		<g id="nava_de_la_asuncion">
			<path id="path3177" class="st3 fase_0" d="M348.76,137.64l-0.51,0.04l-1.19,0.6l-0.22-0.22l-0.33-0.1l-0.04,0.07l-0.53,0.18
				l-0.44,0.34L345.3,139l0.03,0.1l-0.37,0.53l-0.11,0.15l-0.35,0.38l-0.21,0.08l-0.33,0.3l-0.43,0.17l-0.12,0.14l0.18,0.7
				l-0.17,0.44l0.41,0.34l0.08,0.2l0.08-0.04l0.18,0.31l0.17,0.75l0.21,0.58l0.46,0.44l0.11,0.09l0.33,0.69l0.68,0.66l0.25,0.2
				l0.23,0.65l0.18,0.18l-0.13,0.27l0.18,0.39l-0.09,0.05l0.01,0.57l-0.15,0.29l0.29,0.71l0.22,0l-0.07,0.36l0.08,0.07l-0.12,0.09
				l-0.28,0.66l0.01,0.15l0.39,0.64l0.04,0.1l0.14,0.28l0.1-0.03l0.64,0.4l0.33-0.03l0.72,0.36l-0.04,0.08l0.19,0.42l-0.06,0.15
				l0.08,0.08l-0.19,0.51l0.06,0.22l-0.18,0.35l-0.43-0.04l-0.03-0.1l-0.24-0.04l-0.07,0.42l-0.16,0.02l-0.82,0.61l-0.15,0.32
				l-0.28,0.32l0.2,0.27l0.14,0.02l0.16-0.01l0.14-0.18l0.15-0.01l0.54,0.17l0.37,0.21l0.06-0.04l0.12,0.29l0.2-0.15l0.09-0.03
				l0.04,0.08l0.1-0.05l0.02-0.11l0.22-0.09l0.09,0.04l-0.03,0.11l0.15-0.08l0,0.08l0.1-0.14l0.27,0.16l0.13-0.1l0.36,0.07
				l-0.06-0.43l0.38-0.6l0.06-0.24l-0.11-0.14l0.07-0.12l0.14,0.05l0.05-0.23l0.41,0.28l-0.04,0.12l0.07,0.08l0.18,0l0.16,0.16
				l0.56-0.19l0.09,0.11l0.08-0.43l0.12-0.01l0.06-0.11l0.19-0.08l0.26,0l0.05-0.12l0.47,0.38l0.36-0.29l0.22,0.16l-0.04-0.3
				l0.07-0.36l-0.21-0.3l-0.01-0.16l0.53-0.3l0.07,0.04l0.58-0.11l0.12,0.09l0.88-0.29l0.06,0.06l0.23-0.04l0.04-0.73l0.23,0
				l0.44-0.17l0.03-0.08l0.53-0.24l-0.49-0.34l-0.07-0.59l0.05-0.49l0-0.11l-0.15-0.15l-0.94-0.2l-0.33-0.49l0.06-0.03l0.02-0.37
				l0.24-0.24l-0.02-0.13l0.18-0.14l0.05-0.58l0.24-0.11l-0.03-0.2l-0.21-0.17l0.11-0.17l0.58,0.14l0.32-0.39l-0.02-0.25l-0.17-0.34
				l0.18-0.04l-0.12-0.19l0.13-0.12l-0.05-0.04l-0.09,0.09l-0.06-0.05l-0.04-0.56l-0.07-0.02l0.02-0.15l-0.2-0.46l0.04-0.04
				l-0.14-0.12l-0.23-0.55l-0.16-0.16l0.14-0.33l-1.06-0.1l-0.28-0.39l0.62-0.8l-0.02-0.09l-0.28-0.21l-0.2-0.02l-0.18-0.15
				l-0.51,0.01l-0.19-0.22l0.02-0.12l-0.1-0.16l-0.37-0.16l0.19-0.32l-0.18-0.05l-0.1-0.32l-0.33,0.11l-0.09-0.14l-0.28-0.15
				l-0.23,0.02l-0.09-0.26l-0.15-0.17l-0.16-0.04l-0.24-0.23l-0.01-0.09l-0.22,0l-0.27-0.15l-1.16-0.85l-0.29-0.18l-0.07-0.09
				l-0.25-0.04l-0.28-0.42L348.76,137.64L348.76,137.64z"></path>
		</g>
		<g id="carbonero_el_mayor">
			<path id="path3179" class="st3 fase_0" d="M364.69,135.26l-0.2,0.02l-0.11,0.14l-0.2,0.05l-0.36-0.04h-0.31l-0.92,0.3l-0.37,0.25
				l0.09,0.21l0.25,0.15l0.11-0.1l0.1,0.03l0.1,0.16l-0.01,0.28l0.27-0.1l0.1,0.05l0.1,0.22l0.13,0.03l-0.01,0.14l0.39,0.56
				l-0.36,0.42l-0.94-0.21l-0.62,0l-0.32,0.19l-0.46-0.01l-0.3,0.22l-0.02,0.5l-0.46,0.29l-0.36,0.18l-0.19,0.25l-0.36,0.2
				l-0.07,0.2l-0.23,0.04l-0.08,0.13l0.03,0.13l-0.15,0.18l-0.71,0.16l-1.52-0.52l-0.5-0.07l-0.31-0.3l-0.63-0.09l-0.82-0.34
				l0.02-0.1l-0.24-0.08l-0.3,0.11l-0.29,0.69l0.05,0.29l-0.31,0.4l0.09,0.14l0.33-0.11l0.1,0.32l0.18,0.05l-0.19,0.32l0.37,0.16
				l0.1,0.16l-0.02,0.12l0.19,0.22l0.51-0.01l0.18,0.15l0.2,0.02l0.28,0.21l0.02,0.09l-0.62,0.8l0.28,0.39l1.06,0.1l-0.14,0.33
				l0.16,0.16l0.23,0.55l0.14,0.12l-0.04,0.04l0.2,0.46l-0.02,0.15l0.07,0.02l0.04,0.56l0.06,0.05l0.09-0.09l0.05,0.04l-0.13,0.12
				l0.12,0.19l-0.18,0.04l0.17,0.34l0.02,0.25l-0.32,0.39l-0.58-0.14l-0.11,0.17l0.21,0.17l0.03,0.2l-0.24,0.11l-0.05,0.58
				l-0.18,0.14l0.02,0.13l-0.24,0.24l-0.02,0.37l-0.06,0.03l0.33,0.49l0.94,0.2l0.15,0.15l0,0.11l0.64-0.63l0.23,0.27l0.22-0.03
				l0.12-0.12l0.34,0.02l0.09,0.12l0.28-0.16l0.01,0.13l-0.69,0.47l0.17,0.33l-0.12,0.07l0.09,0.32l0.15,0.17l-0.25,0.46l0.15,0.21
				l0.17-0.04l-0.06,0.27l0.1,0.09l0.63-0.53l0.26,0.26l0.13-0.01l0.75-0.66l0.86-0.6l0.03,0.06l0.08-0.25l0.16-0.25l0.14,0.03
				l0.05-0.06l0.24,0.09l0.48,0l0.15,0.17l0.17-0.13l0.3-0.01l0.3,0.14l0.6-0.3l0.05-0.06l0.28-0.75l0.14-0.02l0.55-0.82l0.67-0.19
				l0-0.07l0.33,0.1l0.18-0.2l0.1-0.39l0.48-0.36l0.02-0.12l0.2-0.06l-0.18-0.54l-0.39-0.01l0.16-0.43l-0.19-0.39l-0.23-0.2
				l0.06-0.33l-0.12-0.19l0.43-0.49l-0.21-0.28l0.21-0.36l-0.19-0.08l-0.18-0.34l-0.28-0.65l-0.07-0.4l1.26-1.65l0.42-0.37
				l0.35-0.59l0.04-0.09l0.13,0.01l0.08,0.09l0.52,0.15l0.22-0.04l0.25-1.43l-0.09-0.49l-0.24-0.15l-0.2,0.03l-0.1-0.09l-0.31,0.08
				l-0.07-0.07l0.09-0.11l-0.03-0.13l0.19-0.09l-0.18-0.3l0.08,0.01l-0.08-0.09l0.07-0.2l-0.18,0.03l0.13-0.32l-1.25-1.19
				l-0.13-0.13l-0.59,1.15l-0.4-0.46l-0.26-0.1l-0.33-0.37l0.05-0.11L364.69,135.26L364.69,135.26z"></path>
		</g>
		<g id="riaza">
			<path class="st3 fase_0" d="M401.79,136.22l-0.47-0.15l-0.67-0.43l0.02-0.63l-0.48,0.01l-0.86-0.62l-0.11-0.6l-0.36,0.04l-0.29-0.45
				l-0.59-0.15l-0.13-0.28l-0.94-0.43l-0.49-0.82l-0.38-0.18l0.14-0.11l0-0.4l-0.08-0.14l0.11-0.36l0.2-0.17l0.01-0.34l-0.19-0.31
				l0.24-0.83l0.05-0.46l0.35-0.42l-0.16-0.06l-0.03,0.09l-0.5,0.11l0.03,0.07l-0.07,0.02l-0.33-0.35l-0.27,0.04l0.02-0.17
				l-0.1-0.12l-0.09,0.12l-0.4,0.06l-0.28-0.08l-0.25-0.27l-0.53-0.28l-0.4-0.01l-0.43-0.35l-0.1-0.81l0.15-0.31l0.09-0.5
				l-0.15-0.22l-0.32-0.11l-0.41-0.3l-0.38-0.44l0.01-0.07l-0.18-0.05l-0.08-0.12l-0.25,0.49l-0.62,0.37l-0.22-0.17l-0.68-0.28
				l-0.68-0.55l-0.24-0.31l-0.41-0.19l-0.14-0.18l0.07-0.4l-0.29-0.68l0.09-0.3l-0.2-0.13l-0.26-0.03l0,0l-0.4,0.18l-0.15-0.14
				l-0.18,0.14l-0.14,0.27l0.13,0.2l-0.06,0.18l-0.04,0.39l0.22,0.16l0.02,0.17l-0.27,0.67l0.19,0.22l-0.22-0.04l0.02,0.12
				l-0.35-0.18l-0.03,0.22l-0.16-0.01l-0.85,1.06l-0.38,0.57l-0.25,0.15l-0.14,0.3l-0.35,0.17l-0.22,0.05l-0.17,0.16l-0.05-0.03
				l-0.1,0.28l-0.24,0.11l-0.76,0.59l0.43,0.21l-0.02,0.21l0.09,0.08l-0.18,0.26l-0.03,0.19l0.41,0.3l0.35-0.25l0.08,0.12L384,128.7
				l-0.11,0l0.06,0.34l-0.34,0.11l-0.01,0.11l-0.05-0.11l-0.21,0.14l-0.1,0.34l0.02,0.11l0.11,0.07l-0.14,0.19l0.44,0.27l0.33,0.03
				l0.78,0.35l0.33,0.02l0.3,0.13l0.27,0.12l0.43-0.12l0.38-0.1l0.21-0.51l0.58,0.15l0.49,0.17l0.24,0.17l0.13,0.24l0.04,0.48
				l-0.03,0.33l-0.15,0.05l-0.11,0.06l0.2,0.23l-0.12,0.34l-0.33-0.2l-0.54,0.25l-0.19,0.12l-0.23,0.29l0,0.28l-0.51,0.3l0.11,0.16
				l-0.09,0.34l0.14,0.4l0.57,0.28l0.15,0.27l0.08,0.51l0.1,0.14l0,0l0.1,0.05l0.48,0.93l0.12,0.06l-0.08,0.11l0.03,0.22l-0.14,0.07
				l-0.18,0.25l0.18,0.33l0,0l-0.49-0.25l-0.04-0.11l-0.48-0.41l-0.11,0.04l-0.31-0.1l-0.2-0.2l-0.33-0.16l-0.76,0.04l0.01,0.22
				l-0.49,0.26l-0.13,0.24l0,0.67l0.68,0.16l0.23,0.39l0.22,0.21l-0.08,0.24l0.27,0.24l-0.41,0.27l0.17,0.22l0.13,0.02l0.15,0.16
				l-0.53,0.06l-0.27,0.53l-0.06,0.46l0.02,0.17l0.16,0.15l0.05,0.28l0.26,0.06l0.11,0.15l0.15,0.04l0.2,0l0.35-0.13l0.18,0.15
				l0.28,0.04l0.24-0.2l0.79,0.49l0.62-0.38l0.17,0.03l0.08,0.2l0.11-0.06l0.16-0.13l0.05-0.28l-0.07-0.14l-0.14-0.02l0.52,0.04
				l-0.11,0.45l-0.3,0.32l0.31,0.37l0.14,0.7l-0.02,0.59l0.13,0.31l0.26-0.33l0.09-0.01l0.27-0.56l0.28,0.09l0.27-0.19l0.21,0.03
				l0.29-0.18l0.31,0.02l-0.08-0.16l0.09-0.14l0.25-0.12l0.26-0.59l0.2-0.26l0.2-0.46l0.77,0.06l0.21-0.12l0.44,0.25l0.27-0.1
				l-0.07-0.23l0.05-0.55l-0.13-0.29l0.34-0.52l-0.09-0.32l0.03-0.28l-0.25-0.27l-0.2-0.02l-0.16-0.01l0,0l0.16,0.01l-0.15-0.27
				l0.18,0.24l0.15-0.06l0.48,0.26l1.01-0.18l0.52,0.23l0.43,0.01l0.38,0.16l0.24-0.13l0.02-0.17l0.29-0.16l0.07,0.06l0.33-0.06
				l0.26,0.06l0.14,0.15l0.2,0.04l0.31-0.05l0.09-0.43l0.49-0.49l0.19-0.01l0.18,0.18l0.31,0.12l0.17,0.64l0.69-1.16l-0.08-0.2
				l-0.01-0.11l0.09-0.02l-0.04-0.13l0.51-0.55l0.45-0.02l0.31-0.31l0.02-0.14L401.79,136.22z"></path>
		</g>
		<g id="segovia_rural">
			<path class="st3 fase_0" d="M372.53,152.8l-0.13-0.26l-0.87-0.38l-0.02-0.16l0.13-0.24l-0.09-0.09l-0.09-0.38l-0.14-0.22l-0.37-0.31
				l-0.18-0.52l-0.3-0.24l-0.63,0.05l-0.38-0.24l-0.35,0.13l-0.03-0.05l-0.23-0.06l-0.03-0.21l-0.27-0.1l-0.19-0.32l-0.35-0.31
				l-0.21,0l-0.26-0.12l-0.66,0.18l-0.2-0.4l-0.18-0.01l-0.26-0.21l-0.33-0.1l0,0l0,0.07l-0.67,0.19l-0.55,0.82l-0.14,0.02
				l-0.28,0.75l-0.05,0.06l-0.6,0.3l-0.3-0.14l-0.3,0.01l-0.17,0.13l-0.15-0.17l-0.48,0l-0.24-0.09l-0.05,0.06l-0.14-0.03
				l-0.16,0.25l-0.08,0.25l-0.03-0.06l-0.86,0.6l-0.75,0.66l-0.13,0.01l-0.26-0.26l-0.63,0.53l-0.1-0.09l0.06-0.27l-0.17,0.04
				l-0.15-0.21l0.25-0.46l-0.15-0.17l-0.08-0.32l0.12-0.07l-0.17-0.33l0.69-0.47l-0.01-0.13l-0.28,0.16l-0.09-0.12l-0.34-0.02
				l-0.12,0.12l-0.22,0.03l-0.23-0.27l0,0l-0.64,0.63l-0.05,0.49l0.07,0.59l0.49,0.34l-0.53,0.24l-0.03,0.08l-0.44,0.17l-0.23,0
				l-0.04,0.73l-0.23,0.04l-0.06-0.06l-0.88,0.29l-0.12-0.09l-0.58,0.11l-0.07-0.04l-0.53,0.3l0.01,0.16l0.21,0.3l-0.07,0.36
				l0.04,0.3l-0.22-0.16l-0.36,0.29l-0.47-0.38l-0.05,0.12l-0.26,0l-0.19,0.08l-0.06,0.11l-0.12,0.01l-0.08,0.43l-0.09-0.11
				l-0.56,0.19l-0.16-0.16l-0.18,0l-0.07-0.08l0.04-0.12l-0.41-0.28l-0.05,0.23l-0.14-0.05l-0.07,0.12l0.11,0.14l-0.06,0.24
				l-0.38,0.6l0.06,0.43l-0.36-0.07l-0.13,0.1l-0.27-0.16l-0.1,0.14l0-0.08l-0.15,0.08l0.03-0.11l-0.09-0.04l-0.22,0.09l-0.02,0.11
				l-0.1,0.05l-0.04-0.08l-0.09,0.03l-0.2,0.15l-0.12-0.29l-0.06,0.04l-0.37-0.21l-0.54-0.17l-0.15,0.01l-0.14,0.18l-0.16,0.01
				l-0.04,1.27v0.18l0.28,0.31l-0.16,0.25l0.05,0.42l-0.18,0.49l0.31,0.2l0.04,0.14l0.5,0.24l0.13,0.14l-0.01,0.37l0.08,0.07
				l0.69-0.28l0.29-0.86l0.45,0.02l0.17,0.1l0.56,0.66l0.02,0.13l-0.34,0.24l-0.1-0.04l-0.36,0.14l-0.29,0.24l-0.14,0.49l0.09,0.11
				l-0.02,0.21l0.15-0.02l0.25,0.31l0,0.21h0.2l0.28,0.72l0.03,0.32l0.37,0.36l0,0.23l0.15,0.18l0.11-0.01l0.11-0.2l0.1-0.5
				l0.44-0.7l0.23-0.18l0.4,0.09l0.12-0.06l-0.05-0.09l0.04-0.2l-0.15-0.35l0.22-0.83l0.13-0.23l0.16-0.11l0.14,0.01l0.33,0.16
				l0.44,0.36l0.38-0.02l0.44-0.67l0.54,0.04l0.32-0.02l0.13,0.2l0.78,0.06l0.27,0.14l0.05-0.05l0.21,0.48l-0.24,0.11l0.04,0.11
				l0.12,0.01l0,0l-0.12-0.01l0.01,0.19l-0.14,0.01l0.09,0.23l0.04,0.08l0.08-0.1l0.26-0.02l0.03,0.19l0.15-0.12l0.09,0.02l0-0.24
				l0.13,0.19l0.13-0.08l0-0.1l0.25,0.22l0.03,0.13l0.43-0.23l0.17,0.14l0.2-0.05l0.12,0.07l0.58-0.38l0.1,0.08l0.15-0.11l0.15,0.03
				l0.04,0.14l0.12,0.01l0.15-0.16l0.07,0.04l0.22,0.72l0.38,0.49l-0.51,0.62l-0.03,0.29l-0.07,0.18l0.11,0.32l-0.16,0.1l0.1,0.18
				l0.5-0.18l0.11,0.04l0.18,0.25l0.18,0.05l-0.12,0.19l-0.45,0.16l-0.23,0.29l-0.15-0.06l-0.13-0.17l-0.39,0.03l-0.44,0.61
				l-0.28,0.22l-0.08-0.1l-0.15,0.41l-0.5,0.53l-1.12,0.98l-0.59,0.23l-0.74,0.84l0.07,0.28l-0.17,0.95l-0.09,0.16l0.19,1.16
				l-0.09,0.37l0.08,0.07l-0.02,0.23l-0.36,0.57l1.3,0.02l-0.09-0.13l0.28-0.42l0.34-0.01l-0.15-0.25l-0.06-0.29l0.25-0.21
				l0.05-0.28l0.94-0.52l0.16-0.21l0.2-0.01l0.42,0.23l0.07-0.27l0.32-0.2l0.28-0.05l0.23-0.19l0.06,0.06l0.24-0.08l0.53,0.09
				l0.68-0.25l1.13-0.15l0.16-0.12l0.46,0.3l0.29-0.04l0.19,0.18l0.15-0.13l0.22-0.02l0.24-0.14l0.24-0.94l0.17-0.33l0.35-0.31
				l0.04-0.28l0.41-0.68l0.34-0.21l0.36-0.1l0.14-0.44l-0.1-0.46l0.65-0.65l0.08-0.29l-0.01-0.36l0.14-0.02l0.22-0.25v0l0.93,0.19
				l0.19,0.35l0.84,0.14l0.3-0.12l0.26-0.21l0.2-0.06l0.35,0.04l0.17,0.12l0.25-0.02l0.15,0.09l0.07-0.19l-0.07-0.65l0.17-0.22
				l0.01-0.17l0.3-0.26l0.23-0.55l-0.11-0.88l0.1-0.3l0.21-0.23l0.43-0.89l-0.04-0.46l0.15-0.28l0.03-0.55l-0.17-0.01l-0.22-0.25
				l0.29-0.54l0,0l0.14-0.43l0.19-0.23l-0.02-0.6l0.4-0.02l0.22-0.12l0.1-0.42l0.1-0.27l0.22-0.1l-0.02-0.3l0.2-0.3l0.13-0.08
				l0.22-0.1l0.16-0.42l0.18-0.12L372.53,152.8z"></path>
		</g>
		<g id="el_espinar">
			<path class="st3 fase_0" d="M366.92,163.84l-0.04-0.34l-0.93-0.54l-0.46,0.32l-0.64,0.61l-0.42,0.16l0.02-0.22
				c-0.64,0.32-1.33,0.51-2.08,0.54c-0.45,0.02-0.91-0.02-1.36-0.02c-0.1,0-0.21,0.01-0.31,0.01l-0.07,0.12l-0.45,0.16l-0.23,0.29
				l-0.15-0.06l-0.13-0.17l-0.39,0.03l-0.44,0.61l-0.28,0.22l-0.08-0.1l-0.15,0.41l-0.5,0.53l-1.12,0.98l-0.59,0.23l-0.74,0.84
				l0.07,0.28l-0.17,0.95l-0.09,0.16l0.19,1.16l-0.09,0.37l0.08,0.07l-0.02,0.23l-0.36,0.57l1.3,0.02l-0.09-0.13l0.28-0.42
				l0.34-0.01l-0.15-0.25l-0.06-0.29l0.25-0.21l0.05-0.28l0.94-0.52l0.16-0.21l0.2-0.01l0.42,0.23l0.07-0.27l0.32-0.2l0.28-0.05
				l0.23-0.19l0.06,0.06l0.24-0.08l0.53,0.09l0.68-0.25l1.13-0.15l0.16-0.12l0.46,0.3l0.29-0.04l0.19,0.18l0.15-0.13l0.22-0.02
				l0.24-0.14l0.24-0.94l0.17-0.33l0.35-0.31l0.04-0.28l0.41-0.68l0.34-0.21l0.36-0.1l0.14-0.44l-0.1-0.46l0.65-0.65l0.08-0.29
				l-0.01-0.36l0.14-0.02L366.92,163.84z"></path>
			<path id="path4962" class="st3 fase_0" d="M358.62,170.16l0.21,0.04l0.27,0.46l0.27,0.17l-0.04,0.56l-0.42-0.08l-1.36,0.11
				l-0.73,0.28l-0.15-0.25l-0.06-0.29l0.25-0.21l0.05-0.28l0.94-0.52l0.16-0.21l0.2-0.01L358.62,170.16L358.62,170.16z"></path>
		</g>
		<g id="san_ildefonso">
			<path class="st3 fase_0" d="M372.05,158.74l-0.17-0.01l-0.22-0.25l0.29-0.54h-0.64l-0.25,0.26l-0.21-0.12l-0.46,0.14l-0.03-0.13
				l0.19-0.15l0.34-0.81l-0.14,0.01l-0.13-0.16l-0.16,0.06l-0.53-0.12v0l-0.26,0.04l-0.24,0.33l-0.08-0.06l-0.45,0.19l-0.38-0.01
				l-0.07-0.13l-0.19,0.28l-0.46,0.26l-0.17,0.2l-0.48,1.09l-0.67,0.47l-0.04,0.21l0.07,0.76l0.69,0.21l-0.49,0.21l-0.31-0.08
				c0.1,0.61,0.09,1.24-0.09,1.84c-0.04,0.13-0.09,0.25-0.14,0.37l0.72,0.41l0.04,0.34l0.93,0.19l0.19,0.35l0.84,0.14l0.3-0.12
				l0.26-0.21l0.2-0.06l0.35,0.04l0.17,0.12l0.25-0.02l0.15,0.09l0.07-0.19l-0.07-0.65l0.17-0.22l0.01-0.17l0.3-0.26l0.23-0.55
				l-0.11-0.88l0.1-0.3l0.21-0.23l0.43-0.89l-0.04-0.46l0.15-0.28L372.05,158.74z"></path>
		</g>
		<g id="segovia_i">
			<path class="st3 fase_0" d="M362.18,155.05l0.12,0.16l0.05,0.24l0.23,0.19l-0.01,0.03c0.5,0.03,0.99,0.05,1.49,0.01c0,0,0.01,0,0.01,0
				c0.04-0.01,0.07-0.01,0.11-0.02c0.14-0.03,0.28-0.06,0.42-0.11c0.04-0.01,0.08-0.03,0.12-0.04c0.08-0.09,0.31-0.15,0.4-0.21
				c0.1-0.06,0.29-0.14,0.35-0.24c-0.13,0.1-0.16,0.12-0.08,0.06c0.04-0.03,0.07-0.06,0.11-0.09c0.05-0.04,0.09-0.08,0.14-0.12
				l-0.02-0.02l-0.18,0.04l0.04-0.19l-0.08-0.1l-0.19,0.2l-0.24-0.22l0.15-0.69l0,0l-0.23-0.15l-0.51-0.03l0.13-0.24l0-0.17
				l-0.33,0.21l-0.35-0.13l-0.26,0.04l0.05,0.48l-0.14,0.3l-0.47,0.29l-0.55-0.22l-0.09,0.13l-0.12,0.22l-0.16,0.16l-0.06-0.06
				l-0.06,0.15l0.12,0.2L362.18,155.05z"></path>
		</g>
	</g>
	<g id="castillas_y_leon_provincias">
		<g id="leon_p">
			<path class="st6" d="M339.84,43.06l0.19-0.34l-0.4-0.5l-0.23,0.03l-0.13-0.07l-0.53,0.09l-0.02-1.02L338.45,41L338,40.75
				l-0.16-0.02l-0.34-0.21l-0.16,0l-0.27-0.22l-0.38-0.05l-0.02-0.18l-0.17-0.18l-0.46-0.18l-0.09-0.33L335.92,39l-0.38-0.28
				l-0.02-0.13l-0.23-0.11l0.35-0.4l-0.05-0.68l0.21-0.06l0.14-0.48l0.02-0.12l-0.24-0.22l0.07-0.24l-0.52-0.33l0.1-0.22l0.34-0.2
				l0.07-0.48l-0.55-0.35l-0.17-0.42l-0.2-0.08l0.06-0.15l-0.1-0.2l0.12-0.18l-0.02-0.15l-0.29-0.34l-0.26-0.27l-0.34-0.13
				l-0.46-0.35l0,0l-0.56,0.03l-0.21,0.06l-0.28,0.5l-0.52,0.14l-0.4,0l-0.39,0.3l0,0.26l-0.48,0.11l-0.29,0.43l0.03,0.11l-0.35,0.1
				l-0.71,0.81l-0.01,0.66l-0.19-0.17l-0.65-0.3l-0.14-0.17l-0.4,0.08l-0.41-0.06l-0.5,0.1l-0.04,0.17l-0.22-0.07l-0.16,0.07
				l-0.49-0.11l-0.09,0.34l0.08,0.14l-0.09,0.66l-0.18,0.12l-0.19,0.28l0.07,0.26l-0.37,0.28l0.18,0.48l-0.03,0.24l-0.33,0.74
				l0.15,0.44l-0.25,0.31l-0.14-0.02l-0.62,0.08l-0.82-0.12l-0.29-0.66l-0.23,0.22l-0.17,0.05l-0.1-0.06l-0.12,0.1l-0.19,0.37
				l-0.3,0.13l-0.53,0.01l-0.11,0.24l-0.31,0.14l-0.09,0.14l-0.23-0.12l-0.12,0.3l-0.42,0.17l-0.23-0.1l-0.11,0.07l-0.28-0.09
				l-0.17,0.09h0l0.17-0.09l-0.18-0.14l-1.14,0.05l-0.39,0.23l-0.16,0.32l-0.6-0.1l-0.59-0.27l-0.23,0.08l-0.13,0.18l-0.66-0.14
				l-0.19-0.1l0.1-0.42l-0.24-0.14l-0.98-0.02l0,0l-0.35,0.34l-0.22,0.06l-0.3,2.04l-0.18,0.09l-0.22-0.01l-0.22-0.11l-0.2,0.04
				l-0.25-0.16l-0.69-0.03l-0.58-0.15l-0.75,0.26l-0.36-0.27l-0.12,0.03l-0.09,0.07l-0.01,0.62l-0.26,0.57l-0.11,0.05l-0.14,0.5v0
				l-0.5-0.39l-0.14,0.08l-0.42-0.01l-0.34,0.08l-0.07,0.17l-0.41,0.23l-0.64-0.12l-0.08-0.11l-0.49-0.01l-0.18-0.51l-0.72-0.11
				l-0.12,0.04l-0.35-0.14l-0.21,0.11l-0.16-0.07l-0.58,0.07l-0.37-0.06l-0.08-0.17l-0.18,0.03l-0.28-0.37l-0.37-0.05l-0.04-0.09
				l-0.18-0.03l-0.26-0.24v0l-0.04-0.03L302,42.45l-0.59-0.26l0,0l-0.2,0.02l-0.31,0.25l-0.64,0.3l-0.18,0.18l-0.29,0.02l-0.05,0.27
				l-0.18,0.17L299.48,44l-0.18,0.14l-0.15,0l-0.18,0.19l-0.11,0.63l-0.32,0.34l-0.02,0.35L298.24,46l0.21,0.66l-0.06,0.14
				l-0.11,0.1l-0.59,0.12l-0.43-0.05l-0.17,0.11l-0.21,0.02l-0.16,0.15l-0.13-0.13l-0.7-0.17l-0.18,0.08l-0.43,0.02l-0.15,0.1
				L294.84,47l-0.54-0.62l-0.01-0.13l-0.51,0.11l-0.15-0.08l-0.35,0.09l-0.12-0.32l0.02-0.23l0.18-0.25l-0.79,0.22l-0.32-0.14
				l0.06-0.19l-0.27-0.35l-0.22-0.06l-0.01-0.11l-0.24,0.01l-0.1-0.27l-0.39-0.11l-0.54-0.3l-0.18-0.56l-0.37-0.66l0.1-0.2
				l-0.12-0.31l0.15-0.41l-0.11,0.1l-0.06-0.04l-0.11-0.42l-0.26,0.04l-0.35-0.19l-0.29,0.43l-0.31,0.06l-0.65,0.2l-0.32-0.15
				l-0.38-0.3l-0.5-0.13l-0.1-0.13l-0.41,0.18l-0.2-0.05l-0.59-0.54l0,0l-0.12,0.35l-0.28,0.12l0.06,0.38l-0.19,0.59l0.18,0.26
				l0.02,0.31l-0.23,0.23l-0.06,0.18l-0.18-0.07l-0.19,0.14l-0.26,0l-0.32,0.28l-0.66,0.18l-0.48-0.66l-0.44-0.13l-0.13,0.14
				l-0.2-0.51l-0.15-0.03l-0.3-0.26l-0.31-0.09l-0.33,0.36l-0.79-0.21l-0.52,2.03l-0.47-0.29l-0.17,0.05l-0.69-0.11l-0.21-0.09
				l-0.17-0.23l-0.03-0.19l-0.15-0.06l-0.47-0.55l-0.21,0.1l-0.2,0.4l-0.65-0.11l-0.1-0.13l-0.52,0l-0.13-0.6l-0.14-0.21l-0.2,0.03
				l-0.1-0.18l-0.3-0.03l-0.11-0.18l-0.28-0.19l-0.05-0.16l-0.18,0.12l0.06,0.17l-0.1,0.21l-0.38,0.21l0.02,0.28l-0.67,0.03
				l-0.28,0.12l0.08,0.3l0.14,0.08l0.04,0.29l-0.14,0.49l-0.32,0.02l-0.23-0.12l-0.13,0.08l-0.04,0.44l0.07,0.38l0.12,0.11
				l-0.71,0.64l-0.1,0l-0.39-0.29l-0.38,0.58l-0.51-0.08l-0.28-0.15l-0.2-0.29l-0.26-0.01l-0.06,0.12l0.03,0.52l0.23,0.15
				l-0.09,0.36l0.07,0.15l0.18,0.09l0.25-0.08l0.43,0.09l0.38-0.05l0.35,0.15l-0.32,0.91l0.34,0.41l0,0l-0.38,0.02l-0.36,0.21
				l-1.05,0.08l-0.36,0.27l-0.15,0.47l-0.27-0.07l-0.57-0.39l-0.37,0.15l-0.12,0.12l-0.47,0.08l-0.15,0.13l-0.12,0.39l-1.13-0.12
				l0,0l-0.09-0.12l-0.33-0.08l-0.25,0l-0.19,0.09l-0.31-0.1l-0.35,0.13l-0.26-0.45l-0.63,0.27l-0.15-0.2l-0.27-0.12l0,0l-0.32,0.09
				l-0.41-0.04l-0.15,0.1l-0.27-0.09l-0.96,0.12l-0.2,0.18l-0.68,0.21l-0.4-0.13l-0.48,0.53l-0.25,0.44l-0.35,0.29l-0.3,0.6
				L258,51.48l-0.54,0.06l0.01-0.21l0.22-0.19l-0.01-0.25l-0.1-0.2l-0.77-0.32l-0.46-0.3l-0.43-0.06l-0.51-0.2l-0.96,0.04l0.16,0.43
				l0.44,0.45l0.22,0.5l0,0.47l0.07,0.16l0.45,0.33l-0.2,0.19l-0.63-0.34l-0.08-0.2l-0.12-0.04l0.27-0.21l-1.52,0.44l0.48,1.36
				l0.26,0.53l0.2,0.29l0.28,0.06l-0.11,0.23l0.15,0.26l-0.45,0.25l-0.12,0.44l0,0v0l-0.35,0.2l-0.26,0.33l0.02,0.09l0.17,0.09
				l0.02,0.25l-0.14,0.31l-0.34,0.25l-0.17,0.01l-0.31,0.25l-0.13,0.02l-0.25,0.29l-0.04,0.28l0.07,0.09l-0.13,0l-0.2,0.3
				l-1.27,1.01l-0.46-0.03l-0.39-0.76l-0.12,0.03l-0.34,0.4l0.08,0.28l-0.03,0.19l-0.25,0.04l0.15,0.46l-0.15,0.24l-0.44,0.34
				l-0.26-0.43L248.53,60l-0.21-0.07l-0.15,0.17l-0.32,0.01l-0.44,0.52l0.07,0.2l-0.21-0.12l-0.16,0.11l-0.31,0.56l-0.34,0.29
				l0.45,0.58l0.09,0.45l0.41,0.34l0.04,0.15l0.21,0.1l-0.08,0.19l0.19,0.18l-0.31,0.16l-0.04,0.15l0.5,0.15l0.19,0.34l-0.35,0.06
				l-0.19-0.18l-0.25,0.12l-0.2,0.19l-0.11-0.01l-0.26,0.19l-0.16,0l-0.21,0.18l-0.04,0.36l0.09,0.2l-0.05,0.53l0.07,0.44
				l-0.19,0.34l0.23,0.45l0.39,0.11l0.03,0.17l-0.45,0.39l-0.02,0.14l-0.11,0.1l-0.25,0.05l-0.1,0.39l0.05,0.25l-0.28,0.38
				l-0.25-0.09l0.17,0.18l0.01,0.15l-0.27,0.55l0.14,0.09l0.12-0.09l0.11,0.02l0.08,0.44l-0.36,0.6l-0.03,0.27l-0.09,0.15
				l-0.21,0.12l0.13,0.08l0.56,0.01l0.32-0.11l0.25,0.05l0.53,0.18l0.16,0.35l0.37-0.2l0.61-0.13l0.14-0.1l0.01,0.2l0.14,0.13
				l0.74,0.34l0.22-0.04l0.32-1.01l0.22-0.27l1.02-0.08l0.11-0.07l0.29,0.06l0.42,0.5l-0.07,0.09l0.13,0.07l0.28-0.04l0.37,0.21
				l0.83,0.25l0.68,0.56l0.46,0.1l0.32-0.1l0.15-0.36l0.14,0.12l0.12-0.26l0,0l-0.13,0.26l0.03,0.06l0.26-0.04l0.16,0.11l-0.11,0.28
				l0.25,0.18l0.18,0.45l0.25,0.24l-0.12,0.44l-0.44,0.39l0.09,0.87l-0.06,0.24l0.29,0.34l-0.41,0.06l0.16,0.24l-0.32,0.25
				l-0.04,0.36l-0.06,0.09l-0.27-0.04l-0.22,0.18l0.4,0.56l0.41,0.19l0.11,0.34l0.12-0.04l0.33,0.09l0.3-0.09l0.01,0.14l0.24,0.09
				l0.32-0.03l0.56,0.2l0.21,0.31l0.1,0.43l0.35-0.13l0.33,0.25l0.63,0.2l-0.21,0.77l0.03,0.26l-0.1,0.63l0.15,0.18l0.12,0.38
				l-0.23-0.17l0.13,0.14l0.01,0.4l-0.28,0.42l0.08,0.25l-0.46,0.53l-0.11,0.32l-0.43,0.64l0.05,0.16l-0.09,0.56l-0.17,0.15
				l-0.12,0.03l-0.07-0.08l-0.34,0.1l0.74,0.43l0.28,0.37l0.65-0.08l0.33,0.25l0.07,0.21l-0.1,0.08l0,0.12l0.09,0.21l0.23,0.08
				l0.09-0.04l0.1,0.2l-0.03,0.15l0.16,0.19l0.33-0.04l0.31,0.25l0.19-0.02l0.4-0.27l0.6,0.01l0.14,0.11l0.48-0.07l0.18,0.04
				l0.28-0.21l0.38-0.02l0.35,0.29l0.26-0.01l0.19,0.15l0.1,0.27l0.12,0.06l0.25-0.14l0.13,0.1l0.27,0.03l0.14-0.16l0.24-0.08
				l0.56,0.17l0.65-0.12l0.09,0.05l1.05,0.19l0.15-0.03l0.2,0.17l0.09-0.02l0.35,0.2l0.17,0.16l-0.02,0.18l0.09,0.08l-0.16,0.26
				l0.34,0.03l0.09,0.1l0.22-0.02l0.06-0.09l0.49,0.1l0.06-0.07l0.32-0.5l0.3-0.12l0.06-0.1l0.71,0.19l0.17-0.26l0.41-0.19l0.2,0.16
				l0.51,0.17l0.48-0.11l0.04-0.16l0.64-0.13l0.04,0.09l1.35,0.05l0.22-0.25l0.5,0.28l0.24-0.2l0.11,0.06l0.34-0.23l0.87-0.23
				l0.13-0.1l-0.13,0.1l-0.87,0.23l0.23,0.2l0.16,0.58l0.19,0.46l0.17,0.07l0.02,0.23l0.64-0.05l1.05,0.32l-0.09,0.32l0.48-0.03
				l0.66,0.3l0.29,0.59l0.26,0.16l1.7-0.54l0.41-0.41l0.25,0.24l0.43,0.13l0.25-0.32l0.51,0.31l0.78-0.17l0.38,0.06l0.27-0.04
				l0.49,0.14l0.08-0.15l0.33,0.09l1.57,0.14l0.49,0.27l0.38,0.12l0.7,0.38l0.63,0.71l0.54,0.16l0.17,0.17l0.13,0.02l0.05-0.61
				l0.17-0.37l0.5,0.04l0.17,0.1l0.2-0.07l0.61,0.01l0.22-0.18l0.35-0.65l0.28,0.33l0.39,0.25l-0.15,0.37l0.73,0.99l0.34,0.22
				l0.55,0.28l0.16-0.1l0.11-0.33l0.24-0.03l0.81-0.38l0.23-0.22l0.08,0.04l-0.07,0.11l0.04,0.17l0.29-0.16l0.28,0.02l0.13-0.33
				l0.32-0.1l0.41-0.04l0.42,0.09l0.17-0.12l0.17,0.66l-0.04,0.16l0.19,0.29l0.17,0.14l0.24-0.02l0.8-0.07l0.84-0.31l0.81,0.02
				l-0.12-0.66l-0.24-0.17l-0.06-0.14l0.25-0.19l0.05,0.08l0.49-0.1l0.09,0.05l-0.02,0.23l0.84,0.25l0.29,0.54l0.67,0.72l0.09,0.2
				l-0.19,0.66l-0.57,0.41l0.05,0.07l0.12,0.13l0.38-0.12l0.75,0.22l0.08-0.06l0.02,0.06l0.03-0.05l0.26,0.06l-0.02,0.14l0.14,0.03
				l0.1-0.14l0.12,0.07l-0.01,0.09l-0.03,0.17l0.12,0.07l0.12,0.28l-0.22,0.71l0.09,0.1l0.26-0.41l0.03-0.32l0.07-0.06l0.12,0.05
				l0.2,0.36l0.2-0.07l0.17,0.18l0.13-0.25l0.19-0.16l0.13-0.36l0.05-0.47l0.17-0.38l-0.01-0.33l0.1-0.3l-0.06-0.07l0.66-0.26
				l0.25,0.39h0l0.11,0.34l0,0.01l-0.02,0.32l0.38,0.41l0.3,0.56l-0.05,0.21l-0.2,0.27l0.47,0.45l0.02,0.19v0l0.38,0.36l-0.25,0.3
				l0.18,0.18l0.02,0.22v0l0.6,0.66l0.04,0.39l-0.27,0.1l-0.01,0.07l0,0l-0.01,0.2l0,0l-0.2,0.16l-0.17,0.04l0,0l0.06,0.23
				l0.09,0.36l0.89,0.37l-0.01,0.12v0l0.09,0.04l0.19-0.05l0.45,0.21h0l0.27,0.24l0.19-0.84l0.1-1.04l-0.21-0.35l-0.27-0.16
				l-0.19,0.07l0.02-0.16l1.05-0.92l0.05,0.08l0.33-0.07l0.58-0.22l0.54-0.4l0.33,0.03l0.08-0.09l-0.05-0.19l0.18-0.25l-0.13-0.9
				l0.36,0.14l0.17,0.19l0.32,0.16l1.26,0.03l0.79-0.41l0.07,0.07l0.13-0.26l0.08-0.73l-0.02-0.15l-0.1-0.03l0.02-0.12l0.33,0.14
				l0.66,0.08l0.24-0.15l0.43-0.1l0.14-0.25l-0.18-0.29l-0.01-0.3l-0.11-0.12l0-0.23l-0.1-0.13l0.3-0.28l-0.35-0.47l-0.42-0.16
				l-0.3,0.01l0.07,0.13l-0.14-0.05l-0.18,0.21l-0.15,0.03l-0.17,0.19l-0.51,0.21l-0.2,0.19l-0.3,0.09l-0.59-0.26l-0.04-0.21
				l-0.14,0.17l-0.25,0.12l-0.16,0l-0.09,0.09l-0.19-0.07l-0.1,0.04l0.04,0.07l-0.11,0.05l0.04,0.1l-0.15,0.12l-0.08-0.09
				l-0.15,0.09l-0.11-0.02l-0.18-0.25l-0.1,0v0l0.1,0l-0.01-0.07l0.21-0.22l-0.1-0.17l0-0.29l0.15-0.18l-0.39-1.11h0l-0.01-0.47
				l0.08-0.21l-0.29-0.28l-0.09-0.37l0.07-0.75l0.28-0.46l-0.34-0.4l-0.02-0.68l0.04-0.19l0.29-0.21l0.21,0.16l0.3-0.02l0.29-0.12
				l0.34,0.32l0.15-0.07l0.14,0.04l0.35-0.6l-0.17-0.1l0.03-0.75l-0.22-0.23l-0.05-0.44l0.07-0.46l0.33-0.1l0.35-0.24l0.33,0.37
				l0.56-0.24l-0.27-0.21l0.04-0.53l-0.14-0.14l-0.21-0.06l-0.06-0.25h0l0.06,0.25l0.21,0.06l0.14,0.14l-0.04,0.53l0.27,0.21
				l0.19,0.13l0.24-0.04l0.07-0.37l0.17,0l0.16,0.1l0.06-0.03l-0.02-0.13l0.14-0.04l0.12-0.15l0.6,0.08l0.17-0.35l0.11-0.13
				l0.11,0.12l-0.11-0.12l-0.11,0.13l0.11,0.4l-0.09,0.2l0.34,0.56l0.24,0.06l0.24-0.12l0.13-0.28l-0.05-0.11l0.25-0.12l-0.05,0.65
				l0.12,0.06l0.24-0.69l0.09-0.14l0.15,0l0.66,0.71l0.1,0.22l0.61-0.2l-0.03-0.21l0.22-0.16l0.56,0l0.33-0.15l0-0.2l0.48-0.21
				l0.73-1.05l0.12-0.04l0.15-0.32l0.23-0.11l0.07-0.11l0.1,0l0.03,0.06l0.61,0.07l0.27,0.63l0.04,0.28l0.15-0.08l0.18,0.11
				l-0.04,0.1l0.14,0.25l0.01,0.33l0.13,0.06l0.05-0.07l0.19,0.16l0.16-0.09l-0.24,0.9l0.54,0.2l0.39-0.24l0.14-0.3l-0.03-0.21
				l0.14-0.31l0.08-0.64l-0.19-0.2l0.13-0.23l1.47,0.43l0.3-0.21l0.2,0.07l0.13-0.11l0.53,0.28l0.13-0.18l0.68-0.27l0.11-0.6
				l0.11-0.13l0.04-0.3l-0.08-0.13l0.12-0.28l0.15-0.19l0.01-0.13l0.06-0.31l-0.07-0.59l-0.42,0l-0.05,0.13l-1.13,0.35l-0.08-0.01
				l0.03-0.25l0.26-0.35l-0.02-0.35l0.24-0.66l-0.03-0.35l0.1-0.02l0.05-0.13l0.17-0.13l0.13,0.19l0.67-0.02l-0.03-0.07l0.09-0.09
				l-0.12-0.12l0.41-0.04l0.12-0.14l0.89,0.17l0.33-0.27l-0.32-0.31l0.12-0.25l-0.24-0.23l0.07-0.19l-0.09-0.53l0.35-0.32l0.04-0.23
				l-0.15-0.21l-0.05-0.3l-0.57-0.12l0.12-0.06l0.3-0.21l0.32-0.42l0.23-0.08l0.3-1.91l0.19-1.22l-0.1-0.01l-0.04-0.5l0.13-1.01
				l-0.89-0.22l-0.19-0.19l-0.05-0.28l0.02-0.56l0.51-0.95l0.05-0.35l0-0.55l-0.15-0.24v-0.22l0.41-1.11l-0.15-0.63l0.05-0.45
				l0.04-0.63l-0.17-0.73l0.4-1.27l0.18-0.14l0.25-0.46l0.34-0.21l-0.49-0.25l-0.04-0.3l-0.26-0.28l-0.24-0.05l-0.56,0.3l0,0
				l-0.2-0.1l-0.1-0.73l-0.4-0.14l-0.32-0.41l0.12-0.48l-0.08-0.17l0.28,0.05l0.14-0.21l0.18,0.1l0.44-0.9l0.02-0.28l0.26-0.27
				l0.05,0.08l0.17-0.03l0.06-0.08l-0.03-0.78l-0.1-0.13l0.01-0.16l-0.16-0.22l0.24-0.43l0.04-0.53l0.16-0.28l0.14-0.48l-0.5-0.56
				l-0.01-0.31l0.11-0.31l-0.18-0.25l0,0l0.18,0.25l0.72-0.15l0.49,0.17l0.12,0.12l0.2-0.56L335,51.8l-0.04-0.3l0.16-0.43l0.38-0.31
				l0.31,0.12l0.09-0.21l-0.07-0.4l0.3-0.28l0.08-0.27l0.35-0.29l0.24-0.09l0.03-0.18l0.16-0.16l-0.07-0.31l0.23-0.31l0.1-0.41
				l-0.16-0.38l0.05-0.28l-0.08-0.32l0.47-0.33l0.03-0.69l0.27-0.32l0.22-0.05l0.07-0.24l0.57-0.19l0.4-0.75l0.73-0.16l0.07-0.14
				l-0.25-0.38L339.84,43.06z"></path>
		</g>
		<g id="salamanca_p">
			<path class="st6" d="M325.63,152.94l-0.09-0.1l0-0.45l-0.36-0.4l0.03-0.14l-0.1,0l0.03-0.27l-0.13-0.07l0.06-0.18l-0.17-0.02
				l0.05-0.14l-1-0.36l-0.27-0.17l-0.12,0.12l-0.02-0.18l-0.21-0.17l0.05-0.16l-0.12-0.07l0.06-0.33l-0.09-0.06l0-0.22l-0.15-0.24
				l0.11-0.84l-0.08-0.14l0.09-0.1l-0.05-0.08l0.11-0.37l0.21-0.27l0.12,0.02l0.04-0.16l0.42-0.3l0.17-1.43l-0.14-0.09l-0.16-0.13
				l-0.17-0.27l0.01-0.15l-0.36-0.19l-0.15-0.19l-0.52-0.29l-0.18-0.19l-0.34-0.06l0-0.11l-0.14,0.01l-0.03-0.08l0.08-0.04
				l-0.12-0.11l-0.08,0.07l-0.14-0.04l-0.25,0.25l-0.21-0.06l-0.17-0.18l-0.27,0.08l-0.03-0.1l-0.42,0.39l-0.17-0.14l0.2-0.29
				l-0.2-0.16l-0.09-0.32l0.04-0.09l-0.08-0.01l0.03-0.16l-0.14-0.1l0.03-0.18l-0.07-0.17l-0.39-0.22l-0.15-0.17l-0.25,0.05
				l0.03-0.11l-0.21-0.09l-0.11,0.12l-0.35,0.02l-0.18-0.11l-0.13,0.07l0.05,0.13l-0.14-0.09l-0.36,0.06l-0.23,0.03l-0.14,0.45
				l0.06,0.04l-0.05,0.14l0.11,0.1l-0.33,0.04l-0.19-0.23l-0.23,0.12l-0.04-0.06l-0.75,0.37l-0.18-0.05L316,143l-0.07-0.15
				l-0.36-0.22l0.1-0.09l-0.17-0.22l-0.09-0.16l0.05-0.46l-0.06-0.12l-0.19,0.09l-0.55,0.35l-0.38,0.12l-0.84-0.36l-0.66,0.99
				l0.19,0.49l-0.05,0.04l-0.14-0.53l-0.43-0.11l-0.33,0.01l-2.04,0.49l0.07,0.14l-0.02,0.38l0.23,0.57l-0.14,0.24l-0.16,0.1
				l-0.11,0.09l-0.04,0.16l-0.14,0.07l-0.16-0.5l0.01-0.91l0.1-0.39l-0.46,0.08l-0.4-0.16l-0.27-0.26l-0.05-0.31l-0.14-0.21
				l-0.29-0.05l-0.72-0.51l-0.48-0.46l-0.31-0.08l-0.1-0.1l0,0l-0.25,0.04l-0.68,0.75v0l-0.68-0.3l-0.24-0.31l-0.37-0.09l-0.35,0.06
				l-0.25-0.94l-0.13-0.01l-0.78-0.72l-0.24-0.01l-0.2,0.08l-0.46,0.67l-0.9-0.14l-1.36,0.36l-0.66,0.1l-0.2,0.11l-0.37,0.43
				l-1.03-0.01l-0.21-0.19l-0.34-0.07l-0.18-0.2l-1.07,0.22l-0.35-0.1l-0.28-0.15l-0.21,0.02l-0.22-0.21l-0.14,0.03l-0.19-0.25
				l-0.35-0.25l-0.54,0.08l-0.26-0.15l-0.43,0.34l-0.21,0.08l0.01,0.15l-0.33,0.53l0.14,0.71l0.07,0.93l0.09,0.2l-0.05,0.07
				l0.09,0.22l-0.1,0.22l-0.26-0.08l-0.19,0.22l-0.24,0.31l0.08,0.35l-0.04,0.1l-0.2-0.06l0,0.11l0.11,0.28l-0.41,0.05l-0.15,0.15
				l0.11,0.19l-0.23,0.19l-0.1,0.03l-0.09-0.19l-0.31-0.13l0.01-0.16l-0.15-0.37l-0.09-0.05l-0.13-0.3l0.02-0.35l-0.11-0.2
				l-0.21-0.11l-0.24,0.07l-0.71,1.09l-0.29,0.06l-0.49-0.17l-0.26-0.61l0.03-0.68l0.38-0.6l0.29-0.19l0.09-0.36l-0.63-0.14
				l-0.63-0.28l-0.27-0.04l-0.48-0.31l-0.14-0.34l-0.15,0.18l-0.15,0.05l-0.19,0.29l-0.75-0.06l-0.3,0.12l-0.38-0.12l-0.48,0.48
				l-0.13,0.04l-0.08,0.34l-0.33,0.34l-0.37-0.4l-0.84-0.53l-0.57-0.06l-0.52-0.28l-0.07-0.06l-0.32-0.27l-0.12,0.01l-0.85-0.42
				l-0.24-0.27l-0.08-0.22l-0.77-0.3l-0.6-0.61l-0.04-0.1l-0.25-0.08l-0.14,0.13l-0.44,0l-0.42-0.14l-0.1-0.14l-0.41-0.19l-0.25-0.4
				l-0.3,0.14l-0.24-0.06l-0.16-0.12l-0.51-0.06l-0.05-0.15l-0.21-0.17l-0.4-0.16l-0.09-0.16l-0.25-0.09l-0.42,0.03l-0.35,0.37
				l-0.35,0.01l-0.59,0.15l0.03-0.27l-0.12-0.1l-0.19,0.01l-0.09,0.15l-0.62-0.23l-0.25,0l-0.36-0.44l-0.45,0.32l-0.49-0.07
				l-0.06-0.04l-0.03-0.32l-0.42,0.3l-0.06,0.23l0.52,0.4l0.01,0.11l-0.69,0.47l-0.21,0.06l-0.43,0.05l-0.17-0.49l-0.24-0.14
				l-0.35,0.5l-0.69,0.51l-0.16,0.62l-0.64,0.16l-0.15,0.2l-0.41-0.02l-0.15-0.13l-0.04-0.49l-0.07-0.15l-0.15-0.09l-0.13,0.1
				l-0.13,0.48l-0.22-0.02l-0.08-0.18l-0.15-0.07l-0.44,0.11l-0.41,0.27l-0.57-0.3l-0.15,0.04l-0.2,0.64l-0.48,0.26l-0.24,0.37
				l-0.4,0.28l-0.21,0.36l-0.26,0.19l0.14,0.21l-0.01,0.2l-0.17,0.38l-0.22,0.01l-0.03,0.07l0.1,0.16l-0.06,0.31l-0.19,0.22
				l-0.14,0.02l-0.11,0.29l-0.42,0.36l-0.37,0.18l-0.4,0.55l-0.03,0.16l-0.29,0.22l-0.11,0.18l-0.74,0.41l-0.02,0.09l-0.1,0.15
				l0,0.34l0.67,1.12l-0.1,0.24l-0.26,0.24l-0.05,0.2l-0.32,0.33l-0.08,0.28l0.06,0.14l-0.09,0.16l-0.19,0.17l-0.02,0.18l-0.28,0.21
				l-0.14,0.35l-0.76,0.69l0.08,0.26l-0.05,0.14l-1.03,0.42l-0.78,0.09l-0.54,0.01l-0.52-0.09l-1.45-0.56l-0.61,0.48l-0.05,0.67
				l0.07,0.16l0.22,0.18l0.11,0.44l0.5,0.21l0.22,0.54l0.48,0.42l0.2,0.52l0.43,0.57l0.3,0.16l0.35,0.47l0.1,0.34l0.18,0.4
				l-0.05,0.23l0.24,0.36l-0.02,1.2l0.46,0.7l0.33,0.24l0.28,0.02l0.13,0.19l0.41-0.1l0.1,0.33l0.11,0.4l-0.02,0.23l0.12,0.23
				l-0.07,0.25l0.14,0.42l-0.08,0.17l-0.23-0.05l-0.19,0.22l-0.16-0.08l-0.12,0.06l-0.12-0.04l-0.06-0.11l-0.08,0.12l0,0.15l0.2,0.2
				l-0.13,0.57l0.28,1l0.02,0.32l-0.12,0.36l-0.33,0.93l0.09,0.44l-0.1,0.52l-0.11,0.38l0.09,0.44l-0.07,0.07l0.04,0.14l0.19,0.17
				l0.01,0.39l0.29,0.22l0.26,0.44l-0.09,0.49l0.1,0.4l0.15,0.15l-0.02,0.23l0.08,0.03l-0.03,0.12l0.15,0.58l-0.12,0.6l0.47,0.38
				l-0.23,0.39l-0.34,1.33l-0.21,0.39l-0.32,0.66l-0.68,0.85l-0.01,0.78l-0.13,0.18l-0.16,0.57l0.58,0.15l0.31,0.26l0.61,0.2
				l0.09-0.03l0.04,0.23l0.14,0.01l-0.18,0.54l0.09,0.58l0.24,0.37l0.08,0.59l-0.63,0.37l-0.17,0.19l-0.13,0.37l0.03,0.2l-0.11,0.16
				l-0.11,0.24l-0.51,0.45l-0.58,1.23l0.19,0.88l0.15,0.73l0.27,0.78l0.48,0.6l0.4,0.29l0.16,0.28l-0.03,0.09l0.38,0.31l0.31,0.37
				l0.35,0.11l0.14,0.28l-0.34,0.23l-0.16,0.23l0.1,1.2l-0.13,0.27l-0.31,0.18l-0.64-0.04l-0.11,0.13l-0.14,0.02l-0.07,0.17
				l-0.29,0.08l-0.14,0.22l-0.16-0.01l-0.28,0.36l-0.55,0.41l-0.13,0.28l0.07,0.93l-0.13,0.39l0.74,0.87l0.28-0.06l0.11,0.38
				l0.11,0.05l0.48-0.03l-0.02,0.13l0.08,0.06l0.43,0.01l0.16,0.15l0.27-0.03l-0.08-0.4l0.83,0.02l0.14,0.19l0.91,0.35l0-0.39
				l0.15-0.19l0.42-0.34l0.25-0.05l0.34-0.42l0.34-0.23l0.27,0.12l0.07,0.35l0.27,0.51l0.53,0.45l0.07-0.21l0.43-0.23l0-0.38
				l0.24-0.28l1.29-0.31l0.22,0.23l0.38-0.1l0.42,0.03l0.7-0.3l0.42,0.05l0.14-0.2l0.51-0.27l0.09-0.18l0.51-0.43l-0.14-0.27
				l0.14-0.45l-0.03-0.12l-0.32-0.44l0.18-0.78l0.98-0.98l0.81-0.38l0.93-0.42l0.31-0.07l0.45-0.37l0.51-0.14l0.48,0.1l0.48-0.1
				l-0.11-0.28l0.06-0.19l0.62-0.53l0.22-0.42l0.45-0.15l0.67,0.16l0.33-0.09l0.29,0.03l0.57-0.37l-0.05-0.25l0.06-0.2l-0.18-0.11
				l0.01-0.27l0.79-0.79l0.05-0.31l0.4-0.25l0.38-0.01l0.12-0.15l1.13-0.08l0.27-0.3l0.51-0.05l0.17-0.13l0.1-0.36l0.15-0.18
				l0.4,0.05l0.61-0.85l0.35-0.04l0.3,0.12l0.12-0.03l0.36,0.12l0.43-0.01l0.11,0.08l0.3,0.5l0.14,0.04l0.04,0.27l0.25,0.22
				l0.45,0.22l0.47,0.06l0.21,0.24l-0.14,0.25l0.36,0.63l0.6-0.04l0.49-0.31l0.16,0.06l-0.04,0.34l-0.09,0.01l-0.24,0.31l-0.18,0.46
				l0.43,0.08l0.04,0.18l0.44,0.1l0.12-0.06l0.27,0.1l0.19,0.1l0.04,0.17l0.17-0.04l0.11,0.09l0.09,0.22l0.32-0.02l0.05,0.14
				l0.23,0.08l0.13,0.15l-0.08,0.08l-0.12-0.07l-0.16,0.03l-0.08,0.26l-0.3-0.07l-0.07,0.13l0.06,0.08l0.46,0.04l-0.09,0.29
				l-0.38,0.12l-0.08,0.1l0,0.16l0.21,0.11l-0.05,0.53l-0.14,0.13l-0.27-0.05l-0.18,0.28l-0.21-0.1l-0.14,0.09l1.54,0.5l0.63,0.32
				l0.26-0.08l0.37,0.04l0.36,0.14l0.47-0.04l0.29-0.45l0.08-0.48l-0.08,0.48l-0.29,0.45l-0.14,0.95l0.13,0.33l0.2,0.17l0.21,0.39
				l0.57,0.38l0.54,0.11l0.13,0.2l0.29,0.19l0.14-0.1l0.94,0.38l0.16,0.18l0.69,0.01l0.15,0.17l0.13-0.5l0.23-0.16l0.49-0.81
				l0.15-0.35l-0.06-0.34l0.38-0.45l0.41,0l0.26-0.26l0.4-0.16l0.11-0.22l0.17,0.1l-0.11,0.27l0.27,0.18l0.9-0.87l0.09-0.2
				l0.23-0.16l0.34,0.05l0.44-0.17l0.14,0.25l0.48,1.94l-0.3,0.32l-0.16,0.34l-0.38,0.14l0.39,0.59l1.14,0.32l0.29-0.09l0.39-0.25
				l0.16-0.1l0.28-0.28l0.35-0.52l-0.01-0.35l0.6-0.8l0.26-0.26l0.72-0.41l0.16-0.32l0.53-0.26l-0.54-0.37l0.05-0.13l-0.08-0.18
				l0.01-0.82l0.34-0.56l-0.26-0.47l-0.11-1.09l-0.17-0.2l0.03-0.14l0.3-0.13l0.36-0.15l0.17-0.26l0.58-0.26l0.52-0.51l0.12-0.28
				l0.27-0.49l0.31-0.12l0.15-0.16l0.53-0.12l0.81,0.04l0.6-0.49l0.57-0.04l-0.13,0.23l0.16,0.33l-0.04,0.29l-0.23,0.23l-0.16,0.04
				l-0.1,0.45l-0.09,0.19l0.46,0.21l0.38,0.36l0.4,0.58l0.2-0.07l0.51,0.1l0.55-0.07l0.1-0.23l0.06-0.56l-0.07-0.27l-0.34-0.16
				l-0.02-0.07l0.07-0.22l0.37-0.38l0.06-0.25l0.31,0.26l0.22-0.02l0.27,0.1l0.08-0.05l0.15,0.09l0.18-0.18l0.35-0.12l0.16-0.2
				l-0.09-0.06l0.29-0.37l-0.05-0.59l-0.01-0.33l0.39-0.16l0.08-0.18l0.13-1.21l0.19-0.34l0.24-0.16l0.13-0.26l0.14-0.95l0.17-0.25
				l-0.26-0.56l-0.24,0.04l-0.62,0.3l-0.29,0.07l-0.18-0.05l-0.67,0.19l-1.08,0.43l-0.38,0.17l-0.29,0.26l-0.07-0.04l0.02-0.38
				l-0.12-0.11l0.02-0.19l0.42-0.52l0.09-0.36l-0.06-0.15l0.03-0.21l0.02-0.43l0.25-0.27l0.13-0.25l0.22-0.2l0.1,0.47l-0.03,0.45
				l0.48-0.09l0.09,0.09l0.76,0.06l0.1,0.04l0,0.11l0.68-0.11l0.06-0.33l0.13-0.06l0.08-0.16l0.09,0.06l0.21-0.11l0.17-0.26
				l0.21-0.09l0.06-0.42l0.34-0.24l0.09-0.03l0.14,0.08l0.26-0.03l0.21-0.25l0.18-0.25l0.24-0.05l0.09-0.12l0.06,0.07l0.15-0.35
				l-0.1-0.19l0.35-0.44l-0.21-0.36l0.51-0.18l-0.81-0.78l-0.05-0.32l0.06-0.12l0.2-0.01l0.12-0.34l0.37-0.17l0.15,0.08l0.63-0.41
				l0.3,0.01l0.18-0.13l0.21-0.29l-0.04-0.16l-0.17-0.1l0.28-0.1l0.06-0.22l0.27,0l0.01,0.13l0.35-0.03l0.22-0.12l0.6-0.24
				l0.13-0.22l0.42-0.32l0.09-0.19l0.46-0.25l0.25-0.22l0.09-0.2l0.14,0.11l-0.05-0.28l0.21-0.18l0.72-0.31l0.19-0.25l0.53-0.25
				l-0.1-0.69l-0.08-0.12l0.42-0.33l0.98-0.51l0.25-0.41l-0.09-0.06l0.12-0.42l0.44-0.32l0.03-0.13l0.18-0.03l-0.01-0.21l0.12-0.13
				l0.05-0.2l-0.18-0.21l0.03-0.08l-0.11-0.08l0-0.18l-0.1,0.01l-0.07-0.12l-0.03-0.19l0.14-0.17l0.1-0.59l-0.04-0.15l0.11-0.29
				l-0.07-0.17l-0.02-0.16l0.27-0.09l0.19-0.21l0.02-0.12l0.12-0.06l-0.02-0.23l0.38-0.32l0.13-0.19l-0.11-0.05l0.03-0.09l0.2-0.09
				l-0.04-0.06l0.2-0.2l0.12,0.01l0.16-0.12l0.07,0.07l0.24-0.07l-0.06-0.12l0.18-0.41l-0.08-0.07l0.14-0.07l-0.08-0.14l0.05-0.06
				l-0.2-0.29l0.09-0.12l-0.11-0.07l0.11-0.09l-0.05-0.08l-0.2,0.03l-0.03-0.07l0.14-0.08l-0.27-0.57l0.07-0.1l-0.01-0.17l-0.2-0.01
				l0.02-0.3l0.26-0.11l0.18,0.1l0.32-0.22l0-0.11l0.16,0.01l-0.06-0.17l0.34-0.74l0.06-0.07l0-0.34l-0.13-0.26L325.63,152.94z"></path>
		</g>
		<g id="zamora_p">
			<path class="st6" d="M319.83,124.22l-0.05-0.23l-0.21-0.29l-0.33-0.25l-0.01-0.09l-0.39-0.02l0.02-0.06l-0.29-0.19l-0.35,0.08
				l-0.06-0.13l0.06-0.35l-0.32-0.16l0.05-0.11l-0.16-0.24l-0.4-0.31l0.49-0.55l0.52-0.29l0.05-0.16l-0.17-0.05l-0.05-0.13
				l0.06-0.26l-0.28-0.21l-0.61-0.21l-0.54,0.03l-0.11-0.18l-0.47,0.04l-0.35-0.27l0.19-0.49l-0.04-0.27l0.07-0.22l0.16-0.14
				l-0.21-0.36l0.05-0.31l-0.02-0.07l0.07-0.65l-0.05-0.41l-0.18-0.62l-0.41-0.27l-0.1-0.66l-0.2-0.16l-0.5,0.13l0.08-0.93
				l-0.22-0.63l-0.16-0.03l0,0l-0.18,0.13l-0.06,0.3l-0.15,0.08l-0.01-0.44l-0.61-1.32l0-0.27h0l-0.03-0.16l0.86-0.93l0.84-0.16
				l0.32-0.45l0.32-0.01l0.29-0.03l0.15,0.09l0.09-0.11l0.16-0.04l0.1-0.15l1.39-1.06l0.11-0.51l-0.42-0.26l-0.14,0.08l-0.18-0.07
				l-0.14,0.11l-0.32-0.07l-0.05,0.06l-0.76-0.18l-0.12-0.11l0.1-0.23l0.29-0.24l-0.07-0.06l0.62-0.92l-0.12-0.33l0.17-0.17
				l-0.25-0.84l0.04-0.57l0.4-0.74l0.19-0.22l0.09-0.47l-0.21-0.12l-0.45-0.55l-0.44-0.28l-0.58,0l0.02-0.14l0.13-0.13l1-0.55
				l0.29-0.46l0.45-0.36l-0.02-0.16l0.15-0.17l0.5-0.32l0.39-0.1l0.35,0.08l0-0.07l-0.24-0.16l-0.82,0.31l-0.91-0.66l0.04-0.09
				l-0.29-0.54l-0.16,0.01l-0.18-0.21l-0.22-0.66l-0.09-0.08l-0.95-0.12l-0.78-0.58l-0.33-0.34l0-0.1l-0.33-0.03l0,0l-0.54,0.4
				l-0.58,0.22l-0.33,0.07l-0.05-0.08l-1.05,0.91l-0.02,0.16l0.19-0.07l0.27,0.16l0.21,0.35l-0.1,1.04l-0.19,0.84l-0.27-0.24
				l-0.45-0.21l-0.19,0.05l-0.09-0.04l0.01-0.12l-0.89-0.37l-0.15-0.59l0.17-0.04l0.2-0.16l0.03-0.28l0.27-0.1l-0.04-0.39l-0.6-0.66
				l-0.02-0.22l-0.18-0.18l0.25-0.3l-0.38-0.36l-0.02-0.19l-0.47-0.45l0.2-0.27l0.05-0.21l-0.3-0.56l-0.38-0.41l0.02-0.32
				l-0.12-0.35l-0.25-0.39l-0.66,0.26l0.06,0.07l-0.1,0.3l0.01,0.34l-0.17,0.38l-0.05,0.47l-0.13,0.36l-0.19,0.16l-0.13,0.25
				l-0.17-0.18l-0.2,0.07l-0.2-0.36l-0.12-0.05l-0.07,0.06l-0.03,0.32l-0.26,0.4l-0.09-0.1l0.22-0.71l-0.12-0.28l-0.12-0.07
				l0.03-0.17l0.01-0.09l-0.12-0.07l-0.1,0.14l-0.14-0.03l0.02-0.14l-0.26-0.06l-0.03,0.05l-0.02-0.06l-0.08,0.06l-0.75-0.22
				l-0.38,0.12l-0.12-0.13l-0.05-0.07l0.57-0.41l0.19-0.66l-0.09-0.2l-0.67-0.72l-0.29-0.54l-0.84-0.25l0.02-0.23l-0.09-0.05v0
				l-0.49,0.1l-0.05-0.08l-0.25,0.19l0.06,0.14l0.24,0.17l0.12,0.66l0.21-0.02l0.22,0.16v0l-0.22-0.16l-0.21,0.02l-0.81-0.02
				l-0.84,0.31l-0.8,0.07l-0.24,0.02l-0.17-0.14l-0.19-0.29l0.04-0.16l-0.17-0.66v0l-0.17,0.12l-0.42-0.09l-0.41,0.04l-0.32,0.1
				l-0.13,0.33l-0.28-0.02l-0.29,0.16l-0.04-0.17l0.07-0.11l-0.08-0.04l-0.23,0.22l-0.81,0.39l-0.24,0.03l-0.11,0.33l-0.16,0.1
				l-0.55-0.28l0,0l-0.34-0.22l-0.73-0.99l0.15-0.37l-0.39-0.25l-0.28-0.33l-0.35,0.65l-0.22,0.18l-0.61-0.01l-0.2,0.07l-0.17-0.1
				l-0.5-0.04l-0.17,0.37l-0.05,0.61l-0.13-0.02l-0.17-0.17L289.82,92l-0.63-0.71l-0.7-0.38l-0.38-0.12l-0.49-0.27l-1.57-0.14
				l-0.33-0.09l-0.08,0.15l-0.49-0.14l-0.27,0.04l-0.38-0.06l-0.78,0.17l-0.51-0.31l-0.25,0.32l-0.43-0.13l-0.25-0.24l0,0
				l-0.41,0.41l-1.7,0.54l0.42,0.81v0l-0.42-0.8l-0.26-0.16l-0.29-0.59l-0.66-0.3l-0.48,0.03l0.09-0.32l-1.05-0.32l-0.64,0.05
				l-0.02-0.23l-0.17-0.07l-0.19-0.46l-0.16-0.58l-0.23-0.2l0,0l-0.34,0.23l-0.11-0.06l-0.24,0.2l-0.5-0.28l-0.22,0.25l-1.35-0.05
				l-0.04-0.09l-0.64,0.13l-0.04,0.16l-0.48,0.11l-0.51-0.17l-0.2-0.16l-0.41,0.19l-0.17,0.26l-0.71-0.19l-0.06,0.1l-0.3,0.12
				l-0.32,0.5l0,0l-0.06,0.07l-0.49-0.1l-0.06,0.09l-0.22,0.02l-0.09-0.1l-0.34-0.03l0.16-0.26l-0.09-0.08l0.02-0.18l-0.17-0.16
				l-0.35-0.2l-0.09,0.02l-0.2-0.17l-0.15,0.03l-1.05-0.19l-0.09-0.05l-0.65,0.12l-0.56-0.16l-0.24,0.08l-0.14,0.16l-0.27-0.03
				l-0.13-0.1l-0.25,0.14l-0.12-0.06l-0.1-0.27l-0.19-0.15l-0.26,0.01l-0.35-0.29l-0.38,0.02l-0.28,0.21l-0.18-0.04l-0.48,0.07
				l-0.14-0.11l-0.6-0.01l-0.4,0.27l-0.19,0.02l-0.31-0.25l-0.33,0.04l-0.16-0.19l0.03-0.15l-0.1-0.2l-0.09,0.04l-0.23-0.08
				l-0.09-0.21l0-0.12l0.1-0.08l-0.07-0.21l-0.33-0.25l-0.65,0.08l-0.28-0.37l-0.74-0.43l-0.13-0.03l0,0l-0.27,0.25l0.05,0.15
				l-0.12,0.23l-0.23,0.08l-0.07,0.11l0.05,0.22l0.28,0.37l0,0.17l0.16,0.33l0.16,0.14l-0.31,0.19l-0.08,0.17l-0.23-0.51l-0.55-0.13
				l-0.56-0.28l-0.33-0.02l-0.23-0.1l-0.53,0.06l-0.45,0.22l-0.08-0.03l-0.69,0.51l-0.26,0.38l0.18,0.29l-0.5,0.66l-0.4,0.07
				l-0.15-0.06l-0.95,0.64l-0.21,0.28l0.1,0.29l-0.23,1.14l0.08,0.21l-0.48,0.66l-0.36,0.05l-0.32,0.2l-0.33,0.07l-0.2-0.05
				l-0.14,0.16l0.03,0.26l-0.07,0.07l0.08,0.17l-0.02,0.23l-0.1,0.05l-0.02,0.11l0.09,0.11l-0.06,0.16l-0.13-0.01l-0.02-0.1
				l-0.12,0.04l-0.2,0.25l0.03,0.28l0.07,0.07l-0.11,0.11l0.04,0.15l-0.04,0.05l-0.38-0.06l-0.18,0.21l-0.13-0.1l-0.55,0.48
				l-0.02,0.14l0.12,0.44l0.36,0.13l0.38,0.44l0.73,0.07l0.81-0.2l0.07,0.86l0.34,0.29l0.14,0.35l-0.21,0.18l-0.13,0.46l-0.27,0.34
				l-0.13,0.47l-0.31,0.17l0.02,0.12l-0.16,0.28l-0.05,0.2l0.05,0.18l0.28,0.52l0.2-0.06l0.84,0.25l0.3,0.82l0.25,0.48l0.1,0.04
				l0.29-0.13l0.39,0.12l0.51,0.01l0.81,0.24l0.28-0.03l0.45-0.2l0.2-0.18l1.02,0.28l0.35-0.16l0.23,0.08l0.36-0.13l0.44-1.28
				l-0.05-1.2l0.24,0.09l0,0l0.32,0.12l0.49-0.02l0.3,0.07l0.39,0.2l0.07,0.5l0.66,1.67l0.98,0.34l0.48-0.09l0,0.06l0.62,0.23
				l2.58-0.47l0.42,0.12l0.33-0.07l0.15-0.29l0.29-0.12l0.28,0.04l0.42-0.79l0.02-0.26l0.44,0.33l0,0l0.15,0.06l1,0.87l0.13,0.46
				l-0.01,0.28l-0.15,0.07l0.08,0.11l-0.1,0.09l0.06,0.09l-0.37,0.73l0.04,0.15l-0.17,0.62l-0.19,0.09l-0.21,0.57l1.34,0.37
				l0.81,0.07l0.09,0.25l-0.16,0.31l-0.23,0.22l0.13,0.35l-0.19,0.2l-0.06,0.2l0.1,0.2l-0.25,0.43l0.15,0.38l-0.13,0.25l0.16,0.15
				l-0.3,0.12l0.08,0.22l-0.17,0.08l-0.05,0.06l0.05,0.26l-0.21,0.09l0.13,0.22l-0.15,0.04l0.01,0.19l-0.17,0.1l0.15,0.38
				l-0.14,0.43l0.13,0.08l-0.24,0.08l-0.02,0.24l-0.15,0.16l-0.06,0.23l0.01,0.43l-0.13,0.21l-0.27,0.09l-0.16,0.43l0.05,0.07
				l0.25,0.01l0.09,0.18l0.15,0.04l-0.09,0.18l0.09,0.5l0.15-0.08l0.1,0.04l-0.24,0.26l0.21,0.14l-0.16,0.04l0.22,0.21l-0.07,0.12
				l0.17,0.21l-0.03,0.21l0.16,0l0.02,0.14l-0.11,0.14l-0.18-0.2l-0.18,0.06l0.02,0.42l0.21,0.42l0.26,0.28l0.29,0.21l0.28,0.08
				l0.63,0.74l0.55,0.22l0.41-0.02l0.81-0.49l0.43-0.01l0.21-0.46l0.05-0.33l0.28-0.18l0.19,0.03l0.14,0.18l0.09-0.08l0.36,0.17
				l0.23-0.04l0.35,0.08l0.05-0.11l0.16-0.04l0.42,0.21l0.83,0.18l0.49-0.09l0.09-0.11l0.16,0.03l1.37,0.54l0.81,0.21l0.59,0.34
				l1.41,1.26l0.74,1.39l0.24,0.19l0.59,0.22l0.26,0.01l0.31,0.27l0.47,1.04h0l-0.54,0.39l-0.35,0.4l-0.25,0.52l-0.01,0.41
				l-0.4,0.48l-0.36,0.63l-0.18,0.11l-0.11,0.19l-0.51,0.16l0.22,0.71l-0.1,0.13l-0.17,0.1l-0.31-0.09l-0.08,0.04l-0.07,0.17
				l0.24,0.41l-0.34,0.01l-0.13,0.15l-0.39,0.2l-0.1,0.15l0.03,0.44l-0.09,0.2l-0.34,0.27l-0.36,0.5l-0.06,0.12l0.05,0.16l0.4,0.54
				l-0.17,0.3l-0.39,0.55l-0.28,0.21l-0.52,0.04l-0.16,0.21l0,0.33l0.5,0.24l0.12,0.23l0.07,0.32l-0.1,0.24l-0.27-0.06l-0.74,0.4
				l-0.38,0.08l-0.34-0.23l-0.22-0.36l0.02-0.25l-0.2,0.08l-0.26,0.26l-0.12-0.02l-0.38,0.14l-0.16,0.41l-0.01,0.37l0.26,0.38
				l0.29-0.02l0.14,0.12l-0.1,0.25l-0.31,0.21l-0.27,0.18l-0.57,0l-0.36,0.13l-0.42,0.71l0.34,0.41l-0.57,0.32l-0.12,0.15
				l-0.14,0.31l0.07,0.35l-0.57,0.4l-0.29,0.02l-0.43-0.17l-0.42,0.36l0.03,0.32l0.06,0.04l0.49,0.07l0.45-0.32l0.36,0.44l0.25,0
				l0.62,0.23l0.09-0.15l0.19-0.01l0.12,0.1l-0.03,0.27l0.59-0.15l0.35-0.01l0.35-0.37l0.42-0.03l0.25,0.09l0.09,0.16l0.4,0.16
				l0.21,0.17l0.05,0.15l0.51,0.06l0.16,0.12l0.24,0.06l0.3-0.14l0.25,0.4l0.41,0.19l0.1,0.14l0.42,0.14l0.44,0l0.14-0.13l0.25,0.08
				l0.04,0.1l0.6,0.61l0.72,0.22l0.05,0.08l0.08,0.22l0.24,0.27l0.85,0.42l0.12-0.01l0.32,0.27l0.07,0.06l0.52,0.28l0.57,0.06
				l0.84,0.53l0.37,0.4l0.33-0.34l0.08-0.34l0.13-0.04l0.48-0.48l0.38,0.12l0.3-0.12l0.75,0.06l0.19-0.29l0.15-0.05l0.15-0.18
				l0.14,0.34l0.48,0.31l0.27,0.04l0.63,0.28l0.63,0.14l-0.09,0.36l-0.29,0.19l-0.37,0.6l-0.03,0.68l0.26,0.61l0.49,0.17l0.29-0.06
				l0.71-1.09l0.24-0.07l0.21,0.11l0.11,0.2l-0.02,0.35l0.13,0.3l0.09,0.05l0.15,0.37l-0.01,0.16l0.31,0.13l0.09,0.19l0.08-0.01
				l0.25-0.21l-0.11-0.19l0.15-0.15l0.41-0.05l-0.11-0.28l0-0.11l0.2,0.05l0.04-0.1l-0.08-0.35l0.24-0.31l0.19-0.22l0.26,0.08
				l0.1-0.22l-0.09-0.22l0.05-0.07l-0.09-0.2l-0.07-0.93l-0.14-0.71l0.33-0.53l-0.01-0.15l0.21-0.08l0.43-0.34l0,0l0.26,0.15
				l0.54-0.08l0.35,0.25l0.19,0.25l0.14-0.03l0.22,0.21l0.21-0.02l0.28,0.15l0.35,0.1l1.07-0.22l0.18,0.2l0.34,0.07l0.21,0.19
				l1.03,0.01l0.37-0.43l0.2-0.11l0.66-0.1l1.36-0.36l0.9,0.13l0.46-0.67l0,0l0.2-0.08l0.24,0.01l0.78,0.72l0.13,0.01l0.25,0.94
				l0.35-0.06l0.37,0.09l0.24,0.31l0.68,0.3l0.68-0.75l0.25-0.04l0.1,0.1l0.31,0.08l0.48,0.46l0.72,0.51l0.29,0.05l0.14,0.21
				l0.05,0.31l0.27,0.26l0.4,0.16l0.46-0.08l-0.1,0.39l-0.01,0.91l0.16,0.5l0.14-0.07l0.04-0.16l0.15-0.05l-0.04-0.04l0.12-0.14
				l0.04,0.03l0.14-0.24l-0.23-0.57l0.02-0.38l-0.07-0.14l2.04-0.49l0.33-0.01l0.43,0.11l0.66-0.99l0.84,0.36l0.38-0.12l0.55-0.35
				l0.19-0.09l0.06,0.12l-0.05,0.46l0.09,0.16l0.42-0.54l0.59-0.42l0.17-0.25l0.05-0.17l-0.06-0.09l0.13-0.04l0.03-0.15l0.12-0.09
				l0.08,0.13l0.37-0.12l-0.18,0.01l0.1-0.18l-0.09-0.03l-0.03-0.1l0.07-0.23l0.09-0.02l-0.09-0.06l0.07-0.17l-0.29-1l0.03-0.19
				l0.4-0.34l-0.2-0.08l0.07-0.64l-0.09-0.53l-0.26-0.43l0.02-0.43l-0.13-0.17l0.12-0.24l-0.08-0.12l-0.42-0.12l-0.25-0.32
				l-0.09-0.76l-0.35-0.96l-0.01-0.35l-0.05-0.31l0.06-0.59l0,0l0.47-0.77l0.04-0.3l0.38-0.39l0.15-0.29l-0.44-0.37l-0.85-0.26
				l0.24-0.57l0.02-0.42l0.23-0.17l-0.2-0.16l0.3-0.28l-0.2-0.23l-0.22-0.01l-0.1-0.08l-0.29-0.97l0.11-0.11l0.2,0.27l0.09,0.06
				l0.12-0.04l-0.01-0.11l0.34-0.12l0.52-0.55l0.48-0.53l0.25-0.07l0.09,0.04l0.55-0.3l0.62,0.14l0.32-0.32l0.5-0.54l0.32,0.11
				l0.2-0.58L319.83,124.22z"></path>
		</g>
		<g id="soria_p">
			<path class="st6" d="M459.82,113.8l-0.36-0.2l-0.14-0.3l-0.41,0.08l-0.46-0.22l0.26-0.78l-0.75-0.8l-0.16-0.41h0l-0.01-0.25
				l-0.58-0.47l-0.54-0.13l0.37-0.88l0.4-0.47l0.34-0.23l0.28-0.59l-0.09-0.92l0.02-0.2l0.11-0.15L458,106.4l-0.1-0.16l-0.39,0.11
				l-0.32-0.64l0-0.12l0.05-0.62l-0.75-1.1l-0.05-1.24l0.15-0.23l-0.07-0.49l0.08-0.71l-0.07-0.39l-0.85,0.75l-0.53,0.25l-0.65,0.12
				l0.06,0.14l-0.25,0.13l-0.16,0.45l-0.14-0.06l-0.12-0.05l-0.33,0.25l-0.13-0.05l-0.11,0.19l-0.25,0.08l-0.54-0.11l-0.26,0.38
				l-0.84,0.06l-0.66-0.62l-0.12-0.42l-0.48-0.12l-0.12-0.22l-0.14,0.02l-0.08-0.21l-0.27-0.25l-0.1,0.27l-0.15,0.09l0.03,0.15
				l-0.09-0.01l-0.02,0.23l-0.32-0.36l-0.04-0.21l-0.24-0.04l-0.44-0.19l-0.57,0.19l0,0l-0.09-0.02l-0.07-0.16l-0.25,0l-0.2-0.1
				l-0.38,0.06l-0.39-0.14l-0.25-0.31l0.04-0.14l-0.05-0.46l-0.12-0.1l-0.02-0.51l-0.17-0.49l0.19-0.09l0.3,0l0.18-0.31l-0.31-0.47
				l-0.07-0.54l-0.48-0.25l-0.65-0.83l-0.23-0.03l0.03-0.15l-0.2-0.11L444.29,96l0.02-0.22l-0.14-0.36l0.42-0.12l0.38-0.29
				l0.27-0.02l0.34-0.34l0.01-0.39l-0.06-0.08l0.04-0.41l-0.38-0.32l-0.08,0.04l-0.38-0.11l-1.11-0.02l-0.93,0.12l-0.42-0.12
				l-0.31,0.24l-0.3,0.1l-0.27-0.18l-0.29-0.04l-0.85,0.8l-0.43-0.07l-0.49-1.07l0.05-0.32l-0.08-0.24l0.15-0.19l-0.13-0.5
				l-0.25-0.18l-0.79-0.28l-0.32-0.23l-0.14-0.04l-0.31,0.08l-0.21-0.13l-0.48,0.03l-0.17-0.06v0l-0.12,0.1l-0.79,0.08l-0.22,0.11
				l-0.47-0.06l-0.38,0.3l-0.17-0.14l-0.66,0.03l-0.38,0.17l-0.47-0.08l-0.11,0.04l-0.21,0.23l-0.08,0.44l-0.2,0.15l-0.21,0.32
				l-0.53,0.05l-0.11,0.17l-0.4,0.25l-0.24-0.25l-0.23-0.11l-0.21,0.03l-0.3-0.16l-0.25,0.05l-0.07,0.2l0.01,0.44l-0.13,0.25
				l0.05,0.39l-0.13,0.17l-0.04,0.25l0.13,0.1l0.19,0.36l-0.01,0.29l0,0l0,0l-0.71,0.73l-0.12,0.06l-0.17-0.04l-0.39,0.19
				l-0.12,0.32l0.03,0.36l-0.19,0.22l-0.38,1.12l-0.4,0.39l-0.19,0.56l-0.68-0.31l-0.53-0.05l-0.22-0.15l-0.38,0.05l-0.09-0.08
				l-0.95,0.15l-0.4-0.08l-0.08,0.13l-0.47,0.26l-0.55-0.18l-0.37-0.5l-0.39-0.24l-0.18,0.08l-0.41-0.11l-0.28,0.05l-0.77,0.34
				l-0.09-0.15v0l-0.02-0.2l-0.28-0.47l0.03-0.33l-0.1-0.14l0.77-0.92l0.16-0.55l0.29-0.3l0.07-0.31l0.12-0.06l0.57-0.74l0.03-0.23
				l-0.11-0.32l0.07-0.27l0.11-0.06l-0.03-0.17l-0.63-0.54l-0.25-0.39l-0.4-0.1l-0.32,0.05l-0.19,0.33l-0.58-0.38l-0.38,0.07
				l-0.3,0.51l-0.38,0.29l-0.07,0.42l0.12,0.18l0,0.18l-0.53,0.74l0.01,0.13l0.19,0.15l0.07,0.15l0.08,0.48l-0.14,0.23l0.02,0.67
				l-0.25,0.03l-0.34,0.52l-0.1,0l-0.35-0.24l-0.4,0.15l-0.27,0.19l-0.18,0l-0.49,0.3l-0.06,0.33l-0.16,0.19l-0.22,0.08l-0.07,0.15
				l-0.29,0.17l-0.29,0.13l-0.23-0.05l-0.07-0.36l-0.91-0.31l-0.75,0.26l-0.21,0.54l-0.53,0.76l-0.05,0.31l-0.57,1.19l-0.07,0.43
				l0.31,0.36l-0.19,0.78l-0.25,0.41l1.01,0.53l0.05,0.29l0.2,0.21l0.12,0.12v0l-0.12-0.12l-0.2-0.21l-0.05-0.29l-1.01-0.53
				l-0.47,0.06l-0.21-0.18l-0.44-0.06v0l-0.39,0.56l-0.13,0.43l0.14,0.29l-0.36,0.56l0.06,0.12l-0.07,0.5l-0.7-0.12l-0.16,0.18
				l-0.27-0.08l-0.02-0.12l-0.54-0.1l-0.2,0.43l-0.48,0.01l0,0l-0.39-0.29l0.05-0.25l-0.24-0.49l0-0.37l-0.38-0.22l0,0l0.2,0.62
				l-0.18,0.49l-0.15,0.2l-0.94-0.69l-0.83-0.49l-0.33,0.9l0.1,0.33l-0.04,0.21l-0.11,0.03l-0.17-0.12l-0.25-0.01l-0.03,0.19
				l-1.16,0.42l-0.21,0.29l-0.75-0.57l-0.96,0.84l0,0l-0.13-0.09l-0.41-0.27l-0.25,0.15l-0.24-0.07l-0.08-0.52l-0.15-0.18
				l-0.36-0.09l0,0l-0.24,0.4l0.23,0.37l0.2,0.09l-0.24,0.87l0.04,0.32l-0.21,0.18l0.01,0.75l0.17,0.07l0.14,0.23l-0.15,0.45
				l0.26-0.01l-0.04,0.12l0.1,0.21l0.06,0.47v0.3l-0.42,0.33l0.09,0.54l0.33,0.51l-0.34,0.35l-0.09,0.12l0.28,0.22l0.17,0.55
				l0.33,0.32l0.17,0.13l0.24,0.01l0,0l-0.24-0.01l-0.17-0.13l-0.33-0.32l-0.17-0.55l-0.28-0.22l-0.41-0.11v0l-0.45,0.1l-0.1,0.49
				l-0.49,0.1l-0.05,0.44l-0.25,0.02l-0.05,0.07l-0.21-0.11l-0.31,0.15l-0.39,0.04l-0.09,0.22l0.11,0.25l-0.33,0.41l0.03,0.43
				l-0.08,0.18l0.04,0.58l0.1,0.32l-0.49,0.09l-0.6-0.06l-0.33,0.13l-0.05,0.33l-0.51,0.48l0.01,0.13l-0.31,0.45l-0.12,0.46
				l0.05,0.45l-0.1,0.2l-0.1-0.11l-0.33-0.01l-0.13,0.35l-0.09,0.02l-0.06,0.28l-0.16,0.25l0.04,0.24l0.16,0.16l-0.23,0.2
				l-0.07,0.36l-0.29,0.32l-0.38,0.87l-0.26,0.02l-0.12,0.02l0.01-0.18l-0.29-0.25l-0.17,0.13l-0.07-0.01l-0.01-0.15l-0.53,0.03
				l-0.45-0.07l-0.08,0.08l-0.09-0.03l-0.06-0.06l0.04-0.11l-0.25-0.17l-0.28-0.35l-0.23-0.15l-0.12,0.09l-0.32,0.13l-0.02,0.58
				l-0.2-0.04l-0.19,0.11l0.58,0.54l-0.22,0.48l-0.09,0.3l0.29,0.68l-0.07,0.4l0.14,0.18l0.41,0.19l0.24,0.31l0.68,0.55l0.68,0.28
				l0.22,0.17l0.62-0.37l0.25-0.49l0.08,0.12l0.17,0.12l0.38,0.44l0.41,0.3l0.32,0.11l0.15,0.22l-0.09,0.5l-0.15,0.31l0.1,0.81
				l0.43,0.36l0.4,0.01l0.53,0.28l0.25,0.27l0.28,0.08l0.46-0.04l0.04-0.13l0.1,0.12l-0.02,0.17l0.27-0.04l0.33,0.35l0.04-0.08
				l0.49-0.11l0.03-0.09l0.16,0.06l-0.35,0.42l-0.05,0.46l-0.24,0.83l0.19,0.31l-0.01,0.34l-0.2,0.17l-0.11,0.36l0.08,0.14l0,0.41
				l-0.14,0.11l0.38,0.18l0.49,0.82l0.94,0.43l0.13,0.28l0.59,0.15l0.29,0.45l0.36-0.04l0.11,0.6l0.86,0.62l0.48-0.01l-0.02,0.63
				l0.67,0.43l0.47,0.15l0.07,0.05l-0.02,0.13l0.15,0.06l0.16-0.07l0.31,0.09l0.15-0.07l0.4,0.22l0.31,0.04l0.24,0.2l0,0l0.67,0.18
				l0.29-0.01l0.46,0.22l0.2-0.15l0.66,0.16l0.34,0.42l0.09,0.01l0.16-0.22l0.16,0.03l0.26,0.2l0.42,0.1l0.45,0.23l0.05-0.17
				l0.29-0.02l0.73-0.23l0.71-0.42l0.27,0.2l0.22,0.05l1.08-0.34l0.48-0.06l0.05-0.07l0.65,0.03l0.25-0.26l0.16-0.46l-0.04-0.43
				l0.29,0.03l0.4-0.28l0.54-0.14l0.17-0.34l0.35,0.15l0.39-0.02l0.23,0.19l0.29,0.02l0.26,0.23l0.09,0.21l-0.25,0.78l0.29,0.36
				l0.07,0.24l0.02,0.33l-0.13,0.37l1.39,0.58l0.43,0.28l-0.02,0.09l0.76,0.01l0.39,0.12l0.47,0.03l0.28,0.2l-0.04-0.4l0.37-0.63
				l0.26-0.26l0,0l1.25,0.13l0.54-0.22l0.11,0.78l0.18,0.49l0.57,0.34l1.05,0.46l0.59-0.19l0.36,0.02l0.04,0.12l0.42,0.32
				l-0.06,0.44l0.29,0.46l0.16-0.35l0.05-0.11l0.28,0.23l1.11-0.27l0,0l0.04,0.08l0.04,0.5l-0.23,0.69l-0.31,0.33l-0.2,0.07
				l-0.58-0.1l-0.08,0.4l0.11,0.25l0.68-0.29l0.99,0.51l0.04,0.08l-0.22,0.72l-0.09,0.16l-0.36,0.26l-0.09,0.27l0.6,0.58l0.23,0.07
				l0.32-0.29l0.5-0.09l0.54-0.24l0.22-0.62l0.79,0.86l-0.02,0.51l0.23,0.29l0.11,0.68l0.21,0.07l0.67,0.03l0.07-0.05l0.21,0.23
				l0.44,1.28l0,0.63l0.81,0.02l0.06,0.15l0.23-0.13l0.09,0.22l-0.04,0.1l0.45-0.05l0,0.35l0.38,0.3l0.06,0.14l0.19-0.17l0.49-0.04
				l0.77-0.67l0.07-0.15l0.35-0.11l0.7-0.39l0.25,0.49l0.62,0.54l0.13,0.03l0.3,0.55l0.32-0.27l0.5-0.18l0.39-0.34l0.1,0.02
				l0.27,0.3l0.85-0.12l1.27-1.58l0.39,0.19l1.91,0.52l-0.02-0.15l0.34-0.47l0.97-0.39l0.31-0.76l0.16-0.08l0.58,0.82l0.32,0.29
				l0.48,0.27l0.42,0.13l0.23,0.2l0.19,0.24l-0.08,0.24l0.28,0.27l0.3,0.13l0.31,0.02l0.33-0.47l-0.38-0.34l-0.33-0.6l0.08-0.42
				l0.13-0.09l-0.09-0.43l0.14-0.36l0.35-0.33l0-0.19l-0.04-0.94l-1.04-0.88l-0.1-0.42l-1.09-0.23l-1.6-0.49l-0.11-1.85l-0.24-1.16
				l-0.15-1.32l-0.2-0.98l-0.41-0.26l0.03-0.41l0.16-0.29l0.07-0.54l-0.13-0.41l0.06-0.2l0.19-0.33l0.26-0.16l0.05-0.31l-0.24-0.56
				l-0.03-0.4l0.29-0.08l0.25,0.05l0.73-0.67l0.24-0.07l0.23-0.54l0.17-0.13l-0.14-0.12l0-0.32h0l-0.04-0.21l0.09-0.1l0.09-0.67
				l-0.03-0.87l0.19-0.54l0.18-0.16l0.16-0.09l0.09-0.33l2.2,0.62l0.46,0.4l0.03,1.21l-0.36,0.39l0.05,0.44l0.16-0.02l0.17,0.16
				l0.09-0.03l0.11,0.14l0.17,0.02l0.35,0.01l-0.01-0.08l0.29-0.05l0.31-0.43l0.34-0.09l0.25-0.03l0.22-0.34l0.46,0.07l0.33-0.13
				l0.36-0.28l0.64,0.07l0.02-0.23l-0.07-0.06l-0.03-0.23l0.14-0.16l-0.23,0l-0.46-0.24l0.2-0.29l0-0.33l-0.15-0.52l0.04-0.57
				l-0.07-0.34l-0.68-1.02l-0.45-0.27l0.18-0.55l-0.09-0.4l0.15-0.64l-0.13-0.57l0.25-0.33l-0.04-0.07l0.23-0.64l-0.13-0.14
				l0.07-0.11l-0.2-0.39l-0.13,0.03l-0.21-0.25l0.05-0.14l-0.53-0.67l-0.05-0.25l0.12-0.46L451,120.5l0.36-0.37l0,0l0.34,0.22
				l0.27-0.15l0.46,0.04l1.41,0.35l0.19-0.44l0.17-0.11l0.02-0.4l0.23-0.26l0.31-0.28l0.1-0.33l0.77-0.46l0.2,0.16l0.61-1.27
				l0.99,0.05l0.22-0.11l0.65,0.11l0.28-0.2l-0.12-1.1l-0.12-0.1l0.42-0.52l0.27-0.15l0.04,0.07l0.28-0.09l-0.05-0.18l0.13-0.77
				L459.82,113.8z"></path>
		</g>
		<g id="avila_p">
			<path class="st6" d="M362.92,172.87l-0.18-0.27l0.1-0.18l-0.05-0.21l0.15-0.29l0.01-0.24l-0.05-0.86l-0.18-0.16l0-0.37l0.07-0.32
				l0.47-0.55l-0.19-0.18l-0.29,0.04l-0.46-0.3l0,0l-0.16,0.12l-1.13,0.16l-0.68,0.25l-0.53-0.09l-0.24,0.08l-0.06-0.06l-0.23,0.19
				l-0.28,0.05l-0.32,0.2l-0.07,0.27l0.21,0.04l0.27,0.46l0.27,0.17l-0.04,0.56l-0.42-0.08l-1.36,0.11l-0.73,0.28l-0.34,0.01
				l-0.28,0.42l0.09,0.13l-1.3-0.02l-0.97,0.14l-0.22,0.09l-0.54-0.06l-0.5,0.05l0.02-1.16l-0.12-1.09l-0.4-1.55l-0.11-0.11
				l-0.04-0.52l0.14-0.07l0.09-0.59l0.13-0.08l-0.15-0.35l-0.02-0.33l-0.31-0.4l-0.64-0.26l-0.22-0.23l-0.29-0.55l-0.27-0.3
				l0.05-0.4l-0.33-0.22l-0.03-0.28c0.02-0.03,0.04-0.06,0.06-0.09c0.01-0.01,0.01-0.02,0.02-0.03c0.03-0.13,0.08-0.34,0.14-0.5
				l-0.02-0.02l-0.01-0.23l-0.37-0.36l-0.03-0.32l-0.28-0.71h-0.2l-0.01,0l0-0.21l-0.25-0.31l-0.15,0.02l0.02-0.21l-0.09-0.11
				l0.14-0.49l0.29-0.24l0.36-0.14l0.1,0.04l0.34-0.24l-0.02-0.13l-0.56-0.66l-0.17-0.1l-0.45-0.02l-0.29,0.86l-0.69,0.28
				l-0.08-0.07l0.02-0.37l-0.14-0.14l-0.49-0.24l-0.04-0.14l-0.32-0.2l0-0.01l0.18-0.49l-0.05-0.42l0.16-0.25l-0.28-0.31v-0.18
				l0.04-1.27l-0.14-0.02l-0.2-0.27l0.28-0.32l0.15-0.32l0.82-0.61l0.16-0.02l0.07-0.42l0.24,0.04l0.03,0.1l0.43,0.04l0.18-0.35
				l-0.06-0.22l0.19-0.51l-0.08-0.08l0.06-0.15l-0.19-0.42l0.04-0.08l-0.72-0.36l-0.33,0.03l-0.64-0.4l-0.1,0.03l-0.14-0.28
				l-0.04-0.1l-0.39-0.64l-0.01-0.15l0.28-0.66l0.12-0.09l-0.08-0.07l0.07-0.36l-0.22,0l-0.29-0.71l0.15-0.29l-0.01-0.57l0.09-0.05
				l-0.18-0.39l0.13-0.27l-0.18-0.18l-0.23-0.65l-0.25-0.2l-0.68-0.66l-0.33-0.69l-0.11-0.09l-0.46-0.44l-0.21-0.58l-0.3,0.14
				l-0.35,0l-0.9-0.66l-0.59-0.29l-0.1,0.07l-0.06,0.2l-0.72,0.33l-0.27,0.41l-0.5,0.53l-0.03,0.41v0l-0.28-0.08l-0.34,0.02
				l-0.07,0.08l-0.4-0.04l-0.73,0.4l-0.47-0.14l-0.3,0.08l-0.11,0.2l-0.29,0.22l-0.39,0.03l-0.31,0.31l0.05,0.16l-0.19,0.52
				l-0.08,0.47l0.16,0.24l-0.53,0.23l-1.12-0.08l-0.33,0.36l0,0l-0.34-0.21l-0.02-0.07l0.1-0.06l-0.17-0.17l-0.18-0.1l-0.08,0.04
				l-0.53-0.47l-0.31-0.07l-0.13-0.23l-0.16,0l-0.11-0.05l-0.49-0.24l-0.29-0.28l-0.33-0.14l-0.47-0.48l-0.29,0.09l-0.2-0.06
				l-0.07-0.52l-0.12-0.14l-0.39,0.06l-0.29-0.06l0.08-0.14l-0.21-0.06l-0.08-0.26l-0.6,0.1l-0.12-0.16l-0.14,0l-0.12-0.12l0,0
				l0,0.11l-0.4,0.29l0.09,0.17l-1.13,0.61l-0.41,0.2l-0.33,0.01l-0.32-0.06l-0.31-0.19l-0.67-0.23l-0.09,0.24l-0.37,0.41
				l-0.06-0.07l-0.45,0.04l-0.04,0.19l-0.17,1.43l-0.42,0.3l-0.04,0.16l-0.12-0.02l-0.21,0.27l-0.11,0.37l0.05,0.08l-0.09,0.1
				l0.08,0.14l-0.11,0.84l0.15,0.24l0,0.22l0.09,0.06l-0.06,0.33l0.12,0.07l-0.05,0.16l0.21,0.17l0.02,0.18l0.12-0.12l0.27,0.17
				l1,0.36l-0.05,0.14l0.17,0.02l-0.06,0.18l0.13,0.07l-0.03,0.27l0.1,0l-0.03,0.14l0.36,0.4l0,0.45l0.09,0.1l-0.4,0.38l0.13,0.26
				l0,0.34l-0.06,0.07l0,0l-0.34,0.74l0.06,0.17l-0.16-0.01l0,0.11l-0.32,0.22l-0.18-0.1l-0.26,0.11l-0.02,0.3l0.2,0.01l0.01,0.17
				l-0.07,0.1l0.27,0.57l-0.14,0.08l0.03,0.07l0.2-0.03l0.05,0.08l-0.11,0.09l0.11,0.07l-0.09,0.12l0.2,0.29l-0.05,0.06l0.08,0.14
				l-0.14,0.07l0.08,0.07l-0.18,0.41l0.06,0.12l-0.24,0.07l-0.07-0.07l-0.16,0.12l-0.12-0.01l-0.2,0.2l0.04,0.06l-0.2,0.09
				l-0.03,0.09l0.11,0.05l-0.13,0.19l-0.38,0.32l0.02,0.23l-0.12,0.06l-0.02,0.12l-0.19,0.21l-0.27,0.09l0.02,0.16l0.06-0.05l0,0
				l-0.06,0.05l0.07,0.17l-0.11,0.29l0.04,0.15l-0.1,0.59l-0.14,0.17l0.03,0.19l0.07,0.12l0.1-0.01l0,0.18l0.11,0.08l-0.03,0.08
				l0.18,0.21l-0.05,0.2l0,0l-0.12,0.13l0.01,0.21l-0.18,0.03l-0.03,0.13l-0.44,0.32l-0.12,0.42l0.09,0.06l-0.25,0.41l-0.98,0.52
				l-0.42,0.33l0.08,0.12l0.11,0.69l-0.53,0.25l-0.19,0.25l-0.72,0.31l-0.21,0.18l-0.09,0.17l0,0l-0.09,0.2l-0.25,0.22l-0.46,0.25
				l-0.09,0.19l-0.42,0.32l-0.13,0.22l-0.6,0.24l-0.22,0.12l-0.35,0.03l-0.01-0.13l-0.27,0l-0.06,0.22l-0.28,0.1l0.17,0.1l0.04,0.16
				l-0.21,0.29l-0.18,0.13l-0.31-0.01l-0.63,0.41l-0.15-0.08l-0.37,0.17l-0.12,0.34l-0.2,0.01l-0.06,0.12l0.05,0.32l0.81,0.78
				l-0.51,0.18l0.21,0.36l-0.35,0.44l0.1,0.19l-0.15,0.35l-0.06-0.07l-0.09,0.12l-0.24,0.05l-0.18,0.25l-0.21,0.25l-0.26,0.03
				l-0.14-0.08l-0.09,0.03l-0.34,0.24l-0.06,0.42l-0.21,0.09l-0.17,0.26l-0.21,0.11l-0.09-0.06l-0.08,0.16l-0.13,0.06l-0.06,0.33
				l-0.68,0.11l0-0.11l-0.1-0.04l-0.76-0.06l-0.09-0.09l-0.48,0.09l0.03-0.45l-0.08-0.4l-0.2,0.39l-0.17-0.02l-0.25,0.27l-0.02,0.43
				l-0.03,0.21l0.06,0.15l-0.09,0.36l-0.42,0.52l-0.02,0.19l0.12,0.1l-0.02,0.38l0.07,0.04l0.29-0.26l0.38-0.17l1.08-0.43l0.67-0.19
				l0.18,0.04l0.29-0.07l0.62-0.3l0.28-0.05l0.22,0.57l-0.17,0.25l-0.14,0.95l-0.13,0.26l-0.24,0.16l-0.19,0.34l-0.13,1.21
				l-0.08,0.18l-0.39,0.16l0.01,0.33l0.05,0.59l-0.29,0.37l0.09,0.06h0l-0.16,0.2l-0.35,0.12l-0.18,0.18l-0.15-0.09l-0.08,0.05
				l-0.27-0.1l-0.22,0.02l-0.31-0.21l-0.07-0.04l0.02,0.24l-0.37,0.38l-0.07,0.22l0.02,0.07l0.34,0.16l0.07,0.27l-0.06,0.56
				l-0.1,0.23l-0.55,0.07l-0.51-0.1l-0.2,0.07l-0.4-0.58l-0.38-0.36l-0.43-0.2l0.06-0.21l0.1-0.45l0.16-0.04l0.23-0.23l0.04-0.29
				l-0.16-0.33l0.13-0.23l-0.57,0.04l-0.6,0.49l-0.81-0.04l-0.53,0.12l-0.15,0.16l-0.31,0.12l-0.27,0.49l-0.12,0.28l-0.52,0.51
				l-0.58,0.26l-0.17,0.26l-0.36,0.15l-0.3,0.13l-0.03,0.14l0.17,0.2l0.11,1.09l0.26,0.47l-0.34,0.56l-0.01,0.82l0.08,0.18
				l-0.05,0.13l0.54,0.37l-0.53,0.26l-0.16,0.32l-0.72,0.41l-0.26,0.26l-0.6,0.8l0.01,0.35l-0.35,0.52l0.13,0.09l0.25-0.04
				l0.43,0.11l0.5-0.01l0.2,0.12l0.31-0.17l0.4,0.44l1.06,0.74l-0.13,0.69l0.43,0.41l0.93,0.11l0.17,0.1l0.04,0.24l0.19,0.28
				l0.08,0.02l0.03,0.14l-0.21,0.51l0.36,0.45l0.47-0.07l0.3,0.08l0.93,0.48l0.19,0.2l0.47,0.11l0.15,0.18l0.28,0.07l0.19-0.04
				l0.22,0.1l0.49-0.12l0.37-0.27l0.39,0.06l0.03,0.1l0.08-0.06l0.36-0.25l0.69-0.17l0.11-0.88l0.64-0.16l0.51-0.35l0.4-1l0.7-0.16
				l0.71,0.26l0.66-0.4l0.59,0.05l0.26-0.32l0.36-0.12l0.23,0.06l-0.61,1.73l0,0.2l-0.18,0.28l-0.28,0.25l0.01,0.39l0.22,0.42
				l-0.17,0.22l0.07,0.17l0,0.68l0.08,0.08l0,0.4l-0.13,0.56l0.04,0.2l0.28,0.19l0,0.14l0.12,0.08l0.01,0.24l0.44,0.8l0,0.21
				l0.16,0.2l0.2,0.09l0.04,0.45l0.82,0.47l0.58-0.02l0.19-0.14l0.14,0.03l0.92-0.01l0.14-0.07l0.26,0.17l0.46-0.04l1.47,0.1
				l0.1,0.09l-0.05,0.68l0.16,0.17l0.04,0.21l0.19,0.15l0.4-0.04l0.29-0.24l0.13,0.01l0.09-0.54l0,0l-0.09,0.55l0.42,0.16l0.19-0.17
				l0.52-0.27l0.2,0.11l0.29-0.04l0.08-0.08l-0.01-0.2l0.63-0.4l0.78-0.79l0.16-0.45l0.27-0.02l0.15,0.21l0.08,0l0.12-0.13
				l0.04-0.51l0.3-0.04l-0.03-0.26l0.18-0.14l0.15-0.27l0.29-0.15l0.24,0.13l0.2-0.12l0.4,0.02l0.33-0.3l0.62-0.13l0.06,0.71
				l0.13,0.12l0.27,0.67l-0.03,0.51l-0.23,0.57l0.29,0.07l0.69-0.33l0.28,0.03l0.47-0.24l0.39-0.09l0.27,0.02l-0.18-0.27l0.14-0.01
				l-0.16-0.23l0.12-0.16l0.31,0.01l0,0l0.12,0.03l0.83-0.18l0-0.12l0.13-0.18l0.18-0.74l-0.23-0.52l-0.04-0.27l-0.23,0.14
				l-0.28,0.33l-0.4,0.13l0.4-0.13l0.28-0.33l0.23-0.14l0.58-0.3l0.55-0.51l0.49-0.32l0.25,0.04l0.12-0.4l0.53-0.4l0.1,0.09
				l0.42-0.3l0.37-0.05l0.2-0.13l0.59-0.59l-0.07-0.33l0.15-0.08l0.45-0.23l-0.1-0.3l-0.07,0.01l0.04-0.17l-0.09-0.31l-0.2-0.28
				l0.33-0.25l-0.05-0.25l0.17-0.21l-0.03-0.17l0.14-0.22v0l0.62,0.16l0.22,0.1l0.07,0.15l0.14,0.1l0.69,0.25l0.32-0.17l0.43-0.6
				l0.42,0.23l0.14,0.02l0.04-0.16l0.62-0.13l-0.06-0.26l0.63-0.08l0.16,0.85l-0.18,0.13l-0.17,0.02l0.03,0.34l0.36,0.18l0.23,0.23
				l-0.26,0.46l0.29,0.28l-0.12,0.35l0.14,0.58l-0.02,0.4l0.3,0.22l0.22-0.15l0.14,0.12l0.3,0.08l0.2,0.34l0.15,0.08l0.56,0.05
				l0.17-0.18l0.24-0.07l0.53,0.09l0.13-0.05l0.1-0.15l0.18-0.05l0.1-0.18l0.16,0l0.31-0.17l0.24-0.26l0.31-0.06l0.18-0.12
				l0.36,0.09l0.2-0.25l0.07-0.61l-0.05-0.13l-0.23-0.18l-0.13-0.94l0.52-0.39l0.02-0.08l0.07-0.56l-0.06-0.03l0.35-0.16l0.19-0.35
				l0.34-0.12l0.2-0.21l-0.35-1l-0.09-0.89l-0.18-0.13l0.06-0.03l-0.03-0.15l-0.17-0.31l0.05-0.15l0,0l0.33,0.08l0.48-0.23
				l0.47-0.15l0.54-0.39l0.11-0.34l0.32-0.34l0.19-0.14l0.23-0.05v0l-0.23,0.05l-0.19,0.14l-0.32,0.34l-0.11,0.34l-0.54,0.39
				l-0.47,0.15l-0.01,0.36l0.23,0.13l0.22,0.38l0.1,0.44l0.35,0.29l0.27-0.08l0.28-0.2l0.22-0.03l0.53,0.08l0.31,0.05l0.38-0.07
				l0.07-0.7l-0.28-0.4l0.14-0.23l0.31-0.06l0.04-0.08l0.37-0.16l0.05-0.22l-0.08-0.51l0.04-0.86l-0.12-0.31l0.06-0.15l0.17-0.08
				l-0.01-0.17l0.2-0.24l0.35-0.25l0.06-0.21l0.18,0l0.14-0.22l0.53-0.05l0.96,0.21l0.45-0.02l0.19-0.21l0.36-0.09l0.73,0.25
				l0.22-0.1l0.32,0l-0.22-0.16l0.09-0.27l0-0.31l-0.12-0.02l-0.09,0.07l-0.18-0.1l-0.06-0.13l-0.2-0.07l0.04-0.15l-0.06-0.32
				l0.35-0.19l-0.12-0.1l0.17-0.07l-0.06-0.22l0.38-0.04l0.1-0.11l-0.04-0.25l-0.21-0.06l0.02-0.16l0.05-0.05l0.19,0.12l0.07-0.03
				l-0.08-0.19l0.17-0.13l-0.12-0.1l0.07-0.13l-0.12-0.04l-0.11-0.34l0-1.32l-0.06-0.3l-0.11-0.02l0.18-0.55l0,0l0.07-0.78
				l-0.29-0.14l0.16-0.33l0-0.36l0.07-0.19l0.59-0.41l0.76-0.3l0.16-0.96l0.16-0.39l-0.07-0.31l0.17-0.31l0.01-0.17l-0.46-0.23
				l0.06-0.95l-0.05-0.11l0.42-0.14l0.28,0.08l0.06,0.35l0.36,0.92l0.26,0.11l0.54,0.06l1.1-0.14l0.9-0.45l0.67-0.16l0.61-0.38
				l-0.07-0.12L362.92,172.87z"></path>
		</g>
		<g id="burgos_p">
			<path class="st6" d="M429.54,63.26l0.05-1.02l0,0l0,0l-0.55-0.19l-0.16,0.05l-0.36,0.96l-0.78-0.09l-0.23,0.13l-0.18-0.01
				l-0.07-0.04l0.05-0.27l0,0l0,0l-0.65-0.86l-0.39-0.3l-0.21-0.02l0.04-0.6l0.2-0.08l0.11-0.35l0.32,0.25l0.1-0.01l0.7-0.77
				l0.1-0.4l0.13-0.26l0.1,0.02l0.08-0.23l0,0l0,0l-0.06-0.31l0.21-0.33l0.05-0.38v0l0,0L428.12,58l-0.26-0.01l-0.09,0.21
				l-0.32,0.23l-0.58,0.24l-0.1-0.07l-0.03-0.41l-0.41-0.36l0-0.15l0-0.1l-0.27-0.01l-0.21,0.13l-0.14-0.13l-0.12,0.08l-0.14-0.06
				l-0.28,0.21l-0.07,0.05l-0.03,0.01l-0.06,0.02l-0.2-0.08l-0.15,0.09l-0.08-0.13l-0.24-0.02l-0.07-0.25l-0.56-0.13l-0.03,0.11
				l-0.45-0.39l-0.59,0.04l-0.05-0.15l-0.29-0.13l-0.35,0.04l-0.32-0.27l-0.38-0.14l-0.14-0.18h0l-0.01,0l-0.48,0.12l-0.59-0.09
				l-0.37,0.08l-0.23-0.09h0l-0.25,0.07v0l-0.59-0.02l-0.67,0.09l-0.31,0.08l-0.36,0.3l-0.15-0.41l-0.2-0.22l-0.26,1.08l-0.14-0.1
				l-0.34,0.53l0,0.05l0,0.03l0.22,0.18l-0.16,0.06l-0.04,0.19l0,0l0,0l0.08,0.31l-0.19,0.01l0,0h0l0.07,0.3l-0.31,0.08l-0.26,0.3
				l0,0l0,0l0.14,0.15l0.43,0.04l0.3,0.3l-0.34,0.11l0,0l0,0l0.15,0.2l0.3,0.03l0.24,0.16l0.38,0.13l0.15,0.54l0.58,0.12l0.17,0.24
				l0.45,0.24l0.44-0.05l0.12,0.36l0.12,0.36l0.32,0.49l-0.01,0.21l-0.2,0.13l0.21,0.08l0.11,0.14l0.77,0l0.45,0.15l0.11,0.05
				l0.35-0.08l0.34,0.09l0.14-0.1v0l0-0.14l0-0.16l0.38-0.23l0.25-0.05l0.78,0.17l0.08,0.13l0.29-0.03l0.17,0.19l0.29-0.34
				l0.21,0.13l0.09,0.48l0.43,0.46l0.14,0.15l0.11,0.1h0l0.12-0.08l0,0l0.53,0.03l0.22,0.1l0.38-0.15l0.11,0.09l0.05-0.15l0.61-0.05
				l0.81,0.38l0.19-0.15l0,0l0,0l-0.02-0.05l0.16-0.07l0.2,0.02l0.17,0.22l-0.15,0.23l0,0l0,0l0.1,0.17l0.44-0.06l0.58-0.34
				l0.26-0.07v0l0-0.43L429.54,63.26z M415.58,64.67l-0.15-0.08l-0.11-0.15l-0.59-0.33l-0.28-0.31l-0.19-0.69l0.14-0.09l-0.11-0.19
				l0.12-0.24l0.07-0.1l-0.12-0.16l-0.14-0.01l-0.1-0.21l0.05-0.39l-0.65-0.23l-0.13-0.18l-0.35,0.08l0.09-0.27l-0.1-0.19
				l-0.67-0.08v0l-0.38-0.04l-0.29,0.13l-0.43-0.04l-0.27,0.26l-0.26-0.27l-0.1-0.56l-0.2-0.12l-0.53-0.09l-0.13-0.28l0.2-0.22
				l-0.05-0.06l-0.37-0.01l0-0.36l-0.06-0.04l-0.62,0.22l-0.31-0.08l-0.1,0.15l0.1-0.15l-0.09-0.28l-0.23-0.26l-0.07-0.39l0.12-0.27
				l-0.04-0.21l-0.04-0.04l-0.33,0.11l-0.1,0.08l-0.15,0.1l-0.18-0.05l-0.5-0.34l-0.06,0.06l0.04,0.23l-0.13,0.26l-0.19-0.17
				l-0.08-0.21l-0.39-0.16l-0.37,0.2l-0.23,0.23l-0.64,0.06l-0.39,0.96v0l0.39-0.96l-0.68,0.07l0.24-0.33l0.22-0.14l0.34-1.01
				l0.2-0.27l-0.3-0.02l-0.38-0.33l-0.77-0.48l0.45-0.53l-0.03-0.33l0.32-0.85l0.32-0.06l0.11-0.17l-0.05-0.22l0.14-0.26l0.2,0.04
				l0.08-0.33l0.31-0.33l-0.02-0.16l0.22-0.38l0.12-0.48l-0.33-0.09l0.03-0.26l-0.1-0.14l-0.16-0.11l-0.27,0l0,0l-0.5,0.81
				l-0.03,0.26l-0.24,0.14l-0.21,0.4l-0.44,0.27l-0.5-0.05l-0.71-0.66l-0.4,0.77l-0.26,0.2l-0.1,0.38l-0.52,0.3l0.67,0.6l-0.46,0.05
				l-0.36-0.19l-2.3-2.44l-0.17-0.52l0.42-0.1l-0.03-0.78l1.16-0.57l0.11-0.63l0.23-0.41l0.19-0.08l-0.01-0.58l0.44-0.31l0.19,0.01
				l0.14,0.09v0.13l0.12-0.03l0.09,0.09l0.07-0.01l0-0.12l0.13-0.01l0.16,0.11l0.24-0.08l0.64,0.1l0.46,0.23l1,0.3l-0.12,0.11
				l-0.12,0.71l0.05,0.12l0.32,0.22l0.39,0.03l0.22,0.47l0.73-0.11l0.15-0.22l0.25,0.22l0.23,0.31l-0.04,0.06l0.17,0.01l0.58-0.11
				l0.25-0.16l0.36,0.12l1.27-0.26l0.4-0.11l0.11-0.11l0.07-0.45l0.38-0.43l0.11-0.04l0.19,0.09l0.25-0.45l0.26-0.05l0.14-0.18
				l-0.14-0.29l-0.24-0.15l-0.22-0.32l-0.63,0.02l-0.32-0.14l-0.07-0.14l0.04-0.18l-0.18-0.05l-0.53-0.4l-0.11,0l-0.17-0.21
				l0.24-0.26l0.08-0.21l-0.17,0l-0.48-0.65l-0.23-0.13l-0.1-0.12l0.04-0.23l-0.47-0.11l-0.22-0.16l-0.09,0.2l-0.39,0.17l-0.69-0.2
				l-0.58-0.03l-0.16-0.1l-0.21-0.01l-0.61,0.06l-0.99-0.45l-0.34-0.36l0.28-0.19l0.03-0.19l0.7,0.11l0.38-0.02l0.06-0.13
				l-0.23-0.27l0.12-0.17l-0.03-0.6l0.13,0.02l-0.08-0.25l0.12-0.31l0.01-0.42l-0.03-0.08l-0.26-0.01l-0.43-0.28L404.1,41l0.16-0.36
				l0.55-0.62l-0.31-0.72l-0.26,0.13l-0.17,0.22l-0.42-0.26l-0.29-0.04L402.94,39l-0.05-0.25l0.82-1.2l-0.21-0.2l0.2-0.39l0.54-0.37
				l0.29-0.02l-0.06-0.36l-0.22-0.31l-0.37-0.21l-0.21,0.04l-0.22-0.08L403,35.83l-0.27,0.32h-0.08l-0.46-0.09l-0.24-0.19
				l-0.12-0.27l-0.13,0.08l-0.07,0.2l-0.06,0.04l-0.16-0.1l-0.25,0.28l0.02-0.11l-0.17-0.22l-0.09,0l-0.04-0.13l-0.09-0.01
				l-0.17-0.36l-0.64-0.69h0l-0.12,0.18l-0.25-0.05l-0.22,0.11l-0.41-0.01l0.02,0.24l-0.8,0.4l-0.12,0.19l-0.21,0.11l-0.14,0.24
				l-0.42,0.07l-0.03,0.08l-0.55,0.22l-0.2-0.1l-0.14,0.04l-0.35,0.67l-0.38,0.12l-0.45,0.03l-0.04,0.14l-0.68,0.47l-0.74,0.14
				l-0.13,0.15l-0.3,0.06l-0.17,0.06l-0.31-0.07l-0.16,0.1l-0.33-0.05l-0.13,0.07l-0.57-0.25h-0.87l-0.27,0.15l-0.61-0.18l-0.26,0
				l-0.9-0.34l-0.52-0.28l-0.71-0.07l-0.47-0.18l-0.52-0.09l-0.57,0.14l-0.16,0.12l-0.19-0.76l-0.17-0.31l-0.45-0.24l-0.3,0.06
				l-0.1-0.14l-0.3-0.03l-0.16-0.18l-0.12,0.01l-0.12-0.1l-0.29,0.18l0.08,0.37l-0.2,0.11l-0.05,0.19l-0.18,0.13l-0.05,0.28
				l-0.35,0.14l0,0.21l-0.31,0.47l-0.06,0.28l0.07,0.05l-0.45,0.52l-0.36,0.17l0.04,0.29l-0.07,0.1l-0.39,0.12l-0.2,0.3l0,0l0,0
				l-0.37,0.19l-0.41,0.12l-0.39,0.02l-0.32,0.17l-0.16,0.38l0.11,0.38l-0.14,0.08l-0.28,0.17l-0.33-0.12l-0.1,0.11l-0.45,0.13
				l-0.47-0.27l-0.21-0.01l-0.16-0.17l-0.19,0.05l-0.24,0.26l-0.23,0.02l-0.36,0.01l-0.37-0.11l-0.1-0.1l-0.15,0.23l0,0.16
				l0.19,0.33l0.58,0.39l-0.09,0.05l0,0.23l-0.35,0.39l-0.29,0.18l0.03,0.37l-0.08,0.11l-0.19-0.01l-0.23-0.22l-0.14,0.06
				l-0.08-0.09l-0.33,0.01l-0.12,0.07L374.18,43l-0.08,0.3l-0.67,1.09l-0.62,0.31l-0.84,0.25l-0.62,0.72l-0.54,1.28l-0.63,1.87
				l-0.05,0.16l0.13,0.27l0.41,0.4l0.19,0.36l0.77,0.02l0.33,0.1l0.26-0.04l0.39-0.24l-0.27,0.35l-0.05,0.22l0.16,0.24l0.29-0.26
				l0.12-0.6l0.21-0.03l0.27-0.38l0.16-0.07l0.66-0.04l0.1-0.24l-0.03-0.28l0.08-0.14l0.01-0.59l0.39,0.05l0.31-0.1l0.12-0.28
				l1.12,0.81l0.09,0.3l0.3,0.35l-0.15,0.52l-0.19,0.09l-0.23-0.04l-0.11-0.12l-0.27,0.05l-0.16,0.57l-0.12,0.16l0.06,0.05
				l-0.39,0.25l-0.42-0.12l-0.17-0.16l-0.31,0.09l-0.48-0.37l-0.21-0.27l-0.01,0.12l-0.1,0.07l0.14,0.18l0.32,0.18l0.25,0.9
				l-0.4,0.06l-0.25-0.17l-0.06-0.13l-0.25,0.15l-0.15-0.08l-0.16,0.08l0.88,0.23l0.2,0.25l-0.33,0.54l-0.31,0.13l-0.09,0.37
				l0.1,0.1l0.56-0.05l0.11,0.2l0.35-0.03l0.31,0.29l0.26-0.08l0.21-0.2l-0.22-0.35l0.01-0.38l0.11-0.22l-0.12-0.04l0.24-0.58
				l0.17-0.21l0.48,0.34l0.61,0.25l0.46,0.55l0.32-0.14l0,0l-0.32,0.14L376.47,53l0.01,0.19l0.7,0.87l-0.01,0.31l-0.12,0.13
				l0.14,0.46l0.01,0.47l-0.18,0.23l-0.04,0.32l-0.07,0.08l-0.23,0.01l-1.02,0.6l-0.37,0.06l-0.92-0.63l-0.34-0.34l-0.21,0.74
				l-0.08,0.12l-0.26,0.03l-0.08,0.08l0.03,0.51l-0.09,0.46l-0.63-0.09l-0.17-0.23l-0.74,0.38l0.09,0.3l-0.4-0.02l-0.37,0.19
				l-0.53,0.03l0,0l-0.11-0.55l-0.37-0.14l-0.16,0.13l-0.21-0.02l-0.11,0.03l0.06-0.01l0.06-0.01l0,0l0-0.02l0.15-0.9l-0.06-0.08
				l0,0l0.04-0.6l0,0v0L369.88,56l-0.16-0.52l0.03-0.77l0,0v0l-0.14-0.39l-0.3,0.06l-0.39,0.59l0,0l0,0l0.24,0.62l-0.37,0.08l0,0
				l-0.36,0.27l0.05,0.41l0.06,0.07l0.08,0.1l-0.06,0.11l-0.15,0.09l-0.17-0.23l-0.24,0.11l-0.26,0.48v0l0,0l0.01,0.25l0.01,0.12
				l0,0v0l0.15,0.34l-0.35,0.19l-0.69,0.13l-0.39-0.05l-0.16-0.58l-0.35-0.06l-0.05-0.31l-0.18-0.3l0.05-0.35l-0.59-0.18l-0.39,0
				l-0.6,0.23l-0.24-0.05c0,0,0,0.01-0.01,0.01c0,0,0,0,0,0c0,0.03-0.01,0.06-0.01,0.09c-0.04,0.3-0.12,0.58-0.24,0.86
				c-0.1,0.22-0.23,0.42-0.32,0.65c-0.1,0.25-0.19,0.5-0.31,0.74c-0.21,0.44-0.48,0.8-0.84,1.11L362.17,60l0,0l0.05-0.08l-0.09-0.16
				l-0.11,0.01l-0.41,0.47l-0.01,0.18l-0.55,0.19l-0.06,0.15l0.06,0.13l-0.14,0.22l-0.29,0.07l-0.46,0.31l-0.37,0.03l-0.11-0.07
				l0.05,0.23l-0.38,0.32l-0.28-0.33l-0.15,0l0.04-0.59l-0.1,0.03l-0.48,0.28l-0.12,0.21l-0.37,0.26l0.26,0.24l0.03,0.15l-0.16,0
				l0,0.23l0.21,0.21l0.36,0.12l0.01,0.33l0.16,0.57l-0.22,0.22l0.12-0.03l0.04,0.39l-0.16,0.11l0.11,0.33l0.15,0.08l0.13,0.21
				l-0.12,0.46l0.27,0.29l-0.41,0.05l-0.35,0.39l-0.41,0.2l-0.08,0.16l0.23,0.24l0.03,0.28l0.11,0.1l-0.03,0.11l0.15,0.14
				l-0.16,0.19l0.09,0.09l0.37-0.1l0.17,0.09l0,0l0,0l-0.24,0.09l-0.23,0.21l-0.45,0.09l-0.2,0.2l-0.27-0.01l0.31,0.56l-0.22,0.07
				l-0.01,0.2l0.14,0.15l0.89,0.42l-0.05,0.1l0.06,0.39l-0.09,0.09l0.17,0.27l0.13,0.47l-0.18,0.29l-0.39,0.14l0.53,0.08l0.42-0.08
				l0.42,0.46l0.21,0.46l0.04,0.12l-0.18,0.7l0.23,0.4l-0.22,0.02l-0.16,0.16l0.09,0.13l0.19-0.06l0.08,0.23l-0.06,0.14l0.03,0.08
				l0.11-0.01l0.04,0.14l-0.36-0.02l-0.7,0.17l-0.29-0.09l-0.26,0l-0.1,0.09l-0.89-0.09l-0.01,0.09l-0.41-0.3l-0.24,0.77l-0.5,0.75
				l0.08,0.4l-0.07,0.17l0.07,0.2l0.11,0.06l0.08,0.29l-0.07,0.27l0.06,0.49l-0.1,0.24l0.38,0.59l0.65-0.19l0.49-0.25l0.47-0.05
				l0.24-0.15l-0.07,0.29l0.2,0.05l0.05,0.26l-0.1,0.31l-0.27-0.14l-0.02,0.25l0.17,0.25l0.01,0.18l-0.09-0.04l-0.07,0.11l0.06,0.14
				l0.2,0.14l0.19,0.08l0.19,0.25l-0.06,0.28l-0.15,0.21l0.01,0.22l0.1,0.06l-0.01,0.13l0.21,0.43l0.21,0.16l0,0.09l-0.08,0.02
				l0-0.1l-0.14-0.07l-0.12,0.13l0.06,0.19l0.09,0.01l-0.07,0.4l0.05,0.14l0.4-0.02l0.08,0.84l-0.11,0.21l0.25,0.31l-0.11,0.24
				l0.11,0.1l0.01,0.24l0.17,0.1l0.04,0.17l-0.13,0.52l0.51,0.18l0.11,0.35L360.47,86l-0.32,0.28v0.08l0.12,0.12l0.26,0.07
				l0.04,0.12l-0.21,0.66l0.03,0.48l0.43-0.01l0.22,0.09l0.13,0.32l0.26,0.2l0,0.25l0.6,0.62l0.09,0.25l-0.09,0.67l-0.25,0.36
				l-0.36,0.18l0.22,0.2l-0.11,0.19l-0.08,0.65l0.2,0.14l0.55,0.3l0.35-0.23l-0.11-0.48l0.1-0.57l0.34-0.2l-0.27-0.1l0.27,0.1
				l-0.34,0.2l-0.1,0.57l0.11,0.48l0.06,0.26l0.35-0.2l0.57-0.11l0.4,0.12l0.11,0.32L364,92.57l0.17,0l0.22-0.2l0.22,0.06l0.24-0.32
				l0.23,0.19l0.28,0.31l0.03,0.27l0.15,0.23l0.46,0.1l0.28,0.25l0.38-0.12l0.54,0.23l0.28,0.01l-0.17,0.52l-0.23,0.12l-0.01,0.17
				l-0.44,0.18l-0.69-0.22l-0.34-0.01l-0.04,1.37l0.17,0.2l0.08,0.29l0.23,0.16l0.32,0.57l0.18,0.11l0.66-0.37l0.27,0.21l0.37,0.04
				l0-0.4l0.08-0.23l0,0l0.27-0.14l0.31,0.21l0.84-0.11l0.46,0.14l0.13,0.21l0.31-0.09l0.25-0.25l0.09,0.44l0.16,0.11l0.21,0.42
				l-0.36,0.65l-0.13-0.15l-0.08,0.01l-0.38,0.97l-0.81,0.37l-0.37,0.9l-0.21,0.28l0.07,0.18l-0.14,0.23l-0.08,0.8l0.35,0l0.19-0.23
				l1.17-0.08l0.32-0.3l0.18,0.03l0.21-0.1l0.03-0.13l0.66-0.65l0.22,0.08l1.11-0.59l0.35-0.08l0.19-0.27l0.76,0.02l0.06,0.54
				l-0.16,0.79l0.12,0.61l-0.06,0.16l-0.8,0.08l-0.11-0.13l-0.33-0.11l-0.56,0.19l-0.38,0.57l0.06,0.1l-0.11,0.27l-0.2,0.22
				l-1.18,0.25l-0.1,0.2l0.03,0.73l0.1,0.68l-0.71,0.44l-0.36,0.39l-0.5,0.36l-0.63,0.25l-0.1,0.05l-0.4-0.05l-0.44-0.03l-0.15-0.15
				l-0.17,0.03l-0.26,0.13l-0.28,0l-0.37,0.19l0.18,0.36l0.14,0.1l0.07,0.73l-0.35,0.2l-0.11,0.32l0.03,0.42l1.27,1.4l0.4,0.11
				l0.24,0.63l0.2,0.07l-0.04,0.92l-0.08,0.23l-0.35,0.01l-0.25,1.48l-0.4,0.07l-0.14-0.09l-0.28,0.31l0.05,0.36l-0.07,0.14
				l0.33-0.02l0.37,0.29l0.24,0.09l0.07,0.15l0.37,0.31l0.12,0.24l0.53,0.77l0.07,0.91l-0.06,0.86l-0.04,0.1l-0.01,0.03l-0.07,0.68
				l-0.06,0.75l-0.18,0.26l-0.02,0.27l0.13,0.19l0.14,0.15l0.25-0.09l0.24-0.23l0.2,0.18l0.23-0.13l0.11,0.18l-0.21,0.22l-0.01,0.1
				l0.64,0.07l0.02,0.44l0.1,0.11l0.2,0.01l0,0.42l0.06,0.25l0.12,0.14l0.31-0.01l0.37-0.13l0.13,0.44l0.3,0.24l0.44,0.23l0.55,0.02
				l0.57,0.28l0.36,0.35l0.45,0.26l0.32,0.84l0.04,0.22l0.12-0.09l0.31-0.04l0.29,0.12l0.17,0.19l0.44,0.88l0.42,0.64l0.33,0.05
				l-0.32-1.28l1.16-0.24l0.38,0.14l0.03,0.09l-0.02-0.16l0.15-0.06l0.04-0.09l0,0l-0.04,0.09l-0.15,0.06l0.02,0.16l-0.04,0.39
				l0.2,0.4l-0.12,0.24l-0.04,0.43l0.37,0.27l0.26,0.3l0.44-0.04l0.27,0.23v0l-0.05,0.63l0.07,0.56l0.49-0.07l0.55,0.02l0.19,0.55
				l0.42-0.37l-0.13-0.66l0.35-0.5l-0.22-0.25l0.04-0.12l0.56,0.55l0.49,0.21l0.11-0.09l0.13,0.13l0.33,0.03l0.31-0.18l0.22,0.23
				l0.76-0.59l0.24-0.11l0.1-0.28l0.05,0.03l0.17-0.16l0.22-0.05l0.35-0.17l0.14-0.3l0.25-0.15l0.38-0.57l0.85-1.06l0.16,0.01
				l0.03-0.22l0.35,0.18l-0.02-0.12l0.22,0.04l-0.19-0.22l0.27-0.67l-0.02-0.17l-0.22-0.16l0.04-0.39l0.06-0.18l-0.13-0.2l0.14-0.27
				l0.18-0.14l0.15,0.14l0.4-0.18l0.26,0.04l0.2,0.13l0.22-0.48l-0.58-0.54l0.19-0.1l0.2,0.04l0.04-0.53l-0.07-0.03l0.38-0.15
				l0.12-0.09l0.23,0.15l0.28,0.35l0.25,0.17l-0.04,0.11l0.06,0.06l0.09,0.03l0.08-0.08l0.45,0.07l0.53-0.03l0.01,0.15l0.07,0.01
				l0.17-0.13l0.29,0.25l-0.01,0.18l0.12-0.02l0.26-0.02l0.38-0.87l0.29-0.32l0.07-0.36l0.23-0.2l-0.16-0.16l-0.04-0.24l0.16-0.25
				l0.06-0.28l0.09-0.02l0.13-0.35l0.33,0.01l0.1,0.11l0.1-0.2l-0.05-0.45l-0.37-0.08l0,0l0.36,0.08l0.12-0.46l0.31-0.45l-0.01-0.13
				l0.51-0.48l0.05-0.33l0.33-0.13l0.6,0.06l0.49-0.09l-0.1-0.32l-0.04-0.58l0.08-0.18l-0.03-0.43l0.33-0.41l-0.11-0.25l0.09-0.22
				l0.39-0.04l0.31-0.15l0.21,0.11l0.05-0.07l0.25-0.02l0.05-0.44l0.49-0.1l0.1-0.49l0.45-0.1l0.37,0.1l0.13-0.11l0.34-0.35
				l-0.33-0.51l-0.09-0.54l0.42-0.33v-0.3l-0.06-0.47l-0.04-0.24l-0.02-0.1l-0.04,0.12l-0.09,0.29l-0.21,0.1l0.11-0.23l0.04-0.11
				l-0.07-0.17l0.15-0.45l-0.14-0.23l-0.17-0.07l-0.01-0.75l0.21-0.18l-0.04-0.32l0.23-0.85l-0.24-0.2l-0.2-0.31l0.24-0.36l0.5,0.25
				l0.11,0.52l0.2,0.08l0.29-0.13l0.54,0.34l0.97-0.85l0.75,0.55l0.24-0.27l1.1-0.39l0.08-0.24l0.45,0.14l0.13-0.24
				c0,0-0.1-0.65-0.07-0.65c0.03,0.01,0.05,0.01,0.08,0.02l0-0.03l0.02-0.06l0.18-0.5l0,0l0.83,0.49l0.94,0.69l0.15-0.2l0.18-0.49
				l-0.2-0.62l0.38,0.22l0,0.37l0.24,0.49l-0.05,0.25l0.39,0.29l0.48-0.01l0.2-0.43l0.54,0.1l0.07-0.06l0.18,0.09l0.2-0.01l0.7,0.12
				l0.07-0.5l-0.06-0.12l0.36-0.56l-0.14-0.29l0.13-0.43l0.39-0.56l0.44,0.06l0.21,0.18l0.47-0.06l0.25-0.41l0.19-0.78l-0.31-0.36
				l0.07-0.43l0.57-1.19l0.05-0.31l0.53-0.76l0.21-0.54l0.75-0.26l-0.07-0.87l-0.21-0.55l-0.1-0.64l-0.16-0.29l-0.04-0.41
				l-0.18-0.37l0.05-0.32l-0.17-0.14l0,0l-0.21,0.01l-0.59,0.19l-0.14-0.01l-0.08,0.08l-0.77,0.01l-0.56-0.18l-0.43,0.23l-0.44-0.27
				l-0.36-0.05l-0.44,0.18l0,0l-0.55-1.31l-0.45-0.89l-0.29-0.34l-0.3-0.14l-0.41,0.13l-0.1-0.11l-0.18-0.7l0.03-0.58l-0.4-0.43
				l0.13-0.11l-0.22-0.03l0.01-0.1l-0.3-0.24l-0.14,0.02l-0.1-0.16l-0.01-0.51l-0.4-0.27L405,88.28l0.02-0.08l0.41-0.19l0.22-0.27
				l0.3-0.12l0.18-0.29l0.07-0.54l-0.11-0.27l0,0l0.41-0.67l0.15-0.05l-0.37-0.93l0.33-0.53L406.35,84l0.01-0.45l-0.31-0.99
				l0.04-0.16l-0.16-0.2l0.13-0.44l0.22-0.08l0.09-0.25l-0.18-0.08l-0.42-0.55l0.05-0.41l0.01-0.31l0.88-0.02l0.97-0.21l0.02-0.36
				l-0.3-0.36l-0.57-0.86l-0.41,0.09l-0.38-0.09l0.04-0.26l0.26-0.44l0.05-0.35l-0.13-0.18l-0.1-0.44l0.2-0.01l0.23-0.16l0.01-0.13
				l0.49,0.36l-0.02,0.44l-0.16,0.54l0.16,0.32l0.15-0.01l0.06,0.32l0.23,0.3l0.14,0.36l0.27,0.17l0.18-0.18l-0.12-0.63l-0.32-1.03
				l0.21-0.3l0.06-0.39l-0.24-0.42l0.23-0.48l0.47-0.6L408,74.5l-0.23-0.33l-0.13-0.48l-0.37-0.3l-0.36,0.11l-0.02-0.08l-0.26-0.16
				l-0.05,0.05l-0.04-0.45l-0.1,0l-0.03-0.1l0.06-0.19l0.57-0.41l0.16-0.4l0.06-0.49l-0.09-0.24l0.17-0.4l-0.19-0.24l-0.19,0.06
				l-0.11-0.05l0.06-0.08l-0.06-0.09l-0.21-0.16l-0.3,0.2l-0.13-0.09l-0.31,0.4l-0.4-0.19l-0.32-0.05l-0.36-0.48l0.32-0.09l0.46-0.4
				l0.38-0.14l0.07,0.04l-0.02-0.2l0.07-0.14l0.57-0.41l0.24-0.37l-0.16-0.44l0.28-0.08l0.45-0.28l0.34-0.01l-0.21-0.23l0.01-0.26
				l0.17-0.12l0-0.11l-0.2-0.21l-0.21-0.8l-0.31-0.12l-0.05-0.13l-0.28-0.09l0.22-0.19l-0.15-0.22l0.02-0.29l0.6,0.1l1.16-0.07
				l0.46,0.1l0.46,0.32l0.14-0.44l0.15-0.07l0.08,0.04l0.16-0.18l0,0l0.39,0.13l0.21-0.02l0.16,0.32l0.24,0.05l0.09-0.07l0.08-0.14
				l0.22-0.1l0.72,0.14l0.25-0.04l0.53,0.26l0.38,0.69l0.33,0.12l0.47-0.03l0-0.34l0.47-0.07l0.09,0.11l-0.14,0.24l0.48,0.03
				l-0.06-0.2l0.29-0.35l0.49-0.12l0.24-0.27l-0.03-0.03L415.58,64.67z"></path>
		</g>
		<g id="palencia_p">
			<path class="st6" d="M374.21,99.72l-0.06-0.54l-0.76-0.02l-0.19,0.27l-0.35,0.08l-1.11,0.6l-0.22-0.08l-0.66,0.65l-0.03,0.13
				l-0.21,0.1l-0.18-0.03l-0.32,0.3l-1.17,0.08l0.19,1.36v0l-0.19-1.36l-0.19,0.23l-0.35,0l0.08-0.8l0.14-0.23l-0.07-0.18l0.21-0.29
				l0.37-0.9l0.81-0.37l0.38-0.97l0.08-0.01l0.13,0.15l0.36-0.65l-0.21-0.42l-0.16-0.11l-0.09-0.45l-0.25,0.25l-0.31,0.09
				l-0.13-0.21l-0.46-0.14l-0.84,0.11l-0.31-0.21l-0.27,0.14l-0.08,0.23l0,0.4l-0.37-0.04l-0.27-0.21l-0.66,0.37l-0.18-0.11
				l-0.32-0.57l-0.23-0.16l-0.08-0.29l-0.17-0.2l0.04-1.37l0.34,0.01l0.69,0.22l0.44-0.18l0.01-0.17l0.23-0.12l0.17-0.52l-0.28-0.01
				l-0.54-0.23l-0.38,0.12l-0.28-0.25l-0.46-0.1l-0.15-0.23l-0.03-0.27l-0.28-0.31l-0.23-0.19l-0.24,0.32l-0.22-0.06l-0.22,0.2
				l-0.17,0l0.02-0.21l-0.11-0.31l-0.4-0.12l-0.57,0.11l-0.35,0.2l-0.06-0.26l-0.35,0.23l-0.55-0.3l0,0l-0.2-0.14l0.08-0.65
				l0.11-0.19l-0.22-0.2l0.36-0.18l0.31-0.37l-0.03-0.15l0.07-0.51l-0.09-0.25l-0.6-0.61l0-0.25l-0.26-0.2l-0.13-0.32l-0.22-0.09
				l-0.43,0.01l-0.03-0.48l0.21-0.66l-0.04-0.12l-0.26-0.07l-0.12-0.12v-0.08l0.32-0.28l0.03-0.41l-0.11-0.35l-0.51-0.18l0.13-0.52
				l-0.04-0.17l-0.17-0.1l-0.01-0.24l-0.11-0.1l0.11-0.24l-0.25-0.31l0.11-0.21l-0.08-0.84l-0.4,0.02l-0.05-0.14l-0.18-0.05l0,0
				l0.18,0.05l0.07-0.4l-0.09-0.01l-0.06-0.19l0.17-0.16l-0.05-0.06l-0.21-0.43l0.01-0.13l-0.1-0.06l-0.01-0.22l0.15-0.21l0.06-0.29
				l-0.19-0.25l-0.19-0.08l-0.2-0.14l-0.06-0.14l0.07-0.11l0.09,0.04l-0.01-0.18l-0.17-0.25l0.02-0.25l0.27,0.14l0.1-0.31
				l-0.05-0.26l-0.2-0.05l0.07-0.29l-0.24,0.16l-0.47,0.05l-0.49,0.25l-0.65,0.19l-0.38-0.59l0.1-0.24l-0.06-0.49l0.07-0.27
				l-0.08-0.29l-0.11-0.06l-0.07-0.2l0.07-0.17l-0.08-0.4l0.5-0.75l0.24-0.77l0,0l0.41,0.3l0.01-0.09l0.89,0.09l0.1-0.09l0.26,0
				l0.29,0.09l0.7-0.17l0.36,0.02l-0.05-0.14l-0.11,0.01l-0.03-0.08l0.06-0.14l-0.08-0.23l-0.19,0.06l-0.09-0.13l0.16-0.16
				l0.22-0.02l-0.23-0.41l0.18-0.7l-0.04-0.12l-0.21-0.46l-0.42-0.46l-0.42,0.08l-0.53-0.08l0.39-0.14l0.18-0.29l-0.13-0.47
				l-0.17-0.27l0.09-0.09l-0.06-0.39l0.05-0.1l-0.89-0.42l-0.14-0.15l0.01-0.2l0.22-0.07l-0.31-0.56l0.27,0.01l0.2-0.2l0.45-0.09
				l0.23-0.21l0.24-0.09l-0.17-0.09l-0.37,0.1l-0.09-0.09l0.16-0.19l-0.15-0.14l0.03-0.11l-0.11-0.1l-0.03-0.28l-0.23-0.24
				l0.08-0.16l0.41-0.2l0.35-0.39l0.41-0.05l-0.27-0.29l0.12-0.46l-0.13-0.21l-0.15-0.08l-0.11-0.33l0.16-0.11l-0.09-0.44l0.15-0.14
				l-0.16-0.57l-0.01-0.33l-0.36-0.12l-0.21-0.21l0-0.23l0.16,0l-0.03-0.15l-0.26-0.24l0.37-0.26l0.12-0.21l0.48-0.28v0l0.1-0.03
				l-0.04,0.59l0.15,0l0.28,0.33l0.38-0.32l-0.05-0.23l0.11,0.07l0.37-0.03l0.46-0.31l0.29-0.07l0.14-0.22l-0.06-0.13l0.06-0.15
				l0.55-0.19l0.01-0.18l0.41-0.47l0,0l0.06-0.01c0.04-0.13,0.1-0.38,0.21-0.46c0.4-0.29,0.74-0.64,0.93-1.1
				c0.12-0.3,0.25-0.6,0.39-0.89c0-0.01,0-0.02,0-0.02c0.01-0.05,0.01-0.11,0.02-0.16c0.02-0.16,0.06-0.31,0.11-0.46
				c0.02-0.06,0.05-0.11,0.07-0.17l-0.12,0.06l-0.96-1.04l-0.21-0.12l-0.16,0.03l-0.11-0.37l0.1-0.18l0.18-0.3l0.53-0.11l0.09-0.1
				l0.06-0.32l0.21-0.09l0.23,0.17l0.02,0.17l0.2,0.13l0.21,0l-0.02-0.07l0.15-0.17l0.01-0.32l0.35-0.37l0.1-0.56l-0.53-0.14
				l-0.08-0.13l-0.17-0.03l-0.15-0.19l0.19-0.15l-0.23-0.14l-0.79,0.65l-0.48,0.01l-0.18,0.23l-0.4,0.29l-0.15,0.23l-0.39,0
				l-0.4-0.31l-0.27-0.45l0.11-0.31l-0.02-0.55l-0.29-1.16l-0.01-0.35l0.25-0.17l0.09-0.24l-0.57-0.33l-0.1-0.85l0.02-1.15
				l-0.3-0.27l-0.66-0.31l-0.64-0.01l-0.46-0.2l-0.39,0.14l-0.19-0.02l-0.48-0.15l-1.29-0.13l-0.63-0.69l0,0l-0.17-0.33l-0.21-0.19
				l-0.11-0.27l0.02-0.35l-0.48-0.56l-0.12-0.5l-0.13-0.09l-0.24-0.4l-0.24-0.18l-0.4-0.21l-0.34-0.04l-0.38-0.16l-0.43-0.55
				l-0.21-0.14l-0.37,0.08l-0.17-0.2l0,0l-0.08,0.03l-0.33,0.45l-0.22,0.56l0.03,0.14l-0.74-0.05l-0.08,0.07l-1.01-0.48l-1.52,1.5
				l-0.64-0.4l-0.66,0.06l-0.63-0.53l-0.23,0.28l-0.32,0.22l-0.4,0.14l-0.35,0.39l-0.09-0.03l-0.45-0.2l-0.88-0.03l-0.45-0.25
				l-0.24-0.02l-0.37-0.01l-0.38,0.09l-0.23,0.29l-0.21,0.13l-0.36-0.08l-0.1-0.19l-0.24,0.07l-0.01,0.14l-0.01,0.08l0.01-0.08v0
				l-0.8,0.15l-0.4,0.75l-0.57,0.19l-0.07,0.24l-0.22,0.05l-0.27,0.32l-0.03,0.69l-0.47,0.33l0.08,0.32l-0.05,0.28l0.16,0.38
				l-0.1,0.41l-0.23,0.31l0.07,0.3l-0.16,0.16l-0.03,0.18l-0.24,0.09l-0.35,0.29l-0.08,0.27l-0.3,0.28l0.07,0.4l-0.09,0.21
				l-0.31-0.12l-0.38,0.31l-0.16,0.43l0.04,0.3l-0.09,0.38l-0.2,0.56l-0.12-0.12l-0.49-0.17l-0.72,0.15l-0.11,0.31l0.01,0.31
				l0.5,0.56l-0.14,0.48l-0.16,0.28l-0.04,0.53l-0.24,0.43l0.16,0.22l-0.01,0.16l0.1,0.13l0.03,0.78l-0.06,0.08l-0.17,0.03
				l-0.05-0.08l-0.26,0.27l-0.02,0.28l-0.44,0.9l-0.18-0.1l-0.14,0.21l-0.28-0.05l0.08,0.17l-0.12,0.48l0.32,0.41l0.4,0.14l0.1,0.73
				l0.2,0.1l0.56-0.3l0.24,0.05l0.26,0.28l0.04,0.3l0.49,0.25l-0.34,0.21l-0.25,0.46l-0.18,0.14l-0.4,1.27l0.17,0.73l-0.04,0.63
				l-0.05,0.45l0.15,0.63l-0.41,1.11v0.22l0.15,0.24l0,0.55l-0.05,0.35l-0.51,0.95l-0.02,0.56l0.05,0.28l0.19,0.19l0.89,0.22
				l-0.13,1.01l0.04,0.49l0.1,0.01l-0.19,1.22l-0.3,1.91l-0.23,0.08l-0.32,0.42l-0.3,0.21l-0.12,0.06l0.57,0.12l0.05,0.3l0.15,0.21
				l-0.04,0.23l-0.35,0.32l0.09,0.53l-0.07,0.19l0.24,0.23l-0.12,0.25l0.32,0.31l-0.33,0.27l-0.89-0.17l-0.12,0.14l-0.41,0.04
				l0.12,0.12l-0.09,0.08l-0.63,0.09l-0.13-0.19l-0.17,0.13l-0.05,0.13l-0.1,0.02l0.03,0.35l-0.24,0.66l0.02,0.35l-0.26,0.35
				l-0.03,0.25l0.08,0.01l1.13-0.35l0.05-0.13l0.42,0l0.07,0.59l-0.06,0.31l-0.01,0.13l-0.15,0.19l-0.12,0.28l0.08,0.13l-0.04,0.3
				l-0.11,0.13l-0.11,0.6l-0.68,0.27l-0.13,0.18l-0.53-0.28l-0.13,0.11l-0.2-0.07l-0.3,0.21l-0.31,1.21l0.05,0.27l-0.32,0.13
				l-0.09,0.27l0.4,0.62l-0.06,0.76l-0.58,0.44l-0.01,0.14l0.18,0.63l0.3,0.59l-0.07,0.07l0.09,0.2l-0.29,0.59l-0.5,0.28l0.23,0.43
				l-0.03,0.63l0.48-0.15l-0.01,0.16l0.27,0.28l0.25-0.02l0.13,0.14l0.17-0.07l0.03,0.16l0.24,0.03l0.29-0.13L329.99,91l0.08-0.1
				l0.57-0.27l0.03,0.23l0.13,0.1l0.06-0.11l0.68-0.06l0.68,0.14l0,0l-0.09,0.3l0.03,0.76l0.16,0.53l-0.15,0.13l0.18,0.42l0.16,0.76
				l0.33,0.28l-0.23,0.03l0.55,0.67l-0.51,0l-0.44,0.26l-0.89,1.21l0.47,1.67l0.09,0.11l-0.61,0.31l-0.59,0.27l-0.19,0.46
				l-0.21,0.27l0.16,0.24l-0.12,0.62l-0.07,0.2l-0.28,0.2l-0.01,0.17l-0.44,0.22l-0.05,0.25l-0.29,0.54l-0.11,0.04l0.15,0.22
				l-0.14,0.28l0.3,0.49l0.3,0.18l0.12,0.2l0.01,0.45l0.13,0.18l0.02-0.1l0.27-0.09l-0.04-0.27l0.22-0.05l0.2-0.28l0.4-0.19
				l0.07-0.15l0.24-0.14l0.69,0.39l0.18,0.41l-0.08,0.05l0.02,0.09l0.07,0.07l0.46-0.15l0.03,0.09l0.16,0.03l0.48-0.68l0.46-0.28
				l0.58,0.02l0.2,0.07l0.06,0.76l0.19-0.04l0.16,0.17l0.13-0.03l-0.01,0.13l0.72,0.79l0.24,0.63l0,0.34l0.12,0.29l0.26,0.41
				l0.37,0.24l0.59,1.55l0.17,0.29l0.06,0.36l0.17,0.22l0.78-0.27l0.03-0.15l0.44,0l0.16-0.42l0.26-0.38l0.88-0.75l-0.24-0.39
				l0.16-0.38l0.18-0.12l0.2-0.13l0.72-0.74l0.61-0.2l0.33-0.38l-0.02-0.1l0.35-0.03l0.46,0.2l-0.1-0.31l0,0l0.1,0.3l0.01,0.12
				l0.34,0.19l0.24-0.01l0.27,0.17l0.1,0.24l-0.06,0.24l0.11,0.18l-0.05,0.25l0.12,0.15l0.1,0l-0.13,0.27l0.26,0.26l0.1,0.23
				l0.24,0.15l0.58,0.68l0.52,0.1l0.36,0.77l0.65,0.53l-0.05-0.2l0.11-0.11l0.39,0.11l0.16-0.02l0.12-0.12l0.1-0.38l-0.19-0.15
				l0.04-0.12l0.8,0.17l0.34,0.25l0.36-0.01l0.12-0.16l0.14-0.03l0.01-0.17l-0.13-0.12l0.37-0.05l0.31-0.28l0.51,0.01l0.04-0.28
				l0.11,0.06l0.09-0.04l0.06,0.13l-0.06,0.46l-0.2,0.19l0.7,0.44l-0.19,0.06l-0.11-0.07l-0.43,0.13l-0.18-0.11l-0.15,0.01
				l-0.26,2.14l0.05,0.09l-0.36,0.39l0.67,0.1l1.23,0.56l0.3,0.05l0.19-0.02l0.84-0.51l0.71-0.81l0.36-0.11l0.13-0.11l0.46,0.04
				l-0.09,0.15l0.05,0.07l1.17,0.46l0.59,0.12l0.16-0.03l0.18,0.18l0.48,0.29l0.02-0.27l0.15-0.05l0.27-0.35l0.82,0.06l0.32-0.74
				l0.3-0.19l0.45-0.71l0.16-0.06l0.36,0.19l0.51-0.48l0,0l0.08,0.2l0.72,0.46l0.84-0.24l0.16,0.13l0.18-0.04l0.42-0.02l0.22,0.14
				l0.27-0.06l0.39,0.27l0.13-0.14l0.61-0.24l0.51-0.22l0.21,0.38l0.29,0.3l0.08,0.5l0.37,0.59l0.12-0.02l0.26-0.25l0.2,0.02
				l0.55,0.46l0.31,0.02l0.11-0.24l0.83,0.34l0.08-0.23l0.04-0.92l-0.2-0.07l-0.24-0.63l-0.4-0.11l-1.27-1.4l-0.06-0.44l0,0
				l0.06,0.44l-0.03-0.42l0.11-0.32l0.35-0.2l-0.07-0.74l-0.14-0.1l-0.18-0.36l0.37-0.18h0.28l0.25-0.13l0,0l0.17-0.03l0.15,0.14
				l0.44,0.03l0.38,0.07l0.12-0.07l0,0l0.63-0.25l0.5-0.36l0.36-0.39l0.71-0.44l-0.1-0.68l-0.62-0.01l-0.35,0.21l-0.16-0.23
				l-0.22-0.62v0l0.22,0.62l0.16,0.23l0.35-0.21l0.62,0.01l-0.03-0.73l0.1-0.2l1.18-0.25l0.2-0.22l0.11-0.27l-0.06-0.1l0.38-0.57
				l0.56-0.19l0.33,0.11l0.11,0.13l0.8-0.08l0.06-0.16l-0.12-0.6L374.21,99.72z"></path>
		</g>
		<g id="valladolid_p">
			<path class="st6" d="M370.42,121.26l-0.06-0.25l0-0.42l-0.2-0.01l-0.1-0.11l-0.02-0.44l-0.64-0.07l0.01-0.1l0.18-0.12l0.07-0.05
				l-0.15-0.23l-0.23,0.13l-0.2-0.18l-0.24,0.23l-0.25,0.09l-0.14-0.15l-0.13-0.19l0.02-0.27l0.18-0.26l0.06-0.75l0.07-0.68
				l0.06-0.13l0.06-0.86l-0.07-0.91l-0.53-0.77l-0.12-0.24l-0.37-0.31l-0.07-0.14l-0.24-0.09L367,113.7l-0.33,0.02l0,0l0.07-0.14
				l-0.05-0.36l0.28-0.31l0.14,0.09l0.4-0.07l0.25-1.48l0.35-0.01l-0.83-0.34l-0.11,0.24l-0.31-0.02l-0.55-0.46l-0.2-0.02
				l-0.26,0.25l-0.12,0.02l-0.37-0.59l-0.08-0.5l-0.29-0.3l-0.21-0.38l-0.51,0.22l-0.61,0.24l-0.13,0.14l-0.39-0.27l-0.27,0.06
				l-0.22-0.14l-0.42,0.02l-0.18,0.04l-0.16-0.13l-0.84,0.24l-0.72-0.46l-0.08-0.2l0,0l-0.51,0.48l-0.36-0.19l-0.16,0.06l-0.45,0.71
				l-0.3,0.19l-0.32,0.74l-0.82-0.06l-0.27,0.35l-0.15,0.05l-0.02,0.27l-0.48-0.29l-0.18-0.18l-0.16,0.03l-0.59-0.12l-1.17-0.46
				l-0.05-0.07l0.09-0.15l-0.46-0.04l-0.13,0.11l-0.36,0.11l-0.71,0.81l-0.84,0.51l-0.19,0.02l-0.3-0.05v0l-1.23-0.56l-0.67-0.1
				l0.36-0.39l-0.05-0.09l0.26-2.14l0.15-0.01l0.18,0.11l0.43-0.13l0.11,0.07l0.19-0.06l-0.7-0.44l0.2-0.19l0.06-0.46l-0.06-0.13
				l-0.09,0.04l-0.11-0.06l-0.04,0.28l-0.51-0.01l-0.31,0.28l-0.37,0.05l0.13,0.12l-0.01,0.17l-0.14,0.03l-0.12,0.16l-0.36,0.01
				l-0.34-0.25l-0.8-0.17l-0.04,0.12l0.19,0.15l-0.1,0.38l-0.12,0.12l-0.16,0.02l-0.39-0.11l-0.11,0.11l0.05,0.2l-0.65-0.53
				l-0.36-0.78l-0.52-0.1l-0.58-0.68l-0.24-0.15l-0.1-0.23l-0.26-0.26l0.13-0.27l-0.1,0l-0.12-0.15l0.05-0.25l-0.11-0.18l0.06-0.24
				l-0.1-0.24l-0.27-0.17l-0.24,0.01l-0.34-0.19l-0.01-0.12l-0.46-0.2v0l-0.35,0.03l0.02,0.1l-0.33,0.38l-0.61,0.2l-0.72,0.75
				l-0.2,0.13l-0.18,0.12l-0.16,0.38l0.24,0.39l-0.88,0.75l-0.26,0.38l-0.16,0.42l-0.44,0l-0.03,0.15l-0.78,0.27l0,0l-0.17-0.22
				l-0.06-0.36l-0.17-0.29l-0.59-1.55l-0.37-0.24l-0.26-0.41l-0.12-0.29l0-0.34l-0.24-0.63l-0.72-0.79l0.01-0.13l-0.13,0.03
				l-0.16-0.17l-0.19,0.04l-0.06-0.76l-0.2-0.07l-0.58-0.02l-0.46,0.28l-0.48,0.68l-0.16-0.03l-0.03-0.09l-0.46,0.15l-0.07-0.07
				l-0.02-0.09l0.08-0.06l-0.18-0.41l-0.69-0.39l-0.24,0.14l-0.07,0.15l-0.4,0.19l-0.2,0.28l-0.22,0.05l0.04,0.27l-0.27,0.09
				l-0.02,0.1l-0.13-0.18l-0.01-0.45l-0.12-0.2l-0.3-0.18l-0.3-0.49l0.14-0.28l-0.15-0.22l0.11-0.04l0.29-0.54l0.05-0.25l0.44-0.22
				l0.01-0.17l0.28-0.2l0.07-0.2l0.12-0.62l-0.16-0.24l0,0l0.21-0.27l0.19-0.46l0.59-0.27l0.61-0.31l-0.09-0.11l-0.47-1.67
				l0.89-1.21l0.44-0.26l0.51,0l-0.55-0.67l0.23-0.03l-0.33-0.28l-0.16-0.76l-0.18-0.42l0.15-0.13l-0.16-0.53l-0.03-0.76l0.09-0.3
				l-0.68-0.14l-0.68,0.06l-0.06,0.11l-0.13-0.1l-0.03-0.23l-0.56,0.27l-0.08,0.1l0.04,0.11l-0.29,0.13l-0.24-0.03l-0.03-0.16
				l-0.17,0.07l-0.13-0.14l-0.25,0.02l-0.27-0.28l0.01-0.16l-0.48,0.16l0.03-0.63l-0.23-0.43l0.5-0.28l0.29-0.59l-0.09-0.2
				l0.07-0.07l-0.3-0.59l-0.18-0.63l0.01-0.14l0.58-0.44l0.06-0.76l-0.4-0.62l0.09-0.28l0.32-0.13l-0.05-0.27l0.31-1.21l-1.47-0.43
				l0,0l-0.13,0.23l0.19,0.2l-0.08,0.64l-0.14,0.31l0.03,0.21l-0.14,0.3l-0.39,0.24l0.11,0.21l0.02,0.28l0,0l-0.02-0.28l-0.11-0.21
				l-0.54-0.2l0.24-0.9l-0.16,0.09L326.41,84l-0.05,0.07l-0.13-0.06l-0.01-0.33l-0.14-0.26l0.04-0.1l-0.18-0.11l-0.15,0.08
				l-0.04-0.28l-0.27-0.64l-0.64-0.14l-0.1,0l-0.07,0.11l-0.23,0.11l-0.15,0.32l-0.12,0.04l-0.73,1.05l-0.48,0.21l0,0.2l-0.33,0.15
				l-0.56,0l-0.22,0.16l0.03,0.21l-0.61,0.2l-0.1-0.22l-0.66-0.71l-0.15,0l-0.09,0.14l-0.24,0.69l-0.12-0.06l0.05-0.65l-0.25,0.12
				l0.05,0.11l-0.13,0.28l-0.24,0.12l-0.24-0.06l-0.34-0.56l0.09-0.2l-0.11-0.4l-0.17,0.35l-0.6-0.08l-0.12,0.15l-0.14,0.04
				l0.02,0.13l-0.06,0.03l-0.16-0.1l-0.17,0l-0.07,0.37l-0.24,0.04l-0.19-0.13l-0.56,0.24l-0.33-0.37l-0.35,0.24l-0.33,0.1v0
				l-0.07,0.46v0l0.05,0.44v0l0.22,0.23l-0.03,0.75v0l0.17,0.1l-0.35,0.6l-0.14-0.04l-0.15,0.07l-0.34-0.32l-0.29,0.12l-0.3,0.02
				l-0.21-0.16l-0.29,0.21l-0.04,0.19l0.02,0.68l0.34,0.4l-0.28,0.46l-0.07,0.75l0.09,0.37l0.29,0.28l-0.08,0.21l0.01,0.47
				l0.39,1.11l-0.15,0.18l0,0.29l0.1,0.17l-0.2,0.29l0.18,0.25l0.11,0.02l0.15-0.09l0.08,0.09l0.15-0.12l-0.04-0.1l0.11-0.05
				l-0.04-0.07l0.1-0.04l0.19,0.07l0.09-0.09l0.16,0l0.25-0.12l0.14-0.17l0.04,0.21l0.59,0.26l0.3-0.09l0.2-0.19l0.51-0.21
				l0.17-0.19l0.15-0.03l0.18-0.21l0.07-0.08l0.3-0.01l0.42,0.16l0.35,0.47l-0.3,0.28l0.1,0.13l0,0.23l0.11,0.12l0.01,0.3l0.18,0.29
				l-0.14,0.25l-0.43,0.1l-0.24,0.15l-0.66-0.08l-0.33-0.14l-0.02,0.12l0.1,0.03l0.02,0.15l-0.08,0.73l-0.13,0.26l-0.07-0.07
				l-0.79,0.41l-1.26-0.03l-0.32-0.16l-0.17-0.19l-0.36-0.14l0.13,0.9l-0.18,0.25l0.05,0.19l-0.08,0.09l0,0.1l0.33,0.34l0.78,0.58
				l0.95,0.12l0.09,0.08l0,0l0.22,0.66l0.18,0.21l0.16-0.01l0.29,0.54l-0.04,0.09l0.91,0.66l0.3-0.11l0.52-0.2l0.24,0.16l-0.35-0.01
				l-0.41,0.05l-0.48,0.36l-0.15,0.17l0.02,0.16l-0.45,0.36l-0.29,0.46l-1,0.55l-0.13,0.13l-0.02,0.14l0.58,0l0.44,0.28l0.45,0.55
				l0.21,0.12l-0.09,0.47L317.1,104l-0.4,0.74l-0.04,0.57l0.25,0.84l-0.17,0.17l0.12,0.33l-0.62,0.92l0.07,0.06l-0.29,0.24
				l-0.1,0.23l0.12,0.11l0.76,0.18l0.05-0.06l0.32,0.07l0.14-0.11l0.18,0.07l0.14-0.08l0.42,0.26l-0.11,0.51l-1.39,1.06l-0.1,0.15
				l-0.16,0.04l-0.09,0.11l-0.15-0.09l-0.29,0.03l-0.32,0.01l-0.32,0.45l0,0l-0.84,0.16l-0.86,0.93l0.03,0.16l0,0.27l0.61,1.32
				l0.01,0.44l0.15-0.08l0.06-0.3l0.18-0.13l0.16,0.03l0.22,0.63l-0.08,0.93l0.5-0.13l0.2,0.16l0.1,0.66l0.41,0.27l0.18,0.62
				l0.05,0.41l-0.06,0.65l-0.04,0.38l0.21,0.36l-0.16,0.14l-0.07,0.22l0.04,0.27l-0.19,0.5l0.35,0.27l0.47-0.04l0.12,0.18l0.54-0.03
				l0.61,0.21l0.28,0.21l-0.06,0.26l0.05,0.13l0.17,0.05l-0.05,0.16l-0.52,0.29l-0.49,0.55l0.4,0.31l0.16,0.24l-0.05,0.11l0.32,0.16
				l-0.06,0.35l0.06,0.13l0.35-0.08l0.27,0.25l0.39,0.02v0l0.01,0.09l0.33,0.25l0.21,0.29l0.05,0.23l0.17,0.13l-0.2,0.58l-0.32-0.11
				l-0.5,0.54l-0.32,0.32l-0.62-0.14l-0.55,0.3l-0.09-0.04l-0.25,0.07l-0.48,0.53l-0.52,0.55l-0.34,0.12l0.01,0.11l-0.12,0.04
				l-0.09-0.06l-0.2-0.27l-0.11,0.11l0.29,0.97l0.1,0.08l0.22,0.01l0.2,0.23l0.3,0.33l0.55-0.1l0.09,0.13l0.08,0.6l-0.08-0.6
				l-0.09-0.13l-0.55,0.1l-0.3-0.33l-0.3,0.28l0.2,0.16l-0.23,0.17l-0.02,0.42l-0.24,0.57l0.85,0.25l0.44,0.37l-0.15,0.29
				l-0.38,0.39l-0.04,0.3l-0.47,0.77l-0.06,0.59l0.05,0.32l0.01,0.35l0.35,0.96l0.09,0.76l0.25,0.32l0.42,0.12l0.08,0.12l-0.12,0.24
				l0.13,0.17l-0.02,0.43l0.26,0.43l0.09,0.53l-0.07,0.64l0.2,0.08l-0.4,0.34l-0.03,0.19l0.29,1l-0.07,0.17l-0.07,0.3l0.03,0.1
				l0.09,0.03l-0.09,0.22l-0.2,0.06l-0.08-0.13l-0.12,0.09l-0.03,0.14l-0.13,0.04l0.06,0.09l-0.05,0.17l-0.17,0.25l-0.59,0.42
				l-0.42,0.54l0.17,0.22l-0.1,0.09l0.36,0.22L316,143l-0.14,0.16l0.18,0.05l0.75-0.37l0.04,0.06l0.23-0.12l0.19,0.23l0.33-0.04
				l-0.12-0.1l0.05-0.14l-0.06-0.04l0.14-0.45l0.23-0.03l0.36-0.06l0.14,0.09l-0.05-0.13l0.13-0.07l0.18,0.11l0.35-0.02l0.11-0.12
				l0.21,0.09l-0.03,0.11l0.25-0.05l0.15,0.17l0.39,0.22l0.13-0.03l0,0l-0.13,0.03l0.07,0.17l-0.03,0.18l0.14,0.1l-0.03,0.16
				l0.08,0.01l-0.04,0.09l0.09,0.32l0.2,0.16l-0.2,0.29l0.17,0.14l0.42-0.39l0.03,0.1l0.27-0.08l0.17,0.18l0.21,0.06l0.25-0.25
				l0.14,0.04l0.08-0.07l0.12,0.11l-0.08,0.04l0.03,0.08l0.14-0.01l0,0.11l0.34,0.06l0.18,0.19l0.52,0.29l0.15,0.19l0.36,0.2
				l-0.01,0.15l0.17,0.27l0.16,0.13l0.14,0.09l0.04-0.19l0.45-0.04l0.06,0.07l0.37-0.41l0.09-0.24l0.67,0.23l0.31,0.19l0.32,0.06
				l0.33-0.01l0.41-0.2l1.13-0.6l-0.09-0.17l0.4-0.29l0-0.11l0,0l0.12,0.12l0.14,0l0.12,0.16l0.6-0.1l0.08,0.26l0.21,0.06
				l-0.08,0.14l0.29,0.06l0.39-0.06l0.12,0.14l0.07,0.52l0.2,0.06l0.29-0.09l0.47,0.48l0.33,0.14l0.29,0.28l0.49,0.24l0.11,0.05
				l0.16,0l0.13,0.23l0.31,0.07l0.53,0.47l0.08-0.04l0.18,0.1l0.17,0.17l-0.1,0.06l0.36,0.28l0.33-0.36l1.12,0.09l0.53-0.23
				l-0.16-0.24l0.08-0.47l0.19-0.52l-0.05-0.16l0.31-0.31l0.39-0.03l0.29-0.22l0.11-0.2l0.3-0.08l0.47,0.14l0.74-0.4l0.4,0.04
				l0.07-0.08l0.34-0.02l0.28,0.08l0.03-0.41l0.5-0.54l0.27-0.41l0,0l0.72-0.33l0.06-0.2l0.1-0.07l0.59,0.29l0.9,0.66l0.35,0
				l0.3-0.14l-0.17-0.75l-0.18-0.31l-0.08,0.04l-0.08-0.2l-0.41-0.35l0.17-0.44l-0.18-0.71l0.12-0.14l0.43-0.17l0.33-0.3l0.21-0.08
				l0.35-0.38l0.11-0.15l0.37-0.53l-0.03-0.1l0.22-0.45l0.44-0.34l0.53-0.18l0.04-0.07l0,0l0.33,0.1l0.22,0.22l1.19-0.6l0.51-0.04
				l0.97,0.16l0.28,0.42l0.25,0.04l0.07,0.09l0.33-1.27l0.15-0.23l-0.11-1.21l-0.16-0.26l0.11-0.58l-0.01-0.47l-0.14-0.51
				l-0.18-0.24l-0.16-0.47c0,0,0,0,0-0.06l0.28-0.15l0.4-0.05l0.25-0.1l0.38-0.43l-0.05-0.2l-0.21-0.13l0.17-0.05l0.14-0.51
				l-0.56-0.96l0,0l-0.05-0.09l0.14-0.01l-0.05-0.12l0.2-0.12l0.13,0.05l0.08-0.05l0.02-0.13l0.11,0.03l0.03-0.08l0.24-0.05
				l0.11-0.11l0.18,0.03l0.23-0.08l-0.17-0.1l0.12-0.03l0.02-0.09l0.05-0.06l0.18,0.06l0.27-0.38l0.46-0.11l0.55,0.12l0.01,0.1
				l0.12,0l0.1-0.14l0.25,0.12l0.13-0.18l0.09-0.01l0.16,0.16l0.2-0.03l0.12,0.17l0.14-0.47l-0.04-0.28l0.18,0.04l0.38-0.07
				l0.07,0.13l0.37-0.05l0.23,0.21l0.15,0.01l0.3-0.2l0.05-0.26l0.75-0.24l0.26,0.15l0.16-0.02l0.1-0.16l0.55,0.02l0.16-0.18
				l0.23,0.27l0.13,0.3l0.05-0.31l0.51-0.31l0.16,0.07l0.39,0.28l0.23-0.23l1.23-0.6l0.77-0.23l0.36-0.02l0.45-0.34l0.64,0.65
				l0.24-0.38l0.55-0.18l0.57-0.16l0.34-0.19l0.87-0.46l0.17,0.02l0.38,0.01l0.29-0.14l0.06-0.18l0.16-0.11l0.16-0.28l0.49-0.11
				l0.37-0.42l0.37-0.05l0.28,0.06l0.04,0.09l0.15-0.07l0.05,0.08l0.23-0.02l0.17-0.27l0.08-0.54l0-0.45l-0.01-0.89l0.34-0.48
				l0.19-0.52l0-0.48L370.42,121.26z"></path>
		</g>
		<g id="segovia">
			<path class="st6 fase_0" d="M401.79,136.22l-0.47-0.15l-0.67-0.43l0.02-0.63l-0.48,0.01l-0.86-0.62l-0.11-0.6l-0.36,0.04l-0.29-0.45
				l-0.59-0.15l-0.13-0.28l-0.94-0.43l-0.49-0.82l-0.38-0.18l0.14-0.11l0-0.4l-0.08-0.14l0.11-0.36l0.2-0.17l0.01-0.34l-0.19-0.31
				l0.24-0.83l0.05-0.46l0.35-0.42l-0.16-0.06l-0.03,0.09l-0.5,0.11l0.03,0.07l-0.07,0.02l-0.33-0.35l-0.27,0.04l0.02-0.17
				l-0.1-0.12l-0.09,0.12l-0.4,0.06l-0.28-0.08l-0.25-0.27l-0.53-0.28l-0.4-0.01l-0.43-0.35l-0.1-0.81l0.15-0.31l0.09-0.5
				l-0.15-0.22l-0.32-0.11l-0.41-0.3l-0.38-0.44l0.01-0.07l-0.18-0.05l-0.08-0.12l-0.25,0.49l-0.62,0.37l-0.22-0.17l-0.68-0.28
				l-0.68-0.55l-0.24-0.31l-0.41-0.19l-0.14-0.18l0.07-0.4l-0.29-0.68l0.09-0.3l-0.2-0.13l-0.26-0.03l0,0l-0.4,0.18l-0.15-0.14
				l-0.18,0.14l-0.14,0.27l0.13,0.2l-0.06,0.18l-0.04,0.39l0.22,0.16l0.02,0.17l-0.27,0.67l0.19,0.22l-0.22-0.04l0.02,0.12
				l-0.35-0.18l-0.03,0.22l-0.16-0.01l-0.85,1.06l-0.38,0.57l-0.25,0.15l-0.14,0.3l-0.35,0.17l-0.22,0.05l-0.17,0.16l-0.05-0.03
				l-0.1,0.28l-0.24,0.11l-0.76,0.59l0,0l-0.22-0.23l-0.31,0.18l-0.33-0.03l-0.13-0.13l-0.11,0.09l-0.49-0.21l-0.56-0.55l-0.04,0.12
				l0.22,0.25l-0.35,0.5l0.13,0.66l-0.42,0.37l-0.19-0.55l-0.55-0.02l-0.49,0.07l-0.07-0.56l0.05-0.63l-0.27-0.23l0,0l-0.44,0.04
				l-0.34,0.07l-0.17-0.07l-0.68,0.13l-0.28,0.13l0,0l-0.41-0.99l0.99,0.07l0.26,0.1l0.04-0.43l0.12-0.24l-0.2-0.4l0.04-0.39
				l-0.03-0.09l-0.38-0.14l-1.16,0.24l0.32,1.28l-0.33-0.05l-0.42-0.64l-0.44-0.88l-0.17-0.19l-0.29-0.12l-0.31,0.04l-0.12,0.09
				l-0.04-0.22l-0.32-0.84l-0.45-0.26l-0.36-0.35l-0.57-0.28l-0.55-0.02l-0.44-0.23l-0.3-0.24l-0.13-0.44l0,0l-0.37,0.13l-0.31,0.01
				l0,0.48l-0.19,0.51l-0.34,0.48l0.01,0.89l0,0.45l-0.03,0.01l-0.06,0.52l-0.17,0.27l-0.23,0.02l-0.05-0.08l-0.15,0.07l-0.04-0.09
				l-0.28-0.06l-0.37,0.05l-0.37,0.42l-0.49,0.11l-0.16,0.28l-0.16,0.11l-0.06,0.18l-0.29,0.14l-0.38-0.01l-0.17-0.02l-0.87,0.46
				l-0.34,0.19l-0.57,0.16l-0.55,0.18l-0.24,0.38l-0.64-0.65l-0.45,0.34l-0.36,0.02l-0.77,0.23l-1.23,0.6l0,0l-0.23,0.23l-0.39-0.28
				l-0.16-0.07l0,0l-0.51,0.31l-0.05,0.31l-0.13-0.3l-0.23-0.27l-0.16,0.18l-0.55-0.02l-0.1,0.16l-0.16,0.02l-0.26-0.15l-0.75,0.24
				l-0.05,0.26l-0.3,0.2l-0.15-0.01l-0.23-0.21l-0.37,0.05l-0.07-0.13l-0.38,0.07l-0.18-0.04l0.04,0.28l-0.14,0.47l-0.12-0.17
				l-0.2,0.03l-0.16-0.16l-0.09,0.01l-0.13,0.18l-0.25-0.12l-0.1,0.14l-0.12,0l-0.01-0.1l-0.55-0.12l-0.42,0.16l-0.04-0.05
				l-0.27,0.38l-0.18-0.06l-0.05,0.06l-0.02,0.09l-0.12,0.03l0.17,0.1l-0.23,0.08l-0.18-0.03l-0.11,0.11l-0.24,0.05l-0.03,0.08
				l-0.11-0.03l-0.02,0.13l-0.08,0.05l-0.13-0.05l-0.2,0.12l0.05,0.12l-0.14,0.01l0.62,1.05l-0.14,0.51l-0.17,0.05l0.21,0.13
				l0.05,0.2l-0.38,0.43l-0.25,0.1l-0.4,0.05l-0.28,0.15v0.06l0.16,0.47l0.18,0.24l0.14,0.51l0.01,0.47l-0.11,0.59l0.02,0.06
				l0.13,0.2l0.11,1.21l-0.15,0.23l-0.33,1.27l0,0l-0.07-0.09l-0.25-0.04l-0.28-0.42l-0.97-0.16l0,0l-0.51,0.04l-1.19,0.6
				l-0.22-0.22l-0.33-0.1l-0.04,0.07l-0.53,0.18l-0.44,0.34L345.3,139l0.03,0.1l-0.37,0.53l-0.11,0.15l-0.35,0.38l-0.21,0.08
				l-0.33,0.3l-0.43,0.17l-0.12,0.14l0.18,0.7l-0.17,0.44l0.41,0.34l0.08,0.2l0.08-0.04l0.18,0.31l0.17,0.75l0.21,0.58l0.46,0.44
				l0.11,0.09l0.33,0.69l0.68,0.66l0.25,0.2l0.23,0.65l0.18,0.18l-0.13,0.27l0.18,0.39l-0.09,0.05l0.01,0.57l-0.15,0.29l0.29,0.71
				l0.22,0l-0.07,0.36l0.08,0.07l-0.12,0.09l-0.28,0.66l0.01,0.15l0.39,0.64l0.04,0.1l0.14,0.28l0.1-0.03l0.64,0.4l0.33-0.03
				l0.72,0.36l-0.04,0.08l0.19,0.42l-0.06,0.15l0.08,0.08l-0.19,0.51l0.06,0.22l-0.18,0.35l-0.43-0.04l-0.03-0.1l-0.24-0.04
				l-0.07,0.42l-0.16,0.02l-0.82,0.61l-0.15,0.32l-0.28,0.32l0.2,0.27l0.14,0.02h0l-0.04,1.27v0.18l0.28,0.31l-0.16,0.25l0.05,0.42
				l-0.18,0.49l0.31,0.2l0.04,0.14l0.5,0.24l0.13,0.14l-0.01,0.37l0.08,0.07l0.69-0.28l0.29-0.86l0.45,0.02l0.17,0.1l0.56,0.66
				l0.02,0.13l-0.34,0.24l-0.1-0.04l-0.36,0.14l-0.29,0.24l-0.14,0.49l0.09,0.11l-0.02,0.21l0.15-0.02l0.25,0.31l0,0.21h0.2
				l0.28,0.72l0.03,0.32l0.37,0.36l0,0.23l0.14,0.17c-0.01,0.04-0.02,0.09-0.04,0.13c-0.07,0.21-0.16,0.41-0.26,0.61c0,0,0,0,0,0
				l0,0.01l0.33,0.22l-0.05,0.4l0.27,0.3l0.29,0.55l0.22,0.23l0.19,0.08l0.45,0.18l0.3,0.39l0.02,0.33l0.15,0.36l-0.13,0.08
				l-0.09,0.59l-0.14,0.07l0.04,0.52l0.11,0.11l0.4,1.55l0.12,1.09l-0.01,1.16l0.5-0.05l0.54,0.06l0.22-0.09l0.97-0.14l0.36-0.57
				l-0.36,0.57l1.3,0.02l-0.09-0.13l0.28-0.42l0.34-0.01l-0.09-0.16l0.09,0.16v0l0.73-0.28l1.36-0.11l0.42,0.08l0.04-0.56l0,0l0,0
				l-0.27-0.17l-0.27-0.46l-0.21-0.04l0.07-0.27l0.32-0.2l0.28-0.05l0.23-0.19l0.06,0.06l0.24-0.08l0.53,0.09l0.68-0.25l1.13-0.15
				l0.16-0.12l0.46,0.3l0.29-0.04l0.19,0.18l0.15-0.13l0.22-0.02l0.24-0.14l0.24-0.94l0.17-0.33l0.35-0.31l0.04-0.28l0.41-0.68
				l0.34-0.21l0.36-0.1l0.14-0.44l-0.1-0.46l0.65-0.65l0.08-0.29l-0.01-0.36l0.14-0.02l0.22-0.25v0l0.93,0.19l0.19,0.35l0.84,0.14
				l0.3-0.12l0.26-0.21l0.2-0.06l0.35,0.04l0.17,0.12l0.25-0.02l0.15,0.09l0.07-0.19l-0.07-0.65l0.17-0.22l0.01-0.17l0.3-0.26
				l0.23-0.55l-0.11-0.88l0.1-0.3l0.21-0.23l0.43-0.89l-0.04-0.46l0.15-0.28l0.03-0.55l-0.17-0.01l-0.22-0.25l0.29-0.54l0,0
				l0.14-0.43l0.19-0.23l-0.02-0.6l0.4-0.02l0.22-0.12l0.1-0.42l0.1-0.27l0.22-0.1l-0.02-0.3l0.2-0.3l0.13-0.08l0.22-0.1l0.16-0.42
				l0.18-0.12l0,0l0.4-0.2l0.03-0.09l0.25-0.03l0.19,0.2l0.39-0.4l0.46-0.06l0.21-0.16l0.52-0.18l0.32,0.08l0.3-0.08l0.48-0.33
				l0.86-0.42l0.43-0.65l0.16-0.21l0.02-0.15l0.32-0.43l0.14-0.04l0.24-0.39l0.42-1.01l0.39-0.45l0.18-0.03l0.19-0.15l0.18-0.31
				l0.41-0.32l0.43-0.3l0.3-0.11l0.29,0.05l0.15-0.12l0.19-0.29l0.87-0.83l0.29-0.71l0.38-0.28l0.09-0.44l0.5-0.47l0.24-0.34
				l0.54-0.33l0.35,0.05l0.07-0.21l0.27-0.15l0.58,0.09l0.47-0.27l0.72-0.03l0.51-0.06l0.45,0.2h0l0.26-0.33l0.09-0.01l0.27-0.56
				l0.28,0.09l0.27-0.19l0.21,0.03l0.29-0.18l0.31,0.02l-0.08-0.16l0.09-0.14l0.25-0.12l0.26-0.59l0.2-0.26l0.2-0.46l0.77,0.06
				l0.21-0.12l0.44,0.25l0.27-0.1l-0.07-0.23l0.05-0.55l-0.13-0.29l0.34-0.52l-0.09-0.32l0.03-0.28l-0.25-0.27l-0.2-0.02l-0.16-0.01
				l0,0l0.16,0.01l-0.15-0.27l0.18,0.24l0.15-0.06l0.48,0.26l1.01-0.18l0.52,0.23l0.43,0.01l0.38,0.16l0.24-0.13l0.02-0.17
				l0.29-0.16l0.07,0.06l0.33-0.06l0.26,0.06l0.14,0.15l0.2,0.04l0.31-0.05l0.09-0.43l0.49-0.49l0.19-0.01l0.18,0.18l0.31,0.12
				l0.17,0.64l0.69-1.16l-0.08-0.2l-0.01-0.11l0.09-0.02l-0.04-0.13l0.51-0.55l0.45-0.02l0.31-0.31l0.02-0.14L401.79,136.22z"></path>
		</g>
	</g>
</g>
<g id="leyenda">
	<path class="st8" d="M702.27,487.1H580.12V334.54h122.15V487.1z"></path>
	<g>
		<path class="st9" d="M636.53,348.68v33.75h-44.24v-33.75l22.3-6.37L636.53,348.68z"></path>
		<path class="st10" d="M636.53,380.51v33.75h-44.24v-33.75l22.3-6.37L636.53,380.51z"></path>
		<path class="st11" d="M636.53,413.94v33.75h-44.24v-33.75l22.3-6.37L636.53,413.94z"></path>
		<path class="st12" d="M636.53,446.89v33.75h-44.24v-33.75l22.3-6.37L636.53,446.89z"></path>
	</g>
</g>
<g id="CapaTexto">
	<g id="Regular">
		<rect x="645.63" y="392.94" class="st13" width="48.88" height="13.44"></rect>
		<text transform="matrix(1 0 0 1 645.6303 402.0851)" class="st14 st15 st16">FASE 2</text>
		<rect x="645.63" y="359.34" class="st13" width="49.91" height="11.34"></rect>
		<text transform="matrix(1 0 0 1 645.6303 368.4814)" class="st14 st15 st16">FASE 3</text>
		<rect x="645.63" y="426.2" class="st13" width="53.28" height="14.86"></rect>
		<text transform="matrix(1 0 0 1 645.6303 435.3392)" class="st14 st15 st16">FASE 1</text>
		<rect x="645.63" y="460.05" class="st13" width="51.21" height="12.57"></rect>
		<text transform="matrix(1 0 0 1 645.6303 469.1946)" class="st14 st15 st16">FASE 0</text>
	</g>
</g>
</svg>